A cute little demo showing the simplest usage of minGPT. Configured to run fine on Macbook Air in like a minute.

In [ ]:
DualCurveBootStrapping.m

Settle = datenum('20-Aug-2013');

% Deposit data
EONIADepositRates = [.0007 .00067]';
EONIADepositMat = datenum({'3-Sep-2013','20-Sep-2013'});
EONIADepositBasis = 2; % act/360
EONIADepositPeriod = 0;

% FRA
EONIAFRARates = [.00025 .0003 .00043 .00054]';
EONIAFRAStartDate = datenum({'11-Sep-2013','9-Oct-2013','13-Nov-2013','11-Dec-2013'});
EONIAFRAEndDate = datenum({'9-Oct-2013','13-Nov-2013','11-Dec-2013','11-Jan-2014'});
EONIAFRABasis = 2; % act/360
EONIAFRAPeriod = 0;

% Swap data
EONIASwapRates = [.0003 .001 .002 .004 .008 .012 .0155 .018 .0193 .02]';
EONIASwapMat = datemnth(Settle,12*[2:5 7 10 15 20 25 30]');
EONIASwapBasis = 5; % 30/360 ISDA
EONIASwapPeriod = 1;

% EURIBOR Deposit data
EURIBORDepositRates = [.0022 .0021 .002 .0019]';
EURIBORDepositMat = datenum({'3-Sep-2013','20-Sep-2013','21-Oct-2013','20-Nov-2013'});
EURIBORDepositBasis = 2; % act/360
EURIBORDepositPeriod = 0;

% EURIBOR Futures
EURIBORFRARates = [9982 9978 9976 9975]';
EURIBORFRAStartDate = datenum({'18-Dec-2013','19-Mar-2014','18-Jun-2014','17-Sep-2014'});
EURIBORFRAEndDate = datenum({'18-Mar-2014','19-Jun-2014','18-Sep-2014','17-Dec-2014'});
EURIBORFRABasis = 2; % act/360
EURIBORFRAPeriod = 4;

% EURIBOR Swap data
EURIBORSwapRates = [.0026 .0044 .0062 .0082 .012 .015 .018 .02 .021 .0215]';
EURIBORSwapMat = datemnth(Settle,12*[2:5 7 10 15 20 25 30]');
EURIBORSwapBasis = 5; % 30/360 ISDA
EURIBORSwapPeriod = 1;

CurveType = 'zero';
CurveCompounding = 1;
CurveBasis = 3; % act/365

nEONIADeposits = length(EONIADepositMat);
nEONIAFRA = length(EONIAFRAEndDate);
nEONIASwaps = length(EONIASwapMat);

EONIAInstrumentTypes = [repmat({'deposit'},nEONIADeposits,1);
    repmat({'fra'},nEONIAFRA,1);repmat({'swap'},nEONIASwaps,1)];

EONIAPeriod = [repmat(EONIADepositPeriod,nEONIADeposits,1);
    repmat(EONIAFRAPeriod,nEONIAFRA,1);repmat(EONIASwapPeriod,nEONIASwaps,1)];

EONIABasis = [repmat(EONIADepositBasis,nEONIADeposits,1);
    repmat(EONIAFRABasis,nEONIAFRA,1);repmat(EONIASwapBasis,nEONIASwaps,1)];

EONIAInstrumentData = [[repmat(Settle,[nEONIADeposits 1]);EONIAFRAStartDate;repmat(Settle,[nEONIASwaps 1])] ...
    [EONIADepositMat;EONIAFRAEndDate;EONIASwapMat] ...
    [EONIADepositRates;EONIAFRARates;EONIASwapRates]];

EONIACurve = IRDataCurve.bootstrap(CurveType,Settle,EONIAInstrumentTypes,...
    EONIAInstrumentData,'Compounding',CurveCompounding,'Basis',CurveBasis,...
    'InstrumentPeriod',EONIAPeriod,'InstrumentBasis',EONIABasis)

nEURIBORDeposits = length(EURIBORDepositMat);
nEURIBORFRA = length(EURIBORFRAEndDate);
nEURIBORSwaps = length(EURIBORSwapMat);

EURIBORInstrumentTypes = [repmat({'deposit'},nEURIBORDeposits,1);
    repmat({'futures'},nEURIBORFRA,1);repmat({'swap'},nEURIBORSwaps,1)];

EURIBORPeriod = [repmat(EURIBORDepositPeriod,nEURIBORDeposits,1);
    repmat(EURIBORFRAPeriod,nEURIBORFRA,1);repmat(EURIBORSwapPeriod,nEURIBORSwaps,1)];

EURIBORBasis = [repmat(EURIBORDepositBasis,nEURIBORDeposits,1);
    repmat(EURIBORFRABasis,nEURIBORFRA,1);repmat(EURIBORSwapBasis,nEURIBORSwaps,1)];

EURIBORInstrumentData = [repmat(Settle,size(EURIBORInstrumentTypes)) ...
    [EURIBORDepositMat;EURIBORFRAEndDate;EURIBORSwapMat] ...
    [EURIBORDepositRates;EURIBORFRARates;EURIBORSwapRates]];

EURIBORCurve_Single = IRDataCurve.bootstrap(CurveType,Settle,EURIBORInstrumentTypes,...
    EURIBORInstrumentData,'Compounding',CurveCompounding,'Basis',CurveBasis,...
    'InstrumentPeriod',EURIBORPeriod,'InstrumentBasis',EURIBORBasis)

EURIBORCurve = IRDataCurve.bootstrap(CurveType,Settle,EURIBORInstrumentTypes,...
    EURIBORInstrumentData,'DiscountCurve',EONIACurve,'Compounding',...
    CurveCompounding,'Basis',CurveBasis,'InstrumentPeriod',EURIBORPeriod,...
    'InstrumentBasis',EURIBORBasis)

PlottingDates = (Settle+20:30:Settle+365*30)';
TimeToMaturity = yearfrac(Settle,PlottingDates);

figure
plot(TimeToMaturity, getZeroRates(EONIACurve, PlottingDates),'b')
hold on
plot(TimeToMaturity, getZeroRates(EURIBORCurve_Single, PlottingDates),'r')
plot(TimeToMaturity, getZeroRates(EURIBORCurve, PlottingDates),'g')
title('Comparison of Single Curve and Dual Curve Bootstrapping')
legend({'EONIA','EURIBOR','EURIBOR w/ EONIA Discounting'},'location','southeast')

In [ ]:
function [rm,rp,p1,p2,p3,q1,q2,q3] = dh_Pade33_coeff(alpha,rho,a)
    
    aa  = sqrt(a.*(a+(0+1i)) - rho^2.*a.^2);
    rm  = -(0+1i)*rho.*a - aa;
    rp  = -(0+1i)*rho.*a + aa;
    
    b1  = -a .* (a+1i) / (2*gamma(1+alpha));
    b2  = (1-a*1i) .* a.^2 * rho / (2*gamma(1+2*alpha));  
    b3  = gamma(1+2*alpha) / gamma(1+3*alpha) * (a.^2.*(1i+a).^2 / (8*gamma(1+alpha)^2) + (a+1i).*a.^3*rho^2 / (2*gamma(1+2*alpha)));
    
    g0  = rm;
    g1  = -rm ./ (aa*gamma(1-alpha));
    g2  = rm ./ aa.^2 / gamma(1-2*alpha) .* (1 + rm./(2*aa)*gamma(1-2*alpha)/gamma(1-alpha)^2);
  
    den = g0.^3 + 2*b1.*g0.*g1 - b2.*g1.^2 + b1.^2.*g2 + b2.*g0.*g2;
    
    p1  = b1;
    p2  = (b1.^2.*g0.^2 + b2.*g0.^3 + b1.^3.*g1 + b1.*b2.*g0.*g1 - b2.^2.*g1.^2 + b1.*b3.*g1.^2 + b2.^2.*g0.*g2 - b1.*b3.*g0.*g2) ./ den;
    q1  = (b1.*g0.^2 + b1.^2.*g1 - b2.*g0.*g1 + b3.*g1.^2 - b1.*b2.*g2 - b3.*g0.*g2) ./ den;
    q2  = (b1.^2.*g0 + b2.*g0.^2 - b1.*b2.*g1 - b3.*g0.*g1 + b2.^2.*g2 - b1.*b3.*g2) ./ den;
    q3  = (b1.^3 + 2*b1.*b2.*g0 + b3.*g0.^2 - b2.^2.*g1 + b1.*b3.*g1) ./ den;
    p3  = g0 .* q3;

end

In [ ]:
function out = dh_Pade33(y,rm,rp,p1,p2,p3,q1,q2,q3)

    h_pade = (p1*y + p2*y.^2 + p3*y.^3) ./ (1 + q1*y + q2*y.^2 + q3*y.^3);
    dh  = 1/2 * (h_pade-rm) .* (h_pade-rp);
    
    out = dh;

end

In [ ]:
classdef DepositRateHelper < H_RootObject
    % FixedRateLeg
    % properties: fixedSchedule 
    
    properties(SetAccess = public)
        rateType
        
        rate
        rateTenor
        
        iborIndex
        evaluationDate
        
        earliestDate
        latestDate
        fixingDate
        
        calcCalendar
        spotLag
    end
    
    methods
        % constructor
        function depositRateHelper = DepositRateHelper(params)
                if nargin > 0
                    depositRateHelper.rateType = 'Deposit';
                    
                    depositRateHelper.rate = params('rate');
                    depositRateHelper.rateTenor = params('rateTenor');
                    depositRateHelper.calcCalendar = params('calcCalendar');
                    depositRateHelper.spotLag = params('spotLag');
                    
                    iborIndexParams = params('IborIndexParams');
                    iborIndexParams('tenor') = params('rateTenor');
                    depositRateHelper.iborIndex = IborIndex(iborIndexParams);
                    depositRateHelper.evaluationDate = depositRateHelper.iborIndex.forwardCurve.asOfDate;
                else
                    depositRateHelper.rate = 0.02;
                    depositRateHelper.iborIndex = IborIndex();
                end
                
                out = InitializeDates(depositRateHelper);
                
                depositRateHelper.earliestDate = out.earliestDate;
                depositRateHelper.latestDate = out.latestDate;
                depositRateHelper.fixingDate = out.fixingDate;
        end
        
        function out = InitializeDates(depositRateHelper)
            iborIndex = depositRateHelper.iborIndex;
            
            if strcmp(depositRateHelper.rateTenor,'ON')
                %earliestDate
                earliestDate = depositRateHelper.evaluationDate;
                latestDate   = depositRateHelper.calcCalendar.Advance(earliestDate,iborIndex.convention,...
                                                                    1,'day',iborIndex.endOfMonth);
                fixingDate   = earliestDate;
            elseif strcmp(depositRateHelper.rateTenor,'SN')
                %earliestDate
                earliestDate   = depositRateHelper.calcCalendar.Advance(iborIndex.forwardCurve.asOfDate,iborIndex.convention,...
                                                                    depositRateHelper.spotLag,'day',iborIndex.endOfMonth);
                
                latestDate = depositRateHelper.calcCalendar.Advance(earliestDate,iborIndex.convention,...
                    1,'day',iborIndex.endOfMonth);
                
                fixingDate = depositRateHelper.evaluationDate;
                
            else
                %earliestDate
                earliestDate = iborIndex.fixingCalendar.Advance(iborIndex.forwardCurve.asOfDate,iborIndex.convention,...
                                                                    depositRateHelper.spotLag,'day',iborIndex.endOfMonth);
                % latestDate or maturityDate                                               
                if strcmp(iborIndex.tenor(end),'D')
                    latestDate = iborIndex.fixingCalendar.Advance(earliestDate,iborIndex.convention,...
                                                                    str2num(iborIndex.tenor(1:end-1)),'day',iborIndex.endOfMonth);

                elseif strcmp(iborIndex.tenor(end),'M')
                    latestDate = iborIndex.fixingCalendar.Advance(earliestDate,iborIndex.convention,...
                                                                    str2num(iborIndex.tenor(1:end-1)),'month',iborIndex.endOfMonth);
                else
                    error('unimplemented')
                end
                
                fixingDate = depositRateHelper.evaluationDate;

%                 fixingDate = iborIndex.fixingCalendar.Advance(earliestDate,iborIndex.convention,...
%                                                                     -1*iborIndex.fixingDays,'day',iborIndex.endOfMonth);
            end
            
            out.earliestDate = earliestDate;
            out.latestDate = latestDate;
            out.fixingDate = fixingDate;
           
        end
        
        % core function
        
        function out = ImpliedQuote(depositRateHelper)
            if strcmp(depositRateHelper.rateTenor,'ON')
                iborIndex = depositRateHelper.iborIndex;
                dcf = DayCountFraction(depositRateHelper.earliestDate,depositRateHelper.latestDate,iborIndex.dayCounter);
                t1 = DateDiff(depositRateHelper.earliestDate,iborIndex.forwardCurve.params('asOfDate'))/365.0;
                t2 = DateDiff(depositRateHelper.latestDate,iborIndex.forwardCurve.params('asOfDate'))/365.0;
                disc1 = iborIndex.forwardCurve.DF(t1);
                disc2 = iborIndex.forwardCurve.DF(t2);
                out = (disc1/disc2 - 1.0)/dcf;
            
            elseif strcmp(depositRateHelper.rateTenor,'SN')
                iborIndex = depositRateHelper.iborIndex;
                dcf = DayCountFraction(depositRateHelper.earliestDate,depositRateHelper.latestDate,iborIndex.dayCounter);
                t1 = DateDiff(depositRateHelper.earliestDate,iborIndex.forwardCurve.params('asOfDate'))/365.0;
                t2 = DateDiff(depositRateHelper.latestDate,iborIndex.forwardCurve.params('asOfDate'))/365.0;
                disc1 = iborIndex.forwardCurve.DF(t1);
                disc2 = iborIndex.forwardCurve.DF(t2);
                out = (disc1/disc2 - 1.0)/dcf;
                
            else
                iborIndex = depositRateHelper.iborIndex;
                dcf = DayCountFraction(depositRateHelper.earliestDate,depositRateHelper.latestDate,iborIndex.dayCounter);
                t1 = DateDiff(depositRateHelper.earliestDate,iborIndex.forwardCurve.params('asOfDate'))/365.0;
                t2 = DateDiff(depositRateHelper.latestDate,iborIndex.forwardCurve.params('asOfDate'))/365.0;
                disc1 = iborIndex.forwardCurve.DF(t1);
                disc2 = iborIndex.forwardCurve.DF(t2);
                out = (disc1/disc2 - 1.0)/dcf;
                
                % bug fixing when we have multiple iborIndex in the deposit strips 
                
%                 out = depositRateHelper.iborIndex.Fixing(depositRateHelper.fixingDate);
            end
        end
        
        % bootstrapping
        function out = QuoteError(depositRateHelper)
            out = depositRateHelper.rate - depositRateHelper.ImpliedQuote();
        end
        
        function out = BootStrapError(depositRateHelper,guess,idx)
            depositRateHelper.Update(idx,guess);
            out = depositRateHelper.QuoteError();
        end
        
        function Update(depositRateHelper,idx,guess)
             % forwardCurve update
            forwardRawData = depositRateHelper.iborIndex.forwardCurve.params('rawData');
            forwardRawData(idx,2) = guess;
            depositRateHelper.iborIndex.forwardCurve.params('rawData')= forwardRawData;
            
        end
    end
    
end



In [ ]:
classdef DefaultCurve < H_RootObject
    % FixedRateLeg
    % properties: fixedSchedule 
    
    properties(SetAccess = public)
        rateHelpers
        survProbCurve
        evaluationDate
        
    end
    
    methods
        % constructor
        function defaultCurve = DefaultCurve(params)
                if nargin > 0
                    defaultCurve.rateHelpers = params('rateHelpers');
                    
                    defaultCurve.survProbCurve = params('survProbCurve');
                    
                    defaultCurve.evaluationDate = defaultCurve.survProbCurve.asOfDate;
                    
                end
                
                Initialize(defaultCurve);
                
        end
        
        function Initialize(defaultCurve)
            
            %earliestDate
            times = zeros(length(defaultCurve.rateHelpers),1);
            spotValues = zeros(length(defaultCurve.rateHelpers),1);
            for i=1:length(defaultCurve.rateHelpers)
                date = defaultCurve.rateHelpers{i}.latestDate;
                times(i) = DateDiff(date,defaultCurve.evaluationDate)/365.0;
                if times(i) < 0
                    error('termstructure should be positive')
                end
                spotValues(i) = exp(-defaultCurve.rateHelpers{i}.rate*times(i));
            end
            
            rawData = zeros(length(defaultCurve.rateHelpers),2);
            rawData(:,1) = times;
            rawData(:,2) = spotValues;
            
            
            % curve init
            defaultCurve.survProbCurve.params('rawData') = rawData;
            
            % yieldCurve.forwardCurve is a handle Object(Pointer in C)
            % therefore assign means handle assign and all rateHelpers's
            % iborIndex.forwardCurve point to the same handle object
            % and change the value of a single forwardCurve gives
            % simultaneous changes in all forwardCurve 
            
            % survProbCurve update on rateHelpers
            for i=1:length(defaultCurve.rateHelpers)
                defaultCurve.rateHelpers{i}.survProbCurve = defaultCurve.survProbCurve;
            end
            
        end
        
        function out = BootStrap(defaultCurve)
            out.sols = zeros(length(defaultCurve.rateHelpers),1);
            out.errors = zeros(length(defaultCurve.rateHelpers),1);
            out.iters = zeros(length(defaultCurve.rateHelpers),1);
            out.times = zeros(length(defaultCurve.rateHelpers),1);
            
            for i=1:length(defaultCurve.rateHelpers)
                date = defaultCurve.rateHelpers{i}.latestDate;
                times = DateDiff(date,defaultCurve.evaluationDate)/365.0;
                
                out.times(i) = times;
                %max hazard rate 100%
                minV = exp(-1.0*times);
                %min hazard rate 0.01%
                maxV = exp(-0.0001*times);
                
                guess = 0.05;
                tol = 1e-12;
                maxIter = 100;
%                 yieldCurve.Update(i,guess);
%                 yieldCurve.rateHelpers{i}.Update(i,guess)
                if i == 6
                    aaa = 1.0;
                end
                outInfo = defaultCurve.Solve(defaultCurve.rateHelpers{i},i,minV,maxV,guess,tol,maxIter);
                
                out.sols(i) = outInfo.sol;
                out.errors(i) = outInfo.error;
                out.iters(i) = outInfo.iter;
%                            Solve(rateHelper,idx,minV,maxV,guess,accuracy)
            end
            
            out.survProbCurve= defaultCurve.survProbCurve;
%             out.discountCurve = defaultCurve.discountCurve;
            
        end
        
        function Update(defaultCurve,idx,value)
            % forwardCurve update
            forwardRawData = defaultCurve.survProbCurve.params('rawData');
            forwardRawData(idx,2) = value;
            defaultCurve.survProbCurve.params('rawData')= forwardRawData;
            
            % forwardCurve update on rateHelpers
            for i=1:length(defaultCurve.rateHelpers)
                defaultCurve.rateHelpers{i}.survProbCurve = defaultCurve.survProbCurve;
            end
            
        end
        
        function out = Solve(defaultCurve,rateHelper,idx,a,b,guess,tol,maxIter)
                
                minV = a;
                maxV = b;
                fMin = rateHelper.BootStrapError(minV,idx);
                fMax = rateHelper.BootStrapError(maxV,idx);
                
                if fMin*fMax > 0
                    error('solution is not in the given inteval')
                else
                    % Orient the search so that f < 0 lies at the r.h.s
            
                    if fMin < 0 
                        minV = b;
                        maxV = a;
                    end
                    
                    for i=1:maxIter
                        midV = (minV + maxV)/2.0;
                        fMid = rateHelper.BootStrapError(midV,idx);
                        if abs(fMid) < tol
                            break;
                        else
                            if fMid > 0
                                minV = midV;
                            else
                                maxV = midV;
                            end
                        end
                    end
                end
                
                rateHelper.Update(idx,midV);
                out.sol = midV;
                out.error = fMid;
                out.iter = i;
        end
    end
    
end



In [ ]:
classdef DailyWeeklyRiskyPricerNFOneTime < H_RootObject
    %Pricer(Security,Model,NumMethod)
    %Prcer is constructed from Security, Model, NumMethod 
    
    properties
        m_security
        m_model
        m_numMethod
        overHedgeYN
        nFactor
    end
    
    methods
        function pricer = DailyWeeklyRiskyPricerNFOneTime(security,model,numMethod,overHedgeYN,nFactor)
                pricer.m_security = security;
                pricer.m_model = model;
                pricer.m_numMethod = numMethod;
                pricer.overHedgeYN = overHedgeYN;
                pricer.nFactor = nFactor;
        end
        
        function out = computePriceMCCallable(pricer,valueDate)
                security = pricer.m_security;
                numMethod = pricer.m_numMethod;
                model = pricer.m_model;
                schedule = security.couponSchedule;
                callSchedule = security.callSchedule;
                
                % numMethodTimeSteps Generate
                mcOneTimeStep = numMethod.mcOneTimeStep;
                outInfo = numMethod.numMethodTimeStepsGenerateDW(valueDate,schedule,mcOneTimeStep);
            
                numMethodInfo.timeStepsInfo = outInfo;
                
                % mc random number init
                %pathSize = 2^16;
                pathSize = numMethod.pathSize;

                % generate random numbers and modelstates forward
                rng('default');
                rng(0);

                numOfFactors = pricer.nFactor;

                %Z= randn(numOfFactors*outInfo.totalTimeStepsSize,pathSize/2);
%                 Z= randn(pathSize/2,numOfFactors*outInfo.totalTimeStepsSize);
                Z= randn(pathSize/2,numOfFactors*1);
                Z=[Z;-Z];
                Z=Z';
                
%                 
%                 %TimeInversion
%                 sizeZ = size(Z,1);
%                 A = zeros(sizeZ,sizeZ);
%                 for i=1:sizeZ
%                     A(i,sizeZ + 1 -i) = 1;
%                 end
%                 Z = A*Z;

                % we check with the c code, so we use generated random numbers
%                 mc_random = importdata('mc_random.xlsx');
%                 mc_random = importdata('D:\mc_random_number_new_report.txt');
%                 Z= mc_random;
                
                numMethodInfo.pathSize = pathSize;
                numMethodInfo.numOfFactors = numOfFactors;
                numMethodInfo.totalTimeSteps = outInfo.totalTimeSteps;
                numMethodInfo.totalTimeStepsSize = outInfo.totalTimeStepsSize;
                numMethodInfo.Z = Z;

                % cacluate past_time_steps_size
                pastTimeStepsSize = DateDiff(valueDate,security.startDate);
                numMethodInfo.pastTimeStepsSize = pastTimeStepsSize;
                
                %generate model states for the given timeSteps
%                 modelStates = numMethod.generateModelStatesNF(valueDate,schedule,numMethodInfo,model);
                modelStates = numMethod.generateModelStatesNFOneTime(valueDate,schedule,numMethodInfo,model);

                % numeraire info
                numMatTime = numMethodInfo.totalTimeSteps(end);
                days = DateDiff(schedule.payDates(schedule.numOfPeriod),valueDate);
                numMatTime = max(numMatTime, days);
                dfNumMat = model.DFRisky(numMatTime/365.0);
                
                numeraire.numMatTime = numMatTime;
                numeraire.dfNumMat = dfNumMat;
                
                % induct backward and calculate cashflow
            
                periodCount = schedule.numOfPeriod;
                
                % pricer single info initialize
                
                nonCall = PricerInfo(periodCount +1, pathSize);
                callable = PricerInfo(periodCount +1, pathSize);
                dfCol = PricerInfo(periodCount +1, pathSize);
                 
                
                % intermediate variables
                payoffStateA.cashflow = zeros(1, pathSize);
                payoffStateA.cashflow_npv = zeros(1, pathSize);
                
            
                % we calculate cashflow backward from maturityDate to valueDate
                % rangeAccrual coupons are added at the startDate of the Period
                % At maturity

                nominal = security.nominal;
                coupon = security.coupon;
                callYN = security.callYN;
                
                currentTime = numMethodInfo.totalTimeSteps(end);
                totalTimeStepsSize = numMethodInfo.totalTimeStepsSize;
                currentTimeIdx = totalTimeStepsSize;
                
                currentNodeIdx = periodCount + 1;
                
                modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:numOfFactors*(currentTimeIdx-1)+numOfFactors,:);
                payoffStateA.cashflow = nominal*ones(1,pathSize);
                
                payTime = DateDiff(schedule.payDates(schedule.numOfPeriod),valueDate);
                df_ep = model.discountFactorRisky(currentTime, payTime,numMatTime,modelStateT);
                for k=1:pathSize
                    payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                end
                
                %process payoff at maturity
                nonCall.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                
                callable.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                
                %df column
                payoffStateA.cashflow = ones(1,pathSize);
                for k=1:pathSize
                    payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                end
                
                dfCol.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                  

                lastTimeIdx = currentTimeIdx;
                totalTimeSteps = numMethodInfo.timeStepsInfo.totalTimeSteps;
                
                % flag for inclusive schedule
%                 simpleYN = false;
%                 simpleYN = true;
                if pricer.overHedgeYN == true 
                    simpleYN = 2;
                else
                    simpleYN = 1;
                end
                
                for i=periodCount:-1:1
                    % past schedule neglect
                    if DateDiff(valueDate,schedule.payDates(i)) >= 0, continue;end 
                    startTimeIdx = numMethodInfo.timeStepsInfo.startTimeIdx(i);
                    endTimeIdx = numMethodInfo.timeStepsInfo.endTimeIdx(i);

                    inclusiveYN = numMethodInfo.timeStepsInfo.inclusiveYN(i);
                 %% FromToInduction start

                    %modelsStates induct backward(lastTimeIdx to endTimeIdx)
                    while endTimeIdx < lastTimeIdx
                        lastTimeIdx = lastTimeIdx -1;
                        if lastTimeIdx == endTimeIdx break;end
                    end

                    currentTimeIdx = lastTimeIdx;
                    currentTime = totalTimeSteps(currentTimeIdx);
                    % now we are at endTimeIdx
                    % variables for coupon valuation initialized
                    ralower = security.lower;
                    raupper = security.upper;
                    tenorLong = security.tenorL;
                    tenorShort = security.tenorS;

                    cD = 0; % cumulated day count fraction
                    cP = 0; 
                    dI = 0; % dummy idx for loop
                    dcfC = 0;
                    dcfN = 0;
                    dD = 0;
                    idxL = zeros(1,pathSize);
                    idxS = zeros(1,pathSize);
                    spreadC = zeros(1,pathSize);
                    spreadN = zeros(1,pathSize);
                    probSpread = zeros(1,pathSize);
                    
                    numOfTimeSteps = numMethodInfo.timeStepsInfo.numOfTimeSteps(i);
                    
                    % if inclusive then we use simpleYN rangeProb
                    %if inclusive then we do not evaluate the last step but
                    % use fixing instead 
                    endTimeStep = 1;
                    if inclusiveYN
%                         simpleYN = true;
                        simpleYN = 1;
                        endTimeStep = 2;
                        if pricer.overHedgeYN == true 
                            simpleYN = 2;
                        end
                    end
                    
                    for j=numOfTimeSteps:-1:endTimeStep
                        % evaluate dealdescription
                        % last step is omitted
                        currentTime = totalTimeSteps(currentTimeIdx);
                        %dcfC = currentTime/365.0;
                        dcfC = currentTime;
                        
                        if j~= numOfTimeSteps
                            dD = (dcfN-dcfC);
                        end

                        modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:numOfFactors*(currentTimeIdx-1)+numOfFactors,:);
                        idxL = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorLong,modelStateT);
                        idxS = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                        spreadC = idxL - idxS;

                        probSpread = model.SRPr(spreadC,spreadN,ralower,raupper,simpleYN,pathSize)*dD;
                        cD = cD + dD;
                        cP = cP + probSpread;

                        dcfN = dcfC;
                        spreadN = spreadC;

                        %induct backward to the next time step
                        % one time backward induction
                        lastTimeIdx = currentTimeIdx;

                        if startTimeIdx < lastTimeIdx
                            lastTimeIdx = lastTimeIdx -1;
                            currentTimeIdx = lastTimeIdx;
                        end


                    end

                    %% FromToInduction end
        % if the current period is inclusive, then the start date is not evaluated in the above for loop.
		% instead the fixing will be treated separately below.
		% however current_time_idx is updated at the end of the for loop.
		% so we need to update model_states accordingly.
		% if model_states were updated previously then the following additionaly step doesn't affect the result anyway.
                    currentTime = totalTimeSteps(currentTimeIdx);
                    modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:numOfFactors*(currentTimeIdx-1)+numOfFactors,:);
                    
                    %% AtDate evaluation
                    % Adding a new cashflow to the payoff

                    dcf = schedule.dayCountFraction(i);
                    
                    if inclusiveYN
                        % we add valuation date
                        cD = schedule.dayCount(i);
                        rangeDays =  security.rangeDays(i);
                        payoffStateA.cashflow = nominal * coupon * dcf * (rangeDays + cP)/cD;
                    else
                        payoffStateA.cashflow = nominal * coupon * dcf * cP/cD;
                    end
                    % eventDate(i) = startDate(i) therefore there is no induction in modelStates
                    % i.e. currentTimeIdx has not changed
                    % no need to update modelStateT
                    % modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:end,:);
                    % since cashflow's are added at the startDate(i) we need to
                    % multiply cashflow with the df(startDate(i), payDate(i))
                    payTime = DateDiff(schedule.payDates(i),valueDate);
                    
                    %event is at startDate so we need to discount the
                    %cashflow first from paydate to startdate
                    df_ep = model.discountFactorRisky(currentTime, payTime,numMatTime,modelStateT);

                    for k=1:pathSize
                        payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                    end
                    
                    % process payoff
                    
                    currentNodeIdx = i;
                    
                    nonCall.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                
                    callable.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                    
                    %df column
                    payoffStateA.cashflow = ones(1,pathSize);
                    for k=1:pathSize
                        payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                    end
                    
                    dfCol.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                    
                    % if exercise is possible
                    if i == 1
                        dummaaa = 1;
                    end
                    
                    % call event happens  at the start of the period
                    % at call date, by default, call option is expired
                    
                    if callYN(i) && (DateDiff(valueDate,callSchedule.resetDates(i)) < 0)
                        % 1.undiscount callable.payoff
                        % 2.build the state variables matrix
                        % 3.regress 1's with the given state variables
                        % (model states 1,2)
                        % 4. exercise per each path in
                        % discount the new payoff and assign into callable
                        % column
                        
                        %1 undiscount callable.payoff and get the
                        %contiValue
                       
                        %exercise value
                        exerciseValue = security.nominal*ones(1,pathSize);
                        payTime = DateDiff(callSchedule.payDates(i),valueDate);
                        %event is at startDate so we need to discount the
                        %cashflow first from paydate to startdate
                        df_ep = model.discountFactorRisky(currentTime, payTime,numMatTime,modelStateT);
                        for k=1:pathSize
                            exerciseValue(k) = exerciseValue(k) * df_ep(k);
                        end
                        
                        contiValue = zeros(1,pathSize);
                        onePayoff = ones(1,pathSize);
                        onePayoffD = model.discountPayoffRisky(currentTime,numMatTime ...
                        ,modelStateT,onePayoff);
                        contiValue = callable.payoff./onePayoffD;
                        
                        % build the state variabls matrix
                        decisionVar = zeros(numOfFactors,pathSize);
                        for idx_nf=1:numOfFactors
                            decisionVar(idx_nf,:) = modelStateT(idx_nf,:);
                        end
                        
%                         decisionVar1 = modelStateT(1,:);
%                         decisionVar2 = modelStateT(2,:);
                        % we use upto 2nd order polynomial as regression
                        % equatioin
%                         dimensionLS = 6;
                        dimensionLS = 1 + numOfFactors + numOfFactors*(numOfFactors+1)/2;
                        
                        regMatrix = ones(pathSize,dimensionLS);
                        
                        for idx_r=1:pathSize
                            
                            idx_dummy = 1;
                            for idx_nf=1:numOfFactors
                                idx_dummy = idx_dummy +1;
                                regMatrix(idx_r,idx_dummy) = decisionVar(idx_nf,idx_r);
                                
                            end
                            
                            for idx_nf1=1:numOfFactors
                                for idx_nf2=idx_nf1:numOfFactors
                                    idx_dummy = idx_dummy +1;
                                    regMatrix(idx_r,idx_dummy) = decisionVar(idx_nf1,idx_r)*decisionVar(idx_nf2,idx_r);
                                end
                            end
                            
%                             regMatrix(idx_r,2) = decisionVar1(idx_r);
%                             regMatrix(idx_r,3) = decisionVar2(idx_r);
%                             regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r);
%                             regMatrix(idx_r,5) = decisionVar1(idx_r)*decisionVar2(idx_r);
%                             regMatrix(idx_r,6) = decisionVar2(idx_r)*decisionVar2(idx_r);
                        end
                        
                        % simple regression <- for path dependent option
                        % we should change it
                        % beta  : regression coefficients
                        beta =zeros(dimensionLS,1);
                        beta = regress(contiValue',regMatrix);
                        regContiValue = regMatrix*beta;
                        
                        exercisedPayoff = regContiValue;
                        % if regContiValue is bigger than exerciseValue
                        % then issue exercise call option
                        for idx_r=1:pathSize
                            if regContiValue(idx_r) >= exerciseValue(idx_r) 
                                exercisedPayoff(idx_r) = exerciseValue(idx_r);
                            end
                        end
                        
                        % we discount the new payoff and 
                        % assign it into the existing callable payoff
                        callable.payoff = model.discountPayoffRisky(currentTime,numMatTime ...
                        ,modelStateT,exercisedPayoff);
                        
                    end
                end

                nonCall.npv = mean(nonCall.payoff);
                out.nonCall = nonCall;
                callable.npv = mean(callable.payoff);
                out.callable = callable;
                out.dfCol = dfCol;
                out.daycountFraction = schedule.dayCountFraction;
                
        end
        
        function out = computePriceClosedTRZ(pricer,valueDate)
                security = pricer.m_security;
                numMethod = pricer.m_numMethod;
                model = pricer.m_model;
                schedule = security.couponSchedule;
                
                % numMethodTimeSteps Generate
                mcOneTimeStep = numMethod.mcOneTimeStep;
                outInfo = numMethod.numMethodTimeStepsGenerateDW(valueDate,schedule,mcOneTimeStep);
            
                numMethodInfo.timeStepsInfo = outInfo;

                % mc random number init
                %pathSize = 2^16;
                pathSize = numMethod.pathSize;
                
                pathSize = 1;
                
%                 % generate random numbers and modelstates forward
%                 rng('default');
%                 rng(0);
% 
                numOfFactors = pricer.nFactor;
% 
%                 %Z= randn(numOfFactors*outInfo.totalTimeStepsSize,pathSize/2);
%                 Z= randn(pathSize/2,numOfFactors*outInfo.totalTimeStepsSize);
%                 Z=[Z;-Z];
%                 Z=Z';
%                 
%                 
%                 %TimeInversion
%                 sizeZ = size(Z,1);
%                 A = zeros(sizeZ,sizeZ);
%                 for i=1:sizeZ
%                     A(i,sizeZ + 1 -i) = 1;
%                 end
%                 Z = A*Z;
% 
%                 % we check with the c, so we use generated random numbers
% %                 mc_random = importdata('mc_random.xlsx');
% %                 mc_random = importdata('D:\mc_random_number_new_report.txt');
% %                 Z= mc_random;
%                 
                numMethodInfo.pathSize = pathSize;
                numMethodInfo.numOfFactors = numOfFactors;
                numMethodInfo.totalTimeSteps = outInfo.totalTimeSteps;
                numMethodInfo.totalTimeStepsSize = outInfo.totalTimeStepsSize;
%                 numMethodInfo.Z = Z;
% 
% 
% 
%                 %generate model states for the given timeSteps
%                 modelStates = numMethod.generateModelStates(valueDate,schedule,numMethodInfo,model);

                % numeraire info
                numMatTime = numMethodInfo.totalTimeSteps(end);
                dfNumMat = model.DF(numMatTime/365.0);
                
                % induct backward and calculate cashflow
            
                periodCount = schedule.numOfPeriod;
                
                % cashflow columns initialize
                % 1st nonCall
                nonCall.npv = 0;
                nonCall.cashflow_e = zeros(periodCount + 1,1);
                nonCall.cashflow_npv_e = zeros(periodCount + 1,1);
                nonCall.rangeProb_e = zeros(periodCount + 1,1);
                nonCall.df_e = zeros(periodCount + 1,1);
                nonCall.df_c = zeros(periodCount + 1,1);

                % debug information, cashflow & npv per path
                nonCall.payoff = zeros(1,pathSize);
                nonCall.payoffStates.cashflow = zeros(periodCount+1,pathSize);
                nonCall.payoffStates.cashflow_npv = zeros(periodCount+1,pathSize);
                nonCall.payoffStates.df = zeros(periodCount + 1,pathSize);
                
                % intermediate variables
                payoffStateA.cashflow = zeros(1, pathSize);
                payoffStateA.cashflow_npv = zeros(1, pathSize);
                payoffStateA.df = zeros(1,pathSize);
                
                % we calculate cashflow backward from maturityDate to valueDate
                % rangeAccrual coupons are added at the startDate of the Period
                % At maturity

                nominal = security.nominal;
                coupon = security.coupon;
                callYN = security.callYN;
                
                currentTime = numMethodInfo.totalTimeSteps(end);
                totalTimeSteps = numMethodInfo.timeStepsInfo.totalTimeSteps;
                totalTimeStepsSize = numMethodInfo.timeStepsInfo.totalTimeStepsSize;
                if totalTimeStepsSize == 0 || (totalTimeStepsSize == 1 && totalTimeSteps(1) == 0) 
                    out.nonCall = nonCall;
                    return;
                end
                
                currentTimeIdx = totalTimeStepsSize;
                
                currentNodeIdx = periodCount + 1;
                
%                 modelStateT = modelStates(numOfFactors*(totalTimeStepsSize-1)+1:numOfFactors*(totalTimeStepsSize-1)+2,:);
                payoffStateA.cashflow = nominal*ones(1,pathSize);
                %process payoff at maturity
                nonCall.payoffStates.cashflow(currentNodeIdx,:) = payoffStateA.cashflow;
                
%                 payoffStateA.cashflow_npv = model.discountPayoff(currentTime,numMatTime ...
%                     ,modelStateT,payoffStateA.cashflow);
%                 
                payoffStateA.cashflow_npv = model.DFRisky(currentTime/365.0)*payoffStateA.cashflow;
                
                nonCall.payoffStates.cashflow_npv(currentNodeIdx,:) = payoffStateA.cashflow_npv ;
                
%                 payoffStateA.df = model.discountPayoff(currentTime,numMatTime ...
%                     ,modelStateT,1.0*ones(1,pathSize));
                payoffStateA.df = model.DFRisky(currentTime/365.0) *ones(1,pathSize);
                    

                nonCall.payoffStates.df(currentNodeIdx,:) = payoffStateA.df;
                nonCall.payoff = nonCall.payoff + payoffStateA.cashflow_npv;
                
                nonCall.cashflow_e(currentNodeIdx) = mean(payoffStateA.cashflow);
                nonCall.cashflow_npv_e(currentNodeIdx) = mean(payoffStateA.cashflow_npv);
                nonCall.df_e(currentNodeIdx) = mean(payoffStateA.df);
                nonCall.df_c(currentNodeIdx) = model.DFRisky(currentTime/365.0);
                
%                 % callable at maturity is the same as noncallable
%                 callable.payoffStates.cashflow(currentNodeIdx,:) = nonCall.payoffStates.cashflow(currentNodeIdx,:);
%                 callable.payoffStates.cashflow_npv(currentNodeIdx,:) = nonCall.payoffStates.cashflow_npv(currentNodeIdx,:);
%                 callable.payoff = callable.payoff + callable.payoffStates.cashflow_npv(currentNodeIdx,:);
%                 
%                 callable.cashflow_e(currentNodeIdx) = nonCall.cashflow_e(currentNodeIdx);
%                 callable.cashflow_npv_e(currentNodeIdx) = nonCall.cashflow_npv_e(currentNodeIdx);
                  
                lastTimeIdx = currentTimeIdx;
                
                
                % flag for inclusive schedule
%                 simpleYN = false;
                simpleYN = 1;
                
                for i=periodCount:-1:1
                    % past schedule neglect
                    if DateDiff(valueDate,schedule.payDates(i)) >= 0, continue;end 
                    startTimeIdx = numMethodInfo.timeStepsInfo.startTimeIdx(i);
                    endTimeIdx = numMethodInfo.timeStepsInfo.endTimeIdx(i);

                    inclusiveYN = numMethodInfo.timeStepsInfo.inclusiveYN(i);
                     %% FromToInduction start

                    %modelsStates induct backward(lastTimeIdx to endTimeIdx)
                    while endTimeIdx < lastTimeIdx
                        lastTimeIdx = lastTimeIdx -1;
                        if lastTimeIdx == endTimeIdx break;end
                    end

                    currentTimeIdx = lastTimeIdx;
                    currentTime = totalTimeSteps(currentTimeIdx);
                    % now we are at endTimeIdx
                    % variables for coupon valuation initialized
                    ralower = security.lower;
                    raupper = security.upper;
                    tenorLong = security.tenorL;
                    tenorShort = security.tenorS;

                    cD = 0; % cumulated day count fraction
                    cP = 0; 
                    dI = 0; % dummy idx for loop
                    dcfC = 0;
                    dcfN = 0;
                    dD = 0;
                    idxL = zeros(1,pathSize);
                    idxS = zeros(1,pathSize);
                    spreadC = zeros(1,pathSize);
                    spreadN = zeros(1,pathSize);
                    probSpread = zeros(1,pathSize);
                    
                    numOfTimeSteps = numMethodInfo.timeStepsInfo.numOfTimeSteps(i);
                    
                    % if inclusive then we use simpleYN rangeProb
                    %if inclusive then we do not evaluate the last step but
                    % use fixing instead 
                    endTimeStep = 1;
                    if inclusiveYN
%                         simpleYN = true;
                        simpleYN = 1;
                        endTimeStep = 2;
                    end
                    
                    for j=numOfTimeSteps:-1:endTimeStep
                        % evaluate dealdescription
                        % last step is omitted
                        currentTime = totalTimeSteps(currentTimeIdx);
                        %dcfC = currentTime/365.0;
                        dcfC = currentTime;
                        
                        if j~= numOfTimeSteps
                            dD = (dcfN-dcfC);
                        end

                        toDate = AddDate(valueDate,currentTime,'day');
                        tpDate = schedule.payDates(i);
                        spreadC = model.CMSSpreadDigitalRangeDate(valueDate,toDate,tpDate,tenorLong,tenorShort,0.25,ralower,raupper);
                        
                        %modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:numOfFactors*(currentTimeIdx-1)+2,:);
%                         idxL = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorLong,modelStateT);
%                         idxS = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorShort,modelStateT);
%                         spreadC = idxL - idxS;
%                         probSpread = model.SRPr(spreadC,spreadN,ralower,raupper,simpleYN,pathSize)*dD; 

                        if simpleYN
                            probSpread = spreadC*dD;
                        else
                            probSpread = spreadC*dD;
%                             probSpread = 0.5*(spreadC + spreadN)*dD;
                            
                        end

                        
                        cD = cD + dD;
                        cP = cP + probSpread;

                        dcfN = dcfC;
                        spreadN = spreadC;

                        %induct backward to the next time step
                        % one time backward induction
                        lastTimeIdx = currentTimeIdx;

                        if startTimeIdx < lastTimeIdx
                            lastTimeIdx = lastTimeIdx -1;
                            currentTimeIdx = lastTimeIdx;
                        end


                    end

                    %% FromToInduction end
                    %% AtDate evaluation
                    % Adding a new cashflow to the payoff

                    dcf = schedule.dayCountFraction(i);
                    
                    if inclusiveYN
                        % we add valuation date
                        cD = schedule.dayCount(i);
                        rangeDays =  security.rangeDays(i);
                        payoffStateA.cashflow = nominal * coupon * dcf * (rangeDays(i) + cP)/cD;
                    else
                        payoffStateA.cashflow = nominal * coupon * dcf * cP/cD;
                    end
                    % eventDate(i) = startDate(i) therefore there is no induction in modelStates
                    % i.e. currentTimeIdx has not changed
                    % no need to update modelStateT
                    % modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:end,:);
                    % since cashflow's are added at the startDate(i) we need to
                    % multiply cashflow with the df(startDate(i), payDate(i))
                    payTime = DateDiff(schedule.payDates(i),valueDate);
                    
                    %event is at startDate so we need to discount the
                    %cashflow first from paydate to startdate
%                     df_sp = model.discountFactor(currentTime, payTime,numMatTime,modelStateT);

%                     for k=1:pathSize
%                         payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_sp(k);
%                     end
                    
                    % process payoff
                    
                    currentNodeIdx = i;
                    
                    % 1st nonCall column
                    nonCall.payoffStates.cashflow(currentNodeIdx,:) = payoffStateA.cashflow;
                    payoffStateA.cashflow_npv = payoffStateA.cashflow * model.DFRisky(payTime/365.0);
                    
%                     payoffStateA.cashflow_npv = model.discountPayoff(currentTime,numMatTime ...
%                     ,modelStateT,payoffStateA.cashflow);
                    nonCall.payoffStates.cashflow_npv(currentNodeIdx,:) = payoffStateA.cashflow_npv;
                    payoffStateA.df = model.DFRisky(payTime/365.0);
%                     payoffStateA.df = model.discountPayoff(currentTime,numMatTime ...
%                     ,modelStateT,1.0*ones(1,pathSize));
                    nonCall.payoffStates.df(currentNodeIdx,:) = payoffStateA.df;
                    
                    nonCall.payoff = nonCall.payoff + payoffStateA.cashflow_npv;
                    
                    nonCall.cashflow_e(currentNodeIdx) = mean(payoffStateA.cashflow);
                    nonCall.rangeProb_e(currentNodeIdx) = mean(nonCall.cashflow_e(currentNodeIdx)/(nominal * coupon * dcf));
                    nonCall.cashflow_npv_e(currentNodeIdx) = mean(payoffStateA.cashflow_npv);

                    nonCall.df_e(currentNodeIdx) = mean(payoffStateA.df);
                    nonCall.df_c(currentNodeIdx) = model.DFRisky(currentTime/365.0);
                    
%                     %2nd callable column
%                     callable.payoffStates.cashflow(currentNodeIdx,:) = nonCall.payoffStates.cashflow(currentNodeIdx,:);
%                     callable.payoffStates.cashflow_npv(currentNodeIdx,:) = nonCall.payoffStates.cashflow_npv(currentNodeIdx,:);
%                     callable.payoff = callable.payoff + callable.payoffStates.cashflow_npv(currentNodeIdx,:);
%                     
%                     callable.cashflow_e(currentNodeIdx) = nonCall.cashflow_e(currentNodeIdx);
%                     callable.cashflow_npv_e(currentNodeIdx) = nonCall.cashflow_npv_e(currentNodeIdx);

                    % if exercise is possible
                    if i == 1
                        dummaaa = 1;
                    end
                    
                    % call event happens  at the start of the period
                    % at call date, by default, call option is expired
                    
%                     if callYN(i) && (DateDiff(valueDate,schedule.startDates(i)) < 0)
%                         % 1.undiscount callable.payoff
%                         % 2.build the state variables matrix
%                         % 3.regress 1's with the given state variables
%                         % (model states 1,2)
%                         % 4. exercise per each path in
%                         % discount the new payoff and assign into callable
%                         % column
%                         
%                         %1 undiscount callable.payoff and get the
%                         %contiValue
%                        
%                         %exercise value
%                         exerciseValue = security.nominal*ones(1,pathSize);
%                         contiValue = zeros(1,pathSize);
%                         onePayoff = ones(1,pathSize);
%                         onePayoffD = model.discountPayoff(currentTime,numMatTime ...
%                         ,modelStateT,onePayoff);
%                         contiValue = callable.payoff./onePayoffD;
%                         
%                         % build the state variabls matrix
%                         decisionVar1 = modelStateT(1,:);
%                         decisionVar2 = modelStateT(2,:);
%                         % we use upto 2nd order polynomial as regression
%                         % equatioin
%                         dimensionLS = 6;
%                         regMatrix = ones(pathSize,dimensionLS);
%                         
%                         for idx_r=1:pathSize
%                             regMatrix(idx_r,2) = decisionVar1(idx_r);
%                             regMatrix(idx_r,3) = decisionVar2(idx_r);
%                             regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r);
%                             regMatrix(idx_r,5) = decisionVar1(idx_r)*decisionVar2(idx_r);
%                             regMatrix(idx_r,6) = decisionVar2(idx_r)*decisionVar2(idx_r);
%                         end
%                         
%                         % simple regression <- for path dependent option
%                         % we should change it
%                         % beta  : regression coefficients
%                         beta =zeros(dimensionLS,1);
%                         beta = regress(contiValue',regMatrix);
%                         regContiValue = regMatrix*beta;
%                         
%                         exercisedPayoff = regContiValue;
%                         % if regContiValue is bigger than exerciseValue
%                         % then issue exercise call option
%                         for idx_r=1:pathSize
%                             if regContiValue(idx_r) >= exerciseValue(idx_r) 
%                                 exercisedPayoff(idx_r) = exerciseValue(idx_r);
%                             end
%                         end
%                         
%                         % we discount the new payoff and 
%                         % assign it into the existing callable payoff
%                         callable.payoff = model.discountPayoff(currentTime,numMatTime ...
%                         ,modelStateT,exercisedPayoff);
%                         
%                     end
                end

                nonCall.npv = mean(nonCall.payoff);
                out.pricerInfo.nonCall = nonCall;
                
%                 callable.npv = mean(callable.payoff);
%                 out.pricerInfo.callable = callable;
                out.daycountFraction = schedule.dayCountFraction;
                
        end
        
        
        
        
    end
    
end



In [ ]:
classdef DailyWeeklyRiskyPricerNF < H_RootObject
    %Pricer(Security,Model,NumMethod)
    %Prcer is constructed from Security, Model, NumMethod 
    
    properties
        m_security
        m_model
        m_numMethod
        overHedgeYN
        nFactor
    end
    
    methods
        function pricer = DailyWeeklyRiskyPricerNF(security,model,numMethod,overHedgeYN,nFactor)
                pricer.m_security = security;
                pricer.m_model = model;
                pricer.m_numMethod = numMethod;
                pricer.overHedgeYN = overHedgeYN;
                pricer.nFactor = nFactor;
        end
        
        function out = computePriceMCCallable(pricer,valueDate)
                security = pricer.m_security;
                numMethod = pricer.m_numMethod;
                model = pricer.m_model;
                schedule = security.couponSchedule;
                callSchedule = security.callSchedule;
                
                % numMethodTimeSteps Generate
                mcOneTimeStep = numMethod.mcOneTimeStep;
                outInfo = numMethod.numMethodTimeStepsGenerateDW(valueDate,schedule,mcOneTimeStep);
            
                numMethodInfo.timeStepsInfo = outInfo;

                % mc random number init
                %pathSize = 2^16;
                pathSize = numMethod.pathSize;

                % generate random numbers and modelstates forward
                rng('default');
                rng(0);

                numOfFactors = pricer.nFactor;

                %Z= randn(numOfFactors*outInfo.totalTimeStepsSize,pathSize/2);
                Z= randn(pathSize/2,numOfFactors*outInfo.totalTimeStepsSize);
                Z=[Z;-Z];
                Z=Z';
                
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;

                % we check with the c code, so we use generated random numbers
%                 mc_random = importdata('mc_random.xlsx');
%                 mc_random = importdata('D:\mc_random_number_new_report.txt');
%                 Z= mc_random;
                
                numMethodInfo.pathSize = pathSize;
                numMethodInfo.numOfFactors = numOfFactors;
                numMethodInfo.totalTimeSteps = outInfo.totalTimeSteps;
                numMethodInfo.totalTimeStepsSize = outInfo.totalTimeStepsSize;
                numMethodInfo.Z = Z;



                %generate model states for the given timeSteps
                modelStates = numMethod.generateModelStatesNF(valueDate,schedule,numMethodInfo,model);

                % numeraire info
                numMatTime = numMethodInfo.totalTimeSteps(end);
                days = DateDiff(schedule.payDates(schedule.numOfPeriod),valueDate);
                numMatTime = max(numMatTime, days);
                dfNumMat = model.DFRisky(numMatTime/365.0);
                
                numeraire.numMatTime = numMatTime;
                numeraire.dfNumMat = dfNumMat;
                
                % induct backward and calculate cashflow
            
                periodCount = schedule.numOfPeriod;
                
                % pricer single info initialize
                
                nonCall = PricerInfo(periodCount +1, pathSize);
                callable = PricerInfo(periodCount +1, pathSize);
                dfCol = PricerInfo(periodCount +1, pathSize);
                 
                
                % intermediate variables
                payoffStateA.cashflow = zeros(1, pathSize);
                payoffStateA.cashflow_npv = zeros(1, pathSize);
                
            
                % we calculate cashflow backward from maturityDate to valueDate
                % rangeAccrual coupons are added at the startDate of the Period
                % At maturity

                nominal = security.nominal;
                coupon = security.coupon;
                callYN = security.callYN;
                
                currentTime = numMethodInfo.totalTimeSteps(end);
                totalTimeStepsSize = numMethodInfo.totalTimeStepsSize;
                currentTimeIdx = totalTimeStepsSize;
                
                currentNodeIdx = periodCount + 1;
                
                modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:numOfFactors*(currentTimeIdx-1)+numOfFactors,:);
                payoffStateA.cashflow = nominal*ones(1,pathSize);
                
                payTime = DateDiff(schedule.payDates(schedule.numOfPeriod),valueDate);
                df_ep = model.discountFactorRisky(currentTime, payTime,numMatTime,modelStateT);
                for k=1:pathSize
                    payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                end
                
                %process payoff at maturity
                nonCall.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                
                callable.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                
                %df column
                payoffStateA.cashflow = ones(1,pathSize);
                for k=1:pathSize
                    payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                end
                
                dfCol.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                  

                lastTimeIdx = currentTimeIdx;
                totalTimeSteps = numMethodInfo.timeStepsInfo.totalTimeSteps;
                
                % flag for inclusive schedule
%                 simpleYN = false;
%                 simpleYN = true;
                if pricer.overHedgeYN == true 
                    simpleYN = 2;
                else
                    simpleYN = 1;
                end
                
                for i=periodCount:-1:1
                    % past schedule neglect
                    if DateDiff(valueDate,schedule.payDates(i)) >= 0, continue;end 
                    startTimeIdx = numMethodInfo.timeStepsInfo.startTimeIdx(i);
                    endTimeIdx = numMethodInfo.timeStepsInfo.endTimeIdx(i);

                    inclusiveYN = numMethodInfo.timeStepsInfo.inclusiveYN(i);
                 %% FromToInduction start

                    %modelsStates induct backward(lastTimeIdx to endTimeIdx)
                    while endTimeIdx < lastTimeIdx
                        lastTimeIdx = lastTimeIdx -1;
                        if lastTimeIdx == endTimeIdx break;end
                    end

                    currentTimeIdx = lastTimeIdx;
                    currentTime = totalTimeSteps(currentTimeIdx);
                    % now we are at endTimeIdx
                    % variables for coupon valuation initialized
                    ralower = security.lower;
                    raupper = security.upper;
                    tenorLong = security.tenorL;
                    tenorShort = security.tenorS;

                    cD = 0; % cumulated day count fraction
                    cP = 0; 
                    dI = 0; % dummy idx for loop
                    dcfC = 0;
                    dcfN = 0;
                    dD = 0;
                    idxL = zeros(1,pathSize);
                    idxS = zeros(1,pathSize);
                    spreadC = zeros(1,pathSize);
                    spreadN = zeros(1,pathSize);
                    probSpread = zeros(1,pathSize);
                    
                    numOfTimeSteps = numMethodInfo.timeStepsInfo.numOfTimeSteps(i);
                    
                    % if inclusive then we use simpleYN rangeProb
                    %if inclusive then we do not evaluate the last step but
                    % use fixing instead 
                    endTimeStep = 1;
                    if inclusiveYN
%                         simpleYN = true;
                        simpleYN = 1;
                        endTimeStep = 2;
                        if pricer.overHedgeYN == true 
                            simpleYN = 2;
                        end
                    end
                    
                    for j=numOfTimeSteps:-1:endTimeStep
                        % evaluate dealdescription
                        % last step is omitted
                        currentTime = totalTimeSteps(currentTimeIdx);
                        %dcfC = currentTime/365.0;
                        dcfC = currentTime;
                        
                        if j~= numOfTimeSteps
                            dD = (dcfN-dcfC);
                        end

                        modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:numOfFactors*(currentTimeIdx-1)+numOfFactors,:);
                        idxL = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorLong,modelStateT);
                        idxS = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                        spreadC = idxL - idxS;

                        probSpread = model.SRPr(spreadC,spreadN,ralower,raupper,simpleYN,pathSize)*dD;
                        cD = cD + dD;
                        cP = cP + probSpread;

                        dcfN = dcfC;
                        spreadN = spreadC;

                        %induct backward to the next time step
                        % one time backward induction
                        lastTimeIdx = currentTimeIdx;

                        if startTimeIdx < lastTimeIdx
                            lastTimeIdx = lastTimeIdx -1;
                            currentTimeIdx = lastTimeIdx;
                        end


                    end

                    %% FromToInduction end
        % if the current period is inclusive, then the start date is not evaluated in the above for loop.
		% instead the fixing will be treated separately below.
		% however current_time_idx is updated at the end of the for loop.
		% so we need to update model_states accordingly.
		% if model_states were updated previously then the following additionaly step doesn't affect the result anyway.
                    currentTime = totalTimeSteps(currentTimeIdx);
                    modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:numOfFactors*(currentTimeIdx-1)+numOfFactors,:);
                    
                    %% AtDate evaluation
                    % Adding a new cashflow to the payoff

                    dcf = schedule.dayCountFraction(i);
                    
                    if inclusiveYN
                        % we add valuation date
                        cD = schedule.dayCount(i);
                        rangeDays =  security.rangeDays(i);
                        payoffStateA.cashflow = nominal * coupon * dcf * (rangeDays(i) + cP)/cD;
                    else
                        payoffStateA.cashflow = nominal * coupon * dcf * cP/cD;
                    end
                    % eventDate(i) = startDate(i) therefore there is no induction in modelStates
                    % i.e. currentTimeIdx has not changed
                    % no need to update modelStateT
                    % modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:end,:);
                    % since cashflow's are added at the startDate(i) we need to
                    % multiply cashflow with the df(startDate(i), payDate(i))
                    payTime = DateDiff(schedule.payDates(i),valueDate);
                    
                    %event is at startDate so we need to discount the
                    %cashflow first from paydate to startdate
                    df_ep = model.discountFactorRisky(currentTime, payTime,numMatTime,modelStateT);

                    for k=1:pathSize
                        payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                    end
                    
                    % process payoff
                    
                    currentNodeIdx = i;
                    
                    nonCall.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                
                    callable.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                    
                    %df column
                    payoffStateA.cashflow = ones(1,pathSize);
                    for k=1:pathSize
                        payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                    end
                    
                    dfCol.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                    
                    % if exercise is possible
                    if i == 1
                        dummaaa = 1;
                    end
                    
                    % call event happens  at the start of the period
                    % at call date, by default, call option is expired
                    
                    if callYN(i) && (DateDiff(valueDate,callSchedule.resetDates(i)) < 0)
                        % 1.undiscount callable.payoff
                        % 2.build the state variables matrix
                        % 3.regress 1's with the given state variables
                        % (model states 1,2)
                        % 4. exercise per each path in
                        % discount the new payoff and assign into callable
                        % column
                        
                        %1 undiscount callable.payoff and get the
                        %contiValue
                       
                        %exercise value
                        exerciseValue = security.nominal*ones(1,pathSize);
                        payTime = DateDiff(callSchedule.payDates(i),valueDate);
                        %event is at startDate so we need to discount the
                        %cashflow first from paydate to startdate
                        df_ep = model.discountFactorRisky(currentTime, payTime,numMatTime,modelStateT);
                        for k=1:pathSize
                            exerciseValue(k) = exerciseValue(k) * df_ep(k);
                        end
                        
                        contiValue = zeros(1,pathSize);
                        onePayoff = ones(1,pathSize);
                        onePayoffD = model.discountPayoffRisky(currentTime,numMatTime ...
                        ,modelStateT,onePayoff);
                        contiValue = callable.payoff./onePayoffD;
                        
                        % build the state variabls matrix
                        decisionVar = zeros(numOfFactors,pathSize);
                        for idx_nf=1:numOfFactors
                            decisionVar(idx_nf,:) = modelStateT(idx_nf,:);
                        end
                        
%                         decisionVar1 = modelStateT(1,:);
%                         decisionVar2 = modelStateT(2,:);
                        % we use upto 2nd order polynomial as regression
                        % equatioin
%                         dimensionLS = 6;
                        dimensionLS = 1 + numOfFactors + numOfFactors*(numOfFactors+1)/2;
                        
                        regMatrix = ones(pathSize,dimensionLS);
                        
                        for idx_r=1:pathSize
                            
                            idx_dummy = 1;
                            for idx_nf=1:numOfFactors
                                idx_dummy = idx_dummy +1;
                                regMatrix(idx_r,idx_dummy) = decisionVar(idx_nf,idx_r);
                                
                            end
                            
                            for idx_nf1=1:numOfFactors
                                for idx_nf2=idx_nf1:numOfFactors
                                    idx_dummy = idx_dummy +1;
                                    regMatrix(idx_r,idx_dummy) = decisionVar(idx_nf1,idx_r)*decisionVar(idx_nf2,idx_r);
                                end
                            end
                            
%                             regMatrix(idx_r,2) = decisionVar1(idx_r);
%                             regMatrix(idx_r,3) = decisionVar2(idx_r);
%                             regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r);
%                             regMatrix(idx_r,5) = decisionVar1(idx_r)*decisionVar2(idx_r);
%                             regMatrix(idx_r,6) = decisionVar2(idx_r)*decisionVar2(idx_r);
                        end
                        
                        % simple regression <- for path dependent option
                        % we should change it
                        % beta  : regression coefficients
                        beta =zeros(dimensionLS,1);
                        beta = regress(contiValue',regMatrix);
                        regContiValue = regMatrix*beta;
                        
                        exercisedPayoff = regContiValue;
                        % if regContiValue is bigger than exerciseValue
                        % then issue exercise call option
                        for idx_r=1:pathSize
                            if regContiValue(idx_r) >= exerciseValue(idx_r) 
                                exercisedPayoff(idx_r) = exerciseValue(idx_r);
                            end
                        end
                        
                        % we discount the new payoff and 
                        % assign it into the existing callable payoff
                        callable.payoff = model.discountPayoffRisky(currentTime,numMatTime ...
                        ,modelStateT,exercisedPayoff);
                        
                    end
                end

                nonCall.npv = mean(nonCall.payoff);
                out.nonCall = nonCall;
                callable.npv = mean(callable.payoff);
                out.callable = callable;
                out.dfCol = dfCol;
                out.daycountFraction = schedule.dayCountFraction;
                
        end
        
        function out = computePriceClosedTRZ(pricer,valueDate)
                security = pricer.m_security;
                numMethod = pricer.m_numMethod;
                model = pricer.m_model;
                schedule = security.couponSchedule;
                
                % numMethodTimeSteps Generate
                mcOneTimeStep = numMethod.mcOneTimeStep;
                outInfo = numMethod.numMethodTimeStepsGenerateDW(valueDate,schedule,mcOneTimeStep);
            
                numMethodInfo.timeStepsInfo = outInfo;

                % mc random number init
                %pathSize = 2^16;
                pathSize = numMethod.pathSize;
                
                pathSize = 1;
                
%                 % generate random numbers and modelstates forward
%                 rng('default');
%                 rng(0);
% 
                numOfFactors = pricer.nFactor;
% 
%                 %Z= randn(numOfFactors*outInfo.totalTimeStepsSize,pathSize/2);
%                 Z= randn(pathSize/2,numOfFactors*outInfo.totalTimeStepsSize);
%                 Z=[Z;-Z];
%                 Z=Z';
%                 
%                 
%                 %TimeInversion
%                 sizeZ = size(Z,1);
%                 A = zeros(sizeZ,sizeZ);
%                 for i=1:sizeZ
%                     A(i,sizeZ + 1 -i) = 1;
%                 end
%                 Z = A*Z;
% 
%                 % we check with the c, so we use generated random numbers
% %                 mc_random = importdata('mc_random.xlsx');
% %                 mc_random = importdata('D:\mc_random_number_new_report.txt');
% %                 Z= mc_random;
%                 
                numMethodInfo.pathSize = pathSize;
                numMethodInfo.numOfFactors = numOfFactors;
                numMethodInfo.totalTimeSteps = outInfo.totalTimeSteps;
                numMethodInfo.totalTimeStepsSize = outInfo.totalTimeStepsSize;
%                 numMethodInfo.Z = Z;
% 
% 
% 
%                 %generate model states for the given timeSteps
%                 modelStates = numMethod.generateModelStates(valueDate,schedule,numMethodInfo,model);

                % numeraire info
                numMatTime = numMethodInfo.totalTimeSteps(end);
                dfNumMat = model.DF(numMatTime/365.0);
                
                % induct backward and calculate cashflow
            
                periodCount = schedule.numOfPeriod;
                
                % cashflow columns initialize
                % 1st nonCall
                nonCall.npv = 0;
                nonCall.cashflow_e = zeros(periodCount + 1,1);
                nonCall.cashflow_npv_e = zeros(periodCount + 1,1);
                nonCall.rangeProb_e = zeros(periodCount + 1,1);
                nonCall.df_e = zeros(periodCount + 1,1);
                nonCall.df_c = zeros(periodCount + 1,1);

                % debug information, cashflow & npv per path
                nonCall.payoff = zeros(1,pathSize);
                nonCall.payoffStates.cashflow = zeros(periodCount+1,pathSize);
                nonCall.payoffStates.cashflow_npv = zeros(periodCount+1,pathSize);
                nonCall.payoffStates.df = zeros(periodCount + 1,pathSize);
                
                % intermediate variables
                payoffStateA.cashflow = zeros(1, pathSize);
                payoffStateA.cashflow_npv = zeros(1, pathSize);
                payoffStateA.df = zeros(1,pathSize);
                
                % we calculate cashflow backward from maturityDate to valueDate
                % rangeAccrual coupons are added at the startDate of the Period
                % At maturity

                nominal = security.nominal;
                coupon = security.coupon;
                callYN = security.callYN;
                
                currentTime = numMethodInfo.totalTimeSteps(end);
                totalTimeSteps = numMethodInfo.timeStepsInfo.totalTimeSteps;
                totalTimeStepsSize = numMethodInfo.timeStepsInfo.totalTimeStepsSize;
                if totalTimeStepsSize == 0 || (totalTimeStepsSize == 1 && totalTimeSteps(1) == 0) 
                    out.nonCall = nonCall;
                    return;
                end
                
                currentTimeIdx = totalTimeStepsSize;
                
                currentNodeIdx = periodCount + 1;
                
%                 modelStateT = modelStates(numOfFactors*(totalTimeStepsSize-1)+1:numOfFactors*(totalTimeStepsSize-1)+2,:);
                payoffStateA.cashflow = nominal*ones(1,pathSize);
                %process payoff at maturity
                nonCall.payoffStates.cashflow(currentNodeIdx,:) = payoffStateA.cashflow;
                
%                 payoffStateA.cashflow_npv = model.discountPayoff(currentTime,numMatTime ...
%                     ,modelStateT,payoffStateA.cashflow);
%                 
                payoffStateA.cashflow_npv = model.DFRisky(currentTime/365.0)*payoffStateA.cashflow;
                
                nonCall.payoffStates.cashflow_npv(currentNodeIdx,:) = payoffStateA.cashflow_npv ;
                
%                 payoffStateA.df = model.discountPayoff(currentTime,numMatTime ...
%                     ,modelStateT,1.0*ones(1,pathSize));
                payoffStateA.df = model.DFRisky(currentTime/365.0) *ones(1,pathSize);
                    

                nonCall.payoffStates.df(currentNodeIdx,:) = payoffStateA.df;
                nonCall.payoff = nonCall.payoff + payoffStateA.cashflow_npv;
                
                nonCall.cashflow_e(currentNodeIdx) = mean(payoffStateA.cashflow);
                nonCall.cashflow_npv_e(currentNodeIdx) = mean(payoffStateA.cashflow_npv);
                nonCall.df_e(currentNodeIdx) = mean(payoffStateA.df);
                nonCall.df_c(currentNodeIdx) = model.DFRisky(currentTime/365.0);
                
%                 % callable at maturity is the same as noncallable
%                 callable.payoffStates.cashflow(currentNodeIdx,:) = nonCall.payoffStates.cashflow(currentNodeIdx,:);
%                 callable.payoffStates.cashflow_npv(currentNodeIdx,:) = nonCall.payoffStates.cashflow_npv(currentNodeIdx,:);
%                 callable.payoff = callable.payoff + callable.payoffStates.cashflow_npv(currentNodeIdx,:);
%                 
%                 callable.cashflow_e(currentNodeIdx) = nonCall.cashflow_e(currentNodeIdx);
%                 callable.cashflow_npv_e(currentNodeIdx) = nonCall.cashflow_npv_e(currentNodeIdx);
                  
                lastTimeIdx = currentTimeIdx;
                
                
                % flag for inclusive schedule
%                 simpleYN = false;
                simpleYN = 1;
                
                for i=periodCount:-1:1
                    % past schedule neglect
                    if DateDiff(valueDate,schedule.payDates(i)) >= 0, continue;end 
                    startTimeIdx = numMethodInfo.timeStepsInfo.startTimeIdx(i);
                    endTimeIdx = numMethodInfo.timeStepsInfo.endTimeIdx(i);

                    inclusiveYN = numMethodInfo.timeStepsInfo.inclusiveYN(i);
                     %% FromToInduction start

                    %modelsStates induct backward(lastTimeIdx to endTimeIdx)
                    while endTimeIdx < lastTimeIdx
                        lastTimeIdx = lastTimeIdx -1;
                        if lastTimeIdx == endTimeIdx break;end
                    end

                    currentTimeIdx = lastTimeIdx;
                    currentTime = totalTimeSteps(currentTimeIdx);
                    % now we are at endTimeIdx
                    % variables for coupon valuation initialized
                    ralower = security.lower;
                    raupper = security.upper;
                    tenorLong = security.tenorL;
                    tenorShort = security.tenorS;

                    cD = 0; % cumulated day count fraction
                    cP = 0; 
                    dI = 0; % dummy idx for loop
                    dcfC = 0;
                    dcfN = 0;
                    dD = 0;
                    idxL = zeros(1,pathSize);
                    idxS = zeros(1,pathSize);
                    spreadC = zeros(1,pathSize);
                    spreadN = zeros(1,pathSize);
                    probSpread = zeros(1,pathSize);
                    
                    numOfTimeSteps = numMethodInfo.timeStepsInfo.numOfTimeSteps(i);
                    
                    % if inclusive then we use simpleYN rangeProb
                    %if inclusive then we do not evaluate the last step but
                    % use fixing instead 
                    endTimeStep = 1;
                    if inclusiveYN
%                         simpleYN = true;
                        simpleYN = 1;
                        endTimeStep = 2;
                    end
                    
                    for j=numOfTimeSteps:-1:endTimeStep
                        % evaluate dealdescription
                        % last step is omitted
                        currentTime = totalTimeSteps(currentTimeIdx);
                        %dcfC = currentTime/365.0;
                        dcfC = currentTime;
                        
                        if j~= numOfTimeSteps
                            dD = (dcfN-dcfC);
                        end

                        toDate = AddDate(valueDate,currentTime,'day');
                        tpDate = schedule.payDates(i);
                        spreadC = model.CMSSpreadDigitalRangeDate(valueDate,toDate,tpDate,tenorLong,tenorShort,0.25,ralower,raupper);
                        
                        %modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:numOfFactors*(currentTimeIdx-1)+2,:);
%                         idxL = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorLong,modelStateT);
%                         idxS = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorShort,modelStateT);
%                         spreadC = idxL - idxS;
%                         probSpread = model.SRPr(spreadC,spreadN,ralower,raupper,simpleYN,pathSize)*dD; 

                        if simpleYN
                            probSpread = spreadC*dD;
                        else
                            probSpread = spreadC*dD;
%                             probSpread = 0.5*(spreadC + spreadN)*dD;
                            
                        end

                        
                        cD = cD + dD;
                        cP = cP + probSpread;

                        dcfN = dcfC;
                        spreadN = spreadC;

                        %induct backward to the next time step
                        % one time backward induction
                        lastTimeIdx = currentTimeIdx;

                        if startTimeIdx < lastTimeIdx
                            lastTimeIdx = lastTimeIdx -1;
                            currentTimeIdx = lastTimeIdx;
                        end


                    end

                    %% FromToInduction end
                    %% AtDate evaluation
                    % Adding a new cashflow to the payoff

                    dcf = schedule.dayCountFraction(i);
                    
                    if inclusiveYN
                        % we add valuation date
                        cD = schedule.dayCount(i);
                        rangeDays =  security.rangeDays(i);
                        payoffStateA.cashflow = nominal * coupon * dcf * (rangeDays(i) + cP)/cD;
                    else
                        payoffStateA.cashflow = nominal * coupon * dcf * cP/cD;
                    end
                    % eventDate(i) = startDate(i) therefore there is no induction in modelStates
                    % i.e. currentTimeIdx has not changed
                    % no need to update modelStateT
                    % modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:end,:);
                    % since cashflow's are added at the startDate(i) we need to
                    % multiply cashflow with the df(startDate(i), payDate(i))
                    payTime = DateDiff(schedule.payDates(i),valueDate);
                    
                    %event is at startDate so we need to discount the
                    %cashflow first from paydate to startdate
%                     df_sp = model.discountFactor(currentTime, payTime,numMatTime,modelStateT);

%                     for k=1:pathSize
%                         payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_sp(k);
%                     end
                    
                    % process payoff
                    
                    currentNodeIdx = i;
                    
                    % 1st nonCall column
                    nonCall.payoffStates.cashflow(currentNodeIdx,:) = payoffStateA.cashflow;
                    payoffStateA.cashflow_npv = payoffStateA.cashflow * model.DFRisky(payTime/365.0);
                    
%                     payoffStateA.cashflow_npv = model.discountPayoff(currentTime,numMatTime ...
%                     ,modelStateT,payoffStateA.cashflow);
                    nonCall.payoffStates.cashflow_npv(currentNodeIdx,:) = payoffStateA.cashflow_npv;
                    payoffStateA.df = model.DFRisky(payTime/365.0);
%                     payoffStateA.df = model.discountPayoff(currentTime,numMatTime ...
%                     ,modelStateT,1.0*ones(1,pathSize));
                    nonCall.payoffStates.df(currentNodeIdx,:) = payoffStateA.df;
                    
                    nonCall.payoff = nonCall.payoff + payoffStateA.cashflow_npv;
                    
                    nonCall.cashflow_e(currentNodeIdx) = mean(payoffStateA.cashflow);
                    nonCall.rangeProb_e(currentNodeIdx) = mean(nonCall.cashflow_e(currentNodeIdx)/(nominal * coupon * dcf));
                    nonCall.cashflow_npv_e(currentNodeIdx) = mean(payoffStateA.cashflow_npv);

                    nonCall.df_e(currentNodeIdx) = mean(payoffStateA.df);
                    nonCall.df_c(currentNodeIdx) = model.DFRisky(currentTime/365.0);
                    
%                     %2nd callable column
%                     callable.payoffStates.cashflow(currentNodeIdx,:) = nonCall.payoffStates.cashflow(currentNodeIdx,:);
%                     callable.payoffStates.cashflow_npv(currentNodeIdx,:) = nonCall.payoffStates.cashflow_npv(currentNodeIdx,:);
%                     callable.payoff = callable.payoff + callable.payoffStates.cashflow_npv(currentNodeIdx,:);
%                     
%                     callable.cashflow_e(currentNodeIdx) = nonCall.cashflow_e(currentNodeIdx);
%                     callable.cashflow_npv_e(currentNodeIdx) = nonCall.cashflow_npv_e(currentNodeIdx);

                    % if exercise is possible
                    if i == 1
                        dummaaa = 1;
                    end
                    
                    % call event happens  at the start of the period
                    % at call date, by default, call option is expired
                    
%                     if callYN(i) && (DateDiff(valueDate,schedule.startDates(i)) < 0)
%                         % 1.undiscount callable.payoff
%                         % 2.build the state variables matrix
%                         % 3.regress 1's with the given state variables
%                         % (model states 1,2)
%                         % 4. exercise per each path in
%                         % discount the new payoff and assign into callable
%                         % column
%                         
%                         %1 undiscount callable.payoff and get the
%                         %contiValue
%                        
%                         %exercise value
%                         exerciseValue = security.nominal*ones(1,pathSize);
%                         contiValue = zeros(1,pathSize);
%                         onePayoff = ones(1,pathSize);
%                         onePayoffD = model.discountPayoff(currentTime,numMatTime ...
%                         ,modelStateT,onePayoff);
%                         contiValue = callable.payoff./onePayoffD;
%                         
%                         % build the state variabls matrix
%                         decisionVar1 = modelStateT(1,:);
%                         decisionVar2 = modelStateT(2,:);
%                         % we use upto 2nd order polynomial as regression
%                         % equatioin
%                         dimensionLS = 6;
%                         regMatrix = ones(pathSize,dimensionLS);
%                         
%                         for idx_r=1:pathSize
%                             regMatrix(idx_r,2) = decisionVar1(idx_r);
%                             regMatrix(idx_r,3) = decisionVar2(idx_r);
%                             regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r);
%                             regMatrix(idx_r,5) = decisionVar1(idx_r)*decisionVar2(idx_r);
%                             regMatrix(idx_r,6) = decisionVar2(idx_r)*decisionVar2(idx_r);
%                         end
%                         
%                         % simple regression <- for path dependent option
%                         % we should change it
%                         % beta  : regression coefficients
%                         beta =zeros(dimensionLS,1);
%                         beta = regress(contiValue',regMatrix);
%                         regContiValue = regMatrix*beta;
%                         
%                         exercisedPayoff = regContiValue;
%                         % if regContiValue is bigger than exerciseValue
%                         % then issue exercise call option
%                         for idx_r=1:pathSize
%                             if regContiValue(idx_r) >= exerciseValue(idx_r) 
%                                 exercisedPayoff(idx_r) = exerciseValue(idx_r);
%                             end
%                         end
%                         
%                         % we discount the new payoff and 
%                         % assign it into the existing callable payoff
%                         callable.payoff = model.discountPayoff(currentTime,numMatTime ...
%                         ,modelStateT,exercisedPayoff);
%                         
%                     end
                end

                nonCall.npv = mean(nonCall.payoff);
                out.pricerInfo.nonCall = nonCall;
                
%                 callable.npv = mean(callable.payoff);
%                 out.pricerInfo.callable = callable;
                out.daycountFraction = schedule.dayCountFraction;
                
        end
        
        
        
        
    end
    
end



In [ ]:
classdef DailyWeeklyRiskyPricerExFlag_CMSAVGSteepener < DailyWeeklyRiskyPricer
    %Pricer(Security,Model,NumMethod)
    %Prcer is constructed from Security, Model, NumMethod 
    
    properties
    end
    
    methods
%         function pricer = DailyWeeklyRiskyPricerExFlag_CMSAVGSteepener(security,model,numMethod,overHedgeYN)
%                 pricer.m_security = security;
%                 pricer.m_model = model;
%                 pricer.m_numMethod = numMethod;
%                 pricer.overHedgeYN = overHedgeYN;
%         end
        
        function out = computePriceMCCallable(pricer,valueDate)
                security = pricer.m_security;
                numMethod = pricer.m_numMethod;
                model = pricer.m_model;
                schedule = security.couponSchedule;
                callSchedule = security.callSchedule;
                isExercised = pricer.m_security.isExercised;
                
                % numMethodTimeSteps Generate
                mcOneTimeStep = numMethod.mcOneTimeStep;
                outInfo = numMethod.numMethodTimeStepsGenerateDW(valueDate,schedule,mcOneTimeStep);
            
                numMethodInfo.timeStepsInfo = outInfo;

                % mc random number init
                %pathSize = 2^16;
                pathSize = numMethod.pathSize;

                % generate random numbers and modelstates forward
                rng('default');
                rng(0);

                numOfFactors = 2;

                %Z= randn(numOfFactors*outInfo.totalTimeStepsSize,pathSize/2);
                Z= randn(pathSize/2,numOfFactors*outInfo.totalTimeStepsSize);
                Z=[Z;-Z];
                Z=Z';
                
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;

                % we check with the c code, so we use generated random numbers
%                 mc_random = importdata('mc_random.xlsx');
%                 mc_random = importdata('D:\mc_random_number_new_report.txt');
%                 Z= mc_random;
                
                numMethodInfo.pathSize = pathSize;
                numMethodInfo.numOfFactors = numOfFactors;
                numMethodInfo.totalTimeSteps = outInfo.totalTimeSteps;
                numMethodInfo.totalTimeStepsSize = outInfo.totalTimeStepsSize;
                numMethodInfo.Z = Z;



                %generate model states for the given timeSteps
                modelStates = numMethod.generateModelStates(valueDate,schedule,numMethodInfo,model);

                % numeraire info
                numMatTime = numMethodInfo.totalTimeSteps(end);
                days = DateDiff(schedule.payDates(schedule.numOfPeriod),valueDate);
                numMatTime = max(numMatTime, days);
                dfNumMat = model.DFRisky(numMatTime/365.0);
                
                numeraire.numMatTime = numMatTime;
                numeraire.dfNumMat = dfNumMat;
                
                % induct backward and calculate cashflow
            
                periodCount = schedule.numOfPeriod;
                
                % pricer single info initialize
                
                nonCall = PricerInfo(periodCount +1, pathSize);
                callable = PricerInfo(periodCount +1, pathSize);
                dfCol = PricerInfo(periodCount +1, pathSize);
                
                % if isExercised flag has true value return npv from
                % nonCalllExercised pricerInfo
                nonCallExercised = PricerInfo(periodCount + 1, pathSize);
                
                dummy1 = PricerInfo(periodCount +1, pathSize);
                dummy2 = PricerInfo(periodCount +1, pathSize);
                
                
                % intermediate variables
                payoffStateA.cashflow = zeros(1, pathSize);
                payoffStateA.cashflow_npv = zeros(1, pathSize);
                
            
                % we calculate cashflow backward from maturityDate to valueDate
                % rangeAccrual coupons are added at the startDate of the Period
                % At maturity

                nominal = security.nominal;
%                 coupon = security.coupon;

                localCap = security.localCap;
                localFloor = security.localFloor;
                multiplier = security.multiplier;
                spread = security.spread;
                
                callYN = security.callYN;
                
%                 ralower = security.lower;
%                 raupper = security.upper;

                tenorLong = security.tenorL;
                tenorShort = security.tenorS;
                
                currentTime = numMethodInfo.totalTimeSteps(end);
                totalTimeStepsSize = numMethodInfo.totalTimeStepsSize;
                currentTimeIdx = totalTimeStepsSize;
                
                currentNodeIdx = periodCount + 1;
                
                modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:numOfFactors*(currentTimeIdx-1)+2,:);
                payoffStateA.cashflow = nominal*ones(1,pathSize);
                
                payTime = DateDiff(schedule.payDates(schedule.numOfPeriod),valueDate);
                df_ep = model.discountFactorRisky(currentTime, payTime,numMatTime,modelStateT);
                for k=1:pathSize
                    payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                end
                
                %process payoff at maturity
                nonCall.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                
                callable.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                
                %df column
                payoffStateA.cashflow = ones(1,pathSize);
                for k=1:pathSize
                    payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                end
                
                dfCol.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                                 
                %dummy column
                payoffStateA.cashflow = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorLong,modelStateT);
                dummy1.ProcessPayoff(model,currentTime,numeraire, ...
                                 modelStateT,payoffStateA.cashflow,currentNodeIdx);

                payoffStateA.cashflow = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                dummy2.ProcessPayoff(model,currentTime,numeraire, ...
                                 modelStateT,payoffStateA.cashflow,currentNodeIdx);
                                 
                  

                lastTimeIdx = currentTimeIdx;
                totalTimeSteps = numMethodInfo.timeStepsInfo.totalTimeSteps;
                
                % flag for inclusive schedule
%                 simpleYN = false;
%                 simpleYN = true;
                if pricer.overHedgeYN == true 
                    simpleYN = 2;
                else
                    simpleYN = 1;
                end
                
                for i=periodCount:-1:1
                    % past schedule neglect
                    if DateDiff(valueDate,schedule.payDates(i)) >= 0, continue;end 
                    
                    startTimeIdx = numMethodInfo.timeStepsInfo.startTimeIdx(i);
                    endTimeIdx = numMethodInfo.timeStepsInfo.endTimeIdx(i);

                    inclusiveYN = numMethodInfo.timeStepsInfo.inclusiveYN(i);
                    
                    % if isExercised flag has true value process current
                    % period's cashflow only
                    if isExercised && ~inclusiveYN
                        continue;
                    end
                    
                 %% FromToInduction start

                    %modelsStates induct backward(lastTimeIdx to endTimeIdx)
                    while endTimeIdx < lastTimeIdx
                        lastTimeIdx = lastTimeIdx -1;
                        if lastTimeIdx == endTimeIdx break;end
                    end

                    currentTimeIdx = lastTimeIdx;
                    currentTime = totalTimeSteps(currentTimeIdx);
                    % now we are at endTimeIdx
                    % variables for coupon valuation initialized
                    

                    cD = 0; % cumulated day count fraction
                    cP = 0; 
                    dI = 0; % dummy idx for loop
                    dcfC = 0;
                    dcfN = 0;
                    dD = 0;
                    idxL = zeros(1,pathSize);
                    idxS = zeros(1,pathSize);
                    spreadC = zeros(1,pathSize);
                    spreadN = zeros(1,pathSize);
                    probSpread = zeros(1,pathSize);
                    
                    numOfTimeSteps = numMethodInfo.timeStepsInfo.numOfTimeSteps(i);
                    
                    % if inclusive then we use simpleYN rangeProb
                    %if inclusive then we do not evaluate the last step but
                    % use fixing instead 
                    endTimeStep = 1;
                    if inclusiveYN
%                         simpleYN = true;
                        simpleYN = 1;
                        endTimeStep = 2;
                        if pricer.overHedgeYN == true 
                            simpleYN = 2;
                        end
                    end
                    
                    for j=numOfTimeSteps:-1:endTimeStep
                        % evaluate dealdescription
                        % last step is omitted
                        currentTime = totalTimeSteps(currentTimeIdx);
                        %dcfC = currentTime/365.0;
                        dcfC = currentTime;
                        
                        if j~= numOfTimeSteps
                            dD = (dcfN-dcfC);
                        end

                        modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:numOfFactors*(currentTimeIdx-1)+2,:);
                        idxL = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorLong,modelStateT);
                        idxS = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                        spreadC = idxL - idxS;

%                         probSpread = model.SRPr(spreadC,spreadN,ralower,raupper,simpleYN,pathSize)*dD;
                        probSpread = spreadC*dD;
                        
                        cD = cD + dD;
                        cP = cP + probSpread;

                        dcfN = dcfC;
                        spreadN = spreadC;

                        %induct backward to the next time step
                        % one time backward induction
                        lastTimeIdx = currentTimeIdx;

                        if startTimeIdx < lastTimeIdx
                            lastTimeIdx = lastTimeIdx -1;
                            currentTimeIdx = lastTimeIdx;
                        end


                    end

                    %% FromToInduction end
        % if the current period is inclusive, then the start date is not evaluated in the above for loop.
		% instead the fixing will be treated separately below.
		% however current_time_idx is updated at the end of the for loop.
		% so we need to update model_states accordingly.
		% if model_states were updated previously then the following additionaly step doesn't affect the result anyway.
                    currentTime = totalTimeSteps(currentTimeIdx);
                    modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:numOfFactors*(currentTimeIdx-1)+2,:);
                    
                    %% AtDate evaluation
                    % Adding a new cashflow to the payoff

                    dcf = schedule.dayCountFraction(i);
                    
                    if inclusiveYN
                        % we add valuation date
                        cD = schedule.dayCount(i);
                        rangeDays =  security.rangeDays(i);
                        payoffStateA.cashflow = max(min(multiplier * (rangeDays + cP)/cD + spread,localCap),localFloor)*dcf*nominal;
                    else
                        payoffStateA.cashflow = max(min(multiplier * (0         + cP)/cD + spread,localCap),localFloor)*dcf*nominal;
                    end
                    % eventDate(i) = startDate(i) therefore there is no induction in modelStates
                    % i.e. currentTimeIdx has not changed
                    % no need to update modelStateT
                    % modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:end,:);
                    % since cashflow's are added at the startDate(i) we need to
                    % multiply cashflow with the df(startDate(i), payDate(i))
                    payTime = DateDiff(schedule.payDates(i),valueDate);
                    
                    %event is at startDate so we need to discount the
                    %cashflow first from paydate to startdate
                    df_ep = model.discountFactorRisky(currentTime, payTime,numMatTime,modelStateT);

                    for k=1:pathSize
                        payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                    end
                    
                    % process payoff
                    
                    currentNodeIdx = i;
                    
                    nonCall.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                
                    callable.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                    
                    % Process payoff for nonCallExercised which is 1 period
                    % noncallable range accrual bond
                    if isExercised
                        for k=1:pathSize
                            payoffStateA.cashflow(k) = payoffStateA.cashflow(k) + (nominal * df_ep(k));
                        end
                        
                        nonCallExercised.ProcessPayoff(model, currentTime, numeraire, ...
                                     modelStateT, payoffStateA.cashflow, currentNodeIdx);
                    end
                    
                    %df column
                    payoffStateA.cashflow = ones(1,pathSize);
                    for k=1:pathSize
                        payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                    end
                    
                    dfCol.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                                 
                    %dummy column
                    payoffStateA.cashflow = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorLong,modelStateT);
                    dummy1.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                    
                    payoffStateA.cashflow = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                    dummy2.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                    
                    
                    % if exercise is possible
                    if i == 1
                        dummaaa = 1;
                    end
                    
                    % call event happens  at the start of the period
                    % at call date, by default, call option is expired
                    
                    if callYN(i) && (DateDiff(valueDate,callSchedule.resetDates(i)) < 0) && not(isExercised)
                        % 1.undiscount callable.payoff
                        % 2.build the state variables matrix
                        % 3.regress 1's with the given state variables
                        % (model states 1,2)
                        % 4. exercise per each path in
                        % discount the new payoff and assign into callable
                        % column
                        
                        %1 undiscount callable.payoff and get the
                        %contiValue
                       
                        %exercise value
                        exerciseValue = security.nominal*ones(1,pathSize);
                        payTime = DateDiff(callSchedule.payDates(i),valueDate);
                        %event is at startDate so we need to discount the
                        %cashflow first from paydate to startdate
                        df_ep = model.discountFactorRisky(currentTime, payTime,numMatTime,modelStateT);
                        for k=1:pathSize
                            exerciseValue(k) = exerciseValue(k) * df_ep(k);
                        end
                        
                        contiValue = zeros(1,pathSize);
                        onePayoff = ones(1,pathSize);
                        onePayoffD = model.discountPayoffRisky(currentTime,numMatTime ...
                        ,modelStateT,onePayoff);
                        contiValue = callable.payoff./onePayoffD;
                        
                        %Nominal Regression test
                        contiValue = contiValue/10000;
                        
                        % build the state variabls matrix
                        decisionVar1 = modelStateT(1,:);
                        decisionVar2 = modelStateT(2,:);
                        % we use upto 2nd order polynomial as regression
                        % equatioin
                        dimensionLS = 6;
                        regMatrix = ones(pathSize,dimensionLS);
                        
                        for idx_r=1:pathSize
                            regMatrix(idx_r,2) = decisionVar1(idx_r);
                            regMatrix(idx_r,3) = decisionVar2(idx_r);
                            regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r);
                            regMatrix(idx_r,5) = decisionVar1(idx_r)*decisionVar2(idx_r);
                            regMatrix(idx_r,6) = decisionVar2(idx_r)*decisionVar2(idx_r);
                        end
                        
                        % simple regression <- for path dependent option
                        % we should change it
                        % beta  : regression coefficients
                        beta =zeros(dimensionLS,1);
                        beta = regress(contiValue',regMatrix);
                        regContiValue = regMatrix*beta;
                       
                         %Nominal Regression test
                        regContiValue = regContiValue * 10000;
                        
                        exercisedPayoff = regContiValue;
                        % if regContiValue is bigger than exerciseValue
                        % then issue exercise call option
                        for idx_r=1:pathSize
                            if regContiValue(idx_r) >= exerciseValue(idx_r) 
                                exercisedPayoff(idx_r) = exerciseValue(idx_r);
                            end
                        end
                        
                        % we discount the new payoff and 
                        % assign it into the existing callable payoff
                        callable.payoff = model.discountPayoffRisky(currentTime,numMatTime ...
                        ,modelStateT,exercisedPayoff);
                        
                    end
                end

                nonCall.npv = mean(nonCall.payoff);
                out.nonCall = nonCall;
                callable.npv = mean(callable.payoff);
                out.callable = callable;
                out.dfCol = dfCol;
                out.dummy1 = dummy1;
                out.dummy2 = dummy2;
                out.daycountFraction = schedule.dayCountFraction;
                
                if isExercised
                    nonCallExercised.npv = mean(nonCallExercised.payoff);
                    out.nonCall = nonCallExercised;
                    out.callable = nonCallExercised;
                end
        end
    end
end

In [ ]:
classdef DailyWeeklyRiskyPricerExFlag < DailyWeeklyRiskyPricer
    %Pricer(Security,Model,NumMethod)
    %Prcer is constructed from Security, Model, NumMethod 
    
    properties
    end
    
    methods
%         function pricer = DailyWeeklyRiskyPricerExFlag(security,model,numMethod,overHedgeYN)
%                 pricer.m_security = security;
%                 pricer.m_model = model;
%                 pricer.m_numMethod = numMethod;
%                 pricer.overHedgeYN = overHedgeYN;
%         end
        
        function out = computePriceMCCallable(pricer,valueDate)
                security = pricer.m_security;
                numMethod = pricer.m_numMethod;
                model = pricer.m_model;
                schedule = security.couponSchedule;
                callSchedule = security.callSchedule;
                isExercised = pricer.m_security.isExercised;
                
                % numMethodTimeSteps Generate
                mcOneTimeStep = numMethod.mcOneTimeStep;
                outInfo = numMethod.numMethodTimeStepsGenerateDW(valueDate,schedule,mcOneTimeStep);
            
                numMethodInfo.timeStepsInfo = outInfo;

                % mc random number init
                %pathSize = 2^16;
                pathSize = numMethod.pathSize;

                % generate random numbers and modelstates forward
                rng('default');
                rng(0);

                numOfFactors = 2;

                %Z= randn(numOfFactors*outInfo.totalTimeStepsSize,pathSize/2);
                Z= randn(pathSize/2,numOfFactors*outInfo.totalTimeStepsSize);
                Z=[Z;-Z];
                Z=Z';
                
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;

                % we check with the c code, so we use generated random numbers
%                 mc_random = importdata('mc_random.xlsx');
%                 mc_random = importdata('D:\mc_random_number_new_report.txt');
%                 Z= mc_random;
                
                numMethodInfo.pathSize = pathSize;
                numMethodInfo.numOfFactors = numOfFactors;
                numMethodInfo.totalTimeSteps = outInfo.totalTimeSteps;
                numMethodInfo.totalTimeStepsSize = outInfo.totalTimeStepsSize;
                numMethodInfo.Z = Z;



                %generate model states for the given timeSteps
                modelStates = numMethod.generateModelStates(valueDate,schedule,numMethodInfo,model);

                % numeraire info
                numMatTime = numMethodInfo.totalTimeSteps(end);
                days = DateDiff(schedule.payDates(schedule.numOfPeriod),valueDate);
                numMatTime = max(numMatTime, days);
                dfNumMat = model.DFRisky(numMatTime/365.0);
                
                numeraire.numMatTime = numMatTime;
                numeraire.dfNumMat = dfNumMat;
                
                % induct backward and calculate cashflow
            
                periodCount = schedule.numOfPeriod;
                
                % pricer single info initialize
                
                nonCall = PricerInfo(periodCount +1, pathSize);
                callable = PricerInfo(periodCount +1, pathSize);
                dfCol = PricerInfo(periodCount +1, pathSize);
                
                % if isExercised flag has true value return npv from
                % nonCalllExercised pricerInfo
                nonCallExercised = PricerInfo(periodCount + 1, pathSize);
                
                dummy1 = PricerInfo(periodCount +1, pathSize);
                dummy2 = PricerInfo(periodCount +1, pathSize);
                
                
                % intermediate variables
                payoffStateA.cashflow = zeros(1, pathSize);
                payoffStateA.cashflow_npv = zeros(1, pathSize);
                
            
                % we calculate cashflow backward from maturityDate to valueDate
                % rangeAccrual coupons are added at the startDate of the Period
                % At maturity

                nominal = security.nominal;
                coupon = security.coupon;
                callYN = security.callYN;
                
                ralower = security.lower;
                raupper = security.upper;
                tenorLong = security.tenorL;
                tenorShort = security.tenorS;
                
                currentTime = numMethodInfo.totalTimeSteps(end);
                totalTimeStepsSize = numMethodInfo.totalTimeStepsSize;
                currentTimeIdx = totalTimeStepsSize;
                
                currentNodeIdx = periodCount + 1;
                
                modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:numOfFactors*(currentTimeIdx-1)+2,:);
                payoffStateA.cashflow = nominal*ones(1,pathSize);
                
                payTime = DateDiff(schedule.payDates(schedule.numOfPeriod),valueDate);
                df_ep = model.discountFactorRisky(currentTime, payTime,numMatTime,modelStateT);
                for k=1:pathSize
                    payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                end
                
                %process payoff at maturity
                nonCall.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                
                callable.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                
                %df column
                payoffStateA.cashflow = ones(1,pathSize);
                for k=1:pathSize
                    payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                end
                
                dfCol.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                                 
                %dummy column
                payoffStateA.cashflow = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorLong,modelStateT);
                dummy1.ProcessPayoff(model,currentTime,numeraire, ...
                                 modelStateT,payoffStateA.cashflow,currentNodeIdx);

                payoffStateA.cashflow = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                dummy2.ProcessPayoff(model,currentTime,numeraire, ...
                                 modelStateT,payoffStateA.cashflow,currentNodeIdx);
                                 
                  

                lastTimeIdx = currentTimeIdx;
                totalTimeSteps = numMethodInfo.timeStepsInfo.totalTimeSteps;
                
                % flag for inclusive schedule
%                 simpleYN = false;
%                 simpleYN = true;
                if pricer.overHedgeYN == true 
                    simpleYN = 2;
                else
                    simpleYN = 1;
                end
                
                for i=periodCount:-1:1
                    % past schedule neglect
                    if DateDiff(valueDate,schedule.payDates(i)) >= 0, continue;end 
                    
                    startTimeIdx = numMethodInfo.timeStepsInfo.startTimeIdx(i);
                    endTimeIdx = numMethodInfo.timeStepsInfo.endTimeIdx(i);

                    inclusiveYN = numMethodInfo.timeStepsInfo.inclusiveYN(i);
                    
                    % if isExercised flag has true value process current
                    % period's cashflow only
                    if isExercised && ~inclusiveYN
                        continue;
                    end
                    
                 %% FromToInduction start

                    %modelsStates induct backward(lastTimeIdx to endTimeIdx)
                    while endTimeIdx < lastTimeIdx
                        lastTimeIdx = lastTimeIdx -1;
                        if lastTimeIdx == endTimeIdx break;end
                    end

                    currentTimeIdx = lastTimeIdx;
                    currentTime = totalTimeSteps(currentTimeIdx);
                    % now we are at endTimeIdx
                    % variables for coupon valuation initialized
                    

                    cD = 0; % cumulated day count fraction
                    cP = 0; 
                    dI = 0; % dummy idx for loop
                    dcfC = 0;
                    dcfN = 0;
                    dD = 0;
                    idxL = zeros(1,pathSize);
                    idxS = zeros(1,pathSize);
                    spreadC = zeros(1,pathSize);
                    spreadN = zeros(1,pathSize);
                    probSpread = zeros(1,pathSize);
                    
                    numOfTimeSteps = numMethodInfo.timeStepsInfo.numOfTimeSteps(i);
                    
                    % if inclusive then we use simpleYN rangeProb
                    %if inclusive then we do not evaluate the last step but
                    % use fixing instead 
                    endTimeStep = 1;
                    if inclusiveYN
%                         simpleYN = true;
                        simpleYN = 1;
                        endTimeStep = 2;
                        if pricer.overHedgeYN == true 
                            simpleYN = 2;
                        end
                    end
                    
                    for j=numOfTimeSteps:-1:endTimeStep
                        % evaluate dealdescription
                        % last step is omitted
                        currentTime = totalTimeSteps(currentTimeIdx);
                        %dcfC = currentTime/365.0;
                        dcfC = currentTime;
                        
                        if j~= numOfTimeSteps
                            dD = (dcfN-dcfC);
                        end

                        modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:numOfFactors*(currentTimeIdx-1)+2,:);
                        idxL = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorLong,modelStateT);
                        idxS = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                        spreadC = idxL - idxS;

                        probSpread = model.SRPr(spreadC,spreadN,ralower,raupper,simpleYN,pathSize)*dD;
                        cD = cD + dD;
                        cP = cP + probSpread;

                        dcfN = dcfC;
                        spreadN = spreadC;

                        %induct backward to the next time step
                        % one time backward induction
                        lastTimeIdx = currentTimeIdx;

                        if startTimeIdx < lastTimeIdx
                            lastTimeIdx = lastTimeIdx -1;
                            currentTimeIdx = lastTimeIdx;
                        end


                    end

                    %% FromToInduction end
        % if the current period is inclusive, then the start date is not evaluated in the above for loop.
		% instead the fixing will be treated separately below.
		% however current_time_idx is updated at the end of the for loop.
		% so we need to update model_states accordingly.
		% if model_states were updated previously then the following additionaly step doesn't affect the result anyway.
                    currentTime = totalTimeSteps(currentTimeIdx);
                    modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:numOfFactors*(currentTimeIdx-1)+2,:);
                    
                    %% AtDate evaluation
                    % Adding a new cashflow to the payoff

                    dcf = schedule.dayCountFraction(i);
                    
                    if inclusiveYN
                        % we add valuation date
                        cD = schedule.dayCount(i);
                        rangeDays =  security.rangeDays(i);
                        payoffStateA.cashflow = nominal * coupon * dcf * (rangeDays + cP)/cD;
                    else
                        payoffStateA.cashflow = nominal * coupon * dcf * cP/cD;
                    end
                    % eventDate(i) = startDate(i) therefore there is no induction in modelStates
                    % i.e. currentTimeIdx has not changed
                    % no need to update modelStateT
                    % modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:end,:);
                    % since cashflow's are added at the startDate(i) we need to
                    % multiply cashflow with the df(startDate(i), payDate(i))
                    payTime = DateDiff(schedule.payDates(i),valueDate);
                    
                    %event is at startDate so we need to discount the
                    %cashflow first from paydate to startdate
                    df_ep = model.discountFactorRisky(currentTime, payTime,numMatTime,modelStateT);

                    for k=1:pathSize
                        payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                    end
                    
                    % process payoff
                    
                    currentNodeIdx = i;
                    
                    nonCall.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                
                    callable.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                    
                    % Process payoff for nonCallExercised which is 1 period
                    % noncallable range accrual bond
                    if isExercised
                        for k=1:pathSize
                            payoffStateA.cashflow(k) = payoffStateA.cashflow(k) + (nominal * df_ep(k));
                        end
                        
                        nonCallExercised.ProcessPayoff(model, currentTime, numeraire, ...
                                     modelStateT, payoffStateA.cashflow, currentNodeIdx);
                    end
                    
                    %df column
                    payoffStateA.cashflow = ones(1,pathSize);
                    for k=1:pathSize
                        payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                    end
                    
                    dfCol.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                                 
                    %dummy column
                    payoffStateA.cashflow = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorLong,modelStateT);
                    dummy1.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                    
                    payoffStateA.cashflow = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                    dummy2.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                    
                    
                    % if exercise is possible
                    if i == 1
                        dummaaa = 1;
                    end
                    
                    % call event happens  at the start of the period
                    % at call date, by default, call option is expired
                    
                    if callYN(i) && (DateDiff(valueDate,callSchedule.resetDates(i)) < 0) && not(isExercised)
                        % 1.undiscount callable.payoff
                        % 2.build the state variables matrix
                        % 3.regress 1's with the given state variables
                        % (model states 1,2)
                        % 4. exercise per each path in
                        % discount the new payoff and assign into callable
                        % column
                        
                        %1 undiscount callable.payoff and get the
                        %contiValue
                       
                        %exercise value
                        exerciseValue = security.nominal*ones(1,pathSize);
                        payTime = DateDiff(callSchedule.payDates(i),valueDate);
                        %event is at startDate so we need to discount the
                        %cashflow first from paydate to startdate
                        df_ep = model.discountFactorRisky(currentTime, payTime,numMatTime,modelStateT);
                        for k=1:pathSize
                            exerciseValue(k) = exerciseValue(k) * df_ep(k);
                        end
                        
                        contiValue = zeros(1,pathSize);
                        onePayoff = ones(1,pathSize);
                        onePayoffD = model.discountPayoffRisky(currentTime,numMatTime ...
                        ,modelStateT,onePayoff);
                        contiValue = callable.payoff./onePayoffD;
                        
                        %Nominal Regression test
                        contiValue = contiValue/10000;
                        
                        % build the state variabls matrix
                        decisionVar1 = modelStateT(1,:);
                        decisionVar2 = modelStateT(2,:);
                        % we use upto 2nd order polynomial as regression
                        % equatioin
                        dimensionLS = 6;
                        regMatrix = ones(pathSize,dimensionLS);
                        
                        for idx_r=1:pathSize
                            regMatrix(idx_r,2) = decisionVar1(idx_r);
                            regMatrix(idx_r,3) = decisionVar2(idx_r);
                            regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r);
                            regMatrix(idx_r,5) = decisionVar1(idx_r)*decisionVar2(idx_r);
                            regMatrix(idx_r,6) = decisionVar2(idx_r)*decisionVar2(idx_r);
                        end
                        
                        % simple regression <- for path dependent option
                        % we should change it
                        % beta  : regression coefficients
                        beta =zeros(dimensionLS,1);
                        beta = regress(contiValue',regMatrix);
                        regContiValue = regMatrix*beta;
                       
                         %Nominal Regression test
                        regContiValue = regContiValue * 10000;
                        
                        exercisedPayoff = regContiValue;
                        % if regContiValue is bigger than exerciseValue
                        % then issue exercise call option
                        for idx_r=1:pathSize
                            if regContiValue(idx_r) >= exerciseValue(idx_r) 
                                exercisedPayoff(idx_r) = exerciseValue(idx_r);
                            end
                        end
                        
                        % we discount the new payoff and 
                        % assign it into the existing callable payoff
                        callable.payoff = model.discountPayoffRisky(currentTime,numMatTime ...
                        ,modelStateT,exercisedPayoff);
                        
                    end
                end

                nonCall.npv = mean(nonCall.payoff);
                out.nonCall = nonCall;
                callable.npv = mean(callable.payoff);
                out.callable = callable;
                out.dfCol = dfCol;
                out.dummy1 = dummy1;
                out.dummy2 = dummy2;
                out.daycountFraction = schedule.dayCountFraction;
                
                if isExercised
                    nonCallExercised.npv = mean(nonCallExercised.payoff);
                    out.nonCall = nonCallExercised;
                    out.callable = nonCallExercised;
                end
        end
    end
end

In [ ]:
classdef DailyWeeklyRiskyPricer3 < H_RootObject
    %Pricer(Security,Model,NumMethod)
    %Prcer is constructed from Security, Model, NumMethod 
    
    properties
        m_security
        m_model
        m_numMethod
        overHedgeYN
    end
    
    methods
        function pricer = DailyWeeklyRiskyPricer3(security,model,numMethod,overHedgeYN)
                pricer.m_security = security;
                pricer.m_model = model;
                pricer.m_numMethod = numMethod;
                pricer.overHedgeYN = overHedgeYN;
        end
        
        function out = computePriceMCCallable(pricer,valueDate)
                security = pricer.m_security;
                numMethod = pricer.m_numMethod;
                model = pricer.m_model;
                schedule = security.couponSchedule;
                callSchedule = security.callSchedule;
                
                % numMethodTimeSteps Generate
                mcOneTimeStep = numMethod.mcOneTimeStep;
                outInfo = numMethod.numMethodTimeStepsGenerateDW(valueDate,schedule,mcOneTimeStep);
            
                numMethodInfo.timeStepsInfo = outInfo;

                % mc random number init
                %pathSize = 2^16;
                pathSize = numMethod.pathSize;

                % generate random numbers and modelstates forward
                rng('default');
                rng(0);

                numOfFactors = 3;

                %Z= randn(numOfFactors*outInfo.totalTimeStepsSize,pathSize/2);
                Z= randn(pathSize/2,numOfFactors*outInfo.totalTimeStepsSize);
                Z=[Z;-Z];
                Z=Z';
                
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;

                % we check with the c code, so we use generated random numbers
%                 mc_random = importdata('mc_random.xlsx');
%                 mc_random = importdata('D:\mc_random_number_new_report.txt');
%                 Z= mc_random;
                
                numMethodInfo.pathSize = pathSize;
                numMethodInfo.numOfFactors = numOfFactors;
                numMethodInfo.totalTimeSteps = outInfo.totalTimeSteps;
                numMethodInfo.totalTimeStepsSize = outInfo.totalTimeStepsSize;
                numMethodInfo.Z = Z;



                %generate model states for the given timeSteps
                modelStates = numMethod.generateModelStates3(valueDate,schedule,numMethodInfo,model);

                % numeraire info
                numMatTime = numMethodInfo.totalTimeSteps(end);
                days = DateDiff(schedule.payDates(schedule.numOfPeriod),valueDate);
                numMatTime = max(numMatTime, days);
                dfNumMat = model.DFRisky(numMatTime/365.0);
                
                numeraire.numMatTime = numMatTime;
                numeraire.dfNumMat = dfNumMat;
                
                % induct backward and calculate cashflow
            
                periodCount = schedule.numOfPeriod;
                
                % pricer single info initialize
                
                nonCall = PricerInfo(periodCount +1, pathSize);
                callable = PricerInfo(periodCount +1, pathSize);
                dfCol = PricerInfo(periodCount +1, pathSize);
                
                dummy1 = PricerInfo(periodCount +1, pathSize);
                dummy2 = PricerInfo(periodCount +1, pathSize);
                
                
                % intermediate variables
                payoffStateA.cashflow = zeros(1, pathSize);
                payoffStateA.cashflow_npv = zeros(1, pathSize);
                
            
                % we calculate cashflow backward from maturityDate to valueDate
                % rangeAccrual coupons are added at the startDate of the Period
                % At maturity

                nominal = security.nominal;
                coupon = security.coupon;
                callYN = security.callYN;
                
                ralower = security.lower;
                raupper = security.upper;
                tenorLong = security.tenorL;
                tenorShort = security.tenorS;
                
                currentTime = numMethodInfo.totalTimeSteps(end);
                totalTimeStepsSize = numMethodInfo.totalTimeStepsSize;
                currentTimeIdx = totalTimeStepsSize;
                
                currentNodeIdx = periodCount + 1;
                
                modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:numOfFactors*(currentTimeIdx-1)+3,:);
                payoffStateA.cashflow = nominal*ones(1,pathSize);
                
                payTime = DateDiff(schedule.payDates(schedule.numOfPeriod),valueDate);
                df_ep = model.discountFactorRisky(currentTime, payTime,numMatTime,modelStateT);
                for k=1:pathSize
                    payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                end
                
                %process payoff at maturity
                nonCall.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                
                callable.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                
                %df column
                payoffStateA.cashflow = ones(1,pathSize);
                for k=1:pathSize
                    payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                end
                
                dfCol.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                                 
                %dummy column
                payoffStateA.cashflow = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorLong,modelStateT);
                dummy1.ProcessPayoff(model,currentTime,numeraire, ...
                                 modelStateT,payoffStateA.cashflow,currentNodeIdx);

                payoffStateA.cashflow = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                dummy2.ProcessPayoff(model,currentTime,numeraire, ...
                                 modelStateT,payoffStateA.cashflow,currentNodeIdx);
                                 
                  

                lastTimeIdx = currentTimeIdx;
                totalTimeSteps = numMethodInfo.timeStepsInfo.totalTimeSteps;
                
                % flag for inclusive schedule
%                 simpleYN = false;
%                 simpleYN = true;
                if pricer.overHedgeYN == true 
                    simpleYN = 2;
                else
                    simpleYN = 1;
                end
                
                for i=periodCount:-1:1
                    % past schedule neglect
                    if DateDiff(valueDate,schedule.payDates(i)) >= 0, continue;end 
                    startTimeIdx = numMethodInfo.timeStepsInfo.startTimeIdx(i);
                    endTimeIdx = numMethodInfo.timeStepsInfo.endTimeIdx(i);

                    inclusiveYN = numMethodInfo.timeStepsInfo.inclusiveYN(i);
                 %% FromToInduction start

                    %modelsStates induct backward(lastTimeIdx to endTimeIdx)
                    while endTimeIdx < lastTimeIdx
                        lastTimeIdx = lastTimeIdx -1;
                        if lastTimeIdx == endTimeIdx break;end
                    end

                    currentTimeIdx = lastTimeIdx;
                    currentTime = totalTimeSteps(currentTimeIdx);
                    % now we are at endTimeIdx
                    % variables for coupon valuation initialized
                    

                    cD = 0; % cumulated day count fraction
                    cP = 0; 
                    dI = 0; % dummy idx for loop
                    dcfC = 0;
                    dcfN = 0;
                    dD = 0;
                    idxL = zeros(1,pathSize);
                    idxS = zeros(1,pathSize);
                    spreadC = zeros(1,pathSize);
                    spreadN = zeros(1,pathSize);
                    probSpread = zeros(1,pathSize);
                    
                    numOfTimeSteps = numMethodInfo.timeStepsInfo.numOfTimeSteps(i);
                    
                    % if inclusive then we use simpleYN rangeProb
                    %if inclusive then we do not evaluate the last step but
                    % use fixing instead 
                    endTimeStep = 1;
                    if inclusiveYN
%                         simpleYN = true;
                        simpleYN = 1;
                        endTimeStep = 2;
                        if pricer.overHedgeYN == true 
                            simpleYN = 2;
                        end
                    end
                    
                    for j=numOfTimeSteps:-1:endTimeStep
                        % evaluate dealdescription
                        % last step is omitted
                        currentTime = totalTimeSteps(currentTimeIdx);
                        %dcfC = currentTime/365.0;
                        dcfC = currentTime;
                        
                        if j~= numOfTimeSteps
                            dD = (dcfN-dcfC);
                        end

                        modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:numOfFactors*(currentTimeIdx-1)+3,:);
                        idxL = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorLong,modelStateT);
                        idxS = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                        spreadC = idxL - idxS;

                        probSpread = model.SRPr(spreadC,spreadN,ralower,raupper,simpleYN,pathSize)*dD;
                        cD = cD + dD;
                        cP = cP + probSpread;

                        dcfN = dcfC;
                        spreadN = spreadC;

                        %induct backward to the next time step
                        % one time backward induction
                        lastTimeIdx = currentTimeIdx;

                        if startTimeIdx < lastTimeIdx
                            lastTimeIdx = lastTimeIdx -1;
                            currentTimeIdx = lastTimeIdx;
                        end


                    end

                    %% FromToInduction end
        % if the current period is inclusive, then the start date is not evaluated in the above for loop.
		% instead the fixing will be treated separately below.
		% however current_time_idx is updated at the end of the for loop.
		% so we need to update model_states accordingly.
		% if model_states were updated previously then the following additionaly step doesn't affect the result anyway.
                    currentTime = totalTimeSteps(currentTimeIdx);
                    modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:numOfFactors*(currentTimeIdx-1)+3,:);
                    
                    %% AtDate evaluation
                    % Adding a new cashflow to the payoff

                    dcf = schedule.dayCountFraction(i);
                    
                    if inclusiveYN
                        % we add valuation date
                        cD = schedule.dayCount(i);
                        rangeDays =  security.rangeDays(i);
                        payoffStateA.cashflow = nominal * coupon * dcf * (rangeDays(i) + cP)/cD;
                    else
                        payoffStateA.cashflow = nominal * coupon * dcf * cP/cD;
                    end
                    % eventDate(i) = startDate(i) therefore there is no induction in modelStates
                    % i.e. currentTimeIdx has not changed
                    % no need to update modelStateT
                    % modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:end,:);
                    % since cashflow's are added at the startDate(i) we need to
                    % multiply cashflow with the df(startDate(i), payDate(i))
                    payTime = DateDiff(schedule.payDates(i),valueDate);
                    
                    %event is at startDate so we need to discount the
                    %cashflow first from paydate to startdate
                    df_ep = model.discountFactorRisky(currentTime, payTime,numMatTime,modelStateT);

                    for k=1:pathSize
                        payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                    end
                    
                    % process payoff
                    
                    currentNodeIdx = i;
                    
                    nonCall.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                
                    callable.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                    
                    %df column
                    payoffStateA.cashflow = ones(1,pathSize);
                    for k=1:pathSize
                        payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                    end
                    
                    dfCol.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                                 
                    %dummy column
                    payoffStateA.cashflow = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorLong,modelStateT);
                    dummy1.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                    
                    payoffStateA.cashflow = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                    dummy2.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                    
                    
                    % if exercise is possible
                    if i == 1
                        dummaaa = 1;
                    end
                    
                    % call event happens  at the start of the period
                    % at call date, by default, call option is expired
                    
                    if callYN(i) && (DateDiff(valueDate,callSchedule.resetDates(i)) < 0)
                        % 1.undiscount callable.payoff
                        % 2.build the state variables matrix
                        % 3.regress 1's with the given state variables
                        % (model states 1,2)
                        % 4. exercise per each path in
                        % discount the new payoff and assign into callable
                        % column
                        
                        %1 undiscount callable.payoff and get the
                        %contiValue
                       
                        %exercise value
                        exerciseValue = security.nominal*ones(1,pathSize);
                        payTime = DateDiff(callSchedule.payDates(i),valueDate);
                        %event is at startDate so we need to discount the
                        %cashflow first from paydate to startdate
                        df_ep = model.discountFactorRisky(currentTime, payTime,numMatTime,modelStateT);
                        for k=1:pathSize
                            exerciseValue(k) = exerciseValue(k) * df_ep(k);
                        end
                        
                        contiValue = zeros(1,pathSize);
                        onePayoff = ones(1,pathSize);
                        onePayoffD = model.discountPayoffRisky(currentTime,numMatTime ...
                        ,modelStateT,onePayoff);
                        contiValue = callable.payoff./onePayoffD;
                        
                        %Nominal Regression test
                        contiValue = contiValue/10000;
                        
                        % build the state variabls matrix
                        decisionVar1 = modelStateT(1,:);
                        decisionVar2 = modelStateT(2,:);
                        decisionVar3 = modelStateT(3,:);
                        
                        % we use upto 2nd order polynomial as regression
                        % equatioin
                        dimensionLS = 10;
                        regMatrix = ones(pathSize,dimensionLS);
                        
                        for idx_r=1:pathSize
                            regMatrix(idx_r,2) = decisionVar1(idx_r);
                            regMatrix(idx_r,3) = decisionVar2(idx_r);
                            regMatrix(idx_r,4) = decisionVar3(idx_r);
                            regMatrix(idx_r,5) = decisionVar1(idx_r)*decisionVar1(idx_r);
                            regMatrix(idx_r,6) = decisionVar1(idx_r)*decisionVar2(idx_r);
                            regMatrix(idx_r,7) = decisionVar1(idx_r)*decisionVar3(idx_r);
                            
                            regMatrix(idx_r,8) = decisionVar2(idx_r)*decisionVar2(idx_r);
                            regMatrix(idx_r,9) = decisionVar2(idx_r)*decisionVar3(idx_r);
                            regMatrix(idx_r,10) = decisionVar3(idx_r)*decisionVar3(idx_r);
                        end
                        
                        % simple regression <- for path dependent option
                        % we should change it
                        % beta  : regression coefficients
                        beta =zeros(dimensionLS,1);
                        beta = regress(contiValue',regMatrix);
                        regContiValue = regMatrix*beta;
                       
                         %Nominal Regression test
                        regContiValue = regContiValue * 10000;
                        
                        exercisedPayoff = regContiValue;
                        % if regContiValue is bigger than exerciseValue
                        % then issue exercise call option
                        for idx_r=1:pathSize
                            if regContiValue(idx_r) >= exerciseValue(idx_r) 
                                exercisedPayoff(idx_r) = exerciseValue(idx_r);
                            end
                        end
                        
                        % we discount the new payoff and 
                        % assign it into the existing callable payoff
                        callable.payoff = model.discountPayoffRisky(currentTime,numMatTime ...
                        ,modelStateT,exercisedPayoff);
                        
                    end
                end

                nonCall.npv = mean(nonCall.payoff);
                out.nonCall = nonCall;
                callable.npv = mean(callable.payoff);
                out.callable = callable;
                out.dfCol = dfCol;
                out.dummy1 = dummy1;
                out.dummy2 = dummy2;
                out.daycountFraction = schedule.dayCountFraction;
                
        end
        
        function out = computePriceClosedTRZ(pricer,valueDate)
                security = pricer.m_security;
                numMethod = pricer.m_numMethod;
                model = pricer.m_model;
                schedule = security.couponSchedule;
                
                % numMethodTimeSteps Generate
                mcOneTimeStep = numMethod.mcOneTimeStep;
                outInfo = numMethod.numMethodTimeStepsGenerateDW(valueDate,schedule,mcOneTimeStep);
            
                numMethodInfo.timeStepsInfo = outInfo;

                % mc random number init
                %pathSize = 2^16;
                pathSize = numMethod.pathSize;
                
                pathSize = 1;
                
%                 % generate random numbers and modelstates forward
%                 rng('default');
%                 rng(0);
% 
                numOfFactors = 2;
% 
%                 %Z= randn(numOfFactors*outInfo.totalTimeStepsSize,pathSize/2);
%                 Z= randn(pathSize/2,numOfFactors*outInfo.totalTimeStepsSize);
%                 Z=[Z;-Z];
%                 Z=Z';
%                 
%                 
%                 %TimeInversion
%                 sizeZ = size(Z,1);
%                 A = zeros(sizeZ,sizeZ);
%                 for i=1:sizeZ
%                     A(i,sizeZ + 1 -i) = 1;
%                 end
%                 Z = A*Z;
% 
%                 % we check with the c, so we use generated random numbers
% %                 mc_random = importdata('mc_random.xlsx');
% %                 mc_random = importdata('D:\mc_random_number_new_report.txt');
% %                 Z= mc_random;
%                 
                numMethodInfo.pathSize = pathSize;
                numMethodInfo.numOfFactors = numOfFactors;
                numMethodInfo.totalTimeSteps = outInfo.totalTimeSteps;
                numMethodInfo.totalTimeStepsSize = outInfo.totalTimeStepsSize;
%                 numMethodInfo.Z = Z;
% 
% 
% 
%                 %generate model states for the given timeSteps
%                 modelStates = numMethod.generateModelStates(valueDate,schedule,numMethodInfo,model);

                % numeraire info
                numMatTime = numMethodInfo.totalTimeSteps(end);
                dfNumMat = model.DF(numMatTime/365.0);
                
                % induct backward and calculate cashflow
            
                periodCount = schedule.numOfPeriod;
                
                % cashflow columns initialize
                % 1st nonCall
                nonCall.npv = 0;
                nonCall.cashflow_e = zeros(periodCount + 1,1);
                nonCall.cashflow_npv_e = zeros(periodCount + 1,1);
                nonCall.rangeProb_e = zeros(periodCount + 1,1);
                nonCall.df_e = zeros(periodCount + 1,1);
                nonCall.df_c = zeros(periodCount + 1,1);

                % debug information, cashflow & npv per path
                nonCall.payoff = zeros(1,pathSize);
                nonCall.payoffStates.cashflow = zeros(periodCount+1,pathSize);
                nonCall.payoffStates.cashflow_npv = zeros(periodCount+1,pathSize);
                nonCall.payoffStates.df = zeros(periodCount + 1,pathSize);
                
                % intermediate variables
                payoffStateA.cashflow = zeros(1, pathSize);
                payoffStateA.cashflow_npv = zeros(1, pathSize);
                payoffStateA.df = zeros(1,pathSize);
                
                % we calculate cashflow backward from maturityDate to valueDate
                % rangeAccrual coupons are added at the startDate of the Period
                % At maturity

                nominal = security.nominal;
                coupon = security.coupon;
                callYN = security.callYN;
                
                currentTime = numMethodInfo.totalTimeSteps(end);
                totalTimeSteps = numMethodInfo.timeStepsInfo.totalTimeSteps;
                totalTimeStepsSize = numMethodInfo.timeStepsInfo.totalTimeStepsSize;
                if totalTimeStepsSize == 0 || (totalTimeStepsSize == 1 && totalTimeSteps(1) == 0) 
                    out.nonCall = nonCall;
                    return;
                end
                
                currentTimeIdx = totalTimeStepsSize;
                
                currentNodeIdx = periodCount + 1;
                
%                 modelStateT = modelStates(numOfFactors*(totalTimeStepsSize-1)+1:numOfFactors*(totalTimeStepsSize-1)+2,:);
                payoffStateA.cashflow = nominal*ones(1,pathSize);
                %process payoff at maturity
                nonCall.payoffStates.cashflow(currentNodeIdx,:) = payoffStateA.cashflow;
                
%                 payoffStateA.cashflow_npv = model.discountPayoff(currentTime,numMatTime ...
%                     ,modelStateT,payoffStateA.cashflow);
%                 
                payoffStateA.cashflow_npv = model.DFRisky(currentTime/365.0)*payoffStateA.cashflow;
                
                nonCall.payoffStates.cashflow_npv(currentNodeIdx,:) = payoffStateA.cashflow_npv ;
                
%                 payoffStateA.df = model.discountPayoff(currentTime,numMatTime ...
%                     ,modelStateT,1.0*ones(1,pathSize));
                payoffStateA.df = model.DFRisky(currentTime/365.0) *ones(1,pathSize);
                    

                nonCall.payoffStates.df(currentNodeIdx,:) = payoffStateA.df;
                nonCall.payoff = nonCall.payoff + payoffStateA.cashflow_npv;
                
                nonCall.cashflow_e(currentNodeIdx) = mean(payoffStateA.cashflow);
                nonCall.cashflow_npv_e(currentNodeIdx) = mean(payoffStateA.cashflow_npv);
                nonCall.df_e(currentNodeIdx) = mean(payoffStateA.df);
                nonCall.df_c(currentNodeIdx) = model.DFRisky(currentTime/365.0);
                
%                 % callable at maturity is the same as noncallable
%                 callable.payoffStates.cashflow(currentNodeIdx,:) = nonCall.payoffStates.cashflow(currentNodeIdx,:);
%                 callable.payoffStates.cashflow_npv(currentNodeIdx,:) = nonCall.payoffStates.cashflow_npv(currentNodeIdx,:);
%                 callable.payoff = callable.payoff + callable.payoffStates.cashflow_npv(currentNodeIdx,:);
%                 
%                 callable.cashflow_e(currentNodeIdx) = nonCall.cashflow_e(currentNodeIdx);
%                 callable.cashflow_npv_e(currentNodeIdx) = nonCall.cashflow_npv_e(currentNodeIdx);
                  
                lastTimeIdx = currentTimeIdx;
                
                
                % flag for inclusive schedule
%                 simpleYN = false;
                simpleYN = 1;
                
                for i=periodCount:-1:1
                    % past schedule neglect
                    if DateDiff(valueDate,schedule.payDates(i)) >= 0, continue;end 
                    startTimeIdx = numMethodInfo.timeStepsInfo.startTimeIdx(i);
                    endTimeIdx = numMethodInfo.timeStepsInfo.endTimeIdx(i);

                    inclusiveYN = numMethodInfo.timeStepsInfo.inclusiveYN(i);
                     %% FromToInduction start

                    %modelsStates induct backward(lastTimeIdx to endTimeIdx)
                    while endTimeIdx < lastTimeIdx
                        lastTimeIdx = lastTimeIdx -1;
                        if lastTimeIdx == endTimeIdx break;end
                    end

                    currentTimeIdx = lastTimeIdx;
                    currentTime = totalTimeSteps(currentTimeIdx);
                    % now we are at endTimeIdx
                    % variables for coupon valuation initialized
                    ralower = security.lower;
                    raupper = security.upper;
                    tenorLong = security.tenorL;
                    tenorShort = security.tenorS;

                    cD = 0; % cumulated day count fraction
                    cP = 0; 
                    dI = 0; % dummy idx for loop
                    dcfC = 0;
                    dcfN = 0;
                    dD = 0;
                    idxL = zeros(1,pathSize);
                    idxS = zeros(1,pathSize);
                    spreadC = zeros(1,pathSize);
                    spreadN = zeros(1,pathSize);
                    probSpread = zeros(1,pathSize);
                    
                    numOfTimeSteps = numMethodInfo.timeStepsInfo.numOfTimeSteps(i);
                    
                    % if inclusive then we use simpleYN rangeProb
                    %if inclusive then we do not evaluate the last step but
                    % use fixing instead 
                    endTimeStep = 1;
                    if inclusiveYN
%                         simpleYN = true;
                        simpleYN = 1;
                        endTimeStep = 2;
                    end
                    
                    for j=numOfTimeSteps:-1:endTimeStep
                        % evaluate dealdescription
                        % last step is omitted
                        currentTime = totalTimeSteps(currentTimeIdx);
                        %dcfC = currentTime/365.0;
                        dcfC = currentTime;
                        
                        if j~= numOfTimeSteps
                            dD = (dcfN-dcfC);
                        end

                        toDate = AddDate(valueDate,currentTime,'day');
                        tpDate = schedule.payDates(i);
                        spreadC = model.CMSSpreadDigitalRangeDate(valueDate,toDate,tpDate,tenorLong,tenorShort,0.25,ralower,raupper);
                        
                        %modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:numOfFactors*(currentTimeIdx-1)+2,:);
%                         idxL = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorLong,modelStateT);
%                         idxS = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorShort,modelStateT);
%                         spreadC = idxL - idxS;
%                         probSpread = model.SRPr(spreadC,spreadN,ralower,raupper,simpleYN,pathSize)*dD; 

                        if simpleYN
                            probSpread = spreadC*dD;
                        else
                            probSpread = spreadC*dD;
%                             probSpread = 0.5*(spreadC + spreadN)*dD;
                            
                        end

                        
                        cD = cD + dD;
                        cP = cP + probSpread;

                        dcfN = dcfC;
                        spreadN = spreadC;

                        %induct backward to the next time step
                        % one time backward induction
                        lastTimeIdx = currentTimeIdx;

                        if startTimeIdx < lastTimeIdx
                            lastTimeIdx = lastTimeIdx -1;
                            currentTimeIdx = lastTimeIdx;
                        end


                    end

                    %% FromToInduction end
                    %% AtDate evaluation
                    % Adding a new cashflow to the payoff

                    dcf = schedule.dayCountFraction(i);
                    
                    if inclusiveYN
                        % we add valuation date
                        cD = schedule.dayCount(i);
                        rangeDays =  security.rangeDays(i);
                        payoffStateA.cashflow = nominal * coupon * dcf * (rangeDays(i) + cP)/cD;
                    else
                        payoffStateA.cashflow = nominal * coupon * dcf * cP/cD;
                    end
                    % eventDate(i) = startDate(i) therefore there is no induction in modelStates
                    % i.e. currentTimeIdx has not changed
                    % no need to update modelStateT
                    % modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:end,:);
                    % since cashflow's are added at the startDate(i) we need to
                    % multiply cashflow with the df(startDate(i), payDate(i))
                    payTime = DateDiff(schedule.payDates(i),valueDate);
                    
                    %event is at startDate so we need to discount the
                    %cashflow first from paydate to startdate
%                     df_sp = model.discountFactor(currentTime, payTime,numMatTime,modelStateT);

%                     for k=1:pathSize
%                         payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_sp(k);
%                     end
                    
                    % process payoff
                    
                    currentNodeIdx = i;
                    
                    % 1st nonCall column
                    nonCall.payoffStates.cashflow(currentNodeIdx,:) = payoffStateA.cashflow;
                    payoffStateA.cashflow_npv = payoffStateA.cashflow * model.DFRisky(payTime/365.0);
                    
%                     payoffStateA.cashflow_npv = model.discountPayoff(currentTime,numMatTime ...
%                     ,modelStateT,payoffStateA.cashflow);
                    nonCall.payoffStates.cashflow_npv(currentNodeIdx,:) = payoffStateA.cashflow_npv;
                    payoffStateA.df = model.DFRisky(payTime/365.0);
%                     payoffStateA.df = model.discountPayoff(currentTime,numMatTime ...
%                     ,modelStateT,1.0*ones(1,pathSize));
                    nonCall.payoffStates.df(currentNodeIdx,:) = payoffStateA.df;
                    
                    nonCall.payoff = nonCall.payoff + payoffStateA.cashflow_npv;
                    
                    nonCall.cashflow_e(currentNodeIdx) = mean(payoffStateA.cashflow);
                    nonCall.rangeProb_e(currentNodeIdx) = mean(nonCall.cashflow_e(currentNodeIdx)/(nominal * coupon * dcf));
                    nonCall.cashflow_npv_e(currentNodeIdx) = mean(payoffStateA.cashflow_npv);

                    nonCall.df_e(currentNodeIdx) = mean(payoffStateA.df);
                    nonCall.df_c(currentNodeIdx) = model.DFRisky(currentTime/365.0);
                    
%                     %2nd callable column
%                     callable.payoffStates.cashflow(currentNodeIdx,:) = nonCall.payoffStates.cashflow(currentNodeIdx,:);
%                     callable.payoffStates.cashflow_npv(currentNodeIdx,:) = nonCall.payoffStates.cashflow_npv(currentNodeIdx,:);
%                     callable.payoff = callable.payoff + callable.payoffStates.cashflow_npv(currentNodeIdx,:);
%                     
%                     callable.cashflow_e(currentNodeIdx) = nonCall.cashflow_e(currentNodeIdx);
%                     callable.cashflow_npv_e(currentNodeIdx) = nonCall.cashflow_npv_e(currentNodeIdx);

                    % if exercise is possible
                    if i == 1
                        dummaaa = 1;
                    end
                    
                    % call event happens  at the start of the period
                    % at call date, by default, call option is expired
                    
%                     if callYN(i) && (DateDiff(valueDate,schedule.startDates(i)) < 0)
%                         % 1.undiscount callable.payoff
%                         % 2.build the state variables matrix
%                         % 3.regress 1's with the given state variables
%                         % (model states 1,2)
%                         % 4. exercise per each path in
%                         % discount the new payoff and assign into callable
%                         % column
%                         
%                         %1 undiscount callable.payoff and get the
%                         %contiValue
%                        
%                         %exercise value
%                         exerciseValue = security.nominal*ones(1,pathSize);
%                         contiValue = zeros(1,pathSize);
%                         onePayoff = ones(1,pathSize);
%                         onePayoffD = model.discountPayoff(currentTime,numMatTime ...
%                         ,modelStateT,onePayoff);
%                         contiValue = callable.payoff./onePayoffD;
%                         
%                         % build the state variabls matrix
%                         decisionVar1 = modelStateT(1,:);
%                         decisionVar2 = modelStateT(2,:);
%                         % we use upto 2nd order polynomial as regression
%                         % equatioin
%                         dimensionLS = 6;
%                         regMatrix = ones(pathSize,dimensionLS);
%                         
%                         for idx_r=1:pathSize
%                             regMatrix(idx_r,2) = decisionVar1(idx_r);
%                             regMatrix(idx_r,3) = decisionVar2(idx_r);
%                             regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r);
%                             regMatrix(idx_r,5) = decisionVar1(idx_r)*decisionVar2(idx_r);
%                             regMatrix(idx_r,6) = decisionVar2(idx_r)*decisionVar2(idx_r);
%                         end
%                         
%                         % simple regression <- for path dependent option
%                         % we should change it
%                         % beta  : regression coefficients
%                         beta =zeros(dimensionLS,1);
%                         beta = regress(contiValue',regMatrix);
%                         regContiValue = regMatrix*beta;
%                         
%                         exercisedPayoff = regContiValue;
%                         % if regContiValue is bigger than exerciseValue
%                         % then issue exercise call option
%                         for idx_r=1:pathSize
%                             if regContiValue(idx_r) >= exerciseValue(idx_r) 
%                                 exercisedPayoff(idx_r) = exerciseValue(idx_r);
%                             end
%                         end
%                         
%                         % we discount the new payoff and 
%                         % assign it into the existing callable payoff
%                         callable.payoff = model.discountPayoff(currentTime,numMatTime ...
%                         ,modelStateT,exercisedPayoff);
%                         
%                     end
                end

                nonCall.npv = mean(nonCall.payoff);
                out.pricerInfo.nonCall = nonCall;
                
%                 callable.npv = mean(callable.payoff);
%                 out.pricerInfo.callable = callable;
                out.daycountFraction = schedule.dayCountFraction;
                
        end
        
        
        
        
    end
    
end



In [ ]:
classdef DailyWeeklyRiskyPricer_USDCMS_KRWCMSSpreadDualRASimpleCall < H_RootObject
    %Pricer(Security,Model,NumMethod)
    %Prcer is constructed from Security, Model, NumMethod 
    
    properties
        m_security
        m_model
        m_numMethod
        overHedgeYN
    end
    
    methods
        function pricer = DailyWeeklyRiskyPricer_USDCMS_KRWCMSSpreadDualRASimpleCall(security,model,numMethod,overHedgeYN)
                pricer.m_security = security;
                pricer.m_model = model;
                pricer.m_numMethod = numMethod;
                pricer.overHedgeYN = overHedgeYN;
        end
        
        function out = computePriceMCCallableGeneric(pricer,valueDate,pricerOption)
                security = pricer.m_security;
                numMethod = pricer.m_numMethod;
                model = pricer.m_model;
                schedule = security.couponSchedule;
                callSchedule = security.callSchedule;
                
                % numMethodTimeSteps Generate
                mcOneTimeStep = numMethod.mcOneTimeStep;
%                 outInfo = numMethod.numMethodTimeStepsGenerateDW(valueDate,schedule,mcOneTimeStep);
                
                floatingSchedule = struct;
                outInfo = numMethod.numMethodTimeStepsGenerateDWSwapSimpleCall(valueDate,schedule,floatingSchedule,...
                                    callSchedule,mcOneTimeStep);
            
                numMethodInfo.timeStepsInfo = outInfo;

                % mc random number init
                %pathSize = 2^16;
                pathSize = numMethod.pathSize;

                % generate random numbers and modelstates forward
                rng('default');
                rng(0);

                % LGM2F : two factor
                % CK1F  : one factor
                
                numOfFactors = model.numOfFactors;

                %Z= randn(numOfFactors*outInfo.totalTimeStepsSize,pathSize/2);
%                 Z= randn(pathSize/2,numOfFactors*outInfo.totalTimeStepsSize);
                Z= randn(pathSize/2,numOfFactors*outInfo.modelTimeStepsSize);
                Z=[Z;-Z];
                Z=Z';
                
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;

                % we check with the c code, so we use generated random numbers
%                 mc_random = importdata('mc_random.xlsx');
%                 mc_random = importdata('D:\mc_random_number_new_report.txt');
%                 Z= mc_random;
                
                numMethodInfo.pathSize = pathSize;
                numMethodInfo.numOfFactors = numOfFactors;
                numMethodInfo.totalTimeSteps = outInfo.totalTimeSteps;
                numMethodInfo.totalTimeStepsSize = outInfo.totalTimeStepsSize;
                
                % for modelStateGeneration
                numMethodInfo.modelTimeSteps = outInfo.modelTimeSteps;
                numMethodInfo.modelTimeStepsSize = outInfo.modelTimeStepsSize;
                
                numMethodInfo.pastTimeStepsSize = outInfo.pastTimeStepsSize;
                
                pastTimeStepsSize = numMethodInfo.pastTimeStepsSize;
                
                numMethodInfo.Z = Z;



                %numMethod generate model states for the given timeSteps
%                 modelStates = numMethod.generateModelStatesNF(valueDate,schedule,numMethodInfo,model);
                 modelStates = numMethod.generateModelStatesNFNew(valueDate,schedule,numMethodInfo,model);
                % numeraire info
                numMatTime = numMethodInfo.totalTimeSteps(end);
                days = DateDiff(schedule.payDates(schedule.numOfPeriod),valueDate);
                numMatTime = max(numMatTime, days);
                dfNumMat = model.DFRisky(numMatTime/365.0);
                
                numeraire.numMatTime = numMatTime;
                numeraire.dfNumMat = dfNumMat;
                
                % induct backward and calculate cashflow
            
                periodCount = schedule.numOfPeriod;
                
                % pricer single info initialize
                
                nonCall = PricerInfo(periodCount +1, pathSize);
                callable = PricerInfo(periodCount +1, pathSize);
                dfCol = PricerInfo(periodCount +1, pathSize);
                
                dummy1 = PricerInfo(periodCount +1, pathSize);
                dummy2 = PricerInfo(periodCount +1, pathSize);
                
%                 % additional floating leg info
%                 if ~isempty(fieldnames(floatingSchedule))
%                     floatingLeg = PricerInfo(floatingSchedule.numOfPeriod, pathSize);
%                     % currentFloatingLegIdx for iteration in dealing with 
%                     % the floatingLeg cashflow
%                     currentFloatingLegIdx = floatingSchedule.numOfPeriod;
%                 end
                
                % intermediate variables
                payoffStateA.cashflow = zeros(1, pathSize);
                payoffStateA.cashflow_npv = zeros(1, pathSize);
                
            
                % we calculate cashflow backward from maturityDate to valueDate
                % rangeAccrual coupons are added at the startDate of the Period
                % At maturity

                nominal = security.nominal;
                coupon = security.coupon;
                callYN = security.callYN;
                
                tenorCMS = security.tenor;
                
                ralower = security.lower;
                raupper = security.upper;
                
                tenorLong = security.tenorL;
                tenorShort = security.tenorS;
                
                ralowerS = security.lowerS;
                raupperS = security.upperS;
                
                currentTime = numMethodInfo.totalTimeSteps(end);
                modelTimeStepsSize = numMethodInfo.modelTimeStepsSize;
                currentTimeIdx = numMethodInfo.totalTimeStepsSize;
                
                currentNodeIdx = periodCount + 1;
                
                modelStateT = modelStates(numOfFactors*(max(currentTimeIdx-pastTimeStepsSize-1,0))+1:numOfFactors*(max(currentTimeIdx-pastTimeStepsSize-1,0))+numOfFactors,:);
%                 modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:numOfFactors*(currentTimeIdx-1)+numOfFactors,:);
                
                payTime = DateDiff(schedule.payDates(schedule.numOfPeriod),valueDate);
                
                %even though there is no cashflow at the last structured
                %enddate, we add dummy cashflow in line with the note
                if payTime > 0
                    payoffStateA.cashflow = nominal*ones(1,pathSize);

                    df_ep = model.discountFactorRisky(currentTime, payTime,numMatTime,modelStateT);
                    for k=1:pathSize
                        payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                    end

                    %process payoff at maturity
                    nonCall.ProcessPayoff(model,currentTime,numeraire, ...
                                         modelStateT,payoffStateA.cashflow,currentNodeIdx);

                    callable.ProcessPayoff(model,currentTime,numeraire, ...
                                         modelStateT,payoffStateA.cashflow,currentNodeIdx);

                    %df column
                    payoffStateA.cashflow = ones(1,pathSize);
                    for k=1:pathSize
                        payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                    end

                    dfCol.ProcessPayoff(model,currentTime,numeraire, ...
                                         modelStateT,payoffStateA.cashflow,currentNodeIdx);

                    %dummy column
                    foreignModelName = model.foreignModelName;

                    if strcmp(pricerOption,'PSA')
                        payoffStateA.cashflow = model.CMS_PSA_SimpleDate(model.refModelName,valueDate,currentTime,numMatTime, tenorLong,modelStateT);
                        dummy1.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);

                        payoffStateA.cashflow = model.CMS_PSA_SimpleDate(foreignModelName,valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                        dummy2.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);

                    else
                        payoffStateA.cashflow = model.CMS_SimpleDate(model.refModelName,valueDate,currentTime,numMatTime, tenorLong,modelStateT);
                        dummy1.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);

                        payoffStateA.cashflow = model.CMS_SimpleDate(foreignModelName,valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                        dummy2.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);

                    end
                
                end

                lastTimeIdx = currentTimeIdx;
                totalTimeSteps = numMethodInfo.timeStepsInfo.totalTimeSteps;
                
                % flag for inclusive schedule
%                 simpleYN = false;
%                 simpleYN = true;
                if pricer.overHedgeYN == true 
                    simpleYN = 2;
                else
                    simpleYN = 1;
                end
                
                for i=periodCount:-1:1
                    % we induct in the pastTimeSteps too
%                     if DateDiff(valueDate,schedule.payDates(i)) >= 0, continue;end 
                    
                    startTimeIdx = numMethodInfo.timeStepsInfo.startTimeIdx(i);
                    endTimeIdx = numMethodInfo.timeStepsInfo.endTimeIdx(i);

                    inclusiveYN = numMethodInfo.timeStepsInfo.inclusiveYN(i);
                 %% FromToInduction start

                    %modelsStates induct backward(lastTimeIdx to endTimeIdx)
                    while endTimeIdx < lastTimeIdx
                        lastTimeIdx = lastTimeIdx -1;
                        if lastTimeIdx == endTimeIdx break;end
                    end

                    currentTimeIdx = lastTimeIdx;
                    currentTime = totalTimeSteps(currentTimeIdx);
                    % now we are at endTimeIdx
                    % variables for coupon valuation initialized
                    

                    cD = 0; % cumulated day count fraction
                     %cP shoul be declared as an array
%                     cP = 0;
                    cP = zeros(1,pathSize);
                    dI = 0; % dummy idx for loop
                    dcfC = 0;
                    dcfN = 0;
                    dD = 0;
                    idxCMS = zeros(1,pathSize);
                    
                    idxC = zeros(1,pathSize);
                    idxN = zeros(1,pathSize);
                    
                    idxL = zeros(1,pathSize);
                    idxS = zeros(1,pathSize);
                    
                    spreadC = zeros(1,pathSize);
                    spreadN = zeros(1,pathSize);
                    probSpread = zeros(1,pathSize);
                    
                    numOfTimeSteps = numMethodInfo.timeStepsInfo.numOfTimeSteps(i);
                    
                    % if inclusive then we use simpleYN rangeProb
                    %if inclusive then we do not evaluate the last step but
                    % use fixing instead 
                    endTimeStep = 1;
                     % we let endTimeStep to be 1 and
                    % inside the for loop if currentTime ==0
                    % then we skip the range accrual check part
                    
                    if inclusiveYN
%                         simpleYN = true;
                        simpleYN = 1;
%                         endTimeStep = 2;
                        if pricer.overHedgeYN == true 
                            simpleYN = 2;
                        end
                    end
                    
                    for j=numOfTimeSteps:-1:endTimeStep
                        % we initialize payoffStateA as zeros to prevent
                        % reuse it;
                        payoffStateA.cashflow = zeros(1,pathSize);
                        % evaluate FromTo dealdescription
                        % last step is omitted
                        currentTime = totalTimeSteps(currentTimeIdx);
                        if currentTime > 0
                            dcfC = currentTime;

                            if j~= numOfTimeSteps
                                dD = (dcfN-dcfC);
                            end

%                             modelStateT = modelStates(max(currentTimeIdx-pastTimeStepsSize-1,0)+1:numOfFactors*(currentTimeIdx-1)+numOfFactors,:);
                            modelStateT = modelStates(numOfFactors*(max(currentTimeIdx-pastTimeStepsSize-1,0))+1:numOfFactors*(max(currentTimeIdx-pastTimeStepsSize-1,0))+numOfFactors,:);
                            if strcmp(pricerOption,'PSA')
                                idxCMS = model.CMS_PSA_SimpleDate(foreignModelName,valueDate,currentTime,numMatTime, tenorCMS,modelStateT);
                                idxC = idxCMS;

                                idxL = model.CMS_PSA_SimpleDate(model.refModelName,valueDate,currentTime,numMatTime, tenorLong,modelStateT);
                                idxS = model.CMS_PSA_SimpleDate(model.refModelName,valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                                spreadC = idxL - idxS;
                            else
                                idxCMS = model.CMS_SimpleDate(foreignModelName,valueDate,currentTime,numMatTime, tenorCMS,modelStateT);
                                idxC = idxCMS;

                                idxL = model.CMS_SimpleDate(model.refModelName,valueDate,currentTime,numMatTime, tenorLong,modelStateT);
                                idxS = model.CMS_SimpleDate(model.refModelName,valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                                spreadC = idxL - idxS;
                            end
    %                         probSpread = model.SRPr(spreadC,spreadN,ralower,raupper,simpleYN,pathSize)*dD;
                            probSpread = model.DRPr(idxC,idxN,ralower,raupper,spreadC,spreadN,ralowerS,raupperS,simpleYN,pathSize)*dD;
                            cD = cD + dD;
                            cP = cP + probSpread;

                            dcfN = dcfC;

                            idxN = idxC;
                            spreadN = spreadC;
                        else
                            dcfC = currentTime;

                            if j~= numOfTimeSteps
                                dD = (dcfN-dcfC);
                            end

                            cD = cD + dD;

                            dcfN = dcfC;
                            
                        end
                        
                        %floating event begins here
                        %floating event end here
                        
                        % callable event begin
                        idxCall = find(currentTime == outInfo.callEventTime);
                        
                        % we do not evaluate callable event at the start of
                        % coupon Schedule for avoding double counting
                        % and that make sense
                        if ~isempty(idxCall) && (j~= endTimeStep)
                            iterCall = 1;
                            foundIterCallYN= false;
                            for idx11 = length(callSchedule.resetDates):-1:1
                                % call event happens at call startDate !
                                % we change call event happens at call resetDate !
                                
                                if outInfo.callEventTime(idxCall) == DateDiff(callSchedule.resetDates(idx11),valueDate)
                                    iterCall = idx11;
                                    foundIterCallYN = true;
                                    break;
                                end
                            end
                            
                            if ~foundIterCallYN
                                error('callEventTime is not in the callSchedule startDates')
                            end
                            
                            % only those cashflow that paydate is after
                            % valueDate will be added
                            if DateDiff(callSchedule.payDates(iterCall),valueDate) > 0
                                % callEvent is not fixed
                                if DateDiff(valueDate,callSchedule.resetDates(iterCall)) < 0
                                    % 1.undiscount callable.payoff
                                    % 2.build the state variables matrix
                                    % 3.regress 1's with the given state variables
                                    % (model states 1,2)
                                    % 4. exercise per each path in
                                    % discount the new payoff and assign into callable
                                    % column

                                    %1 undiscount callable.payoff and get the
                                    %contiValue

                                    %exercise value Note
                                    exerciseValue = security.nominal*ones(1,pathSize) * 1.0;
                                    payTime = DateDiff(callSchedule.payDates(iterCall),valueDate);
                                    %event is at startDate so we need to discount the
                                    %cashflow first from paydate to startdate
                                    df_ep = model.discountFactorRisky(currentTime, payTime,numMatTime,modelStateT);
                                    for k=1:pathSize
                                        exerciseValue(k) = exerciseValue(k) * df_ep(k);
                                    end
                                    
                                    contiValue = zeros(1,pathSize);
                                    onePayoff = ones(1,pathSize);
                                    onePayoffD = model.discountPayoffRisky(currentTime,numMatTime ...
                                    ,modelStateT,onePayoff);
                                    contiValue = callable.payoff./onePayoffD;

                                    %Nominal Regression test
                                    contiValue = contiValue/10000;

                                    % build the state variabls matrix
                                    if numMethod.regressionOrder == 3
            %                             dimensionLS = (numOfFactors + 1)*(numOfFactors + 2)/2;

                                        decisionVar = zeros(numOfFactors,size(modelStateT,2));

                                        for idxLS =1:numOfFactors
                                            decisionVar(idxLS,:) = modelStateT(idxLS,:); 
                                        end

                                        % calculate number of dimension first
                                        % 0th order
                                        dimensionLS = 1;

                                        % 1st order
                                        for idxLS=1:numOfFactors
                                            dimensionLS = dimensionLS + 1;
                                        end

                                        %2nd order
                                        for idxLS1=1:numOfFactors
                                            for idxLS2=idxLS1:numOfFactors
                                                dimensionLS = dimensionLS + 1;
                                            end
                                        end

                                        %3rd order
                                        for idxLS1=1:numOfFactors
                                            for idxLS2=idxLS1:numOfFactors
                                                for idxLS3=idxLS2:numOfFactors
                                                    dimensionLS = dimensionLS + 1;
                                                end
                                            end
                                        end


                %                         decisionVar1 = modelStateT(1,:);
                %                         decisionVar2 = modelStateT(2,:);
                                        % we use upto 2nd order polynomial as regression
                                        % equatioin

                                        regMatrix = ones(pathSize,dimensionLS);

                                        for idx_r=1:pathSize
                                            for idxLS=1:numOfFactors
                                                regMatrix(idx_r,idxLS+1) = decisionVar(idxLS,idx_r);
                                            end

                                            tempIdx = 0;
                                            for idxLS1=1:numOfFactors
                                                for idxLS2=idxLS1:numOfFactors
                                                    tempIdx = tempIdx + 1;
                                                    regMatrix(idx_r,numOfFactors+1+tempIdx) = decisionVar(idxLS1,idx_r)*decisionVar(idxLS2,idx_r);
                                                end
                                            end

                                            tempIdx2 = 0;
                                            for idxLS1=1:numOfFactors
                                                for idxLS2=idxLS1:numOfFactors
                                                    for idxLS3=idxLS2:numOfFactors
                                                        tempIdx2 = tempIdx2 + 1;
                                                        regMatrix(idx_r,numOfFactors+1+tempIdx + tempIdx2) = decisionVar(idxLS1,idx_r)*decisionVar(idxLS2,idx_r)*decisionVar(idxLS3,idx_r);
                                                    end
                                                end
                                            end


                                        end

                                    else
                                        dimensionLS = (numOfFactors + 1)*(numOfFactors + 2)/2;

                                        decisionVar = zeros(numOfFactors,size(modelStateT,2));

                                        for idxLS =1:numOfFactors
                                            decisionVar(idxLS,:) = modelStateT(idxLS,:); 
                                        end
                %                         decisionVar1 = modelStateT(1,:);
                %                         decisionVar2 = modelStateT(2,:);
                                        % we use upto 2nd order polynomial as regression
                                        % equatioin

                                        regMatrix = ones(pathSize,dimensionLS);

                                        for idx_r=1:pathSize
                                            for idxLS=1:numOfFactors
                                                regMatrix(idx_r,idxLS+1) = decisionVar(idxLS,idx_r);
                                            end

                                            tempIdx = 1;
                                            for idxLS1=1:numOfFactors
                                                for idxLS2=idxLS1:numOfFactors
                                                    regMatrix(idx_r,numOfFactors+1+tempIdx) = decisionVar(idxLS1,idx_r)*decisionVar(idxLS2,idx_r);
                                                    tempIdx = tempIdx + 1;
                                                end
                                            end

                                        end
                                    end
                                    % simple regression <- for path dependent option
                                    % we should change it
                                    % beta  : regression coefficients
                                    beta =zeros(dimensionLS,1);
                                    beta = regress(contiValue',regMatrix);
                                    regContiValue = regMatrix*beta;

                                     %Nominal Regression test
                                    regContiValue = regContiValue * 10000;

                                    exercisedPayoff = regContiValue;
                                    % if regContiValue is bigger than exerciseValue
                                    % then issue exercise call option
                                    for idx_r=1:pathSize
                                        if regContiValue(idx_r) >= exerciseValue(idx_r) 
                                            exercisedPayoff(idx_r) = exerciseValue(idx_r);
                                        end
                                    end

                                    % we discount the new payoff and 
                                    % assign it into the existing callable payoff
                                    callable.payoff = model.discountPayoffRisky(currentTime,numMatTime ...
                                    ,modelStateT,exercisedPayoff);

%                                     contiValue = zeros(1,pathSize);
%                                     onePayoff = ones(1,pathSize);
%                                     onePayoffD = model.discountPayoffRisky(currentTime,numMatTime ...
%                                     ,modelStateT,onePayoff);
%                                     contiValue = callable.payoff./onePayoffD;
% 
%                                     %Nominal Regression test
%                                     contiValue = contiValue/10000;
% 
%                                     % build the state variabls matrix
%                                     decisionVar1 = modelStateT(1,:);
%                                     decisionVar2 = modelStateT(2,:);
%                                     % we use upto 2nd order polynomial as regression
%                                     % equatioin
%                                     dimensionLS = 6;
%                                     regMatrix = ones(pathSize,dimensionLS);
% 
%                                     for idx_r=1:pathSize
%                                         regMatrix(idx_r,2) = decisionVar1(idx_r);
%                                         regMatrix(idx_r,3) = decisionVar2(idx_r);
%                                         regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r);
%                                         regMatrix(idx_r,5) = decisionVar1(idx_r)*decisionVar2(idx_r);
%                                         regMatrix(idx_r,6) = decisionVar2(idx_r)*decisionVar2(idx_r);
%                                     end
% 
%                                     % simple regression <- for path dependent option
%                                     % we should change it
%                                     % beta  : regression coefficients
%                                     beta =zeros(dimensionLS,1);
%                                     beta = regress(contiValue',regMatrix);
%                                     regContiValue = regMatrix*beta;
% 
%                                      %Nominal Regression test
%                                     regContiValue = regContiValue * 10000;
% 
%                                     exercisedPayoff = regContiValue;
%                                     % if regContiValue is bigger than exerciseValue
%                                     % then issue exercise call option
%                                     for idx_r=1:pathSize
%                                         if regContiValue(idx_r) >= exerciseValue(idx_r) 
%                                             exercisedPayoff(idx_r) = exerciseValue(idx_r);
%                                         end
%                                     end
% 
%                                     % we discount the new payoff and 
%                                     % assign it into the existing callable payoff
%                                     callable.payoff = model.discountPayoffRisky(currentTime,numMatTime ...
%                                     ,modelStateT,exercisedPayoff);

                                end
                                
                                
                            end
                        end
                        
                        % callable event end
                        
                        %induct backward to the next time step
                        % one time backward induction
                        lastTimeIdx = currentTimeIdx;

                        if startTimeIdx < lastTimeIdx
                            lastTimeIdx = lastTimeIdx -1;
                            currentTimeIdx = lastTimeIdx;
                        end


                    end

                    %% FromToInduction end
        % if the current period is inclusive, then the start date is not evaluated in the above for loop.
		% instead the fixing will be treated separately below.
		% however current_time_idx is updated at the end of the for loop.
		% so we need to update model_states accordingly.
		% if model_states were updated previously then the following additionaly step doesn't affect the result anyway.
                    currentTime = totalTimeSteps(currentTimeIdx);
%                     modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:numOfFactors*(currentTimeIdx-1)+numOfFactors,:);
                    modelStateT = modelStates(numOfFactors*(max(currentTimeIdx-pastTimeStepsSize-1,0))+1:numOfFactors*(max(currentTimeIdx-pastTimeStepsSize-1,0))+numOfFactors,:);
                    
                    %% AtDate evaluation
                    % Adding a new cashflow to the payoff
                    if DateDiff(schedule.payDates(i),valueDate) > 0
                        dcf = schedule.dayCountFraction(i);

                        if inclusiveYN
                            % we add valuation date
                            cD = schedule.dayCount(i);
                            rangeDays =  security.rangeDays(i);
                            payoffStateA.cashflow = nominal * coupon * dcf * (rangeDays + cP)/cD;
                        else
                            payoffStateA.cashflow = nominal * coupon * dcf * cP/cD;
                        end
                        % eventDate(i) = startDate(i) therefore there is no induction in modelStates
                        % i.e. currentTimeIdx has not changed
                        % no need to update modelStateT
                        % modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:end,:);
                        % since cashflow's are added at the startDate(i) we need to
                        % multiply cashflow with the df(startDate(i), payDate(i))
                        payTime = DateDiff(schedule.payDates(i),valueDate);

                        %event is at startDate so we need to discount the
                        %cashflow first from paydate to startdate
                        df_ep = model.discountFactorRisky(currentTime, payTime,numMatTime,modelStateT);

                        for k=1:pathSize
                            payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                        end

                        % process payoff

                        currentNodeIdx = i;

                        nonCall.ProcessPayoff(model,currentTime,numeraire, ...
                                         modelStateT,payoffStateA.cashflow,currentNodeIdx);

                        callable.ProcessPayoff(model,currentTime,numeraire, ...
                                         modelStateT,payoffStateA.cashflow,currentNodeIdx);

                        %df column
                        payoffStateA.cashflow = ones(1,pathSize);
                        for k=1:pathSize
                            payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                        end

                        dfCol.ProcessPayoff(model,currentTime,numeraire, ...
                                         modelStateT,payoffStateA.cashflow,currentNodeIdx);

                        %dummy column
                        if strcmp(pricerOption,'PSA')
                            payoffStateA.cashflow = model.CMS_PSA_SimpleDate(model.refModelName,valueDate,currentTime,numMatTime, tenorLong,modelStateT);
                            dummy1.ProcessPayoff(model,currentTime,numeraire, ...
                                             modelStateT,payoffStateA.cashflow,currentNodeIdx);



                            payoffStateA.cashflow = model.CMS_PSA_SimpleDate(foreignModelName,valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                            dummy2.ProcessPayoff(model,currentTime,numeraire, ...
                                             modelStateT,payoffStateA.cashflow,currentNodeIdx);

                        else
                            payoffStateA.cashflow = model.CMS_SimpleDate(model.refModelName,valueDate,currentTime,numMatTime, tenorLong,modelStateT);
                            dummy1.ProcessPayoff(model,currentTime,numeraire, ...
                                             modelStateT,payoffStateA.cashflow,currentNodeIdx);



                            payoffStateA.cashflow = model.CMS_SimpleDate(foreignModelName,valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                            dummy2.ProcessPayoff(model,currentTime,numeraire, ...
                                             modelStateT,payoffStateA.cashflow,currentNodeIdx);

                        end

                    end           
                    
                    
                    
                    % if exercise is possible
                    if i == 1
                        dummaaa = 1;
                    end
                    
                    % call event happens  at the start of the period
                    % at call date, by default, call option is expired
                    
%                     if callYN(i) && (DateDiff(valueDate,callSchedule.resetDates(i)) < 0)
%                         % 1.undiscount callable.payoff
%                         % 2.build the state variables matrix
%                         % 3.regress 1's with the given state variables
%                         % (model states 1,2)
%                         % 4. exercise per each path in
%                         % discount the new payoff and assign into callable
%                         % column
%                         
%                         %1 undiscount callable.payoff and get the
%                         %contiValue
%                        
%                         %exercise value
%                         exerciseValue = security.nominal*ones(1,pathSize);
%                         payTime = DateDiff(callSchedule.payDates(i),valueDate);
%                         %event is at startDate so we need to discount the
%                         %cashflow first from paydate to startdate
%                         df_ep = model.discountFactorRisky(currentTime, payTime,numMatTime,modelStateT);
%                         for k=1:pathSize
%                             exerciseValue(k) = exerciseValue(k) * df_ep(k);
%                         end
%                         
%                         contiValue = zeros(1,pathSize);
%                         onePayoff = ones(1,pathSize);
%                         onePayoffD = model.discountPayoffRisky(currentTime,numMatTime ...
%                         ,modelStateT,onePayoff);
%                         contiValue = callable.payoff./onePayoffD;
%                         
%                         %Nominal Regression test
%                         contiValue = contiValue/10000;
%                         
%                         % build the state variabls matrix
%                         if numMethod.regressionOrder == 3
% %                             dimensionLS = (numOfFactors + 1)*(numOfFactors + 2)/2;
% 
%                             decisionVar = zeros(numOfFactors,size(modelStateT,2));
% 
%                             for idxLS =1:numOfFactors
%                                 decisionVar(idxLS,:) = modelStateT(idxLS,:); 
%                             end
%                             
%                             % calculate number of dimension first
%                             % 0th order
%                             dimensionLS = 1;
%                             
%                             % 1st order
%                             for idxLS=1:numOfFactors
%                                 dimensionLS = dimensionLS + 1;
%                             end
%                             
%                             %2nd order
%                             for idxLS1=1:numOfFactors
%                                 for idxLS2=idxLS1:numOfFactors
%                                     dimensionLS = dimensionLS + 1;
%                                 end
%                             end
%                             
%                             %3rd order
%                             for idxLS1=1:numOfFactors
%                                 for idxLS2=idxLS1:numOfFactors
%                                     for idxLS3=idxLS2:numOfFactors
%                                         dimensionLS = dimensionLS + 1;
%                                     end
%                                 end
%                             end
% 
%                             
%     %                         decisionVar1 = modelStateT(1,:);
%     %                         decisionVar2 = modelStateT(2,:);
%                             % we use upto 2nd order polynomial as regression
%                             % equatioin
% 
%                             regMatrix = ones(pathSize,dimensionLS);
% 
%                             for idx_r=1:pathSize
%                                 for idxLS=1:numOfFactors
%                                     regMatrix(idx_r,idxLS+1) = decisionVar(idxLS,idx_r);
%                                 end
% 
%                                 tempIdx = 0;
%                                 for idxLS1=1:numOfFactors
%                                     for idxLS2=idxLS1:numOfFactors
%                                         tempIdx = tempIdx + 1;
%                                         regMatrix(idx_r,numOfFactors+1+tempIdx) = decisionVar(idxLS1,idx_r)*decisionVar(idxLS2,idx_r);
%                                     end
%                                 end
%                                 
%                                 tempIdx2 = 0;
%                                 for idxLS1=1:numOfFactors
%                                     for idxLS2=idxLS1:numOfFactors
%                                         for idxLS3=idxLS2:numOfFactors
%                                             tempIdx2 = tempIdx2 + 1;
%                                             regMatrix(idx_r,numOfFactors+1+tempIdx + tempIdx2) = decisionVar(idxLS1,idx_r)*decisionVar(idxLS2,idx_r)*decisionVar(idxLS3,idx_r);
%                                         end
%                                     end
%                                 end
%                                 
% 
%                             end
%                         
%                         else
%                             dimensionLS = (numOfFactors + 1)*(numOfFactors + 2)/2;
% 
%                             decisionVar = zeros(numOfFactors,size(modelStateT,2));
% 
%                             for idxLS =1:numOfFactors
%                                 decisionVar(idxLS,:) = modelStateT(idxLS,:); 
%                             end
%     %                         decisionVar1 = modelStateT(1,:);
%     %                         decisionVar2 = modelStateT(2,:);
%                             % we use upto 2nd order polynomial as regression
%                             % equatioin
% 
%                             regMatrix = ones(pathSize,dimensionLS);
% 
%                             for idx_r=1:pathSize
%                                 for idxLS=1:numOfFactors
%                                     regMatrix(idx_r,idxLS+1) = decisionVar(idxLS,idx_r);
%                                 end
% 
%                                 tempIdx = 1;
%                                 for idxLS1=1:numOfFactors
%                                     for idxLS2=idxLS1:numOfFactors
%                                         regMatrix(idx_r,numOfFactors+1+tempIdx) = decisionVar(idxLS1,idx_r)*decisionVar(idxLS2,idx_r);
%                                         tempIdx = tempIdx + 1;
%                                     end
%                                 end
% 
%                             end
%                         end
%                         % simple regression <- for path dependent option
%                         % we should change it
%                         % beta  : regression coefficients
%                         beta =zeros(dimensionLS,1);
%                         beta = regress(contiValue',regMatrix);
%                         regContiValue = regMatrix*beta;
%                        
%                          %Nominal Regression test
%                         regContiValue = regContiValue * 10000;
%                         
%                         exercisedPayoff = regContiValue;
%                         % if regContiValue is bigger than exerciseValue
%                         % then issue exercise call option
%                         for idx_r=1:pathSize
%                             if regContiValue(idx_r) >= exerciseValue(idx_r) 
%                                 exercisedPayoff(idx_r) = exerciseValue(idx_r);
%                             end
%                         end
%                         
%                         % we discount the new payoff and 
%                         % assign it into the existing callable payoff
%                         callable.payoff = model.discountPayoffRisky(currentTime,numMatTime ...
%                         ,modelStateT,exercisedPayoff);
%                         
%                     end
                end

                nonCall.npv = mean(nonCall.payoff);
                out.nonCall = nonCall;
                callable.npv = mean(callable.payoff);
                dfCol.npv = mean(dfCol.payoff);
                dummy1.npv = mean(dummy1.npv);
                dummy2.npv = mean(dummy2.npv);
                out.callable = callable;
                out.dfCol = dfCol;
                out.dummy1 = dummy1;
                out.dummy2 = dummy2;
                out.daycountFraction = schedule.dayCountFraction;
                
        end
        
        
        function out = computePriceClosedSimple(pricer,valueDate)
                security = pricer.m_security;
                numMethod = pricer.m_numMethod;
                model = pricer.m_model;
                schedule = security.couponSchedule;
                
                % numMethodTimeSteps Generate
                mcOneTimeStep = numMethod.mcOneTimeStep;
                outInfo = numMethod.numMethodTimeStepsGenerateDW(valueDate,schedule,mcOneTimeStep);
            
                numMethodInfo.timeStepsInfo = outInfo;

                % mc random number init
                %pathSize = 2^16;
                pathSize = numMethod.pathSize;
                
                pathSize = 1;
                
%                 % generate random numbers and modelstates forward
%                 rng('default');
%                 rng(0);
% 
                numOfFactors = model.numOfFactor;
% 
%                 %Z= randn(numOfFactors*outInfo.totalTimeStepsSize,pathSize/2);
%                 Z= randn(pathSize/2,numOfFactors*outInfo.totalTimeStepsSize);
%                 Z=[Z;-Z];
%                 Z=Z';
%                 
%                 
%                 %TimeInversion
%                 sizeZ = size(Z,1);
%                 A = zeros(sizeZ,sizeZ);
%                 for i=1:sizeZ
%                     A(i,sizeZ + 1 -i) = 1;
%                 end
%                 Z = A*Z;
% 
%                 % we check with the c, so we use generated random numbers
% %                 mc_random = importdata('mc_random.xlsx');
% %                 mc_random = importdata('D:\mc_random_number_new_report.txt');
% %                 Z= mc_random;
%                 
                numMethodInfo.pathSize = pathSize;
                numMethodInfo.numOfFactors = numOfFactors;
                numMethodInfo.totalTimeSteps = outInfo.totalTimeSteps;
                numMethodInfo.totalTimeStepsSize = outInfo.totalTimeStepsSize;
%                 numMethodInfo.Z = Z;
% 
% 
% 
%                 %generate model states for the given timeSteps
%                 modelStates = numMethod.generateModelStates(valueDate,schedule,numMethodInfo,model);

                % numeraire info
                numMatTime = numMethodInfo.totalTimeSteps(end);
                dfNumMat = model.DFRisky(numMatTime/365.0);
                
                % induct backward and calculate cashflow
            
                periodCount = schedule.numOfPeriod;
                
                % cashflow columns initialize
                % 1st nonCall
                nonCall.npv = 0;
                nonCall.cashflow_e = zeros(periodCount + 1,1);
                nonCall.cashflow_npv_e = zeros(periodCount + 1,1);
                nonCall.rangeProb_e = zeros(periodCount + 1,1);
                nonCall.df_e = zeros(periodCount + 1,1);
                nonCall.df_c = zeros(periodCount + 1,1);

                % debug information, cashflow & npv per path
                nonCall.payoff = zeros(1,pathSize);
                nonCall.payoffStates.cashflow = zeros(periodCount+1,pathSize);
                nonCall.payoffStates.cashflow_npv = zeros(periodCount+1,pathSize);
                nonCall.payoffStates.df = zeros(periodCount + 1,pathSize);
                
                % intermediate variables
                payoffStateA.cashflow = zeros(1, pathSize);
                payoffStateA.cashflow_npv = zeros(1, pathSize);
                payoffStateA.df = zeros(1,pathSize);
                
                % we calculate cashflow backward from maturityDate to valueDate
                % rangeAccrual coupons are added at the startDate of the Period
                % At maturity

                nominal = security.nominal;
                coupon = security.coupon;
                callYN = security.callYN;
                
                tenorCMS = security.tenor;
                
                ralower = security.lower;
                raupper = security.upper;
                
                tenorLong = security.tenorL;
                tenorShort = security.tenorS;
                
                ralowerS = security.lowerS;
                raupperS = security.upperS;
                
                currentTime = numMethodInfo.totalTimeSteps(end);
                totalTimeSteps = numMethodInfo.timeStepsInfo.totalTimeSteps;
                totalTimeStepsSize = numMethodInfo.timeStepsInfo.totalTimeStepsSize;
                if totalTimeStepsSize == 0 || (totalTimeStepsSize == 1 && totalTimeSteps(1) == 0) 
                    out.nonCall = nonCall;
                    return;
                end
                
                currentTimeIdx = totalTimeStepsSize;
                
                currentNodeIdx = periodCount + 1;
                
%                 modelStateT = modelStates(numOfFactors*(totalTimeStepsSize-1)+1:numOfFactors*(totalTimeStepsSize-1)+2,:);
                payoffStateA.cashflow = nominal*ones(1,pathSize);
                %process payoff at maturity
                nonCall.payoffStates.cashflow(currentNodeIdx,:) = payoffStateA.cashflow;
                
%                 payoffStateA.cashflow_npv = model.discountPayoff(currentTime,numMatTime ...
%                     ,modelStateT,payoffStateA.cashflow);
%                 
                payoffStateA.cashflow_npv = model.DFRisky(currentTime/365.0)*payoffStateA.cashflow;
                
                nonCall.payoffStates.cashflow_npv(currentNodeIdx,:) = payoffStateA.cashflow_npv ;
                
%                 payoffStateA.df = model.discountPayoff(currentTime,numMatTime ...
%                     ,modelStateT,1.0*ones(1,pathSize));
                payoffStateA.df = model.DFRisky(currentTime/365.0) *ones(1,pathSize);
                    

                nonCall.payoffStates.df(currentNodeIdx,:) = payoffStateA.df;
                nonCall.payoff = nonCall.payoff + payoffStateA.cashflow_npv;
                
                nonCall.cashflow_e(currentNodeIdx) = mean(payoffStateA.cashflow);
                nonCall.cashflow_npv_e(currentNodeIdx) = mean(payoffStateA.cashflow_npv);
                nonCall.df_e(currentNodeIdx) = mean(payoffStateA.df);
                nonCall.df_c(currentNodeIdx) = model.DFRisky(currentTime/365.0);
                
%                 % callable at maturity is the same as noncallable
%                 callable.payoffStates.cashflow(currentNodeIdx,:) = nonCall.payoffStates.cashflow(currentNodeIdx,:);
%                 callable.payoffStates.cashflow_npv(currentNodeIdx,:) = nonCall.payoffStates.cashflow_npv(currentNodeIdx,:);
%                 callable.payoff = callable.payoff + callable.payoffStates.cashflow_npv(currentNodeIdx,:);
%                 
%                 callable.cashflow_e(currentNodeIdx) = nonCall.cashflow_e(currentNodeIdx);
%                 callable.cashflow_npv_e(currentNodeIdx) = nonCall.cashflow_npv_e(currentNodeIdx);
                  
                lastTimeIdx = currentTimeIdx;
                
                
                % flag for inclusive schedule
%                 simpleYN = false;
                simpleYN = 1;
                
                for i=periodCount:-1:1
                    % past schedule neglect
                    if DateDiff(valueDate,schedule.payDates(i)) >= 0, continue;end 
                    startTimeIdx = numMethodInfo.timeStepsInfo.startTimeIdx(i);
                    endTimeIdx = numMethodInfo.timeStepsInfo.endTimeIdx(i);

                    inclusiveYN = numMethodInfo.timeStepsInfo.inclusiveYN(i);
                     %% FromToInduction start

                    %modelsStates induct backward(lastTimeIdx to endTimeIdx)
                    while endTimeIdx < lastTimeIdx
                        lastTimeIdx = lastTimeIdx -1;
                        if lastTimeIdx == endTimeIdx break;end
                    end

                    currentTimeIdx = lastTimeIdx;
                    currentTime = totalTimeSteps(currentTimeIdx);
                    % now we are at endTimeIdx
                    % variables for coupon valuation initialized
%                     ralower = security.lower;
%                     raupper = security.upper;
%                     tenorLong = security.tenorL;
%                     tenorShort = security.tenorS;

                    cD = 0; % cumulated day count fraction
                    cP = 0; 
                    dI = 0; % dummy idx for loop
                    dcfC = 0;
                    dcfN = 0;
                    dD = 0;
                    idxCMS = zeros(1,pathSize);
                    
                    idxC = zeros(1,pathSize);
                    idxN = zeros(1,pathSize);
                    
                    idxL = zeros(1,pathSize);
                    idxS = zeros(1,pathSize);
                    spreadC = zeros(1,pathSize);
                    spreadN = zeros(1,pathSize);
                    probSpread = zeros(1,pathSize);
                    
                    numOfTimeSteps = numMethodInfo.timeStepsInfo.numOfTimeSteps(i);
                    
                    % if inclusive then we use simpleYN rangeProb
                    %if inclusive then we do not evaluate the last step but
                    % use fixing instead 
                    endTimeStep = 1;
                    if inclusiveYN
%                         simpleYN = true;
                        simpleYN = 1;
                        endTimeStep = 2;
                    end
                    
                    for j=numOfTimeSteps:-1:endTimeStep
                        % evaluate dealdescription
                        % last step is omitted
                        currentTime = totalTimeSteps(currentTimeIdx);
                        %dcfC = currentTime/365.0;
                        dcfC = currentTime;
                        
                        if j~= numOfTimeSteps
                            dD = (dcfN-dcfC);
                        end

                        toDate = AddDate(valueDate,currentTime,'day');
                        tpDate = schedule.payDates(i);
%                         spreadC = model.CMSCMTSpreadDigitalRangeDate(valueDate,toDate,tpDate,tenorLong,tenorShort,ralower,raupper);
                        spreadC = model.CMSCMSSpreadDigitalRangeDate(valueDate,toDate,tpDate,tenorCMS,tenorLong,tenorShort,0.25,ralower,raupper,ralowerS,raupperS);
                        
                        %modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:numOfFactors*(currentTimeIdx-1)+2,:);
%                         idxL = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorLong,modelStateT);
%                         idxS = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorShort,modelStateT);
%                         spreadC = idxL - idxS;
%                         probSpread = model.SRPr(spreadC,spreadN,ralower,raupper,simpleYN,pathSize)*dD; 

                        if simpleYN
                            probSpread = spreadC*dD;
                        else
                            probSpread = spreadC*dD;
%                             probSpread = 0.5*(spreadC + spreadN)*dD;
                            
                        end

                        
                        cD = cD + dD;
                        cP = cP + probSpread;

                        dcfN = dcfC;
                        spreadN = spreadC;

                        %induct backward to the next time step
                        % one time backward induction
                        lastTimeIdx = currentTimeIdx;

                        if startTimeIdx < lastTimeIdx
                            lastTimeIdx = lastTimeIdx -1;
                            currentTimeIdx = lastTimeIdx;
                        end


                    end

                    %% FromToInduction end
                    %% AtDate evaluation
                    % Adding a new cashflow to the payoff

                    dcf = schedule.dayCountFraction(i);
                    
                    if inclusiveYN
                        % we add valuation date
                        cD = schedule.dayCount(i);
                        rangeDays =  security.rangeDays(i);
                        payoffStateA.cashflow = nominal * coupon * dcf * (rangeDays + cP)/cD;
                    else
                        payoffStateA.cashflow = nominal * coupon * dcf * cP/cD;
                    end
                    % eventDate(i) = startDate(i) therefore there is no induction in modelStates
                    % i.e. currentTimeIdx has not changed
                    % no need to update modelStateT
                    % modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:end,:);
                    % since cashflow's are added at the startDate(i) we need to
                    % multiply cashflow with the df(startDate(i), payDate(i))
                    payTime = DateDiff(schedule.payDates(i),valueDate);
                    
                    %event is at startDate so we need to discount the
                    %cashflow first from paydate to startdate
%                     df_sp = model.discountFactor(currentTime, payTime,numMatTime,modelStateT);

%                     for k=1:pathSize
%                         payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_sp(k);
%                     end
                    
                    % process payoff
                    
                    currentNodeIdx = i;
                    
                    % 1st nonCall column
                    nonCall.payoffStates.cashflow(currentNodeIdx,:) = payoffStateA.cashflow;
                    payoffStateA.cashflow_npv = payoffStateA.cashflow * model.DFRisky(payTime/365.0);
                    
%                     payoffStateA.cashflow_npv = model.discountPayoff(currentTime,numMatTime ...
%                     ,modelStateT,payoffStateA.cashflow);
                    nonCall.payoffStates.cashflow_npv(currentNodeIdx,:) = payoffStateA.cashflow_npv;
                    payoffStateA.df = model.DFRisky(payTime/365.0);
%                     payoffStateA.df = model.discountPayoff(currentTime,numMatTime ...
%                     ,modelStateT,1.0*ones(1,pathSize));
                    nonCall.payoffStates.df(currentNodeIdx,:) = payoffStateA.df;
                    
                    nonCall.payoff = nonCall.payoff + payoffStateA.cashflow_npv;
                    
                    nonCall.cashflow_e(currentNodeIdx) = mean(payoffStateA.cashflow);
                    nonCall.rangeProb_e(currentNodeIdx) = mean(nonCall.cashflow_e(currentNodeIdx)/(nominal * coupon * dcf));
                    nonCall.cashflow_npv_e(currentNodeIdx) = mean(payoffStateA.cashflow_npv);

                    nonCall.df_e(currentNodeIdx) = mean(payoffStateA.df);
                    nonCall.df_c(currentNodeIdx) = model.DFRisky(currentTime/365.0);
                    

                end

                nonCall.npv = mean(nonCall.payoff);
                out.pricerInfo.nonCall = nonCall;
                
%                 callable.npv = mean(callable.payoff);
%                 out.pricerInfo.callable = callable;
                out.daycountFraction = schedule.dayCountFraction;
                
        end
        

        
        
        
        
    end
    
end



In [ ]:
classdef DailyWeeklyRiskyPricer_USDCMS_KRWCMSSpreadDualRAGeneric < H_RootObject
    %SimpleCall + assetName for tenor,tenorL,tenorS
    %Pricer(Security,Model,NumMethod)
    %Prcer is constructed from Security, Model, NumMethod 
    
    properties
        m_security
        m_model
        m_numMethod
        overHedgeYN
    end
    
    methods
        function pricer = DailyWeeklyRiskyPricer_USDCMS_KRWCMSSpreadDualRAGeneric(security,model,numMethod,overHedgeYN)
                pricer.m_security = security;
                pricer.m_model = model;
                pricer.m_numMethod = numMethod;
                pricer.overHedgeYN = overHedgeYN;
        end
        
        function out = computePriceMCCallableGeneric(pricer,valueDate,PSAOption)
                security = pricer.m_security;
                numMethod = pricer.m_numMethod;
                model = pricer.m_model;
                schedule = security.couponSchedule;
                callSchedule = security.callSchedule;
                
                % numMethodTimeSteps Generate
                mcOneTimeStep = numMethod.mcOneTimeStep;
%                 outInfo = numMethod.numMethodTimeStepsGenerateDW(valueDate,schedule,mcOneTimeStep);
                
                floatingSchedule = struct;
                outInfo = numMethod.numMethodTimeStepsGenerateDWSwapSimpleCall(valueDate,schedule,floatingSchedule,...
                                    callSchedule,mcOneTimeStep);
            
                numMethodInfo.timeStepsInfo = outInfo;

                % mc random number init
                %pathSize = 2^16;
                pathSize = numMethod.pathSize;

                % generate random numbers and modelstates forward
                rng('default');
                rng(0);

                % LGM2F : two factor
                % CK1F  : one factor
                
                numOfFactors = model.numOfFactor;

                %Z= randn(numOfFactors*outInfo.totalTimeStepsSize,pathSize/2);
%                 Z= randn(pathSize/2,numOfFactors*outInfo.totalTimeStepsSize);
                Z= randn(pathSize/2,numOfFactors*outInfo.modelTimeStepsSize);
                Z=[Z;-Z];
                Z=Z';
                
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;

                % we check with the c code, so we use generated random numbers
%                 mc_random = importdata('mc_random.xlsx');
%                 mc_random = importdata('D:\mc_random_number_new_report.txt');
%                 Z= mc_random;
                
                numMethodInfo.pathSize = pathSize;
                numMethodInfo.numOfFactors = numOfFactors;
                numMethodInfo.totalTimeSteps = outInfo.totalTimeSteps;
                numMethodInfo.totalTimeStepsSize = outInfo.totalTimeStepsSize;
                
                % for modelStateGeneration
                numMethodInfo.modelTimeSteps = outInfo.modelTimeSteps;
                numMethodInfo.modelTimeStepsSize = outInfo.modelTimeStepsSize;
                
                numMethodInfo.pastTimeStepsSize = outInfo.pastTimeStepsSize;
                
                pastTimeStepsSize = numMethodInfo.pastTimeStepsSize;
                
                numMethodInfo.Z = Z;



                %numMethod generate model states for the given timeSteps
%                 modelStates = numMethod.generateModelStatesNF(valueDate,schedule,numMethodInfo,model);
                 modelStates = numMethod.generateModelStatesNFNew(valueDate,schedule,numMethodInfo,model);
                % numeraire info
                numMatTime = numMethodInfo.totalTimeSteps(end);
                days = DateDiff(schedule.payDates(schedule.numOfPeriod),valueDate);
                numMatTime = max(numMatTime, days);
                dfNumMat = model.DFRisky(numMatTime/365.0);
                
                numeraire.numMatTime = numMatTime;
                numeraire.dfNumMat = dfNumMat;
                
                % induct backward and calculate cashflow
            
                periodCount = schedule.numOfPeriod;
                
                % pricer single info initialize
                
                nonCall = PricerInfo(periodCount +1, pathSize);
                callable = PricerInfo(periodCount +1, pathSize);
                dfCol = PricerInfo(periodCount +1, pathSize);
                
                dummy1 = PricerInfo(periodCount +1, pathSize);
                dummy2 = PricerInfo(periodCount +1, pathSize);
                
%                 % additional floating leg info
%                 if ~isempty(fieldnames(floatingSchedule))
%                     floatingLeg = PricerInfo(floatingSchedule.numOfPeriod, pathSize);
%                     % currentFloatingLegIdx for iteration in dealing with 
%                     % the floatingLeg cashflow
%                     currentFloatingLegIdx = floatingSchedule.numOfPeriod;
%                 end
                
                % intermediate variables
                payoffStateA.cashflow = zeros(1, pathSize);
                payoffStateA.cashflow_npv = zeros(1, pathSize);
                
            
                % we calculate cashflow backward from maturityDate to valueDate
                % rangeAccrual coupons are added at the startDate of the Period
                % At maturity

                nominal = security.nominal;
                coupon = security.coupon;
                callYN = security.callYN;
                
                tenorCMS = security.tenor;
                
                tenorAssetName = security.tenorAssetName;
                tenorRateType = security.tenorRateType;
        
                ralower = security.lower;
                raupper = security.upper;
                
                tenorLong = security.tenorL;
                tenorShort = security.tenorS;
                
                tenorLAssetName = security.tenorLAssetName;
                tenorSAssetName = security.tenorSAssetName;
                
                tenorLRateType = security.tenorLRateType;
                tenorSRateType = security.tenorSRateType;
        
                ralowerS = security.lowerS;
                raupperS = security.upperS;
                
                currentTime = numMethodInfo.totalTimeSteps(end);
                modelTimeStepsSize = numMethodInfo.modelTimeStepsSize;
                currentTimeIdx = numMethodInfo.totalTimeStepsSize;
                
                currentNodeIdx = periodCount + 1;
                
                modelStateT = modelStates(numOfFactors*(max(currentTimeIdx-pastTimeStepsSize-1,0))+1:numOfFactors*(max(currentTimeIdx-pastTimeStepsSize-1,0))+numOfFactors,:);
%                 modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:numOfFactors*(currentTimeIdx-1)+numOfFactors,:);
                
                payTime = DateDiff(schedule.payDates(schedule.numOfPeriod),valueDate);
                
                %even though there is no cashflow at the last structured
                %enddate, we add dummy cashflow in line with the note
                if payTime > 0
                    payoffStateA.cashflow = nominal*ones(1,pathSize);

                    df_ep = model.discountFactorRisky(currentTime, payTime,numMatTime,modelStateT);
                    for k=1:pathSize
                        payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                    end

                    %process payoff at maturity
                    nonCall.ProcessPayoff(model,currentTime,numeraire, ...
                                         modelStateT,payoffStateA.cashflow,currentNodeIdx);

                    callable.ProcessPayoff(model,currentTime,numeraire, ...
                                         modelStateT,payoffStateA.cashflow,currentNodeIdx);

                    %df column
                    payoffStateA.cashflow = ones(1,pathSize);
                    for k=1:pathSize
                        payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                    end

                    dfCol.ProcessPayoff(model,currentTime,numeraire, ...
                                         modelStateT,payoffStateA.cashflow,currentNodeIdx);
                
                end

                lastTimeIdx = currentTimeIdx;
                totalTimeSteps = numMethodInfo.timeStepsInfo.totalTimeSteps;
                
                % flag for inclusive schedule
%                 simpleYN = false;
%                 simpleYN = true;
                if pricer.overHedgeYN == true 
                    simpleYN = 2;
                else
                    simpleYN = 1;
                end
                
                for i=periodCount:-1:1
                    % we induct in the pastTimeSteps too
%                     if DateDiff(valueDate,schedule.payDates(i)) >= 0, continue;end 
                    
                    startTimeIdx = numMethodInfo.timeStepsInfo.startTimeIdx(i);
                    endTimeIdx = numMethodInfo.timeStepsInfo.endTimeIdx(i);

                    inclusiveYN = numMethodInfo.timeStepsInfo.inclusiveYN(i);
                 %% FromToInduction start

                    %modelsStates induct backward(lastTimeIdx to endTimeIdx)
                    while endTimeIdx < lastTimeIdx
                        lastTimeIdx = lastTimeIdx -1;
                        if lastTimeIdx == endTimeIdx break;end
                    end

                    currentTimeIdx = lastTimeIdx;
                    currentTime = totalTimeSteps(currentTimeIdx);
                    % now we are at endTimeIdx
                    % variables for coupon valuation initialized
                    

                    cD = 0; % cumulated day count fraction
                     %cP shoul be declared as an array
%                     cP = 0;
                    cP = zeros(1,pathSize);
                    dI = 0; % dummy idx for loop
                    dcfC = 0;
                    dcfN = 0;
                    dD = 0;
                    idxCMS = zeros(1,pathSize);
                    
                    idxC = zeros(1,pathSize);
                    idxN = zeros(1,pathSize);
                    
                    idxL = zeros(1,pathSize);
                    idxS = zeros(1,pathSize);
                    
                    spreadC = zeros(1,pathSize);
                    spreadN = zeros(1,pathSize);
                    probSpread = zeros(1,pathSize);
                    
                    numOfTimeSteps = numMethodInfo.timeStepsInfo.numOfTimeSteps(i);
                    
                    % if inclusive then we use simpleYN rangeProb
                    %if inclusive then we do not evaluate the last step but
                    % use fixing instead 
                    endTimeStep = 1;
                     % we let endTimeStep to be 1 and
                    % inside the for loop if currentTime ==0
                    % then we skip the range accrual check part
                    
                    if inclusiveYN
%                         simpleYN = true;
                        simpleYN = 1;
%                         endTimeStep = 2;
                        if pricer.overHedgeYN == true 
                            simpleYN = 2;
                        end
                    end
                    
                    for j=numOfTimeSteps:-1:endTimeStep
                        % we initialize payoffStateA as zeros to prevent
                        % reuse it;
                        payoffStateA.cashflow = zeros(1,pathSize);
                        % evaluate FromTo dealdescription
                        % last step is omitted
                        currentTime = totalTimeSteps(currentTimeIdx);
                        if currentTime > 0
                            dcfC = currentTime;

                            if j~= numOfTimeSteps
                                dD = (dcfN-dcfC);
                            end

%                             modelStateT = modelStates(max(currentTimeIdx-pastTimeStepsSize-1,0)+1:numOfFactors*(currentTimeIdx-1)+numOfFactors,:);
                            modelStateT = modelStates(numOfFactors*(max(currentTimeIdx-pastTimeStepsSize-1,0))+1:numOfFactors*(max(currentTimeIdx-pastTimeStepsSize-1,0))+numOfFactors,:);
                            %% generic Rate_Index for Libor & CMS too
                            idxCMS = model.Rate_Index(tenorRateType,tenorAssetName,valueDate,currentTime,numMatTime, tenorCMS,modelStateT,PSAOption);
                            idxC = idxCMS;

                            idxL = model.Rate_Index(tenorLRateType,tenorLAssetName,valueDate,currentTime,numMatTime, tenorLong,modelStateT,PSAOption);
                            idxS = model.Rate_Index(tenorSRateType,tenorSAssetName,valueDate,currentTime,numMatTime, tenorShort,modelStateT,PSAOption);
                            spreadC = idxL - idxS;

%                             if strcmp(pricerOption,'PSA')
%                                 idxCMS = model.CMS_PSA_SimpleDate(tenorAssetName,valueDate,currentTime,numMatTime, tenorCMS,modelStateT);
%                                 idxC = idxCMS;
% 
%                                 idxL = model.CMS_PSA_SimpleDate(tenorLAssetName,valueDate,currentTime,numMatTime, tenorLong,modelStateT);
%                                 idxS = model.CMS_PSA_SimpleDate(tenorSAssetName,valueDate,currentTime,numMatTime, tenorShort,modelStateT);
%                                 spreadC = idxL - idxS;
%                             else
%                                 idxCMS = model.CMS_SimpleDate(tenorAssetName,valueDate,currentTime,numMatTime, tenorCMS,modelStateT);
%                                 idxC = idxCMS;
% 
%                                 idxL = model.CMS_SimpleDate(tenorLAssetName,valueDate,currentTime,numMatTime, tenorLong,modelStateT);
%                                 idxS = model.CMS_SimpleDate(tenorSAssetName,valueDate,currentTime,numMatTime, tenorShort,modelStateT);
%                                 spreadC = idxL - idxS;
%                             end

    %                         probSpread = model.SRPr(spreadC,spreadN,ralower,raupper,simpleYN,pathSize)*dD;
                            probSpread = model.DRPr(idxC,idxN,ralower,raupper,spreadC,spreadN,ralowerS,raupperS,simpleYN,pathSize)*dD;
                            cD = cD + dD;
                            cP = cP + probSpread;

                            dcfN = dcfC;

                            idxN = idxC;
                            spreadN = spreadC;
                        else
                            dcfC = currentTime;

                            if j~= numOfTimeSteps
                                dD = (dcfN-dcfC);
                            end

                            cD = cD + dD;

                            dcfN = dcfC;
                            
                        end
                        
                        %floating event begins here
                        %floating event end here
                        
                        % callable event begin
                        idxCall = find(currentTime == outInfo.callEventTime);
                        
                        % we do not evaluate callable event at the start of
                        % coupon Schedule for avoding double counting
                        % and that make sense
                        if ~isempty(idxCall) && (j~= endTimeStep)
                            iterCall = 1;
                            foundIterCallYN= false;
                            for idx11 = length(callSchedule.resetDates):-1:1
                                % call event happens at call startDate !
                                % we change call event happens at call resetDate !
                                
                                if outInfo.callEventTime(idxCall) == DateDiff(callSchedule.resetDates(idx11),valueDate)
                                    iterCall = idx11;
                                    foundIterCallYN = true;
                                    break;
                                end
                            end
                            
                            if ~foundIterCallYN
                                error('callEventTime is not in the callSchedule startDates')
                            end
                            
                            % only those cashflow that paydate is after
                            % valueDate will be added
                            if DateDiff(callSchedule.payDates(iterCall),valueDate) > 0
                                % callEvent is not fixed
                                if DateDiff(valueDate,callSchedule.resetDates(iterCall)) < 0
                                    % 1.undiscount callable.payoff
                                    % 2.build the state variables matrix
                                    % 3.regress 1's with the given state variables
                                    % (model states 1,2)
                                    % 4. exercise per each path in
                                    % discount the new payoff and assign into callable
                                    % column

                                    %1 undiscount callable.payoff and get the
                                    %contiValue

                                    %exercise value Note
                                    exerciseValue = security.nominal*ones(1,pathSize) * 1.0;
                                    payTime = DateDiff(callSchedule.payDates(iterCall),valueDate);
                                    %event is at startDate so we need to discount the
                                    %cashflow first from paydate to startdate
                                    df_ep = model.discountFactorRisky(currentTime, payTime,numMatTime,modelStateT);
                                    for k=1:pathSize
                                        exerciseValue(k) = exerciseValue(k) * df_ep(k);
                                    end
                                    
                                    contiValue = zeros(1,pathSize);
                                    onePayoff = ones(1,pathSize);
                                    onePayoffD = model.discountPayoffRisky(currentTime,numMatTime ...
                                    ,modelStateT,onePayoff);
                                    contiValue = callable.payoff./onePayoffD;

                                    %Nominal Regression test
                                    contiValue = contiValue/10000;

                                    % build the state variabls matrix
                                    if numMethod.regressionOrder == 3
            %                             dimensionLS = (numOfFactors + 1)*(numOfFactors + 2)/2;

                                        decisionVar = zeros(numOfFactors,size(modelStateT,2));

                                        for idxLS =1:numOfFactors
                                            decisionVar(idxLS,:) = modelStateT(idxLS,:); 
                                        end

                                        % calculate number of dimension first
                                        % 0th order
                                        dimensionLS = 1;

                                        % 1st order
                                        for idxLS=1:numOfFactors
                                            dimensionLS = dimensionLS + 1;
                                        end

                                        %2nd order
                                        for idxLS1=1:numOfFactors
                                            for idxLS2=idxLS1:numOfFactors
                                                dimensionLS = dimensionLS + 1;
                                            end
                                        end

                                        %3rd order
                                        for idxLS1=1:numOfFactors
                                            for idxLS2=idxLS1:numOfFactors
                                                for idxLS3=idxLS2:numOfFactors
                                                    dimensionLS = dimensionLS + 1;
                                                end
                                            end
                                        end


                                        % we use upto 2nd order polynomial as regression
                                        % equatioin

                                        regMatrix = ones(pathSize,dimensionLS);

                                        for idx_r=1:pathSize
                                            for idxLS=1:numOfFactors
                                                regMatrix(idx_r,idxLS+1) = decisionVar(idxLS,idx_r);
                                            end

                                            tempIdx = 0;
                                            for idxLS1=1:numOfFactors
                                                for idxLS2=idxLS1:numOfFactors
                                                    tempIdx = tempIdx + 1;
                                                    regMatrix(idx_r,numOfFactors+1+tempIdx) = decisionVar(idxLS1,idx_r)*decisionVar(idxLS2,idx_r);
                                                end
                                            end

                                            tempIdx2 = 0;
                                            for idxLS1=1:numOfFactors
                                                for idxLS2=idxLS1:numOfFactors
                                                    for idxLS3=idxLS2:numOfFactors
                                                        tempIdx2 = tempIdx2 + 1;
                                                        regMatrix(idx_r,numOfFactors+1+tempIdx + tempIdx2) = decisionVar(idxLS1,idx_r)*decisionVar(idxLS2,idx_r)*decisionVar(idxLS3,idx_r);
                                                    end
                                                end
                                            end


                                        end

                                    else
                                        dimensionLS = (numOfFactors + 1)*(numOfFactors + 2)/2;

                                        decisionVar = zeros(numOfFactors,size(modelStateT,2));

                                        for idxLS =1:numOfFactors
                                            decisionVar(idxLS,:) = modelStateT(idxLS,:); 
                                        end
            
                                        % we use upto 2nd order polynomial as regression
                                        % equatioin

                                        regMatrix = ones(pathSize,dimensionLS);

                                        for idx_r=1:pathSize
                                            for idxLS=1:numOfFactors
                                                regMatrix(idx_r,idxLS+1) = decisionVar(idxLS,idx_r);
                                            end

                                            tempIdx = 1;
                                            for idxLS1=1:numOfFactors
                                                for idxLS2=idxLS1:numOfFactors
                                                    regMatrix(idx_r,numOfFactors+1+tempIdx) = decisionVar(idxLS1,idx_r)*decisionVar(idxLS2,idx_r);
                                                    tempIdx = tempIdx + 1;
                                                end
                                            end

                                        end
                                    end
                                    % simple regression <- for path dependent option
                                    % we should change it
                                    % beta  : regression coefficients
                                    beta =zeros(dimensionLS,1);
                                    beta = regress(contiValue',regMatrix);
                                    regContiValue = regMatrix*beta;

                                     %Nominal Regression test
                                    regContiValue = regContiValue * 10000;

                                    exercisedPayoff = regContiValue;
                                    % if regContiValue is bigger than exerciseValue
                                    % then issue exercise call option
                                    for idx_r=1:pathSize
                                        if regContiValue(idx_r) >= exerciseValue(idx_r) 
                                            exercisedPayoff(idx_r) = exerciseValue(idx_r);
                                        end
                                    end

                                    % we discount the new payoff and 
                                    % assign it into the existing callable payoff
                                    callable.payoff = model.discountPayoffRisky(currentTime,numMatTime ...
                                    ,modelStateT,exercisedPayoff);

                                end
                                
                                
                            end
                        end
                        
                        % callable event end
                        
                        %induct backward to the next time step
                        % one time backward induction
                        lastTimeIdx = currentTimeIdx;

                        if startTimeIdx < lastTimeIdx
                            lastTimeIdx = lastTimeIdx -1;
                            currentTimeIdx = lastTimeIdx;
                        end


                    end

                    %% FromToInduction end
        % if the current period is inclusive, then the start date is not evaluated in the above for loop.
		% instead the fixing will be treated separately below.
		% however current_time_idx is updated at the end of the for loop.
		% so we need to update model_states accordingly.
		% if model_states were updated previously then the following additionaly step doesn't affect the result anyway.
                    currentTime = totalTimeSteps(currentTimeIdx);
%                     modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:numOfFactors*(currentTimeIdx-1)+numOfFactors,:);
                    modelStateT = modelStates(numOfFactors*(max(currentTimeIdx-pastTimeStepsSize-1,0))+1:numOfFactors*(max(currentTimeIdx-pastTimeStepsSize-1,0))+numOfFactors,:);
                    
                    %% AtDate evaluation
                    % Adding a new cashflow to the payoff
                    if DateDiff(schedule.payDates(i),valueDate) > 0
                        dcf = schedule.dayCountFraction(i);

                        if inclusiveYN
                            % we add valuation date
                            cD = schedule.dayCount(i);
                            rangeDays =  security.rangeDays(i);
                            payoffStateA.cashflow = nominal * coupon * dcf * (rangeDays + cP)/cD;
                        else
                            payoffStateA.cashflow = nominal * coupon * dcf * cP/cD;
                        end
                        % eventDate(i) = startDate(i) therefore there is no induction in modelStates
                        % i.e. currentTimeIdx has not changed
                        % no need to update modelStateT
                        % modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:end,:);
                        % since cashflow's are added at the startDate(i) we need to
                        % multiply cashflow with the df(startDate(i), payDate(i))
                        payTime = DateDiff(schedule.payDates(i),valueDate);

                        %event is at startDate so we need to discount the
                        %cashflow first from paydate to startdate
                        df_ep = model.discountFactorRisky(currentTime, payTime,numMatTime,modelStateT);

                        for k=1:pathSize
                            payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                        end

                        % process payoff

                        currentNodeIdx = i;

                        nonCall.ProcessPayoff(model,currentTime,numeraire, ...
                                         modelStateT,payoffStateA.cashflow,currentNodeIdx);

                        callable.ProcessPayoff(model,currentTime,numeraire, ...
                                         modelStateT,payoffStateA.cashflow,currentNodeIdx);

                        %df column
                        payoffStateA.cashflow = ones(1,pathSize);
                        for k=1:pathSize
                            payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                        end

                        dfCol.ProcessPayoff(model,currentTime,numeraire, ...
                                         modelStateT,payoffStateA.cashflow,currentNodeIdx);

                    end           
                    
   
  
                end

                nonCall.npv = mean(nonCall.payoff);
                out.nonCall = nonCall;
                callable.npv = mean(callable.payoff);
                dfCol.npv = mean(dfCol.payoff);
%                 dummy1.npv = mean(dummy1.npv);
%                 dummy2.npv = mean(dummy2.npv);
                out.callable = callable;
                out.dfCol = dfCol;
%                 out.dummy1 = dummy1;
%                 out.dummy2 = dummy2;
                out.daycountFraction = schedule.dayCountFraction;
                
        end
        
        
        function out = computePriceClosedSimple(pricer,valueDate)
                security = pricer.m_security;
                numMethod = pricer.m_numMethod;
                model = pricer.m_model;
                schedule = security.couponSchedule;
                
                % numMethodTimeSteps Generate
                mcOneTimeStep = numMethod.mcOneTimeStep;
                outInfo = numMethod.numMethodTimeStepsGenerateDW(valueDate,schedule,mcOneTimeStep);
            
                numMethodInfo.timeStepsInfo = outInfo;

                % mc random number init
                %pathSize = 2^16;
                pathSize = numMethod.pathSize;
                
                pathSize = 1;
                
%                 % generate random numbers and modelstates forward
%                 rng('default');
%                 rng(0);
% 
                numOfFactors = model.numOfFactor;
% 
%                 %Z= randn(numOfFactors*outInfo.totalTimeStepsSize,pathSize/2);
%                 Z= randn(pathSize/2,numOfFactors*outInfo.totalTimeStepsSize);
%                 Z=[Z;-Z];
%                 Z=Z';
%                 
%                 
%                 %TimeInversion
%                 sizeZ = size(Z,1);
%                 A = zeros(sizeZ,sizeZ);
%                 for i=1:sizeZ
%                     A(i,sizeZ + 1 -i) = 1;
%                 end
%                 Z = A*Z;
% 
%                 % we check with the c, so we use generated random numbers
% %                 mc_random = importdata('mc_random.xlsx');
% %                 mc_random = importdata('D:\mc_random_number_new_report.txt');
% %                 Z= mc_random;
%                 
                numMethodInfo.pathSize = pathSize;
                numMethodInfo.numOfFactors = numOfFactors;
                numMethodInfo.totalTimeSteps = outInfo.totalTimeSteps;
                numMethodInfo.totalTimeStepsSize = outInfo.totalTimeStepsSize;
%                 numMethodInfo.Z = Z;
% 
% 
% 
%                 %generate model states for the given timeSteps
%                 modelStates = numMethod.generateModelStates(valueDate,schedule,numMethodInfo,model);

                % numeraire info
                numMatTime = numMethodInfo.totalTimeSteps(end);
                dfNumMat = model.DFRisky(numMatTime/365.0);
                
                % induct backward and calculate cashflow
            
                periodCount = schedule.numOfPeriod;
                
                % cashflow columns initialize
                % 1st nonCall
                nonCall.npv = 0;
                nonCall.cashflow_e = zeros(periodCount + 1,1);
                nonCall.cashflow_npv_e = zeros(periodCount + 1,1);
                nonCall.rangeProb_e = zeros(periodCount + 1,1);
                nonCall.df_e = zeros(periodCount + 1,1);
                nonCall.df_c = zeros(periodCount + 1,1);

                % debug information, cashflow & npv per path
                nonCall.payoff = zeros(1,pathSize);
                nonCall.payoffStates.cashflow = zeros(periodCount+1,pathSize);
                nonCall.payoffStates.cashflow_npv = zeros(periodCount+1,pathSize);
                nonCall.payoffStates.df = zeros(periodCount + 1,pathSize);
                
                % intermediate variables
                payoffStateA.cashflow = zeros(1, pathSize);
                payoffStateA.cashflow_npv = zeros(1, pathSize);
                payoffStateA.df = zeros(1,pathSize);
                
                % we calculate cashflow backward from maturityDate to valueDate
                % rangeAccrual coupons are added at the startDate of the Period
                % At maturity

                nominal = security.nominal;
                coupon = security.coupon;
                callYN = security.callYN;
                
                tenorCMS = security.tenor;
                tenorAssetName = security.tenorAssetName;
                
                ralower = security.lower;
                raupper = security.upper;
                
                tenorLong = security.tenorL;
                tenorShort = security.tenorS;
                
                tenorLAssetName = security.tenorLAssetName;
                tenorSAssetName = security.tenorSAssetName;
                
                ralowerS = security.lowerS;
                raupperS = security.upperS;
                
                currentTime = numMethodInfo.totalTimeSteps(end);
                totalTimeSteps = numMethodInfo.timeStepsInfo.totalTimeSteps;
                totalTimeStepsSize = numMethodInfo.timeStepsInfo.totalTimeStepsSize;
                if totalTimeStepsSize == 0 || (totalTimeStepsSize == 1 && totalTimeSteps(1) == 0) 
                    out.nonCall = nonCall;
                    return;
                end
                
                currentTimeIdx = totalTimeStepsSize;
                
                currentNodeIdx = periodCount + 1;
                
%                 modelStateT = modelStates(numOfFactors*(totalTimeStepsSize-1)+1:numOfFactors*(totalTimeStepsSize-1)+2,:);
                payoffStateA.cashflow = nominal*ones(1,pathSize);
                %process payoff at maturity
                nonCall.payoffStates.cashflow(currentNodeIdx,:) = payoffStateA.cashflow;
                
%                 payoffStateA.cashflow_npv = model.discountPayoff(currentTime,numMatTime ...
%                     ,modelStateT,payoffStateA.cashflow);
%                 
                payoffStateA.cashflow_npv = model.DFRisky(currentTime/365.0)*payoffStateA.cashflow;
                
                nonCall.payoffStates.cashflow_npv(currentNodeIdx,:) = payoffStateA.cashflow_npv ;
                
%                 payoffStateA.df = model.discountPayoff(currentTime,numMatTime ...
%                     ,modelStateT,1.0*ones(1,pathSize));
                payoffStateA.df = model.DFRisky(currentTime/365.0) *ones(1,pathSize);
                    

                nonCall.payoffStates.df(currentNodeIdx,:) = payoffStateA.df;
                nonCall.payoff = nonCall.payoff + payoffStateA.cashflow_npv;
                
                nonCall.cashflow_e(currentNodeIdx) = mean(payoffStateA.cashflow);
                nonCall.cashflow_npv_e(currentNodeIdx) = mean(payoffStateA.cashflow_npv);
                nonCall.df_e(currentNodeIdx) = mean(payoffStateA.df);
                nonCall.df_c(currentNodeIdx) = model.DFRisky(currentTime/365.0);
                
%                 % callable at maturity is the same as noncallable
%                 callable.payoffStates.cashflow(currentNodeIdx,:) = nonCall.payoffStates.cashflow(currentNodeIdx,:);
%                 callable.payoffStates.cashflow_npv(currentNodeIdx,:) = nonCall.payoffStates.cashflow_npv(currentNodeIdx,:);
%                 callable.payoff = callable.payoff + callable.payoffStates.cashflow_npv(currentNodeIdx,:);
%                 
%                 callable.cashflow_e(currentNodeIdx) = nonCall.cashflow_e(currentNodeIdx);
%                 callable.cashflow_npv_e(currentNodeIdx) = nonCall.cashflow_npv_e(currentNodeIdx);
                  
                lastTimeIdx = currentTimeIdx;
                
                
                % flag for inclusive schedule
%                 simpleYN = false;
                simpleYN = 1;
                
                for i=periodCount:-1:1
                    % past schedule neglect
                    if DateDiff(valueDate,schedule.payDates(i)) >= 0, continue;end 
                    startTimeIdx = numMethodInfo.timeStepsInfo.startTimeIdx(i);
                    endTimeIdx = numMethodInfo.timeStepsInfo.endTimeIdx(i);

                    inclusiveYN = numMethodInfo.timeStepsInfo.inclusiveYN(i);
                     %% FromToInduction start

                    %modelsStates induct backward(lastTimeIdx to endTimeIdx)
                    while endTimeIdx < lastTimeIdx
                        lastTimeIdx = lastTimeIdx -1;
                        if lastTimeIdx == endTimeIdx break;end
                    end

                    currentTimeIdx = lastTimeIdx;
                    currentTime = totalTimeSteps(currentTimeIdx);
                    % now we are at endTimeIdx
                    % variables for coupon valuation initialized
%                     ralower = security.lower;
%                     raupper = security.upper;
%                     tenorLong = security.tenorL;
%                     tenorShort = security.tenorS;

                    cD = 0; % cumulated day count fraction
                    cP = 0; 
                    dI = 0; % dummy idx for loop
                    dcfC = 0;
                    dcfN = 0;
                    dD = 0;
                    idxCMS = zeros(1,pathSize);
                    
                    idxC = zeros(1,pathSize);
                    idxN = zeros(1,pathSize);
                    
                    idxL = zeros(1,pathSize);
                    idxS = zeros(1,pathSize);
                    spreadC = zeros(1,pathSize);
                    spreadN = zeros(1,pathSize);
                    probSpread = zeros(1,pathSize);
                    
                    numOfTimeSteps = numMethodInfo.timeStepsInfo.numOfTimeSteps(i);
                    
                    % if inclusive then we use simpleYN rangeProb
                    %if inclusive then we do not evaluate the last step but
                    % use fixing instead 
                    endTimeStep = 1;
                    if inclusiveYN
%                         simpleYN = true;
                        simpleYN = 1;
                        endTimeStep = 2;
                    end
                    
                    for j=numOfTimeSteps:-1:endTimeStep
                        % evaluate dealdescription
                        % last step is omitted
                        currentTime = totalTimeSteps(currentTimeIdx);
                        %dcfC = currentTime/365.0;
                        dcfC = currentTime;
                        
                        if j~= numOfTimeSteps
                            dD = (dcfN-dcfC);
                        end

                        toDate = AddDate(valueDate,currentTime,'day');
                        tpDate = schedule.payDates(i);
%                         spreadC = model.CMSCMTSpreadDigitalRangeDate(valueDate,toDate,tpDate,tenorLong,tenorShort,ralower,raupper);
                        spreadC = model.CMSCMSSpreadDigitalRangeDate(valueDate,toDate,tpDate,tenorCMS,tenorLong,tenorShort,0.25,ralower,raupper,ralowerS,raupperS);
                        
                        %modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:numOfFactors*(currentTimeIdx-1)+2,:);
%                         idxL = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorLong,modelStateT);
%                         idxS = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorShort,modelStateT);
%                         spreadC = idxL - idxS;
%                         probSpread = model.SRPr(spreadC,spreadN,ralower,raupper,simpleYN,pathSize)*dD; 

                        if simpleYN
                            probSpread = spreadC*dD;
                        else
                            probSpread = spreadC*dD;
%                             probSpread = 0.5*(spreadC + spreadN)*dD;
                            
                        end

                        
                        cD = cD + dD;
                        cP = cP + probSpread;

                        dcfN = dcfC;
                        spreadN = spreadC;

                        %induct backward to the next time step
                        % one time backward induction
                        lastTimeIdx = currentTimeIdx;

                        if startTimeIdx < lastTimeIdx
                            lastTimeIdx = lastTimeIdx -1;
                            currentTimeIdx = lastTimeIdx;
                        end


                    end

                    %% FromToInduction end
                    %% AtDate evaluation
                    % Adding a new cashflow to the payoff

                    dcf = schedule.dayCountFraction(i);
                    
                    if inclusiveYN
                        % we add valuation date
                        cD = schedule.dayCount(i);
                        rangeDays =  security.rangeDays(i);
                        payoffStateA.cashflow = nominal * coupon * dcf * (rangeDays + cP)/cD;
                    else
                        payoffStateA.cashflow = nominal * coupon * dcf * cP/cD;
                    end
                    % eventDate(i) = startDate(i) therefore there is no induction in modelStates
                    % i.e. currentTimeIdx has not changed
                    % no need to update modelStateT
                    % modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:end,:);
                    % since cashflow's are added at the startDate(i) we need to
                    % multiply cashflow with the df(startDate(i), payDate(i))
                    payTime = DateDiff(schedule.payDates(i),valueDate);
                    
                    %event is at startDate so we need to discount the
                    %cashflow first from paydate to startdate
%                     df_sp = model.discountFactor(currentTime, payTime,numMatTime,modelStateT);

%                     for k=1:pathSize
%                         payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_sp(k);
%                     end
                    
                    % process payoff
                    
                    currentNodeIdx = i;
                    
                    % 1st nonCall column
                    nonCall.payoffStates.cashflow(currentNodeIdx,:) = payoffStateA.cashflow;
                    payoffStateA.cashflow_npv = payoffStateA.cashflow * model.DFRisky(payTime/365.0);
                    
%                     payoffStateA.cashflow_npv = model.discountPayoff(currentTime,numMatTime ...
%                     ,modelStateT,payoffStateA.cashflow);
                    nonCall.payoffStates.cashflow_npv(currentNodeIdx,:) = payoffStateA.cashflow_npv;
                    payoffStateA.df = model.DFRisky(payTime/365.0);
%                     payoffStateA.df = model.discountPayoff(currentTime,numMatTime ...
%                     ,modelStateT,1.0*ones(1,pathSize));
                    nonCall.payoffStates.df(currentNodeIdx,:) = payoffStateA.df;
                    
                    nonCall.payoff = nonCall.payoff + payoffStateA.cashflow_npv;
                    
                    nonCall.cashflow_e(currentNodeIdx) = mean(payoffStateA.cashflow);
                    nonCall.rangeProb_e(currentNodeIdx) = mean(nonCall.cashflow_e(currentNodeIdx)/(nominal * coupon * dcf));
                    nonCall.cashflow_npv_e(currentNodeIdx) = mean(payoffStateA.cashflow_npv);

                    nonCall.df_e(currentNodeIdx) = mean(payoffStateA.df);
                    nonCall.df_c(currentNodeIdx) = model.DFRisky(currentTime/365.0);
                    

                end

                nonCall.npv = mean(nonCall.payoff);
                out.pricerInfo.nonCall = nonCall;
                
%                 callable.npv = mean(callable.payoff);
%                 out.pricerInfo.callable = callable;
                out.daycountFraction = schedule.dayCountFraction;
                
        end
        
        function out = computePriceClosedSimpleGeneric(pricer,valueDate)
                security = pricer.m_security;
                numMethod = pricer.m_numMethod;
                model = pricer.m_model;
                schedule = security.couponSchedule;
                callSchedule = security.callSchedule;
                
                % numMethodTimeSteps Generate
                mcOneTimeStep = numMethod.mcOneTimeStep;
                outInfo = numMethod.numMethodTimeStepsGenerateDW(valueDate,schedule,mcOneTimeStep);
                
%                 floatingSchedule = struct;
%                 outInfo = numMethod.numMethodTimeStepsGenerateDWSwapSimpleCall(valueDate,schedule,floatingSchedule,...
%                                     callSchedule,mcOneTimeStep);
                                
                numMethodInfo.timeStepsInfo = outInfo;

                pathSize = numMethod.pathSize;
                
                pathSize = 1;
                
 
                numOfFactors = model.numOfFactor;

%                 
                numMethodInfo.pathSize = pathSize;
                numMethodInfo.numOfFactors = numOfFactors;
                numMethodInfo.totalTimeSteps = outInfo.totalTimeSteps;
                numMethodInfo.totalTimeStepsSize = outInfo.totalTimeStepsSize;

                % numeraire info
                numMatTime = numMethodInfo.totalTimeSteps(end);
                dfNumMat = model.DFRisky(numMatTime/365.0);
                
                % induct backward and calculate cashflow
            
                periodCount = schedule.numOfPeriod;
                
                % cashflow columns initialize
                % 1st nonCall
                nonCall.npv = 0;
                nonCall.cashflow_e = zeros(periodCount + 1,1);
                nonCall.cashflow_npv_e = zeros(periodCount + 1,1);
                nonCall.rangeProb_e = zeros(periodCount + 1,1);
                nonCall.df_e = zeros(periodCount + 1,1);
                nonCall.df_c = zeros(periodCount + 1,1);

                % debug information, cashflow & npv per path
                nonCall.payoff = zeros(1,pathSize);
                nonCall.payoffStates.cashflow = zeros(periodCount+1,pathSize);
                nonCall.payoffStates.cashflow_npv = zeros(periodCount+1,pathSize);
                nonCall.payoffStates.df = zeros(periodCount + 1,pathSize);
                
                % intermediate variables
                payoffStateA.cashflow = zeros(1, pathSize);
                payoffStateA.cashflow_npv = zeros(1, pathSize);
                payoffStateA.df = zeros(1,pathSize);
                
                % we calculate cashflow backward from maturityDate to valueDate
                % rangeAccrual coupons are added at the startDate of the Period
                % At maturity

                nominal = security.nominal;
                coupon = security.coupon;
                callYN = security.callYN;
                
                tenorCMS = security.tenor;
                tenorAssetName = security.tenorAssetName;
                
                ralower = security.lower;
                raupper = security.upper;
                
                tenorLong = security.tenorL;
                tenorShort = security.tenorS;
                
                tenorLAssetName = security.tenorLAssetName;
                tenorSAssetName = security.tenorSAssetName;
                
                ralowerS = security.lowerS;
                raupperS = security.upperS;
                
                currentTime = numMethodInfo.totalTimeSteps(end);
                totalTimeSteps = numMethodInfo.timeStepsInfo.totalTimeSteps;
                totalTimeStepsSize = numMethodInfo.timeStepsInfo.totalTimeStepsSize;
                if totalTimeStepsSize == 0 || (totalTimeStepsSize == 1 && totalTimeSteps(1) == 0) 
                    out.nonCall = nonCall;
                    return;
                end
                
                currentTimeIdx = totalTimeStepsSize;
                
                currentNodeIdx = periodCount + 1;
                
                payoffStateA.cashflow = nominal*ones(1,pathSize);
                %process payoff at maturity
                nonCall.payoffStates.cashflow(currentNodeIdx,:) = payoffStateA.cashflow;
                
                payoffStateA.cashflow_npv = model.DFRisky(currentTime/365.0)*payoffStateA.cashflow;
                
                nonCall.payoffStates.cashflow_npv(currentNodeIdx,:) = payoffStateA.cashflow_npv ;
                
                payoffStateA.df = model.DFRisky(currentTime/365.0) *ones(1,pathSize);
                    

                nonCall.payoffStates.df(currentNodeIdx,:) = payoffStateA.df;
                nonCall.payoff = nonCall.payoff + payoffStateA.cashflow_npv;
                
                nonCall.cashflow_e(currentNodeIdx) = mean(payoffStateA.cashflow);
                nonCall.cashflow_npv_e(currentNodeIdx) = mean(payoffStateA.cashflow_npv);
                nonCall.df_e(currentNodeIdx) = mean(payoffStateA.df);
                nonCall.df_c(currentNodeIdx) = model.DFRisky(currentTime/365.0);
                  
                lastTimeIdx = currentTimeIdx;
                
                % flag for inclusive schedule
%                 simpleYN = false;
                simpleYN = 1;
                
                for i=periodCount:-1:1
                    % past schedule neglect
                    if DateDiff(valueDate,schedule.payDates(i)) >= 0, continue;end 
                    startTimeIdx = numMethodInfo.timeStepsInfo.startTimeIdx(i);
                    endTimeIdx = numMethodInfo.timeStepsInfo.endTimeIdx(i);

                    inclusiveYN = numMethodInfo.timeStepsInfo.inclusiveYN(i);
                     %% FromToInduction start

                    %modelsStates induct backward(lastTimeIdx to endTimeIdx)
                    while endTimeIdx < lastTimeIdx
                        lastTimeIdx = lastTimeIdx -1;
                        if lastTimeIdx == endTimeIdx break;end
                    end

                    currentTimeIdx = lastTimeIdx;
                    currentTime = totalTimeSteps(currentTimeIdx);
                    % now we are at endTimeIdx
                    % variables for coupon valuation initialized
%                     ralower = security.lower;
%                     raupper = security.upper;
%                     tenorLong = security.tenorL;
%                     tenorShort = security.tenorS;

                    cD = 0; % cumulated day count fraction
                    cP = 0; 
                    dI = 0; % dummy idx for loop
                    dcfC = 0;
                    dcfN = 0;
                    dD = 0;
                    idxCMS = zeros(1,pathSize);
                    
                    idxC = zeros(1,pathSize);
                    idxN = zeros(1,pathSize);
                    
                    idxL = zeros(1,pathSize);
                    idxS = zeros(1,pathSize);
                    spreadC = zeros(1,pathSize);
                    spreadN = zeros(1,pathSize);
                    probSpread = zeros(1,pathSize);
                    
                    numOfTimeSteps = numMethodInfo.timeStepsInfo.numOfTimeSteps(i);
                    
                    % if inclusive then we use simpleYN rangeProb
                    %if inclusive then we do not evaluate the last step but
                    % use fixing instead 
                    endTimeStep = 1;
                    if inclusiveYN
%                         simpleYN = true;
                        simpleYN = 1;
                        endTimeStep = 2;
                    end
                    
                    for j=numOfTimeSteps:-1:endTimeStep
                        % evaluate dealdescription
                        % last step is omitted
                        currentTime = totalTimeSteps(currentTimeIdx);
                        %dcfC = currentTime/365.0;
                        dcfC = currentTime;
                        
                        if j~= numOfTimeSteps
                            dD = (dcfN-dcfC);
                        end

                        toDate = AddDate(valueDate,currentTime,'day');
                        tpDate = schedule.payDates(i);
%                         spreadC = model.CMSCMTSpreadDigitalRangeDate(valueDate,toDate,tpDate,tenorLong,tenorShort,ralower,raupper);
                        spreadC = model.CMSCMSSpreadDigitalRangeDate(valueDate,toDate,tpDate,tenorCMS,tenorLong,tenorShort,0.25,ralower,raupper,ralowerS,raupperS);
                      

                        if simpleYN
                            probSpread = spreadC*dD;
                        else
                            probSpread = spreadC*dD;
%                             probSpread = 0.5*(spreadC + spreadN)*dD;
                            
                        end

                        
                        cD = cD + dD;
                        cP = cP + probSpread;

                        dcfN = dcfC;
                        spreadN = spreadC;

                        %induct backward to the next time step
                        % one time backward induction
                        lastTimeIdx = currentTimeIdx;

                        if startTimeIdx < lastTimeIdx
                            lastTimeIdx = lastTimeIdx -1;
                            currentTimeIdx = lastTimeIdx;
                        end


                    end

                    %% FromToInduction end
                    %% AtDate evaluation
                    % Adding a new cashflow to the payoff

                    dcf = schedule.dayCountFraction(i);
                    
                    if inclusiveYN
                        % we add valuation date
                        cD = schedule.dayCount(i);
                        rangeDays =  security.rangeDays(i);
                        payoffStateA.cashflow = nominal * coupon * dcf * (rangeDays + cP)/cD;
                    else
                        payoffStateA.cashflow = nominal * coupon * dcf * cP/cD;
                    end
                    % eventDate(i) = startDate(i) therefore there is no induction in modelStates
                    % i.e. currentTimeIdx has not changed
                    % no need to update modelStateT
                    % modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:end,:);
                    % since cashflow's are added at the startDate(i) we need to
                    % multiply cashflow with the df(startDate(i), payDate(i))
                    payTime = DateDiff(schedule.payDates(i),valueDate);
                    
                    %event is at startDate so we need to discount the
                    %cashflow first from paydate to startdate
%                     df_sp = model.discountFactor(currentTime, payTime,numMatTime,modelStateT);

%                     for k=1:pathSize
%                         payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_sp(k);
%                     end
                    
                    % process payoff
                    
                    currentNodeIdx = i;
                    
                    % 1st nonCall column
                    nonCall.payoffStates.cashflow(currentNodeIdx,:) = payoffStateA.cashflow;
                    payoffStateA.cashflow_npv = payoffStateA.cashflow * model.DFRisky(payTime/365.0);
                    
%                     payoffStateA.cashflow_npv = model.discountPayoff(currentTime,numMatTime ...
%                     ,modelStateT,payoffStateA.cashflow);
                    nonCall.payoffStates.cashflow_npv(currentNodeIdx,:) = payoffStateA.cashflow_npv;
                    payoffStateA.df = model.DFRisky(payTime/365.0);
%                     payoffStateA.df = model.discountPayoff(currentTime,numMatTime ...
%                     ,modelStateT,1.0*ones(1,pathSize));
                    nonCall.payoffStates.df(currentNodeIdx,:) = payoffStateA.df;
                    
                    nonCall.payoff = nonCall.payoff + payoffStateA.cashflow_npv;
                    
                    nonCall.cashflow_e(currentNodeIdx) = mean(payoffStateA.cashflow);
                    nonCall.rangeProb_e(currentNodeIdx) = mean(nonCall.cashflow_e(currentNodeIdx)/(nominal * coupon * dcf));
                    nonCall.cashflow_npv_e(currentNodeIdx) = mean(payoffStateA.cashflow_npv);

                    nonCall.df_e(currentNodeIdx) = mean(payoffStateA.df);
                    nonCall.df_c(currentNodeIdx) = model.DFRisky(currentTime/365.0);
                    

                end

                nonCall.npv = mean(nonCall.payoff);
                out.nonCall = nonCall;
                
%                 callable.npv = mean(callable.payoff);
%                 out.pricerInfo.callable = callable;
                out.daycountFraction = schedule.dayCountFraction;
                
        end
        
        
        
        
        
    end
    
end



In [ ]:
classdef DailyWeeklyRiskyPricer_USDCMS_KRWCMSSpreadDualRA < H_RootObject
    %Pricer(Security,Model,NumMethod)
    %Prcer is constructed from Security, Model, NumMethod 
    
    properties
        m_security
        m_model
        m_numMethod
        overHedgeYN
    end
    
    methods
        function pricer = DailyWeeklyRiskyPricer_USDCMS_KRWCMSSpreadDualRA(security,model,numMethod,overHedgeYN)
                pricer.m_security = security;
                pricer.m_model = model;
                pricer.m_numMethod = numMethod;
                pricer.overHedgeYN = overHedgeYN;
        end
        
        function out = computePriceMCCallableGeneric(pricer,valueDate,pricerOption)
                security = pricer.m_security;
                numMethod = pricer.m_numMethod;
                model = pricer.m_model;
                schedule = security.couponSchedule;
                callSchedule = security.callSchedule;
                
                % numMethodTimeSteps Generate
                mcOneTimeStep = numMethod.mcOneTimeStep;
                outInfo = numMethod.numMethodTimeStepsGenerateDW(valueDate,schedule,mcOneTimeStep);
            
                numMethodInfo.timeStepsInfo = outInfo;

                % mc random number init
                %pathSize = 2^16;
                pathSize = numMethod.pathSize;

                % generate random numbers and modelstates forward
                rng('default');
                rng(0);

                % LGM2F : two factor
                % CK1F  : one factor
                
                numOfFactors = model.numOfFactor;

                %Z= randn(numOfFactors*outInfo.totalTimeStepsSize,pathSize/2);
                Z= randn(pathSize/2,numOfFactors*outInfo.totalTimeStepsSize);
                Z=[Z;-Z];
                Z=Z';
                
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;

                % we check with the c code, so we use generated random numbers
%                 mc_random = importdata('mc_random.xlsx');
%                 mc_random = importdata('D:\mc_random_number_new_report.txt');
%                 Z= mc_random;
                
                numMethodInfo.pathSize = pathSize;
                numMethodInfo.numOfFactors = numOfFactors;
                numMethodInfo.totalTimeSteps = outInfo.totalTimeSteps;
                numMethodInfo.totalTimeStepsSize = outInfo.totalTimeStepsSize;
                numMethodInfo.Z = Z;



                %numMethod generate model states for the given timeSteps
                modelStates = numMethod.generateModelStatesNF(valueDate,schedule,numMethodInfo,model);

                % numeraire info
                numMatTime = numMethodInfo.totalTimeSteps(end);
                days = DateDiff(schedule.payDates(schedule.numOfPeriod),valueDate);
                numMatTime = max(numMatTime, days);
                dfNumMat = model.DFRisky(numMatTime/365.0);
                
                numeraire.numMatTime = numMatTime;
                numeraire.dfNumMat = dfNumMat;
                
                % induct backward and calculate cashflow
            
                periodCount = schedule.numOfPeriod;
                
                % pricer single info initialize
                
                nonCall = PricerInfo(periodCount +1, pathSize);
                callable = PricerInfo(periodCount +1, pathSize);
                dfCol = PricerInfo(periodCount +1, pathSize);
                
                dummy1 = PricerInfo(periodCount +1, pathSize);
                dummy2 = PricerInfo(periodCount +1, pathSize);
                
                
                % intermediate variables
                payoffStateA.cashflow = zeros(1, pathSize);
                payoffStateA.cashflow_npv = zeros(1, pathSize);
                
            
                % we calculate cashflow backward from maturityDate to valueDate
                % rangeAccrual coupons are added at the startDate of the Period
                % At maturity

                nominal = security.nominal;
                coupon = security.coupon;
                callYN = security.callYN;
                
                tenorCMS = security.tenor;
                
                ralower = security.lower;
                raupper = security.upper;
                
                tenorLong = security.tenorL;
                tenorShort = security.tenorS;
                
                ralowerS = security.lowerS;
                raupperS = security.upperS;
                
                currentTime = numMethodInfo.totalTimeSteps(end);
                totalTimeStepsSize = numMethodInfo.totalTimeStepsSize;
                currentTimeIdx = totalTimeStepsSize;
                
                currentNodeIdx = periodCount + 1;
                
                modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:numOfFactors*(currentTimeIdx-1)+numOfFactors,:);
                
                payoffStateA.cashflow = nominal*ones(1,pathSize);
                
                payTime = DateDiff(schedule.payDates(schedule.numOfPeriod),valueDate);
                df_ep = model.discountFactorRisky(currentTime, payTime,numMatTime,modelStateT);
                for k=1:pathSize
                    payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                end
                
                %process payoff at maturity
                nonCall.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                
                callable.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                
                %df column
                payoffStateA.cashflow = ones(1,pathSize);
                for k=1:pathSize
                    payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                end
                
                dfCol.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                                 
                %dummy column
                foreignModelName = model.foreignModelName;
                
                if strcmp(pricerOption,'PSA')
                    payoffStateA.cashflow = model.CMS_PSA_SimpleDate(model.refModelName,valueDate,currentTime,numMatTime, tenorLong,modelStateT);
                    dummy1.ProcessPayoff(model,currentTime,numeraire, ...
                                 modelStateT,payoffStateA.cashflow,currentNodeIdx);
                    
                    payoffStateA.cashflow = model.CMS_PSA_SimpleDate(foreignModelName,valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                    dummy2.ProcessPayoff(model,currentTime,numeraire, ...
                                 modelStateT,payoffStateA.cashflow,currentNodeIdx);
                
                else
                    payoffStateA.cashflow = model.CMS_SimpleDate(model.refModelName,valueDate,currentTime,numMatTime, tenorLong,modelStateT);
                    dummy1.ProcessPayoff(model,currentTime,numeraire, ...
                                 modelStateT,payoffStateA.cashflow,currentNodeIdx);
                    
                    payoffStateA.cashflow = model.CMS_SimpleDate(foreignModelName,valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                    dummy2.ProcessPayoff(model,currentTime,numeraire, ...
                                 modelStateT,payoffStateA.cashflow,currentNodeIdx);
                    
                end
                

%                 payoffStateA.cashflow = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                
%                 payoffStateA.cashflow = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                
                
                                 
                  

                lastTimeIdx = currentTimeIdx;
                totalTimeSteps = numMethodInfo.timeStepsInfo.totalTimeSteps;
                
                % flag for inclusive schedule
%                 simpleYN = false;
%                 simpleYN = true;
                if pricer.overHedgeYN == true 
                    simpleYN = 2;
                else
                    simpleYN = 1;
                end
                
                for i=periodCount:-1:1
                    % past schedule neglect
                    if DateDiff(valueDate,schedule.payDates(i)) >= 0, continue;end 
                    startTimeIdx = numMethodInfo.timeStepsInfo.startTimeIdx(i);
                    endTimeIdx = numMethodInfo.timeStepsInfo.endTimeIdx(i);

                    inclusiveYN = numMethodInfo.timeStepsInfo.inclusiveYN(i);
                 %% FromToInduction start

                    %modelsStates induct backward(lastTimeIdx to endTimeIdx)
                    while endTimeIdx < lastTimeIdx
                        lastTimeIdx = lastTimeIdx -1;
                        if lastTimeIdx == endTimeIdx break;end
                    end

                    currentTimeIdx = lastTimeIdx;
                    currentTime = totalTimeSteps(currentTimeIdx);
                    % now we are at endTimeIdx
                    % variables for coupon valuation initialized
                    

                    cD = 0; % cumulated day count fraction
                    cP = 0; 
                    dI = 0; % dummy idx for loop
                    dcfC = 0;
                    dcfN = 0;
                    dD = 0;
                    idxCMS = zeros(1,pathSize);
                    
                    idxC = zeros(1,pathSize);
                    idxN = zeros(1,pathSize);
                    
                    idxL = zeros(1,pathSize);
                    idxS = zeros(1,pathSize);
                    
                    spreadC = zeros(1,pathSize);
                    spreadN = zeros(1,pathSize);
                    probSpread = zeros(1,pathSize);
                    
                    numOfTimeSteps = numMethodInfo.timeStepsInfo.numOfTimeSteps(i);
                    
                    % if inclusive then we use simpleYN rangeProb
                    %if inclusive then we do not evaluate the last step but
                    % use fixing instead 
                    endTimeStep = 1;
                    if inclusiveYN
%                         simpleYN = true;
                        simpleYN = 1;
                        endTimeStep = 2;
                        if pricer.overHedgeYN == true 
                            simpleYN = 2;
                        end
                    end
                    
                    for j=numOfTimeSteps:-1:endTimeStep
                        % evaluate dealdescription
                        % last step is omitted
                        currentTime = totalTimeSteps(currentTimeIdx);
                        %dcfC = currentTime/365.0;
                        dcfC = currentTime;
                        
                        if j~= numOfTimeSteps
                            dD = (dcfN-dcfC);
                        end

                        modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:numOfFactors*(currentTimeIdx-1)+numOfFactors,:);
                        
                        if strcmp(pricerOption,'PSA')
                            idxCMS = model.CMS_PSA_SimpleDate(foreignModelName,valueDate,currentTime,numMatTime, tenorCMS,modelStateT);
                            idxC = idxCMS;

                            idxL = model.CMS_PSA_SimpleDate(model.refModelName,valueDate,currentTime,numMatTime, tenorLong,modelStateT);
                            idxS = model.CMS_PSA_SimpleDate(model.refModelName,valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                            spreadC = idxL - idxS;
                        else
                            idxCMS = model.CMS_SimpleDate(foreignModelName,valueDate,currentTime,numMatTime, tenorCMS,modelStateT);
                            idxC = idxCMS;

                            idxL = model.CMS_SimpleDate(model.refModelName,valueDate,currentTime,numMatTime, tenorLong,modelStateT);
                            idxS = model.CMS_SimpleDate(model.refModelName,valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                            spreadC = idxL - idxS;
                        end
%                         probSpread = model.SRPr(spreadC,spreadN,ralower,raupper,simpleYN,pathSize)*dD;
                        probSpread = model.DRPr(idxC,idxN,ralower,raupper,spreadC,spreadN,ralowerS,raupperS,simpleYN,pathSize)*dD;
                        cD = cD + dD;
                        cP = cP + probSpread;

                        dcfN = dcfC;
                        
                        idxN = idxC;
                        spreadN = spreadC;

                        %induct backward to the next time step
                        % one time backward induction
                        lastTimeIdx = currentTimeIdx;

                        if startTimeIdx < lastTimeIdx
                            lastTimeIdx = lastTimeIdx -1;
                            currentTimeIdx = lastTimeIdx;
                        end


                    end

                    %% FromToInduction end
        % if the current period is inclusive, then the start date is not evaluated in the above for loop.
		% instead the fixing will be treated separately below.
		% however current_time_idx is updated at the end of the for loop.
		% so we need to update model_states accordingly.
		% if model_states were updated previously then the following additionaly step doesn't affect the result anyway.
                    currentTime = totalTimeSteps(currentTimeIdx);
                    modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:numOfFactors*(currentTimeIdx-1)+numOfFactors,:);
                    
                    %% AtDate evaluation
                    % Adding a new cashflow to the payoff

                    dcf = schedule.dayCountFraction(i);
                    
                    if inclusiveYN
                        % we add valuation date
                        cD = schedule.dayCount(i);
                        rangeDays =  security.rangeDays(i);
                        payoffStateA.cashflow = nominal * coupon * dcf * (rangeDays + cP)/cD;
                    else
                        payoffStateA.cashflow = nominal * coupon * dcf * cP/cD;
                    end
                    % eventDate(i) = startDate(i) therefore there is no induction in modelStates
                    % i.e. currentTimeIdx has not changed
                    % no need to update modelStateT
                    % modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:end,:);
                    % since cashflow's are added at the startDate(i) we need to
                    % multiply cashflow with the df(startDate(i), payDate(i))
                    payTime = DateDiff(schedule.payDates(i),valueDate);
                    
                    %event is at startDate so we need to discount the
                    %cashflow first from paydate to startdate
                    df_ep = model.discountFactorRisky(currentTime, payTime,numMatTime,modelStateT);

                    for k=1:pathSize
                        payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                    end
                    
                    % process payoff
                    
                    currentNodeIdx = i;
                    
                    nonCall.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                
                    callable.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                    
                    %df column
                    payoffStateA.cashflow = ones(1,pathSize);
                    for k=1:pathSize
                        payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                    end
                    
                    dfCol.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                                 
                    %dummy column
                    if strcmp(pricerOption,'PSA')
                        payoffStateA.cashflow = model.CMS_PSA_SimpleDate(model.refModelName,valueDate,currentTime,numMatTime, tenorLong,modelStateT);
                        dummy1.ProcessPayoff(model,currentTime,numeraire, ...
                                         modelStateT,payoffStateA.cashflow,currentNodeIdx);



                        payoffStateA.cashflow = model.CMS_PSA_SimpleDate(foreignModelName,valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                        dummy2.ProcessPayoff(model,currentTime,numeraire, ...
                                         modelStateT,payoffStateA.cashflow,currentNodeIdx);
                    
                    else
                        payoffStateA.cashflow = model.CMS_SimpleDate(model.refModelName,valueDate,currentTime,numMatTime, tenorLong,modelStateT);
                        dummy1.ProcessPayoff(model,currentTime,numeraire, ...
                                         modelStateT,payoffStateA.cashflow,currentNodeIdx);



                        payoffStateA.cashflow = model.CMS_SimpleDate(foreignModelName,valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                        dummy2.ProcessPayoff(model,currentTime,numeraire, ...
                                         modelStateT,payoffStateA.cashflow,currentNodeIdx);
                        
                    end

                                 
                    
                    
                    
                    % if exercise is possible
                    if i == 1
                        dummaaa = 1;
                    end
                    
                    % call event happens  at the start of the period
                    % at call date, by default, call option is expired
                    
                    if callYN(i) && (DateDiff(valueDate,callSchedule.resetDates(i)) < 0)
                        % 1.undiscount callable.payoff
                        % 2.build the state variables matrix
                        % 3.regress 1's with the given state variables
                        % (model states 1,2)
                        % 4. exercise per each path in
                        % discount the new payoff and assign into callable
                        % column
                        
                        %1 undiscount callable.payoff and get the
                        %contiValue
                       
                        %exercise value
                        exerciseValue = security.nominal*ones(1,pathSize);
                        payTime = DateDiff(callSchedule.payDates(i),valueDate);
                        %event is at startDate so we need to discount the
                        %cashflow first from paydate to startdate
                        df_ep = model.discountFactorRisky(currentTime, payTime,numMatTime,modelStateT);
                        for k=1:pathSize
                            exerciseValue(k) = exerciseValue(k) * df_ep(k);
                        end
                        
                        contiValue = zeros(1,pathSize);
                        onePayoff = ones(1,pathSize);
                        onePayoffD = model.discountPayoffRisky(currentTime,numMatTime ...
                        ,modelStateT,onePayoff);
                        contiValue = callable.payoff./onePayoffD;
                        
                        %Nominal Regression test
                        contiValue = contiValue/10000;
                        
                        % build the state variabls matrix
                        if numMethod.regressionOrder == 3
%                             dimensionLS = (numOfFactors + 1)*(numOfFactors + 2)/2;

                            decisionVar = zeros(numOfFactors,size(modelStateT,2));

                            for idxLS =1:numOfFactors
                                decisionVar(idxLS,:) = modelStateT(idxLS,:); 
                            end
                            
                            % calculate number of dimension first
                            % 0th order
                            dimensionLS = 1;
                            
                            % 1st order
                            for idxLS=1:numOfFactors
                                dimensionLS = dimensionLS + 1;
                            end
                            
                            %2nd order
                            for idxLS1=1:numOfFactors
                                for idxLS2=idxLS1:numOfFactors
                                    dimensionLS = dimensionLS + 1;
                                end
                            end
                            
                            %3rd order
                            for idxLS1=1:numOfFactors
                                for idxLS2=idxLS1:numOfFactors
                                    for idxLS3=idxLS2:numOfFactors
                                        dimensionLS = dimensionLS + 1;
                                    end
                                end
                            end

                            
    %                         decisionVar1 = modelStateT(1,:);
    %                         decisionVar2 = modelStateT(2,:);
                            % we use upto 2nd order polynomial as regression
                            % equatioin

                            regMatrix = ones(pathSize,dimensionLS);

                            for idx_r=1:pathSize
                                for idxLS=1:numOfFactors
                                    regMatrix(idx_r,idxLS+1) = decisionVar(idxLS,idx_r);
                                end

                                tempIdx = 0;
                                for idxLS1=1:numOfFactors
                                    for idxLS2=idxLS1:numOfFactors
                                        tempIdx = tempIdx + 1;
                                        regMatrix(idx_r,numOfFactors+1+tempIdx) = decisionVar(idxLS1,idx_r)*decisionVar(idxLS2,idx_r);
                                    end
                                end
                                
                                tempIdx2 = 0;
                                for idxLS1=1:numOfFactors
                                    for idxLS2=idxLS1:numOfFactors
                                        for idxLS3=idxLS2:numOfFactors
                                            tempIdx2 = tempIdx2 + 1;
                                            regMatrix(idx_r,numOfFactors+1+tempIdx + tempIdx2) = decisionVar(idxLS1,idx_r)*decisionVar(idxLS2,idx_r)*decisionVar(idxLS3,idx_r);
                                        end
                                    end
                                end
                                

                            end
                        
                        else
                            dimensionLS = (numOfFactors + 1)*(numOfFactors + 2)/2;

                            decisionVar = zeros(numOfFactors,size(modelStateT,2));

                            for idxLS =1:numOfFactors
                                decisionVar(idxLS,:) = modelStateT(idxLS,:); 
                            end
    %                         decisionVar1 = modelStateT(1,:);
    %                         decisionVar2 = modelStateT(2,:);
                            % we use upto 2nd order polynomial as regression
                            % equatioin

                            regMatrix = ones(pathSize,dimensionLS);

                            for idx_r=1:pathSize
                                for idxLS=1:numOfFactors
                                    regMatrix(idx_r,idxLS+1) = decisionVar(idxLS,idx_r);
                                end

                                tempIdx = 1;
                                for idxLS1=1:numOfFactors
                                    for idxLS2=idxLS1:numOfFactors
                                        regMatrix(idx_r,numOfFactors+1+tempIdx) = decisionVar(idxLS1,idx_r)*decisionVar(idxLS2,idx_r);
                                        tempIdx = tempIdx + 1;
                                    end
                                end

                            end
                        end
                        % simple regression <- for path dependent option
                        % we should change it
                        % beta  : regression coefficients
                        beta =zeros(dimensionLS,1);
                        beta = regress(contiValue',regMatrix);
                        regContiValue = regMatrix*beta;
                       
                         %Nominal Regression test
                        regContiValue = regContiValue * 10000;
                        
                        exercisedPayoff = regContiValue;
                        % if regContiValue is bigger than exerciseValue
                        % then issue exercise call option
                        for idx_r=1:pathSize
                            if regContiValue(idx_r) >= exerciseValue(idx_r) 
                                exercisedPayoff(idx_r) = exerciseValue(idx_r);
                            end
                        end
                        
                        % we discount the new payoff and 
                        % assign it into the existing callable payoff
                        callable.payoff = model.discountPayoffRisky(currentTime,numMatTime ...
                        ,modelStateT,exercisedPayoff);
                        
                    end
                end

                nonCall.npv = mean(nonCall.payoff);
                out.nonCall = nonCall;
                callable.npv = mean(callable.payoff);
                dfCol.npv = mean(dfCol.payoff);
                dummy1.npv = mean(dummy1.npv);
                dummy2.npv = mean(dummy2.npv);
                out.callable = callable;
                out.dfCol = dfCol;
                out.dummy1 = dummy1;
                out.dummy2 = dummy2;
                out.daycountFraction = schedule.dayCountFraction;
                
        end
        
        
        function out = computePriceClosedSimple(pricer,valueDate)
                security = pricer.m_security;
                numMethod = pricer.m_numMethod;
                model = pricer.m_model;
                schedule = security.couponSchedule;
                
                % numMethodTimeSteps Generate
                mcOneTimeStep = numMethod.mcOneTimeStep;
                outInfo = numMethod.numMethodTimeStepsGenerateDW(valueDate,schedule,mcOneTimeStep);
            
                numMethodInfo.timeStepsInfo = outInfo;

                % mc random number init
                %pathSize = 2^16;
                pathSize = numMethod.pathSize;
                
                pathSize = 1;
                
%                 % generate random numbers and modelstates forward
%                 rng('default');
%                 rng(0);
% 
                numOfFactors = model.numOfFactor;
% 
%                 %Z= randn(numOfFactors*outInfo.totalTimeStepsSize,pathSize/2);
%                 Z= randn(pathSize/2,numOfFactors*outInfo.totalTimeStepsSize);
%                 Z=[Z;-Z];
%                 Z=Z';
%                 
%                 
%                 %TimeInversion
%                 sizeZ = size(Z,1);
%                 A = zeros(sizeZ,sizeZ);
%                 for i=1:sizeZ
%                     A(i,sizeZ + 1 -i) = 1;
%                 end
%                 Z = A*Z;
% 
%                 % we check with the c, so we use generated random numbers
% %                 mc_random = importdata('mc_random.xlsx');
% %                 mc_random = importdata('D:\mc_random_number_new_report.txt');
% %                 Z= mc_random;
%                 
                numMethodInfo.pathSize = pathSize;
                numMethodInfo.numOfFactors = numOfFactors;
                numMethodInfo.totalTimeSteps = outInfo.totalTimeSteps;
                numMethodInfo.totalTimeStepsSize = outInfo.totalTimeStepsSize;
%                 numMethodInfo.Z = Z;
% 
% 
% 
%                 %generate model states for the given timeSteps
%                 modelStates = numMethod.generateModelStates(valueDate,schedule,numMethodInfo,model);

                % numeraire info
                numMatTime = numMethodInfo.totalTimeSteps(end);
                dfNumMat = model.DFRisky(numMatTime/365.0);
                
                % induct backward and calculate cashflow
            
                periodCount = schedule.numOfPeriod;
                
                % cashflow columns initialize
                % 1st nonCall
                nonCall.npv = 0;
                nonCall.cashflow_e = zeros(periodCount + 1,1);
                nonCall.cashflow_npv_e = zeros(periodCount + 1,1);
                nonCall.rangeProb_e = zeros(periodCount + 1,1);
                nonCall.df_e = zeros(periodCount + 1,1);
                nonCall.df_c = zeros(periodCount + 1,1);

                % debug information, cashflow & npv per path
                nonCall.payoff = zeros(1,pathSize);
                nonCall.payoffStates.cashflow = zeros(periodCount+1,pathSize);
                nonCall.payoffStates.cashflow_npv = zeros(periodCount+1,pathSize);
                nonCall.payoffStates.df = zeros(periodCount + 1,pathSize);
                
                % intermediate variables
                payoffStateA.cashflow = zeros(1, pathSize);
                payoffStateA.cashflow_npv = zeros(1, pathSize);
                payoffStateA.df = zeros(1,pathSize);
                
                % we calculate cashflow backward from maturityDate to valueDate
                % rangeAccrual coupons are added at the startDate of the Period
                % At maturity

                nominal = security.nominal;
                coupon = security.coupon;
                callYN = security.callYN;
                
                tenorCMS = security.tenor;
                
                ralower = security.lower;
                raupper = security.upper;
                
                tenorLong = security.tenorL;
                tenorShort = security.tenorS;
                
                ralowerS = security.lowerS;
                raupperS = security.upperS;
                
                currentTime = numMethodInfo.totalTimeSteps(end);
                totalTimeSteps = numMethodInfo.timeStepsInfo.totalTimeSteps;
                totalTimeStepsSize = numMethodInfo.timeStepsInfo.totalTimeStepsSize;
                if totalTimeStepsSize == 0 || (totalTimeStepsSize == 1 && totalTimeSteps(1) == 0) 
                    out.nonCall = nonCall;
                    return;
                end
                
                currentTimeIdx = totalTimeStepsSize;
                
                currentNodeIdx = periodCount + 1;
                
%                 modelStateT = modelStates(numOfFactors*(totalTimeStepsSize-1)+1:numOfFactors*(totalTimeStepsSize-1)+2,:);
                payoffStateA.cashflow = nominal*ones(1,pathSize);
                %process payoff at maturity
                nonCall.payoffStates.cashflow(currentNodeIdx,:) = payoffStateA.cashflow;
                
%                 payoffStateA.cashflow_npv = model.discountPayoff(currentTime,numMatTime ...
%                     ,modelStateT,payoffStateA.cashflow);
%                 
                payoffStateA.cashflow_npv = model.DFRisky(currentTime/365.0)*payoffStateA.cashflow;
                
                nonCall.payoffStates.cashflow_npv(currentNodeIdx,:) = payoffStateA.cashflow_npv ;
                
%                 payoffStateA.df = model.discountPayoff(currentTime,numMatTime ...
%                     ,modelStateT,1.0*ones(1,pathSize));
                payoffStateA.df = model.DFRisky(currentTime/365.0) *ones(1,pathSize);
                    

                nonCall.payoffStates.df(currentNodeIdx,:) = payoffStateA.df;
                nonCall.payoff = nonCall.payoff + payoffStateA.cashflow_npv;
                
                nonCall.cashflow_e(currentNodeIdx) = mean(payoffStateA.cashflow);
                nonCall.cashflow_npv_e(currentNodeIdx) = mean(payoffStateA.cashflow_npv);
                nonCall.df_e(currentNodeIdx) = mean(payoffStateA.df);
                nonCall.df_c(currentNodeIdx) = model.DFRisky(currentTime/365.0);
                
%                 % callable at maturity is the same as noncallable
%                 callable.payoffStates.cashflow(currentNodeIdx,:) = nonCall.payoffStates.cashflow(currentNodeIdx,:);
%                 callable.payoffStates.cashflow_npv(currentNodeIdx,:) = nonCall.payoffStates.cashflow_npv(currentNodeIdx,:);
%                 callable.payoff = callable.payoff + callable.payoffStates.cashflow_npv(currentNodeIdx,:);
%                 
%                 callable.cashflow_e(currentNodeIdx) = nonCall.cashflow_e(currentNodeIdx);
%                 callable.cashflow_npv_e(currentNodeIdx) = nonCall.cashflow_npv_e(currentNodeIdx);
                  
                lastTimeIdx = currentTimeIdx;
                
                
                % flag for inclusive schedule
%                 simpleYN = false;
                simpleYN = 1;
                
                for i=periodCount:-1:1
                    % past schedule neglect
                    if DateDiff(valueDate,schedule.payDates(i)) >= 0, continue;end 
                    startTimeIdx = numMethodInfo.timeStepsInfo.startTimeIdx(i);
                    endTimeIdx = numMethodInfo.timeStepsInfo.endTimeIdx(i);

                    inclusiveYN = numMethodInfo.timeStepsInfo.inclusiveYN(i);
                     %% FromToInduction start

                    %modelsStates induct backward(lastTimeIdx to endTimeIdx)
                    while endTimeIdx < lastTimeIdx
                        lastTimeIdx = lastTimeIdx -1;
                        if lastTimeIdx == endTimeIdx break;end
                    end

                    currentTimeIdx = lastTimeIdx;
                    currentTime = totalTimeSteps(currentTimeIdx);
                    % now we are at endTimeIdx
                    % variables for coupon valuation initialized
%                     ralower = security.lower;
%                     raupper = security.upper;
%                     tenorLong = security.tenorL;
%                     tenorShort = security.tenorS;

                    cD = 0; % cumulated day count fraction
                    cP = 0; 
                    dI = 0; % dummy idx for loop
                    dcfC = 0;
                    dcfN = 0;
                    dD = 0;
                    idxCMS = zeros(1,pathSize);
                    
                    idxC = zeros(1,pathSize);
                    idxN = zeros(1,pathSize);
                    
                    idxL = zeros(1,pathSize);
                    idxS = zeros(1,pathSize);
                    spreadC = zeros(1,pathSize);
                    spreadN = zeros(1,pathSize);
                    probSpread = zeros(1,pathSize);
                    
                    numOfTimeSteps = numMethodInfo.timeStepsInfo.numOfTimeSteps(i);
                    
                    % if inclusive then we use simpleYN rangeProb
                    %if inclusive then we do not evaluate the last step but
                    % use fixing instead 
                    endTimeStep = 1;
                    if inclusiveYN
%                         simpleYN = true;
                        simpleYN = 1;
                        endTimeStep = 2;
                    end
                    
                    for j=numOfTimeSteps:-1:endTimeStep
                        % evaluate dealdescription
                        % last step is omitted
                        currentTime = totalTimeSteps(currentTimeIdx);
                        %dcfC = currentTime/365.0;
                        dcfC = currentTime;
                        
                        if j~= numOfTimeSteps
                            dD = (dcfN-dcfC);
                        end

                        toDate = AddDate(valueDate,currentTime,'day');
                        tpDate = schedule.payDates(i);
%                         spreadC = model.CMSCMTSpreadDigitalRangeDate(valueDate,toDate,tpDate,tenorLong,tenorShort,ralower,raupper);
                        spreadC = model.CMSCMSSpreadDigitalRangeDate(valueDate,toDate,tpDate,tenorCMS,tenorLong,tenorShort,0.25,ralower,raupper,ralowerS,raupperS);
                        
                        %modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:numOfFactors*(currentTimeIdx-1)+2,:);
%                         idxL = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorLong,modelStateT);
%                         idxS = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorShort,modelStateT);
%                         spreadC = idxL - idxS;
%                         probSpread = model.SRPr(spreadC,spreadN,ralower,raupper,simpleYN,pathSize)*dD; 

                        if simpleYN
                            probSpread = spreadC*dD;
                        else
                            probSpread = spreadC*dD;
%                             probSpread = 0.5*(spreadC + spreadN)*dD;
                            
                        end

                        
                        cD = cD + dD;
                        cP = cP + probSpread;

                        dcfN = dcfC;
                        spreadN = spreadC;

                        %induct backward to the next time step
                        % one time backward induction
                        lastTimeIdx = currentTimeIdx;

                        if startTimeIdx < lastTimeIdx
                            lastTimeIdx = lastTimeIdx -1;
                            currentTimeIdx = lastTimeIdx;
                        end


                    end

                    %% FromToInduction end
                    %% AtDate evaluation
                    % Adding a new cashflow to the payoff

                    dcf = schedule.dayCountFraction(i);
                    
                    if inclusiveYN
                        % we add valuation date
                        cD = schedule.dayCount(i);
                        rangeDays =  security.rangeDays(i);
                        payoffStateA.cashflow = nominal * coupon * dcf * (rangeDays + cP)/cD;
                    else
                        payoffStateA.cashflow = nominal * coupon * dcf * cP/cD;
                    end
                    % eventDate(i) = startDate(i) therefore there is no induction in modelStates
                    % i.e. currentTimeIdx has not changed
                    % no need to update modelStateT
                    % modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:end,:);
                    % since cashflow's are added at the startDate(i) we need to
                    % multiply cashflow with the df(startDate(i), payDate(i))
                    payTime = DateDiff(schedule.payDates(i),valueDate);
                    
                    %event is at startDate so we need to discount the
                    %cashflow first from paydate to startdate
%                     df_sp = model.discountFactor(currentTime, payTime,numMatTime,modelStateT);

%                     for k=1:pathSize
%                         payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_sp(k);
%                     end
                    
                    % process payoff
                    
                    currentNodeIdx = i;
                    
                    % 1st nonCall column
                    nonCall.payoffStates.cashflow(currentNodeIdx,:) = payoffStateA.cashflow;
                    payoffStateA.cashflow_npv = payoffStateA.cashflow * model.DFRisky(payTime/365.0);
                    
%                     payoffStateA.cashflow_npv = model.discountPayoff(currentTime,numMatTime ...
%                     ,modelStateT,payoffStateA.cashflow);
                    nonCall.payoffStates.cashflow_npv(currentNodeIdx,:) = payoffStateA.cashflow_npv;
                    payoffStateA.df = model.DFRisky(payTime/365.0);
%                     payoffStateA.df = model.discountPayoff(currentTime,numMatTime ...
%                     ,modelStateT,1.0*ones(1,pathSize));
                    nonCall.payoffStates.df(currentNodeIdx,:) = payoffStateA.df;
                    
                    nonCall.payoff = nonCall.payoff + payoffStateA.cashflow_npv;
                    
                    nonCall.cashflow_e(currentNodeIdx) = mean(payoffStateA.cashflow);
                    nonCall.rangeProb_e(currentNodeIdx) = mean(nonCall.cashflow_e(currentNodeIdx)/(nominal * coupon * dcf));
                    nonCall.cashflow_npv_e(currentNodeIdx) = mean(payoffStateA.cashflow_npv);

                    nonCall.df_e(currentNodeIdx) = mean(payoffStateA.df);
                    nonCall.df_c(currentNodeIdx) = model.DFRisky(currentTime/365.0);
                    

                end

                nonCall.npv = mean(nonCall.payoff);
                out.pricerInfo.nonCall = nonCall;
                
%                 callable.npv = mean(callable.payoff);
%                 out.pricerInfo.callable = callable;
                out.daycountFraction = schedule.dayCountFraction;
                
        end
        

        
        
        
        
    end
    
end



In [ ]:
classdef DailyWeeklyRiskyPricer_Swaption < H_RootObject
    %Pricer(Security,Model,NumMethod)
    %Prcer is constructed from Security, Model, NumMethod 
    
    properties
        m_security
        m_model
        m_numMethod
        overHedgeYN
    end
    
    methods
        function pricer = DailyWeeklyRiskyPricer_Swaption(security,model,numMethod,overHedgeYN)
                pricer.m_security = security;
                pricer.m_model = model;
                pricer.m_numMethod = numMethod;
                pricer.overHedgeYN = overHedgeYN;
        end
        
        function out = computePriceMCCallableGeneric(pricer,valueDate)
                security = pricer.m_security;
                numMethod = pricer.m_numMethod;
                model = pricer.m_model;
                schedule = security.couponSchedule;
                callSchedule = security.callSchedule;
                
                % numMethodTimeSteps Generate
                mcOneTimeStep = numMethod.mcOneTimeStep;
                outInfo = numMethod.numMethodTimeStepsGenerateDW(valueDate,schedule,mcOneTimeStep);
            
                numMethodInfo.timeStepsInfo = outInfo;

                % mc random number init
                %pathSize = 2^16;
                pathSize = numMethod.pathSize;

                % generate random numbers and modelstates forward
                rng('default');
                rng(0);

                % LGM1F : one factor 
                numOfFactors = 1;

                %Z= randn(numOfFactors*outInfo.totalTimeStepsSize,pathSize/2);
                Z= randn(pathSize/2,numOfFactors*outInfo.totalTimeStepsSize);
                Z=[Z;-Z];
                Z=Z';
                
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;

                % we check with the c code, so we use generated random numbers
%                 mc_random = importdata('mc_random.xlsx');
%                 mc_random = importdata('D:\mc_random_number_new_report.txt');
%                 Z= mc_random;
                
                numMethodInfo.pathSize = pathSize;
                numMethodInfo.numOfFactors = numOfFactors;
                numMethodInfo.totalTimeSteps = outInfo.totalTimeSteps;
                numMethodInfo.totalTimeStepsSize = outInfo.totalTimeStepsSize;
                numMethodInfo.Z = Z;



                %generate model states for the given timeSteps
                modelStates = numMethod.generateModelStates1(valueDate,schedule,numMethodInfo,model);

                % numeraire info
                numMatTime = numMethodInfo.totalTimeSteps(end);
                days = DateDiff(schedule.payDates(schedule.numOfPeriod),valueDate);
                numMatTime = max(numMatTime, days);
                dfNumMat = model.DFRisky(numMatTime/365.0);
                
                numeraire.numMatTime = numMatTime;
                numeraire.dfNumMat = dfNumMat;
                
                % induct backward and calculate cashflow
            
                periodCount = schedule.numOfPeriod;
                
                % pricer single info initialize
                
                nonCall = PricerInfo(periodCount +1, pathSize);
                callable = PricerInfo(periodCount +1, pathSize);
                dfCol = PricerInfo(periodCount +1, pathSize);
                
%                 dummy1 = PricerInfo(periodCount +1, pathSize);
%                 dummy2 = PricerInfo(periodCount +1, pathSize);
                
                
                % intermediate variables
                payoffStateA.cashflow = zeros(1, pathSize);
                payoffStateA.cashflow_npv = zeros(1, pathSize);
                
            
                % we calculate cashflow backward from maturityDate to valueDate
                % rangeAccrual coupons are added at the startDate of the Period
                % At maturity

                nominal = security.nominal;
                coupon = security.coupon;
                callYN = security.callYN;
                
                ralower = security.lower;
                raupper = security.upper;
                
                tenorCD = security.tenor;
                
                currentTime = numMethodInfo.totalTimeSteps(end);
                totalTimeStepsSize = numMethodInfo.totalTimeStepsSize;
                currentTimeIdx = totalTimeStepsSize;
                
                currentNodeIdx = periodCount + 1;
                
                modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1,:);
                payoffStateA.cashflow = nominal*ones(1,pathSize);
                
                payTime = DateDiff(schedule.payDates(schedule.numOfPeriod),valueDate);
                df_ep = model.discountFactorRisky(currentTime, payTime,numMatTime,modelStateT);
                for k=1:pathSize
                    payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                end
                
                %process payoff at maturity
                nonCall.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                
                callable.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                
                %df column
                payoffStateA.cashflow = ones(1,pathSize);
                for k=1:pathSize
                    payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                end
                
                dfCol.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                                 
%                 %dummy column
%                 payoffStateA.cashflow = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorLong,modelStateT);
%                 dummy1.ProcessPayoff(model,currentTime,numeraire, ...
%                                  modelStateT,payoffStateA.cashflow,currentNodeIdx);
% 
%                 payoffStateA.cashflow = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorShort,modelStateT);
%                 dummy2.ProcessPayoff(model,currentTime,numeraire, ...
%                                  modelStateT,payoffStateA.cashflow,currentNodeIdx);
                                 
                  

                lastTimeIdx = currentTimeIdx;
                totalTimeSteps = numMethodInfo.timeStepsInfo.totalTimeSteps;
                
                % flag for inclusive schedule
%                 simpleYN = false;
%                 simpleYN = true;
                if pricer.overHedgeYN == true 
                    simpleYN = 2;
                else
                    simpleYN = 1;
                end
                
                for i=periodCount:-1:1
                    % past schedule neglect
                    if DateDiff(valueDate,schedule.payDates(i)) >= 0, continue;end 
                    startTimeIdx = numMethodInfo.timeStepsInfo.startTimeIdx(i);
                    endTimeIdx = numMethodInfo.timeStepsInfo.endTimeIdx(i);

                    inclusiveYN = numMethodInfo.timeStepsInfo.inclusiveYN(i);
                 %% FromToInduction start

                    %modelsStates induct backward(lastTimeIdx to endTimeIdx)
                    while endTimeIdx < lastTimeIdx
                        lastTimeIdx = lastTimeIdx -1;
                        if lastTimeIdx == endTimeIdx break;end
                    end

                    currentTimeIdx = lastTimeIdx;
                    currentTime = totalTimeSteps(currentTimeIdx);
                    % now we are at endTimeIdx
                    % variables for coupon valuation initialized
                    

                    cD = 0; % cumulated day count fraction
                    cP = 0; 
                    dI = 0; % dummy idx for loop
                    dcfC = 0;
                    dcfN = 0;
                    dD = 0;
                    idxCD = zeros(1,pathSize);
                    
                    idxC = zeros(1,pathSize);
                    idxN = zeros(1,pathSize);
                
                    probSpread = zeros(1,pathSize);
                    
                    numOfTimeSteps = numMethodInfo.timeStepsInfo.numOfTimeSteps(i);
                    
                    % if inclusive then we use simpleYN rangeProb
                    %if inclusive then we do not evaluate the last step but
                    % use fixing instead 
                    endTimeStep = 1;
                    if inclusiveYN
%                         simpleYN = true;
                        simpleYN = 1;
                        endTimeStep = 2;
                        if pricer.overHedgeYN == true 
                            simpleYN = 2;
                        end
                    end
                    
                    for j=numOfTimeSteps:-1:endTimeStep
                        % evaluate dealdescription
                        % last step is omitted
                        currentTime = totalTimeSteps(currentTimeIdx);
                        %dcfC = currentTime/365.0;
                        dcfC = currentTime;
                        
                        if j~= numOfTimeSteps
                            dD = (dcfN-dcfC);
                        end

                        modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1,:);
                        
                        idxCD = model.Libor_SimpleDate(valueDate,currentTime,currentTime,numMatTime, tenorCD,modelStateT);
                        idxC = idxCD;
                        probSpread = model.SRPr(idxC,idxN,ralower,raupper,simpleYN,pathSize)*dD;
                    
                        cD = cD + dD;
                        cP = cP + probSpread;

                        dcfN = dcfC;
                        
                        idxN = idxC;
                        
                        %induct backward to the next time step
                        % one time backward induction
                        lastTimeIdx = currentTimeIdx;

                        if startTimeIdx < lastTimeIdx
                            lastTimeIdx = lastTimeIdx -1;
                            currentTimeIdx = lastTimeIdx;
                        end


                    end

                    %% FromToInduction end
        % if the current period is inclusive, then the start date is not evaluated in the above for loop.
		% instead the fixing will be treated separately below.
		% however current_time_idx is updated at the end of the for loop.
		% so we need to update model_states accordingly.
		% if model_states were updated previously then the following additionaly step doesn't affect the result anyway.
                    currentTime = totalTimeSteps(currentTimeIdx);
                    modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1,:);
                    
                    %% AtDate evaluation
                    % Adding a new cashflow to the payoff

                    dcf = schedule.dayCountFraction(i);
                    
                    if inclusiveYN
                        % we add valuation date
                        cD = schedule.dayCount(i);
                        rangeDays =  security.rangeDays(i);
                        payoffStateA.cashflow = nominal * coupon * dcf * (rangeDays(i) + cP)/cD;
                    else
                        payoffStateA.cashflow = nominal * coupon * dcf * cP/cD;
                    end
                    % eventDate(i) = startDate(i) therefore there is no induction in modelStates
                    % i.e. currentTimeIdx has not changed
                    % no need to update modelStateT
                    % modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:end,:);
                    % since cashflow's are added at the startDate(i) we need to
                    % multiply cashflow with the df(startDate(i), payDate(i))
                    payTime = DateDiff(schedule.payDates(i),valueDate);
                    
                    %event is at startDate so we need to discount the
                    %cashflow first from paydate to startdate
                    df_ep = model.discountFactorRisky(currentTime, payTime,numMatTime,modelStateT);

                    for k=1:pathSize
                        payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                    end
                    
                    % process payoff
                    
                    currentNodeIdx = i;
                    
                    nonCall.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                
                    callable.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                    
                    %df column
                    payoffStateA.cashflow = ones(1,pathSize);
                    for k=1:pathSize
                        payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                    end
                    
                    dfCol.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                                 
%                     %dummy column
%                     payoffStateA.cashflow = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorLong,modelStateT);
%                     dummy1.ProcessPayoff(model,currentTime,numeraire, ...
%                                      modelStateT,payoffStateA.cashflow,currentNodeIdx);
%                     
%                     payoffStateA.cashflow = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorShort,modelStateT);
%                     dummy2.ProcessPayoff(model,currentTime,numeraire, ...
%                                      modelStateT,payoffStateA.cashflow,currentNodeIdx);
                    
                    
                    % if exercise is possible
                    if i == 1
                        dummaaa = 1;
                    end
                    
                    % call event happens  at the start of the period
                    % at call date, by default, call option is expired
                    
                    if callYN(i) && (DateDiff(valueDate,callSchedule.resetDates(i)) < 0)
                        % 1.undiscount callable.payoff
                        % 2.build the state variables matrix
                        % 3.regress 1's with the given state variables
                        % (model states 1,2)
                        % 4. exercise per each path in
                        % discount the new payoff and assign into callable
                        % column
                        
                        %1 undiscount callable.payoff and get the
                        %contiValue
                       
                        %exercise value
                        exerciseValue = security.nominal*ones(1,pathSize);
                        payTime = DateDiff(callSchedule.payDates(i),valueDate);
                        %event is at startDate so we need to discount the
                        %cashflow first from paydate to startdate
                        df_ep = model.discountFactorRisky(currentTime, payTime,numMatTime,modelStateT);
                        for k=1:pathSize
                            exerciseValue(k) = exerciseValue(k) * df_ep(k);
                        end
                        
                        contiValue = zeros(1,pathSize);
                        onePayoff = ones(1,pathSize);
                        onePayoffD = model.discountPayoffRisky(currentTime,numMatTime ...
                        ,modelStateT,onePayoff);
                        contiValue = callable.payoff./onePayoffD;
                        
                        %Nominal Regression test
                        contiValue = contiValue/10000;
                        
                        % build the state variabls matrix
                        decisionVar1 = modelStateT(1,:);
%                         decisionVar2 = modelStateT(2,:);
                        % we use upto 2nd order polynomial as regression
                        % equatioin
                        dimensionLS = 3;
                        regMatrix = ones(pathSize,dimensionLS);
                        
                        for idx_r=1:pathSize
                            regMatrix(idx_r,2) = decisionVar1(idx_r);
                            regMatrix(idx_r,3) = decisionVar1(idx_r)*decisionVar1(idx_r);
                        end
                        
                        % simple regression <- for path dependent option
                        % we should change it
                        % beta  : regression coefficients
                        beta =zeros(dimensionLS,1);
                        beta = regress(contiValue',regMatrix);
                        regContiValue = regMatrix*beta;
                       
                         %Nominal Regression test
                        regContiValue = regContiValue * 10000;
                        
                        exercisedPayoff = regContiValue;
                        % if regContiValue is bigger than exerciseValue
                        % then issue exercise call option
                        for idx_r=1:pathSize
                            if regContiValue(idx_r) >= exerciseValue(idx_r) 
                                exercisedPayoff(idx_r) = exerciseValue(idx_r);
                            end
                        end
                        
                        % we discount the new payoff and 
                        % assign it into the existing callable payoff
                        callable.payoff = model.discountPayoffRisky(currentTime,numMatTime ...
                        ,modelStateT,exercisedPayoff);
                        
                    end
                end

                nonCall.npv = mean(nonCall.payoff);
                out.nonCall = nonCall;
                callable.npv = mean(callable.payoff);
                out.callable = callable;
                out.dfCol = dfCol;
%                 out.dummy1 = dummy1;
%                 out.dummy2 = dummy2;
                out.daycountFraction = schedule.dayCountFraction;
                
        end
        
        function out = computePriceMCCallable(pricer,valueDate,PSAFlag)
                security = pricer.m_security;
                numMethod = pricer.m_numMethod;
                model = pricer.m_model;
                schedule = security.schedule;
                
                % numMethodTimeSteps Generate
                mcOneTimeStep = numMethod.mcOneTimeStep;
                outInfo = numMethod.numMethodTimeStepsGenerateDW(valueDate,schedule,mcOneTimeStep);
            
                numMethodInfo.timeStepsInfo = outInfo;

                % mc random number init
                %pathSize = 2^16;
                pathSize = numMethod.pathSize;

                % generate random numbers and modelstates forward
                rng('default');
                rng(0);

                % LGM1F : one factor 
                numOfFactors = 1;

                %Z= randn(numOfFactors*outInfo.totalTimeStepsSize,pathSize/2);
                Z= randn(pathSize/2,numOfFactors*outInfo.totalTimeStepsSize);
                Z=[Z;-Z];
                Z=Z';
                
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;

                % we check with the c code, so we use generated random numbers
%                 mc_random = importdata('mc_random.xlsx');
%                 mc_random = importdata('D:\mc_random_number_new_report.txt');
%                 Z= mc_random;
                
                numMethodInfo.pathSize = pathSize;
                numMethodInfo.numOfFactors = numOfFactors;
                numMethodInfo.totalTimeSteps = outInfo.totalTimeSteps;
                numMethodInfo.totalTimeStepsSize = outInfo.totalTimeStepsSize;
                numMethodInfo.Z = Z;



                %generate model states for the given timeSteps
                modelStates = numMethod.generateModelStates1(valueDate,schedule,numMethodInfo,model);

                % numeraire info
                numMatTime = numMethodInfo.totalTimeSteps(end);
                days = DateDiff(schedule.payDates(schedule.numOfPeriod),valueDate);
                numMatTime = max(numMatTime, days);
                dfNumMat = model.DFRisky(numMatTime/365.0);
                
                numeraire.numMatTime = numMatTime;
                numeraire.dfNumMat = dfNumMat;
                
                % induct backward and calculate cashflow
            
                periodCount = schedule.numOfPeriod;
                
                % pricer single info initialize
                
                nonCall = PricerInfo(periodCount +1, pathSize);
                callable = PricerInfo(periodCount +1, pathSize);
                dfCol = PricerInfo(periodCount +1, pathSize);
                
%                 dummy1 = PricerInfo(periodCount +1, pathSize);
%                 dummy2 = PricerInfo(periodCount +1, pathSize);
                
                
                % intermediate variables
                payoffStateA.cashflow = zeros(1, pathSize);
                payoffStateA.cashflow_npv = zeros(1, pathSize);
                
            
                % we calculate cashflow backward from maturityDate to valueDate
                % rangeAccrual coupons are added at the startDate of the Period
                % At maturity

                nominal = security.nominal;
                
                strike = security.strike;
                annuityIn = security.annuityIn;
                
                tenorLong = security.tenor;
                
                currentTime = numMethodInfo.totalTimeSteps(end);
                totalTimeStepsSize = numMethodInfo.totalTimeStepsSize;
                currentTimeIdx = totalTimeStepsSize;
                
                currentNodeIdx = periodCount + 1;
                
                modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1,:);
%                 payoffStateA.cashflow = nominal*ones(1,pathSize);
                if strcmp(PSAFlag,'PSA')
                    idxLong = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorLong,modelStateT);
                elseif strcmp(PSAFlag,'PSAFallback')
                    idxLong = model.CMS_PSA_SimpleDate_Fallback(valueDate,currentTime,numMatTime, tenorLong,modelStateT);
                elseif strcmp(PSAFlag,'PSAFallbackSimple')
                    idxLong = model.CMS_PSA_SimpleDate_FallbackSimple(valueDate,currentTime,numMatTime, tenorLong,modelStateT);
                else
                    error('not implemented')
                end
                for k=1:pathSize
                    payoffStateA.cashflow(k) = annuityIn * max(idxLong(k)-strike,0.0);
                end
                
                payTime = DateDiff(schedule.payDates(schedule.numOfPeriod),valueDate);
                df_ep = model.discountFactorRisky(currentTime, payTime,numMatTime,modelStateT);
                for k=1:pathSize
                    payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                end
                
                %process payoff at maturity
                nonCall.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                
%                 callable.ProcessPayoff(model,currentTime,numeraire, ...
%                                      modelStateT,payoffStateA.cashflow,currentNodeIdx);
%                 
%                 %df column
%                 payoffStateA.cashflow = ones(1,pathSize);
%                 for k=1:pathSize
%                     payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
%                 end
%                 
%                 dfCol.ProcessPayoff(model,currentTime,numeraire, ...
%                                      modelStateT,payoffStateA.cashflow,currentNodeIdx);
                                 
%                 %dummy column
%                 payoffStateA.cashflow = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorLong,modelStateT);
%                 dummy1.ProcessPayoff(model,currentTime,numeraire, ...
%                                  modelStateT,payoffStateA.cashflow,currentNodeIdx);
% 
%                 payoffStateA.cashflow = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorShort,modelStateT);
%                 dummy2.ProcessPayoff(model,currentTime,numeraire, ...
%                                  modelStateT,payoffStateA.cashflow,currentNodeIdx);
                                 
                  

                lastTimeIdx = currentTimeIdx;
                totalTimeSteps = numMethodInfo.timeStepsInfo.totalTimeSteps;
                
                % flag for inclusive schedule
%                 simpleYN = false;
%                 simpleYN = true;
%                 if pricer.overHedgeYN == true 
%                     simpleYN = 2;
%                 else
%                     simpleYN = 1;
%                 end
%                 
%                 for i=periodCount:-1:1
%                     % past schedule neglect
%                     if DateDiff(valueDate,schedule.payDates(i)) >= 0, continue;end 
%                     startTimeIdx = numMethodInfo.timeStepsInfo.startTimeIdx(i);
%                     endTimeIdx = numMethodInfo.timeStepsInfo.endTimeIdx(i);
% 
%                     inclusiveYN = numMethodInfo.timeStepsInfo.inclusiveYN(i);
%                  %% FromToInduction start
% 
%                     %modelsStates induct backward(lastTimeIdx to endTimeIdx)
%                     while endTimeIdx < lastTimeIdx
%                         lastTimeIdx = lastTimeIdx -1;
%                         if lastTimeIdx == endTimeIdx break;end
%                     end
% 
%                     currentTimeIdx = lastTimeIdx;
%                     currentTime = totalTimeSteps(currentTimeIdx);
%                     % now we are at endTimeIdx
%                     % variables for coupon valuation initialized
%                     
% 
%                     cD = 0; % cumulated day count fraction
%                     cP = 0; 
%                     dI = 0; % dummy idx for loop
%                     dcfC = 0;
%                     dcfN = 0;
%                     dD = 0;
%                     idxCD = zeros(1,pathSize);
%                     
%                     idxC = zeros(1,pathSize);
%                     idxN = zeros(1,pathSize);
%                 
%                     probSpread = zeros(1,pathSize);
%                     
%                     numOfTimeSteps = numMethodInfo.timeStepsInfo.numOfTimeSteps(i);
%                     
%                     % if inclusive then we use simpleYN rangeProb
%                     %if inclusive then we do not evaluate the last step but
%                     % use fixing instead 
%                     endTimeStep = 1;
%                     if inclusiveYN
% %                         simpleYN = true;
%                         simpleYN = 1;
%                         endTimeStep = 2;
%                         if pricer.overHedgeYN == true 
%                             simpleYN = 2;
%                         end
%                     end
%                     
%                     for j=numOfTimeSteps:-1:endTimeStep
%                         % evaluate dealdescription
%                         % last step is omitted
%                         currentTime = totalTimeSteps(currentTimeIdx);
%                         %dcfC = currentTime/365.0;
%                         dcfC = currentTime;
%                         
%                         if j~= numOfTimeSteps
%                             dD = (dcfN-dcfC);
%                         end
% 
%                         modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1,:);
%                         
%                         idxCD = model.Libor_SimpleDate(valueDate,currentTime,currentTime,numMatTime, tenorCD,modelStateT);
%                         idxC = idxCD;
%                         probSpread = model.SRPr(idxC,idxN,ralower,raupper,simpleYN,pathSize)*dD;
%                     
%                         cD = cD + dD;
%                         cP = cP + probSpread;
% 
%                         dcfN = dcfC;
%                         
%                         idxN = idxC;
%                         
%                         %induct backward to the next time step
%                         % one time backward induction
%                         lastTimeIdx = currentTimeIdx;
% 
%                         if startTimeIdx < lastTimeIdx
%                             lastTimeIdx = lastTimeIdx -1;
%                             currentTimeIdx = lastTimeIdx;
%                         end
% 
% 
%                     end
% 
%                     %% FromToInduction end
%         % if the current period is inclusive, then the start date is not evaluated in the above for loop.
% 		% instead the fixing will be treated separately below.
% 		% however current_time_idx is updated at the end of the for loop.
% 		% so we need to update model_states accordingly.
% 		% if model_states were updated previously then the following additionaly step doesn't affect the result anyway.
%                     currentTime = totalTimeSteps(currentTimeIdx);
%                     modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1,:);
%                     
%                     %% AtDate evaluation
%                     % Adding a new cashflow to the payoff
% 
%                     dcf = schedule.dayCountFraction(i);
%                     
%                     if inclusiveYN
%                         % we add valuation date
%                         cD = schedule.dayCount(i);
%                         rangeDays =  security.rangeDays(i);
%                         payoffStateA.cashflow = nominal * coupon * dcf * (rangeDays + cP)/cD;
%                     else
%                         payoffStateA.cashflow = nominal * coupon * dcf * cP/cD;
%                     end
%                     % eventDate(i) = startDate(i) therefore there is no induction in modelStates
%                     % i.e. currentTimeIdx has not changed
%                     % no need to update modelStateT
%                     % modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:end,:);
%                     % since cashflow's are added at the startDate(i) we need to
%                     % multiply cashflow with the df(startDate(i), payDate(i))
%                     payTime = DateDiff(schedule.payDates(i),valueDate);
%                     
%                     %event is at startDate so we need to discount the
%                     %cashflow first from paydate to startdate
%                     df_ep = model.discountFactorRisky(currentTime, payTime,numMatTime,modelStateT);
% 
%                     for k=1:pathSize
%                         payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
%                     end
%                     
%                     % process payoff
%                     
%                     currentNodeIdx = i;
%                     
%                     nonCall.ProcessPayoff(model,currentTime,numeraire, ...
%                                      modelStateT,payoffStateA.cashflow,currentNodeIdx);
%                 
%                     callable.ProcessPayoff(model,currentTime,numeraire, ...
%                                      modelStateT,payoffStateA.cashflow,currentNodeIdx);
%                     
%                     %df column
%                     payoffStateA.cashflow = ones(1,pathSize);
%                     for k=1:pathSize
%                         payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
%                     end
%                     
%                     dfCol.ProcessPayoff(model,currentTime,numeraire, ...
%                                      modelStateT,payoffStateA.cashflow,currentNodeIdx);
%                                  
% %                     %dummy column
% %                     payoffStateA.cashflow = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorLong,modelStateT);
% %                     dummy1.ProcessPayoff(model,currentTime,numeraire, ...
% %                                      modelStateT,payoffStateA.cashflow,currentNodeIdx);
% %                     
% %                     payoffStateA.cashflow = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorShort,modelStateT);
% %                     dummy2.ProcessPayoff(model,currentTime,numeraire, ...
% %                                      modelStateT,payoffStateA.cashflow,currentNodeIdx);
%                     
%                     
%                     % if exercise is possible
%                     if i == 1
%                         dummaaa = 1;
%                     end
%                     
%                     % call event happens  at the start of the period
%                     % at call date, by default, call option is expired
%                     
%                     if callYN(i) && (DateDiff(valueDate,callSchedule.resetDates(i)) < 0)
%                         % 1.undiscount callable.payoff
%                         % 2.build the state variables matrix
%                         % 3.regress 1's with the given state variables
%                         % (model states 1,2)
%                         % 4. exercise per each path in
%                         % discount the new payoff and assign into callable
%                         % column
%                         
%                         %1 undiscount callable.payoff and get the
%                         %contiValue
%                        
%                         %exercise value
% %                         exerciseValue = security.nominal*ones(1,pathSize);
%                         exerciseValue = security.nominal*security.callStrike*ones(1,pathSize);
%                         
%                         payTime = DateDiff(callSchedule.payDates(i),valueDate);
%                         %event is at startDate so we need to discount the
%                         %cashflow first from paydate to startdate
%                         df_ep = model.discountFactorRisky(currentTime, payTime,numMatTime,modelStateT);
%                         for k=1:pathSize
%                             exerciseValue(k) = exerciseValue(k) * df_ep(k);
%                         end
%                         
%                         contiValue = zeros(1,pathSize);
%                         onePayoff = ones(1,pathSize);
%                         onePayoffD = model.discountPayoffRisky(currentTime,numMatTime ...
%                         ,modelStateT,onePayoff);
%                         contiValue = callable.payoff./onePayoffD;
%                         
%                         %Nominal Regression test
%                         contiValue = contiValue/10000;
%                         
%                         % build the state variabls matrix
%                         decisionVar1 = modelStateT(1,:);
% %                         decisionVar2 = modelStateT(2,:);
%                         % we use upto 2nd order polynomial as regression
%                         % equatioin
%                         dimensionLS = 3;
%                         regMatrix = ones(pathSize,dimensionLS);
%                         
%                         for idx_r=1:pathSize
%                             regMatrix(idx_r,2) = decisionVar1(idx_r);
%                             regMatrix(idx_r,3) = decisionVar1(idx_r)*decisionVar1(idx_r);
%                         end
%                         
%                         % simple regression <- for path dependent option
%                         % we should change it
%                         % beta  : regression coefficients
%                         beta =zeros(dimensionLS,1);
%                         beta = regress(contiValue',regMatrix);
%                         regContiValue = regMatrix*beta;
%                        
%                          %Nominal Regression test
%                         regContiValue = regContiValue * 10000;
%                         
%                         exercisedPayoff = regContiValue;
%                         % if regContiValue is bigger than exerciseValue
%                         % then issue exercise call option
%                         for idx_r=1:pathSize
%                             if regContiValue(idx_r) >= exerciseValue(idx_r) 
%                                 exercisedPayoff(idx_r) = exerciseValue(idx_r);
%                             end
%                         end
%                         
%                         % we discount the new payoff and 
%                         % assign it into the existing callable payoff
%                         callable.payoff = model.discountPayoffRisky(currentTime,numMatTime ...
%                         ,modelStateT,exercisedPayoff);
%                         
%                     end
%                 end

                nonCall.npv = mean(nonCall.payoff);
                out.nonCall = nonCall;
%                 callable.npv = mean(callable.payoff);
%                 out.callable = callable;
%                 out.dfCol = dfCol;
% %                 out.dummy1 = dummy1;
% %                 out.dummy2 = dummy2;
                out.daycountFraction = schedule.dayCountFraction;
                
        end
        
        function out = computePriceClosedSimple(pricer,valueDate)
                security = pricer.m_security;
                numMethod = pricer.m_numMethod;
                model = pricer.m_model;
                schedule = security.couponSchedule;
                
                % numMethodTimeSteps Generate
                mcOneTimeStep = numMethod.mcOneTimeStep;
                outInfo = numMethod.numMethodTimeStepsGenerateDW(valueDate,schedule,mcOneTimeStep);
            
                numMethodInfo.timeStepsInfo = outInfo;

                % mc random number init
                %pathSize = 2^16;
                pathSize = numMethod.pathSize;
                
                pathSize = 1;
                
%                 % generate random numbers and modelstates forward
%                 rng('default');
%                 rng(0);
% 
                numOfFactors = 1;
% 
%                 %Z= randn(numOfFactors*outInfo.totalTimeStepsSize,pathSize/2);
%                 Z= randn(pathSize/2,numOfFactors*outInfo.totalTimeStepsSize);
%                 Z=[Z;-Z];
%                 Z=Z';
%                 
%                 
%                 %TimeInversion
%                 sizeZ = size(Z,1);
%                 A = zeros(sizeZ,sizeZ);
%                 for i=1:sizeZ
%                     A(i,sizeZ + 1 -i) = 1;
%                 end
%                 Z = A*Z;
% 
%                 % we check with the c, so we use generated random numbers
% %                 mc_random = importdata('mc_random.xlsx');
% %                 mc_random = importdata('D:\mc_random_number_new_report.txt');
% %                 Z= mc_random;
%                 
                numMethodInfo.pathSize = pathSize;
                numMethodInfo.numOfFactors = numOfFactors;
                numMethodInfo.totalTimeSteps = outInfo.totalTimeSteps;
                numMethodInfo.totalTimeStepsSize = outInfo.totalTimeStepsSize;
%                 numMethodInfo.Z = Z;
% 
% 
% 
%                 %generate model states for the given timeSteps
%                 modelStates = numMethod.generateModelStates(valueDate,schedule,numMethodInfo,model);

                % numeraire info
                numMatTime = numMethodInfo.totalTimeSteps(end);
                dfNumMat = model.DF(numMatTime/365.0);
                
                % induct backward and calculate cashflow
            
                periodCount = schedule.numOfPeriod;
                
                % cashflow columns initialize
                % 1st nonCall
                nonCall.npv = 0;
                nonCall.cashflow_e = zeros(periodCount + 1,1);
                nonCall.cashflow_npv_e = zeros(periodCount + 1,1);
                nonCall.rangeProb_e = zeros(periodCount + 1,1);
                nonCall.df_e = zeros(periodCount + 1,1);
                nonCall.df_c = zeros(periodCount + 1,1);

                % debug information, cashflow & npv per path
                nonCall.payoff = zeros(1,pathSize);
                nonCall.payoffStates.cashflow = zeros(periodCount+1,pathSize);
                nonCall.payoffStates.cashflow_npv = zeros(periodCount+1,pathSize);
                nonCall.payoffStates.df = zeros(periodCount + 1,pathSize);
                
                % intermediate variables
                payoffStateA.cashflow = zeros(1, pathSize);
                payoffStateA.cashflow_npv = zeros(1, pathSize);
                payoffStateA.df = zeros(1,pathSize);
                
                % we calculate cashflow backward from maturityDate to valueDate
                % rangeAccrual coupons are added at the startDate of the Period
                % At maturity

                nominal = security.nominal;
                coupon = security.coupon;
                callYN = security.callYN;
                
                ralower = security.lower;
                raupper = security.upper;
                
                tenorCD = security.tenor;
                
                currentTime = numMethodInfo.totalTimeSteps(end);
                totalTimeSteps = numMethodInfo.timeStepsInfo.totalTimeSteps;
                totalTimeStepsSize = numMethodInfo.timeStepsInfo.totalTimeStepsSize;
                if totalTimeStepsSize == 0 || (totalTimeStepsSize == 1 && totalTimeSteps(1) == 0) 
                    out.nonCall = nonCall;
                    return;
                end
                
                currentTimeIdx = totalTimeStepsSize;
                
                currentNodeIdx = periodCount + 1;
                
%                 modelStateT = modelStates(numOfFactors*(totalTimeStepsSize-1)+1:numOfFactors*(totalTimeStepsSize-1)+2,:);
                payoffStateA.cashflow = nominal*ones(1,pathSize);
                %process payoff at maturity
                nonCall.payoffStates.cashflow(currentNodeIdx,:) = payoffStateA.cashflow;
                
%                 payoffStateA.cashflow_npv = model.discountPayoff(currentTime,numMatTime ...
%                     ,modelStateT,payoffStateA.cashflow);
%                 
                payoffStateA.cashflow_npv = model.DFRisky(currentTime/365.0)*payoffStateA.cashflow;
                
                nonCall.payoffStates.cashflow_npv(currentNodeIdx,:) = payoffStateA.cashflow_npv ;
                
%                 payoffStateA.df = model.discountPayoff(currentTime,numMatTime ...
%                     ,modelStateT,1.0*ones(1,pathSize));
                payoffStateA.df = model.DFRisky(currentTime/365.0) *ones(1,pathSize);
                    

                nonCall.payoffStates.df(currentNodeIdx,:) = payoffStateA.df;
                nonCall.payoff = nonCall.payoff + payoffStateA.cashflow_npv;
                
                nonCall.cashflow_e(currentNodeIdx) = mean(payoffStateA.cashflow);
                nonCall.cashflow_npv_e(currentNodeIdx) = mean(payoffStateA.cashflow_npv);
                nonCall.df_e(currentNodeIdx) = mean(payoffStateA.df);
                nonCall.df_c(currentNodeIdx) = model.DFRisky(currentTime/365.0);
                
%                 % callable at maturity is the same as noncallable
%                 callable.payoffStates.cashflow(currentNodeIdx,:) = nonCall.payoffStates.cashflow(currentNodeIdx,:);
%                 callable.payoffStates.cashflow_npv(currentNodeIdx,:) = nonCall.payoffStates.cashflow_npv(currentNodeIdx,:);
%                 callable.payoff = callable.payoff + callable.payoffStates.cashflow_npv(currentNodeIdx,:);
%                 
%                 callable.cashflow_e(currentNodeIdx) = nonCall.cashflow_e(currentNodeIdx);
%                 callable.cashflow_npv_e(currentNodeIdx) = nonCall.cashflow_npv_e(currentNodeIdx);
                  
                lastTimeIdx = currentTimeIdx;
                
                
                % flag for inclusive schedule
%                 simpleYN = false;
                simpleYN = 1;
                
                for i=periodCount:-1:1
                    % past schedule neglect
                    if DateDiff(valueDate,schedule.payDates(i)) >= 0, continue;end 
                    startTimeIdx = numMethodInfo.timeStepsInfo.startTimeIdx(i);
                    endTimeIdx = numMethodInfo.timeStepsInfo.endTimeIdx(i);

                    inclusiveYN = numMethodInfo.timeStepsInfo.inclusiveYN(i);
                     %% FromToInduction start

                    %modelsStates induct backward(lastTimeIdx to endTimeIdx)
                    while endTimeIdx < lastTimeIdx
                        lastTimeIdx = lastTimeIdx -1;
                        if lastTimeIdx == endTimeIdx break;end
                    end

                    currentTimeIdx = lastTimeIdx;
                    currentTime = totalTimeSteps(currentTimeIdx);
                    % now we are at endTimeIdx
                    % variables for coupon valuation initialized
%                     ralower = security.lower;
%                     raupper = security.upper;
%                     tenorLong = security.tenorL;
%                     tenorShort = security.tenorS;

                    cD = 0; % cumulated day count fraction
                    cP = 0; 
                    dI = 0; % dummy idx for loop
                    dcfC = 0;
                    dcfN = 0;
                    dD = 0;
                    idxCD = zeros(1,pathSize);
                    
                    idxC = zeros(1,pathSize);
                    idxN = zeros(1,pathSize);
                    
                    probSpread = zeros(1,pathSize);
                    
                    numOfTimeSteps = numMethodInfo.timeStepsInfo.numOfTimeSteps(i);
                    
                    % if inclusive then we use simpleYN rangeProb
                    %if inclusive then we do not evaluate the last step but
                    % use fixing instead 
                    endTimeStep = 1;
                    if inclusiveYN
%                         simpleYN = true;
                        simpleYN = 1;
                        endTimeStep = 2;
                    end
                    
                    for j=numOfTimeSteps:-1:endTimeStep
                        % evaluate dealdescription
                        % last step is omitted
                        currentTime = totalTimeSteps(currentTimeIdx);
                        %dcfC = currentTime/365.0;
                        dcfC = currentTime;
                        
                        if j~= numOfTimeSteps
                            dD = (dcfN-dcfC);
                        end

                        toDate = AddDate(valueDate,currentTime,'day');
                        tpDate = schedule.payDates(i);
                        
                                                                     
                        spreadC = model.CDDigitalRangeDate(valueDate,toDate,tpDate,tenorCD,0.25,ralower,raupper);
%                         spreadC = model.CMSSpreadDigitalRangeDate(valueDate,toDate,tpDate,tenorLong,tenorShort,0.25,ralower,raupper);
                        
                        %modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:numOfFactors*(currentTimeIdx-1)+2,:);
%                         idxL = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorLong,modelStateT);
%                         idxS = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorShort,modelStateT);
%                         spreadC = idxL - idxS;
%                         probSpread = model.SRPr(spreadC,spreadN,ralower,raupper,simpleYN,pathSize)*dD; 

                        if simpleYN
                            probSpread = spreadC*dD;
                        else
                            probSpread = spreadC*dD;
%                             probSpread = 0.5*(spreadC + spreadN)*dD;
                            
                        end

                        
                        cD = cD + dD;
                        cP = cP + probSpread;

                        dcfN = dcfC;
                        spreadN = spreadC;

                        %induct backward to the next time step
                        % one time backward induction
                        lastTimeIdx = currentTimeIdx;

                        if startTimeIdx < lastTimeIdx
                            lastTimeIdx = lastTimeIdx -1;
                            currentTimeIdx = lastTimeIdx;
                        end


                    end

                    %% FromToInduction end
                    %% AtDate evaluation
                    % Adding a new cashflow to the payoff

                    dcf = schedule.dayCountFraction(i);
                    
                    if inclusiveYN
                        % we add valuation date
                        cD = schedule.dayCount(i);
                        rangeDays =  security.rangeDays(i);
                        payoffStateA.cashflow = nominal * coupon * dcf * (rangeDays + cP)/cD;
                    else
                        payoffStateA.cashflow = nominal * coupon * dcf * cP/cD;
                    end
                    % eventDate(i) = startDate(i) therefore there is no induction in modelStates
                    % i.e. currentTimeIdx has not changed
                    % no need to update modelStateT
                    % modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:end,:);
                    % since cashflow's are added at the startDate(i) we need to
                    % multiply cashflow with the df(startDate(i), payDate(i))
                    payTime = DateDiff(schedule.payDates(i),valueDate);
                    
                    %event is at startDate so we need to discount the
                    %cashflow first from paydate to startdate
%                     df_sp = model.discountFactor(currentTime, payTime,numMatTime,modelStateT);

%                     for k=1:pathSize
%                         payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_sp(k);
%                     end
                    
                    % process payoff
                    
                    currentNodeIdx = i;
                    
                    % 1st nonCall column
                    nonCall.payoffStates.cashflow(currentNodeIdx,:) = payoffStateA.cashflow;
                    payoffStateA.cashflow_npv = payoffStateA.cashflow * model.DFRisky(payTime/365.0);
                    
%                     payoffStateA.cashflow_npv = model.discountPayoff(currentTime,numMatTime ...
%                     ,modelStateT,payoffStateA.cashflow);
                    nonCall.payoffStates.cashflow_npv(currentNodeIdx,:) = payoffStateA.cashflow_npv;
                    payoffStateA.df = model.DFRisky(payTime/365.0);
%                     payoffStateA.df = model.discountPayoff(currentTime,numMatTime ...
%                     ,modelStateT,1.0*ones(1,pathSize));
                    nonCall.payoffStates.df(currentNodeIdx,:) = payoffStateA.df;
                    
                    nonCall.payoff = nonCall.payoff + payoffStateA.cashflow_npv;
                    
                    nonCall.cashflow_e(currentNodeIdx) = mean(payoffStateA.cashflow);
                    nonCall.rangeProb_e(currentNodeIdx) = mean(nonCall.cashflow_e(currentNodeIdx)/(nominal * coupon * dcf));
                    nonCall.cashflow_npv_e(currentNodeIdx) = mean(payoffStateA.cashflow_npv);

                    nonCall.df_e(currentNodeIdx) = mean(payoffStateA.df);
                    nonCall.df_c(currentNodeIdx) = model.DFRisky(currentTime/365.0);
                    
%                     %2nd callable column
%                     callable.payoffStates.cashflow(currentNodeIdx,:) = nonCall.payoffStates.cashflow(currentNodeIdx,:);
%                     callable.payoffStates.cashflow_npv(currentNodeIdx,:) = nonCall.payoffStates.cashflow_npv(currentNodeIdx,:);
%                     callable.payoff = callable.payoff + callable.payoffStates.cashflow_npv(currentNodeIdx,:);
%                     
%                     callable.cashflow_e(currentNodeIdx) = nonCall.cashflow_e(currentNodeIdx);
%                     callable.cashflow_npv_e(currentNodeIdx) = nonCall.cashflow_npv_e(currentNodeIdx);

                    % if exercise is possible
                    if i == 1
                        dummaaa = 1;
                    end
                    
                    % call event happens  at the start of the period
                    % at call date, by default, call option is expired
                    
%                     if callYN(i) && (DateDiff(valueDate,schedule.startDates(i)) < 0)
%                         % 1.undiscount callable.payoff
%                         % 2.build the state variables matrix
%                         % 3.regress 1's with the given state variables
%                         % (model states 1,2)
%                         % 4. exercise per each path in
%                         % discount the new payoff and assign into callable
%                         % column
%                         
%                         %1 undiscount callable.payoff and get the
%                         %contiValue
%                        
%                         %exercise value
%                         exerciseValue = security.nominal*ones(1,pathSize);
%                         contiValue = zeros(1,pathSize);
%                         onePayoff = ones(1,pathSize);
%                         onePayoffD = model.discountPayoff(currentTime,numMatTime ...
%                         ,modelStateT,onePayoff);
%                         contiValue = callable.payoff./onePayoffD;
%                         
%                         % build the state variabls matrix
%                         decisionVar1 = modelStateT(1,:);
%                         decisionVar2 = modelStateT(2,:);
%                         % we use upto 2nd order polynomial as regression
%                         % equatioin
%                         dimensionLS = 6;
%                         regMatrix = ones(pathSize,dimensionLS);
%                         
%                         for idx_r=1:pathSize
%                             regMatrix(idx_r,2) = decisionVar1(idx_r);
%                             regMatrix(idx_r,3) = decisionVar2(idx_r);
%                             regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r);
%                             regMatrix(idx_r,5) = decisionVar1(idx_r)*decisionVar2(idx_r);
%                             regMatrix(idx_r,6) = decisionVar2(idx_r)*decisionVar2(idx_r);
%                         end
%                         
%                         % simple regression <- for path dependent option
%                         % we should change it
%                         % beta  : regression coefficients
%                         beta =zeros(dimensionLS,1);
%                         beta = regress(contiValue',regMatrix);
%                         regContiValue = regMatrix*beta;
%                         
%                         exercisedPayoff = regContiValue;
%                         % if regContiValue is bigger than exerciseValue
%                         % then issue exercise call option
%                         for idx_r=1:pathSize
%                             if regContiValue(idx_r) >= exerciseValue(idx_r) 
%                                 exercisedPayoff(idx_r) = exerciseValue(idx_r);
%                             end
%                         end
%                         
%                         % we discount the new payoff and 
%                         % assign it into the existing callable payoff
%                         callable.payoff = model.discountPayoff(currentTime,numMatTime ...
%                         ,modelStateT,exercisedPayoff);
%                         
%                     end
                end

                nonCall.npv = mean(nonCall.payoff);
                out.pricerInfo.nonCall = nonCall;
                
%                 callable.npv = mean(callable.payoff);
%                 out.pricerInfo.callable = callable;
                out.daycountFraction = schedule.dayCountFraction;
                
        end
        
        
        
        
    end
    
end



In [ ]:
classdef DailyWeeklyRiskyPricer_CDKTBSpreadCMSSpreadDualRASimpleCall < H_RootObject
    %Pricer(Security,Model,NumMethod)
    %Prcer is constructed from Security, Model, NumMethod 
    
    properties
        m_security
        m_model
        m_numMethod
        overHedgeYN
    end
    
    methods
        function pricer = DailyWeeklyRiskyPricer_KRWCMSSpread_CDKTBSpreadDualRASimpleCall(security,model,numMethod,overHedgeYN)
                pricer.m_security = security;
                pricer.m_model = model;
                pricer.m_numMethod = numMethod;
                pricer.overHedgeYN = overHedgeYN;
        end
        
        function out = computePriceMCCallableGeneric(pricer,valueDate,pricerOption)
                security = pricer.m_security;
                numMethod = pricer.m_numMethod;
                model = pricer.m_model;
                schedule = security.couponSchedule;
                callSchedule = security.callSchedule;
                
                % numMethodTimeSteps Generate
                mcOneTimeStep = numMethod.mcOneTimeStep;
%                 outInfo = numMethod.numMethodTimeStepsGenerateDW(valueDate,schedule,mcOneTimeStep);
                
                floatingSchedule = struct;
                outInfo = numMethod.numMethodTimeStepsGenerateDWSwapSimpleCall(valueDate,schedule,floatingSchedule,...
                                    callSchedule,mcOneTimeStep);
            
                numMethodInfo.timeStepsInfo = outInfo;

                % mc random number init
                %pathSize = 2^16;
                pathSize = numMethod.pathSize;

                % generate random numbers and modelstates forward
                rng('default');
                rng(0);

                % LGM2F : two factor
                % CK1F  : one factor
                
                numOfFactors = model.numOfFactors;

                %Z= randn(numOfFactors*outInfo.totalTimeStepsSize,pathSize/2);
%                 Z= randn(pathSize/2,numOfFactors*outInfo.totalTimeStepsSize);
                Z= randn(pathSize/2,numOfFactors*outInfo.modelTimeStepsSize);
                Z=[Z;-Z];
                Z=Z';
                
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;

                % we check with the c code, so we use generated random numbers
%                 mc_random = importdata('mc_random.xlsx');
%                 mc_random = importdata('D:\mc_random_number_new_report.txt');
%                 Z= mc_random;
                
                numMethodInfo.pathSize = pathSize;
                numMethodInfo.numOfFactors = numOfFactors;
                numMethodInfo.totalTimeSteps = outInfo.totalTimeSteps;
                numMethodInfo.totalTimeStepsSize = outInfo.totalTimeStepsSize;
                
                % for modelStateGeneration
                numMethodInfo.modelTimeSteps = outInfo.modelTimeSteps;
                numMethodInfo.modelTimeStepsSize = outInfo.modelTimeStepsSize;
                
                numMethodInfo.pastTimeStepsSize = outInfo.pastTimeStepsSize;
                
                pastTimeStepsSize = numMethodInfo.pastTimeStepsSize;
                
                numMethodInfo.Z = Z;



                %numMethod generate model states for the given timeSteps
%                 modelStates = numMethod.generateModelStatesNF(valueDate,schedule,numMethodInfo,model);
                 modelStates = numMethod.generateModelStatesNFNew(valueDate,schedule,numMethodInfo,model);
                % numeraire info
                numMatTime = numMethodInfo.totalTimeSteps(end);
                days = DateDiff(schedule.payDates(schedule.numOfPeriod),valueDate);
                numMatTime = max(numMatTime, days);
                dfNumMat = model.DFRisky(numMatTime/365.0);
                
                numeraire.numMatTime = numMatTime;
                numeraire.dfNumMat = dfNumMat;
                
                % induct backward and calculate cashflow
            
                periodCount = schedule.numOfPeriod;
                
                % pricer single info initialize
                
                nonCall = PricerInfo(periodCount +1, pathSize);
                callable = PricerInfo(periodCount +1, pathSize);
                dfCol = PricerInfo(periodCount +1, pathSize);
                
                dummy1 = PricerInfo(periodCount +1, pathSize);
                dummy2 = PricerInfo(periodCount +1, pathSize);
                
%                 % additional floating leg info
%                 if ~isempty(fieldnames(floatingSchedule))
%                     floatingLeg = PricerInfo(floatingSchedule.numOfPeriod, pathSize);
%                     % currentFloatingLegIdx for iteration in dealing with 
%                     % the floatingLeg cashflow
%                     currentFloatingLegIdx = floatingSchedule.numOfPeriod;
%                 end
                
                % intermediate variables
                payoffStateA.cashflow = zeros(1, pathSize);
                payoffStateA.cashflow_npv = zeros(1, pathSize);
                
            
                % we calculate cashflow backward from maturityDate to valueDate
                % rangeAccrual coupons are added at the startDate of the Period
                % At maturity

                nominal = security.nominal;
                coupon = security.coupon;
                callYN = security.callYN;
                
                tenorCMS = security.tenor;
                
                ralower = security.lower;
                raupper = security.upper;
                
                tenorLong = security.tenorL;
                tenorShort = security.tenorS;
                
                ralowerS = security.lowerS;
                raupperS = security.upperS;
                
                currentTime = numMethodInfo.totalTimeSteps(end);
                modelTimeStepsSize = numMethodInfo.modelTimeStepsSize;
                currentTimeIdx = numMethodInfo.totalTimeStepsSize;
                
                currentNodeIdx = periodCount + 1;
                
                modelStateT = modelStates(numOfFactors*(max(currentTimeIdx-pastTimeStepsSize-1,0))+1:numOfFactors*(max(currentTimeIdx-pastTimeStepsSize-1,0))+numOfFactors,:);
%                 modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:numOfFactors*(currentTimeIdx-1)+numOfFactors,:);
                
                payTime = DateDiff(schedule.payDates(schedule.numOfPeriod),valueDate);
                
                %even though there is no cashflow at the last structured
                %enddate, we add dummy cashflow in line with the note
                if payTime > 0
                    payoffStateA.cashflow = nominal*ones(1,pathSize);

                    df_ep = model.discountFactorRisky(currentTime, payTime,numMatTime,modelStateT);
                    for k=1:pathSize
                        payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                    end

                    %process payoff at maturity
                    nonCall.ProcessPayoff(model,currentTime,numeraire, ...
                                         modelStateT,payoffStateA.cashflow,currentNodeIdx);

                    callable.ProcessPayoff(model,currentTime,numeraire, ...
                                         modelStateT,payoffStateA.cashflow,currentNodeIdx);

                    %df column
                    payoffStateA.cashflow = ones(1,pathSize);
                    for k=1:pathSize
                        payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                    end

                    dfCol.ProcessPayoff(model,currentTime,numeraire, ...
                                         modelStateT,payoffStateA.cashflow,currentNodeIdx);

                    %dummy column
                    foreignModelName = model.foreignModelName;

                    if strcmp(pricerOption,'PSA')
                        payoffStateA.cashflow = model.CMS_PSA_SimpleDate(model.refModelName,valueDate,currentTime,numMatTime, tenorLong,modelStateT);
                        dummy1.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);

                        payoffStateA.cashflow = model.CMS_PSA_SimpleDate(foreignModelName,valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                        dummy2.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);

                    else
                        payoffStateA.cashflow = model.CMS_SimpleDate(model.refModelName,valueDate,currentTime,numMatTime, tenorLong,modelStateT);
                        dummy1.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);

                        payoffStateA.cashflow = model.CMS_SimpleDate(foreignModelName,valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                        dummy2.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);

                    end
                
                end

                lastTimeIdx = currentTimeIdx;
                totalTimeSteps = numMethodInfo.timeStepsInfo.totalTimeSteps;
                
                % flag for inclusive schedule
%                 simpleYN = false;
%                 simpleYN = true;
                if pricer.overHedgeYN == true 
                    simpleYN = 2;
                else
                    simpleYN = 1;
                end
                
                for i=periodCount:-1:1
                    % we induct in the pastTimeSteps too
%                     if DateDiff(valueDate,schedule.payDates(i)) >= 0, continue;end 
                    
                    startTimeIdx = numMethodInfo.timeStepsInfo.startTimeIdx(i);
                    endTimeIdx = numMethodInfo.timeStepsInfo.endTimeIdx(i);

                    inclusiveYN = numMethodInfo.timeStepsInfo.inclusiveYN(i);
                 %% FromToInduction start

                    %modelsStates induct backward(lastTimeIdx to endTimeIdx)
                    while endTimeIdx < lastTimeIdx
                        lastTimeIdx = lastTimeIdx -1;
                        if lastTimeIdx == endTimeIdx break;end
                    end

                    currentTimeIdx = lastTimeIdx;
                    currentTime = totalTimeSteps(currentTimeIdx);
                    % now we are at endTimeIdx
                    % variables for coupon valuation initialized
                    

                    cD = 0; % cumulated day count fraction
                     %cP shoul be declared as an array
%                     cP = 0;
                    cP = zeros(1,pathSize);
                    dI = 0; % dummy idx for loop
                    dcfC = 0;
                    dcfN = 0;
                    dD = 0;
                    idxCMS = zeros(1,pathSize);
                    
                    idxC = zeros(1,pathSize);
                    idxN = zeros(1,pathSize);
                    
                    idxL = zeros(1,pathSize);
                    idxS = zeros(1,pathSize);
                    
                    spreadC = zeros(1,pathSize);
                    spreadN = zeros(1,pathSize);
                    probSpread = zeros(1,pathSize);
                    
                    numOfTimeSteps = numMethodInfo.timeStepsInfo.numOfTimeSteps(i);
                    
                    % if inclusive then we use simpleYN rangeProb
                    %if inclusive then we do not evaluate the last step but
                    % use fixing instead 
                    endTimeStep = 1;
                     % we let endTimeStep to be 1 and
                    % inside the for loop if currentTime ==0
                    % then we skip the range accrual check part
                    
                    if inclusiveYN
%                         simpleYN = true;
                        simpleYN = 1;
%                         endTimeStep = 2;
                        if pricer.overHedgeYN == true 
                            simpleYN = 2;
                        end
                    end
                    
                    for j=numOfTimeSteps:-1:endTimeStep
                        % we initialize payoffStateA as zeros to prevent
                        % reuse it;
                        payoffStateA.cashflow = zeros(1,pathSize);
                        % evaluate FromTo dealdescription
                        % last step is omitted
                        currentTime = totalTimeSteps(currentTimeIdx);
                        if currentTime > 0
                            dcfC = currentTime;

                            if j~= numOfTimeSteps
                                dD = (dcfN-dcfC);
                            end

%                             modelStateT = modelStates(max(currentTimeIdx-pastTimeStepsSize-1,0)+1:numOfFactors*(currentTimeIdx-1)+numOfFactors,:);
                            modelStateT = modelStates(numOfFactors*(max(currentTimeIdx-pastTimeStepsSize-1,0))+1:numOfFactors*(max(currentTimeIdx-pastTimeStepsSize-1,0))+numOfFactors,:);
                            if strcmp(pricerOption,'PSA')
                                idxCMS = model.CMS_PSA_SimpleDate(foreignModelName,valueDate,currentTime,numMatTime, tenorCMS,modelStateT);
                                idxC = idxCMS;

                                idxL = model.CMS_PSA_SimpleDate(model.refModelName,valueDate,currentTime,numMatTime, tenorLong,modelStateT);
                                idxS = model.CMS_PSA_SimpleDate(model.refModelName,valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                                spreadC = idxL - idxS;
                            else
                                idxCMS = model.CMS_SimpleDate(foreignModelName,valueDate,currentTime,numMatTime, tenorCMS,modelStateT);
                                idxC = idxCMS;

                                idxL = model.CMS_SimpleDate(model.refModelName,valueDate,currentTime,numMatTime, tenorLong,modelStateT);
                                idxS = model.CMS_SimpleDate(model.refModelName,valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                                spreadC = idxL - idxS;
                            end
    %                         probSpread = model.SRPr(spreadC,spreadN,ralower,raupper,simpleYN,pathSize)*dD;
                            probSpread = model.DRPr(idxC,idxN,ralower,raupper,spreadC,spreadN,ralowerS,raupperS,simpleYN,pathSize)*dD;
                            cD = cD + dD;
                            cP = cP + probSpread;

                            dcfN = dcfC;

                            idxN = idxC;
                            spreadN = spreadC;
                        else
                            dcfC = currentTime;

                            if j~= numOfTimeSteps
                                dD = (dcfN-dcfC);
                            end

                            cD = cD + dD;

                            dcfN = dcfC;
                            
                        end
                        
                        %floating event begins here
                        %floating event end here
                        
                        % callable event begin
                        idxCall = find(currentTime == outInfo.callEventTime);
                        
                        % we do not evaluate callable event at the start of
                        % coupon Schedule for avoding double counting
                        % and that make sense
                        if ~isempty(idxCall) && (j~= endTimeStep)
                            iterCall = 1;
                            foundIterCallYN= false;
                            for idx11 = length(callSchedule.resetDates):-1:1
                                % call event happens at call startDate !
                                % we change call event happens at call resetDate !
                                
                                if outInfo.callEventTime(idxCall) == DateDiff(callSchedule.resetDates(idx11),valueDate)
                                    iterCall = idx11;
                                    foundIterCallYN = true;
                                    break;
                                end
                            end
                            
                            if ~foundIterCallYN
                                error('callEventTime is not in the callSchedule startDates')
                            end
                            
                            % only those cashflow that paydate is after
                            % valueDate will be added
                            if DateDiff(callSchedule.payDates(iterCall),valueDate) > 0
                                % callEvent is not fixed
                                if DateDiff(valueDate,callSchedule.resetDates(iterCall)) < 0
                                    % 1.undiscount callable.payoff
                                    % 2.build the state variables matrix
                                    % 3.regress 1's with the given state variables
                                    % (model states 1,2)
                                    % 4. exercise per each path in
                                    % discount the new payoff and assign into callable
                                    % column

                                    %1 undiscount callable.payoff and get the
                                    %contiValue

                                    %exercise value Note
                                    exerciseValue = security.nominal*ones(1,pathSize) * 1.0;
                                    payTime = DateDiff(callSchedule.payDates(iterCall),valueDate);
                                    %event is at startDate so we need to discount the
                                    %cashflow first from paydate to startdate
                                    df_ep = model.discountFactorRisky(currentTime, payTime,numMatTime,modelStateT);
                                    for k=1:pathSize
                                        exerciseValue(k) = exerciseValue(k) * df_ep(k);
                                    end
                                    
                                    contiValue = zeros(1,pathSize);
                                    onePayoff = ones(1,pathSize);
                                    onePayoffD = model.discountPayoffRisky(currentTime,numMatTime ...
                                    ,modelStateT,onePayoff);
                                    contiValue = callable.payoff./onePayoffD;

                                    %Nominal Regression test
                                    contiValue = contiValue/10000;

                                    % build the state variabls matrix
                                    if numMethod.regressionOrder == 3
            %                             dimensionLS = (numOfFactors + 1)*(numOfFactors + 2)/2;

                                        decisionVar = zeros(numOfFactors,size(modelStateT,2));

                                        for idxLS =1:numOfFactors
                                            decisionVar(idxLS,:) = modelStateT(idxLS,:); 
                                        end

                                        % calculate number of dimension first
                                        % 0th order
                                        dimensionLS = 1;

                                        % 1st order
                                        for idxLS=1:numOfFactors
                                            dimensionLS = dimensionLS + 1;
                                        end

                                        %2nd order
                                        for idxLS1=1:numOfFactors
                                            for idxLS2=idxLS1:numOfFactors
                                                dimensionLS = dimensionLS + 1;
                                            end
                                        end

                                        %3rd order
                                        for idxLS1=1:numOfFactors
                                            for idxLS2=idxLS1:numOfFactors
                                                for idxLS3=idxLS2:numOfFactors
                                                    dimensionLS = dimensionLS + 1;
                                                end
                                            end
                                        end


                %                         decisionVar1 = modelStateT(1,:);
                %                         decisionVar2 = modelStateT(2,:);
                                        % we use upto 2nd order polynomial as regression
                                        % equatioin

                                        regMatrix = ones(pathSize,dimensionLS);

                                        for idx_r=1:pathSize
                                            for idxLS=1:numOfFactors
                                                regMatrix(idx_r,idxLS+1) = decisionVar(idxLS,idx_r);
                                            end

                                            tempIdx = 0;
                                            for idxLS1=1:numOfFactors
                                                for idxLS2=idxLS1:numOfFactors
                                                    tempIdx = tempIdx + 1;
                                                    regMatrix(idx_r,numOfFactors+1+tempIdx) = decisionVar(idxLS1,idx_r)*decisionVar(idxLS2,idx_r);
                                                end
                                            end

                                            tempIdx2 = 0;
                                            for idxLS1=1:numOfFactors
                                                for idxLS2=idxLS1:numOfFactors
                                                    for idxLS3=idxLS2:numOfFactors
                                                        tempIdx2 = tempIdx2 + 1;
                                                        regMatrix(idx_r,numOfFactors+1+tempIdx + tempIdx2) = decisionVar(idxLS1,idx_r)*decisionVar(idxLS2,idx_r)*decisionVar(idxLS3,idx_r);
                                                    end
                                                end
                                            end


                                        end

                                    else
                                        dimensionLS = (numOfFactors + 1)*(numOfFactors + 2)/2;

                                        decisionVar = zeros(numOfFactors,size(modelStateT,2));

                                        for idxLS =1:numOfFactors
                                            decisionVar(idxLS,:) = modelStateT(idxLS,:); 
                                        end
                %                         decisionVar1 = modelStateT(1,:);
                %                         decisionVar2 = modelStateT(2,:);
                                        % we use upto 2nd order polynomial as regression
                                        % equatioin

                                        regMatrix = ones(pathSize,dimensionLS);

                                        for idx_r=1:pathSize
                                            for idxLS=1:numOfFactors
                                                regMatrix(idx_r,idxLS+1) = decisionVar(idxLS,idx_r);
                                            end

                                            tempIdx = 1;
                                            for idxLS1=1:numOfFactors
                                                for idxLS2=idxLS1:numOfFactors
                                                    regMatrix(idx_r,numOfFactors+1+tempIdx) = decisionVar(idxLS1,idx_r)*decisionVar(idxLS2,idx_r);
                                                    tempIdx = tempIdx + 1;
                                                end
                                            end

                                        end
                                    end
                                    % simple regression <- for path dependent option
                                    % we should change it
                                    % beta  : regression coefficients
                                    beta =zeros(dimensionLS,1);
                                    beta = regress(contiValue',regMatrix);
                                    regContiValue = regMatrix*beta;

                                     %Nominal Regression test
                                    regContiValue = regContiValue * 10000;

                                    exercisedPayoff = regContiValue;
                                    % if regContiValue is bigger than exerciseValue
                                    % then issue exercise call option
                                    for idx_r=1:pathSize
                                        if regContiValue(idx_r) >= exerciseValue(idx_r) 
                                            exercisedPayoff(idx_r) = exerciseValue(idx_r);
                                        end
                                    end

                                    % we discount the new payoff and 
                                    % assign it into the existing callable payoff
                                    callable.payoff = model.discountPayoffRisky(currentTime,numMatTime ...
                                    ,modelStateT,exercisedPayoff);

%                                     contiValue = zeros(1,pathSize);
%                                     onePayoff = ones(1,pathSize);
%                                     onePayoffD = model.discountPayoffRisky(currentTime,numMatTime ...
%                                     ,modelStateT,onePayoff);
%                                     contiValue = callable.payoff./onePayoffD;
% 
%                                     %Nominal Regression test
%                                     contiValue = contiValue/10000;
% 
%                                     % build the state variabls matrix
%                                     decisionVar1 = modelStateT(1,:);
%                                     decisionVar2 = modelStateT(2,:);
%                                     % we use upto 2nd order polynomial as regression
%                                     % equatioin
%                                     dimensionLS = 6;
%                                     regMatrix = ones(pathSize,dimensionLS);
% 
%                                     for idx_r=1:pathSize
%                                         regMatrix(idx_r,2) = decisionVar1(idx_r);
%                                         regMatrix(idx_r,3) = decisionVar2(idx_r);
%                                         regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r);
%                                         regMatrix(idx_r,5) = decisionVar1(idx_r)*decisionVar2(idx_r);
%                                         regMatrix(idx_r,6) = decisionVar2(idx_r)*decisionVar2(idx_r);
%                                     end
% 
%                                     % simple regression <- for path dependent option
%                                     % we should change it
%                                     % beta  : regression coefficients
%                                     beta =zeros(dimensionLS,1);
%                                     beta = regress(contiValue',regMatrix);
%                                     regContiValue = regMatrix*beta;
% 
%                                      %Nominal Regression test
%                                     regContiValue = regContiValue * 10000;
% 
%                                     exercisedPayoff = regContiValue;
%                                     % if regContiValue is bigger than exerciseValue
%                                     % then issue exercise call option
%                                     for idx_r=1:pathSize
%                                         if regContiValue(idx_r) >= exerciseValue(idx_r) 
%                                             exercisedPayoff(idx_r) = exerciseValue(idx_r);
%                                         end
%                                     end
% 
%                                     % we discount the new payoff and 
%                                     % assign it into the existing callable payoff
%                                     callable.payoff = model.discountPayoffRisky(currentTime,numMatTime ...
%                                     ,modelStateT,exercisedPayoff);

                                end
                                
                                
                            end
                        end
                        
                        % callable event end
                        
                        %induct backward to the next time step
                        % one time backward induction
                        lastTimeIdx = currentTimeIdx;

                        if startTimeIdx < lastTimeIdx
                            lastTimeIdx = lastTimeIdx -1;
                            currentTimeIdx = lastTimeIdx;
                        end


                    end

                    %% FromToInduction end
        % if the current period is inclusive, then the start date is not evaluated in the above for loop.
		% instead the fixing will be treated separately below.
		% however current_time_idx is updated at the end of the for loop.
		% so we need to update model_states accordingly.
		% if model_states were updated previously then the following additionaly step doesn't affect the result anyway.
                    currentTime = totalTimeSteps(currentTimeIdx);
%                     modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:numOfFactors*(currentTimeIdx-1)+numOfFactors,:);
                    modelStateT = modelStates(numOfFactors*(max(currentTimeIdx-pastTimeStepsSize-1,0))+1:numOfFactors*(max(currentTimeIdx-pastTimeStepsSize-1,0))+numOfFactors,:);
                    
                    %% AtDate evaluation
                    % Adding a new cashflow to the payoff
                    if DateDiff(schedule.payDates(i),valueDate) > 0
                        dcf = schedule.dayCountFraction(i);

                        if inclusiveYN
                            % we add valuation date
                            cD = schedule.dayCount(i);
                            rangeDays =  security.rangeDays(i);
                            payoffStateA.cashflow = nominal * coupon * dcf * (rangeDays + cP)/cD;
                        else
                            payoffStateA.cashflow = nominal * coupon * dcf * cP/cD;
                        end
                        % eventDate(i) = startDate(i) therefore there is no induction in modelStates
                        % i.e. currentTimeIdx has not changed
                        % no need to update modelStateT
                        % modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:end,:);
                        % since cashflow's are added at the startDate(i) we need to
                        % multiply cashflow with the df(startDate(i), payDate(i))
                        payTime = DateDiff(schedule.payDates(i),valueDate);

                        %event is at startDate so we need to discount the
                        %cashflow first from paydate to startdate
                        df_ep = model.discountFactorRisky(currentTime, payTime,numMatTime,modelStateT);

                        for k=1:pathSize
                            payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                        end

                        % process payoff

                        currentNodeIdx = i;

                        nonCall.ProcessPayoff(model,currentTime,numeraire, ...
                                         modelStateT,payoffStateA.cashflow,currentNodeIdx);

                        callable.ProcessPayoff(model,currentTime,numeraire, ...
                                         modelStateT,payoffStateA.cashflow,currentNodeIdx);

                        %df column
                        payoffStateA.cashflow = ones(1,pathSize);
                        for k=1:pathSize
                            payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                        end

                        dfCol.ProcessPayoff(model,currentTime,numeraire, ...
                                         modelStateT,payoffStateA.cashflow,currentNodeIdx);

                        %dummy column
                        if strcmp(pricerOption,'PSA')
                            payoffStateA.cashflow = model.CMS_PSA_SimpleDate(model.refModelName,valueDate,currentTime,numMatTime, tenorLong,modelStateT);
                            dummy1.ProcessPayoff(model,currentTime,numeraire, ...
                                             modelStateT,payoffStateA.cashflow,currentNodeIdx);



                            payoffStateA.cashflow = model.CMS_PSA_SimpleDate(foreignModelName,valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                            dummy2.ProcessPayoff(model,currentTime,numeraire, ...
                                             modelStateT,payoffStateA.cashflow,currentNodeIdx);

                        else
                            payoffStateA.cashflow = model.CMS_SimpleDate(model.refModelName,valueDate,currentTime,numMatTime, tenorLong,modelStateT);
                            dummy1.ProcessPayoff(model,currentTime,numeraire, ...
                                             modelStateT,payoffStateA.cashflow,currentNodeIdx);



                            payoffStateA.cashflow = model.CMS_SimpleDate(foreignModelName,valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                            dummy2.ProcessPayoff(model,currentTime,numeraire, ...
                                             modelStateT,payoffStateA.cashflow,currentNodeIdx);

                        end

                    end           
                    
                    
                    
                    % if exercise is possible
                    if i == 1
                        dummaaa = 1;
                    end
                    
                    % call event happens  at the start of the period
                    % at call date, by default, call option is expired
                    
%                     if callYN(i) && (DateDiff(valueDate,callSchedule.resetDates(i)) < 0)
%                         % 1.undiscount callable.payoff
%                         % 2.build the state variables matrix
%                         % 3.regress 1's with the given state variables
%                         % (model states 1,2)
%                         % 4. exercise per each path in
%                         % discount the new payoff and assign into callable
%                         % column
%                         
%                         %1 undiscount callable.payoff and get the
%                         %contiValue
%                        
%                         %exercise value
%                         exerciseValue = security.nominal*ones(1,pathSize);
%                         payTime = DateDiff(callSchedule.payDates(i),valueDate);
%                         %event is at startDate so we need to discount the
%                         %cashflow first from paydate to startdate
%                         df_ep = model.discountFactorRisky(currentTime, payTime,numMatTime,modelStateT);
%                         for k=1:pathSize
%                             exerciseValue(k) = exerciseValue(k) * df_ep(k);
%                         end
%                         
%                         contiValue = zeros(1,pathSize);
%                         onePayoff = ones(1,pathSize);
%                         onePayoffD = model.discountPayoffRisky(currentTime,numMatTime ...
%                         ,modelStateT,onePayoff);
%                         contiValue = callable.payoff./onePayoffD;
%                         
%                         %Nominal Regression test
%                         contiValue = contiValue/10000;
%                         
%                         % build the state variabls matrix
%                         if numMethod.regressionOrder == 3
% %                             dimensionLS = (numOfFactors + 1)*(numOfFactors + 2)/2;
% 
%                             decisionVar = zeros(numOfFactors,size(modelStateT,2));
% 
%                             for idxLS =1:numOfFactors
%                                 decisionVar(idxLS,:) = modelStateT(idxLS,:); 
%                             end
%                             
%                             % calculate number of dimension first
%                             % 0th order
%                             dimensionLS = 1;
%                             
%                             % 1st order
%                             for idxLS=1:numOfFactors
%                                 dimensionLS = dimensionLS + 1;
%                             end
%                             
%                             %2nd order
%                             for idxLS1=1:numOfFactors
%                                 for idxLS2=idxLS1:numOfFactors
%                                     dimensionLS = dimensionLS + 1;
%                                 end
%                             end
%                             
%                             %3rd order
%                             for idxLS1=1:numOfFactors
%                                 for idxLS2=idxLS1:numOfFactors
%                                     for idxLS3=idxLS2:numOfFactors
%                                         dimensionLS = dimensionLS + 1;
%                                     end
%                                 end
%                             end
% 
%                             
%     %                         decisionVar1 = modelStateT(1,:);
%     %                         decisionVar2 = modelStateT(2,:);
%                             % we use upto 2nd order polynomial as regression
%                             % equatioin
% 
%                             regMatrix = ones(pathSize,dimensionLS);
% 
%                             for idx_r=1:pathSize
%                                 for idxLS=1:numOfFactors
%                                     regMatrix(idx_r,idxLS+1) = decisionVar(idxLS,idx_r);
%                                 end
% 
%                                 tempIdx = 0;
%                                 for idxLS1=1:numOfFactors
%                                     for idxLS2=idxLS1:numOfFactors
%                                         tempIdx = tempIdx + 1;
%                                         regMatrix(idx_r,numOfFactors+1+tempIdx) = decisionVar(idxLS1,idx_r)*decisionVar(idxLS2,idx_r);
%                                     end
%                                 end
%                                 
%                                 tempIdx2 = 0;
%                                 for idxLS1=1:numOfFactors
%                                     for idxLS2=idxLS1:numOfFactors
%                                         for idxLS3=idxLS2:numOfFactors
%                                             tempIdx2 = tempIdx2 + 1;
%                                             regMatrix(idx_r,numOfFactors+1+tempIdx + tempIdx2) = decisionVar(idxLS1,idx_r)*decisionVar(idxLS2,idx_r)*decisionVar(idxLS3,idx_r);
%                                         end
%                                     end
%                                 end
%                                 
% 
%                             end
%                         
%                         else
%                             dimensionLS = (numOfFactors + 1)*(numOfFactors + 2)/2;
% 
%                             decisionVar = zeros(numOfFactors,size(modelStateT,2));
% 
%                             for idxLS =1:numOfFactors
%                                 decisionVar(idxLS,:) = modelStateT(idxLS,:); 
%                             end
%     %                         decisionVar1 = modelStateT(1,:);
%     %                         decisionVar2 = modelStateT(2,:);
%                             % we use upto 2nd order polynomial as regression
%                             % equatioin
% 
%                             regMatrix = ones(pathSize,dimensionLS);
% 
%                             for idx_r=1:pathSize
%                                 for idxLS=1:numOfFactors
%                                     regMatrix(idx_r,idxLS+1) = decisionVar(idxLS,idx_r);
%                                 end
% 
%                                 tempIdx = 1;
%                                 for idxLS1=1:numOfFactors
%                                     for idxLS2=idxLS1:numOfFactors
%                                         regMatrix(idx_r,numOfFactors+1+tempIdx) = decisionVar(idxLS1,idx_r)*decisionVar(idxLS2,idx_r);
%                                         tempIdx = tempIdx + 1;
%                                     end
%                                 end
% 
%                             end
%                         end
%                         % simple regression <- for path dependent option
%                         % we should change it
%                         % beta  : regression coefficients
%                         beta =zeros(dimensionLS,1);
%                         beta = regress(contiValue',regMatrix);
%                         regContiValue = regMatrix*beta;
%                        
%                          %Nominal Regression test
%                         regContiValue = regContiValue * 10000;
%                         
%                         exercisedPayoff = regContiValue;
%                         % if regContiValue is bigger than exerciseValue
%                         % then issue exercise call option
%                         for idx_r=1:pathSize
%                             if regContiValue(idx_r) >= exerciseValue(idx_r) 
%                                 exercisedPayoff(idx_r) = exerciseValue(idx_r);
%                             end
%                         end
%                         
%                         % we discount the new payoff and 
%                         % assign it into the existing callable payoff
%                         callable.payoff = model.discountPayoffRisky(currentTime,numMatTime ...
%                         ,modelStateT,exercisedPayoff);
%                         
%                     end
                end

                nonCall.npv = mean(nonCall.payoff);
                out.nonCall = nonCall;
                callable.npv = mean(callable.payoff);
                dfCol.npv = mean(dfCol.payoff);
                dummy1.npv = mean(dummy1.npv);
                dummy2.npv = mean(dummy2.npv);
                out.callable = callable;
                out.dfCol = dfCol;
                out.dummy1 = dummy1;
                out.dummy2 = dummy2;
                out.daycountFraction = schedule.dayCountFraction;
                
        end
        
        
        function out = computePriceClosedSimple(pricer,valueDate)
                security = pricer.m_security;
                numMethod = pricer.m_numMethod;
                model = pricer.m_model;
                schedule = security.couponSchedule;
                
                % numMethodTimeSteps Generate
                mcOneTimeStep = numMethod.mcOneTimeStep;
                outInfo = numMethod.numMethodTimeStepsGenerateDW(valueDate,schedule,mcOneTimeStep);
            
                numMethodInfo.timeStepsInfo = outInfo;

                % mc random number init
                %pathSize = 2^16;
                pathSize = numMethod.pathSize;
                
                pathSize = 1;
                
%                 % generate random numbers and modelstates forward
%                 rng('default');
%                 rng(0);
% 
                numOfFactors = model.numOfFactor;
% 
%                 %Z= randn(numOfFactors*outInfo.totalTimeStepsSize,pathSize/2);
%                 Z= randn(pathSize/2,numOfFactors*outInfo.totalTimeStepsSize);
%                 Z=[Z;-Z];
%                 Z=Z';
%                 
%                 
%                 %TimeInversion
%                 sizeZ = size(Z,1);
%                 A = zeros(sizeZ,sizeZ);
%                 for i=1:sizeZ
%                     A(i,sizeZ + 1 -i) = 1;
%                 end
%                 Z = A*Z;
% 
%                 % we check with the c, so we use generated random numbers
% %                 mc_random = importdata('mc_random.xlsx');
% %                 mc_random = importdata('D:\mc_random_number_new_report.txt');
% %                 Z= mc_random;
%                 
                numMethodInfo.pathSize = pathSize;
                numMethodInfo.numOfFactors = numOfFactors;
                numMethodInfo.totalTimeSteps = outInfo.totalTimeSteps;
                numMethodInfo.totalTimeStepsSize = outInfo.totalTimeStepsSize;
%                 numMethodInfo.Z = Z;
% 
% 
% 
%                 %generate model states for the given timeSteps
%                 modelStates = numMethod.generateModelStates(valueDate,schedule,numMethodInfo,model);

                % numeraire info
                numMatTime = numMethodInfo.totalTimeSteps(end);
                dfNumMat = model.DFRisky(numMatTime/365.0);
                
                % induct backward and calculate cashflow
            
                periodCount = schedule.numOfPeriod;
                
                % cashflow columns initialize
                % 1st nonCall
                nonCall.npv = 0;
                nonCall.cashflow_e = zeros(periodCount + 1,1);
                nonCall.cashflow_npv_e = zeros(periodCount + 1,1);
                nonCall.rangeProb_e = zeros(periodCount + 1,1);
                nonCall.df_e = zeros(periodCount + 1,1);
                nonCall.df_c = zeros(periodCount + 1,1);

                % debug information, cashflow & npv per path
                nonCall.payoff = zeros(1,pathSize);
                nonCall.payoffStates.cashflow = zeros(periodCount+1,pathSize);
                nonCall.payoffStates.cashflow_npv = zeros(periodCount+1,pathSize);
                nonCall.payoffStates.df = zeros(periodCount + 1,pathSize);
                
                % intermediate variables
                payoffStateA.cashflow = zeros(1, pathSize);
                payoffStateA.cashflow_npv = zeros(1, pathSize);
                payoffStateA.df = zeros(1,pathSize);
                
                % we calculate cashflow backward from maturityDate to valueDate
                % rangeAccrual coupons are added at the startDate of the Period
                % At maturity

                nominal = security.nominal;
                coupon = security.coupon;
                callYN = security.callYN;
                
                tenorCMS = security.tenor;
                
                ralower = security.lower;
                raupper = security.upper;
                
                tenorLong = security.tenorL;
                tenorShort = security.tenorS;
                
                ralowerS = security.lowerS;
                raupperS = security.upperS;
                
                currentTime = numMethodInfo.totalTimeSteps(end);
                totalTimeSteps = numMethodInfo.timeStepsInfo.totalTimeSteps;
                totalTimeStepsSize = numMethodInfo.timeStepsInfo.totalTimeStepsSize;
                if totalTimeStepsSize == 0 || (totalTimeStepsSize == 1 && totalTimeSteps(1) == 0) 
                    out.nonCall = nonCall;
                    return;
                end
                
                currentTimeIdx = totalTimeStepsSize;
                
                currentNodeIdx = periodCount + 1;
                
%                 modelStateT = modelStates(numOfFactors*(totalTimeStepsSize-1)+1:numOfFactors*(totalTimeStepsSize-1)+2,:);
                payoffStateA.cashflow = nominal*ones(1,pathSize);
                %process payoff at maturity
                nonCall.payoffStates.cashflow(currentNodeIdx,:) = payoffStateA.cashflow;
                
%                 payoffStateA.cashflow_npv = model.discountPayoff(currentTime,numMatTime ...
%                     ,modelStateT,payoffStateA.cashflow);
%                 
                payoffStateA.cashflow_npv = model.DFRisky(currentTime/365.0)*payoffStateA.cashflow;
                
                nonCall.payoffStates.cashflow_npv(currentNodeIdx,:) = payoffStateA.cashflow_npv ;
                
%                 payoffStateA.df = model.discountPayoff(currentTime,numMatTime ...
%                     ,modelStateT,1.0*ones(1,pathSize));
                payoffStateA.df = model.DFRisky(currentTime/365.0) *ones(1,pathSize);
                    

                nonCall.payoffStates.df(currentNodeIdx,:) = payoffStateA.df;
                nonCall.payoff = nonCall.payoff + payoffStateA.cashflow_npv;
                
                nonCall.cashflow_e(currentNodeIdx) = mean(payoffStateA.cashflow);
                nonCall.cashflow_npv_e(currentNodeIdx) = mean(payoffStateA.cashflow_npv);
                nonCall.df_e(currentNodeIdx) = mean(payoffStateA.df);
                nonCall.df_c(currentNodeIdx) = model.DFRisky(currentTime/365.0);
                
%                 % callable at maturity is the same as noncallable
%                 callable.payoffStates.cashflow(currentNodeIdx,:) = nonCall.payoffStates.cashflow(currentNodeIdx,:);
%                 callable.payoffStates.cashflow_npv(currentNodeIdx,:) = nonCall.payoffStates.cashflow_npv(currentNodeIdx,:);
%                 callable.payoff = callable.payoff + callable.payoffStates.cashflow_npv(currentNodeIdx,:);
%                 
%                 callable.cashflow_e(currentNodeIdx) = nonCall.cashflow_e(currentNodeIdx);
%                 callable.cashflow_npv_e(currentNodeIdx) = nonCall.cashflow_npv_e(currentNodeIdx);
                  
                lastTimeIdx = currentTimeIdx;
                
                
                % flag for inclusive schedule
%                 simpleYN = false;
                simpleYN = 1;
                
                for i=periodCount:-1:1
                    % past schedule neglect
                    if DateDiff(valueDate,schedule.payDates(i)) >= 0, continue;end 
                    startTimeIdx = numMethodInfo.timeStepsInfo.startTimeIdx(i);
                    endTimeIdx = numMethodInfo.timeStepsInfo.endTimeIdx(i);

                    inclusiveYN = numMethodInfo.timeStepsInfo.inclusiveYN(i);
                     %% FromToInduction start

                    %modelsStates induct backward(lastTimeIdx to endTimeIdx)
                    while endTimeIdx < lastTimeIdx
                        lastTimeIdx = lastTimeIdx -1;
                        if lastTimeIdx == endTimeIdx break;end
                    end

                    currentTimeIdx = lastTimeIdx;
                    currentTime = totalTimeSteps(currentTimeIdx);
                    % now we are at endTimeIdx
                    % variables for coupon valuation initialized
%                     ralower = security.lower;
%                     raupper = security.upper;
%                     tenorLong = security.tenorL;
%                     tenorShort = security.tenorS;

                    cD = 0; % cumulated day count fraction
                    cP = 0; 
                    dI = 0; % dummy idx for loop
                    dcfC = 0;
                    dcfN = 0;
                    dD = 0;
                    idxCMS = zeros(1,pathSize);
                    
                    idxC = zeros(1,pathSize);
                    idxN = zeros(1,pathSize);
                    
                    idxL = zeros(1,pathSize);
                    idxS = zeros(1,pathSize);
                    spreadC = zeros(1,pathSize);
                    spreadN = zeros(1,pathSize);
                    probSpread = zeros(1,pathSize);
                    
                    numOfTimeSteps = numMethodInfo.timeStepsInfo.numOfTimeSteps(i);
                    
                    % if inclusive then we use simpleYN rangeProb
                    %if inclusive then we do not evaluate the last step but
                    % use fixing instead 
                    endTimeStep = 1;
                    if inclusiveYN
%                         simpleYN = true;
                        simpleYN = 1;
                        endTimeStep = 2;
                    end
                    
                    for j=numOfTimeSteps:-1:endTimeStep
                        % evaluate dealdescription
                        % last step is omitted
                        currentTime = totalTimeSteps(currentTimeIdx);
                        %dcfC = currentTime/365.0;
                        dcfC = currentTime;
                        
                        if j~= numOfTimeSteps
                            dD = (dcfN-dcfC);
                        end

                        toDate = AddDate(valueDate,currentTime,'day');
                        tpDate = schedule.payDates(i);
%                         spreadC = model.CMSCMTSpreadDigitalRangeDate(valueDate,toDate,tpDate,tenorLong,tenorShort,ralower,raupper);
                        spreadC = model.CMSCMSSpreadDigitalRangeDate(valueDate,toDate,tpDate,tenorCMS,tenorLong,tenorShort,0.25,ralower,raupper,ralowerS,raupperS);
                        
                        %modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:numOfFactors*(currentTimeIdx-1)+2,:);
%                         idxL = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorLong,modelStateT);
%                         idxS = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorShort,modelStateT);
%                         spreadC = idxL - idxS;
%                         probSpread = model.SRPr(spreadC,spreadN,ralower,raupper,simpleYN,pathSize)*dD; 

                        if simpleYN
                            probSpread = spreadC*dD;
                        else
                            probSpread = spreadC*dD;
%                             probSpread = 0.5*(spreadC + spreadN)*dD;
                            
                        end

                        
                        cD = cD + dD;
                        cP = cP + probSpread;

                        dcfN = dcfC;
                        spreadN = spreadC;

                        %induct backward to the next time step
                        % one time backward induction
                        lastTimeIdx = currentTimeIdx;

                        if startTimeIdx < lastTimeIdx
                            lastTimeIdx = lastTimeIdx -1;
                            currentTimeIdx = lastTimeIdx;
                        end


                    end

                    %% FromToInduction end
                    %% AtDate evaluation
                    % Adding a new cashflow to the payoff

                    dcf = schedule.dayCountFraction(i);
                    
                    if inclusiveYN
                        % we add valuation date
                        cD = schedule.dayCount(i);
                        rangeDays =  security.rangeDays(i);
                        payoffStateA.cashflow = nominal * coupon * dcf * (rangeDays + cP)/cD;
                    else
                        payoffStateA.cashflow = nominal * coupon * dcf * cP/cD;
                    end
                    % eventDate(i) = startDate(i) therefore there is no induction in modelStates
                    % i.e. currentTimeIdx has not changed
                    % no need to update modelStateT
                    % modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:end,:);
                    % since cashflow's are added at the startDate(i) we need to
                    % multiply cashflow with the df(startDate(i), payDate(i))
                    payTime = DateDiff(schedule.payDates(i),valueDate);
                    
                    %event is at startDate so we need to discount the
                    %cashflow first from paydate to startdate
%                     df_sp = model.discountFactor(currentTime, payTime,numMatTime,modelStateT);

%                     for k=1:pathSize
%                         payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_sp(k);
%                     end
                    
                    % process payoff
                    
                    currentNodeIdx = i;
                    
                    % 1st nonCall column
                    nonCall.payoffStates.cashflow(currentNodeIdx,:) = payoffStateA.cashflow;
                    payoffStateA.cashflow_npv = payoffStateA.cashflow * model.DFRisky(payTime/365.0);
                    
%                     payoffStateA.cashflow_npv = model.discountPayoff(currentTime,numMatTime ...
%                     ,modelStateT,payoffStateA.cashflow);
                    nonCall.payoffStates.cashflow_npv(currentNodeIdx,:) = payoffStateA.cashflow_npv;
                    payoffStateA.df = model.DFRisky(payTime/365.0);
%                     payoffStateA.df = model.discountPayoff(currentTime,numMatTime ...
%                     ,modelStateT,1.0*ones(1,pathSize));
                    nonCall.payoffStates.df(currentNodeIdx,:) = payoffStateA.df;
                    
                    nonCall.payoff = nonCall.payoff + payoffStateA.cashflow_npv;
                    
                    nonCall.cashflow_e(currentNodeIdx) = mean(payoffStateA.cashflow);
                    nonCall.rangeProb_e(currentNodeIdx) = mean(nonCall.cashflow_e(currentNodeIdx)/(nominal * coupon * dcf));
                    nonCall.cashflow_npv_e(currentNodeIdx) = mean(payoffStateA.cashflow_npv);

                    nonCall.df_e(currentNodeIdx) = mean(payoffStateA.df);
                    nonCall.df_c(currentNodeIdx) = model.DFRisky(currentTime/365.0);
                    

                end

                nonCall.npv = mean(nonCall.payoff);
                out.pricerInfo.nonCall = nonCall;
                
%                 callable.npv = mean(callable.payoff);
%                 out.pricerInfo.callable = callable;
                out.daycountFraction = schedule.dayCountFraction;
                
        end
        

        
        
        
        
    end
    
end



In [ ]:
classdef DailyWeeklyRiskyPricer_KRWCMS_KTBCMSSpreadDualRA < H_RootObject
    %Pricer(Security,Model,NumMethod)
    %Prcer is constructed from Security, Model, NumMethod 
    
    properties
        m_security
        m_model
        m_numMethod
        overHedgeYN
    end
    
    methods
        function pricer = DailyWeeklyRiskyPricer_KRWCMS_KTBCMSSpreadDualRA(security,model,numMethod,overHedgeYN)
                pricer.m_security = security;
                pricer.m_model = model;
                pricer.m_numMethod = numMethod;
                pricer.overHedgeYN = overHedgeYN;
        end
        
        function out = computePriceMCCallableGeneric(pricer,valueDate,pricerOption)
                security = pricer.m_security;
                numMethod = pricer.m_numMethod;
                model = pricer.m_model;
                schedule = security.couponSchedule;
                callSchedule = security.callSchedule;
                
                % numMethodTimeSteps Generate
                mcOneTimeStep = numMethod.mcOneTimeStep;
                outInfo = numMethod.numMethodTimeStepsGenerateDW(valueDate,schedule,mcOneTimeStep);
            
                numMethodInfo.timeStepsInfo = outInfo;

                % mc random number init
                %pathSize = 2^16;
                pathSize = numMethod.pathSize;

                % generate random numbers and modelstates forward
                rng('default');
                rng(0);

                % LGM2F : two factor
                % CK1F  : one factor
                
                numOfFactors = model.numOfFactor;

                %Z= randn(numOfFactors*outInfo.totalTimeStepsSize,pathSize/2);
                Z= randn(pathSize/2,numOfFactors*outInfo.totalTimeStepsSize);
                Z=[Z;-Z];
                Z=Z';
                
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;

                % we check with the c code, so we use generated random numbers
%                 mc_random = importdata('mc_random.xlsx');
%                 mc_random = importdata('D:\mc_random_number_new_report.txt');
%                 Z= mc_random;
                
                numMethodInfo.pathSize = pathSize;
                numMethodInfo.numOfFactors = numOfFactors;
                numMethodInfo.totalTimeSteps = outInfo.totalTimeSteps;
                numMethodInfo.totalTimeStepsSize = outInfo.totalTimeStepsSize;
                numMethodInfo.Z = Z;



                %numMethod generate model states for the given timeSteps
                modelStates = numMethod.generateModelStatesNF(valueDate,schedule,numMethodInfo,model);

                % numeraire info
                numMatTime = numMethodInfo.totalTimeSteps(end);
                days = DateDiff(schedule.payDates(schedule.numOfPeriod),valueDate);
                numMatTime = max(numMatTime, days);
                dfNumMat = model.DFRisky(numMatTime/365.0);
                
                numeraire.numMatTime = numMatTime;
                numeraire.dfNumMat = dfNumMat;
                
                % induct backward and calculate cashflow
            
                periodCount = schedule.numOfPeriod;
                
                % pricer single info initialize
                
                nonCall = PricerInfo(periodCount +1, pathSize);
                callable = PricerInfo(periodCount +1, pathSize);
                dfCol = PricerInfo(periodCount +1, pathSize);
                
                dummy1 = PricerInfo(periodCount +1, pathSize);
                dummy2 = PricerInfo(periodCount +1, pathSize);
                
                
                % intermediate variables
                payoffStateA.cashflow = zeros(1, pathSize);
                payoffStateA.cashflow_npv = zeros(1, pathSize);
                
            
                % we calculate cashflow backward from maturityDate to valueDate
                % rangeAccrual coupons are added at the startDate of the Period
                % At maturity

                nominal = security.nominal;
                coupon = security.coupon;
                callYN = security.callYN;
                
                tenorCMS = security.tenor;
                
                ralower = security.lower;
                raupper = security.upper;
                
                tenorLong = security.tenorL;
                tenorShort = security.tenorS;
                
                ralowerS = security.lowerS;
                raupperS = security.upperS;
                
                currentTime = numMethodInfo.totalTimeSteps(end);
                totalTimeStepsSize = numMethodInfo.totalTimeStepsSize;
                currentTimeIdx = totalTimeStepsSize;
                
                currentNodeIdx = periodCount + 1;
                
                modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:numOfFactors*(currentTimeIdx-1)+numOfFactors,:);
                
                payoffStateA.cashflow = nominal*ones(1,pathSize);
                
                payTime = DateDiff(schedule.payDates(schedule.numOfPeriod),valueDate);
                df_ep = model.discountFactorRisky(currentTime, payTime,numMatTime,modelStateT);
                for k=1:pathSize
                    payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                end
                
                %process payoff at maturity
                nonCall.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                
                callable.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                
                %df column
                payoffStateA.cashflow = ones(1,pathSize);
                for k=1:pathSize
                    payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                end
                
                dfCol.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                                 
                %dummy column
                foreignModelName = model.foreignModelName;
                
                if strcmp(pricerOption,'PSA')
                    payoffStateA.cashflow = model.CMS_PSA_SimpleDate(model.refModelName,valueDate,currentTime,numMatTime, tenorLong,modelStateT);
                    dummy1.ProcessPayoff(model,currentTime,numeraire, ...
                                 modelStateT,payoffStateA.cashflow,currentNodeIdx);
                    
                    payoffStateA.cashflow = model.CMS_PSA_SimpleDate(foreignModelName,valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                    dummy2.ProcessPayoff(model,currentTime,numeraire, ...
                                 modelStateT,payoffStateA.cashflow,currentNodeIdx);
                
                else
                    payoffStateA.cashflow = model.CMS_SimpleDate(model.refModelName,valueDate,currentTime,numMatTime, tenorLong,modelStateT);
                    dummy1.ProcessPayoff(model,currentTime,numeraire, ...
                                 modelStateT,payoffStateA.cashflow,currentNodeIdx);
                    
                    payoffStateA.cashflow = model.CMS_SimpleDate(foreignModelName,valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                    dummy2.ProcessPayoff(model,currentTime,numeraire, ...
                                 modelStateT,payoffStateA.cashflow,currentNodeIdx);
                    
                end
                

%                 payoffStateA.cashflow = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                
%                 payoffStateA.cashflow = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                
                
                                 
                  

                lastTimeIdx = currentTimeIdx;
                totalTimeSteps = numMethodInfo.timeStepsInfo.totalTimeSteps;
                
                % flag for inclusive schedule
%                 simpleYN = false;
%                 simpleYN = true;
                if pricer.overHedgeYN == true 
                    simpleYN = 2;
                else
                    simpleYN = 1;
                end
                
                for i=periodCount:-1:1
                    % past schedule neglect
                    if DateDiff(valueDate,schedule.payDates(i)) >= 0, continue;end 
                    startTimeIdx = numMethodInfo.timeStepsInfo.startTimeIdx(i);
                    endTimeIdx = numMethodInfo.timeStepsInfo.endTimeIdx(i);

                    inclusiveYN = numMethodInfo.timeStepsInfo.inclusiveYN(i);
                 %% FromToInduction start

                    %modelsStates induct backward(lastTimeIdx to endTimeIdx)
                    while endTimeIdx < lastTimeIdx
                        lastTimeIdx = lastTimeIdx -1;
                        if lastTimeIdx == endTimeIdx break;end
                    end

                    currentTimeIdx = lastTimeIdx;
                    currentTime = totalTimeSteps(currentTimeIdx);
                    % now we are at endTimeIdx
                    % variables for coupon valuation initialized
                    

                    cD = 0; % cumulated day count fraction
                    cP = 0; 
                    dI = 0; % dummy idx for loop
                    dcfC = 0;
                    dcfN = 0;
                    dD = 0;
                    idxCMS = zeros(1,pathSize);
                    
                    idxC = zeros(1,pathSize);
                    idxN = zeros(1,pathSize);
                    
                    idxL = zeros(1,pathSize);
                    idxS = zeros(1,pathSize);
                    
                    spreadC = zeros(1,pathSize);
                    spreadN = zeros(1,pathSize);
                    probSpread = zeros(1,pathSize);
                    
                    numOfTimeSteps = numMethodInfo.timeStepsInfo.numOfTimeSteps(i);
                    
                    % if inclusive then we use simpleYN rangeProb
                    %if inclusive then we do not evaluate the last step but
                    % use fixing instead 
                    endTimeStep = 1;
                    if inclusiveYN
%                         simpleYN = true;
                        simpleYN = 1;
                        endTimeStep = 2;
                        if pricer.overHedgeYN == true 
                            simpleYN = 2;
                        end
                    end
                    
                    for j=numOfTimeSteps:-1:endTimeStep
                        % evaluate dealdescription
                        % last step is omitted
                        currentTime = totalTimeSteps(currentTimeIdx);
                        %dcfC = currentTime/365.0;
                        dcfC = currentTime;
                        
                        if j~= numOfTimeSteps
                            dD = (dcfN-dcfC);
                        end

                        modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:numOfFactors*(currentTimeIdx-1)+numOfFactors,:);
                        
                        if strcmp(pricerOption,'PSA')
                            idxCMS = model.CMS_PSA_SimpleDate(model.refModelName,valueDate,currentTime,numMatTime, tenorCMS,modelStateT);
                            idxC = idxCMS;

%                             idxL = model.CMS_PSA_SimpleDate(foreignModelName,valueDate,currentTime,numMatTime, tenorLong,modelStateT);
                            idxL = model.CMS_PSA_SimpleDate(model.refModelName,valueDate,currentTime,numMatTime, tenorLong,modelStateT);
                            idxS = model.CMS_PSA_SimpleDate(foreignModelName,valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                            spreadC = idxL - idxS;
                        else
                            idxCMS = model.CMS_SimpleDate(model.refModelName,valueDate,currentTime,numMatTime, tenorCMS,modelStateT);
                            idxC = idxCMS;

%                             idxL = model.CMS_SimpleDate(foreignModelName,valueDate,currentTime,numMatTime, tenorLong,modelStateT);
                            idxL = model.CMS_SimpleDate(model.refModelName,valueDate,currentTime,numMatTime, tenorLong,modelStateT);
                            idxS = model.CMS_SimpleDate(foreignModelName,valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                            spreadC = idxL - idxS;
                        end
%                         probSpread = model.SRPr(spreadC,spreadN,ralower,raupper,simpleYN,pathSize)*dD;
                        probSpread = model.DRPr(idxC,idxN,ralower,raupper,spreadC,spreadN,ralowerS,raupperS,simpleYN,pathSize)*dD;
                        cD = cD + dD;
                        cP = cP + probSpread;

                        dcfN = dcfC;
                        
                        idxN = idxC;
                        spreadN = spreadC;

                        %induct backward to the next time step
                        % one time backward induction
                        lastTimeIdx = currentTimeIdx;

                        if startTimeIdx < lastTimeIdx
                            lastTimeIdx = lastTimeIdx -1;
                            currentTimeIdx = lastTimeIdx;
                        end


                    end

                    %% FromToInduction end
        % if the current period is inclusive, then the start date is not evaluated in the above for loop.
		% instead the fixing will be treated separately below.
		% however current_time_idx is updated at the end of the for loop.
		% so we need to update model_states accordingly.
		% if model_states were updated previously then the following additionaly step doesn't affect the result anyway.
                    currentTime = totalTimeSteps(currentTimeIdx);
                    modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:numOfFactors*(currentTimeIdx-1)+numOfFactors,:);
                    
                    %% AtDate evaluation
                    % Adding a new cashflow to the payoff

                    dcf = schedule.dayCountFraction(i);
                    
                    if inclusiveYN
                        % we add valuation date
                        cD = schedule.dayCount(i);
                        rangeDays =  security.rangeDays(i);
                        payoffStateA.cashflow = nominal * coupon * dcf * (rangeDays + cP)/cD;
                    else
                        payoffStateA.cashflow = nominal * coupon * dcf * cP/cD;
                    end
                    % eventDate(i) = startDate(i) therefore there is no induction in modelStates
                    % i.e. currentTimeIdx has not changed
                    % no need to update modelStateT
                    % modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:end,:);
                    % since cashflow's are added at the startDate(i) we need to
                    % multiply cashflow with the df(startDate(i), payDate(i))
                    payTime = DateDiff(schedule.payDates(i),valueDate);
                    
                    %event is at startDate so we need to discount the
                    %cashflow first from paydate to startdate
                    df_ep = model.discountFactorRisky(currentTime, payTime,numMatTime,modelStateT);

                    for k=1:pathSize
                        payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                    end
                    
                    % process payoff
                    
                    currentNodeIdx = i;
                    
                    nonCall.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                
                    callable.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                    
                    %df column
                    payoffStateA.cashflow = ones(1,pathSize);
                    for k=1:pathSize
                        payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                    end
                    
                    dfCol.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                                 
                    %dummy column
                    if strcmp(pricerOption,'PSA')
                        payoffStateA.cashflow = model.CMS_PSA_SimpleDate(model.refModelName,valueDate,currentTime,numMatTime, tenorLong,modelStateT);
                        dummy1.ProcessPayoff(model,currentTime,numeraire, ...
                                         modelStateT,payoffStateA.cashflow,currentNodeIdx);



                        payoffStateA.cashflow = model.CMS_PSA_SimpleDate(foreignModelName,valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                        dummy2.ProcessPayoff(model,currentTime,numeraire, ...
                                         modelStateT,payoffStateA.cashflow,currentNodeIdx);
                    
                    else
                        payoffStateA.cashflow = model.CMS_SimpleDate(model.refModelName,valueDate,currentTime,numMatTime, tenorLong,modelStateT);
                        dummy1.ProcessPayoff(model,currentTime,numeraire, ...
                                         modelStateT,payoffStateA.cashflow,currentNodeIdx);



                        payoffStateA.cashflow = model.CMS_SimpleDate(foreignModelName,valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                        dummy2.ProcessPayoff(model,currentTime,numeraire, ...
                                         modelStateT,payoffStateA.cashflow,currentNodeIdx);
                        
                    end

                                 
                    
                    
                    
                    % if exercise is possible
                    if i == 1
                        dummaaa = 1;
                    end
                    
                    % call event happens  at the start of the period
                    % at call date, by default, call option is expired
                    
                    if callYN(i) && (DateDiff(valueDate,callSchedule.resetDates(i)) < 0)
                        % 1.undiscount callable.payoff
                        % 2.build the state variables matrix
                        % 3.regress 1's with the given state variables
                        % (model states 1,2)
                        % 4. exercise per each path in
                        % discount the new payoff and assign into callable
                        % column
                        
                        %1 undiscount callable.payoff and get the
                        %contiValue
                       
                        %exercise value
                        exerciseValue = security.nominal*ones(1,pathSize);
                        payTime = DateDiff(callSchedule.payDates(i),valueDate);
                        %event is at startDate so we need to discount the
                        %cashflow first from paydate to startdate
                        df_ep = model.discountFactorRisky(currentTime, payTime,numMatTime,modelStateT);
                        for k=1:pathSize
                            exerciseValue(k) = exerciseValue(k) * df_ep(k);
                        end
                        
                        contiValue = zeros(1,pathSize);
                        onePayoff = ones(1,pathSize);
                        onePayoffD = model.discountPayoffRisky(currentTime,numMatTime ...
                        ,modelStateT,onePayoff);
                        contiValue = callable.payoff./onePayoffD;
                        
                        %Nominal Regression test
                        contiValue = contiValue/10000;
                        
                        % build the state variabls matrix
                        if numMethod.regressionOrder == 3
%                             dimensionLS = (numOfFactors + 1)*(numOfFactors + 2)/2;

                            decisionVar = zeros(numOfFactors,size(modelStateT,2));

                            for idxLS =1:numOfFactors
                                decisionVar(idxLS,:) = modelStateT(idxLS,:); 
                            end
                            
                            % calculate number of dimension first
                            % 0th order
                            dimensionLS = 1;
                            
                            % 1st order
                            for idxLS=1:numOfFactors
                                dimensionLS = dimensionLS + 1;
                            end
                            
                            %2nd order
                            for idxLS1=1:numOfFactors
                                for idxLS2=idxLS1:numOfFactors
                                    dimensionLS = dimensionLS + 1;
                                end
                            end
                            
                            %3rd order
                            for idxLS1=1:numOfFactors
                                for idxLS2=idxLS1:numOfFactors
                                    for idxLS3=idxLS2:numOfFactors
                                        dimensionLS = dimensionLS + 1;
                                    end
                                end
                            end

                            
    %                         decisionVar1 = modelStateT(1,:);
    %                         decisionVar2 = modelStateT(2,:);
                            % we use upto 2nd order polynomial as regression
                            % equatioin

                            regMatrix = ones(pathSize,dimensionLS);

                            for idx_r=1:pathSize
                                for idxLS=1:numOfFactors
                                    regMatrix(idx_r,idxLS+1) = decisionVar(idxLS,idx_r);
                                end

                                tempIdx = 0;
                                for idxLS1=1:numOfFactors
                                    for idxLS2=idxLS1:numOfFactors
                                        tempIdx = tempIdx + 1;
                                        regMatrix(idx_r,numOfFactors+1+tempIdx) = decisionVar(idxLS1,idx_r)*decisionVar(idxLS2,idx_r);
                                    end
                                end
                                
                                tempIdx2 = 0;
                                for idxLS1=1:numOfFactors
                                    for idxLS2=idxLS1:numOfFactors
                                        for idxLS3=idxLS2:numOfFactors
                                            tempIdx2 = tempIdx2 + 1;
                                            regMatrix(idx_r,numOfFactors+1+tempIdx + tempIdx2) = decisionVar(idxLS1,idx_r)*decisionVar(idxLS2,idx_r)*decisionVar(idxLS3,idx_r);
                                        end
                                    end
                                end
                                

                            end
                        
                        else
                            dimensionLS = (numOfFactors + 1)*(numOfFactors + 2)/2;

                            decisionVar = zeros(numOfFactors,size(modelStateT,2));

                            for idxLS =1:numOfFactors
                                decisionVar(idxLS,:) = modelStateT(idxLS,:); 
                            end
    %                         decisionVar1 = modelStateT(1,:);
    %                         decisionVar2 = modelStateT(2,:);
                            % we use upto 2nd order polynomial as regression
                            % equatioin

                            regMatrix = ones(pathSize,dimensionLS);

                            for idx_r=1:pathSize
                                for idxLS=1:numOfFactors
                                    regMatrix(idx_r,idxLS+1) = decisionVar(idxLS,idx_r);
                                end

                                tempIdx = 1;
                                for idxLS1=1:numOfFactors
                                    for idxLS2=idxLS1:numOfFactors
                                        regMatrix(idx_r,numOfFactors+1+tempIdx) = decisionVar(idxLS1,idx_r)*decisionVar(idxLS2,idx_r);
                                        tempIdx = tempIdx + 1;
                                    end
                                end

                            end
                        end
                        % simple regression <- for path dependent option
                        % we should change it
                        % beta  : regression coefficients
                        beta =zeros(dimensionLS,1);
                        beta = regress(contiValue',regMatrix);
                        regContiValue = regMatrix*beta;
                       
                         %Nominal Regression test
                        regContiValue = regContiValue * 10000;
                        
                        exercisedPayoff = regContiValue;
                        % if regContiValue is bigger than exerciseValue
                        % then issue exercise call option
                        for idx_r=1:pathSize
                            if regContiValue(idx_r) >= exerciseValue(idx_r) 
                                exercisedPayoff(idx_r) = exerciseValue(idx_r);
                            end
                        end
                        
                        % we discount the new payoff and 
                        % assign it into the existing callable payoff
                        callable.payoff = model.discountPayoffRisky(currentTime,numMatTime ...
                        ,modelStateT,exercisedPayoff);
                        
                    end
                end

                nonCall.npv = mean(nonCall.payoff);
                out.nonCall = nonCall;
                callable.npv = mean(callable.payoff);
                dfCol.npv = mean(dfCol.payoff);
                dummy1.npv = mean(dummy1.npv);
                dummy2.npv = mean(dummy2.npv);
                out.callable = callable;
                out.dfCol = dfCol;
                out.dummy1 = dummy1;
                out.dummy2 = dummy2;
                out.daycountFraction = schedule.dayCountFraction;
                
        end
        
        
        function out = computePriceClosedSimple(pricer,valueDate)
                security = pricer.m_security;
                numMethod = pricer.m_numMethod;
                model = pricer.m_model;
                schedule = security.couponSchedule;
                
                % numMethodTimeSteps Generate
                mcOneTimeStep = numMethod.mcOneTimeStep;
                outInfo = numMethod.numMethodTimeStepsGenerateDW(valueDate,schedule,mcOneTimeStep);
            
                numMethodInfo.timeStepsInfo = outInfo;

                % mc random number init
                %pathSize = 2^16;
                pathSize = numMethod.pathSize;
                
                pathSize = 1;
                
%                 % generate random numbers and modelstates forward
%                 rng('default');
%                 rng(0);
% 
                numOfFactors = model.numOfFactor;
% 
%                 %Z= randn(numOfFactors*outInfo.totalTimeStepsSize,pathSize/2);
%                 Z= randn(pathSize/2,numOfFactors*outInfo.totalTimeStepsSize);
%                 Z=[Z;-Z];
%                 Z=Z';
%                 
%                 
%                 %TimeInversion
%                 sizeZ = size(Z,1);
%                 A = zeros(sizeZ,sizeZ);
%                 for i=1:sizeZ
%                     A(i,sizeZ + 1 -i) = 1;
%                 end
%                 Z = A*Z;
% 
%                 % we check with the c, so we use generated random numbers
% %                 mc_random = importdata('mc_random.xlsx');
% %                 mc_random = importdata('D:\mc_random_number_new_report.txt');
% %                 Z= mc_random;
%                 
                numMethodInfo.pathSize = pathSize;
                numMethodInfo.numOfFactors = numOfFactors;
                numMethodInfo.totalTimeSteps = outInfo.totalTimeSteps;
                numMethodInfo.totalTimeStepsSize = outInfo.totalTimeStepsSize;
%                 numMethodInfo.Z = Z;
% 
% 
% 
%                 %generate model states for the given timeSteps
%                 modelStates = numMethod.generateModelStates(valueDate,schedule,numMethodInfo,model);

                % numeraire info
                numMatTime = numMethodInfo.totalTimeSteps(end);
                dfNumMat = model.DFRisky(numMatTime/365.0);
                
                % induct backward and calculate cashflow
            
                periodCount = schedule.numOfPeriod;
                
                % cashflow columns initialize
                % 1st nonCall
                nonCall.npv = 0;
                nonCall.cashflow_e = zeros(periodCount + 1,1);
                nonCall.cashflow_npv_e = zeros(periodCount + 1,1);
                nonCall.rangeProb_e = zeros(periodCount + 1,1);
                nonCall.df_e = zeros(periodCount + 1,1);
                nonCall.df_c = zeros(periodCount + 1,1);

                % debug information, cashflow & npv per path
                nonCall.payoff = zeros(1,pathSize);
                nonCall.payoffStates.cashflow = zeros(periodCount+1,pathSize);
                nonCall.payoffStates.cashflow_npv = zeros(periodCount+1,pathSize);
                nonCall.payoffStates.df = zeros(periodCount + 1,pathSize);
                
                % intermediate variables
                payoffStateA.cashflow = zeros(1, pathSize);
                payoffStateA.cashflow_npv = zeros(1, pathSize);
                payoffStateA.df = zeros(1,pathSize);
                
                % we calculate cashflow backward from maturityDate to valueDate
                % rangeAccrual coupons are added at the startDate of the Period
                % At maturity

                nominal = security.nominal;
                coupon = security.coupon;
                callYN = security.callYN;
                
                tenorCMS = security.tenor;
                
                ralower = security.lower;
                raupper = security.upper;
                
                tenorLong = security.tenorL;
                tenorShort = security.tenorS;
                
                ralowerS = security.lowerS;
                raupperS = security.upperS;
                
                currentTime = numMethodInfo.totalTimeSteps(end);
                totalTimeSteps = numMethodInfo.timeStepsInfo.totalTimeSteps;
                totalTimeStepsSize = numMethodInfo.timeStepsInfo.totalTimeStepsSize;
                if totalTimeStepsSize == 0 || (totalTimeStepsSize == 1 && totalTimeSteps(1) == 0) 
                    out.nonCall = nonCall;
                    return;
                end
                
                currentTimeIdx = totalTimeStepsSize;
                
                currentNodeIdx = periodCount + 1;
                
%                 modelStateT = modelStates(numOfFactors*(totalTimeStepsSize-1)+1:numOfFactors*(totalTimeStepsSize-1)+2,:);
                payoffStateA.cashflow = nominal*ones(1,pathSize);
                %process payoff at maturity
                nonCall.payoffStates.cashflow(currentNodeIdx,:) = payoffStateA.cashflow;
                
%                 payoffStateA.cashflow_npv = model.discountPayoff(currentTime,numMatTime ...
%                     ,modelStateT,payoffStateA.cashflow);
%                 
                payoffStateA.cashflow_npv = model.DFRisky(currentTime/365.0)*payoffStateA.cashflow;
                
                nonCall.payoffStates.cashflow_npv(currentNodeIdx,:) = payoffStateA.cashflow_npv ;
                
%                 payoffStateA.df = model.discountPayoff(currentTime,numMatTime ...
%                     ,modelStateT,1.0*ones(1,pathSize));
                payoffStateA.df = model.DFRisky(currentTime/365.0) *ones(1,pathSize);
                    

                nonCall.payoffStates.df(currentNodeIdx,:) = payoffStateA.df;
                nonCall.payoff = nonCall.payoff + payoffStateA.cashflow_npv;
                
                nonCall.cashflow_e(currentNodeIdx) = mean(payoffStateA.cashflow);
                nonCall.cashflow_npv_e(currentNodeIdx) = mean(payoffStateA.cashflow_npv);
                nonCall.df_e(currentNodeIdx) = mean(payoffStateA.df);
                nonCall.df_c(currentNodeIdx) = model.DFRisky(currentTime/365.0);
                
%                 % callable at maturity is the same as noncallable
%                 callable.payoffStates.cashflow(currentNodeIdx,:) = nonCall.payoffStates.cashflow(currentNodeIdx,:);
%                 callable.payoffStates.cashflow_npv(currentNodeIdx,:) = nonCall.payoffStates.cashflow_npv(currentNodeIdx,:);
%                 callable.payoff = callable.payoff + callable.payoffStates.cashflow_npv(currentNodeIdx,:);
%                 
%                 callable.cashflow_e(currentNodeIdx) = nonCall.cashflow_e(currentNodeIdx);
%                 callable.cashflow_npv_e(currentNodeIdx) = nonCall.cashflow_npv_e(currentNodeIdx);
                  
                lastTimeIdx = currentTimeIdx;
                
                
                % flag for inclusive schedule
%                 simpleYN = false;
                simpleYN = 1;
                
                for i=periodCount:-1:1
                    % past schedule neglect
                    if DateDiff(valueDate,schedule.payDates(i)) >= 0, continue;end 
                    startTimeIdx = numMethodInfo.timeStepsInfo.startTimeIdx(i);
                    endTimeIdx = numMethodInfo.timeStepsInfo.endTimeIdx(i);

                    inclusiveYN = numMethodInfo.timeStepsInfo.inclusiveYN(i);
                     %% FromToInduction start

                    %modelsStates induct backward(lastTimeIdx to endTimeIdx)
                    while endTimeIdx < lastTimeIdx
                        lastTimeIdx = lastTimeIdx -1;
                        if lastTimeIdx == endTimeIdx break;end
                    end

                    currentTimeIdx = lastTimeIdx;
                    currentTime = totalTimeSteps(currentTimeIdx);
                    % now we are at endTimeIdx
                    % variables for coupon valuation initialized
%                     ralower = security.lower;
%                     raupper = security.upper;
%                     tenorLong = security.tenorL;
%                     tenorShort = security.tenorS;

                    cD = 0; % cumulated day count fraction
                    cP = 0; 
                    dI = 0; % dummy idx for loop
                    dcfC = 0;
                    dcfN = 0;
                    dD = 0;
                    idxCMS = zeros(1,pathSize);
                    
                    idxC = zeros(1,pathSize);
                    idxN = zeros(1,pathSize);
                    
                    idxL = zeros(1,pathSize);
                    idxS = zeros(1,pathSize);
                    spreadC = zeros(1,pathSize);
                    spreadN = zeros(1,pathSize);
                    probSpread = zeros(1,pathSize);
                    
                    numOfTimeSteps = numMethodInfo.timeStepsInfo.numOfTimeSteps(i);
                    
                    % if inclusive then we use simpleYN rangeProb
                    %if inclusive then we do not evaluate the last step but
                    % use fixing instead 
                    endTimeStep = 1;
                    if inclusiveYN
%                         simpleYN = true;
                        simpleYN = 1;
                        endTimeStep = 2;
                    end
                    
                    for j=numOfTimeSteps:-1:endTimeStep
                        % evaluate dealdescription
                        % last step is omitted
                        currentTime = totalTimeSteps(currentTimeIdx);
                        %dcfC = currentTime/365.0;
                        dcfC = currentTime;
                        
                        if j~= numOfTimeSteps
                            dD = (dcfN-dcfC);
                        end

                        toDate = AddDate(valueDate,currentTime,'day');
                        tpDate = schedule.payDates(i);
%                         spreadC = model.CMSCMTSpreadDigitalRangeDate(valueDate,toDate,tpDate,tenorLong,tenorShort,ralower,raupper);
                        spreadC = model.CMSRACMTSpreadDigitalRangeDate(valueDate,toDate,tpDate,tenorCMS,tenorLong,tenorShort,0.25,ralower,raupper,ralowerS,raupperS);
                        
                        %modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:numOfFactors*(currentTimeIdx-1)+2,:);
%                         idxL = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorLong,modelStateT);
%                         idxS = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorShort,modelStateT);
%                         spreadC = idxL - idxS;
%                         probSpread = model.SRPr(spreadC,spreadN,ralower,raupper,simpleYN,pathSize)*dD; 

                        if simpleYN
                            probSpread = spreadC*dD;
                        else
                            probSpread = spreadC*dD;
%                             probSpread = 0.5*(spreadC + spreadN)*dD;
                            
                        end

                        
                        cD = cD + dD;
                        cP = cP + probSpread;

                        dcfN = dcfC;
                        spreadN = spreadC;

                        %induct backward to the next time step
                        % one time backward induction
                        lastTimeIdx = currentTimeIdx;

                        if startTimeIdx < lastTimeIdx
                            lastTimeIdx = lastTimeIdx -1;
                            currentTimeIdx = lastTimeIdx;
                        end


                    end

                    %% FromToInduction end
                    %% AtDate evaluation
                    % Adding a new cashflow to the payoff

                    dcf = schedule.dayCountFraction(i);
                    
                    if inclusiveYN
                        % we add valuation date
                        cD = schedule.dayCount(i);
                        rangeDays =  security.rangeDays(i);
                        payoffStateA.cashflow = nominal * coupon * dcf * (rangeDays + cP)/cD;
                    else
                        payoffStateA.cashflow = nominal * coupon * dcf * cP/cD;
                    end
                    % eventDate(i) = startDate(i) therefore there is no induction in modelStates
                    % i.e. currentTimeIdx has not changed
                    % no need to update modelStateT
                    % modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:end,:);
                    % since cashflow's are added at the startDate(i) we need to
                    % multiply cashflow with the df(startDate(i), payDate(i))
                    payTime = DateDiff(schedule.payDates(i),valueDate);
                    
                    %event is at startDate so we need to discount the
                    %cashflow first from paydate to startdate
%                     df_sp = model.discountFactor(currentTime, payTime,numMatTime,modelStateT);

%                     for k=1:pathSize
%                         payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_sp(k);
%                     end
                    
                    % process payoff
                    
                    currentNodeIdx = i;
                    
                    % 1st nonCall column
                    nonCall.payoffStates.cashflow(currentNodeIdx,:) = payoffStateA.cashflow;
                    payoffStateA.cashflow_npv = payoffStateA.cashflow * model.DFRisky(payTime/365.0);
                    
%                     payoffStateA.cashflow_npv = model.discountPayoff(currentTime,numMatTime ...
%                     ,modelStateT,payoffStateA.cashflow);
                    nonCall.payoffStates.cashflow_npv(currentNodeIdx,:) = payoffStateA.cashflow_npv;
                    payoffStateA.df = model.DFRisky(payTime/365.0);
%                     payoffStateA.df = model.discountPayoff(currentTime,numMatTime ...
%                     ,modelStateT,1.0*ones(1,pathSize));
                    nonCall.payoffStates.df(currentNodeIdx,:) = payoffStateA.df;
                    
                    nonCall.payoff = nonCall.payoff + payoffStateA.cashflow_npv;
                    
                    nonCall.cashflow_e(currentNodeIdx) = mean(payoffStateA.cashflow);
                    nonCall.rangeProb_e(currentNodeIdx) = mean(nonCall.cashflow_e(currentNodeIdx)/(nominal * coupon * dcf));
                    nonCall.cashflow_npv_e(currentNodeIdx) = mean(payoffStateA.cashflow_npv);

                    nonCall.df_e(currentNodeIdx) = mean(payoffStateA.df);
                    nonCall.df_c(currentNodeIdx) = model.DFRisky(currentTime/365.0);
                    

                end

                nonCall.npv = mean(nonCall.payoff);
                out.pricerInfo.nonCall = nonCall;
                
%                 callable.npv = mean(callable.payoff);
%                 out.pricerInfo.callable = callable;
                out.daycountFraction = schedule.dayCountFraction;
                
        end
        

        
        
        
        
    end
    
end



In [ ]:
classdef DailyWeeklyRiskyPricer_KRWCMS_KRWCMSKTBCMSSpreadDualRA < H_RootObject
    %Pricer(Security,Model,NumMethod)
    %Prcer is constructed from Security, Model, NumMethod 
    
    properties
        m_security
        m_model
        m_numMethod
        overHedgeYN
    end
    
    methods
        function pricer = DailyWeeklyRiskyPricer_KRWCMS_KRWCMSKTBCMSSpreadDualRA(security,model,numMethod,overHedgeYN)
                pricer.m_security = security;
                pricer.m_model = model;
                pricer.m_numMethod = numMethod;
                pricer.overHedgeYN = overHedgeYN;
        end
        
        function out = computePriceMCCallableGeneric(pricer,valueDate,pricerOption)
                security = pricer.m_security;
                numMethod = pricer.m_numMethod;
                model = pricer.m_model;
                schedule = security.couponSchedule;
                callSchedule = security.callSchedule;
                
                % numMethodTimeSteps Generate
                mcOneTimeStep = numMethod.mcOneTimeStep;
                outInfo = numMethod.numMethodTimeStepsGenerateDW(valueDate,schedule,mcOneTimeStep);
            
                numMethodInfo.timeStepsInfo = outInfo;

                % mc random number init
                %pathSize = 2^16;
                pathSize = numMethod.pathSize;

                % generate random numbers and modelstates forward
                rng('default');
                rng(0);

                % LGM2F : two factor
                % CK1F  : one factor
                
                numOfFactors = model.numOfFactor;

                %Z= randn(numOfFactors*outInfo.totalTimeStepsSize,pathSize/2);
                Z= randn(pathSize/2,numOfFactors*outInfo.totalTimeStepsSize);
                Z=[Z;-Z];
                Z=Z';
                
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;

                % we check with the c code, so we use generated random numbers
%                 mc_random = importdata('mc_random.xlsx');
%                 mc_random = importdata('D:\mc_random_number_new_report.txt');
%                 Z= mc_random;
                
                numMethodInfo.pathSize = pathSize;
                numMethodInfo.numOfFactors = numOfFactors;
                numMethodInfo.totalTimeSteps = outInfo.totalTimeSteps;
                numMethodInfo.totalTimeStepsSize = outInfo.totalTimeStepsSize;
                numMethodInfo.Z = Z;



                %numMethod generate model states for the given timeSteps
                modelStates = numMethod.generateModelStatesNF(valueDate,schedule,numMethodInfo,model);

                % numeraire info
                numMatTime = numMethodInfo.totalTimeSteps(end);
                days = DateDiff(schedule.payDates(schedule.numOfPeriod),valueDate);
                numMatTime = max(numMatTime, days);
                dfNumMat = model.DFRisky(numMatTime/365.0);
                
                numeraire.numMatTime = numMatTime;
                numeraire.dfNumMat = dfNumMat;
                
                % induct backward and calculate cashflow
            
                periodCount = schedule.numOfPeriod;
                
                % pricer single info initialize
                
                nonCall = PricerInfo(periodCount +1, pathSize);
                callable = PricerInfo(periodCount +1, pathSize);
                dfCol = PricerInfo(periodCount +1, pathSize);
                
                dummy1 = PricerInfo(periodCount +1, pathSize);
                dummy2 = PricerInfo(periodCount +1, pathSize);
                
                
                % intermediate variables
                payoffStateA.cashflow = zeros(1, pathSize);
                payoffStateA.cashflow_npv = zeros(1, pathSize);
                
            
                % we calculate cashflow backward from maturityDate to valueDate
                % rangeAccrual coupons are added at the startDate of the Period
                % At maturity

                nominal = security.nominal;
                coupon = security.coupon;
                callYN = security.callYN;
                
                tenorCMS = security.tenor;
                
                ralower = security.lower;
                raupper = security.upper;
                
                tenorLong = security.tenorL;
                tenorShort = security.tenorS;
                
                ralowerS = security.lowerS;
                raupperS = security.upperS;
                
                currentTime = numMethodInfo.totalTimeSteps(end);
                totalTimeStepsSize = numMethodInfo.totalTimeStepsSize;
                currentTimeIdx = totalTimeStepsSize;
                
                currentNodeIdx = periodCount + 1;
                
                modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:numOfFactors*(currentTimeIdx-1)+numOfFactors,:);
                
                payoffStateA.cashflow = nominal*ones(1,pathSize);
                
                payTime = DateDiff(schedule.payDates(schedule.numOfPeriod),valueDate);
                df_ep = model.discountFactorRisky(currentTime, payTime,numMatTime,modelStateT);
                for k=1:pathSize
                    payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                end
                
                %process payoff at maturity
                nonCall.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                
                callable.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                
                %df column
                payoffStateA.cashflow = ones(1,pathSize);
                for k=1:pathSize
                    payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                end
                
                dfCol.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                                 
                %dummy column
                foreignModelName = model.foreignModelName;
                
                if strcmp(pricerOption,'PSA')
                    payoffStateA.cashflow = model.CMS_PSA_SimpleDate(model.refModelName,valueDate,currentTime,numMatTime, tenorLong,modelStateT);
                    dummy1.ProcessPayoff(model,currentTime,numeraire, ...
                                 modelStateT,payoffStateA.cashflow,currentNodeIdx);
                    
                    payoffStateA.cashflow = model.CMS_PSA_SimpleDate(foreignModelName,valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                    dummy2.ProcessPayoff(model,currentTime,numeraire, ...
                                 modelStateT,payoffStateA.cashflow,currentNodeIdx);
                
                else
                    payoffStateA.cashflow = model.CMS_SimpleDate(model.refModelName,valueDate,currentTime,numMatTime, tenorLong,modelStateT);
                    dummy1.ProcessPayoff(model,currentTime,numeraire, ...
                                 modelStateT,payoffStateA.cashflow,currentNodeIdx);
                    
                    payoffStateA.cashflow = model.CMS_SimpleDate(foreignModelName,valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                    dummy2.ProcessPayoff(model,currentTime,numeraire, ...
                                 modelStateT,payoffStateA.cashflow,currentNodeIdx);
                    
                end
                

%                 payoffStateA.cashflow = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                
%                 payoffStateA.cashflow = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                
                
                                 
                  

                lastTimeIdx = currentTimeIdx;
                totalTimeSteps = numMethodInfo.timeStepsInfo.totalTimeSteps;
                
                % flag for inclusive schedule
%                 simpleYN = false;
%                 simpleYN = true;
                if pricer.overHedgeYN == true 
                    simpleYN = 2;
                else
                    simpleYN = 1;
                end
                
                for i=periodCount:-1:1
                    % past schedule neglect
                    if DateDiff(valueDate,schedule.payDates(i)) >= 0, continue;end 
                    startTimeIdx = numMethodInfo.timeStepsInfo.startTimeIdx(i);
                    endTimeIdx = numMethodInfo.timeStepsInfo.endTimeIdx(i);

                    inclusiveYN = numMethodInfo.timeStepsInfo.inclusiveYN(i);
                 %% FromToInduction start

                    %modelsStates induct backward(lastTimeIdx to endTimeIdx)
                    while endTimeIdx < lastTimeIdx
                        lastTimeIdx = lastTimeIdx -1;
                        if lastTimeIdx == endTimeIdx break;end
                    end

                    currentTimeIdx = lastTimeIdx;
                    currentTime = totalTimeSteps(currentTimeIdx);
                    % now we are at endTimeIdx
                    % variables for coupon valuation initialized
                    

                    cD = 0; % cumulated day count fraction
                    cP = 0; 
                    dI = 0; % dummy idx for loop
                    dcfC = 0;
                    dcfN = 0;
                    dD = 0;
                    idxCMS = zeros(1,pathSize);
                    
                    idxC = zeros(1,pathSize);
                    idxN = zeros(1,pathSize);
                    
                    idxL = zeros(1,pathSize);
                    idxS = zeros(1,pathSize);
                    
                    spreadC = zeros(1,pathSize);
                    spreadN = zeros(1,pathSize);
                    probSpread = zeros(1,pathSize);
                    
                    numOfTimeSteps = numMethodInfo.timeStepsInfo.numOfTimeSteps(i);
                    
                    % if inclusive then we use simpleYN rangeProb
                    %if inclusive then we do not evaluate the last step but
                    % use fixing instead 
                    endTimeStep = 1;
                    if inclusiveYN
%                         simpleYN = true;
                        simpleYN = 1;
                        endTimeStep = 2;
                        if pricer.overHedgeYN == true 
                            simpleYN = 2;
                        end
                    end
                    
                    for j=numOfTimeSteps:-1:endTimeStep
                        % evaluate dealdescription
                        % last step is omitted
                        currentTime = totalTimeSteps(currentTimeIdx);
                        %dcfC = currentTime/365.0;
                        dcfC = currentTime;
                        
                        if j~= numOfTimeSteps
                            dD = (dcfN-dcfC);
                        end

                        modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:numOfFactors*(currentTimeIdx-1)+numOfFactors,:);
                        
                        if strcmp(pricerOption,'PSA')
                            idxCMS = model.CMS_PSA_SimpleDate(model.refModelName,valueDate,currentTime,numMatTime, tenorCMS,modelStateT);
                            idxC = idxCMS;

                            idxL = model.CMS_PSA_SimpleDate(foreignModelName,valueDate,currentTime,numMatTime, tenorLong,modelStateT);
                            idxS = model.CMS_PSA_SimpleDate(foreignModelName,valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                            spreadC = idxL - idxS;
                        else
                            idxCMS = model.CMS_SimpleDate(model.refModelName,valueDate,currentTime,numMatTime, tenorCMS,modelStateT);
                            idxC = idxCMS;

                            idxL = model.CMS_SimpleDate(foreignModelName,valueDate,currentTime,numMatTime, tenorLong,modelStateT);
                            idxS = model.CMS_SimpleDate(foreignModelName,valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                            spreadC = idxL - idxS;
                        end
%                         probSpread = model.SRPr(spreadC,spreadN,ralower,raupper,simpleYN,pathSize)*dD;
                        probSpread = model.DRPr(idxC,idxN,ralower,raupper,spreadC,spreadN,ralowerS,raupperS,simpleYN,pathSize)*dD;
                        cD = cD + dD;
                        cP = cP + probSpread;

                        dcfN = dcfC;
                        
                        idxN = idxC;
                        spreadN = spreadC;

                        %induct backward to the next time step
                        % one time backward induction
                        lastTimeIdx = currentTimeIdx;

                        if startTimeIdx < lastTimeIdx
                            lastTimeIdx = lastTimeIdx -1;
                            currentTimeIdx = lastTimeIdx;
                        end


                    end

                    %% FromToInduction end
        % if the current period is inclusive, then the start date is not evaluated in the above for loop.
		% instead the fixing will be treated separately below.
		% however current_time_idx is updated at the end of the for loop.
		% so we need to update model_states accordingly.
		% if model_states were updated previously then the following additionaly step doesn't affect the result anyway.
                    currentTime = totalTimeSteps(currentTimeIdx);
                    modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:numOfFactors*(currentTimeIdx-1)+numOfFactors,:);
                    
                    %% AtDate evaluation
                    % Adding a new cashflow to the payoff

                    dcf = schedule.dayCountFraction(i);
                    
                    if inclusiveYN
                        % we add valuation date
                        cD = schedule.dayCount(i);
                        rangeDays =  security.rangeDays(i);
                        payoffStateA.cashflow = nominal * coupon * dcf * (rangeDays + cP)/cD;
                    else
                        payoffStateA.cashflow = nominal * coupon * dcf * cP/cD;
                    end
                    % eventDate(i) = startDate(i) therefore there is no induction in modelStates
                    % i.e. currentTimeIdx has not changed
                    % no need to update modelStateT
                    % modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:end,:);
                    % since cashflow's are added at the startDate(i) we need to
                    % multiply cashflow with the df(startDate(i), payDate(i))
                    payTime = DateDiff(schedule.payDates(i),valueDate);
                    
                    %event is at startDate so we need to discount the
                    %cashflow first from paydate to startdate
                    df_ep = model.discountFactorRisky(currentTime, payTime,numMatTime,modelStateT);

                    for k=1:pathSize
                        payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                    end
                    
                    % process payoff
                    
                    currentNodeIdx = i;
                    
                    nonCall.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                
                    callable.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                    
                    %df column
                    payoffStateA.cashflow = ones(1,pathSize);
                    for k=1:pathSize
                        payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                    end
                    
                    dfCol.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                                 
                    %dummy column
                    if strcmp(pricerOption,'PSA')
                        payoffStateA.cashflow = model.CMS_PSA_SimpleDate(model.refModelName,valueDate,currentTime,numMatTime, tenorLong,modelStateT);
                        dummy1.ProcessPayoff(model,currentTime,numeraire, ...
                                         modelStateT,payoffStateA.cashflow,currentNodeIdx);



                        payoffStateA.cashflow = model.CMS_PSA_SimpleDate(foreignModelName,valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                        dummy2.ProcessPayoff(model,currentTime,numeraire, ...
                                         modelStateT,payoffStateA.cashflow,currentNodeIdx);
                    
                    else
                        payoffStateA.cashflow = model.CMS_SimpleDate(model.refModelName,valueDate,currentTime,numMatTime, tenorLong,modelStateT);
                        dummy1.ProcessPayoff(model,currentTime,numeraire, ...
                                         modelStateT,payoffStateA.cashflow,currentNodeIdx);



                        payoffStateA.cashflow = model.CMS_SimpleDate(foreignModelName,valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                        dummy2.ProcessPayoff(model,currentTime,numeraire, ...
                                         modelStateT,payoffStateA.cashflow,currentNodeIdx);
                        
                    end

                                 
                    
                    
                    
                    % if exercise is possible
                    if i == 1
                        dummaaa = 1;
                    end
                    
                    % call event happens  at the start of the period
                    % at call date, by default, call option is expired
                    
                    if callYN(i) && (DateDiff(valueDate,callSchedule.resetDates(i)) < 0)
                        % 1.undiscount callable.payoff
                        % 2.build the state variables matrix
                        % 3.regress 1's with the given state variables
                        % (model states 1,2)
                        % 4. exercise per each path in
                        % discount the new payoff and assign into callable
                        % column
                        
                        %1 undiscount callable.payoff and get the
                        %contiValue
                       
                        %exercise value
                        exerciseValue = security.nominal*ones(1,pathSize);
                        payTime = DateDiff(callSchedule.payDates(i),valueDate);
                        %event is at startDate so we need to discount the
                        %cashflow first from paydate to startdate
                        df_ep = model.discountFactorRisky(currentTime, payTime,numMatTime,modelStateT);
                        for k=1:pathSize
                            exerciseValue(k) = exerciseValue(k) * df_ep(k);
                        end
                        
                        contiValue = zeros(1,pathSize);
                        onePayoff = ones(1,pathSize);
                        onePayoffD = model.discountPayoffRisky(currentTime,numMatTime ...
                        ,modelStateT,onePayoff);
                        contiValue = callable.payoff./onePayoffD;
                        
                        %Nominal Regression test
                        contiValue = contiValue/10000;
                        
                        % build the state variabls matrix
                        if numMethod.regressionOrder == 3
%                             dimensionLS = (numOfFactors + 1)*(numOfFactors + 2)/2;

                            decisionVar = zeros(numOfFactors,size(modelStateT,2));

                            for idxLS =1:numOfFactors
                                decisionVar(idxLS,:) = modelStateT(idxLS,:); 
                            end
                            
                            % calculate number of dimension first
                            % 0th order
                            dimensionLS = 1;
                            
                            % 1st order
                            for idxLS=1:numOfFactors
                                dimensionLS = dimensionLS + 1;
                            end
                            
                            %2nd order
                            for idxLS1=1:numOfFactors
                                for idxLS2=idxLS1:numOfFactors
                                    dimensionLS = dimensionLS + 1;
                                end
                            end
                            
                            %3rd order
                            for idxLS1=1:numOfFactors
                                for idxLS2=idxLS1:numOfFactors
                                    for idxLS3=idxLS2:numOfFactors
                                        dimensionLS = dimensionLS + 1;
                                    end
                                end
                            end

                            
    %                         decisionVar1 = modelStateT(1,:);
    %                         decisionVar2 = modelStateT(2,:);
                            % we use upto 2nd order polynomial as regression
                            % equatioin

                            regMatrix = ones(pathSize,dimensionLS);

                            for idx_r=1:pathSize
                                for idxLS=1:numOfFactors
                                    regMatrix(idx_r,idxLS+1) = decisionVar(idxLS,idx_r);
                                end

                                tempIdx = 0;
                                for idxLS1=1:numOfFactors
                                    for idxLS2=idxLS1:numOfFactors
                                        tempIdx = tempIdx + 1;
                                        regMatrix(idx_r,numOfFactors+1+tempIdx) = decisionVar(idxLS1,idx_r)*decisionVar(idxLS2,idx_r);
                                    end
                                end
                                
                                tempIdx2 = 0;
                                for idxLS1=1:numOfFactors
                                    for idxLS2=idxLS1:numOfFactors
                                        for idxLS3=idxLS2:numOfFactors
                                            tempIdx2 = tempIdx2 + 1;
                                            regMatrix(idx_r,numOfFactors+1+tempIdx + tempIdx2) = decisionVar(idxLS1,idx_r)*decisionVar(idxLS2,idx_r)*decisionVar(idxLS3,idx_r);
                                        end
                                    end
                                end
                                

                            end
                        
                        else
                            dimensionLS = (numOfFactors + 1)*(numOfFactors + 2)/2;

                            decisionVar = zeros(numOfFactors,size(modelStateT,2));

                            for idxLS =1:numOfFactors
                                decisionVar(idxLS,:) = modelStateT(idxLS,:); 
                            end
    %                         decisionVar1 = modelStateT(1,:);
    %                         decisionVar2 = modelStateT(2,:);
                            % we use upto 2nd order polynomial as regression
                            % equatioin

                            regMatrix = ones(pathSize,dimensionLS);

                            for idx_r=1:pathSize
                                for idxLS=1:numOfFactors
                                    regMatrix(idx_r,idxLS+1) = decisionVar(idxLS,idx_r);
                                end

                                tempIdx = 1;
                                for idxLS1=1:numOfFactors
                                    for idxLS2=idxLS1:numOfFactors
                                        regMatrix(idx_r,numOfFactors+1+tempIdx) = decisionVar(idxLS1,idx_r)*decisionVar(idxLS2,idx_r);
                                        tempIdx = tempIdx + 1;
                                    end
                                end

                            end
                        end
                        % simple regression <- for path dependent option
                        % we should change it
                        % beta  : regression coefficients
                        beta =zeros(dimensionLS,1);
                        beta = regress(contiValue',regMatrix);
                        regContiValue = regMatrix*beta;
                       
                         %Nominal Regression test
                        regContiValue = regContiValue * 10000;
                        
                        exercisedPayoff = regContiValue;
                        % if regContiValue is bigger than exerciseValue
                        % then issue exercise call option
                        for idx_r=1:pathSize
                            if regContiValue(idx_r) >= exerciseValue(idx_r) 
                                exercisedPayoff(idx_r) = exerciseValue(idx_r);
                            end
                        end
                        
                        % we discount the new payoff and 
                        % assign it into the existing callable payoff
                        callable.payoff = model.discountPayoffRisky(currentTime,numMatTime ...
                        ,modelStateT,exercisedPayoff);
                        
                    end
                end

                nonCall.npv = mean(nonCall.payoff);
                out.nonCall = nonCall;
                callable.npv = mean(callable.payoff);
                dfCol.npv = mean(dfCol.payoff);
                dummy1.npv = mean(dummy1.npv);
                dummy2.npv = mean(dummy2.npv);
                out.callable = callable;
                out.dfCol = dfCol;
                out.dummy1 = dummy1;
                out.dummy2 = dummy2;
                out.daycountFraction = schedule.dayCountFraction;
                
        end
        
        
        function out = computePriceClosedSimple(pricer,valueDate)
                security = pricer.m_security;
                numMethod = pricer.m_numMethod;
                model = pricer.m_model;
                schedule = security.couponSchedule;
                
                % numMethodTimeSteps Generate
                mcOneTimeStep = numMethod.mcOneTimeStep;
                outInfo = numMethod.numMethodTimeStepsGenerateDW(valueDate,schedule,mcOneTimeStep);
            
                numMethodInfo.timeStepsInfo = outInfo;

                % mc random number init
                %pathSize = 2^16;
                pathSize = numMethod.pathSize;
                
                pathSize = 1;
                
%                 % generate random numbers and modelstates forward
%                 rng('default');
%                 rng(0);
% 
                numOfFactors = model.numOfFactor;
% 
%                 %Z= randn(numOfFactors*outInfo.totalTimeStepsSize,pathSize/2);
%                 Z= randn(pathSize/2,numOfFactors*outInfo.totalTimeStepsSize);
%                 Z=[Z;-Z];
%                 Z=Z';
%                 
%                 
%                 %TimeInversion
%                 sizeZ = size(Z,1);
%                 A = zeros(sizeZ,sizeZ);
%                 for i=1:sizeZ
%                     A(i,sizeZ + 1 -i) = 1;
%                 end
%                 Z = A*Z;
% 
%                 % we check with the c, so we use generated random numbers
% %                 mc_random = importdata('mc_random.xlsx');
% %                 mc_random = importdata('D:\mc_random_number_new_report.txt');
% %                 Z= mc_random;
%                 
                numMethodInfo.pathSize = pathSize;
                numMethodInfo.numOfFactors = numOfFactors;
                numMethodInfo.totalTimeSteps = outInfo.totalTimeSteps;
                numMethodInfo.totalTimeStepsSize = outInfo.totalTimeStepsSize;
%                 numMethodInfo.Z = Z;
% 
% 
% 
%                 %generate model states for the given timeSteps
%                 modelStates = numMethod.generateModelStates(valueDate,schedule,numMethodInfo,model);

                % numeraire info
                numMatTime = numMethodInfo.totalTimeSteps(end);
                dfNumMat = model.DFRisky(numMatTime/365.0);
                
                % induct backward and calculate cashflow
            
                periodCount = schedule.numOfPeriod;
                
                % cashflow columns initialize
                % 1st nonCall
                nonCall.npv = 0;
                nonCall.cashflow_e = zeros(periodCount + 1,1);
                nonCall.cashflow_npv_e = zeros(periodCount + 1,1);
                nonCall.rangeProb_e = zeros(periodCount + 1,1);
                nonCall.df_e = zeros(periodCount + 1,1);
                nonCall.df_c = zeros(periodCount + 1,1);

                % debug information, cashflow & npv per path
                nonCall.payoff = zeros(1,pathSize);
                nonCall.payoffStates.cashflow = zeros(periodCount+1,pathSize);
                nonCall.payoffStates.cashflow_npv = zeros(periodCount+1,pathSize);
                nonCall.payoffStates.df = zeros(periodCount + 1,pathSize);
                
                % intermediate variables
                payoffStateA.cashflow = zeros(1, pathSize);
                payoffStateA.cashflow_npv = zeros(1, pathSize);
                payoffStateA.df = zeros(1,pathSize);
                
                % we calculate cashflow backward from maturityDate to valueDate
                % rangeAccrual coupons are added at the startDate of the Period
                % At maturity

                nominal = security.nominal;
                coupon = security.coupon;
                callYN = security.callYN;
                
                tenorCMS = security.tenor;
                
                ralower = security.lower;
                raupper = security.upper;
                
                tenorLong = security.tenorL;
                tenorShort = security.tenorS;
                
                ralowerS = security.lowerS;
                raupperS = security.upperS;
                
                currentTime = numMethodInfo.totalTimeSteps(end);
                totalTimeSteps = numMethodInfo.timeStepsInfo.totalTimeSteps;
                totalTimeStepsSize = numMethodInfo.timeStepsInfo.totalTimeStepsSize;
                if totalTimeStepsSize == 0 || (totalTimeStepsSize == 1 && totalTimeSteps(1) == 0) 
                    out.nonCall = nonCall;
                    return;
                end
                
                currentTimeIdx = totalTimeStepsSize;
                
                currentNodeIdx = periodCount + 1;
                
%                 modelStateT = modelStates(numOfFactors*(totalTimeStepsSize-1)+1:numOfFactors*(totalTimeStepsSize-1)+2,:);
                payoffStateA.cashflow = nominal*ones(1,pathSize);
                %process payoff at maturity
                nonCall.payoffStates.cashflow(currentNodeIdx,:) = payoffStateA.cashflow;
                
%                 payoffStateA.cashflow_npv = model.discountPayoff(currentTime,numMatTime ...
%                     ,modelStateT,payoffStateA.cashflow);
%                 
                payoffStateA.cashflow_npv = model.DFRisky(currentTime/365.0)*payoffStateA.cashflow;
                
                nonCall.payoffStates.cashflow_npv(currentNodeIdx,:) = payoffStateA.cashflow_npv ;
                
%                 payoffStateA.df = model.discountPayoff(currentTime,numMatTime ...
%                     ,modelStateT,1.0*ones(1,pathSize));
                payoffStateA.df = model.DFRisky(currentTime/365.0) *ones(1,pathSize);
                    

                nonCall.payoffStates.df(currentNodeIdx,:) = payoffStateA.df;
                nonCall.payoff = nonCall.payoff + payoffStateA.cashflow_npv;
                
                nonCall.cashflow_e(currentNodeIdx) = mean(payoffStateA.cashflow);
                nonCall.cashflow_npv_e(currentNodeIdx) = mean(payoffStateA.cashflow_npv);
                nonCall.df_e(currentNodeIdx) = mean(payoffStateA.df);
                nonCall.df_c(currentNodeIdx) = model.DFRisky(currentTime/365.0);
                
%                 % callable at maturity is the same as noncallable
%                 callable.payoffStates.cashflow(currentNodeIdx,:) = nonCall.payoffStates.cashflow(currentNodeIdx,:);
%                 callable.payoffStates.cashflow_npv(currentNodeIdx,:) = nonCall.payoffStates.cashflow_npv(currentNodeIdx,:);
%                 callable.payoff = callable.payoff + callable.payoffStates.cashflow_npv(currentNodeIdx,:);
%                 
%                 callable.cashflow_e(currentNodeIdx) = nonCall.cashflow_e(currentNodeIdx);
%                 callable.cashflow_npv_e(currentNodeIdx) = nonCall.cashflow_npv_e(currentNodeIdx);
                  
                lastTimeIdx = currentTimeIdx;
                
                
                % flag for inclusive schedule
%                 simpleYN = false;
                simpleYN = 1;
                
                for i=periodCount:-1:1
                    % past schedule neglect
                    if DateDiff(valueDate,schedule.payDates(i)) >= 0, continue;end 
                    startTimeIdx = numMethodInfo.timeStepsInfo.startTimeIdx(i);
                    endTimeIdx = numMethodInfo.timeStepsInfo.endTimeIdx(i);

                    inclusiveYN = numMethodInfo.timeStepsInfo.inclusiveYN(i);
                     %% FromToInduction start

                    %modelsStates induct backward(lastTimeIdx to endTimeIdx)
                    while endTimeIdx < lastTimeIdx
                        lastTimeIdx = lastTimeIdx -1;
                        if lastTimeIdx == endTimeIdx break;end
                    end

                    currentTimeIdx = lastTimeIdx;
                    currentTime = totalTimeSteps(currentTimeIdx);
                    % now we are at endTimeIdx
                    % variables for coupon valuation initialized
%                     ralower = security.lower;
%                     raupper = security.upper;
%                     tenorLong = security.tenorL;
%                     tenorShort = security.tenorS;

                    cD = 0; % cumulated day count fraction
                    cP = 0; 
                    dI = 0; % dummy idx for loop
                    dcfC = 0;
                    dcfN = 0;
                    dD = 0;
                    idxCMS = zeros(1,pathSize);
                    
                    idxC = zeros(1,pathSize);
                    idxN = zeros(1,pathSize);
                    
                    idxL = zeros(1,pathSize);
                    idxS = zeros(1,pathSize);
                    spreadC = zeros(1,pathSize);
                    spreadN = zeros(1,pathSize);
                    probSpread = zeros(1,pathSize);
                    
                    numOfTimeSteps = numMethodInfo.timeStepsInfo.numOfTimeSteps(i);
                    
                    % if inclusive then we use simpleYN rangeProb
                    %if inclusive then we do not evaluate the last step but
                    % use fixing instead 
                    endTimeStep = 1;
                    if inclusiveYN
%                         simpleYN = true;
                        simpleYN = 1;
                        endTimeStep = 2;
                    end
                    
                    for j=numOfTimeSteps:-1:endTimeStep
                        % evaluate dealdescription
                        % last step is omitted
                        currentTime = totalTimeSteps(currentTimeIdx);
                        %dcfC = currentTime/365.0;
                        dcfC = currentTime;
                        
                        if j~= numOfTimeSteps
                            dD = (dcfN-dcfC);
                        end

                        toDate = AddDate(valueDate,currentTime,'day');
                        tpDate = schedule.payDates(i);
%                         spreadC = model.CMSCMTSpreadDigitalRangeDate(valueDate,toDate,tpDate,tenorLong,tenorShort,ralower,raupper);
                        spreadC = model.CMSRACMTSpreadDigitalRangeDate(valueDate,toDate,tpDate,tenorCMS,tenorLong,tenorShort,0.25,ralower,raupper,ralowerS,raupperS);
                        
                        %modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:numOfFactors*(currentTimeIdx-1)+2,:);
%                         idxL = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorLong,modelStateT);
%                         idxS = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorShort,modelStateT);
%                         spreadC = idxL - idxS;
%                         probSpread = model.SRPr(spreadC,spreadN,ralower,raupper,simpleYN,pathSize)*dD; 

                        if simpleYN
                            probSpread = spreadC*dD;
                        else
                            probSpread = spreadC*dD;
%                             probSpread = 0.5*(spreadC + spreadN)*dD;
                            
                        end

                        
                        cD = cD + dD;
                        cP = cP + probSpread;

                        dcfN = dcfC;
                        spreadN = spreadC;

                        %induct backward to the next time step
                        % one time backward induction
                        lastTimeIdx = currentTimeIdx;

                        if startTimeIdx < lastTimeIdx
                            lastTimeIdx = lastTimeIdx -1;
                            currentTimeIdx = lastTimeIdx;
                        end


                    end

                    %% FromToInduction end
                    %% AtDate evaluation
                    % Adding a new cashflow to the payoff

                    dcf = schedule.dayCountFraction(i);
                    
                    if inclusiveYN
                        % we add valuation date
                        cD = schedule.dayCount(i);
                        rangeDays =  security.rangeDays(i);
                        payoffStateA.cashflow = nominal * coupon * dcf * (rangeDays + cP)/cD;
                    else
                        payoffStateA.cashflow = nominal * coupon * dcf * cP/cD;
                    end
                    % eventDate(i) = startDate(i) therefore there is no induction in modelStates
                    % i.e. currentTimeIdx has not changed
                    % no need to update modelStateT
                    % modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:end,:);
                    % since cashflow's are added at the startDate(i) we need to
                    % multiply cashflow with the df(startDate(i), payDate(i))
                    payTime = DateDiff(schedule.payDates(i),valueDate);
                    
                    %event is at startDate so we need to discount the
                    %cashflow first from paydate to startdate
%                     df_sp = model.discountFactor(currentTime, payTime,numMatTime,modelStateT);

%                     for k=1:pathSize
%                         payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_sp(k);
%                     end
                    
                    % process payoff
                    
                    currentNodeIdx = i;
                    
                    % 1st nonCall column
                    nonCall.payoffStates.cashflow(currentNodeIdx,:) = payoffStateA.cashflow;
                    payoffStateA.cashflow_npv = payoffStateA.cashflow * model.DFRisky(payTime/365.0);
                    
%                     payoffStateA.cashflow_npv = model.discountPayoff(currentTime,numMatTime ...
%                     ,modelStateT,payoffStateA.cashflow);
                    nonCall.payoffStates.cashflow_npv(currentNodeIdx,:) = payoffStateA.cashflow_npv;
                    payoffStateA.df = model.DFRisky(payTime/365.0);
%                     payoffStateA.df = model.discountPayoff(currentTime,numMatTime ...
%                     ,modelStateT,1.0*ones(1,pathSize));
                    nonCall.payoffStates.df(currentNodeIdx,:) = payoffStateA.df;
                    
                    nonCall.payoff = nonCall.payoff + payoffStateA.cashflow_npv;
                    
                    nonCall.cashflow_e(currentNodeIdx) = mean(payoffStateA.cashflow);
                    nonCall.rangeProb_e(currentNodeIdx) = mean(nonCall.cashflow_e(currentNodeIdx)/(nominal * coupon * dcf));
                    nonCall.cashflow_npv_e(currentNodeIdx) = mean(payoffStateA.cashflow_npv);

                    nonCall.df_e(currentNodeIdx) = mean(payoffStateA.df);
                    nonCall.df_c(currentNodeIdx) = model.DFRisky(currentTime/365.0);
                    

                end

                nonCall.npv = mean(nonCall.payoff);
                out.pricerInfo.nonCall = nonCall;
                
%                 callable.npv = mean(callable.payoff);
%                 out.pricerInfo.callable = callable;
                out.daycountFraction = schedule.dayCountFraction;
                
        end
        

        
        
        
        
    end
    
end



In [ ]:
classdef DailyWeeklyRiskyPricer_CMSSpreadRASwapSimpleCall < H_RootObject
    %Pricer(Security,Model,NumMethod)
    %Prcer is constructed from Security, Model, NumMethod 
    
    properties
        m_security
        m_model
        m_numMethod
        overHedgeYN
    end
    
    methods
        function pricer = DailyWeeklyRiskyPricer_CMSSpreadRASwapSimpleCall(security,model,numMethod,overHedgeYN)
                pricer.m_security = security;
                pricer.m_model = model;
                pricer.m_numMethod = numMethod;
                pricer.overHedgeYN = overHedgeYN;
        end
        
        function out = computePriceMCCallableGeneric(pricer,valueDate)
                security = pricer.m_security;
                numMethod = pricer.m_numMethod;
                model = pricer.m_model;
                schedule = security.couponSchedule;
                callSchedule = security.callSchedule;
                
                floatingSchedule = security.floatingSchedule;
                floatingSpread = security.floatingSpread;
                floatingIndex = security.floatingIndex;
                callSchedule = security.callSchedule;
                
                % numMethodTimeSteps Generate
                mcOneTimeStep = numMethod.mcOneTimeStep;
%                 outInfo = numMethod.numMethodTimeStepsGenerateDW(valueDate,...
%                                     schedule,mcOneTimeStep);
%                                 

                outInfo = numMethod.numMethodTimeStepsGenerateDWSwapSimpleCall(valueDate,...
                                    schedule,floatingSchedule,callSchedule,mcOneTimeStep);
                
                numMethodInfo.timeStepsInfo = outInfo;

                % mc random number init
                %pathSize = 2^16;
                pathSize = numMethod.pathSize;

                % generate random numbers and modelstates forward
                rng('default');
                rng(0);

                numOfFactors = model.numOfFactor;

                %Z= randn(numOfFactors*outInfo.totalTimeStepsSize,pathSize/2);
%                 Z= randn(pathSize/2,numOfFactors*outInfo.totalTimeStepsSize);
                Z= randn(pathSize/2,numOfFactors*outInfo.modelTimeStepsSize);
                Z=[Z;-Z];
                Z=Z';
                
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;

                % we check with the c code, so we use generated random numbers
%                 mc_random = importdata('mc_random.xlsx');
%                 mc_random = importdata('D:\mc_random_number_new_report.txt');
%                 Z= mc_random;
                
                numMethodInfo.pathSize = pathSize;
                numMethodInfo.numOfFactors = numOfFactors;
                
                numMethodInfo.totalTimeSteps = outInfo.totalTimeSteps;
                numMethodInfo.totalTimeStepsSize = outInfo.totalTimeStepsSize;
                
                totalTimeSteps = numMethodInfo.totalTimeSteps;
                totalTimeStepsSize = numMethodInfo.totalTimeStepsSize;
                
                % for modelStateGeneration
                numMethodInfo.modelTimeSteps = outInfo.modelTimeSteps;
                numMethodInfo.modelTimeStepsSize = outInfo.modelTimeStepsSize;
                
                numMethodInfo.pastTimeStepsSize = outInfo.pastTimeStepsSize;
                
                pastTimeStepsSize = numMethodInfo.pastTimeStepsSize;
                
                numMethodInfo.Z = Z;

                

                %generate model states for the given timeSteps
                modelStates = numMethod.generateModelStatesNFNew(valueDate,schedule,numMethodInfo,model);

                % numeraire info
%                 numMatTime = numMethodInfo.totalTimeSteps(end - numMethodInfo.pastTimeStepsSize);
                numMatTime = numMethodInfo.totalTimeSteps(end);
                
                days = DateDiff(schedule.payDates(schedule.numOfPeriod),valueDate);
                numMatTime = max(numMatTime, days);
                
%                 days = DateDiff(floatingSchedule.payDates(end),valueDate);
%                 numMatTime = max(numMatTime, days);
                
                dfNumMat = model.DFRisky(numMatTime/365.0);
                
                numeraire.numMatTime = numMatTime;
                numeraire.dfNumMat = dfNumMat;
                
                % induct backward and calculate cashflow
                periodCount = schedule.numOfPeriod;
                
                % pricer single info initialize
                
                nonCall = PricerInfo(periodCount +1, pathSize);
                callable = PricerInfo(periodCount +1, pathSize);
                dfCol = PricerInfo(periodCount +1, pathSize);
                
                dummy1 = PricerInfo(periodCount +1, pathSize);
                dummy2 = PricerInfo(periodCount +1, pathSize);
                
                % additional floating leg info
                floatingLeg = PricerInfo(floatingSchedule.numOfPeriod, pathSize);
                % currentFloatingLegIdx for iteration in dealing with 
                % the floatingLeg cashflow
                currentFloatingLegIdx = floatingSchedule.numOfPeriod;
                
                % intermediate variables
                payoffStateA.cashflow = zeros(1, pathSize);
                payoffStateA.cashflow_npv = zeros(1, pathSize);
                
            
                % we calculate cashflow backward from maturityDate to valueDate
                % rangeAccrual coupons are added at the startDate of the Period
                % At maturity

                nominal = security.nominal;
                coupon = security.coupon;
                callYN = security.callYN;
                
                ralower = security.lower;
                raupper = security.upper;
                tenorLong = security.tenorL;
                tenorShort = security.tenorS;
                
                currentTime = numMethodInfo.totalTimeSteps(end);
                modelTimeStepsSize = numMethodInfo.modelTimeStepsSize;
                currentTimeIdx = numMethodInfo.totalTimeStepsSize;
%                 currentTimeIdx = numMethodInfo.totalTimeStepsSize - numMethodInfo.pastTimeStepsSize;
                
                currentNodeIdx = periodCount + 1;
                
                modelStateT = modelStates(numOfFactors*(max(currentTimeIdx-pastTimeStepsSize-1,0))+1:numOfFactors*(max(currentTimeIdx-pastTimeStepsSize-1,0))+2,:);
                
                %swap no nominal payoff
                payTime = DateDiff(schedule.payDates(schedule.numOfPeriod),valueDate);
                
                %even though there is no cashflow at the last structured
                %enddate, we add dummy cashflow in line with the note
                
                if payTime > 0
                    payoffStateA.cashflow = nominal*ones(1,pathSize)*0.0;

                    df_ep = model.discountFactorRisky(currentTime, payTime,numMatTime,modelStateT);
                    for k=1:pathSize
                        payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                    end

                    %process payoff at maturity
                    nonCall.ProcessAddPayoff(model,currentTime,numeraire, ...
                                         modelStateT,payoffStateA.cashflow,currentNodeIdx);

                    callable.ProcessAddPayoff(model,currentTime,numeraire, ...
                                         modelStateT,payoffStateA.cashflow,currentNodeIdx);

                    %df column
                    payoffStateA.cashflow = ones(1,pathSize);
                    for k=1:pathSize
                        payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                    end

                    dfCol.ProcessAddPayoff(model,currentTime,numeraire, ...
                                         modelStateT,payoffStateA.cashflow,currentNodeIdx);

                    %dummy column
                    payoffStateA.cashflow = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorLong,modelStateT);
                    dummy1.ProcessAddPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);

                    payoffStateA.cashflow = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                    dummy2.ProcessAddPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);

                  
                end
                
                lastTimeIdx = currentTimeIdx;
                totalTimeSteps = numMethodInfo.timeStepsInfo.totalTimeSteps;
                
                % flag for inclusive schedule
%                 simpleYN = false;
%                 simpleYN = true;
                if pricer.overHedgeYN == true 
                    simpleYN = 2;
                else
                    simpleYN = 1;
                end
                
                for i=periodCount:-1:1
                    % past schedule neglect
                    % we induct in the pastTimeSteps too
                    
%                     if DateDiff(valueDate,schedule.payDates(i)) >= 0, continue;end 
                    
                    startTimeIdx = numMethodInfo.timeStepsInfo.startTimeIdx(i);
                    endTimeIdx = numMethodInfo.timeStepsInfo.endTimeIdx(i);

                    inclusiveYN = numMethodInfo.timeStepsInfo.inclusiveYN(i);
                 %% FromToInduction start

                    %modelsStates induct backward(lastTimeIdx to endTimeIdx)
                    while endTimeIdx < lastTimeIdx
                        lastTimeIdx = lastTimeIdx -1;
                        if lastTimeIdx == endTimeIdx break;end
                    end

                    currentTimeIdx = lastTimeIdx;
                    currentTime = totalTimeSteps(currentTimeIdx);
                    % now we are at endTimeIdx
                    % variables for coupon valuation initialized
                    

                    cD = 0; % cumulated day count fraction
                    %cP shoul be declared as an array
%                     cP = 0;
                    cP = zeros(1,pathSize);
                    dI = 0; % dummy idx for loop
                    dcfC = 0;
                    dcfN = 0;
                    dD = 0;
                    idxL = zeros(1,pathSize);
                    idxS = zeros(1,pathSize);
                    spreadC = zeros(1,pathSize);
                    spreadN = zeros(1,pathSize);
                    probSpread = zeros(1,pathSize);
                    
                    numOfTimeSteps = numMethodInfo.timeStepsInfo.numOfTimeSteps(i);
                    
                    % if inclusive then we use simpleYN rangeProb
                    %if inclusive then we do not evaluate the last step but
                    % use fixing instead 
                    endTimeStep = 1;
                    % we let endTimeStep to be 1 and
                    % inside the for loop if currentTime ==0
                    % then we skip the range accrual check part
                    
                    if inclusiveYN
%                         simpleYN = true;
                        simpleYN = 1;
%                         endTimeStep = 2;
                        if pricer.overHedgeYN == true 
                            simpleYN = 2;
                        end
                    end
                    
                    for j=numOfTimeSteps:-1:endTimeStep
                        % we initialize payoffStateA as zeros to prevent
                        % reuse it;
                        payoffStateA.cashflow = zeros(1,pathSize);
                        % evaluate FromTo dealdescription
                        % last step is omitted
                        currentTime = totalTimeSteps(currentTimeIdx);
                        if currentTime > 0
                            dcfC = currentTime;

                            if j~= numOfTimeSteps
                                dD = (dcfN-dcfC);
                            end

                            modelStateT = modelStates(numOfFactors*(max(currentTimeIdx-pastTimeStepsSize-1,0))+1:numOfFactors*(max(currentTimeIdx-pastTimeStepsSize-1,0))+2,:);
                            idxL = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorLong,modelStateT);
                            idxS = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                            spreadC = idxL - idxS;

                            probSpread = model.SRPr(spreadC,spreadN,ralower,raupper,simpleYN,pathSize)*dD;
                            cD = cD + dD;
                            cP = cP + probSpread;

                            dcfN = dcfC;
                            spreadN = spreadC;
                        else
                            % we updated cD only since fixing is treated in
                            % rangedays
                            % fix it later !
                            
                            dcfC = currentTime;

                            if j~= numOfTimeSteps
                                dD = (dcfN-dcfC);
                            end
                            cD = cD + dD;
                            
                            dcfN = dcfC;
                            aaa =1.0;
                        end 
                        
                        % floating event begin
                        
                        
                        % evaluate secodary dealDescription(floating)
                        % floatingLeg
                        idxFloat = find(currentTime == outInfo.floatingEventTime);
                        
%                         if currentTime == 5297
%                             aaa =1;
%                         end
                        
                        % we do not evaluate floating leg at the end of
                        % coupon Schedule for avoding double counting
                        if ~isempty(idxFloat) && (j~= numOfTimeSteps)
                            iterFloat = 1;
                            foundIterFloatYN= false;
                            for idx11 = length(floatingSchedule.startDates):-1:1
                                if outInfo.floatingEventTime(idxFloat) == DateDiff(floatingSchedule.startDates(idx11),valueDate)
                                    iterFloat = idx11;
                                    foundIterFloatYN = true;
                                    break;
                                end
                            end
                            
                            if ~foundIterFloatYN
                                error('floatingEventTime is not in the floatingSchedule startDates')
                            end
                            
                            % only those cashflow that paydate is after
                            % valueDate will be added
                            if DateDiff(floatingSchedule.payDates(idx11),valueDate) > 0
                                % floatingIndex is not fixed    
                                if outInfo.floatingEventTime(idxFloat) > 0
                                    dcfFloating = floatingSchedule.dayCountFraction(iterFloat);
                                    if strcmp(floatingIndex,'Libor3M')
                                        tenorFloatingCD = 0.25;
                                        idxFloatingCD = model.Libor_SimpleDate(valueDate,currentTime,currentTime,numMatTime, tenorFloatingCD,modelStateT);
                                        % -1 for floating pay 
                                        payoffStateA.cashflow = -1.0 * nominal * (idxFloatingCD + floatingSpread) * dcfFloating;
                                    end

                                else
                                    % floatingIndex is fixed
                                    sizeIdxFloat = length(idxFloat);
                                    if strcmp(floatingIndex,'Libor3M')
                                        tenorFloatingCD = 0.25;
                                        allFixings = security.allFixings;
                                        KRwLibor3MFixings = allFixings('KRWLibor3M');

                                        dcfFloating = floatingSchedule.dayCountFraction(iterFloat);
                                        idxFloatingCD = KRwLibor3MFixings(StrDate(floatingSchedule.resetDates(iterFloat),'str'));
        %                                 idxFloatingCD = model.Libor_SimpleDate(valueDate,currentTime,currentTime,numMatTime, tenorFloatingCD,modelStateT);
                                        % -1 for floating pay
                                        for k=1:pathSize
                                            payoffStateA.cashflow(k) = -1.0 * nominal * (idxFloatingCD + floatingSpread) * dcfFloating;
                                        end
                                    end
                                end
                                
                                payTime = DateDiff(floatingSchedule.payDates(iterFloat),valueDate);
                                %event is at startDate so we need to discount the
                                %cashflow first from paydate to startdate
                                df_ep = model.discountFactorRisky(currentTime, payTime,numMatTime,modelStateT);

                                for k=1:pathSize
                                    payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                                end
                                
                                % process floatingLeg payoff

                                currentNodeIdx = i;

                                nonCall.ProcessAddPayoff(model,currentTime,numeraire, ...
                                                 modelStateT,payoffStateA.cashflow,currentNodeIdx);

                                callable.ProcessAddPayoff(model,currentTime,numeraire, ...
                                                 modelStateT,payoffStateA.cashflow,currentNodeIdx);

                                % process additional payoff
                                floatingLeg.ProcessAddPayoff(model,currentTime,numeraire, ...
                                                 modelStateT,payoffStateA.cashflow,iterFloat);
                                
                            end
                        end
                        % floating event end
                        
                        % callable event begin
                        idxCall = find(currentTime == outInfo.callEventTime);
                        
                        % we do not evaluate callable event at the start of
                        % coupon Schedule for avoding double counting
                        % and that make sense
                        if ~isempty(idxCall) && (j~= endTimeStep)
                            iterCall = 1;
                            foundIterCallYN= false;
                            for idx11 = length(callSchedule.startDates):-1:1
                                % call event happens at call startDate !
                                % we change call event happens at call resetDate !
                                
                                if outInfo.callEventTime(idxCall) == DateDiff(callSchedule.resetDates(idx11),valueDate)
                                    iterCall = idx11;
                                    foundIterCallYN = true;
                                    break;
                                end
                            end
                            
                            if ~foundIterCallYN
                                error('callEventTime is not in the callSchedule startDates')
                            end
                            
                            % only those cashflow that paydate is after
                            % valueDate will be added
                            if DateDiff(callSchedule.payDates(iterCall),valueDate) > 0
                                % callEvent is not fixed
                                if DateDiff(valueDate,callSchedule.resetDates(iterCall)) < 0
                                    % 1.undiscount callable.payoff
                                    % 2.build the state variables matrix
                                    % 3.regress 1's with the given state variables
                                    % (model states 1,2)
                                    % 4. exercise per each path in
                                    % discount the new payoff and assign into callable
                                    % column

                                    %1 undiscount callable.payoff and get the
                                    %contiValue

                                    %exercise value swap
                                    exerciseValue = security.nominal*ones(1,pathSize) * 0.0;
                                    payTime = DateDiff(callSchedule.payDates(iterCall),valueDate);
                                    %event is at startDate so we need to discount the
                                    %cashflow first from paydate to startdate
                                    df_ep = model.discountFactorRisky(currentTime, payTime,numMatTime,modelStateT);
                                    for k=1:pathSize
                                        exerciseValue(k) = exerciseValue(k) * df_ep(k);
                                    end

                                    contiValue = zeros(1,pathSize);
                                    onePayoff = ones(1,pathSize);
                                    onePayoffD = model.discountPayoffRisky(currentTime,numMatTime ...
                                    ,modelStateT,onePayoff);
                                    contiValue = callable.payoff./onePayoffD;

                                    %Nominal Regression test
                                    contiValue = contiValue/10000;

                                    % build the state variabls matrix
                                    decisionVar1 = modelStateT(1,:);
                                    decisionVar2 = modelStateT(2,:);
                                    % we use upto 2nd order polynomial as regression
                                    % equatioin
                                    dimensionLS = 6;
                                    regMatrix = ones(pathSize,dimensionLS);

                                    for idx_r=1:pathSize
                                        regMatrix(idx_r,2) = decisionVar1(idx_r);
                                        regMatrix(idx_r,3) = decisionVar2(idx_r);
                                        regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r);
                                        regMatrix(idx_r,5) = decisionVar1(idx_r)*decisionVar2(idx_r);
                                        regMatrix(idx_r,6) = decisionVar2(idx_r)*decisionVar2(idx_r);
                                    end

                                    % simple regression <- for path dependent option
                                    % we should change it
                                    % beta  : regression coefficients
                                    beta =zeros(dimensionLS,1);
                                    beta = regress(contiValue',regMatrix);
                                    regContiValue = regMatrix*beta;

                                     %Nominal Regression test
                                    regContiValue = regContiValue * 10000;

                                    exercisedPayoff = regContiValue;
                                    % if regContiValue is bigger than exerciseValue
                                    % then issue exercise call option
                                    for idx_r=1:pathSize
                                        if regContiValue(idx_r) >= exerciseValue(idx_r) 
                                            exercisedPayoff(idx_r) = exerciseValue(idx_r);
                                        end
                                    end

                                    % we discount the new payoff and 
                                    % assign it into the existing callable payoff
                                    callable.payoff = model.discountPayoffRisky(currentTime,numMatTime ...
                                    ,modelStateT,exercisedPayoff);

                                end
                                
                                
                            end
                        end
                        
                        % callable event end    
                            
                        
                        %induct backward to the next time step
                        % one time backward induction
                        lastTimeIdx = currentTimeIdx;

                        if startTimeIdx < lastTimeIdx
                            lastTimeIdx = lastTimeIdx -1;
                            currentTimeIdx = lastTimeIdx;
                        end

                    end
                    
                    %% FromToInduction end
        % if the current period is inclusive, then the start date is not evaluated in the above for loop.
		% instead the fixing will be treated separately below.
		% however current_time_idx is updated at the end of the for loop.
		% so we need to update model_states accordingly.
		% if model_states were updated previously then the following additionaly step doesn't affect the result anyway.
                    currentTime = totalTimeSteps(currentTimeIdx);
                    
                    % if 
                    modelStateT = modelStates(numOfFactors*(max(currentTimeIdx-pastTimeStepsSize-1,0))+1:numOfFactors*(max(currentTimeIdx-pastTimeStepsSize-1,0))+2,:);
                    
                    %% AtDate evaluation
                    % Adding a new cashflow to the payoff
                    % for structured coupon payDate should be after
                    % valueDate
                    
                    if DateDiff(schedule.payDates(i),valueDate) > 0
                        dcf = schedule.dayCountFraction(i);

                        if inclusiveYN
                            % we add valuation date
                            cD = schedule.dayCount(i);
                            rangeDays =  security.rangeDays(i);
                            payoffStateA.cashflow = nominal * coupon * dcf * (rangeDays + cP)/cD;
                        else
                            payoffStateA.cashflow = nominal * coupon * dcf * cP/cD;
                        end
                        % eventDate(i) = startDate(i) therefore there is no induction in modelStates
                        % i.e. currentTimeIdx has not changed
                        % no need to update modelStateT
                        % modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:end,:);
                        % since cashflow's are added at the startDate(i) we need to
                        % multiply cashflow with the df(startDate(i), payDate(i))
                        payTime = DateDiff(schedule.payDates(i),valueDate);

                        %event is at startDate so we need to discount the
                        %cashflow first from paydate to startdate
                        df_ep = model.discountFactorRisky(currentTime, payTime,numMatTime,modelStateT);

                        for k=1:pathSize
                            payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                        end

                        % process payoff

                        currentNodeIdx = i;

                        nonCall.ProcessAddPayoff(model,currentTime,numeraire, ...
                                         modelStateT,payoffStateA.cashflow,currentNodeIdx);

                        callable.ProcessAddPayoff(model,currentTime,numeraire, ...
                                         modelStateT,payoffStateA.cashflow,currentNodeIdx);

                        %df column
                        payoffStateA.cashflow = ones(1,pathSize);
                        for k=1:pathSize
                            payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                        end

                        dfCol.ProcessAddPayoff(model,currentTime,numeraire, ...
                                         modelStateT,payoffStateA.cashflow,currentNodeIdx);

                        %dummy column
                        payoffStateA.cashflow = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorLong,modelStateT);
                        dummy1.ProcessAddPayoff(model,currentTime,numeraire, ...
                                         modelStateT,payoffStateA.cashflow,currentNodeIdx);

                        payoffStateA.cashflow = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                        dummy2.ProcessAddPayoff(model,currentTime,numeraire, ...
                                         modelStateT,payoffStateA.cashflow,currentNodeIdx);

                    end
                    % if exercise is possible
                    if i == 1
                        dummaaa = 1;
                    end
                    
%                     % call event happens  at the start of the period
%                     % at call date, by default, call option is expired
%                     
%                     if callYN(i) && (DateDiff(valueDate,callSchedule.resetDates(i)) < 0)
%                         % 1.undiscount callable.payoff
%                         % 2.build the state variables matrix
%                         % 3.regress 1's with the given state variables
%                         % (model states 1,2)
%                         % 4. exercise per each path in
%                         % discount the new payoff and assign into callable
%                         % column
%                         
%                         %1 undiscount callable.payoff and get the
%                         %contiValue
%                        
%                         %exercise value swap
%                         exerciseValue = security.nominal*ones(1,pathSize) * 0.0;
%                         payTime = DateDiff(callSchedule.payDates(i),valueDate);
%                         %event is at startDate so we need to discount the
%                         %cashflow first from paydate to startdate
%                         df_ep = model.discountFactorRisky(currentTime, payTime,numMatTime,modelStateT);
%                         for k=1:pathSize
%                             exerciseValue(k) = exerciseValue(k) * df_ep(k);
%                         end
%                         
%                         contiValue = zeros(1,pathSize);
%                         onePayoff = ones(1,pathSize);
%                         onePayoffD = model.discountPayoffRisky(currentTime,numMatTime ...
%                         ,modelStateT,onePayoff);
%                         contiValue = callable.payoff./onePayoffD;
%                         
%                         %Nominal Regression test
%                         contiValue = contiValue/10000;
%                         
%                         % build the state variabls matrix
%                         decisionVar1 = modelStateT(1,:);
%                         decisionVar2 = modelStateT(2,:);
%                         % we use upto 2nd order polynomial as regression
%                         % equatioin
%                         dimensionLS = 6;
%                         regMatrix = ones(pathSize,dimensionLS);
%                         
%                         for idx_r=1:pathSize
%                             regMatrix(idx_r,2) = decisionVar1(idx_r);
%                             regMatrix(idx_r,3) = decisionVar2(idx_r);
%                             regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r);
%                             regMatrix(idx_r,5) = decisionVar1(idx_r)*decisionVar2(idx_r);
%                             regMatrix(idx_r,6) = decisionVar2(idx_r)*decisionVar2(idx_r);
%                         end
%                         
%                         % simple regression <- for path dependent option
%                         % we should change it
%                         % beta  : regression coefficients
%                         beta =zeros(dimensionLS,1);
%                         beta = regress(contiValue',regMatrix);
%                         regContiValue = regMatrix*beta;
%                        
%                          %Nominal Regression test
%                         regContiValue = regContiValue * 10000;
%                         
%                         exercisedPayoff = regContiValue;
%                         % if regContiValue is bigger than exerciseValue
%                         % then issue exercise call option
%                         for idx_r=1:pathSize
%                             if regContiValue(idx_r) >= exerciseValue(idx_r) 
%                                 exercisedPayoff(idx_r) = exerciseValue(idx_r);
%                             end
%                         end
%                         
%                         % we discount the new payoff and 
%                         % assign it into the existing callable payoff
%                         callable.payoff = model.discountPayoffRisky(currentTime,numMatTime ...
%                         ,modelStateT,exercisedPayoff);
%                         
%                     end
                end

                
                nonCall.npv = mean(nonCall.payoff);
                out.nonCall = nonCall;
                callable.npv = mean(callable.payoff);
                out.callable = callable;
                
                floatingLeg.npv = mean(floatingLeg.payoff);
                out.floatingLeg = floatingLeg;
                
                out.dfCol = dfCol;
                out.dummy1 = dummy1;
                out.dummy2 = dummy2;
                out.daycountFraction = schedule.dayCountFraction;
                
        end
        
        function out = computePriceClosedSimple(pricer,valueDate)
                security = pricer.m_security;
                numMethod = pricer.m_numMethod;
                model = pricer.m_model;
                schedule = security.couponSchedule;
                
                 %floating
                floatingSchedule = security.floatingSchedule;
                floatingSpread = security.floatingSpread;
                floatingIndex = security.floatingIndex;
                
                % numMethodTimeSteps Generate
                mcOneTimeStep = numMethod.mcOneTimeStep;
%                 outInfo = numMethod.numMethodTimeStepsGenerateDW(valueDate,schedule,mcOneTimeStep);
                
                outInfo = numMethod.numMethodTimeStepsGenerateDWSwapSimple(valueDate,...
                                    schedule,floatingSchedule,mcOneTimeStep);
                                
                numMethodInfo.timeStepsInfo = outInfo;

                % mc random number init
                %pathSize = 2^16;
                pathSize = numMethod.pathSize;
                
                pathSize = 1;
                
%                 % generate random numbers and modelstates forward
%                 rng('default');
%                 rng(0);
% 
                numOfFactors = 2;
% 
%                 %Z= randn(numOfFactors*outInfo.totalTimeStepsSize,pathSize/2);
%                 Z= randn(pathSize/2,numOfFactors*outInfo.totalTimeStepsSize);
%                 Z=[Z;-Z];
%                 Z=Z';
%                 
%                 
%                 %TimeInversion
%                 sizeZ = size(Z,1);
%                 A = zeros(sizeZ,sizeZ);
%                 for i=1:sizeZ
%                     A(i,sizeZ + 1 -i) = 1;
%                 end
%                 Z = A*Z;
% 
%                 % we check with the c, so we use generated random numbers
% %                 mc_random = importdata('mc_random.xlsx');
% %                 mc_random = importdata('D:\mc_random_number_new_report.txt');
% %                 Z= mc_random;
%                 
                numMethodInfo.pathSize = pathSize;
                numMethodInfo.numOfFactors = numOfFactors;
                
                numMethodInfo.totalTimeSteps = outInfo.totalTimeSteps;
                numMethodInfo.totalTimeStepsSize = outInfo.totalTimeStepsSize;
                
                totalTimeSteps = numMethodInfo.totalTimeSteps;
                totalTimeStepsSize = numMethodInfo.totalTimeStepsSize;
                
                numMethodInfo.modelTimeSteps = outInfo.modelTimeSteps;
                numMethodInfo.modelTimeStepsSize = outInfo.modelTimeStepsSize;
                
                numMethodInfo.pastTimeStepsSize = outInfo.pastTimeStepsSize;
                
                pastTimeStepsSize = numMethodInfo.pastTimeStepsSize;
%                 numMethodInfo.Z = Z;
% 
% 
% 
%                 %generate model states for the given timeSteps
%                 modelStates = numMethod.generateModelStates(valueDate,schedule,numMethodInfo,model);

                % numeraire info
                numMatTime = numMethodInfo.totalTimeSteps(end);
                dfNumMat = model.DF(numMatTime/365.0);
                
                % induct backward and calculate cashflow
            
                periodCount = schedule.numOfPeriod;
                fltPeriodCount = floatingSchedule.numOfPeriod;
                % cashflow columns initialize
                % 1st nonCall
                nonCall.npv = 0;
                nonCall.cashflow_e = zeros(periodCount + 1,1);
                nonCall.cashflow_npv_e = zeros(periodCount + 1,1);
                nonCall.rangeProb_e = zeros(periodCount + 1,1);
                nonCall.df_e = zeros(periodCount + 1,1);
                nonCall.df_c = zeros(periodCount + 1,1);

                % debug information, cashflow & npv per path
                nonCall.payoff = zeros(1,pathSize);
                nonCall.payoffStates.cashflow = zeros(periodCount+1,pathSize);
                nonCall.payoffStates.cashflow_npv = zeros(periodCount+1,pathSize);
                nonCall.payoffStates.df = zeros(periodCount + 1,pathSize);
                
                %floatingLeg
                floatingLeg.npv = 0;
                floatingLeg.cashflow_e = zeros(fltPeriodCount,1);
                floatingLeg.cashflow_npv_e = zeros(fltPeriodCount,1);
                floatingLeg.rangeProb_e = zeros(fltPeriodCount,1);
                floatingLeg.df_e = zeros(fltPeriodCount,1);
                floatingLeg.df_c = zeros(fltPeriodCount,1);

                floatingLeg.payoff = zeros(1,pathSize);
                floatingLeg.payoffStates.cashflow = zeros(fltPeriodCount,pathSize);
                floatingLeg.payoffStates.cashflow_npv = zeros(fltPeriodCount,pathSize);
                floatingLeg.payoffStates.df = zeros(fltPeriodCount,pathSize);
                
                % intermediate variables
                payoffStateA.cashflow = zeros(1, pathSize);
                payoffStateA.cashflow_npv = zeros(1, pathSize);
                payoffStateA.df = zeros(1,pathSize);
                
                % we calculate cashflow backward from maturityDate to valueDate
                % rangeAccrual coupons are added at the startDate of the Period
                % At maturity

                nominal = security.nominal;
                coupon = security.coupon;
                callYN = security.callYN;
                
                currentTime = numMethodInfo.totalTimeSteps(end);
                totalTimeSteps = numMethodInfo.timeStepsInfo.totalTimeSteps;
                totalTimeStepsSize = numMethodInfo.timeStepsInfo.totalTimeStepsSize;
                if totalTimeStepsSize == 0 || (totalTimeStepsSize == 1 && totalTimeSteps(1) == 0) 
                    out.nonCall = nonCall;
                    return;
                end
                
                currentTimeIdx = totalTimeStepsSize;
                
                currentNodeIdx = periodCount + 1;
                
%                 modelStateT = modelStates(numOfFactors*(totalTimeStepsSize-1)+1:numOfFactors*(totalTimeStepsSize-1)+2,:);
                payoffStateA.cashflow = nominal*ones(1,pathSize)*0.0;
                %process payoff at maturity
                nonCall.payoffStates.cashflow(currentNodeIdx,:) = payoffStateA.cashflow;
                
%                 payoffStateA.cashflow_npv = model.discountPayoff(currentTime,numMatTime ...
%                     ,modelStateT,payoffStateA.cashflow);
%                 
                payoffStateA.cashflow_npv = model.DFRisky(currentTime/365.0)*payoffStateA.cashflow;
                
                nonCall.payoffStates.cashflow_npv(currentNodeIdx,:) = payoffStateA.cashflow_npv ;
                
%                 payoffStateA.df = model.discountPayoff(currentTime,numMatTime ...
%                     ,modelStateT,1.0*ones(1,pathSize));
                payoffStateA.df = model.DFRisky(currentTime/365.0) *ones(1,pathSize);
                    

                nonCall.payoffStates.df(currentNodeIdx,:) = payoffStateA.df;
                nonCall.payoff = nonCall.payoff + payoffStateA.cashflow_npv;
                
                nonCall.cashflow_e(currentNodeIdx) = mean(payoffStateA.cashflow);
                nonCall.cashflow_npv_e(currentNodeIdx) = mean(payoffStateA.cashflow_npv);
                nonCall.df_e(currentNodeIdx) = mean(payoffStateA.df);
                nonCall.df_c(currentNodeIdx) = model.DFRisky(currentTime/365.0);
                
%                 % callable at maturity is the same as noncallable
%                 callable.payoffStates.cashflow(currentNodeIdx,:) = nonCall.payoffStates.cashflow(currentNodeIdx,:);
%                 callable.payoffStates.cashflow_npv(currentNodeIdx,:) = nonCall.payoffStates.cashflow_npv(currentNodeIdx,:);
%                 callable.payoff = callable.payoff + callable.payoffStates.cashflow_npv(currentNodeIdx,:);
%                 
%                 callable.cashflow_e(currentNodeIdx) = nonCall.cashflow_e(currentNodeIdx);
%                 callable.cashflow_npv_e(currentNodeIdx) = nonCall.cashflow_npv_e(currentNodeIdx);
                  
                lastTimeIdx = currentTimeIdx;
                
                
                % flag for inclusive schedule
%                 simpleYN = false;
                simpleYN = 1;
                
                for i=periodCount:-1:1
                    % past schedule neglect
%                     if DateDiff(valueDate,schedule.payDates(i)) >= 0, continue;end 
                    startTimeIdx = numMethodInfo.timeStepsInfo.startTimeIdx(i);
                    endTimeIdx = numMethodInfo.timeStepsInfo.endTimeIdx(i);

                    inclusiveYN = numMethodInfo.timeStepsInfo.inclusiveYN(i);
                     %% FromToInduction start

                    %modelsStates induct backward(lastTimeIdx to endTimeIdx)
                    while endTimeIdx < lastTimeIdx
                        lastTimeIdx = lastTimeIdx -1;
                        if lastTimeIdx == endTimeIdx break;end
                    end

                    currentTimeIdx = lastTimeIdx;
                    currentTime = totalTimeSteps(currentTimeIdx);
                    % now we are at endTimeIdx
                    % variables for coupon valuation initialized
                    ralower = security.lower;
                    raupper = security.upper;
                    tenorLong = security.tenorL;
                    tenorShort = security.tenorS;

                    cD = 0; % cumulated day count fraction
                    cP = zeros(1,pathSize);
                    dI = 0; % dummy idx for loop
                    dcfC = 0;
                    dcfN = 0;
                    dD = 0;
                    idxL = zeros(1,pathSize);
                    idxS = zeros(1,pathSize);
                    spreadC = zeros(1,pathSize);
                    spreadN = zeros(1,pathSize);
                    probSpread = zeros(1,pathSize);
                    
                    numOfTimeSteps = numMethodInfo.timeStepsInfo.numOfTimeSteps(i);
                    
                    % if inclusive then we use simpleYN rangeProb
                    %if inclusive then we do not evaluate the last step but
                    % use fixing instead 
                    endTimeStep = 1;
                    if inclusiveYN
%                         simpleYN = true;
                        simpleYN = 1;
%                         endTimeStep = 2;
                    end
                    
                    for j=numOfTimeSteps:-1:endTimeStep
                        payoffStateA.cashflow = zeros(1,pathSize);
                        % evaluate dealdescription
                        % last step is omitted
                        currentTime = totalTimeSteps(currentTimeIdx);
                        if currentTime > 0
                            %dcfC = currentTime/365.0;
                            dcfC = currentTime;

                            if j~= numOfTimeSteps
                                dD = (dcfN-dcfC);
                            end

                            toDate = AddDate(valueDate,currentTime,'day');
                            tpDate = schedule.payDates(i);
                            spreadC = model.CMSSpreadDigitalRangeDate(valueDate,toDate,tpDate,tenorLong,tenorShort,0.25,ralower,raupper);

                            if simpleYN
                                probSpread = spreadC*dD;
                            else
                                probSpread = spreadC*dD;
    %                             probSpread = 0.5*(spreadC + spreadN)*dD;

                            end


                            cD = cD + dD;
                            cP = cP + probSpread;

                            dcfN = dcfC;
                            spreadN = spreadC;
                        else
                            % we updated cD only since fixing is treated in
                            % rangedays
                            % fix it later !
                            
                            dcfC = currentTime;

                            if j~= numOfTimeSteps
                                dD = (dcfN-dcfC);
                            end
                            cD = cD + dD;
                            
                            dcfN = dcfC;
                            aaa =1.0;
                        end
                        % evaluate secodary dealDescription(floating)
                        % floatingLeg
                        idxFloat = find(currentTime == outInfo.floatingEventTime);
                        
                        % we do not evaluate floating leg at the endof
                        % coupon Schedule for avoding double counting
                        if ~isempty(idxFloat) && (j~= numOfTimeSteps)
                            iterFloat = 1;
                            foundIterFloatYN= false;
                            for idx11 = length(floatingSchedule.startDates):-1:1
                                if outInfo.floatingEventTime(idxFloat) == DateDiff(floatingSchedule.startDates(idx11),valueDate)
                                    iterFloat = idx11;
                                    foundIterFloatYN = true;
                                    break;
                                end
                            end

                            if ~foundIterFloatYN
                                error('floatingEventTime is not in the floatingSchedule startDates')
                            end
                            
                            if DateDiff(floatingSchedule.payDates(idx11),valueDate) > 0
                                if outInfo.floatingEventTime(idxFloat) > 0

                                    dcfFloating = floatingSchedule.dayCountFraction(iterFloat);
                                    if strcmp(floatingIndex,'Libor3M')
                                        tenorFloatingCD = 0.25;
    %                                         idxFloatingCD = model.Libor_SimpleDate(valueDate,currentTime,currentTime,numMatTime, tenorFloatingCD,modelStateT);
                                        idxFloatingCD = model.LiborDate(valueDate,currentTime,tenorFloatingCD);
                                        % -1 for floating pay 
                                        payoffStateA.cashflow = -1.0 * nominal * (idxFloatingCD + floatingSpread) * dcfFloating;
                                    end

                                else
                                    % floatingIndex is fixed
                                    %if floatingCashflow 1 is fixed not paid
                                    %and also next floatingCashflow2 is fixed
                                    %then
                                    %idxFloat = find(currentTime == outInfo.floatingEventTime);
                                    % will give two answers
                                    % then we use sizeIdxFloat, length(idxfloat) to 
                                    % access the right floating Cashflow
                                    sizeIdxFloat = length(idxFloat);
                                    %
                                    if strcmp(floatingIndex,'Libor3M')
                                        tenorFloatingCD = 0.25;
                                        allFixings = security.allFixings;
                                        KRwLibor3MFixings = allFixings('KRWLibor3M');
        
                                        dcfFloating = floatingSchedule.dayCountFraction(iterFloat);
                                        idxFloatingCD = KRwLibor3MFixings(StrDate(floatingSchedule.resetDates(iterFloat),'str'));
        %                                 idxFloatingCD = model.Libor_SimpleDate(valueDate,currentTime,currentTime,numMatTime, tenorFloatingCD,modelStateT);
                                        % -1 for floating pay
                                        for k=1:pathSize
                                            payoffStateA.cashflow(k) = -1.0 * nominal * (idxFloatingCD + floatingSpread) * dcfFloating;
                                        end
                                    end
                                end
                                
                                payTime = DateDiff(floatingSchedule.payDates(iterFloat),valueDate);
                                %event is at startDate so we need to discount the
                                %cashflow first from paydate to startdate
                                %cashflow first from paydate to startdate
                                currentNodeIdx = i;
                                nonCall.payoffStates.cashflow(currentNodeIdx,:) = nonCall.payoffStates.cashflow(currentNodeIdx,:) + payoffStateA.cashflow;
                                payoffStateA.cashflow_npv = payoffStateA.cashflow * model.DFRisky(payTime/365.0);

                                nonCall.payoffStates.cashflow_npv(currentNodeIdx,:) = nonCall.payoffStates.cashflow_npv(currentNodeIdx,:) + payoffStateA.cashflow_npv;

                                nonCall.payoff = nonCall.payoff + payoffStateA.cashflow_npv;

                                nonCall.cashflow_e(currentNodeIdx) = nonCall.cashflow_e(currentNodeIdx) + mean(payoffStateA.cashflow);
                                nonCall.cashflow_npv_e(currentNodeIdx) = nonCall.cashflow_npv_e(currentNodeIdx) + mean(payoffStateA.cashflow_npv);

                                floatingLeg.payoffStates.cashflow(iterFloat,:) = floatingLeg.payoffStates.cashflow(iterFloat,:) + payoffStateA.cashflow;
                                floatingLeg.payoffStates.cashflow_npv(iterFloat,:) = floatingLeg.payoffStates.cashflow_npv(iterFloat,:) + payoffStateA.cashflow_npv;
                                floatingLeg.payoff = floatingLeg.payoff + payoffStateA.cashflow_npv;

                                floatingLeg.cashflow_e(iterFloat) = floatingLeg.cashflow_e(iterFloat) + mean(payoffStateA.cashflow);
                                floatingLeg.cashflow_npv_e(iterFloat) = floatingLeg.cashflow_npv_e(iterFloat) + mean(payoffStateA.cashflow_npv);

                            end
                        end
                               
                            
                        
                        %induct backward to the next time step
                        % one time backward induction
                        lastTimeIdx = currentTimeIdx;

                        if startTimeIdx < lastTimeIdx
                            lastTimeIdx = lastTimeIdx -1;
                            currentTimeIdx = lastTimeIdx;
                        end
                    end

                    
                    %% FromToInduction end
                    %% AtDate evaluation
                    % Adding a new cashflow to the payoff

                    dcf = schedule.dayCountFraction(i);
                    if DateDiff(schedule.payDates(i),valueDate) > 0
                        if inclusiveYN
                            % we add valuation date
                            cD = schedule.dayCount(i);
                            rangeDays =  security.rangeDays(i);
                            payoffStateA.cashflow = nominal * coupon * dcf * (rangeDays + cP)/cD;
                        else
                            payoffStateA.cashflow = nominal * coupon * dcf * cP/cD;
                        end
           
                        payTime = DateDiff(schedule.payDates(i),valueDate);

                        currentNodeIdx = i;

                        % 1st nonCall column
                        nonCall.payoffStates.cashflow(currentNodeIdx,:) = nonCall.payoffStates.cashflow(currentNodeIdx,:) + payoffStateA.cashflow;
                        payoffStateA.cashflow_npv = payoffStateA.cashflow * model.DFRisky(payTime/365.0);

    %                     payoffStateA.cashflow_npv = model.discountPayoff(currentTime,numMatTime ...
    %                     ,modelStateT,payoffStateA.cashflow);
                        nonCall.payoffStates.cashflow_npv(currentNodeIdx,:) = nonCall.payoffStates.cashflow_npv(currentNodeIdx,:) + payoffStateA.cashflow_npv;
                        payoffStateA.df = model.DFRisky(payTime/365.0);
    %                     payoffStateA.df = model.discountPayoff(currentTime,numMatTime ...
    %                     ,modelStateT,1.0*ones(1,pathSize));
                        nonCall.payoffStates.df(currentNodeIdx,:) = nonCall.payoffStates.df(currentNodeIdx,:) + payoffStateA.df;

                        nonCall.payoff = nonCall.payoff + payoffStateA.cashflow_npv;

                        nonCall.cashflow_e(currentNodeIdx) = nonCall.cashflow_e(currentNodeIdx) + mean(payoffStateA.cashflow);
                        nonCall.cashflow_npv_e(currentNodeIdx) = nonCall.cashflow_npv_e(currentNodeIdx) + mean(payoffStateA.cashflow_npv);

                        nonCall.df_e(currentNodeIdx) = nonCall.df_e(currentNodeIdx) + mean(payoffStateA.df);
                        nonCall.df_c(currentNodeIdx) = nonCall.df_c(currentNodeIdx) + model.DFRisky(currentTime/365.0);
 
                    end
                    % if exercise is possible
                    if i == 1
                        dummaaa = 1;
                    end
       
                end

                nonCall.npv = mean(nonCall.payoff);
                out.nonCall = nonCall;
                floatingLeg.npv = mean(floatingLeg.payoff);
                out.floatingLeg = floatingLeg;
%                 callable.npv = mean(callable.payoff);
%                 out.pricerInfo.callable = callable;
                out.daycountFraction = schedule.dayCountFraction;
                
        end
        
        
        
        
    end
    
end



In [ ]:
classdef DailyWeeklyRiskyPricer_CMSSpreadRASwapSimple < H_RootObject
    %Pricer(Security,Model,NumMethod)
    %Prcer is constructed from Security, Model, NumMethod 
    
    properties
        m_security
        m_model
        m_numMethod
        overHedgeYN
    end
    
    methods
        function pricer = DailyWeeklyRiskyPricer_CMSSpreadRASwapSimple(security,model,numMethod,overHedgeYN)
                pricer.m_security = security;
                pricer.m_model = model;
                pricer.m_numMethod = numMethod;
                pricer.overHedgeYN = overHedgeYN;
        end
        
        function out = computePriceMCCallableGeneric(pricer,valueDate)
                security = pricer.m_security;
                numMethod = pricer.m_numMethod;
                model = pricer.m_model;
                schedule = security.couponSchedule;
                callSchedule = security.callSchedule;
                
                floatingSchedule = security.floatingSchedule;
                floatingSpread = security.floatingSpread;
                floatingIndex = security.floatingIndex;
                
                % numMethodTimeSteps Generate
                mcOneTimeStep = numMethod.mcOneTimeStep;
%                 outInfo = numMethod.numMethodTimeStepsGenerateDW(valueDate,...
%                                     schedule,mcOneTimeStep);
%                                 

                outInfo = numMethod.numMethodTimeStepsGenerateDWSwapSimple(valueDate,...
                                    schedule,floatingSchedule,mcOneTimeStep);
                
                numMethodInfo.timeStepsInfo = outInfo;

                % mc random number init
                %pathSize = 2^16;
                pathSize = numMethod.pathSize;

                % generate random numbers and modelstates forward
                rng('default');
                rng(0);

                numOfFactors = model.numOfFactor;

                %Z= randn(numOfFactors*outInfo.totalTimeStepsSize,pathSize/2);
%                 Z= randn(pathSize/2,numOfFactors*outInfo.totalTimeStepsSize);
                Z= randn(pathSize/2,numOfFactors*outInfo.modelTimeStepsSize);
                Z=[Z;-Z];
                Z=Z';
                
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;

                % we check with the c code, so we use generated random numbers
%                 mc_random = importdata('mc_random.xlsx');
%                 mc_random = importdata('D:\mc_random_number_new_report.txt');
%                 Z= mc_random;
                
                numMethodInfo.pathSize = pathSize;
                numMethodInfo.numOfFactors = numOfFactors;
                
                numMethodInfo.totalTimeSteps = outInfo.totalTimeSteps;
                numMethodInfo.totalTimeStepsSize = outInfo.totalTimeStepsSize;
                
                totalTimeSteps = numMethodInfo.totalTimeSteps;
                totalTimeStepsSize = numMethodInfo.totalTimeStepsSize;
                
                % for modelStateGeneration
                numMethodInfo.modelTimeSteps = outInfo.modelTimeSteps;
                numMethodInfo.modelTimeStepsSize = outInfo.modelTimeStepsSize;
                
                numMethodInfo.pastTimeStepsSize = outInfo.pastTimeStepsSize;
                
                pastTimeStepsSize = numMethodInfo.pastTimeStepsSize;
                
                numMethodInfo.Z = Z;

                

                %generate model states for the given timeSteps
                modelStates = numMethod.generateModelStatesNFNew(valueDate,schedule,numMethodInfo,model);

                % numeraire info
%                 numMatTime = numMethodInfo.totalTimeSteps(end - numMethodInfo.pastTimeStepsSize);
                numMatTime = numMethodInfo.totalTimeSteps(end);
                
                days = DateDiff(schedule.payDates(schedule.numOfPeriod),valueDate);
                numMatTime = max(numMatTime, days);
                dfNumMat = model.DFRisky(numMatTime/365.0);
                
                numeraire.numMatTime = numMatTime;
                numeraire.dfNumMat = dfNumMat;
                
                % induct backward and calculate cashflow
                periodCount = schedule.numOfPeriod;
                
                % pricer single info initialize
                
                nonCall = PricerInfo(periodCount +1, pathSize);
                callable = PricerInfo(periodCount +1, pathSize);
                dfCol = PricerInfo(periodCount +1, pathSize);
                
                dummy1 = PricerInfo(periodCount +1, pathSize);
                dummy2 = PricerInfo(periodCount +1, pathSize);
                
                % additional floating leg info
                floatingLeg = PricerInfo(floatingSchedule.numOfPeriod, pathSize);
                % currentFloatingLegIdx for iteration in dealing with 
                % the floatingLeg cashflow
                currentFloatingLegIdx = floatingSchedule.numOfPeriod;
                
                % intermediate variables
                payoffStateA.cashflow = zeros(1, pathSize);
                payoffStateA.cashflow_npv = zeros(1, pathSize);
                
            
                % we calculate cashflow backward from maturityDate to valueDate
                % rangeAccrual coupons are added at the startDate of the Period
                % At maturity

                nominal = security.nominal;
                coupon = security.coupon;
                callYN = security.callYN;
                
                ralower = security.lower;
                raupper = security.upper;
                tenorLong = security.tenorL;
                tenorShort = security.tenorS;
                
                currentTime = numMethodInfo.totalTimeSteps(end);
                modelTimeStepsSize = numMethodInfo.modelTimeStepsSize;
                currentTimeIdx = numMethodInfo.totalTimeStepsSize;
%                 currentTimeIdx = numMethodInfo.totalTimeStepsSize - numMethodInfo.pastTimeStepsSize;
                
                currentNodeIdx = periodCount + 1;
                
                modelStateT = modelStates(numOfFactors*(max(currentTimeIdx-pastTimeStepsSize-1,0))+1:numOfFactors*(max(currentTimeIdx-pastTimeStepsSize-1,0))+2,:);
                
                %swap no nominal payoff
                payTime = DateDiff(schedule.payDates(schedule.numOfPeriod),valueDate);
                
                %even though there is no cashflow at the last structured
                %enddate, we add dummy cashflow in line with the note
                
                if payTime > 0
                    payoffStateA.cashflow = nominal*ones(1,pathSize)*0.0;

                    df_ep = model.discountFactorRisky(currentTime, payTime,numMatTime,modelStateT);
                    for k=1:pathSize
                        payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                    end

                    %process payoff at maturity
                    nonCall.ProcessAddPayoff(model,currentTime,numeraire, ...
                                         modelStateT,payoffStateA.cashflow,currentNodeIdx);

                    callable.ProcessAddPayoff(model,currentTime,numeraire, ...
                                         modelStateT,payoffStateA.cashflow,currentNodeIdx);

                    %df column
                    payoffStateA.cashflow = ones(1,pathSize);
                    for k=1:pathSize
                        payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                    end

                    dfCol.ProcessAddPayoff(model,currentTime,numeraire, ...
                                         modelStateT,payoffStateA.cashflow,currentNodeIdx);

                    %dummy column
                    payoffStateA.cashflow = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorLong,modelStateT);
                    dummy1.ProcessAddPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);

                    payoffStateA.cashflow = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                    dummy2.ProcessAddPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);

                  
                end
                
                lastTimeIdx = currentTimeIdx;
                totalTimeSteps = numMethodInfo.timeStepsInfo.totalTimeSteps;
                
                % flag for inclusive schedule
%                 simpleYN = false;
%                 simpleYN = true;
                if pricer.overHedgeYN == true 
                    simpleYN = 2;
                else
                    simpleYN = 1;
                end
                
                for i=periodCount:-1:1
                    % past schedule neglect
                    % we induct in the pastTimeSteps too
                    
%                     if DateDiff(valueDate,schedule.payDates(i)) >= 0, continue;end 
                    
                    startTimeIdx = numMethodInfo.timeStepsInfo.startTimeIdx(i);
                    endTimeIdx = numMethodInfo.timeStepsInfo.endTimeIdx(i);

                    inclusiveYN = numMethodInfo.timeStepsInfo.inclusiveYN(i);
                 %% FromToInduction start

                    %modelsStates induct backward(lastTimeIdx to endTimeIdx)
                    while endTimeIdx < lastTimeIdx
                        lastTimeIdx = lastTimeIdx -1;
                        if lastTimeIdx == endTimeIdx break;end
                    end

                    currentTimeIdx = lastTimeIdx;
                    currentTime = totalTimeSteps(currentTimeIdx);
                    % now we are at endTimeIdx
                    % variables for coupon valuation initialized
                    

                    cD = 0; % cumulated day count fraction
                    %cP shoul be declared as an array
%                     cP = 0;
                    cP = zeros(1,pathSize);
                    dI = 0; % dummy idx for loop
                    dcfC = 0;
                    dcfN = 0;
                    dD = 0;
                    idxL = zeros(1,pathSize);
                    idxS = zeros(1,pathSize);
                    spreadC = zeros(1,pathSize);
                    spreadN = zeros(1,pathSize);
                    probSpread = zeros(1,pathSize);
                    
                    numOfTimeSteps = numMethodInfo.timeStepsInfo.numOfTimeSteps(i);
                    
                    % if inclusive then we use simpleYN rangeProb
                    %if inclusive then we do not evaluate the last step but
                    % use fixing instead 
                    endTimeStep = 1;
                    % we let endTimeStep to be 1 and
                    % inside the for loop if currentTime ==0
                    % then we skip the range accrual check part
                    
                    if inclusiveYN
%                         simpleYN = true;
                        simpleYN = 1;
%                         endTimeStep = 2;
                        if pricer.overHedgeYN == true 
                            simpleYN = 2;
                        end
                    end
                    
                    for j=numOfTimeSteps:-1:endTimeStep
                        % we initialize payoffStateA as zeros to prevent
                        % reuse it;
                        payoffStateA.cashflow = zeros(1,pathSize);
                        % evaluate FromTo dealdescription
                        % last step is omitted
                        currentTime = totalTimeSteps(currentTimeIdx);
                        if currentTime > 0
                            dcfC = currentTime;

                            if j~= numOfTimeSteps
                                dD = (dcfN-dcfC);
                            end

                            modelStateT = modelStates(numOfFactors*(max(currentTimeIdx-pastTimeStepsSize-1,0))+1:numOfFactors*(max(currentTimeIdx-pastTimeStepsSize-1,0))+2,:);
                            idxL = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorLong,modelStateT);
                            idxS = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                            spreadC = idxL - idxS;

                            probSpread = model.SRPr(spreadC,spreadN,ralower,raupper,simpleYN,pathSize)*dD;
                            cD = cD + dD;
                            cP = cP + probSpread;

                            dcfN = dcfC;
                            spreadN = spreadC;
                        else
                            % we updated cD only since fixing is treated in
                            % rangedays
                            % fix it later !
                            
                            dcfC = currentTime;

                            if j~= numOfTimeSteps
                                dD = (dcfN-dcfC);
                            end
                            cD = cD + dD;
                            
                            dcfN = dcfC;
                            aaa =1.0;
                        end 
                        % evaluate secodary dealDescription(floating)
                        % floatingLeg
                        idxFloat = find(currentTime == outInfo.floatingEventTime);
                        
                        if currentTime == 5297
                            aaa =1;
                        end
                        
                        % we do not evaluate floating leg at the end of
                        % coupon Schedule for avoding double counting
                        if ~isempty(idxFloat) && (j~= numOfTimeSteps)
                            iterFloat = 1;
                            foundIterFloatYN= false;
                            for idx11 = length(floatingSchedule.startDates):-1:1
                                if outInfo.floatingEventTime(idxFloat) == DateDiff(floatingSchedule.startDates(idx11),valueDate)
                                    iterFloat = idx11;
                                    foundIterFloatYN = true;
                                    break;
                                end
                            end
                            
                            if ~foundIterFloatYN
                                error('floatingEventTime is not in the floatingSchedule startDates')
                            end
                            
                            % only those cashflow that paydate is after
                            % valueDate will be added
                            if DateDiff(floatingSchedule.payDates(idx11),valueDate) > 0
                                % floatingIndex is not fixed    
                                if outInfo.floatingEventTime(idxFloat) > 0
                                    dcfFloating = floatingSchedule.dayCountFraction(iterFloat);
                                    if strcmp(floatingIndex,'Libor3M')
                                        tenorFloatingCD = 0.25;
                                        idxFloatingCD = model.Libor_SimpleDate(valueDate,currentTime,currentTime,numMatTime, tenorFloatingCD,modelStateT);
                                        % -1 for floating pay 
                                        payoffStateA.cashflow = -1.0 * nominal * (idxFloatingCD + floatingSpread) * dcfFloating;
                                    end

                                else
                                    % floatingIndex is fixed
                                    sizeIdxFloat = length(idxFloat);
                                    if strcmp(floatingIndex,'Libor3M')
                                        tenorFloatingCD = 0.25;
                                        allFixings = security.allFixings;
                                        KRWLibor3MFixings = allFixings('KRWLibor3M');

                                        dcfFloating = floatingSchedule.dayCountFraction(iterFloat);
                                        idxFloatingCD = KRwLibor3MFixings(StrDate(floatingSchedule.resetDates(iterFloat),'str'));
        %                                 idxFloatingCD = model.Libor_SimpleDate(valueDate,currentTime,currentTime,numMatTime, tenorFloatingCD,modelStateT);
                                        % -1 for floating pay
                                        for k=1:pathSize
                                            payoffStateA.cashflow(k) = -1.0 * nominal * (idxFloatingCD + floatingSpread) * dcfFloating;
                                        end
                                    end
                                end
                                
                                payTime = DateDiff(floatingSchedule.payDates(iterFloat),valueDate);
                                %event is at startDate so we need to discount the
                                %cashflow first from paydate to startdate
                                df_ep = model.discountFactorRisky(currentTime, payTime,numMatTime,modelStateT);

                                for k=1:pathSize
                                    payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                                end
                                
                                % process floatingLeg payoff

                                currentNodeIdx = i;

                                nonCall.ProcessAddPayoff(model,currentTime,numeraire, ...
                                                 modelStateT,payoffStateA.cashflow,currentNodeIdx);

                                callable.ProcessAddPayoff(model,currentTime,numeraire, ...
                                                 modelStateT,payoffStateA.cashflow,currentNodeIdx);

                                % process additional payoff
                                floatingLeg.ProcessAddPayoff(model,currentTime,numeraire, ...
                                                 modelStateT,payoffStateA.cashflow,iterFloat);
                                
                            end
                        end
                            

                            
                            
                        
                        %induct backward to the next time step
                        % one time backward induction
                        lastTimeIdx = currentTimeIdx;

                        if startTimeIdx < lastTimeIdx
                            lastTimeIdx = lastTimeIdx -1;
                            currentTimeIdx = lastTimeIdx;
                        end

                    end
                    
                    %% FromToInduction end
        % if the current period is inclusive, then the start date is not evaluated in the above for loop.
		% instead the fixing will be treated separately below.
		% however current_time_idx is updated at the end of the for loop.
		% so we need to update model_states accordingly.
		% if model_states were updated previously then the following additionaly step doesn't affect the result anyway.
                    currentTime = totalTimeSteps(currentTimeIdx);
                    
                    % if 
                    modelStateT = modelStates(numOfFactors*(max(currentTimeIdx-pastTimeStepsSize-1,0))+1:numOfFactors*(max(currentTimeIdx-pastTimeStepsSize-1,0))+2,:);
                    
                    %% AtDate evaluation
                    % Adding a new cashflow to the payoff
                    % for structured coupon payDate should be after
                    % valueDate
                    
                    if DateDiff(schedule.payDates(i),valueDate) > 0
                        dcf = schedule.dayCountFraction(i);

                        if inclusiveYN
                            % we add valuation date
                            cD = schedule.dayCount(i);
                            rangeDays =  security.rangeDays(i);
                            payoffStateA.cashflow = nominal * coupon * dcf * (rangeDays + cP)/cD;
                        else
                            payoffStateA.cashflow = nominal * coupon * dcf * cP/cD;
                        end
                        % eventDate(i) = startDate(i) therefore there is no induction in modelStates
                        % i.e. currentTimeIdx has not changed
                        % no need to update modelStateT
                        % modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:end,:);
                        % since cashflow's are added at the startDate(i) we need to
                        % multiply cashflow with the df(startDate(i), payDate(i))
                        payTime = DateDiff(schedule.payDates(i),valueDate);

                        %event is at startDate so we need to discount the
                        %cashflow first from paydate to startdate
                        df_ep = model.discountFactorRisky(currentTime, payTime,numMatTime,modelStateT);

                        for k=1:pathSize
                            payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                        end

                        % process payoff

                        currentNodeIdx = i;

                        nonCall.ProcessAddPayoff(model,currentTime,numeraire, ...
                                         modelStateT,payoffStateA.cashflow,currentNodeIdx);

                        callable.ProcessAddPayoff(model,currentTime,numeraire, ...
                                         modelStateT,payoffStateA.cashflow,currentNodeIdx);

                        %df column
                        payoffStateA.cashflow = ones(1,pathSize);
                        for k=1:pathSize
                            payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                        end

                        dfCol.ProcessAddPayoff(model,currentTime,numeraire, ...
                                         modelStateT,payoffStateA.cashflow,currentNodeIdx);

                        %dummy column
                        payoffStateA.cashflow = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorLong,modelStateT);
                        dummy1.ProcessAddPayoff(model,currentTime,numeraire, ...
                                         modelStateT,payoffStateA.cashflow,currentNodeIdx);

                        payoffStateA.cashflow = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                        dummy2.ProcessAddPayoff(model,currentTime,numeraire, ...
                                         modelStateT,payoffStateA.cashflow,currentNodeIdx);

                    end
                    % if exercise is possible
                    if i == 1
                        dummaaa = 1;
                    end
                    
                    % call event happens  at the start of the period
                    % at call date, by default, call option is expired
                    
                    if callYN(i) && (DateDiff(valueDate,callSchedule.resetDates(i)) < 0)
                        % 1.undiscount callable.payoff
                        % 2.build the state variables matrix
                        % 3.regress 1's with the given state variables
                        % (model states 1,2)
                        % 4. exercise per each path in
                        % discount the new payoff and assign into callable
                        % column
                        
                        %1 undiscount callable.payoff and get the
                        %contiValue
                       
                        %exercise value swap
                        exerciseValue = security.nominal*ones(1,pathSize) * 0.0;
                        payTime = DateDiff(callSchedule.payDates(i),valueDate);
                        %event is at startDate so we need to discount the
                        %cashflow first from paydate to startdate
                        df_ep = model.discountFactorRisky(currentTime, payTime,numMatTime,modelStateT);
                        for k=1:pathSize
                            exerciseValue(k) = exerciseValue(k) * df_ep(k);
                        end
                        
                        contiValue = zeros(1,pathSize);
                        onePayoff = ones(1,pathSize);
                        onePayoffD = model.discountPayoffRisky(currentTime,numMatTime ...
                        ,modelStateT,onePayoff);
                        contiValue = callable.payoff./onePayoffD;
                        
                        %Nominal Regression test
                        contiValue = contiValue/10000;
                        
                        % build the state variabls matrix
                        decisionVar1 = modelStateT(1,:);
                        decisionVar2 = modelStateT(2,:);
                        % we use upto 2nd order polynomial as regression
                        % equatioin
                        dimensionLS = 6;
                        regMatrix = ones(pathSize,dimensionLS);
                        
                        for idx_r=1:pathSize
                            regMatrix(idx_r,2) = decisionVar1(idx_r);
                            regMatrix(idx_r,3) = decisionVar2(idx_r);
                            regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r);
                            regMatrix(idx_r,5) = decisionVar1(idx_r)*decisionVar2(idx_r);
                            regMatrix(idx_r,6) = decisionVar2(idx_r)*decisionVar2(idx_r);
                        end
                        
                        % simple regression <- for path dependent option
                        % we should change it
                        % beta  : regression coefficients
                        beta =zeros(dimensionLS,1);
                        beta = regress(contiValue',regMatrix);
                        regContiValue = regMatrix*beta;
                       
                         %Nominal Regression test
                        regContiValue = regContiValue * 10000;
                        
                        exercisedPayoff = regContiValue;
                        % if regContiValue is bigger than exerciseValue
                        % then issue exercise call option
                        for idx_r=1:pathSize
                            if regContiValue(idx_r) >= exerciseValue(idx_r) 
                                exercisedPayoff(idx_r) = exerciseValue(idx_r);
                            end
                        end
                        
                        % we discount the new payoff and 
                        % assign it into the existing callable payoff
                        callable.payoff = model.discountPayoffRisky(currentTime,numMatTime ...
                        ,modelStateT,exercisedPayoff);
                        
                    end
                end

                
                nonCall.npv = mean(nonCall.payoff);
                out.nonCall = nonCall;
                callable.npv = mean(callable.payoff);
                out.callable = callable;
                
                floatingLeg.npv = mean(floatingLeg.payoff);
                out.floatingLeg = floatingLeg;
                
                out.dfCol = dfCol;
                out.dummy1 = dummy1;
                out.dummy2 = dummy2;
                out.daycountFraction = schedule.dayCountFraction;
                
        end
        
        function out = computePriceClosedSimple(pricer,valueDate)
                security = pricer.m_security;
                numMethod = pricer.m_numMethod;
                model = pricer.m_model;
                schedule = security.couponSchedule;
                
                 %floating
                floatingSchedule = security.floatingSchedule;
                floatingSpread = security.floatingSpread;
                floatingIndex = security.floatingIndex;
                
                % numMethodTimeSteps Generate
                mcOneTimeStep = numMethod.mcOneTimeStep;
%                 outInfo = numMethod.numMethodTimeStepsGenerateDW(valueDate,schedule,mcOneTimeStep);
                
                outInfo = numMethod.numMethodTimeStepsGenerateDWSwapSimple(valueDate,...
                                    schedule,floatingSchedule,mcOneTimeStep);
                                
                numMethodInfo.timeStepsInfo = outInfo;

                % mc random number init
                %pathSize = 2^16;
                pathSize = numMethod.pathSize;
                
                pathSize = 1;
                
%                 % generate random numbers and modelstates forward
%                 rng('default');
%                 rng(0);
% 
                numOfFactors = 2;
% 
%                 %Z= randn(numOfFactors*outInfo.totalTimeStepsSize,pathSize/2);
%                 Z= randn(pathSize/2,numOfFactors*outInfo.totalTimeStepsSize);
%                 Z=[Z;-Z];
%                 Z=Z';
%                 
%                 
%                 %TimeInversion
%                 sizeZ = size(Z,1);
%                 A = zeros(sizeZ,sizeZ);
%                 for i=1:sizeZ
%                     A(i,sizeZ + 1 -i) = 1;
%                 end
%                 Z = A*Z;
% 
%                 % we check with the c, so we use generated random numbers
% %                 mc_random = importdata('mc_random.xlsx');
% %                 mc_random = importdata('D:\mc_random_number_new_report.txt');
% %                 Z= mc_random;
%                 
                numMethodInfo.pathSize = pathSize;
                numMethodInfo.numOfFactors = numOfFactors;
                
                numMethodInfo.totalTimeSteps = outInfo.totalTimeSteps;
                numMethodInfo.totalTimeStepsSize = outInfo.totalTimeStepsSize;
                
                totalTimeSteps = numMethodInfo.totalTimeSteps;
                totalTimeStepsSize = numMethodInfo.totalTimeStepsSize;
                
                numMethodInfo.modelTimeSteps = outInfo.modelTimeSteps;
                numMethodInfo.modelTimeStepsSize = outInfo.modelTimeStepsSize;
                
                numMethodInfo.pastTimeStepsSize = outInfo.pastTimeStepsSize;
                
                pastTimeStepsSize = numMethodInfo.pastTimeStepsSize;
%                 numMethodInfo.Z = Z;
% 
% 
% 
%                 %generate model states for the given timeSteps
%                 modelStates = numMethod.generateModelStates(valueDate,schedule,numMethodInfo,model);

                % numeraire info
                numMatTime = numMethodInfo.totalTimeSteps(end);
                dfNumMat = model.DF(numMatTime/365.0);
                
                % induct backward and calculate cashflow
            
                periodCount = schedule.numOfPeriod;
                fltPeriodCount = floatingSchedule.numOfPeriod;
                % cashflow columns initialize
                % 1st nonCall
                nonCall.npv = 0;
                nonCall.cashflow_e = zeros(periodCount + 1,1);
                nonCall.cashflow_npv_e = zeros(periodCount + 1,1);
                nonCall.rangeProb_e = zeros(periodCount + 1,1);
                nonCall.df_e = zeros(periodCount + 1,1);
                nonCall.df_c = zeros(periodCount + 1,1);

                % debug information, cashflow & npv per path
                nonCall.payoff = zeros(1,pathSize);
                nonCall.payoffStates.cashflow = zeros(periodCount+1,pathSize);
                nonCall.payoffStates.cashflow_npv = zeros(periodCount+1,pathSize);
                nonCall.payoffStates.df = zeros(periodCount + 1,pathSize);
                
                %floatingLeg
                floatingLeg.npv = 0;
                floatingLeg.cashflow_e = zeros(fltPeriodCount,1);
                floatingLeg.cashflow_npv_e = zeros(fltPeriodCount,1);
                floatingLeg.rangeProb_e = zeros(fltPeriodCount,1);
                floatingLeg.df_e = zeros(fltPeriodCount,1);
                floatingLeg.df_c = zeros(fltPeriodCount,1);

                floatingLeg.payoff = zeros(1,pathSize);
                floatingLeg.payoffStates.cashflow = zeros(fltPeriodCount,pathSize);
                floatingLeg.payoffStates.cashflow_npv = zeros(fltPeriodCount,pathSize);
                floatingLeg.payoffStates.df = zeros(fltPeriodCount,pathSize);
                
                % intermediate variables
                payoffStateA.cashflow = zeros(1, pathSize);
                payoffStateA.cashflow_npv = zeros(1, pathSize);
                payoffStateA.df = zeros(1,pathSize);
                
                % we calculate cashflow backward from maturityDate to valueDate
                % rangeAccrual coupons are added at the startDate of the Period
                % At maturity

                nominal = security.nominal;
                coupon = security.coupon;
                callYN = security.callYN;
                
                currentTime = numMethodInfo.totalTimeSteps(end);
                totalTimeSteps = numMethodInfo.timeStepsInfo.totalTimeSteps;
                totalTimeStepsSize = numMethodInfo.timeStepsInfo.totalTimeStepsSize;
                if totalTimeStepsSize == 0 || (totalTimeStepsSize == 1 && totalTimeSteps(1) == 0) 
                    out.nonCall = nonCall;
                    return;
                end
                
                currentTimeIdx = totalTimeStepsSize;
                
                currentNodeIdx = periodCount + 1;
                
%                 modelStateT = modelStates(numOfFactors*(totalTimeStepsSize-1)+1:numOfFactors*(totalTimeStepsSize-1)+2,:);
                payoffStateA.cashflow = nominal*ones(1,pathSize)*0.0;
                %process payoff at maturity
                nonCall.payoffStates.cashflow(currentNodeIdx,:) = payoffStateA.cashflow;
                
%                 payoffStateA.cashflow_npv = model.discountPayoff(currentTime,numMatTime ...
%                     ,modelStateT,payoffStateA.cashflow);
%                 
                payoffStateA.cashflow_npv = model.DFRisky(currentTime/365.0)*payoffStateA.cashflow;
                
                nonCall.payoffStates.cashflow_npv(currentNodeIdx,:) = payoffStateA.cashflow_npv ;
                
%                 payoffStateA.df = model.discountPayoff(currentTime,numMatTime ...
%                     ,modelStateT,1.0*ones(1,pathSize));
                payoffStateA.df = model.DFRisky(currentTime/365.0) *ones(1,pathSize);
                    

                nonCall.payoffStates.df(currentNodeIdx,:) = payoffStateA.df;
                nonCall.payoff = nonCall.payoff + payoffStateA.cashflow_npv;
                
                nonCall.cashflow_e(currentNodeIdx) = mean(payoffStateA.cashflow);
                nonCall.cashflow_npv_e(currentNodeIdx) = mean(payoffStateA.cashflow_npv);
                nonCall.df_e(currentNodeIdx) = mean(payoffStateA.df);
                nonCall.df_c(currentNodeIdx) = model.DFRisky(currentTime/365.0);
                
%                 % callable at maturity is the same as noncallable
%                 callable.payoffStates.cashflow(currentNodeIdx,:) = nonCall.payoffStates.cashflow(currentNodeIdx,:);
%                 callable.payoffStates.cashflow_npv(currentNodeIdx,:) = nonCall.payoffStates.cashflow_npv(currentNodeIdx,:);
%                 callable.payoff = callable.payoff + callable.payoffStates.cashflow_npv(currentNodeIdx,:);
%                 
%                 callable.cashflow_e(currentNodeIdx) = nonCall.cashflow_e(currentNodeIdx);
%                 callable.cashflow_npv_e(currentNodeIdx) = nonCall.cashflow_npv_e(currentNodeIdx);
                  
                lastTimeIdx = currentTimeIdx;
                
                
                % flag for inclusive schedule
%                 simpleYN = false;
                simpleYN = 1;
                
                for i=periodCount:-1:1
                    % past schedule neglect
%                     if DateDiff(valueDate,schedule.payDates(i)) >= 0, continue;end 
                    startTimeIdx = numMethodInfo.timeStepsInfo.startTimeIdx(i);
                    endTimeIdx = numMethodInfo.timeStepsInfo.endTimeIdx(i);

                    inclusiveYN = numMethodInfo.timeStepsInfo.inclusiveYN(i);
                     %% FromToInduction start

                    %modelsStates induct backward(lastTimeIdx to endTimeIdx)
                    while endTimeIdx < lastTimeIdx
                        lastTimeIdx = lastTimeIdx -1;
                        if lastTimeIdx == endTimeIdx break;end
                    end

                    currentTimeIdx = lastTimeIdx;
                    currentTime = totalTimeSteps(currentTimeIdx);
                    % now we are at endTimeIdx
                    % variables for coupon valuation initialized
                    ralower = security.lower;
                    raupper = security.upper;
                    tenorLong = security.tenorL;
                    tenorShort = security.tenorS;

                    cD = 0; % cumulated day count fraction
                    cP = zeros(1,pathSize);
                    dI = 0; % dummy idx for loop
                    dcfC = 0;
                    dcfN = 0;
                    dD = 0;
                    idxL = zeros(1,pathSize);
                    idxS = zeros(1,pathSize);
                    spreadC = zeros(1,pathSize);
                    spreadN = zeros(1,pathSize);
                    probSpread = zeros(1,pathSize);
                    
                    numOfTimeSteps = numMethodInfo.timeStepsInfo.numOfTimeSteps(i);
                    
                    % if inclusive then we use simpleYN rangeProb
                    %if inclusive then we do not evaluate the last step but
                    % use fixing instead 
                    endTimeStep = 1;
                    if inclusiveYN
%                         simpleYN = true;
                        simpleYN = 1;
%                         endTimeStep = 2;
                    end
                    
                    for j=numOfTimeSteps:-1:endTimeStep
                        payoffStateA.cashflow = zeros(1,pathSize);
                        % evaluate dealdescription
                        % last step is omitted
                        currentTime = totalTimeSteps(currentTimeIdx);
                        if currentTime > 0
                            %dcfC = currentTime/365.0;
                            dcfC = currentTime;

                            if j~= numOfTimeSteps
                                dD = (dcfN-dcfC);
                            end

                            toDate = AddDate(valueDate,currentTime,'day');
                            tpDate = schedule.payDates(i);
                            spreadC = model.CMSSpreadDigitalRangeDate(valueDate,toDate,tpDate,tenorLong,tenorShort,0.25,ralower,raupper);

                            if simpleYN
                                probSpread = spreadC*dD;
                            else
                                probSpread = spreadC*dD;
    %                             probSpread = 0.5*(spreadC + spreadN)*dD;

                            end


                            cD = cD + dD;
                            cP = cP + probSpread;

                            dcfN = dcfC;
                            spreadN = spreadC;
                        else
                            % we updated cD only since fixing is treated in
                            % rangedays
                            % fix it later !
                            
                            dcfC = currentTime;

                            if j~= numOfTimeSteps
                                dD = (dcfN-dcfC);
                            end
                            cD = cD + dD;
                            
                            dcfN = dcfC;
                            aaa =1.0;
                        end
                        % evaluate secodary dealDescription(floating)
                        % floatingLeg
                        idxFloat = find(currentTime == outInfo.floatingEventTime);
                        
                        % we do not evaluate floating leg at the endof
                        % coupon Schedule for avoding double counting
                        if ~isempty(idxFloat) && (j~= numOfTimeSteps)
                            iterFloat = 1;
                            foundIterFloatYN= false;
                            for idx11 = length(floatingSchedule.startDates):-1:1
                                if outInfo.floatingEventTime(idxFloat) == DateDiff(floatingSchedule.startDates(idx11),valueDate)
                                    iterFloat = idx11;
                                    foundIterFloatYN = true;
                                    break;
                                end
                            end

                            if ~foundIterFloatYN
                                error('floatingEventTime is not in the floatingSchedule startDates')
                            end
                            
                            if DateDiff(floatingSchedule.payDates(idx11),valueDate) > 0
                                if outInfo.floatingEventTime(idxFloat) > 0

                                    dcfFloating = floatingSchedule.dayCountFraction(iterFloat);
                                    if strcmp(floatingIndex,'Libor3M')
                                        tenorFloatingCD = 0.25;
    %                                         idxFloatingCD = model.Libor_SimpleDate(valueDate,currentTime,currentTime,numMatTime, tenorFloatingCD,modelStateT);
                                        idxFloatingCD = model.LiborDate(valueDate,currentTime,tenorFloatingCD);
                                        % -1 for floating pay 
                                        payoffStateA.cashflow = -1.0 * nominal * (idxFloatingCD + floatingSpread) * dcfFloating;
                                    end

                                else
                                    % floatingIndex is fixed
                                    %if floatingCashflow 1 is fixed not paid
                                    %and also next floatingCashflow2 is fixed
                                    %then
                                    %idxFloat = find(currentTime == outInfo.floatingEventTime);
                                    % will give two answers
                                    % then we use sizeIdxFloat, length(idxfloat) to 
                                    % access the right floating Cashflow
                                    sizeIdxFloat = length(idxFloat);
                                    %
                                    if strcmp(floatingIndex,'Libor3M')
                                        tenorFloatingCD = 0.25;
                                        allFixings = security.allFixings;
                                        KRwLibor3MFixings = allFixings('KRWLibor3M');
        
                                        dcfFloating = floatingSchedule.dayCountFraction(iterFloat);
                                        idxFloatingCD = KRwLibor3MFixings(StrDate(floatingSchedule.resetDates(iterFloat),'str'));
        %                                 idxFloatingCD = model.Libor_SimpleDate(valueDate,currentTime,currentTime,numMatTime, tenorFloatingCD,modelStateT);
                                        % -1 for floating pay
                                        for k=1:pathSize
                                            payoffStateA.cashflow(k) = -1.0 * nominal * (idxFloatingCD + floatingSpread) * dcfFloating;
                                        end
                                    end
                                end
                                
                                payTime = DateDiff(floatingSchedule.payDates(iterFloat),valueDate);
                                %event is at startDate so we need to discount the
                                %cashflow first from paydate to startdate
                                %cashflow first from paydate to startdate
                                currentNodeIdx = i;
                                nonCall.payoffStates.cashflow(currentNodeIdx,:) = nonCall.payoffStates.cashflow(currentNodeIdx,:) + payoffStateA.cashflow;
                                payoffStateA.cashflow_npv = payoffStateA.cashflow * model.DFRisky(payTime/365.0);

                                nonCall.payoffStates.cashflow_npv(currentNodeIdx,:) = nonCall.payoffStates.cashflow_npv(currentNodeIdx,:) + payoffStateA.cashflow_npv;

                                nonCall.payoff = nonCall.payoff + payoffStateA.cashflow_npv;

                                nonCall.cashflow_e(currentNodeIdx) = nonCall.cashflow_e(currentNodeIdx) + mean(payoffStateA.cashflow);
                                nonCall.cashflow_npv_e(currentNodeIdx) = nonCall.cashflow_npv_e(currentNodeIdx) + mean(payoffStateA.cashflow_npv);

                                floatingLeg.payoffStates.cashflow(iterFloat,:) = floatingLeg.payoffStates.cashflow(iterFloat,:) + payoffStateA.cashflow;
                                floatingLeg.payoffStates.cashflow_npv(iterFloat,:) = floatingLeg.payoffStates.cashflow_npv(iterFloat,:) + payoffStateA.cashflow_npv;
                                floatingLeg.payoff = floatingLeg.payoff + payoffStateA.cashflow_npv;

                                floatingLeg.cashflow_e(iterFloat) = floatingLeg.cashflow_e(iterFloat) + mean(payoffStateA.cashflow);
                                floatingLeg.cashflow_npv_e(iterFloat) = floatingLeg.cashflow_npv_e(iterFloat) + mean(payoffStateA.cashflow_npv);

                            end
                        end
                               
                            
                        
                        %induct backward to the next time step
                        % one time backward induction
                        lastTimeIdx = currentTimeIdx;

                        if startTimeIdx < lastTimeIdx
                            lastTimeIdx = lastTimeIdx -1;
                            currentTimeIdx = lastTimeIdx;
                        end
                    end

                    
                    %% FromToInduction end
                    %% AtDate evaluation
                    % Adding a new cashflow to the payoff

                    dcf = schedule.dayCountFraction(i);
                    if DateDiff(schedule.payDates(i),valueDate) > 0
                        if inclusiveYN
                            % we add valuation date
                            cD = schedule.dayCount(i);
                            rangeDays =  security.rangeDays(i);
                            payoffStateA.cashflow = nominal * coupon * dcf * (rangeDays + cP)/cD;
                        else
                            payoffStateA.cashflow = nominal * coupon * dcf * cP/cD;
                        end
           
                        payTime = DateDiff(schedule.payDates(i),valueDate);

                        currentNodeIdx = i;

                        % 1st nonCall column
                        nonCall.payoffStates.cashflow(currentNodeIdx,:) = nonCall.payoffStates.cashflow(currentNodeIdx,:) + payoffStateA.cashflow;
                        payoffStateA.cashflow_npv = payoffStateA.cashflow * model.DFRisky(payTime/365.0);

    %                     payoffStateA.cashflow_npv = model.discountPayoff(currentTime,numMatTime ...
    %                     ,modelStateT,payoffStateA.cashflow);
                        nonCall.payoffStates.cashflow_npv(currentNodeIdx,:) = nonCall.payoffStates.cashflow_npv(currentNodeIdx,:) + payoffStateA.cashflow_npv;
                        payoffStateA.df = model.DFRisky(payTime/365.0);
    %                     payoffStateA.df = model.discountPayoff(currentTime,numMatTime ...
    %                     ,modelStateT,1.0*ones(1,pathSize));
                        nonCall.payoffStates.df(currentNodeIdx,:) = nonCall.payoffStates.df(currentNodeIdx,:) + payoffStateA.df;

                        nonCall.payoff = nonCall.payoff + payoffStateA.cashflow_npv;

                        nonCall.cashflow_e(currentNodeIdx) = nonCall.cashflow_e(currentNodeIdx) + mean(payoffStateA.cashflow);
                        nonCall.cashflow_npv_e(currentNodeIdx) = nonCall.cashflow_npv_e(currentNodeIdx) + mean(payoffStateA.cashflow_npv);

                        nonCall.df_e(currentNodeIdx) = nonCall.df_e(currentNodeIdx) + mean(payoffStateA.df);
                        nonCall.df_c(currentNodeIdx) = nonCall.df_c(currentNodeIdx) + model.DFRisky(currentTime/365.0);
 
                    end
                    % if exercise is possible
                    if i == 1
                        dummaaa = 1;
                    end
       
                end

                nonCall.npv = mean(nonCall.payoff);
                out.nonCall = nonCall;
                floatingLeg.npv = mean(floatingLeg.payoff);
                out.floatingLeg = floatingLeg;
%                 callable.npv = mean(callable.payoff);
%                 out.pricerInfo.callable = callable;
                out.daycountFraction = schedule.dayCountFraction;
                
        end
        
        
        
        
    end
    
end



In [ ]:
classdef DailyWeeklyRiskyPricer_CMSSpreadAndEquityDualRAPutAtMat < H_RootObject
    %Pricer(Security,Model,NumMethod)
    %Prcer is constructed from Security, Model, NumMethod 
    
    properties
        m_security
        m_model
        m_numMethod
        overHedgeYN
        
    end
    
    methods
        function pricer = DailyWeeklyRiskyPricer_CMSSpreadAndEquityDualRAPutAtMat(security,model,numMethod,overHedgeYN)
                pricer.m_security = security;
                pricer.m_model = model;
                pricer.m_numMethod = numMethod;
                pricer.overHedgeYN = overHedgeYN;
                
        end
        
        function out = computePriceMCCallableGeneric(pricer,valueDate,pricerOption)
                security = pricer.m_security;
                numMethod = pricer.m_numMethod;
                model = pricer.m_model;
                schedule = security.couponSchedule;
                callSchedule = security.callSchedule;
                
                % numMethodTimeSteps Generate
                mcOneTimeStep = numMethod.mcOneTimeStep;
                outInfo = numMethod.numMethodTimeStepsGenerateDW(valueDate,schedule,mcOneTimeStep);
            
                numMethodInfo.timeStepsInfo = outInfo;

                % mc random number init
                %pathSize = 2^16;
                pathSize = numMethod.pathSize;

                % generate random numbers and modelstates forward
                rng('default');
                rng(0);

                % LGM1F : one factor
                % LGM2F : two factor
                
                numOfFactors = model.numOfFactor;

                %Z= randn(numOfFactors*outInfo.totalTimeStepsSize,pathSize/2);
                Z= randn(pathSize/2,numOfFactors*outInfo.totalTimeStepsSize);
                Z=[Z;-Z];
                Z=Z';
                
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;

                % we check with the c code, so we use generated random numbers
%                 mc_random = importdata('mc_random.xlsx');
%                 mc_random = importdata('D:\mc_random_number_new_report.txt');
%                 Z= mc_random;
                
                numMethodInfo.pathSize = pathSize;
                numMethodInfo.numOfFactors = numOfFactors;
                numMethodInfo.totalTimeSteps = outInfo.totalTimeSteps;
                numMethodInfo.totalTimeStepsSize = outInfo.totalTimeStepsSize;
                numMethodInfo.Z = Z;



                %numMethod generate model states for the given timeSteps
                modelStates = numMethod.generateModelStatesNF(valueDate,schedule,numMethodInfo,model);

                % numeraire info
                numMatTime = numMethodInfo.totalTimeSteps(end);
                days = DateDiff(schedule.payDates(schedule.numOfPeriod),valueDate);
                numMatTime = max(numMatTime, days);
                dfNumMat = model.DFRisky(numMatTime/365.0);
                
                numeraire.numMatTime = numMatTime;
                numeraire.dfNumMat = dfNumMat;
                
                % induct backward and calculate cashflow
            
                periodCount = schedule.numOfPeriod;
                
                % pricer single info initialize
                
                nonCall = PricerInfo(periodCount +1, pathSize);
                callable = PricerInfo(periodCount +1, pathSize);
                dfCol = PricerInfo(periodCount +1, pathSize);
                
                dummy1 = PricerInfo(periodCount +1, pathSize);
                dummy2 = PricerInfo(periodCount +1, pathSize);
                
                
                % intermediate variables
                payoffStateA.cashflow = zeros(1, pathSize);
                payoffStateA.cashflow_npv = zeros(1, pathSize);
                
            
                % we calculate cashflow backward from maturityDate to valueDate
                % rangeAccrual coupons are added at the startDate of the Period
                % At maturity

                nominal = security.nominal;
                coupon = security.coupon;
                callYN = security.callYN;
                
                ralower = security.lower;
                raupper = security.upper;
                
                ralowerS = security.lowerS;
                raupperS = security.upperS;
                
                tenorCD = security.tenor;
                
                tenorLong = security.tenorL;
                tenorShort = security.tenorS;
                
                currentTime = numMethodInfo.totalTimeSteps(end);
                totalTimeStepsSize = numMethodInfo.totalTimeStepsSize;
                currentTimeIdx = totalTimeStepsSize;
                
                currentNodeIdx = periodCount + 1;
                
                modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:numOfFactors*(currentTimeIdx-1)+numOfFactors,:);
                
                payoffStateA.cashflow = nominal*ones(1,pathSize);
                
                payTime = DateDiff(schedule.payDates(schedule.numOfPeriod),valueDate);
                df_ep = model.discountFactorRisky(currentTime, payTime,numMatTime,modelStateT);
                for k=1:pathSize
                    payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                end
                
                %process payoff at maturity
                nonCall.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                
                callable.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                
                %df column
                payoffStateA.cashflow = ones(1,pathSize);
                for k=1:pathSize
                    payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                end
                
                dfCol.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                                 
                %dummy column

                if strcmp(pricerOption,'PSA')
                    payoffStateA.cashflow = model.CMS_PSA_SimpleDate('KRW',valueDate,currentTime,numMatTime, tenorLong,modelStateT);
                    dummy1.ProcessPayoff(model,currentTime,numeraire, ...
                                 modelStateT,payoffStateA.cashflow,currentNodeIdx);
                    
                    payoffStateA.cashflow = model.CMS_PSA_SimpleDate('USD',valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                    dummy2.ProcessPayoff(model,currentTime,numeraire, ...
                                 modelStateT,payoffStateA.cashflow,currentNodeIdx);
                
                else
                    payoffStateA.cashflow = model.Libor_SimpleDate('KRW',valueDate,currentTime,currentTime,numMatTime, tenorLong,modelStateT); 
                    dummy1.ProcessPayoff(model,currentTime,numeraire, ...
                                 modelStateT,payoffStateA.cashflow,currentNodeIdx);
                             
                    payoffStateA.cashflow = model.Libor_SimpleDate('USD',valueDate,currentTime,currentTime,numMatTime, tenorShort,modelStateT); 
                    dummy2.ProcessPayoff(model,currentTime,numeraire, ...
                                 modelStateT,payoffStateA.cashflow,currentNodeIdx);
                    
                end
                

%                 payoffStateA.cashflow = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                
%                 payoffStateA.cashflow = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                
                
                                 
                  

                lastTimeIdx = currentTimeIdx;
                totalTimeSteps = numMethodInfo.timeStepsInfo.totalTimeSteps;
                
                % flag for inclusive schedule
%                 simpleYN = false;
%                 simpleYN = true;
                if pricer.overHedgeYN == true 
                    simpleYN = 2;
                else
                    simpleYN = 1;
                end
                
                for i=periodCount:-1:1
                    % past schedule neglect
                    if DateDiff(valueDate,schedule.payDates(i)) >= 0, continue;end 
                    startTimeIdx = numMethodInfo.timeStepsInfo.startTimeIdx(i);
                    endTimeIdx = numMethodInfo.timeStepsInfo.endTimeIdx(i);

                    inclusiveYN = numMethodInfo.timeStepsInfo.inclusiveYN(i);
                 %% FromToInduction start

                    %modelsStates induct backward(lastTimeIdx to endTimeIdx)
                    while endTimeIdx < lastTimeIdx
                        lastTimeIdx = lastTimeIdx -1;
                        if lastTimeIdx == endTimeIdx break;end
                    end

                    currentTimeIdx = lastTimeIdx;
                    currentTime = totalTimeSteps(currentTimeIdx);
                    % now we are at endTimeIdx
                    % variables for coupon valuation initialized
                    

                    cD = 0; % cumulated day count fraction
                    cP = 0; 
                    dI = 0; % dummy idx for loop
                    dcfC = 0;
                    dcfN = 0;
                    dD = 0;
                    idxCD = zeros(1,pathSize);
                    idxL = zeros(1,pathSize);
                    idxS = zeros(1,pathSize);
                    
                    idxC = zeros(1,pathSize);
                    idxN = zeros(1,pathSize);
                    
                    spreadC = zeros(1,pathSize);
                    spreadN = zeros(1,pathSize);
                    probSpread = zeros(1,pathSize);
                    
                    numOfTimeSteps = numMethodInfo.timeStepsInfo.numOfTimeSteps(i);
                    
                    % if inclusive then we use simpleYN rangeProb
                    %if inclusive then we do not evaluate the last step but
                    % use fixing instead 
                    endTimeStep = 1;
                    if inclusiveYN
%                         simpleYN = true;
                        simpleYN = 1;
                        endTimeStep = 2;
                        if pricer.overHedgeYN == true 
                            simpleYN = 2;
                        end
                    end
                    
                    for j=numOfTimeSteps:-1:endTimeStep
                        % evaluate dealdescription
                        % last step is omitted
                        currentTime = totalTimeSteps(currentTimeIdx);
                        %dcfC = currentTime/365.0;
                        dcfC = currentTime;
                        
                        if j~= numOfTimeSteps
                            dD = (dcfN-dcfC);
                        end

                        modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:numOfFactors*(currentTimeIdx-1)+numOfFactors,:);
                        
                        idxCD = model.Libor_SimpleDate('KRW',valueDate,currentTime,currentTime,numMatTime, tenorCD,modelStateT);
                        idxC = idxCD;
                        
                        if strcmp(pricerOption,'PSA')
                            idxL = model.CMS_PSA_SimpleDate('KRW',valueDate,currentTime,numMatTime, tenorLong,modelStateT);
                            idxS = model.CMS_PSA_SimpleDate('USD',valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                        else
                            idxL = model.Libor_SimpleDate('KRW',valueDate,currentTime,currentTime,numMatTime, tenorLong,modelStateT);
                            idxS = model.Libor_SimpleDate('USD',valueDate,currentTime,currentTime,numMatTime, tenorShort,modelStateT);
                        end
                        spreadC = idxL - idxS;

                        probSpread = model.DRPr(idxC,idxN,ralower,raupper,spreadC,spreadN,ralowerS,raupperS,simpleYN,pathSize)*dD;
                    
%                         probSpread = model.SRPr(spreadC,spreadN,ralower,raupper,simpleYN,pathSize)*dD;
                        cD = cD + dD;
                        cP = cP + probSpread;

                        dcfN = dcfC;
                        
                        idxN = idxC;
                        spreadN = spreadC;

                        %induct backward to the next time step
                        % one time backward induction
                        lastTimeIdx = currentTimeIdx;

                        if startTimeIdx < lastTimeIdx
                            lastTimeIdx = lastTimeIdx -1;
                            currentTimeIdx = lastTimeIdx;
                        end


                    end

                    %% FromToInduction end
        % if the current period is inclusive, then the start date is not evaluated in the above for loop.
		% instead the fixing will be treated separately below.
		% however current_time_idx is updated at the end of the for loop.
		% so we need to update model_states accordingly.
		% if model_states were updated previously then the following additionaly step doesn't affect the result anyway.
                    currentTime = totalTimeSteps(currentTimeIdx);
                    modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:numOfFactors*(currentTimeIdx-1)+numOfFactors,:);
                    
                    %% AtDate evaluation
                    % Adding a new cashflow to the payoff

                    dcf = schedule.dayCountFraction(i);
                    
                    if inclusiveYN
                        % we add valuation date
                        cD = schedule.dayCount(i);
                        rangeDays =  security.rangeDays(i);
                        payoffStateA.cashflow = nominal * coupon * dcf * (rangeDays + cP)/cD;
                    else
                        payoffStateA.cashflow = nominal * coupon * dcf * cP/cD;
                    end
                    % eventDate(i) = startDate(i) therefore there is no induction in modelStates
                    % i.e. currentTimeIdx has not changed
                    % no need to update modelStateT
                    % modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:end,:);
                    % since cashflow's are added at the startDate(i) we need to
                    % multiply cashflow with the df(startDate(i), payDate(i))
                    payTime = DateDiff(schedule.payDates(i),valueDate);
                    
                    %event is at startDate so we need to discount the
                    %cashflow first from paydate to startdate
                    df_ep = model.discountFactorRisky(currentTime, payTime,numMatTime,modelStateT);

                    for k=1:pathSize
                        payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                    end
                    
                    % process payoff
                    
                    currentNodeIdx = i;
                    
                    nonCall.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                
                    callable.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                    
                    %df column
                    payoffStateA.cashflow = ones(1,pathSize);
                    for k=1:pathSize
                        payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                    end
                    
                    dfCol.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                                 
                    %dummy column
                    if strcmp(pricerOption,'PSA')
                        payoffStateA.cashflow = model.CMS_PSA_SimpleDate('KRW',valueDate,currentTime,numMatTime, tenorLong,modelStateT);
                        dummy1.ProcessPayoff(model,currentTime,numeraire, ...
                                         modelStateT,payoffStateA.cashflow,currentNodeIdx);



                        payoffStateA.cashflow = model.CMS_PSA_SimpleDate('USD',valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                        dummy2.ProcessPayoff(model,currentTime,numeraire, ...
                                         modelStateT,payoffStateA.cashflow,currentNodeIdx);
                    
                    else
                        payoffStateA.cashflow = model.Libor_SimpleDate('KRW',valueDate,currentTime,currentTime,numMatTime, tenorLong,modelStateT); 
                        dummy1.ProcessPayoff(model,currentTime,numeraire, ...
                                 modelStateT,payoffStateA.cashflow,currentNodeIdx);
                        
                        payoffStateA.cashflow = model.Libor_SimpleDate('USD',valueDate,currentTime,currentTime,numMatTime, tenorShort,modelStateT); 
                        dummy2.ProcessPayoff(model,currentTime,numeraire, ...
                                 modelStateT,payoffStateA.cashflow,currentNodeIdx);
                        
                    end

                                 
                    
                    
                    
                    % if exercise is possible
                    if i == 1
                        dummaaa = 1;
                    end
                    
                    % call event happens  at the start of the period
                    % at call date, by default, call option is expired
                    
                    if callYN(i) && (DateDiff(valueDate,callSchedule.resetDates(i)) < 0)
                        % 1.undiscount callable.payoff
                        % 2.build the state variables matrix
                        % 3.regress 1's with the given state variables
                        % (model states 1,2)
                        % 4. exercise per each path in
                        % discount the new payoff and assign into callable
                        % column
                        
                        %1 undiscount callable.payoff and get the
                        %contiValue
                       
                        %exercise value
                        exerciseValue = security.nominal*ones(1,pathSize);
                        payTime = DateDiff(callSchedule.payDates(i),valueDate);
                        %event is at startDate so we need to discount the
                        %cashflow first from paydate to startdate
                        df_ep = model.discountFactorRisky(currentTime, payTime,numMatTime,modelStateT);
                        for k=1:pathSize
                            exerciseValue(k) = exerciseValue(k) * df_ep(k);
                        end
                        
                        contiValue = zeros(1,pathSize);
                        onePayoff = ones(1,pathSize);
                        onePayoffD = model.discountPayoffRisky(currentTime,numMatTime ...
                        ,modelStateT,onePayoff);
                        contiValue = callable.payoff./onePayoffD;
                        
                        %Nominal Regression test
                        contiValue = contiValue/10000;
                        
                        % build the state variabls matrix
                        dimensionLS = (numOfFactors + 1)*(numOfFactors + 2)/2;
                        
                        decisionVar = zeros(numOfFactors,size(modelStateT,2));
                        
                        for idxLS =1:numOfFactors
                            decisionVar(idxLS,:) = modelStateT(idxLS,:); 
                        end
%                         decisionVar1 = modelStateT(1,:);
%                         decisionVar2 = modelStateT(2,:);
                        % we use upto 2nd order polynomial as regression
                        % equatioin
                        
                        regMatrix = ones(pathSize,dimensionLS);
                        
                        for idx_r=1:pathSize
                            for idxLS=1:numOfFactors
                                regMatrix(idx_r,idxLS+1) = decisionVar(idxLS,idx_r);
                            end
                            
                            tempIdx = 1;
                            for idxLS1=1:numOfFactors
                                for idxLS2=idxLS1:numOfFactors
                                    regMatrix(idx_r,numOfFactors+1+tempIdx) = decisionVar(idxLS1,idx_r)*decisionVar(idxLS2,idx_r);
                                    tempIdx = tempIdx + 1;
                                end
                            end
                            
                        end
                        
                        % simple regression <- for path dependent option
                        % we should change it
                        % beta  : regression coefficients
                        beta =zeros(dimensionLS,1);
                        beta = regress(contiValue',regMatrix);
                        regContiValue = regMatrix*beta;
                       
                         %Nominal Regression test
                        regContiValue = regContiValue * 10000;
                        
                        exercisedPayoff = regContiValue;
                        % if regContiValue is bigger than exerciseValue
                        % then issue exercise call option
                        for idx_r=1:pathSize
                            if regContiValue(idx_r) >= exerciseValue(idx_r) 
                                exercisedPayoff(idx_r) = exerciseValue(idx_r);
                            end
                        end
                        
                        % we discount the new payoff and 
                        % assign it into the existing callable payoff
                        callable.payoff = model.discountPayoffRisky(currentTime,numMatTime ...
                        ,modelStateT,exercisedPayoff);
                        
                    end
                end

                nonCall.npv = mean(nonCall.payoff);
                out.nonCall = nonCall;
                callable.npv = mean(callable.payoff);
                dfCol.npv = mean(dfCol.payoff);
                dummy1.npv = mean(dummy1.npv);
                dummy2.npv = mean(dummy2.npv);
                out.callable = callable;
                out.dfCol = dfCol;
                out.dummy1 = dummy1;
                out.dummy2 = dummy2;
                out.daycountFraction = schedule.dayCountFraction;
                
        end
        
        function out = computePriceMCCallableGenericReg2(pricer,valueDate,pricerOption)
                security = pricer.m_security;
                numMethod = pricer.m_numMethod;
                model = pricer.m_model;
                schedule = security.couponSchedule;
                callSchedule = security.callSchedule;
                
                % numMethodTimeSteps Generate
                mcOneTimeStep = numMethod.mcOneTimeStep;
                outInfo = numMethod.numMethodTimeStepsGenerateDW(valueDate,schedule,mcOneTimeStep);
            
                numMethodInfo.timeStepsInfo = outInfo;

                % mc random number init
                %pathSize = 2^16;
                pathSize = numMethod.pathSize;

                % generate random numbers and modelstates forward
                rng('default');
                rng(0);

                % LGM1F : one factor
                % LGM2F : two factor
                
                numOfFactors = model.numOfFactor;

                %Z= randn(numOfFactors*outInfo.totalTimeStepsSize,pathSize/2);
                Z= randn(pathSize/2,numOfFactors*outInfo.totalTimeStepsSize);
                Z=[Z;-Z];
                Z=Z';
                
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;

                % we check with the c code, so we use generated random numbers
%                 mc_random = importdata('mc_random.xlsx');
%                 mc_random = importdata('D:\mc_random_number_new_report.txt');
%                 Z= mc_random;
                
                numMethodInfo.pathSize = pathSize;
                numMethodInfo.numOfFactors = numOfFactors;
                numMethodInfo.totalTimeSteps = outInfo.totalTimeSteps;
                numMethodInfo.totalTimeStepsSize = outInfo.totalTimeStepsSize;
                numMethodInfo.Z = Z;


                % numeraire info
                numMatTime = numMethodInfo.totalTimeSteps(end);
                days = DateDiff(schedule.payDates(schedule.numOfPeriod),valueDate);
                numMatTime = max(numMatTime, days);
                dfNumMat = model.DFRisky(numMatTime/365.0);
                
                numeraire.numMatTime = numMatTime;
                numeraire.dfNumMat = dfNumMat;
                numMethodInfo.numMatTime = numMatTime; 
                %numMethod generate model states for the given timeSteps
%                 modelStates = numMethod.generateModelStatesNF(valueDate,schedule,numMethodInfo,model);
                tic
                modelStates = numMethod.generateModelStatesNFHyb(valueDate,schedule,numMethodInfo,model);
                toc
                disp('modelStates generation finished')
                
                
                % induct backward and calculate cashflow
            
                periodCount = schedule.numOfPeriod;
                
                % pricer single info initialize
                
                nonCall = PricerInfo(periodCount +1, pathSize);
                callable = PricerInfo(periodCount +1, pathSize);
                dfCol = PricerInfo(periodCount +1, pathSize);
                
                dummy1 = PricerInfo(periodCount +1, pathSize);
                dummy2 = PricerInfo(periodCount +1, pathSize);
                
                
                % intermediate variables
                payoffStateA.cashflow = zeros(1, pathSize);
                payoffStateA.cashflow_npv = zeros(1, pathSize);
                
                payoffStateB.cashflow = zeros(1, pathSize);
                payoffStateB.cashflow_npv = zeros(1, pathSize);
                
                % we calculate cashflow backward from maturityDate to valueDate
                % rangeAccrual coupons are added at the startDate of the Period
                % At maturity

                hedgeBuffer = security.hedgeBuffer;
                
                nominal = security.nominal;
                coupon = security.coupon;
                callYN = security.callYN;
                
%                 ralower = security.lower;
%                 raupper = security.upper;
                
                ralowerS = security.lowerS;
                raupperS = security.upperS;
                
                ralowerEQ = security.lowerEQ;
                raupperEQ = security.upperEQ;
                
                putAtMatStrike = security.putAtMatStrike;
                
%                 tenorCD = security.tenor;
                
                tenorLong = security.tenorL;
                tenorShort = security.tenorS;
                
                currentTime = numMethodInfo.totalTimeSteps(end);
                totalTimeStepsSize = numMethodInfo.totalTimeStepsSize;
                currentTimeIdx = totalTimeStepsSize;
                
                currentNodeIdx = periodCount + 1;
                
                modelStateT = modelStates((numOfFactors+1)*(currentTimeIdx-1)+1:(numOfFactors+1)*(currentTimeIdx-1)+(numOfFactors+1),:);
                
                idxEQMat = model.EQSpotMC( model.eqModelName,valueDate,currentTime,numMatTime,modelStateT);
                
                for idxHong=1:pathSize
                    if idxEQMat(idxHong) >= putAtMatStrike
                        payoffStateA.cashflow(idxHong) = nominal;
                    else
                        payoffStateA.cashflow(idxHong) = nominal*idxEQMat(idxHong)/100.0;
                    end
                end
                
%                 payoffStateA.cashflow = nominal*ones(1,pathSize);
                
                payTime = DateDiff(schedule.payDates(schedule.numOfPeriod),valueDate);
                df_ep = model.discountFactorRisky(currentTime, payTime,numMatTime,modelStateT);
                for k=1:pathSize
                    payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                end
                
                %process payoff at maturity
                nonCall.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                
                callable.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                
                %df column
                payoffStateA.cashflow = ones(1,pathSize);
                for k=1:pathSize
                    payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                end
                
                dfCol.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                                 
                  

                lastTimeIdx = currentTimeIdx;
                totalTimeSteps = numMethodInfo.timeStepsInfo.totalTimeSteps;
                
                % flag for inclusive schedule
%                 simpleYN = false;
%                 simpleYN = true;
                if pricer.overHedgeYN == true 
                    simpleYN = 2;
                else
                    simpleYN = 1;
                end
                
                for i=periodCount:-1:1
                    % past schedule neglect
                    if DateDiff(valueDate,schedule.payDates(i)) >= 0, continue;end 
                    startTimeIdx = numMethodInfo.timeStepsInfo.startTimeIdx(i);
                    endTimeIdx = numMethodInfo.timeStepsInfo.endTimeIdx(i);

                    inclusiveYN = numMethodInfo.timeStepsInfo.inclusiveYN(i);
                 %% FromToInduction start

                    %modelsStates induct backward(lastTimeIdx to endTimeIdx)
                    while endTimeIdx < lastTimeIdx
                        lastTimeIdx = lastTimeIdx -1;
                        if lastTimeIdx == endTimeIdx break;end
                    end

                    currentTimeIdx = lastTimeIdx;
                    currentTime = totalTimeSteps(currentTimeIdx);
                    % now we are at endTimeIdx
                    % variables for coupon valuation initialized
                    

                    cD = 0; % cumulated day count fraction
                    cP = 0; 
                    dI = 0; % dummy idx for loop
                    dcfC = 0;
                    dcfN = 0;
                    dD = 0;
                    idxCD = zeros(1,pathSize);
                    
                    idxL = zeros(1,pathSize);
                    idxS = zeros(1,pathSize);
                    idxEQ = zeros(1,pathSize);
                    
                    idxEQMat = zeros(1,pathSize);
                    
                    idxC = zeros(1,pathSize);
                    idxN = zeros(1,pathSize);
                    
                    spreadC = zeros(1,pathSize);
                    spreadN = zeros(1,pathSize);
                    probSpread = zeros(1,pathSize);
                    
                    numOfTimeSteps = numMethodInfo.timeStepsInfo.numOfTimeSteps(i);
                    
                    % if inclusive then we use simpleYN rangeProb
                    %if inclusive then we do not evaluate the last step but
                    % use fixing instead 
                    endTimeStep = 1;
                    if inclusiveYN
%                         simpleYN = true;
                        simpleYN = 1;
                        endTimeStep = 2;
                        if pricer.overHedgeYN == true 
                            simpleYN = 2;
                        end
                    end
                    
                    for j=numOfTimeSteps:-1:endTimeStep
                        % evaluate dealdescription
                        % last step is omitted
                        currentTime = totalTimeSteps(currentTimeIdx);
                        %dcfC = currentTime/365.0;
                        dcfC = currentTime;
                        
                        if j~= numOfTimeSteps
                            dD = (dcfN-dcfC);
                        end

%                         modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:numOfFactors*(currentTimeIdx-1)+numOfFactors,:);
                        modelStateT = modelStates((numOfFactors+1)*(currentTimeIdx-1)+1:(numOfFactors+1)*(currentTimeIdx-1)+(numOfFactors+1),:);
                       
                        
                        idxL = model.CMS_PSA_SimpleDate(model.refModelName,valueDate,currentTime,numMatTime, tenorLong,modelStateT);
                        idxS = model.CMS_PSA_SimpleDate(model.refModelName,valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                        idxEQ = model.EQSpotMC(model.eqModelName,valueDate,currentTime,numMatTime,modelStateT);
                        
                        spreadC = idxL - idxS;
                        idxC = idxEQ;
  
                        probSpread = model.DRPr(idxC,idxN,ralowerEQ,raupperEQ,spreadC,spreadN,ralowerS,raupperEQ,simpleYN,hedgeBuffer,pathSize)*dD;
                    
                        cD = cD + dD;
                        cP = cP + probSpread;

                        dcfN = dcfC;
                        
                        idxN = idxC;
                        spreadN = spreadC;

                        %induct backward to the next time step
                        % one time backward induction
                        lastTimeIdx = currentTimeIdx;

                        if startTimeIdx < lastTimeIdx
                            lastTimeIdx = lastTimeIdx -1;
                            currentTimeIdx = lastTimeIdx;
                        end


                    end

                    %% FromToInduction end
        % if the current period is inclusive, then the start date is not evaluated in the above for loop.
		% instead the fixing will be treated separately below.
		% however current_time_idx is updated at the end of the for loop.
		% so we need to update model_states accordingly.
		% if model_states were updated previously then the following additionaly step doesn't affect the result anyway.
                    currentTime = totalTimeSteps(currentTimeIdx);
%                     modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:numOfFactors*(currentTimeIdx-1)+numOfFactors,:);
                    modelStateT = modelStates((numOfFactors+1)*(currentTimeIdx-1)+1:(numOfFactors+1)*(currentTimeIdx-1)+(numOfFactors+1),:);
                    %% AtDate evaluation
                    % Adding a new cashflow to the payoff

                    dcf = schedule.dayCountFraction(i);
                    
                    if inclusiveYN
                        % we add valuation date
                        cD = schedule.dayCount(i);
                        rangeDays =  security.rangeDays(i);
                        payoffStateA.cashflow = nominal * coupon * dcf * (rangeDays + cP)/cD;
                    else
                        payoffStateA.cashflow = nominal * coupon * dcf * cP/cD;
                    end
                    % eventDate(i) = startDate(i) therefore there is no induction in modelStates
                    % i.e. currentTimeIdx has not changed
                    % no need to update modelStateT
                    % modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:end,:);
                    % since cashflow's are added at the startDate(i) we need to
                    % multiply cashflow with the df(startDate(i), payDate(i))
                    payTime = DateDiff(schedule.payDates(i),valueDate);
                    
                    %event is at startDate so we need to discount the
                    %cashflow first from paydate to startdate
                    df_ep = model.discountFactorRisky(currentTime, payTime,numMatTime,modelStateT);

                    for k=1:pathSize
                        payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                    end
                    
                    % process payoff
                    
                    currentNodeIdx = i;
                    
                    nonCall.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                
                    callable.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                    
                    %df column
                    payoffStateA.cashflow = ones(1,pathSize);
                    for k=1:pathSize
                        payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                    end
                    
                    dfCol.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                                 
%                     %dummy column
%                     if strcmp(pricerOption,'PSA')
%                         payoffStateA.cashflow = model.CMS_PSA_SimpleDate('KRW',valueDate,currentTime,numMatTime, tenorLong,modelStateT);
%                         dummy1.ProcessPayoff(model,currentTime,numeraire, ...
%                                          modelStateT,payoffStateA.cashflow,currentNodeIdx);
% 
% 
% 
%                         payoffStateA.cashflow = model.CMS_PSA_SimpleDate('KRW',valueDate,currentTime,numMatTime, tenorShort,modelStateT);
%                         dummy2.ProcessPayoff(model,currentTime,numeraire, ...
%                                          modelStateT,payoffStateA.cashflow,currentNodeIdx);
%                     
%                     else
% %                         payoffStateA.cashflow = model.Libor_SimpleDate('KRW',valueDate,currentTime,currentTime,numMatTime, tenorLong,modelStateT); 
% %                         dummy1.ProcessPayoff(model,currentTime,numeraire, ...
% %                                  modelStateT,payoffStateA.cashflow,currentNodeIdx);
%                         
% %                         payoffStateA.cashflow = model.Libor_SimpleDate('USD',valueDate,currentTime,currentTime,numMatTime, tenorShort,modelStateT); 
% %                         dummy2.ProcessPayoff(model,currentTime,numeraire, ...
% %                                  modelStateT,payoffStateA.cashflow,currentNodeIdx);
%                         
%                     end

                                 
                    
                    
                    
                    % if exercise is possible
                    if i == 1
                        dummaaa = 1;
                    end
                    
                    % call event happens  at the start of the period
                    % at call date, by default, call option is expired
                    
                    if callYN(i) && (DateDiff(valueDate,callSchedule.resetDates(i)) < 0)
                        % 1.undiscount callable.payoff
                        % 2.build the state variables matrix
                        % 3.regress 1's with the given state variables
                        % (model states 1,2)
                        % 4. exercise per each path in
                        % discount the new payoff and assign into callable
                        % column
                        
                        %1 undiscount callable.payoff and get the
                        %contiValue
                       
                        %exercise value
                        exerciseValue = security.nominal*ones(1,pathSize);
                        payTime = DateDiff(callSchedule.payDates(i),valueDate);
                        %event is at startDate so we need to discount the
                        %cashflow first from paydate to startdate
                        df_ep = model.discountFactorRisky(currentTime, payTime,numMatTime,modelStateT);
                        for k=1:pathSize
                            exerciseValue(k) = exerciseValue(k) * df_ep(k);
                        end
                        
                        contiValue = zeros(1,pathSize);
                        onePayoff = ones(1,pathSize);
                        onePayoffD = model.discountPayoffRisky(currentTime,numMatTime ...
                        ,modelStateT,onePayoff);
                        contiValue = callable.payoff./onePayoffD;
                        
                        %Nominal Regression test
                        contiValue = contiValue/10000;
                        
                        % build the state variabls matrix
                        %zeroth order
                        dimensionLS = 1;
                        
                        %1st order
                        for idxLS =1:numOfFactors
                            dimensionLS =  dimensionLS +1; 
                        end
                        
                        %2nd order
                        for idxLS1=1:numOfFactors
                            for idxLS2=idxLS1:numOfFactors
                                dimensionLS = dimensionLS + 1;
                            end
                        end
                        
                        %3rd order
                        for idxLS1=1:numOfFactors
                            for idxLS2=idxLS1:numOfFactors
                                for idxLS3=idxLS2:numOfFactors
                                    dimensionLS = dimensionLS + 1;
                                end
                            end
                        end
                        
%                         dimensionLS = (numOfFactors + 1)*(numOfFactors + 2)/2;
                        
                        decisionVar = zeros(numOfFactors,size(modelStateT,2));
                        
                        for idxLS =1:numOfFactors
                            decisionVar(idxLS,:) = modelStateT(idxLS,:); 
                        end
%                         decisionVar1 = modelStateT(1,:);
%                         decisionVar2 = modelStateT(2,:);
                        % we use upto 2nd order polynomial as regression
                        % equatioin
                        
                        regMatrix = ones(pathSize,dimensionLS);
                        
                        for idx_r=1:pathSize
                            for idxLS=1:numOfFactors
                                regMatrix(idx_r,idxLS+1) = decisionVar(idxLS,idx_r);
                            end
                            
                            tempIdx = 1;
                            for idxLS1=1:numOfFactors
                                for idxLS2=idxLS1:numOfFactors
                                    regMatrix(idx_r,numOfFactors+1+tempIdx) = decisionVar(idxLS1,idx_r)*decisionVar(idxLS2,idx_r);
                                    tempIdx = tempIdx + 1;
                                end
                            end
                            
                            tempIdx2 = 1;
                            for idxLS1=1:numOfFactors
                                for idxLS2=idxLS1:numOfFactors
                                    for idxLS3=idxLS2:numOfFactors
                                        regMatrix(idx_r,numOfFactors+1+tempIdx-1 + tempIdx2) = decisionVar(idxLS1,idx_r)*decisionVar(idxLS2,idx_r)*decisionVar(idxLS3,idx_r);
                                        tempIdx = tempIdx + 1;
                                    end
                                end
                            end
                            
                        end
                        
                        % simple regression <- for path dependent option
                        % we should change it
                        % beta  : regression coefficients
                        beta =zeros(dimensionLS,1);
                        beta = regress(contiValue',regMatrix);
                        regContiValue = regMatrix*beta;
                       
                         %Nominal Regression test
                        regContiValue = regContiValue * 10000;
                                                
%                         exercisedPayoff = regContiValue;
                        exercisedPayoff = contiValue * 10000;
                        % if regContiValue is bigger than exerciseValue
                        % then issue exercise call option
                        for idx_r=1:pathSize
                            if regContiValue(idx_r) >= exerciseValue(idx_r) 
                                exercisedPayoff(idx_r) = exerciseValue(idx_r);
                            end
                        end
                        
                        % we discount the new payoff and 
                        % assign it into the existing callable payoff
                        callable.payoff = model.discountPayoffRisky(currentTime,numMatTime ...
                        ,modelStateT,exercisedPayoff);
                        
                    end
                end

                nonCall.npv = mean(nonCall.payoff);
                out.nonCall = nonCall;
                callable.npv = mean(callable.payoff);
                dfCol.npv = mean(dfCol.payoff);
%                 dummy1.npv = mean(dummy1.npv);
%                 dummy2.npv = mean(dummy2.npv);
                out.callable = callable;
                out.dfCol = dfCol;
%                 out.dummy1 = dummy1;
%                 out.dummy2 = dummy2;
                out.daycountFraction = schedule.dayCountFraction;
                
        end
        
        
        function out = computePriceClosedSimple(pricer,valueDate)
                security = pricer.m_security;
                numMethod = pricer.m_numMethod;
                model = pricer.m_model;
                schedule = security.couponSchedule;
                
                % numMethodTimeSteps Generate
                mcOneTimeStep = numMethod.mcOneTimeStep;
                outInfo = numMethod.numMethodTimeStepsGenerateDW(valueDate,schedule,mcOneTimeStep);
            
                numMethodInfo.timeStepsInfo = outInfo;

                % mc random number init
                %pathSize = 2^16;
                pathSize = numMethod.pathSize;
                
                pathSize = 1;
                
%                 % generate random numbers and modelstates forward
%                 rng('default');
%                 rng(0);
% 
                numOfFactors = model.numOfFactor;
% 
%                 %Z= randn(numOfFactors*outInfo.totalTimeStepsSize,pathSize/2);
%                 Z= randn(pathSize/2,numOfFactors*outInfo.totalTimeStepsSize);
%                 Z=[Z;-Z];
%                 Z=Z';
%                 
%                 
%                 %TimeInversion
%                 sizeZ = size(Z,1);
%                 A = zeros(sizeZ,sizeZ);
%                 for i=1:sizeZ
%                     A(i,sizeZ + 1 -i) = 1;
%                 end
%                 Z = A*Z;
% 
%                 % we check with the c, so we use generated random numbers
% %                 mc_random = importdata('mc_random.xlsx');
% %                 mc_random = importdata('D:\mc_random_number_new_report.txt');
% %                 Z= mc_random;
%                 
                numMethodInfo.pathSize = pathSize;
                numMethodInfo.numOfFactors = numOfFactors;
                numMethodInfo.totalTimeSteps = outInfo.totalTimeSteps;
                numMethodInfo.totalTimeStepsSize = outInfo.totalTimeStepsSize;
%                 numMethodInfo.Z = Z;
% 
% 
% 
%                 %generate model states for the given timeSteps
%                 modelStates = numMethod.generateModelStates(valueDate,schedule,numMethodInfo,model);

                % numeraire info
                numMatTime = numMethodInfo.totalTimeSteps(end);
                dfNumMat = model.DFRisky(numMatTime/365.0);
                
                % induct backward and calculate cashflow
            
                periodCount = schedule.numOfPeriod;
                
                % cashflow columns initialize
                % 1st nonCall
                nonCall.npv = 0;
                nonCall.cashflow_e = zeros(periodCount + 1,1);
                nonCall.cashflow_npv_e = zeros(periodCount + 1,1);
                nonCall.rangeProb_e = zeros(periodCount + 1,1);
                nonCall.df_e = zeros(periodCount + 1,1);
                nonCall.df_c = zeros(periodCount + 1,1);
                
                nonCall.fwdCD = zeros(periodCount + 1,1);
                nonCall.fwdKTB = zeros(periodCount + 1,1);
                nonCall.fwdSpread = zeros(periodCount + 1,1);
                nonCall.fwdMu = zeros(periodCount + 1,1);
                nonCall.fwdSigma = zeros(periodCount + 1,1);

                % debug information, cashflow & npv per path
                nonCall.payoff = zeros(1,pathSize);
                nonCall.payoffStates.cashflow = zeros(periodCount+1,pathSize);
                nonCall.payoffStates.cashflow_npv = zeros(periodCount+1,pathSize);
                nonCall.payoffStates.df = zeros(periodCount + 1,pathSize);
                
                % intermediate variables
                payoffStateA.cashflow = zeros(1, pathSize);
                payoffStateA.cashflow_npv = zeros(1, pathSize);
                payoffStateA.df = zeros(1,pathSize);
                
                % we calculate cashflow backward from maturityDate to valueDate
                % rangeAccrual coupons are added at the startDate of the Period
                % At maturity

                nominal = security.nominal;
                coupon = security.coupon;
                callYN = security.callYN;
                
                currentTime = numMethodInfo.totalTimeSteps(end);
                totalTimeSteps = numMethodInfo.timeStepsInfo.totalTimeSteps;
                totalTimeStepsSize = numMethodInfo.timeStepsInfo.totalTimeStepsSize;
                if totalTimeStepsSize == 0 || (totalTimeStepsSize == 1 && totalTimeSteps(1) == 0) 
                    out.nonCall = nonCall;
                    return;
                end
                
                currentTimeIdx = totalTimeStepsSize;
                
                currentNodeIdx = periodCount + 1;
                
%                 modelStateT = modelStates(numOfFactors*(totalTimeStepsSize-1)+1:numOfFactors*(totalTimeStepsSize-1)+2,:);
                payoffStateA.cashflow = nominal*ones(1,pathSize);
                %process payoff at maturity
                nonCall.payoffStates.cashflow(currentNodeIdx,:) = payoffStateA.cashflow;
                
%                 payoffStateA.cashflow_npv = model.discountPayoff(currentTime,numMatTime ...
%                     ,modelStateT,payoffStateA.cashflow);
%                 
                payoffStateA.cashflow_npv = model.DFRisky(currentTime/365.0)*payoffStateA.cashflow;
                
                nonCall.payoffStates.cashflow_npv(currentNodeIdx,:) = payoffStateA.cashflow_npv ;
                
%                 payoffStateA.df = model.discountPayoff(currentTime,numMatTime ...
%                     ,modelStateT,1.0*ones(1,pathSize));
                payoffStateA.df = model.DFRisky(currentTime/365.0) *ones(1,pathSize);
                    

                nonCall.payoffStates.df(currentNodeIdx,:) = payoffStateA.df;
                nonCall.payoff = nonCall.payoff + payoffStateA.cashflow_npv;
                
                nonCall.cashflow_e(currentNodeIdx) = mean(payoffStateA.cashflow);
                nonCall.cashflow_npv_e(currentNodeIdx) = mean(payoffStateA.cashflow_npv);
                nonCall.df_e(currentNodeIdx) = mean(payoffStateA.df);
                nonCall.df_c(currentNodeIdx) = model.DFRisky(currentTime/365.0);
                
%                 % callable at maturity is the same as noncallable
%                 callable.payoffStates.cashflow(currentNodeIdx,:) = nonCall.payoffStates.cashflow(currentNodeIdx,:);
%                 callable.payoffStates.cashflow_npv(currentNodeIdx,:) = nonCall.payoffStates.cashflow_npv(currentNodeIdx,:);
%                 callable.payoff = callable.payoff + callable.payoffStates.cashflow_npv(currentNodeIdx,:);
%                 
%                 callable.cashflow_e(currentNodeIdx) = nonCall.cashflow_e(currentNodeIdx);
%                 callable.cashflow_npv_e(currentNodeIdx) = nonCall.cashflow_npv_e(currentNodeIdx);
                  
                lastTimeIdx = currentTimeIdx;
                
                
                % flag for inclusive schedule
%                 simpleYN = false;
                simpleYN = 1;
                
                for i=periodCount:-1:1
                    % past schedule neglect
                    if DateDiff(valueDate,schedule.payDates(i)) >= 0, continue;end 
                    startTimeIdx = numMethodInfo.timeStepsInfo.startTimeIdx(i);
                    endTimeIdx = numMethodInfo.timeStepsInfo.endTimeIdx(i);

                    inclusiveYN = numMethodInfo.timeStepsInfo.inclusiveYN(i);
                     %% FromToInduction start

                    %modelsStates induct backward(lastTimeIdx to endTimeIdx)
                    while endTimeIdx < lastTimeIdx
                        lastTimeIdx = lastTimeIdx -1;
                        if lastTimeIdx == endTimeIdx break;end
                    end

                    currentTimeIdx = lastTimeIdx;
                    currentTime = totalTimeSteps(currentTimeIdx);
                    % now we are at endTimeIdx
                    % variables for coupon valuation initialized
                    ralower = security.lowerS;
                    raupper = security.upperS;
                    tenorLong = security.tenorL;
                    tenorShort = security.tenorS;

                    cD = 0; % cumulated day count fraction
                    cP = 0; 
                    dI = 0; % dummy idx for loop
                    dcfC = 0;
                    dcfN = 0;
                    dD = 0;
                    idxL = zeros(1,pathSize);
                    idxS = zeros(1,pathSize);
                    spreadC = zeros(1,pathSize);
                    spreadN = zeros(1,pathSize);
                    probSpread = zeros(1,pathSize);
                    
                    numOfTimeSteps = numMethodInfo.timeStepsInfo.numOfTimeSteps(i);
                    
                    % if inclusive then we use simpleYN rangeProb
                    %if inclusive then we do not evaluate the last step but
                    % use fixing instead 
                    endTimeStep = 1;
                    if inclusiveYN
%                         simpleYN = true;
                        simpleYN = 1;
                        endTimeStep = 2;
                    end
                    
                    for j=numOfTimeSteps:-1:endTimeStep
                        % evaluate dealdescription
                        % last step is omitted
                        currentTime = totalTimeSteps(currentTimeIdx);
                        %dcfC = currentTime/365.0;
                        dcfC = currentTime;
                        
                        if j~= numOfTimeSteps
                            dD = (dcfN-dcfC);
                        end

                        toDate = AddDate(valueDate,currentTime,'day');
                        tpDate = schedule.payDates(i);
                        spreadCOut = model.CMSCMTSpreadDigitalRangeDate(valueDate,toDate,tpDate,tenorLong,tenorShort,ralower,raupper);
                        spreadC = spreadCOut.value;
                        if j== endTimeStep
                            fwdCD = spreadCOut.cms_fwd;
                            fwdKTB = spreadCOut.cmt_fwd;
                            fwdSpread = spreadCOut.fwdSpread;
                            fwdMu        = spreadCOut.mu;
                            fwdSigma     = spreadCOut.sigma;
                        end
                        %modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:numOfFactors*(currentTimeIdx-1)+2,:);
%                         idxL = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorLong,modelStateT);
%                         idxS = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorShort,modelStateT);
%                         spreadC = idxL - idxS;
%                         probSpread = model.SRPr(spreadC,spreadN,ralower,raupper,simpleYN,pathSize)*dD; 

                        if simpleYN
                            probSpread = spreadC*dD;
                        else
                            probSpread = spreadC*dD;
%                             probSpread = 0.5*(spreadC + spreadN)*dD;
                            
                        end

                        
                        cD = cD + dD;
                        cP = cP + probSpread;

                        dcfN = dcfC;
                        spreadN = spreadC;

                        %induct backward to the next time step
                        % one time backward induction
                        lastTimeIdx = currentTimeIdx;

                        if startTimeIdx < lastTimeIdx
                            lastTimeIdx = lastTimeIdx -1;
                            currentTimeIdx = lastTimeIdx;
                        end


                    end

                    %% FromToInduction end
                    %% AtDate evaluation
                    % Adding a new cashflow to the payoff

                    dcf = schedule.dayCountFraction(i);
                    
                    if inclusiveYN
                        % we add valuation date
                        cD = schedule.dayCount(i);
                        rangeDays =  security.rangeDays(i);
                        payoffStateA.cashflow = nominal * coupon * dcf * (rangeDays + cP)/cD;
                    else
                        payoffStateA.cashflow = nominal * coupon * dcf * cP/cD;
                    end
                    % eventDate(i) = startDate(i) therefore there is no induction in modelStates
                    % i.e. currentTimeIdx has not changed
                    % no need to update modelStateT
                    % modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:end,:);
                    % since cashflow's are added at the startDate(i) we need to
                    % multiply cashflow with the df(startDate(i), payDate(i))
                    payTime = DateDiff(schedule.payDates(i),valueDate);
                    
                    %event is at startDate so we need to discount the
                    %cashflow first from paydate to startdate
%                     df_sp = model.discountFactor(currentTime, payTime,numMatTime,modelStateT);

%                     for k=1:pathSize
%                         payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_sp(k);
%                     end
                    
                    % process payoff
                    
                    currentNodeIdx = i;
                    
                    % 1st nonCall column
                    nonCall.payoffStates.cashflow(currentNodeIdx,:) = payoffStateA.cashflow;
                    payoffStateA.cashflow_npv = payoffStateA.cashflow * model.DFRisky(payTime/365.0);
                    
%                     payoffStateA.cashflow_npv = model.discountPayoff(currentTime,numMatTime ...
%                     ,modelStateT,payoffStateA.cashflow);
                    nonCall.payoffStates.cashflow_npv(currentNodeIdx,:) = payoffStateA.cashflow_npv;
                    payoffStateA.df = model.DFRisky(payTime/365.0);
%                     payoffStateA.df = model.discountPayoff(currentTime,numMatTime ...
%                     ,modelStateT,1.0*ones(1,pathSize));
                    nonCall.payoffStates.df(currentNodeIdx,:) = payoffStateA.df;
                    
                    nonCall.payoff = nonCall.payoff + payoffStateA.cashflow_npv;
                    
                    nonCall.cashflow_e(currentNodeIdx) = mean(payoffStateA.cashflow);
                    nonCall.rangeProb_e(currentNodeIdx) = mean(nonCall.cashflow_e(currentNodeIdx)/(nominal * coupon * dcf));
                    nonCall.cashflow_npv_e(currentNodeIdx) = mean(payoffStateA.cashflow_npv);

                    nonCall.df_e(currentNodeIdx) = mean(payoffStateA.df);
                    nonCall.df_c(currentNodeIdx) = model.DFRisky(currentTime/365.0);
                    
                    nonCall.fwdCD(currentNodeIdx) = fwdCD;
                    nonCall.fwdKTB(currentNodeIdx) = fwdKTB;
                    
                    nonCall.fwdSpread(currentNodeIdx) = fwdSpread;
                    nonCall.fwdMu(currentNodeIdx) = fwdMu;
                    nonCall.fwdSigma(currentNodeIdx) = fwdSigma;

                end

                nonCall.npv = mean(nonCall.payoff);
                out.pricerInfo.nonCall = nonCall;
                
%                 callable.npv = mean(callable.payoff);
%                 out.pricerInfo.callable = callable;
                out.daycountFraction = schedule.dayCountFraction;
                
        end
        

        
        
        
        
    end
    
end



In [ ]:
classdef DailyWeeklyRiskyPricer_CMSCMSSpreadDualRA < H_RootObject
    %Pricer(Security,Model,NumMethod)
    %Prcer is constructed from Security, Model, NumMethod 
    
    properties
        m_security
        m_model
        m_numMethod
        overHedgeYN
    end
    
    methods
        function pricer = DailyWeeklyRiskyPricer_CMSCMSSpreadDualRA(security,model,numMethod,overHedgeYN)
                pricer.m_security = security;
                pricer.m_model = model;
                pricer.m_numMethod = numMethod;
                pricer.overHedgeYN = overHedgeYN;
        end
        
        function out = computePriceMCCallable(pricer,valueDate)
                security = pricer.m_security;
                numMethod = pricer.m_numMethod;
                model = pricer.m_model;
                schedule = security.couponSchedule;
                callSchedule = security.callSchedule;
                
                % numMethodTimeSteps Generate
                mcOneTimeStep = numMethod.mcOneTimeStep;
                outInfo = numMethod.numMethodTimeStepsGenerateDW(valueDate,schedule,mcOneTimeStep);
            
                numMethodInfo.timeStepsInfo = outInfo;

                % mc random number init
                %pathSize = 2^16;
                pathSize = numMethod.pathSize;

                % generate random numbers and modelstates forward
                rng('default');
                rng(0);

                numOfFactors = 2;

                %Z= randn(numOfFactors*outInfo.totalTimeStepsSize,pathSize/2);
                Z= randn(pathSize/2,numOfFactors*outInfo.totalTimeStepsSize);
                Z=[Z;-Z];
                Z=Z';
                
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;

                % we check with the c code, so we use generated random numbers
%                 mc_random = importdata('mc_random.xlsx');
%                 mc_random = importdata('D:\mc_random_number_new_report.txt');
%                 Z= mc_random;
                
                numMethodInfo.pathSize = pathSize;
                numMethodInfo.numOfFactors = numOfFactors;
                numMethodInfo.totalTimeSteps = outInfo.totalTimeSteps;
                numMethodInfo.totalTimeStepsSize = outInfo.totalTimeStepsSize;
                numMethodInfo.Z = Z;



                %generate model states for the given timeSteps
                modelStates = numMethod.generateModelStates(valueDate,schedule,numMethodInfo,model);

                % numeraire info
                numMatTime = numMethodInfo.totalTimeSteps(end);
                days = DateDiff(schedule.payDates(schedule.numOfPeriod),valueDate);
                numMatTime = max(numMatTime, days);
                dfNumMat = model.DFRisky(numMatTime/365.0);
                
                numeraire.numMatTime = numMatTime;
                numeraire.dfNumMat = dfNumMat;
                
                % induct backward and calculate cashflow
            
                periodCount = schedule.numOfPeriod;
                
                % pricer single info initialize
                
                nonCall = PricerInfo(periodCount +1, pathSize);
                callable = PricerInfo(periodCount +1, pathSize);
                dfCol = PricerInfo(periodCount +1, pathSize);
                
                dummy1 = PricerInfo(periodCount +1, pathSize);
                dummy2 = PricerInfo(periodCount +1, pathSize);
                
                
                % intermediate variables
                payoffStateA.cashflow = zeros(1, pathSize);
                payoffStateA.cashflow_npv = zeros(1, pathSize);
                
            
                % we calculate cashflow backward from maturityDate to valueDate
                % rangeAccrual coupons are added at the startDate of the Period
                % At maturity

                nominal = security.nominal;
                coupon = security.coupon;
                callYN = security.callYN;
                
                ralower = security.lower;
                raupper = security.upper;
                
                ralowerS = security.lowerS;
                raupperS = security.upperS;
                
                tenorCMS = security.tenor;
                
                tenorLong = security.tenorL;
                tenorShort = security.tenorS;
                
                currentTime = numMethodInfo.totalTimeSteps(end);
                totalTimeStepsSize = numMethodInfo.totalTimeStepsSize;
                currentTimeIdx = totalTimeStepsSize;
                
                currentNodeIdx = periodCount + 1;
                
                modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:numOfFactors*(currentTimeIdx-1)+2,:);
                payoffStateA.cashflow = nominal*ones(1,pathSize);
                
                payTime = DateDiff(schedule.payDates(schedule.numOfPeriod),valueDate);
                df_ep = model.discountFactorRisky(currentTime, payTime,numMatTime,modelStateT);
                for k=1:pathSize
                    payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                end
                
                %process payoff at maturity
                nonCall.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                
                callable.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                
                %df column
                payoffStateA.cashflow = ones(1,pathSize);
                for k=1:pathSize
                    payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                end
                
                dfCol.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                                 
                %dummy column
                payoffStateA.cashflow = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorLong,modelStateT);
                dummy1.ProcessPayoff(model,currentTime,numeraire, ...
                                 modelStateT,payoffStateA.cashflow,currentNodeIdx);

                payoffStateA.cashflow = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                dummy2.ProcessPayoff(model,currentTime,numeraire, ...
                                 modelStateT,payoffStateA.cashflow,currentNodeIdx);
                                 
                  

                lastTimeIdx = currentTimeIdx;
                totalTimeSteps = numMethodInfo.timeStepsInfo.totalTimeSteps;
                
                % flag for inclusive schedule
%                 simpleYN = false;
%                 simpleYN = true;
                if pricer.overHedgeYN == true 
                    simpleYN = 2;
                else
                    simpleYN = 1;
                end
                
                for i=periodCount:-1:1
                    % past schedule neglect
                    if DateDiff(valueDate,schedule.payDates(i)) >= 0, continue;end 
                    startTimeIdx = numMethodInfo.timeStepsInfo.startTimeIdx(i);
                    endTimeIdx = numMethodInfo.timeStepsInfo.endTimeIdx(i);

                    inclusiveYN = numMethodInfo.timeStepsInfo.inclusiveYN(i);
                 %% FromToInduction start

                    %modelsStates induct backward(lastTimeIdx to endTimeIdx)
                    while endTimeIdx < lastTimeIdx
                        lastTimeIdx = lastTimeIdx -1;
                        if lastTimeIdx == endTimeIdx break;end
                    end

                    currentTimeIdx = lastTimeIdx;
                    currentTime = totalTimeSteps(currentTimeIdx);
                    % now we are at endTimeIdx
                    % variables for coupon valuation initialized
                    

                    cD = 0; % cumulated day count fraction
                    cP = 0; 
                    dI = 0; % dummy idx for loop
                    dcfC = 0;
                    dcfN = 0;
                    dD = 0;
                    idxCMS = zeros(1,pathSize);
                    
                    idxL = zeros(1,pathSize);
                    idxS = zeros(1,pathSize);
                    
                    idxC = zeros(1,pathSize);
                    idxN = zeros(1,pathSize);
                
                    spreadC = zeros(1,pathSize);
                    spreadN = zeros(1,pathSize);
                    probSpread = zeros(1,pathSize);
                    
                    numOfTimeSteps = numMethodInfo.timeStepsInfo.numOfTimeSteps(i);
                    
                    % if inclusive then we use simpleYN rangeProb
                    %if inclusive then we do not evaluate the last step but
                    % use fixing instead 
                    endTimeStep = 1;
                    if inclusiveYN
%                         simpleYN = true;
                        simpleYN = 1;
                        endTimeStep = 2;
                        if pricer.overHedgeYN == true 
                            simpleYN = 2;
                        end
                    end
                    
                    for j=numOfTimeSteps:-1:endTimeStep
                        % evaluate dealdescription
                        % last step is omitted
                        currentTime = totalTimeSteps(currentTimeIdx);
                        %dcfC = currentTime/365.0;
                        dcfC = currentTime;
                        
                        if j~= numOfTimeSteps
                            dD = (dcfN-dcfC);
                        end

                        modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:numOfFactors*(currentTimeIdx-1)+2,:);
                        
                        idxCMS = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorCMS,modelStateT);
                        idxC = idxCMS;
                    
                        idxL = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorLong,modelStateT);
                        idxS = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                        spreadC = idxL - idxS;

%                         probSpread = model.SRPr(spreadC,spreadN,ralower,raupper,simpleYN,pathSize)*dD;
                        
                        probSpread = model.DRPr(idxC,idxN,ralower,raupper,spreadC,spreadN,ralowerS,raupperS,simpleYN,pathSize)*dD;
                    
                        cD = cD + dD;
                        cP = cP + probSpread;

                        dcfN = dcfC;
                        
                        idxN = idxC;
                        spreadN = spreadC;

                        %induct backward to the next time step
                        % one time backward induction
                        lastTimeIdx = currentTimeIdx;

                        if startTimeIdx < lastTimeIdx
                            lastTimeIdx = lastTimeIdx -1;
                            currentTimeIdx = lastTimeIdx;
                        end


                    end

                    %% FromToInduction end
        % if the current period is inclusive, then the start date is not evaluated in the above for loop.
		% instead the fixing will be treated separately below.
		% however current_time_idx is updated at the end of the for loop.
		% so we need to update model_states accordingly.
		% if model_states were updated previously then the following additionaly step doesn't affect the result anyway.
                    currentTime = totalTimeSteps(currentTimeIdx);
                    modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:numOfFactors*(currentTimeIdx-1)+2,:);
                    
                    %% AtDate evaluation
                    % Adding a new cashflow to the payoff

                    dcf = schedule.dayCountFraction(i);
                    
                    if inclusiveYN
                        % we add valuation date
                        cD = schedule.dayCount(i);
                        rangeDays =  security.rangeDays(i);
                        payoffStateA.cashflow = nominal * coupon * dcf * (rangeDays(i) + cP)/cD;
                    else
                        payoffStateA.cashflow = nominal * coupon * dcf * cP/cD;
                    end
                    % eventDate(i) = startDate(i) therefore there is no induction in modelStates
                    % i.e. currentTimeIdx has not changed
                    % no need to update modelStateT
                    % modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:end,:);
                    % since cashflow's are added at the startDate(i) we need to
                    % multiply cashflow with the df(startDate(i), payDate(i))
                    payTime = DateDiff(schedule.payDates(i),valueDate);
                    
                    %event is at startDate so we need to discount the
                    %cashflow first from paydate to startdate
                    df_ep = model.discountFactorRisky(currentTime, payTime,numMatTime,modelStateT);

                    for k=1:pathSize
                        payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                    end
                    
                    % process payoff
                    
                    currentNodeIdx = i;
                    
                    nonCall.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                
                    callable.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                    
                    %df column
                    payoffStateA.cashflow = ones(1,pathSize);
                    for k=1:pathSize
                        payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                    end
                    
                    dfCol.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                                 
                    %dummy column
                    payoffStateA.cashflow = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorLong,modelStateT);
                    dummy1.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                    
                    payoffStateA.cashflow = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                    dummy2.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                    
                    
                    % if exercise is possible
                    if i == 1
                        dummaaa = 1;
                    end
                    
                    % call event happens  at the start of the period
                    % at call date, by default, call option is expired
                    
                    if callYN(i) && (DateDiff(valueDate,callSchedule.resetDates(i)) < 0)
                        % 1.undiscount callable.payoff
                        % 2.build the state variables matrix
                        % 3.regress 1's with the given state variables
                        % (model states 1,2)
                        % 4. exercise per each path in
                        % discount the new payoff and assign into callable
                        % column
                        
                        %1 undiscount callable.payoff and get the
                        %contiValue
                       
                        %exercise value
                        exerciseValue = security.nominal*ones(1,pathSize);
                        payTime = DateDiff(callSchedule.payDates(i),valueDate);
                        %event is at startDate so we need to discount the
                        %cashflow first from paydate to startdate
                        df_ep = model.discountFactorRisky(currentTime, payTime,numMatTime,modelStateT);
                        for k=1:pathSize
                            exerciseValue(k) = exerciseValue(k) * df_ep(k);
                        end
                        
                        contiValue = zeros(1,pathSize);
                        onePayoff = ones(1,pathSize);
                        onePayoffD = model.discountPayoffRisky(currentTime,numMatTime ...
                        ,modelStateT,onePayoff);
                        contiValue = callable.payoff./onePayoffD;
                        
                        %Nominal Regression test
                        contiValue = contiValue/10000;
                        
                        % build the state variabls matrix
                        decisionVar1 = modelStateT(1,:);
                        decisionVar2 = modelStateT(2,:);
                        % we use upto 2nd order polynomial as regression
                        % equatioin
                        dimensionLS = 6;
                        regMatrix = ones(pathSize,dimensionLS);
                        
                        for idx_r=1:pathSize
                            regMatrix(idx_r,2) = decisionVar1(idx_r);
                            regMatrix(idx_r,3) = decisionVar2(idx_r);
                            regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r);
                            regMatrix(idx_r,5) = decisionVar1(idx_r)*decisionVar2(idx_r);
                            regMatrix(idx_r,6) = decisionVar2(idx_r)*decisionVar2(idx_r);
                        end
                        
                        % simple regression <- for path dependent option
                        % we should change it
                        % beta  : regression coefficients
                        beta =zeros(dimensionLS,1);
                        beta = regress(contiValue',regMatrix);
                        regContiValue = regMatrix*beta;
                       
                         %Nominal Regression test
                        regContiValue = regContiValue * 10000;
                        
                        exercisedPayoff = regContiValue;
                        % if regContiValue is bigger than exerciseValue
                        % then issue exercise call option
                        for idx_r=1:pathSize
                            if regContiValue(idx_r) >= exerciseValue(idx_r) 
                                exercisedPayoff(idx_r) = exerciseValue(idx_r);
                            end
                        end
                        
                        % we discount the new payoff and 
                        % assign it into the existing callable payoff
                        callable.payoff = model.discountPayoffRisky(currentTime,numMatTime ...
                        ,modelStateT,exercisedPayoff);
                        
                    end
                end

                nonCall.npv = mean(nonCall.payoff);
                out.nonCall = nonCall;
                callable.npv = mean(callable.payoff);
                out.callable = callable;
                out.dfCol = dfCol;
                out.dummy1 = dummy1;
                out.dummy2 = dummy2;
                out.daycountFraction = schedule.dayCountFraction;
                
        end
        
        function out = computePriceClosedSimple(pricer,valueDate)
                security = pricer.m_security;
                numMethod = pricer.m_numMethod;
                model = pricer.m_model;
                schedule = security.couponSchedule;
                
                % numMethodTimeSteps Generate
                mcOneTimeStep = numMethod.mcOneTimeStep;
                outInfo = numMethod.numMethodTimeStepsGenerateDW(valueDate,schedule,mcOneTimeStep);
            
                numMethodInfo.timeStepsInfo = outInfo;

                % mc random number init
                %pathSize = 2^16;
                pathSize = numMethod.pathSize;
                
                pathSize = 1;
                
%                 % generate random numbers and modelstates forward
%                 rng('default');
%                 rng(0);
% 
                numOfFactors = 2;
% 
%                 %Z= randn(numOfFactors*outInfo.totalTimeStepsSize,pathSize/2);
%                 Z= randn(pathSize/2,numOfFactors*outInfo.totalTimeStepsSize);
%                 Z=[Z;-Z];
%                 Z=Z';
%                 
%                 
%                 %TimeInversion
%                 sizeZ = size(Z,1);
%                 A = zeros(sizeZ,sizeZ);
%                 for i=1:sizeZ
%                     A(i,sizeZ + 1 -i) = 1;
%                 end
%                 Z = A*Z;
% 
%                 % we check with the c, so we use generated random numbers
% %                 mc_random = importdata('mc_random.xlsx');
% %                 mc_random = importdata('D:\mc_random_number_new_report.txt');
% %                 Z= mc_random;
%                 
                numMethodInfo.pathSize = pathSize;
                numMethodInfo.numOfFactors = numOfFactors;
                numMethodInfo.totalTimeSteps = outInfo.totalTimeSteps;
                numMethodInfo.totalTimeStepsSize = outInfo.totalTimeStepsSize;
%                 numMethodInfo.Z = Z;
% 
% 
% 
%                 %generate model states for the given timeSteps
%                 modelStates = numMethod.generateModelStates(valueDate,schedule,numMethodInfo,model);

                % numeraire info
                numMatTime = numMethodInfo.totalTimeSteps(end);
                dfNumMat = model.DF(numMatTime/365.0);
                
                % induct backward and calculate cashflow
            
                periodCount = schedule.numOfPeriod;
                
                % cashflow columns initialize
                % 1st nonCall
                nonCall.npv = 0;
                nonCall.cashflow_e = zeros(periodCount + 1,1);
                nonCall.cashflow_npv_e = zeros(periodCount + 1,1);
                nonCall.rangeProb_e = zeros(periodCount + 1,1);
                nonCall.df_e = zeros(periodCount + 1,1);
                nonCall.df_c = zeros(periodCount + 1,1);

                % debug information, cashflow & npv per path
                nonCall.payoff = zeros(1,pathSize);
                nonCall.payoffStates.cashflow = zeros(periodCount+1,pathSize);
                nonCall.payoffStates.cashflow_npv = zeros(periodCount+1,pathSize);
                nonCall.payoffStates.df = zeros(periodCount + 1,pathSize);
                
                % intermediate variables
                payoffStateA.cashflow = zeros(1, pathSize);
                payoffStateA.cashflow_npv = zeros(1, pathSize);
                payoffStateA.df = zeros(1,pathSize);
                
                % we calculate cashflow backward from maturityDate to valueDate
                % rangeAccrual coupons are added at the startDate of the Period
                % At maturity

                nominal = security.nominal;
                coupon = security.coupon;
                callYN = security.callYN;
                
                ralower = security.lower;
                raupper = security.upper;
                
                ralowerS = security.lowerS;
                raupperS = security.upperS;
                
                tenorCMS = security.tenor;
                
                tenorLong = security.tenorL;
                tenorShort = security.tenorS;
                
                currentTime = numMethodInfo.totalTimeSteps(end);
                totalTimeSteps = numMethodInfo.timeStepsInfo.totalTimeSteps;
                totalTimeStepsSize = numMethodInfo.timeStepsInfo.totalTimeStepsSize;
                if totalTimeStepsSize == 0 || (totalTimeStepsSize == 1 && totalTimeSteps(1) == 0) 
                    out.nonCall = nonCall;
                    return;
                end
                
                currentTimeIdx = totalTimeStepsSize;
                
                currentNodeIdx = periodCount + 1;
                
%                 modelStateT = modelStates(numOfFactors*(totalTimeStepsSize-1)+1:numOfFactors*(totalTimeStepsSize-1)+2,:);
                payoffStateA.cashflow = nominal*ones(1,pathSize);
                %process payoff at maturity
                nonCall.payoffStates.cashflow(currentNodeIdx,:) = payoffStateA.cashflow;
                
%                 payoffStateA.cashflow_npv = model.discountPayoff(currentTime,numMatTime ...
%                     ,modelStateT,payoffStateA.cashflow);
%                 
                payoffStateA.cashflow_npv = model.DFRisky(currentTime/365.0)*payoffStateA.cashflow;
                
                nonCall.payoffStates.cashflow_npv(currentNodeIdx,:) = payoffStateA.cashflow_npv ;
                
%                 payoffStateA.df = model.discountPayoff(currentTime,numMatTime ...
%                     ,modelStateT,1.0*ones(1,pathSize));
                payoffStateA.df = model.DFRisky(currentTime/365.0) *ones(1,pathSize);
                    

                nonCall.payoffStates.df(currentNodeIdx,:) = payoffStateA.df;
                nonCall.payoff = nonCall.payoff + payoffStateA.cashflow_npv;
                
                nonCall.cashflow_e(currentNodeIdx) = mean(payoffStateA.cashflow);
                nonCall.cashflow_npv_e(currentNodeIdx) = mean(payoffStateA.cashflow_npv);
                nonCall.df_e(currentNodeIdx) = mean(payoffStateA.df);
                nonCall.df_c(currentNodeIdx) = model.DFRisky(currentTime/365.0);
                
%                 % callable at maturity is the same as noncallable
%                 callable.payoffStates.cashflow(currentNodeIdx,:) = nonCall.payoffStates.cashflow(currentNodeIdx,:);
%                 callable.payoffStates.cashflow_npv(currentNodeIdx,:) = nonCall.payoffStates.cashflow_npv(currentNodeIdx,:);
%                 callable.payoff = callable.payoff + callable.payoffStates.cashflow_npv(currentNodeIdx,:);
%                 
%                 callable.cashflow_e(currentNodeIdx) = nonCall.cashflow_e(currentNodeIdx);
%                 callable.cashflow_npv_e(currentNodeIdx) = nonCall.cashflow_npv_e(currentNodeIdx);
                  
                lastTimeIdx = currentTimeIdx;
                
                
                % flag for inclusive schedule
%                 simpleYN = false;
                simpleYN = 1;
                
                for i=periodCount:-1:1
                    % past schedule neglect
                    if DateDiff(valueDate,schedule.payDates(i)) >= 0, continue;end 
                    startTimeIdx = numMethodInfo.timeStepsInfo.startTimeIdx(i);
                    endTimeIdx = numMethodInfo.timeStepsInfo.endTimeIdx(i);

                    inclusiveYN = numMethodInfo.timeStepsInfo.inclusiveYN(i);
                     %% FromToInduction start

                    %modelsStates induct backward(lastTimeIdx to endTimeIdx)
                    while endTimeIdx < lastTimeIdx
                        lastTimeIdx = lastTimeIdx -1;
                        if lastTimeIdx == endTimeIdx break;end
                    end

                    currentTimeIdx = lastTimeIdx;
                    currentTime = totalTimeSteps(currentTimeIdx);
                    % now we are at endTimeIdx
                    % variables for coupon valuation initialized
%                     ralower = security.lower;
%                     raupper = security.upper;
%                     tenorLong = security.tenorL;
%                     tenorShort = security.tenorS;

                    cD = 0; % cumulated day count fraction
                    cP = 0; 
                    dI = 0; % dummy idx for loop
                    dcfC = 0;
                    dcfN = 0;
                    dD = 0;
                    idxCMS = zeros(1,pathSize);
                    
                    idxL = zeros(1,pathSize);
                    idxS = zeros(1,pathSize);
                    
                    idxC = zeros(1,pathSize);
                    idxN = zeros(1,pathSize);
                    
                    spreadC = zeros(1,pathSize);
                    spreadN = zeros(1,pathSize);
                    probSpread = zeros(1,pathSize);
                    
                    numOfTimeSteps = numMethodInfo.timeStepsInfo.numOfTimeSteps(i);
                    
                    % if inclusive then we use simpleYN rangeProb
                    %if inclusive then we do not evaluate the last step but
                    % use fixing instead 
                    endTimeStep = 1;
                    if inclusiveYN
%                         simpleYN = true;
                        simpleYN = 1;
                        endTimeStep = 2;
                    end
                    
                    for j=numOfTimeSteps:-1:endTimeStep
                        % evaluate dealdescription
                        % last step is omitted
                        currentTime = totalTimeSteps(currentTimeIdx);
                        %dcfC = currentTime/365.0;
                        dcfC = currentTime;
                        
                        if j~= numOfTimeSteps
                            dD = (dcfN-dcfC);
                        end

                        toDate = AddDate(valueDate,currentTime,'day');
                        tpDate = schedule.payDates(i);
                        
                                                                     
                        spreadC = model.CMSCMSSpreadDigitalRangeDate(valueDate,toDate,tpDate,tenorCMS,tenorLong,tenorShort,0.25,ralower,raupper,ralowerS,raupperS);
%                         spreadC = model.CMSSpreadDigitalRangeDate(valueDate,toDate,tpDate,tenorLong,tenorShort,0.25,ralower,raupper);
                        
                        %modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:numOfFactors*(currentTimeIdx-1)+2,:);
%                         idxL = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorLong,modelStateT);
%                         idxS = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorShort,modelStateT);
%                         spreadC = idxL - idxS;
%                         probSpread = model.SRPr(spreadC,spreadN,ralower,raupper,simpleYN,pathSize)*dD; 

                        if simpleYN
                            probSpread = spreadC*dD;
                        else
                            probSpread = spreadC*dD;
%                             probSpread = 0.5*(spreadC + spreadN)*dD;
                            
                        end

                        
                        cD = cD + dD;
                        cP = cP + probSpread;

                        dcfN = dcfC;
                        spreadN = spreadC;

                        %induct backward to the next time step
                        % one time backward induction
                        lastTimeIdx = currentTimeIdx;

                        if startTimeIdx < lastTimeIdx
                            lastTimeIdx = lastTimeIdx -1;
                            currentTimeIdx = lastTimeIdx;
                        end


                    end

                    %% FromToInduction end
                    %% AtDate evaluation
                    % Adding a new cashflow to the payoff

                    dcf = schedule.dayCountFraction(i);
                    
                    if inclusiveYN
                        % we add valuation date
                        cD = schedule.dayCount(i);
                        rangeDays =  security.rangeDays(i);
                        payoffStateA.cashflow = nominal * coupon * dcf * (rangeDays(i) + cP)/cD;
                    else
                        payoffStateA.cashflow = nominal * coupon * dcf * cP/cD;
                    end
                    % eventDate(i) = startDate(i) therefore there is no induction in modelStates
                    % i.e. currentTimeIdx has not changed
                    % no need to update modelStateT
                    % modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:end,:);
                    % since cashflow's are added at the startDate(i) we need to
                    % multiply cashflow with the df(startDate(i), payDate(i))
                    payTime = DateDiff(schedule.payDates(i),valueDate);
                    
                    %event is at startDate so we need to discount the
                    %cashflow first from paydate to startdate
%                     df_sp = model.discountFactor(currentTime, payTime,numMatTime,modelStateT);

%                     for k=1:pathSize
%                         payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_sp(k);
%                     end
                    
                    % process payoff
                    
                    currentNodeIdx = i;
                    
                    % 1st nonCall column
                    nonCall.payoffStates.cashflow(currentNodeIdx,:) = payoffStateA.cashflow;
                    payoffStateA.cashflow_npv = payoffStateA.cashflow * model.DFRisky(payTime/365.0);
                    
%                     payoffStateA.cashflow_npv = model.discountPayoff(currentTime,numMatTime ...
%                     ,modelStateT,payoffStateA.cashflow);
                    nonCall.payoffStates.cashflow_npv(currentNodeIdx,:) = payoffStateA.cashflow_npv;
                    payoffStateA.df = model.DFRisky(payTime/365.0);
%                     payoffStateA.df = model.discountPayoff(currentTime,numMatTime ...
%                     ,modelStateT,1.0*ones(1,pathSize));
                    nonCall.payoffStates.df(currentNodeIdx,:) = payoffStateA.df;
                    
                    nonCall.payoff = nonCall.payoff + payoffStateA.cashflow_npv;
                    
                    nonCall.cashflow_e(currentNodeIdx) = mean(payoffStateA.cashflow);
                    nonCall.rangeProb_e(currentNodeIdx) = mean(nonCall.cashflow_e(currentNodeIdx)/(nominal * coupon * dcf));
                    nonCall.cashflow_npv_e(currentNodeIdx) = mean(payoffStateA.cashflow_npv);

                    nonCall.df_e(currentNodeIdx) = mean(payoffStateA.df);
                    nonCall.df_c(currentNodeIdx) = model.DFRisky(currentTime/365.0);
                    
%                     %2nd callable column
%                     callable.payoffStates.cashflow(currentNodeIdx,:) = nonCall.payoffStates.cashflow(currentNodeIdx,:);
%                     callable.payoffStates.cashflow_npv(currentNodeIdx,:) = nonCall.payoffStates.cashflow_npv(currentNodeIdx,:);
%                     callable.payoff = callable.payoff + callable.payoffStates.cashflow_npv(currentNodeIdx,:);
%                     
%                     callable.cashflow_e(currentNodeIdx) = nonCall.cashflow_e(currentNodeIdx);
%                     callable.cashflow_npv_e(currentNodeIdx) = nonCall.cashflow_npv_e(currentNodeIdx);

                    % if exercise is possible
                    if i == 1
                        dummaaa = 1;
                    end
                    
                    % call event happens  at the start of the period
                    % at call date, by default, call option is expired
                    
%                     if callYN(i) && (DateDiff(valueDate,schedule.startDates(i)) < 0)
%                         % 1.undiscount callable.payoff
%                         % 2.build the state variables matrix
%                         % 3.regress 1's with the given state variables
%                         % (model states 1,2)
%                         % 4. exercise per each path in
%                         % discount the new payoff and assign into callable
%                         % column
%                         
%                         %1 undiscount callable.payoff and get the
%                         %contiValue
%                        
%                         %exercise value
%                         exerciseValue = security.nominal*ones(1,pathSize);
%                         contiValue = zeros(1,pathSize);
%                         onePayoff = ones(1,pathSize);
%                         onePayoffD = model.discountPayoff(currentTime,numMatTime ...
%                         ,modelStateT,onePayoff);
%                         contiValue = callable.payoff./onePayoffD;
%                         
%                         % build the state variabls matrix
%                         decisionVar1 = modelStateT(1,:);
%                         decisionVar2 = modelStateT(2,:);
%                         % we use upto 2nd order polynomial as regression
%                         % equatioin
%                         dimensionLS = 6;
%                         regMatrix = ones(pathSize,dimensionLS);
%                         
%                         for idx_r=1:pathSize
%                             regMatrix(idx_r,2) = decisionVar1(idx_r);
%                             regMatrix(idx_r,3) = decisionVar2(idx_r);
%                             regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r);
%                             regMatrix(idx_r,5) = decisionVar1(idx_r)*decisionVar2(idx_r);
%                             regMatrix(idx_r,6) = decisionVar2(idx_r)*decisionVar2(idx_r);
%                         end
%                         
%                         % simple regression <- for path dependent option
%                         % we should change it
%                         % beta  : regression coefficients
%                         beta =zeros(dimensionLS,1);
%                         beta = regress(contiValue',regMatrix);
%                         regContiValue = regMatrix*beta;
%                         
%                         exercisedPayoff = regContiValue;
%                         % if regContiValue is bigger than exerciseValue
%                         % then issue exercise call option
%                         for idx_r=1:pathSize
%                             if regContiValue(idx_r) >= exerciseValue(idx_r) 
%                                 exercisedPayoff(idx_r) = exerciseValue(idx_r);
%                             end
%                         end
%                         
%                         % we discount the new payoff and 
%                         % assign it into the existing callable payoff
%                         callable.payoff = model.discountPayoff(currentTime,numMatTime ...
%                         ,modelStateT,exercisedPayoff);
%                         
%                     end
                end

                nonCall.npv = mean(nonCall.payoff);
                out.pricerInfo.nonCall = nonCall;
                
%                 callable.npv = mean(callable.payoff);
%                 out.pricerInfo.callable = callable;
                out.daycountFraction = schedule.dayCountFraction;
                
        end
        
        
        
        
    end
    
end



In [ ]:
classdef DailyWeeklyRiskyPricer_CDUSDSpreadRA < H_RootObject
    %Pricer(Security,Model,NumMethod)
    %Prcer is constructed from Security, Model, NumMethod 
    
    properties
        m_security
        m_model
        m_numMethod
        overHedgeYN
    end
    
    methods
        function pricer = DailyWeeklyRiskyPricer_CDUSDSpreadRA(security,model,numMethod,overHedgeYN)
                pricer.m_security = security;
                pricer.m_model = model;
                pricer.m_numMethod = numMethod;
                pricer.overHedgeYN = overHedgeYN;
        end
        
        function out = computePriceMCCallableGeneric(pricer,valueDate,pricerOption)
                security = pricer.m_security;
                numMethod = pricer.m_numMethod;
                model = pricer.m_model;
                schedule = security.couponSchedule;
                callSchedule = security.callSchedule;
                
                % numMethodTimeSteps Generate
                mcOneTimeStep = numMethod.mcOneTimeStep;
                outInfo = numMethod.numMethodTimeStepsGenerateDW(valueDate,schedule,mcOneTimeStep);
            
                numMethodInfo.timeStepsInfo = outInfo;

                % mc random number init
                %pathSize = 2^16;
                pathSize = numMethod.pathSize;

                % generate random numbers and modelstates forward
                rng('default');
                rng(0);

                % LGM1F : one factor
                % LGM2F : two factor
                
                numOfFactors = model.numOfFactor;

                %Z= randn(numOfFactors*outInfo.totalTimeStepsSize,pathSize/2);
                Z= randn(pathSize/2,numOfFactors*outInfo.totalTimeStepsSize);
                Z=[Z;-Z];
                Z=Z';
                
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;

                % we check with the c code, so we use generated random numbers
%                 mc_random = importdata('mc_random.xlsx');
%                 mc_random = importdata('D:\mc_random_number_new_report.txt');
%                 Z= mc_random;
                
                numMethodInfo.pathSize = pathSize;
                numMethodInfo.numOfFactors = numOfFactors;
                numMethodInfo.totalTimeSteps = outInfo.totalTimeSteps;
                numMethodInfo.totalTimeStepsSize = outInfo.totalTimeStepsSize;
                numMethodInfo.Z = Z;



                %numMethod generate model states for the given timeSteps
                modelStates = numMethod.generateModelStatesNF(valueDate,schedule,numMethodInfo,model);

                % numeraire info
                numMatTime = numMethodInfo.totalTimeSteps(end);
                days = DateDiff(schedule.payDates(schedule.numOfPeriod),valueDate);
                numMatTime = max(numMatTime, days);
                dfNumMat = model.DFRisky(numMatTime/365.0);
                
                numeraire.numMatTime = numMatTime;
                numeraire.dfNumMat = dfNumMat;
                
                % induct backward and calculate cashflow
            
                periodCount = schedule.numOfPeriod;
                
                % pricer single info initialize
                
                nonCall = PricerInfo(periodCount +1, pathSize);
                callable = PricerInfo(periodCount +1, pathSize);
                dfCol = PricerInfo(periodCount +1, pathSize);
                
                dummy1 = PricerInfo(periodCount +1, pathSize);
                dummy2 = PricerInfo(periodCount +1, pathSize);
                
                
                % intermediate variables
                payoffStateA.cashflow = zeros(1, pathSize);
                payoffStateA.cashflow_npv = zeros(1, pathSize);
                
            
                % we calculate cashflow backward from maturityDate to valueDate
                % rangeAccrual coupons are added at the startDate of the Period
                % At maturity

                nominal = security.nominal;
                coupon = security.coupon;
                callYN = security.callYN;
                
                ralower = security.lower;
                raupper = security.upper;
                tenorLong = security.tenorL;
                tenorShort = security.tenorS;
                
                currentTime = numMethodInfo.totalTimeSteps(end);
                totalTimeStepsSize = numMethodInfo.totalTimeStepsSize;
                currentTimeIdx = totalTimeStepsSize;
                
                currentNodeIdx = periodCount + 1;
                
                modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:numOfFactors*(currentTimeIdx-1)+numOfFactors,:);
                
                payoffStateA.cashflow = nominal*ones(1,pathSize);
                
                payTime = DateDiff(schedule.payDates(schedule.numOfPeriod),valueDate);
                df_ep = model.discountFactorRisky(currentTime, payTime,numMatTime,modelStateT);
                for k=1:pathSize
                    payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                end
                
                %process payoff at maturity
                nonCall.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                
                callable.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                
                %df column
                payoffStateA.cashflow = ones(1,pathSize);
                for k=1:pathSize
                    payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                end
                
                dfCol.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                                 
                %dummy column
                foreignModelName = model.foreignModelName;
                
                if strcmp(pricerOption,'PSA')
                    payoffStateA.cashflow = model.CMS_PSA_SimpleDate(model.refModelName,valueDate,currentTime,numMatTime, tenorLong,modelStateT);
                    dummy1.ProcessPayoff(model,currentTime,numeraire, ...
                                 modelStateT,payoffStateA.cashflow,currentNodeIdx);
                    
                    payoffStateA.cashflow = model.CMS_PSA_SimpleDate(foreignModelName,valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                    dummy2.ProcessPayoff(model,currentTime,numeraire, ...
                                 modelStateT,payoffStateA.cashflow,currentNodeIdx);
                
                else
                    payoffStateA.cashflow = model.Libor_SimpleDate(model.refModelName,valueDate,currentTime,currentTime,numMatTime, tenorLong,modelStateT); 
                    dummy1.ProcessPayoff(model,currentTime,numeraire, ...
                                 modelStateT,payoffStateA.cashflow,currentNodeIdx);
                             
                    payoffStateA.cashflow = model.Libor_SimpleDate(foreignModelName,valueDate,currentTime,currentTime,numMatTime, tenorShort,modelStateT); 
                    dummy2.ProcessPayoff(model,currentTime,numeraire, ...
                                 modelStateT,payoffStateA.cashflow,currentNodeIdx);
                    
                end
                

%                 payoffStateA.cashflow = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                
%                 payoffStateA.cashflow = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                
                
                                 
                  

                lastTimeIdx = currentTimeIdx;
                totalTimeSteps = numMethodInfo.timeStepsInfo.totalTimeSteps;
                
                % flag for inclusive schedule
%                 simpleYN = false;
%                 simpleYN = true;
                if pricer.overHedgeYN == true 
                    simpleYN = 2;
                else
                    simpleYN = 1;
                end
                
                for i=periodCount:-1:1
                    % past schedule neglect
                    if DateDiff(valueDate,schedule.payDates(i)) >= 0, continue;end 
                    startTimeIdx = numMethodInfo.timeStepsInfo.startTimeIdx(i);
                    endTimeIdx = numMethodInfo.timeStepsInfo.endTimeIdx(i);

                    inclusiveYN = numMethodInfo.timeStepsInfo.inclusiveYN(i);
                 %% FromToInduction start

                    %modelsStates induct backward(lastTimeIdx to endTimeIdx)
                    while endTimeIdx < lastTimeIdx
                        lastTimeIdx = lastTimeIdx -1;
                        if lastTimeIdx == endTimeIdx break;end
                    end

                    currentTimeIdx = lastTimeIdx;
                    currentTime = totalTimeSteps(currentTimeIdx);
                    % now we are at endTimeIdx
                    % variables for coupon valuation initialized
                    

                    cD = 0; % cumulated day count fraction
                    cP = 0; 
                    dI = 0; % dummy idx for loop
                    dcfC = 0;
                    dcfN = 0;
                    dD = 0;
                    idxL = zeros(1,pathSize);
                    idxS = zeros(1,pathSize);
                    spreadC = zeros(1,pathSize);
                    spreadN = zeros(1,pathSize);
                    probSpread = zeros(1,pathSize);
                    
                    numOfTimeSteps = numMethodInfo.timeStepsInfo.numOfTimeSteps(i);
                    
                    % if inclusive then we use simpleYN rangeProb
                    %if inclusive then we do not evaluate the last step but
                    % use fixing instead 
                    endTimeStep = 1;
                    if inclusiveYN
%                         simpleYN = true;
                        simpleYN = 1;
                        endTimeStep = 2;
                        if pricer.overHedgeYN == true 
                            simpleYN = 2;
                        end
                    end
                    
                    for j=numOfTimeSteps:-1:endTimeStep
                        % evaluate dealdescription
                        % last step is omitted
                        currentTime = totalTimeSteps(currentTimeIdx);
                        %dcfC = currentTime/365.0;
                        dcfC = currentTime;
                        
                        if j~= numOfTimeSteps
                            dD = (dcfN-dcfC);
                        end

                        modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:numOfFactors*(currentTimeIdx-1)+numOfFactors,:);
                        
                        if strcmp(pricerOption,'PSA')
                            idxL = model.CMS_PSA_SimpleDate(model.refModelName,valueDate,currentTime,numMatTime, tenorLong,modelStateT);
                            idxS = model.CMS_PSA_SimpleDate(foreignModelName,valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                        else
                            idxL = model.Libor_SimpleDate(model.refModelName,valueDate,currentTime,currentTime,numMatTime, tenorLong,modelStateT);
                            idxS = model.Libor_SimpleDate(foreignModelName,valueDate,currentTime,currentTime,numMatTime, tenorShort,modelStateT);
                        end
                        spreadC = idxL - idxS;

                        probSpread = model.SRPr(spreadC,spreadN,ralower,raupper,simpleYN,pathSize)*dD;
                        cD = cD + dD;
                        cP = cP + probSpread;

                        dcfN = dcfC;
                        spreadN = spreadC;

                        %induct backward to the next time step
                        % one time backward induction
                        lastTimeIdx = currentTimeIdx;

                        if startTimeIdx < lastTimeIdx
                            lastTimeIdx = lastTimeIdx -1;
                            currentTimeIdx = lastTimeIdx;
                        end


                    end

                    %% FromToInduction end
        % if the current period is inclusive, then the start date is not evaluated in the above for loop.
		% instead the fixing will be treated separately below.
		% however current_time_idx is updated at the end of the for loop.
		% so we need to update model_states accordingly.
		% if model_states were updated previously then the following additionaly step doesn't affect the result anyway.
                    currentTime = totalTimeSteps(currentTimeIdx);
                    modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:numOfFactors*(currentTimeIdx-1)+numOfFactors,:);
                    
                    %% AtDate evaluation
                    % Adding a new cashflow to the payoff

                    dcf = schedule.dayCountFraction(i);
                    
                    if inclusiveYN
                        % we add valuation date
                        cD = schedule.dayCount(i);
                        rangeDays =  security.rangeDays(i);
                        payoffStateA.cashflow = nominal * coupon * dcf * (rangeDays + cP)/cD;
                    else
                        payoffStateA.cashflow = nominal * coupon * dcf * cP/cD;
                    end
                    % eventDate(i) = startDate(i) therefore there is no induction in modelStates
                    % i.e. currentTimeIdx has not changed
                    % no need to update modelStateT
                    % modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:end,:);
                    % since cashflow's are added at the startDate(i) we need to
                    % multiply cashflow with the df(startDate(i), payDate(i))
                    payTime = DateDiff(schedule.payDates(i),valueDate);
                    
                    %event is at startDate so we need to discount the
                    %cashflow first from paydate to startdate
                    df_ep = model.discountFactorRisky(currentTime, payTime,numMatTime,modelStateT);

                    for k=1:pathSize
                        payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                    end
                    
                    % process payoff
                    
                    currentNodeIdx = i;
                    
                    nonCall.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                
                    callable.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                    
                    %df column
                    payoffStateA.cashflow = ones(1,pathSize);
                    for k=1:pathSize
                        payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                    end
                    
                    dfCol.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                                 
                    %dummy column
                    if strcmp(pricerOption,'PSA')
                        payoffStateA.cashflow = model.CMS_PSA_SimpleDate(model.refModelName,valueDate,currentTime,numMatTime, tenorLong,modelStateT);
                        dummy1.ProcessPayoff(model,currentTime,numeraire, ...
                                         modelStateT,payoffStateA.cashflow,currentNodeIdx);



                        payoffStateA.cashflow = model.CMS_PSA_SimpleDate(foreignModelName,valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                        dummy2.ProcessPayoff(model,currentTime,numeraire, ...
                                         modelStateT,payoffStateA.cashflow,currentNodeIdx);
                    
                    else
                        payoffStateA.cashflow = model.Libor_SimpleDate(model.refModelName,valueDate,currentTime,currentTime,numMatTime, tenorLong,modelStateT); 
                        dummy1.ProcessPayoff(model,currentTime,numeraire, ...
                                 modelStateT,payoffStateA.cashflow,currentNodeIdx);
                        
                        payoffStateA.cashflow = model.Libor_SimpleDate(foreignModelName,valueDate,currentTime,currentTime,numMatTime, tenorShort,modelStateT); 
                        dummy2.ProcessPayoff(model,currentTime,numeraire, ...
                                 modelStateT,payoffStateA.cashflow,currentNodeIdx);
                        
                    end

                                 
                    
                    
                    
                    % if exercise is possible
                    if i == 1
                        dummaaa = 1;
                    end
                    
                    % call event happens  at the start of the period
                    % at call date, by default, call option is expired
                    
                    if callYN(i) && (DateDiff(valueDate,callSchedule.resetDates(i)) < 0)
                        % 1.undiscount callable.payoff
                        % 2.build the state variables matrix
                        % 3.regress 1's with the given state variables
                        % (model states 1,2)
                        % 4. exercise per each path in
                        % discount the new payoff and assign into callable
                        % column
                        
                        %1 undiscount callable.payoff and get the
                        %contiValue
                       
                        %exercise value
                        exerciseValue = security.nominal*ones(1,pathSize);
                        payTime = DateDiff(callSchedule.payDates(i),valueDate);
                        %event is at startDate so we need to discount the
                        %cashflow first from paydate to startdate
                        df_ep = model.discountFactorRisky(currentTime, payTime,numMatTime,modelStateT);
                        for k=1:pathSize
                            exerciseValue(k) = exerciseValue(k) * df_ep(k);
                        end
                        
                        contiValue = zeros(1,pathSize);
                        onePayoff = ones(1,pathSize);
                        onePayoffD = model.discountPayoffRisky(currentTime,numMatTime ...
                        ,modelStateT,onePayoff);
                        contiValue = callable.payoff./onePayoffD;
                        
                        %Nominal Regression test
                        contiValue = contiValue/10000;
                        
                        % build the state variabls matrix
                        dimensionLS = (numOfFactors + 1)*(numOfFactors + 2)/2;
                        
                        decisionVar = zeros(numOfFactors,size(modelStateT,2));
                        
                        for idxLS =1:numOfFactors
                            decisionVar(idxLS,:) = modelStateT(idxLS,:); 
                        end
%                         decisionVar1 = modelStateT(1,:);
%                         decisionVar2 = modelStateT(2,:);
                        % we use upto 2nd order polynomial as regression
                        % equatioin
                        
                        regMatrix = ones(pathSize,dimensionLS);
                        
                        for idx_r=1:pathSize
                            for idxLS=1:numOfFactors
                                regMatrix(idx_r,idxLS+1) = decisionVar(idxLS,idx_r);
                            end
                            
                            tempIdx = 1;
                            for idxLS1=1:numOfFactors
                                for idxLS2=idxLS1:numOfFactors
                                    regMatrix(idx_r,numOfFactors+1+tempIdx) = decisionVar(idxLS1,idx_r)*decisionVar(idxLS2,idx_r);
                                    tempIdx = tempIdx + 1;
                                end
                            end
                            
                        end
                        
                        % simple regression <- for path dependent option
                        % we should change it
                        % beta  : regression coefficients
                        beta =zeros(dimensionLS,1);
                        beta = regress(contiValue',regMatrix);
                        regContiValue = regMatrix*beta;
                       
                         %Nominal Regression test
                        regContiValue = regContiValue * 10000;
                        
                        exercisedPayoff = regContiValue;
                        % if regContiValue is bigger than exerciseValue
                        % then issue exercise call option
                        for idx_r=1:pathSize
                            if regContiValue(idx_r) >= exerciseValue(idx_r) 
                                exercisedPayoff(idx_r) = exerciseValue(idx_r);
                            end
                        end
                        
                        % we discount the new payoff and 
                        % assign it into the existing callable payoff
                        callable.payoff = model.discountPayoffRisky(currentTime,numMatTime ...
                        ,modelStateT,exercisedPayoff);
                        
                    end
                end

                nonCall.npv = mean(nonCall.payoff);
                out.nonCall = nonCall;
                callable.npv = mean(callable.payoff);
                dfCol.npv = mean(dfCol.payoff);
                dummy1.npv = mean(dummy1.npv);
                dummy2.npv = mean(dummy2.npv);
                out.callable = callable;
                out.dfCol = dfCol;
                out.dummy1 = dummy1;
                out.dummy2 = dummy2;
                out.daycountFraction = schedule.dayCountFraction;
                
        end
        
        
        function out = computePriceClosedSimple(pricer,valueDate)
                security = pricer.m_security;
                numMethod = pricer.m_numMethod;
                model = pricer.m_model;
                schedule = security.couponSchedule;
                
                % numMethodTimeSteps Generate
                mcOneTimeStep = numMethod.mcOneTimeStep;
                outInfo = numMethod.numMethodTimeStepsGenerateDW(valueDate,schedule,mcOneTimeStep);
            
                numMethodInfo.timeStepsInfo = outInfo;

                % mc random number init
                %pathSize = 2^16;
                pathSize = numMethod.pathSize;
                
                pathSize = 1;
                
%                 % generate random numbers and modelstates forward
%                 rng('default');
%                 rng(0);
% 
                numOfFactors = model.numOfFactor;
% 
%                 %Z= randn(numOfFactors*outInfo.totalTimeStepsSize,pathSize/2);
%                 Z= randn(pathSize/2,numOfFactors*outInfo.totalTimeStepsSize);
%                 Z=[Z;-Z];
%                 Z=Z';
%                 
%                 
%                 %TimeInversion
%                 sizeZ = size(Z,1);
%                 A = zeros(sizeZ,sizeZ);
%                 for i=1:sizeZ
%                     A(i,sizeZ + 1 -i) = 1;
%                 end
%                 Z = A*Z;
% 
%                 % we check with the c, so we use generated random numbers
% %                 mc_random = importdata('mc_random.xlsx');
% %                 mc_random = importdata('D:\mc_random_number_new_report.txt');
% %                 Z= mc_random;
%                 
                numMethodInfo.pathSize = pathSize;
                numMethodInfo.numOfFactors = numOfFactors;
                numMethodInfo.totalTimeSteps = outInfo.totalTimeSteps;
                numMethodInfo.totalTimeStepsSize = outInfo.totalTimeStepsSize;
%                 numMethodInfo.Z = Z;
% 
% 
% 
%                 %generate model states for the given timeSteps
%                 modelStates = numMethod.generateModelStates(valueDate,schedule,numMethodInfo,model);

                % numeraire info
                numMatTime = numMethodInfo.totalTimeSteps(end);
                dfNumMat = model.DFRisky(numMatTime/365.0);
                
                % induct backward and calculate cashflow
            
                periodCount = schedule.numOfPeriod;
                
                % cashflow columns initialize
                % 1st nonCall
                nonCall.npv = 0;
                nonCall.cashflow_e = zeros(periodCount + 1,1);
                nonCall.cashflow_npv_e = zeros(periodCount + 1,1);
                nonCall.rangeProb_e = zeros(periodCount + 1,1);
                nonCall.df_e = zeros(periodCount + 1,1);
                nonCall.df_c = zeros(periodCount + 1,1);

                % debug information, cashflow & npv per path
                nonCall.payoff = zeros(1,pathSize);
                nonCall.payoffStates.cashflow = zeros(periodCount+1,pathSize);
                nonCall.payoffStates.cashflow_npv = zeros(periodCount+1,pathSize);
                nonCall.payoffStates.df = zeros(periodCount + 1,pathSize);
                
                % intermediate variables
                payoffStateA.cashflow = zeros(1, pathSize);
                payoffStateA.cashflow_npv = zeros(1, pathSize);
                payoffStateA.df = zeros(1,pathSize);
                
                % we calculate cashflow backward from maturityDate to valueDate
                % rangeAccrual coupons are added at the startDate of the Period
                % At maturity

                nominal = security.nominal;
                coupon = security.coupon;
                callYN = security.callYN;
                
                currentTime = numMethodInfo.totalTimeSteps(end);
                totalTimeSteps = numMethodInfo.timeStepsInfo.totalTimeSteps;
                totalTimeStepsSize = numMethodInfo.timeStepsInfo.totalTimeStepsSize;
                if totalTimeStepsSize == 0 || (totalTimeStepsSize == 1 && totalTimeSteps(1) == 0) 
                    out.nonCall = nonCall;
                    return;
                end
                
                currentTimeIdx = totalTimeStepsSize;
                
                currentNodeIdx = periodCount + 1;
                
%                 modelStateT = modelStates(numOfFactors*(totalTimeStepsSize-1)+1:numOfFactors*(totalTimeStepsSize-1)+2,:);
                payoffStateA.cashflow = nominal*ones(1,pathSize);
                %process payoff at maturity
                nonCall.payoffStates.cashflow(currentNodeIdx,:) = payoffStateA.cashflow;
                
%                 payoffStateA.cashflow_npv = model.discountPayoff(currentTime,numMatTime ...
%                     ,modelStateT,payoffStateA.cashflow);
%                 
                payoffStateA.cashflow_npv = model.DFRisky(currentTime/365.0)*payoffStateA.cashflow;
                
                nonCall.payoffStates.cashflow_npv(currentNodeIdx,:) = payoffStateA.cashflow_npv ;
                
%                 payoffStateA.df = model.discountPayoff(currentTime,numMatTime ...
%                     ,modelStateT,1.0*ones(1,pathSize));
                payoffStateA.df = model.DFRisky(currentTime/365.0) *ones(1,pathSize);
                    

                nonCall.payoffStates.df(currentNodeIdx,:) = payoffStateA.df;
                nonCall.payoff = nonCall.payoff + payoffStateA.cashflow_npv;
                
                nonCall.cashflow_e(currentNodeIdx) = mean(payoffStateA.cashflow);
                nonCall.cashflow_npv_e(currentNodeIdx) = mean(payoffStateA.cashflow_npv);
                nonCall.df_e(currentNodeIdx) = mean(payoffStateA.df);
                nonCall.df_c(currentNodeIdx) = model.DFRisky(currentTime/365.0);
                
%                 % callable at maturity is the same as noncallable
%                 callable.payoffStates.cashflow(currentNodeIdx,:) = nonCall.payoffStates.cashflow(currentNodeIdx,:);
%                 callable.payoffStates.cashflow_npv(currentNodeIdx,:) = nonCall.payoffStates.cashflow_npv(currentNodeIdx,:);
%                 callable.payoff = callable.payoff + callable.payoffStates.cashflow_npv(currentNodeIdx,:);
%                 
%                 callable.cashflow_e(currentNodeIdx) = nonCall.cashflow_e(currentNodeIdx);
%                 callable.cashflow_npv_e(currentNodeIdx) = nonCall.cashflow_npv_e(currentNodeIdx);
                  
                lastTimeIdx = currentTimeIdx;
                
                
                % flag for inclusive schedule
%                 simpleYN = false;
                simpleYN = 1;
                
                for i=periodCount:-1:1
                    % past schedule neglect
                    if DateDiff(valueDate,schedule.payDates(i)) >= 0, continue;end 
                    startTimeIdx = numMethodInfo.timeStepsInfo.startTimeIdx(i);
                    endTimeIdx = numMethodInfo.timeStepsInfo.endTimeIdx(i);

                    inclusiveYN = numMethodInfo.timeStepsInfo.inclusiveYN(i);
                     %% FromToInduction start

                    %modelsStates induct backward(lastTimeIdx to endTimeIdx)
                    while endTimeIdx < lastTimeIdx
                        lastTimeIdx = lastTimeIdx -1;
                        if lastTimeIdx == endTimeIdx break;end
                    end

                    currentTimeIdx = lastTimeIdx;
                    currentTime = totalTimeSteps(currentTimeIdx);
                    % now we are at endTimeIdx
                    % variables for coupon valuation initialized
                    ralower = security.lower;
                    raupper = security.upper;
                    tenorLong = security.tenorL;
                    tenorShort = security.tenorS;

                    cD = 0; % cumulated day count fraction
                    cP = 0; 
                    dI = 0; % dummy idx for loop
                    dcfC = 0;
                    dcfN = 0;
                    dD = 0;
                    idxL = zeros(1,pathSize);
                    idxS = zeros(1,pathSize);
                    spreadC = zeros(1,pathSize);
                    spreadN = zeros(1,pathSize);
                    probSpread = zeros(1,pathSize);
                    
                    numOfTimeSteps = numMethodInfo.timeStepsInfo.numOfTimeSteps(i);
                    
                    % if inclusive then we use simpleYN rangeProb
                    %if inclusive then we do not evaluate the last step but
                    % use fixing instead 
                    endTimeStep = 1;
                    if inclusiveYN
%                         simpleYN = true;
                        simpleYN = 1;
                        endTimeStep = 2;
                    end
                    
                    for j=numOfTimeSteps:-1:endTimeStep
                        % evaluate dealdescription
                        % last step is omitted
                        currentTime = totalTimeSteps(currentTimeIdx);
                        %dcfC = currentTime/365.0;
                        dcfC = currentTime;
                        
                        if j~= numOfTimeSteps
                            dD = (dcfN-dcfC);
                        end

                        toDate = AddDate(valueDate,currentTime,'day');
                        tpDate = schedule.payDates(i);
                        spreadC = model.CMSCMTSpreadDigitalRangeDate(valueDate,toDate,tpDate,tenorLong,tenorShort,ralower,raupper);
                        
                        %modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:numOfFactors*(currentTimeIdx-1)+2,:);
%                         idxL = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorLong,modelStateT);
%                         idxS = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorShort,modelStateT);
%                         spreadC = idxL - idxS;
%                         probSpread = model.SRPr(spreadC,spreadN,ralower,raupper,simpleYN,pathSize)*dD; 

                        if simpleYN
                            probSpread = spreadC*dD;
                        else
                            probSpread = spreadC*dD;
%                             probSpread = 0.5*(spreadC + spreadN)*dD;
                            
                        end

                        
                        cD = cD + dD;
                        cP = cP + probSpread;

                        dcfN = dcfC;
                        spreadN = spreadC;

                        %induct backward to the next time step
                        % one time backward induction
                        lastTimeIdx = currentTimeIdx;

                        if startTimeIdx < lastTimeIdx
                            lastTimeIdx = lastTimeIdx -1;
                            currentTimeIdx = lastTimeIdx;
                        end


                    end

                    %% FromToInduction end
                    %% AtDate evaluation
                    % Adding a new cashflow to the payoff

                    dcf = schedule.dayCountFraction(i);
                    
                    if inclusiveYN
                        % we add valuation date
                        cD = schedule.dayCount(i);
                        rangeDays =  security.rangeDays(i);
                        payoffStateA.cashflow = nominal * coupon * dcf * (rangeDays + cP)/cD;
                    else
                        payoffStateA.cashflow = nominal * coupon * dcf * cP/cD;
                    end
                    % eventDate(i) = startDate(i) therefore there is no induction in modelStates
                    % i.e. currentTimeIdx has not changed
                    % no need to update modelStateT
                    % modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:end,:);
                    % since cashflow's are added at the startDate(i) we need to
                    % multiply cashflow with the df(startDate(i), payDate(i))
                    payTime = DateDiff(schedule.payDates(i),valueDate);
                    
                    %event is at startDate so we need to discount the
                    %cashflow first from paydate to startdate
%                     df_sp = model.discountFactor(currentTime, payTime,numMatTime,modelStateT);

%                     for k=1:pathSize
%                         payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_sp(k);
%                     end
                    
                    % process payoff
                    
                    currentNodeIdx = i;
                    
                    % 1st nonCall column
                    nonCall.payoffStates.cashflow(currentNodeIdx,:) = payoffStateA.cashflow;
                    payoffStateA.cashflow_npv = payoffStateA.cashflow * model.DFRisky(payTime/365.0);
                    
%                     payoffStateA.cashflow_npv = model.discountPayoff(currentTime,numMatTime ...
%                     ,modelStateT,payoffStateA.cashflow);
                    nonCall.payoffStates.cashflow_npv(currentNodeIdx,:) = payoffStateA.cashflow_npv;
                    payoffStateA.df = model.DFRisky(payTime/365.0);
%                     payoffStateA.df = model.discountPayoff(currentTime,numMatTime ...
%                     ,modelStateT,1.0*ones(1,pathSize));
                    nonCall.payoffStates.df(currentNodeIdx,:) = payoffStateA.df;
                    
                    nonCall.payoff = nonCall.payoff + payoffStateA.cashflow_npv;
                    
                    nonCall.cashflow_e(currentNodeIdx) = mean(payoffStateA.cashflow);
                    nonCall.rangeProb_e(currentNodeIdx) = mean(nonCall.cashflow_e(currentNodeIdx)/(nominal * coupon * dcf));
                    nonCall.cashflow_npv_e(currentNodeIdx) = mean(payoffStateA.cashflow_npv);

                    nonCall.df_e(currentNodeIdx) = mean(payoffStateA.df);
                    nonCall.df_c(currentNodeIdx) = model.DFRisky(currentTime/365.0);
                    

                end

                nonCall.npv = mean(nonCall.payoff);
                out.pricerInfo.nonCall = nonCall;
                
%                 callable.npv = mean(callable.payoff);
%                 out.pricerInfo.callable = callable;
                out.daycountFraction = schedule.dayCountFraction;
                
        end
        

        
        
        
        
    end
    
end



In [ ]:
classdef DailyWeeklyRiskyPricer_CDUSDLibor3MSpreadRA_LGM2FCK1F < H_RootObject
    %Pricer(Security,Model,NumMethod)
    %Prcer is constructed from Security, Model, NumMethod 
    
    properties
        m_security
        m_model
        m_numMethod
        overHedgeYN
    end
    
    methods
        function pricer = DailyWeeklyRiskyPricer_CDUSDLibor3MSpreadRA_LGM2FCK1F(security,model,numMethod,overHedgeYN)
                pricer.m_security = security;
                pricer.m_model = model;
                pricer.m_numMethod = numMethod;
                pricer.overHedgeYN = overHedgeYN;
        end
        
        function out = computePriceMCCallableGeneric(pricer,valueDate,pricerOption)
                security = pricer.m_security;
                numMethod = pricer.m_numMethod;
                model = pricer.m_model;
                schedule = security.couponSchedule;
                callSchedule = security.callSchedule;
                
                % numMethodTimeSteps Generate
                mcOneTimeStep = numMethod.mcOneTimeStep;
                outInfo = numMethod.numMethodTimeStepsGenerateDW(valueDate,schedule,mcOneTimeStep);
            
                numMethodInfo.timeStepsInfo = outInfo;

                % mc random number init
                %pathSize = 2^16;
                pathSize = numMethod.pathSize;

                % generate random numbers and modelstates forward
                rng('default');
                rng(0);

                % LGM1F : one factor
                % LGM2F : two factor
                
                numOfFactors = model.numOfFactor;

                %Z= randn(numOfFactors*outInfo.totalTimeStepsSize,pathSize/2);
                Z= randn(pathSize/2,numOfFactors*outInfo.totalTimeStepsSize);
                Z=[Z;-Z];
                Z=Z';
                
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;

                % we check with the c code, so we use generated random numbers
%                 mc_random = importdata('mc_random.xlsx');
%                 mc_random = importdata('D:\mc_random_number_new_report.txt');
%                 Z= mc_random;
                
                numMethodInfo.pathSize = pathSize;
                numMethodInfo.numOfFactors = numOfFactors;
                numMethodInfo.totalTimeSteps = outInfo.totalTimeSteps;
                numMethodInfo.totalTimeStepsSize = outInfo.totalTimeStepsSize;
                numMethodInfo.Z = Z;



                %numMethod generate model states for the given timeSteps
                modelStates = numMethod.generateModelStatesNF(valueDate,schedule,numMethodInfo,model);

                % numeraire info
                numMatTime = numMethodInfo.totalTimeSteps(end);
                days = DateDiff(schedule.payDates(schedule.numOfPeriod),valueDate);
                numMatTime = max(numMatTime, days);
                dfNumMat = model.DFRisky(numMatTime/365.0);
                
                numeraire.numMatTime = numMatTime;
                numeraire.dfNumMat = dfNumMat;
                
                % induct backward and calculate cashflow
            
                periodCount = schedule.numOfPeriod;
                
                % pricer single info initialize
                
                nonCall = PricerInfo(periodCount +1, pathSize);
                callable = PricerInfo(periodCount +1, pathSize);
                dfCol = PricerInfo(periodCount +1, pathSize);
                
                dummy1 = PricerInfo(periodCount +1, pathSize);
                dummy2 = PricerInfo(periodCount +1, pathSize);
                
                
                % intermediate variables
                payoffStateA.cashflow = zeros(1, pathSize);
                payoffStateA.cashflow_npv = zeros(1, pathSize);
                
            
                % we calculate cashflow backward from maturityDate to valueDate
                % rangeAccrual coupons are added at the startDate of the Period
                % At maturity

                nominal = security.nominal;
                coupon = security.coupon;
                callYN = security.callYN;
                
                ralower = security.lower;
                raupper = security.upper;
                tenorLong = security.tenorL;
                tenorShort = security.tenorS;
                
                currentTime = numMethodInfo.totalTimeSteps(end);
                totalTimeStepsSize = numMethodInfo.totalTimeStepsSize;
                currentTimeIdx = totalTimeStepsSize;
                
                currentNodeIdx = periodCount + 1;
                
                modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:numOfFactors*(currentTimeIdx-1)+numOfFactors,:);
                
                payoffStateA.cashflow = nominal*ones(1,pathSize);
                
                payTime = DateDiff(schedule.payDates(schedule.numOfPeriod),valueDate);
                df_ep = model.discountFactorRisky(currentTime, payTime,numMatTime,modelStateT);
                for k=1:pathSize
                    payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                end
                
                %process payoff at maturity
                nonCall.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                
                callable.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                
                %df column
                payoffStateA.cashflow = ones(1,pathSize);
                for k=1:pathSize
                    payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                end
                
                dfCol.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                                 
                %dummy column
                foreignModelName = model.foreignModelName;
                
                if strcmp(pricerOption,'PSA')
                    payoffStateA.cashflow = model.CMS_PSA_SimpleDate(model.refModelName,valueDate,currentTime,numMatTime, tenorLong,modelStateT);
                    dummy1.ProcessPayoff(model,currentTime,numeraire, ...
                                 modelStateT,payoffStateA.cashflow,currentNodeIdx);
                    
                    payoffStateA.cashflow = model.CMS_PSA_SimpleDate(foreignModelName,valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                    dummy2.ProcessPayoff(model,currentTime,numeraire, ...
                                 modelStateT,payoffStateA.cashflow,currentNodeIdx);
                
                else
                    payoffStateA.cashflow = model.Libor_SimpleDate(model.refModelName,valueDate,currentTime,currentTime,numMatTime, tenorLong,modelStateT); 
                    dummy1.ProcessPayoff(model,currentTime,numeraire, ...
                                 modelStateT,payoffStateA.cashflow,currentNodeIdx);
                             
                    payoffStateA.cashflow = model.Libor_SimpleDate(foreignModelName,valueDate,currentTime,currentTime,numMatTime, tenorShort,modelStateT); 
                    dummy2.ProcessPayoff(model,currentTime,numeraire, ...
                                 modelStateT,payoffStateA.cashflow,currentNodeIdx);
                    
                end
                

%                 payoffStateA.cashflow = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                
%                 payoffStateA.cashflow = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                
                
                                 
                  

                lastTimeIdx = currentTimeIdx;
                totalTimeSteps = numMethodInfo.timeStepsInfo.totalTimeSteps;
                
                % flag for inclusive schedule
%                 simpleYN = false;
%                 simpleYN = true;
                if pricer.overHedgeYN == true 
                    simpleYN = 2;
                else
                    simpleYN = 1;
                end
                
                for i=periodCount:-1:1
                    % past schedule neglect
                    if DateDiff(valueDate,schedule.payDates(i)) >= 0, continue;end 
                    startTimeIdx = numMethodInfo.timeStepsInfo.startTimeIdx(i);
                    endTimeIdx = numMethodInfo.timeStepsInfo.endTimeIdx(i);

                    inclusiveYN = numMethodInfo.timeStepsInfo.inclusiveYN(i);
                 %% FromToInduction start

                    %modelsStates induct backward(lastTimeIdx to endTimeIdx)
                    while endTimeIdx < lastTimeIdx
                        lastTimeIdx = lastTimeIdx -1;
                        if lastTimeIdx == endTimeIdx break;end
                    end

                    currentTimeIdx = lastTimeIdx;
                    currentTime = totalTimeSteps(currentTimeIdx);
                    % now we are at endTimeIdx
                    % variables for coupon valuation initialized
                    

                    cD = 0; % cumulated day count fraction
                    cP = 0; 
                    dI = 0; % dummy idx for loop
                    dcfC = 0;
                    dcfN = 0;
                    dD = 0;
                    idxL = zeros(1,pathSize);
                    idxS = zeros(1,pathSize);
                    spreadC = zeros(1,pathSize);
                    spreadN = zeros(1,pathSize);
                    probSpread = zeros(1,pathSize);
                    
                    numOfTimeSteps = numMethodInfo.timeStepsInfo.numOfTimeSteps(i);
                    
                    % if inclusive then we use simpleYN rangeProb
                    %if inclusive then we do not evaluate the last step but
                    % use fixing instead 
                    endTimeStep = 1;
                    if inclusiveYN
%                         simpleYN = true;
                        simpleYN = 1;
                        endTimeStep = 2;
                        if pricer.overHedgeYN == true 
                            simpleYN = 2;
                        end
                    end
                    
                    for j=numOfTimeSteps:-1:endTimeStep
                        % evaluate dealdescription
                        % last step is omitted
                        currentTime = totalTimeSteps(currentTimeIdx);
                        %dcfC = currentTime/365.0;
                        dcfC = currentTime;
                        
                        if j~= numOfTimeSteps
                            dD = (dcfN-dcfC);
                        end

                        modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:numOfFactors*(currentTimeIdx-1)+numOfFactors,:);
                        
                        if strcmp(pricerOption,'PSA')
                            idxL = model.CMS_PSA_SimpleDate(model.refModelName,valueDate,currentTime,numMatTime, tenorLong,modelStateT);
                            idxS = model.CMS_PSA_SimpleDate(foreignModelName,valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                        else
                            idxL = model.Libor_SimpleDate(model.refModelName,valueDate,currentTime,currentTime,numMatTime, tenorLong,modelStateT);
                            idxS = model.Libor_SimpleDate(foreignModelName,valueDate,currentTime,currentTime,numMatTime, tenorShort,modelStateT);
                        end
                        spreadC = idxL - idxS;

                        probSpread = model.SRPr(spreadC,spreadN,ralower,raupper,simpleYN,pathSize)*dD;
                        cD = cD + dD;
                        cP = cP + probSpread;

                        dcfN = dcfC;
                        spreadN = spreadC;

                        %induct backward to the next time step
                        % one time backward induction
                        lastTimeIdx = currentTimeIdx;

                        if startTimeIdx < lastTimeIdx
                            lastTimeIdx = lastTimeIdx -1;
                            currentTimeIdx = lastTimeIdx;
                        end


                    end

                    %% FromToInduction end
        % if the current period is inclusive, then the start date is not evaluated in the above for loop.
		% instead the fixing will be treated separately below.
		% however current_time_idx is updated at the end of the for loop.
		% so we need to update model_states accordingly.
		% if model_states were updated previously then the following additionaly step doesn't affect the result anyway.
                    currentTime = totalTimeSteps(currentTimeIdx);
                    modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:numOfFactors*(currentTimeIdx-1)+numOfFactors,:);
                    
                    %% AtDate evaluation
                    % Adding a new cashflow to the payoff

                    dcf = schedule.dayCountFraction(i);
                    
                    if inclusiveYN
                        % we add valuation date
                        cD = schedule.dayCount(i);
                        rangeDays =  security.rangeDays(i);
                        payoffStateA.cashflow = nominal * coupon * dcf * (rangeDays + cP)/cD;
                    else
                        payoffStateA.cashflow = nominal * coupon * dcf * cP/cD;
                    end
                    % eventDate(i) = startDate(i) therefore there is no induction in modelStates
                    % i.e. currentTimeIdx has not changed
                    % no need to update modelStateT
                    % modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:end,:);
                    % since cashflow's are added at the startDate(i) we need to
                    % multiply cashflow with the df(startDate(i), payDate(i))
                    payTime = DateDiff(schedule.payDates(i),valueDate);
                    
                    %event is at startDate so we need to discount the
                    %cashflow first from paydate to startdate
                    df_ep = model.discountFactorRisky(currentTime, payTime,numMatTime,modelStateT);

                    for k=1:pathSize
                        payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                    end
                    
                    % process payoff
                    
                    currentNodeIdx = i;
                    
                    nonCall.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                
                    callable.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                    
                    %df column
                    payoffStateA.cashflow = ones(1,pathSize);
                    for k=1:pathSize
                        payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                    end
                    
                    dfCol.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                                 
                    %dummy column
                    if strcmp(pricerOption,'PSA')
                        payoffStateA.cashflow = model.CMS_PSA_SimpleDate(model.refModelName,valueDate,currentTime,numMatTime, tenorLong,modelStateT);
                        dummy1.ProcessPayoff(model,currentTime,numeraire, ...
                                         modelStateT,payoffStateA.cashflow,currentNodeIdx);



                        payoffStateA.cashflow = model.CMS_PSA_SimpleDate(foreignModelName,valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                        dummy2.ProcessPayoff(model,currentTime,numeraire, ...
                                         modelStateT,payoffStateA.cashflow,currentNodeIdx);
                    
                    else
                        payoffStateA.cashflow = model.Libor_SimpleDate(model.refModelName,valueDate,currentTime,currentTime,numMatTime, tenorLong,modelStateT); 
                        dummy1.ProcessPayoff(model,currentTime,numeraire, ...
                                 modelStateT,payoffStateA.cashflow,currentNodeIdx);
                        
                        payoffStateA.cashflow = model.Libor_SimpleDate(foreignModelName,valueDate,currentTime,currentTime,numMatTime, tenorShort,modelStateT); 
                        dummy2.ProcessPayoff(model,currentTime,numeraire, ...
                                 modelStateT,payoffStateA.cashflow,currentNodeIdx);
                        
                    end

                                 
                    
                    
                    
                    % if exercise is possible
                    if i == 1
                        dummaaa = 1;
                    end
                    
                    % call event happens  at the start of the period
                    % at call date, by default, call option is expired
                    
                    if callYN(i) && (DateDiff(valueDate,callSchedule.resetDates(i)) < 0)
                        % 1.undiscount callable.payoff
                        % 2.build the state variables matrix
                        % 3.regress 1's with the given state variables
                        % (model states 1,2)
                        % 4. exercise per each path in
                        % discount the new payoff and assign into callable
                        % column
                        
                        %1 undiscount callable.payoff and get the
                        %contiValue
                       
                        %exercise value
                        exerciseValue = security.nominal*ones(1,pathSize);
                        payTime = DateDiff(callSchedule.payDates(i),valueDate);
                        %event is at startDate so we need to discount the
                        %cashflow first from paydate to startdate
                        df_ep = model.discountFactorRisky(currentTime, payTime,numMatTime,modelStateT);
                        for k=1:pathSize
                            exerciseValue(k) = exerciseValue(k) * df_ep(k);
                        end
                        
                        contiValue = zeros(1,pathSize);
                        onePayoff = ones(1,pathSize);
                        onePayoffD = model.discountPayoffRisky(currentTime,numMatTime ...
                        ,modelStateT,onePayoff);
                        contiValue = callable.payoff./onePayoffD;
                        
                        %Nominal Regression test
                        contiValue = contiValue/10000;
                        
                        % build the state variabls matrix
                        if numMethod.regressionOrder == 3
%                             dimensionLS = (numOfFactors + 1)*(numOfFactors + 2)/2;

                            decisionVar = zeros(numOfFactors,size(modelStateT,2));

                            for idxLS =1:numOfFactors
                                decisionVar(idxLS,:) = modelStateT(idxLS,:); 
                            end
                            
                            % calculate number of dimension first
                            % 0th order
                            dimensionLS = 1;
                            
                            % 1st order
                            for idxLS=1:numOfFactors
                                dimensionLS = dimensionLS + 1;
                            end
                            
                            %2nd order
                            for idxLS1=1:numOfFactors
                                for idxLS2=idxLS1:numOfFactors
                                    dimensionLS = dimensionLS + 1;
                                end
                            end
                            
                            %3rd order
                            for idxLS1=1:numOfFactors
                                for idxLS2=idxLS1:numOfFactors
                                    for idxLS3=idxLS2:numOfFactors
                                        dimensionLS = dimensionLS + 1;
                                    end
                                end
                            end

                            
    %                         decisionVar1 = modelStateT(1,:);
    %                         decisionVar2 = modelStateT(2,:);
                            % we use upto 2nd order polynomial as regression
                            % equatioin

                            regMatrix = ones(pathSize,dimensionLS);

                            for idx_r=1:pathSize
                                for idxLS=1:numOfFactors
                                    regMatrix(idx_r,idxLS+1) = decisionVar(idxLS,idx_r);
                                end

                                tempIdx = 0;
                                for idxLS1=1:numOfFactors
                                    for idxLS2=idxLS1:numOfFactors
                                        tempIdx = tempIdx + 1;
                                        regMatrix(idx_r,numOfFactors+1+tempIdx) = decisionVar(idxLS1,idx_r)*decisionVar(idxLS2,idx_r);
                                    end
                                end
                                
                                tempIdx2 = 0;
                                for idxLS1=1:numOfFactors
                                    for idxLS2=idxLS1:numOfFactors
                                        for idxLS3=idxLS2:numOfFactors
                                            tempIdx2 = tempIdx2 + 1;
                                            regMatrix(idx_r,numOfFactors+1+tempIdx + tempIdx2) = decisionVar(idxLS1,idx_r)*decisionVar(idxLS2,idx_r)*decisionVar(idxLS3,idx_r);
                                        end
                                    end
                                end
                                

                            end
                        
                        else
                            dimensionLS = (numOfFactors + 1)*(numOfFactors + 2)/2;

                            decisionVar = zeros(numOfFactors,size(modelStateT,2));

                            for idxLS =1:numOfFactors
                                decisionVar(idxLS,:) = modelStateT(idxLS,:); 
                            end
    %                         decisionVar1 = modelStateT(1,:);
    %                         decisionVar2 = modelStateT(2,:);
                            % we use upto 2nd order polynomial as regression
                            % equatioin

                            regMatrix = ones(pathSize,dimensionLS);

                            for idx_r=1:pathSize
                                for idxLS=1:numOfFactors
                                    regMatrix(idx_r,idxLS+1) = decisionVar(idxLS,idx_r);
                                end

                                tempIdx = 1;
                                for idxLS1=1:numOfFactors
                                    for idxLS2=idxLS1:numOfFactors
                                        regMatrix(idx_r,numOfFactors+1+tempIdx) = decisionVar(idxLS1,idx_r)*decisionVar(idxLS2,idx_r);
                                        tempIdx = tempIdx + 1;
                                    end
                                end

                            end
                        end
                        % simple regression <- for path dependent option
                        % we should change it
                        % beta  : regression coefficients
                        beta =zeros(dimensionLS,1);
                        beta = regress(contiValue',regMatrix);
                        regContiValue = regMatrix*beta;
                       
                         %Nominal Regression test
                        regContiValue = regContiValue * 10000;
                        
                        exercisedPayoff = regContiValue;
                        % if regContiValue is bigger than exerciseValue
                        % then issue exercise call option
                        for idx_r=1:pathSize
                            if regContiValue(idx_r) >= exerciseValue(idx_r) 
                                exercisedPayoff(idx_r) = exerciseValue(idx_r);
                            end
                        end
                        
                        % we discount the new payoff and 
                        % assign it into the existing callable payoff
                        callable.payoff = model.discountPayoffRisky(currentTime,numMatTime ...
                        ,modelStateT,exercisedPayoff);
                        
                    end
                end

                nonCall.npv = mean(nonCall.payoff);
                out.nonCall = nonCall;
                callable.npv = mean(callable.payoff);
                dfCol.npv = mean(dfCol.payoff);
                dummy1.npv = mean(dummy1.npv);
                dummy2.npv = mean(dummy2.npv);
                out.callable = callable;
                out.dfCol = dfCol;
                out.dummy1 = dummy1;
                out.dummy2 = dummy2;
                out.daycountFraction = schedule.dayCountFraction;
                
        end
        
        
        function out = computePriceClosedSimple(pricer,valueDate)
                security = pricer.m_security;
                numMethod = pricer.m_numMethod;
                model = pricer.m_model;
                schedule = security.couponSchedule;
                
                % numMethodTimeSteps Generate
                mcOneTimeStep = numMethod.mcOneTimeStep;
                outInfo = numMethod.numMethodTimeStepsGenerateDW(valueDate,schedule,mcOneTimeStep);
            
                numMethodInfo.timeStepsInfo = outInfo;

                % mc random number init
                %pathSize = 2^16;
                pathSize = numMethod.pathSize;
                
                pathSize = 1;
                
%                 % generate random numbers and modelstates forward
%                 rng('default');
%                 rng(0);
% 
                numOfFactors = model.numOfFactor;
% 
%                 %Z= randn(numOfFactors*outInfo.totalTimeStepsSize,pathSize/2);
%                 Z= randn(pathSize/2,numOfFactors*outInfo.totalTimeStepsSize);
%                 Z=[Z;-Z];
%                 Z=Z';
%                 
%                 
%                 %TimeInversion
%                 sizeZ = size(Z,1);
%                 A = zeros(sizeZ,sizeZ);
%                 for i=1:sizeZ
%                     A(i,sizeZ + 1 -i) = 1;
%                 end
%                 Z = A*Z;
% 
%                 % we check with the c, so we use generated random numbers
% %                 mc_random = importdata('mc_random.xlsx');
% %                 mc_random = importdata('D:\mc_random_number_new_report.txt');
% %                 Z= mc_random;
%                 
                numMethodInfo.pathSize = pathSize;
                numMethodInfo.numOfFactors = numOfFactors;
                numMethodInfo.totalTimeSteps = outInfo.totalTimeSteps;
                numMethodInfo.totalTimeStepsSize = outInfo.totalTimeStepsSize;
%                 numMethodInfo.Z = Z;
% 
% 
% 
%                 %generate model states for the given timeSteps
%                 modelStates = numMethod.generateModelStates(valueDate,schedule,numMethodInfo,model);

                % numeraire info
                numMatTime = numMethodInfo.totalTimeSteps(end);
                dfNumMat = model.DFRisky(numMatTime/365.0);
                
                % induct backward and calculate cashflow
            
                periodCount = schedule.numOfPeriod;
                
                % cashflow columns initialize
                % 1st nonCall
                nonCall.npv = 0;
                nonCall.cashflow_e = zeros(periodCount + 1,1);
                nonCall.cashflow_npv_e = zeros(periodCount + 1,1);
                nonCall.rangeProb_e = zeros(periodCount + 1,1);
                nonCall.df_e = zeros(periodCount + 1,1);
                nonCall.df_c = zeros(periodCount + 1,1);

                % debug information, cashflow & npv per path
                nonCall.payoff = zeros(1,pathSize);
                nonCall.payoffStates.cashflow = zeros(periodCount+1,pathSize);
                nonCall.payoffStates.cashflow_npv = zeros(periodCount+1,pathSize);
                nonCall.payoffStates.df = zeros(periodCount + 1,pathSize);
                
                % intermediate variables
                payoffStateA.cashflow = zeros(1, pathSize);
                payoffStateA.cashflow_npv = zeros(1, pathSize);
                payoffStateA.df = zeros(1,pathSize);
                
                % we calculate cashflow backward from maturityDate to valueDate
                % rangeAccrual coupons are added at the startDate of the Period
                % At maturity

                nominal = security.nominal;
                coupon = security.coupon;
                callYN = security.callYN;
                
                currentTime = numMethodInfo.totalTimeSteps(end);
                totalTimeSteps = numMethodInfo.timeStepsInfo.totalTimeSteps;
                totalTimeStepsSize = numMethodInfo.timeStepsInfo.totalTimeStepsSize;
                if totalTimeStepsSize == 0 || (totalTimeStepsSize == 1 && totalTimeSteps(1) == 0) 
                    out.nonCall = nonCall;
                    return;
                end
                
                currentTimeIdx = totalTimeStepsSize;
                
                currentNodeIdx = periodCount + 1;
                
%                 modelStateT = modelStates(numOfFactors*(totalTimeStepsSize-1)+1:numOfFactors*(totalTimeStepsSize-1)+2,:);
                payoffStateA.cashflow = nominal*ones(1,pathSize);
                %process payoff at maturity
                nonCall.payoffStates.cashflow(currentNodeIdx,:) = payoffStateA.cashflow;
                
%                 payoffStateA.cashflow_npv = model.discountPayoff(currentTime,numMatTime ...
%                     ,modelStateT,payoffStateA.cashflow);
%                 
                payoffStateA.cashflow_npv = model.DFRisky(currentTime/365.0)*payoffStateA.cashflow;
                
                nonCall.payoffStates.cashflow_npv(currentNodeIdx,:) = payoffStateA.cashflow_npv ;
                
%                 payoffStateA.df = model.discountPayoff(currentTime,numMatTime ...
%                     ,modelStateT,1.0*ones(1,pathSize));
                payoffStateA.df = model.DFRisky(currentTime/365.0) *ones(1,pathSize);
                    

                nonCall.payoffStates.df(currentNodeIdx,:) = payoffStateA.df;
                nonCall.payoff = nonCall.payoff + payoffStateA.cashflow_npv;
                
                nonCall.cashflow_e(currentNodeIdx) = mean(payoffStateA.cashflow);
                nonCall.cashflow_npv_e(currentNodeIdx) = mean(payoffStateA.cashflow_npv);
                nonCall.df_e(currentNodeIdx) = mean(payoffStateA.df);
                nonCall.df_c(currentNodeIdx) = model.DFRisky(currentTime/365.0);
                
%                 % callable at maturity is the same as noncallable
%                 callable.payoffStates.cashflow(currentNodeIdx,:) = nonCall.payoffStates.cashflow(currentNodeIdx,:);
%                 callable.payoffStates.cashflow_npv(currentNodeIdx,:) = nonCall.payoffStates.cashflow_npv(currentNodeIdx,:);
%                 callable.payoff = callable.payoff + callable.payoffStates.cashflow_npv(currentNodeIdx,:);
%                 
%                 callable.cashflow_e(currentNodeIdx) = nonCall.cashflow_e(currentNodeIdx);
%                 callable.cashflow_npv_e(currentNodeIdx) = nonCall.cashflow_npv_e(currentNodeIdx);
                  
                lastTimeIdx = currentTimeIdx;
                
                
                % flag for inclusive schedule
%                 simpleYN = false;
                simpleYN = 1;
                
                for i=periodCount:-1:1
                    % past schedule neglect
                    if DateDiff(valueDate,schedule.payDates(i)) >= 0, continue;end 
                    startTimeIdx = numMethodInfo.timeStepsInfo.startTimeIdx(i);
                    endTimeIdx = numMethodInfo.timeStepsInfo.endTimeIdx(i);

                    inclusiveYN = numMethodInfo.timeStepsInfo.inclusiveYN(i);
                     %% FromToInduction start

                    %modelsStates induct backward(lastTimeIdx to endTimeIdx)
                    while endTimeIdx < lastTimeIdx
                        lastTimeIdx = lastTimeIdx -1;
                        if lastTimeIdx == endTimeIdx break;end
                    end

                    currentTimeIdx = lastTimeIdx;
                    currentTime = totalTimeSteps(currentTimeIdx);
                    % now we are at endTimeIdx
                    % variables for coupon valuation initialized
                    ralower = security.lower;
                    raupper = security.upper;
                    tenorLong = security.tenorL;
                    tenorShort = security.tenorS;

                    cD = 0; % cumulated day count fraction
                    cP = 0; 
                    dI = 0; % dummy idx for loop
                    dcfC = 0;
                    dcfN = 0;
                    dD = 0;
                    idxL = zeros(1,pathSize);
                    idxS = zeros(1,pathSize);
                    spreadC = zeros(1,pathSize);
                    spreadN = zeros(1,pathSize);
                    probSpread = zeros(1,pathSize);
                    
                    numOfTimeSteps = numMethodInfo.timeStepsInfo.numOfTimeSteps(i);
                    
                    % if inclusive then we use simpleYN rangeProb
                    %if inclusive then we do not evaluate the last step but
                    % use fixing instead 
                    endTimeStep = 1;
                    if inclusiveYN
%                         simpleYN = true;
                        simpleYN = 1;
                        endTimeStep = 2;
                    end
                    
                    for j=numOfTimeSteps:-1:endTimeStep
                        % evaluate dealdescription
                        % last step is omitted
                        currentTime = totalTimeSteps(currentTimeIdx);
                        %dcfC = currentTime/365.0;
                        dcfC = currentTime;
                        
                        if j~= numOfTimeSteps
                            dD = (dcfN-dcfC);
                        end

                        toDate = AddDate(valueDate,currentTime,'day');
                        tpDate = schedule.payDates(i);
                        spreadC = model.CMSCMTSpreadDigitalRangeDate(valueDate,toDate,tpDate,tenorLong,tenorShort,ralower,raupper);
                        
                        %modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:numOfFactors*(currentTimeIdx-1)+2,:);
%                         idxL = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorLong,modelStateT);
%                         idxS = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorShort,modelStateT);
%                         spreadC = idxL - idxS;
%                         probSpread = model.SRPr(spreadC,spreadN,ralower,raupper,simpleYN,pathSize)*dD; 

                        if simpleYN
                            probSpread = spreadC*dD;
                        else
                            probSpread = spreadC*dD;
%                             probSpread = 0.5*(spreadC + spreadN)*dD;
                            
                        end

                        
                        cD = cD + dD;
                        cP = cP + probSpread;

                        dcfN = dcfC;
                        spreadN = spreadC;

                        %induct backward to the next time step
                        % one time backward induction
                        lastTimeIdx = currentTimeIdx;

                        if startTimeIdx < lastTimeIdx
                            lastTimeIdx = lastTimeIdx -1;
                            currentTimeIdx = lastTimeIdx;
                        end


                    end

                    %% FromToInduction end
                    %% AtDate evaluation
                    % Adding a new cashflow to the payoff

                    dcf = schedule.dayCountFraction(i);
                    
                    if inclusiveYN
                        % we add valuation date
                        cD = schedule.dayCount(i);
                        rangeDays =  security.rangeDays(i);
                        payoffStateA.cashflow = nominal * coupon * dcf * (rangeDays + cP)/cD;
                    else
                        payoffStateA.cashflow = nominal * coupon * dcf * cP/cD;
                    end
                    % eventDate(i) = startDate(i) therefore there is no induction in modelStates
                    % i.e. currentTimeIdx has not changed
                    % no need to update modelStateT
                    % modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:end,:);
                    % since cashflow's are added at the startDate(i) we need to
                    % multiply cashflow with the df(startDate(i), payDate(i))
                    payTime = DateDiff(schedule.payDates(i),valueDate);
                    
                    %event is at startDate so we need to discount the
                    %cashflow first from paydate to startdate
%                     df_sp = model.discountFactor(currentTime, payTime,numMatTime,modelStateT);

%                     for k=1:pathSize
%                         payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_sp(k);
%                     end
                    
                    % process payoff
                    
                    currentNodeIdx = i;
                    
                    % 1st nonCall column
                    nonCall.payoffStates.cashflow(currentNodeIdx,:) = payoffStateA.cashflow;
                    payoffStateA.cashflow_npv = payoffStateA.cashflow * model.DFRisky(payTime/365.0);
                    
%                     payoffStateA.cashflow_npv = model.discountPayoff(currentTime,numMatTime ...
%                     ,modelStateT,payoffStateA.cashflow);
                    nonCall.payoffStates.cashflow_npv(currentNodeIdx,:) = payoffStateA.cashflow_npv;
                    payoffStateA.df = model.DFRisky(payTime/365.0);
%                     payoffStateA.df = model.discountPayoff(currentTime,numMatTime ...
%                     ,modelStateT,1.0*ones(1,pathSize));
                    nonCall.payoffStates.df(currentNodeIdx,:) = payoffStateA.df;
                    
                    nonCall.payoff = nonCall.payoff + payoffStateA.cashflow_npv;
                    
                    nonCall.cashflow_e(currentNodeIdx) = mean(payoffStateA.cashflow);
                    nonCall.rangeProb_e(currentNodeIdx) = mean(nonCall.cashflow_e(currentNodeIdx)/(nominal * coupon * dcf));
                    nonCall.cashflow_npv_e(currentNodeIdx) = mean(payoffStateA.cashflow_npv);

                    nonCall.df_e(currentNodeIdx) = mean(payoffStateA.df);
                    nonCall.df_c(currentNodeIdx) = model.DFRisky(currentTime/365.0);
                    

                end

                nonCall.npv = mean(nonCall.payoff);
                out.pricerInfo.nonCall = nonCall;
                
%                 callable.npv = mean(callable.payoff);
%                 out.pricerInfo.callable = callable;
                out.daycountFraction = schedule.dayCountFraction;
                
        end
        

        
        
        
        
    end
    
end



In [ ]:
classdef DailyWeeklyRiskyPricer_CDUSDLibor3MSpreadRA < H_RootObject
    %Pricer(Security,Model,NumMethod)
    %Prcer is constructed from Security, Model, NumMethod 
    
    properties
        m_security
        m_model
        m_numMethod
        overHedgeYN
    end
    
    methods
        function pricer = DailyWeeklyRiskyPricer_CDUSDLibor3MSpreadRA(security,model,numMethod,overHedgeYN)
                pricer.m_security = security;
                pricer.m_model = model;
                pricer.m_numMethod = numMethod;
                pricer.overHedgeYN = overHedgeYN;
        end
        
        function out = computePriceMCCallableGeneric(pricer,valueDate,pricerOption)
                security = pricer.m_security;
                numMethod = pricer.m_numMethod;
                model = pricer.m_model;
                schedule = security.couponSchedule;
                callSchedule = security.callSchedule;
                
                % numMethodTimeSteps Generate
                mcOneTimeStep = numMethod.mcOneTimeStep;
                
                % outInfo = numMethod.numMethodTimeStepsGenerateDW(valueDate,schedule,mcOneTimeStep);
                
                % generate schedule using coupon schedule, floating schedule and call schedule
                floatingSchedule = struct;
                outInfo = numMethod.numMethodTimeStepsGenerateDWSwapSimpleCall(valueDate,schedule,floatingSchedule,...
                                callSchedule,mcOneTimeStep);
                
                numMethodInfo.timeStepsInfo = outInfo;

                % mc random number init
                %pathSize = 2^16;
                pathSize = numMethod.pathSize;

                % generate random numbers and modelstates forward
                rng('default');
                rng(0);

                % LGM1F : one factor
                % LGM2F : two factor
                
                numOfFactors = model.numOfFactor;

                %Z= randn(numOfFactors*outInfo.totalTimeStepsSize,pathSize/2);
                Z= randn(pathSize/2,numOfFactors*outInfo.totalTimeStepsSize);
                Z=[Z;-Z];
                Z=Z';
                
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;

                % we check with the c code, so we use generated random numbers
%                 mc_random = importdata('mc_random.xlsx');
%                 mc_random = importdata('D:\mc_random_number_new_report.txt');
%                 Z= mc_random;
                
                numMethodInfo.pathSize = pathSize;
                numMethodInfo.numOfFactors = numOfFactors;
                numMethodInfo.totalTimeSteps = outInfo.totalTimeSteps;
                numMethodInfo.totalTimeStepsSize = outInfo.totalTimeStepsSize;
                numMethodInfo.Z = Z;



                %numMethod generate model states for the given timeSteps
                modelStates = numMethod.generateModelStatesNF(valueDate,schedule,numMethodInfo,model);

                % numeraire info
                numMatTime = numMethodInfo.totalTimeSteps(end);
                days = DateDiff(schedule.payDates(schedule.numOfPeriod),valueDate);
                numMatTime = max(numMatTime, days);
                dfNumMat = model.DFRisky(numMatTime/365.0);
                
                numeraire.numMatTime = numMatTime;
                numeraire.dfNumMat = dfNumMat;
                
                % induct backward and calculate cashflow
            
                periodCount = schedule.numOfPeriod;
                
                % pricer single info initialize
                
                nonCall = PricerInfo(periodCount +1, pathSize);
                callable = PricerInfo(periodCount +1, pathSize);
                dfCol = PricerInfo(periodCount +1, pathSize);
                
                dummy1 = PricerInfo(periodCount +1, pathSize);
                dummy2 = PricerInfo(periodCount +1, pathSize);
                
                
                % intermediate variables
                payoffStateA.cashflow = zeros(1, pathSize);
                payoffStateA.cashflow_npv = zeros(1, pathSize);
                
            
                % we calculate cashflow backward from maturityDate to valueDate
                % rangeAccrual coupons are added at the startDate of the Period
                % At maturity

                nominal = security.nominal;
                coupon = security.coupon;
                callYN = security.callYN;
                
                ralower = security.lower;
                raupper = security.upper;
                tenorLong = security.tenorL;
                tenorShort = security.tenorS;
                
                currentTime = numMethodInfo.totalTimeSteps(end);
                totalTimeStepsSize = numMethodInfo.totalTimeStepsSize;
                currentTimeIdx = totalTimeStepsSize;
                
                currentNodeIdx = periodCount + 1;
                
                modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:numOfFactors*(currentTimeIdx-1)+numOfFactors,:);
                
                payoffStateA.cashflow = nominal*ones(1,pathSize);
                
                payTime = DateDiff(schedule.payDates(schedule.numOfPeriod),valueDate);
                df_ep = model.discountFactorRisky(currentTime, payTime,numMatTime,modelStateT);
                for k=1:pathSize
                    payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                end
                
                %process payoff at maturity
                nonCall.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                
                callable.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                
                %df column
                payoffStateA.cashflow = ones(1,pathSize);
                for k=1:pathSize
                    payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                end
                
                dfCol.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                                 
                %dummy column
                foreignModelName = model.foreignModelName;
                
                if strcmp(pricerOption,'PSA')
                    payoffStateA.cashflow = model.CMS_PSA_SimpleDate(model.refModelName,valueDate,currentTime,numMatTime, tenorLong,modelStateT);
                    dummy1.ProcessPayoff(model,currentTime,numeraire, ...
                                 modelStateT,payoffStateA.cashflow,currentNodeIdx);
                    
                    payoffStateA.cashflow = model.CMS_PSA_SimpleDate(foreignModelName,valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                    dummy2.ProcessPayoff(model,currentTime,numeraire, ...
                                 modelStateT,payoffStateA.cashflow,currentNodeIdx);
                
                else
                    payoffStateA.cashflow = model.Libor_SimpleDate(model.refModelName,valueDate,currentTime,currentTime,numMatTime, tenorLong,modelStateT); 
                    dummy1.ProcessPayoff(model,currentTime,numeraire, ...
                                 modelStateT,payoffStateA.cashflow,currentNodeIdx);
                             
                    payoffStateA.cashflow = model.Libor_SimpleDate(foreignModelName,valueDate,currentTime,currentTime,numMatTime, tenorShort,modelStateT); 
                    dummy2.ProcessPayoff(model,currentTime,numeraire, ...
                                 modelStateT,payoffStateA.cashflow,currentNodeIdx);
                    
                end
                

%                 payoffStateA.cashflow = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                
%                 payoffStateA.cashflow = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                
                
                                 
                  

                lastTimeIdx = currentTimeIdx;
                totalTimeSteps = numMethodInfo.timeStepsInfo.totalTimeSteps;
                
                % flag for inclusive schedule
%                 simpleYN = false;
%                 simpleYN = true;
                if pricer.overHedgeYN == true 
                    simpleYN = 2;
                else
                    simpleYN = 1;
                end
                
                for i=periodCount:-1:1
                    % past schedule neglect
                    if DateDiff(valueDate,schedule.payDates(i)) >= 0, continue;end 
                    startTimeIdx = numMethodInfo.timeStepsInfo.startTimeIdx(i);
                    endTimeIdx = numMethodInfo.timeStepsInfo.endTimeIdx(i);

                    inclusiveYN = numMethodInfo.timeStepsInfo.inclusiveYN(i);
                 %% FromToInduction start

                    %modelsStates induct backward(lastTimeIdx to endTimeIdx)
                    while endTimeIdx < lastTimeIdx
                        lastTimeIdx = lastTimeIdx -1;
                        if lastTimeIdx == endTimeIdx break;end
                    end

                    currentTimeIdx = lastTimeIdx;
                    currentTime = totalTimeSteps(currentTimeIdx);
                    % now we are at endTimeIdx
                    % variables for coupon valuation initialized
                    

                    cD = 0; % cumulated day count fraction
                    cP = 0; 
                    dI = 0; % dummy idx for loop
                    dcfC = 0;
                    dcfN = 0;
                    dD = 0;
                    idxL = zeros(1,pathSize);
                    idxS = zeros(1,pathSize);
                    spreadC = zeros(1,pathSize);
                    spreadN = zeros(1,pathSize);
                    probSpread = zeros(1,pathSize);
                    
                    numOfTimeSteps = numMethodInfo.timeStepsInfo.numOfTimeSteps(i);
                    
                    % if inclusive then we use simpleYN rangeProb
                    %if inclusive then we do not evaluate the last step but
                    % use fixing instead 
                    endTimeStep = 1;
                    if inclusiveYN
%                         simpleYN = true;
                        simpleYN = 1;
                        endTimeStep = 2;
                        if pricer.overHedgeYN == true 
                            simpleYN = 2;
                        end
                    end
                    
                    for j=numOfTimeSteps:-1:endTimeStep
                        % evaluate dealdescription
                        % last step is omitted
                        currentTime = totalTimeSteps(currentTimeIdx);
                        %dcfC = currentTime/365.0;
                        dcfC = currentTime;
                        
                        if j~= numOfTimeSteps
                            dD = (dcfN-dcfC);
                        end

                        modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:numOfFactors*(currentTimeIdx-1)+numOfFactors,:);
                        
                        if strcmp(pricerOption,'PSA')
                            idxL = model.CMS_PSA_SimpleDate(model.refModelName,valueDate,currentTime,numMatTime, tenorLong,modelStateT);
                            idxS = model.CMS_PSA_SimpleDate(foreignModelName,valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                        else
                            idxL = model.Libor_SimpleDate(model.refModelName,valueDate,currentTime,currentTime,numMatTime, tenorLong,modelStateT);
                            idxS = model.Libor_SimpleDate(foreignModelName,valueDate,currentTime,currentTime,numMatTime, tenorShort,modelStateT);
                        end
                        spreadC = idxL - idxS;

                        probSpread = model.SRPr(spreadC,spreadN,ralower,raupper,simpleYN,pathSize)*dD;
                        cD = cD + dD;
                        cP = cP + probSpread;

                        dcfN = dcfC;
                        spreadN = spreadC;

                        %induct backward to the next time step
                        % one time backward induction
                        lastTimeIdx = currentTimeIdx;

                        if startTimeIdx < lastTimeIdx
                            lastTimeIdx = lastTimeIdx -1;
                            currentTimeIdx = lastTimeIdx;
                        end


                    end

                    %% FromToInduction end
        % if the current period is inclusive, then the start date is not evaluated in the above for loop.
		% instead the fixing will be treated separately below.
		% however current_time_idx is updated at the end of the for loop.
		% so we need to update model_states accordingly.
		% if model_states were updated previously then the following additionaly step doesn't affect the result anyway.
                    currentTime = totalTimeSteps(currentTimeIdx);
                    modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:numOfFactors*(currentTimeIdx-1)+numOfFactors,:);
                    
                    %% AtDate evaluation
                    % Adding a new cashflow to the payoff

                    dcf = schedule.dayCountFraction(i);
                    
                    if inclusiveYN
                        % we add valuation date
                        cD = schedule.dayCount(i);
                        rangeDays =  security.rangeDays(i);
                        payoffStateA.cashflow = nominal * coupon * dcf * (rangeDays + cP)/cD;
                    else
                        payoffStateA.cashflow = nominal * coupon * dcf * cP/cD;
                    end
                    % eventDate(i) = startDate(i) therefore there is no induction in modelStates
                    % i.e. currentTimeIdx has not changed
                    % no need to update modelStateT
                    % modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:end,:);
                    % since cashflow's are added at the startDate(i) we need to
                    % multiply cashflow with the df(startDate(i), payDate(i))
                    payTime = DateDiff(schedule.payDates(i),valueDate);
                    
                    %event is at startDate so we need to discount the
                    %cashflow first from paydate to startdate
                    df_ep = model.discountFactorRisky(currentTime, payTime,numMatTime,modelStateT);

                    for k=1:pathSize
                        payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                    end
                    
                    % process payoff
                    
                    currentNodeIdx = i;
                    
                    nonCall.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                
                    callable.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                    
                    %df column
                    payoffStateA.cashflow = ones(1,pathSize);
                    for k=1:pathSize
                        payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                    end
                    
                    dfCol.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                                 
                    %dummy column
                    if strcmp(pricerOption,'PSA')
                        payoffStateA.cashflow = model.CMS_PSA_SimpleDate(model.refModelName,valueDate,currentTime,numMatTime, tenorLong,modelStateT);
                        dummy1.ProcessPayoff(model,currentTime,numeraire, ...
                                         modelStateT,payoffStateA.cashflow,currentNodeIdx);



                        payoffStateA.cashflow = model.CMS_PSA_SimpleDate(foreignModelName,valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                        dummy2.ProcessPayoff(model,currentTime,numeraire, ...
                                         modelStateT,payoffStateA.cashflow,currentNodeIdx);
                    
                    else
                        payoffStateA.cashflow = model.Libor_SimpleDate(model.refModelName,valueDate,currentTime,currentTime,numMatTime, tenorLong,modelStateT); 
                        dummy1.ProcessPayoff(model,currentTime,numeraire, ...
                                 modelStateT,payoffStateA.cashflow,currentNodeIdx);
                        
                        payoffStateA.cashflow = model.Libor_SimpleDate(foreignModelName,valueDate,currentTime,currentTime,numMatTime, tenorShort,modelStateT); 
                        dummy2.ProcessPayoff(model,currentTime,numeraire, ...
                                 modelStateT,payoffStateA.cashflow,currentNodeIdx);
                        
                    end

                                 
                    
                    
                    
                    % if exercise is possible
                    if i == 1
                        dummaaa = 1;
                    end
                    
                    % call event happens  at the start of the period
                    % at call date, by default, call option is expired
                    
                    if callYN(i) && (DateDiff(valueDate,callSchedule.resetDates(i)) < 0)
                        % 1.undiscount callable.payoff
                        % 2.build the state variables matrix
                        % 3.regress 1's with the given state variables
                        % (model states 1,2)
                        % 4. exercise per each path in
                        % discount the new payoff and assign into callable
                        % column
                        
                        %1 undiscount callable.payoff and get the
                        %contiValue
                       
                        %exercise value
                        exerciseValue = security.nominal*ones(1,pathSize);
                        payTime = DateDiff(callSchedule.payDates(i),valueDate);
                        %event is at startDate so we need to discount the
                        %cashflow first from paydate to startdate
                        df_ep = model.discountFactorRisky(currentTime, payTime,numMatTime,modelStateT);
                        for k=1:pathSize
                            exerciseValue(k) = exerciseValue(k) * df_ep(k);
                        end
                        
                        contiValue = zeros(1,pathSize);
                        onePayoff = ones(1,pathSize);
                        onePayoffD = model.discountPayoffRisky(currentTime,numMatTime ...
                        ,modelStateT,onePayoff);
                        contiValue = callable.payoff./onePayoffD;
                        
                        %Nominal Regression test
                        contiValue = contiValue/10000;
                        
                        % build the state variabls matrix
                        if numMethod.regressionOrder == 3
%                             dimensionLS = (numOfFactors + 1)*(numOfFactors + 2)/2;

                            decisionVar = zeros(numOfFactors,size(modelStateT,2));

                            for idxLS =1:numOfFactors
                                decisionVar(idxLS,:) = modelStateT(idxLS,:); 
                            end
                            
                            % calculate number of dimension first
                            % 0th order
                            dimensionLS = 1;
                            
                            % 1st order
                            for idxLS=1:numOfFactors
                                dimensionLS = dimensionLS + 1;
                            end
                            
                            %2nd order
                            for idxLS1=1:numOfFactors
                                for idxLS2=idxLS1:numOfFactors
                                    dimensionLS = dimensionLS + 1;
                                end
                            end
                            
                            %3rd order
                            for idxLS1=1:numOfFactors
                                for idxLS2=idxLS1:numOfFactors
                                    for idxLS3=idxLS2:numOfFactors
                                        dimensionLS = dimensionLS + 1;
                                    end
                                end
                            end

                            
    %                         decisionVar1 = modelStateT(1,:);
    %                         decisionVar2 = modelStateT(2,:);
                            % we use upto 2nd order polynomial as regression
                            % equatioin

                            regMatrix = ones(pathSize,dimensionLS);

                            for idx_r=1:pathSize
                                for idxLS=1:numOfFactors
                                    regMatrix(idx_r,idxLS+1) = decisionVar(idxLS,idx_r);
                                end

                                tempIdx = 0;
                                for idxLS1=1:numOfFactors
                                    for idxLS2=idxLS1:numOfFactors
                                        tempIdx = tempIdx + 1;
                                        regMatrix(idx_r,numOfFactors+1+tempIdx) = decisionVar(idxLS1,idx_r)*decisionVar(idxLS2,idx_r);
                                    end
                                end
                                
                                tempIdx2 = 0;
                                for idxLS1=1:numOfFactors
                                    for idxLS2=idxLS1:numOfFactors
                                        for idxLS3=idxLS2:numOfFactors
                                            tempIdx2 = tempIdx2 + 1;
                                            regMatrix(idx_r,numOfFactors+1+tempIdx + tempIdx2) = decisionVar(idxLS1,idx_r)*decisionVar(idxLS2,idx_r)*decisionVar(idxLS3,idx_r);
                                        end
                                    end
                                end
                                

                            end
                        
                        else
                            dimensionLS = (numOfFactors + 1)*(numOfFactors + 2)/2;

                            decisionVar = zeros(numOfFactors,size(modelStateT,2));

                            for idxLS =1:numOfFactors
                                decisionVar(idxLS,:) = modelStateT(idxLS,:); 
                            end
    %                         decisionVar1 = modelStateT(1,:);
    %                         decisionVar2 = modelStateT(2,:);
                            % we use upto 2nd order polynomial as regression
                            % equatioin

                            regMatrix = ones(pathSize,dimensionLS);

                            for idx_r=1:pathSize
                                for idxLS=1:numOfFactors
                                    regMatrix(idx_r,idxLS+1) = decisionVar(idxLS,idx_r);
                                end

                                tempIdx = 1;
                                for idxLS1=1:numOfFactors
                                    for idxLS2=idxLS1:numOfFactors
                                        regMatrix(idx_r,numOfFactors+1+tempIdx) = decisionVar(idxLS1,idx_r)*decisionVar(idxLS2,idx_r);
                                        tempIdx = tempIdx + 1;
                                    end
                                end

                            end
                        end
                        % simple regression <- for path dependent option
                        % we should change it
                        % beta  : regression coefficients
                        beta =zeros(dimensionLS,1);
                        beta = regress(contiValue',regMatrix);
                        regContiValue = regMatrix*beta;
                       
                         %Nominal Regression test
                        regContiValue = regContiValue * 10000;
                        
                        exercisedPayoff = regContiValue;
                        % if regContiValue is bigger than exerciseValue
                        % then issue exercise call option
                        for idx_r=1:pathSize
                            if regContiValue(idx_r) >= exerciseValue(idx_r) 
                                exercisedPayoff(idx_r) = exerciseValue(idx_r);
                            end
                        end
                        
                        % we discount the new payoff and 
                        % assign it into the existing callable payoff
                        callable.payoff = model.discountPayoffRisky(currentTime,numMatTime ...
                        ,modelStateT,exercisedPayoff);
                        
                    end
                end

                nonCall.npv = mean(nonCall.payoff);
                out.nonCall = nonCall;
                callable.npv = mean(callable.payoff);
                dfCol.npv = mean(dfCol.payoff);
                dummy1.npv = mean(dummy1.npv);
                dummy2.npv = mean(dummy2.npv);
                out.callable = callable;
                out.dfCol = dfCol;
                out.dummy1 = dummy1;
                out.dummy2 = dummy2;
                out.daycountFraction = schedule.dayCountFraction;
                
        end
        
        
        function out = computePriceClosedSimple(pricer,valueDate)
                security = pricer.m_security;
                numMethod = pricer.m_numMethod;
                model = pricer.m_model;
                schedule = security.couponSchedule;
                
                % numMethodTimeSteps Generate
                mcOneTimeStep = numMethod.mcOneTimeStep;
                outInfo = numMethod.numMethodTimeStepsGenerateDW(valueDate,schedule,mcOneTimeStep);
            
                numMethodInfo.timeStepsInfo = outInfo;

                % mc random number init
                %pathSize = 2^16;
                pathSize = numMethod.pathSize;
                
                pathSize = 1;
                
%                 % generate random numbers and modelstates forward
%                 rng('default');
%                 rng(0);
% 
                numOfFactors = model.numOfFactor;
% 
%                 %Z= randn(numOfFactors*outInfo.totalTimeStepsSize,pathSize/2);
%                 Z= randn(pathSize/2,numOfFactors*outInfo.totalTimeStepsSize);
%                 Z=[Z;-Z];
%                 Z=Z';
%                 
%                 
%                 %TimeInversion
%                 sizeZ = size(Z,1);
%                 A = zeros(sizeZ,sizeZ);
%                 for i=1:sizeZ
%                     A(i,sizeZ + 1 -i) = 1;
%                 end
%                 Z = A*Z;
% 
%                 % we check with the c, so we use generated random numbers
% %                 mc_random = importdata('mc_random.xlsx');
% %                 mc_random = importdata('D:\mc_random_number_new_report.txt');
% %                 Z= mc_random;
%                 
                numMethodInfo.pathSize = pathSize;
                numMethodInfo.numOfFactors = numOfFactors;
                numMethodInfo.totalTimeSteps = outInfo.totalTimeSteps;
                numMethodInfo.totalTimeStepsSize = outInfo.totalTimeStepsSize;
%                 numMethodInfo.Z = Z;
% 
% 
% 
%                 %generate model states for the given timeSteps
%                 modelStates = numMethod.generateModelStates(valueDate,schedule,numMethodInfo,model);

                % numeraire info
                numMatTime = numMethodInfo.totalTimeSteps(end);
                dfNumMat = model.DFRisky(numMatTime/365.0);
                
                % induct backward and calculate cashflow
            
                periodCount = schedule.numOfPeriod;
                
                % cashflow columns initialize
                % 1st nonCall
                nonCall.npv = 0;
                nonCall.cashflow_e = zeros(periodCount + 1,1);
                nonCall.cashflow_npv_e = zeros(periodCount + 1,1);
                nonCall.rangeProb_e = zeros(periodCount + 1,1);
                nonCall.df_e = zeros(periodCount + 1,1);
                nonCall.df_c = zeros(periodCount + 1,1);

                % debug information, cashflow & npv per path
                nonCall.payoff = zeros(1,pathSize);
                nonCall.payoffStates.cashflow = zeros(periodCount+1,pathSize);
                nonCall.payoffStates.cashflow_npv = zeros(periodCount+1,pathSize);
                nonCall.payoffStates.df = zeros(periodCount + 1,pathSize);
                
                % intermediate variables
                payoffStateA.cashflow = zeros(1, pathSize);
                payoffStateA.cashflow_npv = zeros(1, pathSize);
                payoffStateA.df = zeros(1,pathSize);
                
                % we calculate cashflow backward from maturityDate to valueDate
                % rangeAccrual coupons are added at the startDate of the Period
                % At maturity

                nominal = security.nominal;
                coupon = security.coupon;
                callYN = security.callYN;
                
                currentTime = numMethodInfo.totalTimeSteps(end);
                totalTimeSteps = numMethodInfo.timeStepsInfo.totalTimeSteps;
                totalTimeStepsSize = numMethodInfo.timeStepsInfo.totalTimeStepsSize;
                if totalTimeStepsSize == 0 || (totalTimeStepsSize == 1 && totalTimeSteps(1) == 0) 
                    out.nonCall = nonCall;
                    return;
                end
                
                currentTimeIdx = totalTimeStepsSize;
                
                currentNodeIdx = periodCount + 1;
                
%                 modelStateT = modelStates(numOfFactors*(totalTimeStepsSize-1)+1:numOfFactors*(totalTimeStepsSize-1)+2,:);
                payoffStateA.cashflow = nominal*ones(1,pathSize);
                %process payoff at maturity
                nonCall.payoffStates.cashflow(currentNodeIdx,:) = payoffStateA.cashflow;
                
%                 payoffStateA.cashflow_npv = model.discountPayoff(currentTime,numMatTime ...
%                     ,modelStateT,payoffStateA.cashflow);
%                 
                payoffStateA.cashflow_npv = model.DFRisky(currentTime/365.0)*payoffStateA.cashflow;
                
                nonCall.payoffStates.cashflow_npv(currentNodeIdx,:) = payoffStateA.cashflow_npv ;
                
%                 payoffStateA.df = model.discountPayoff(currentTime,numMatTime ...
%                     ,modelStateT,1.0*ones(1,pathSize));
                payoffStateA.df = model.DFRisky(currentTime/365.0) *ones(1,pathSize);
                    

                nonCall.payoffStates.df(currentNodeIdx,:) = payoffStateA.df;
                nonCall.payoff = nonCall.payoff + payoffStateA.cashflow_npv;
                
                nonCall.cashflow_e(currentNodeIdx) = mean(payoffStateA.cashflow);
                nonCall.cashflow_npv_e(currentNodeIdx) = mean(payoffStateA.cashflow_npv);
                nonCall.df_e(currentNodeIdx) = mean(payoffStateA.df);
                nonCall.df_c(currentNodeIdx) = model.DFRisky(currentTime/365.0);
                
%                 % callable at maturity is the same as noncallable
%                 callable.payoffStates.cashflow(currentNodeIdx,:) = nonCall.payoffStates.cashflow(currentNodeIdx,:);
%                 callable.payoffStates.cashflow_npv(currentNodeIdx,:) = nonCall.payoffStates.cashflow_npv(currentNodeIdx,:);
%                 callable.payoff = callable.payoff + callable.payoffStates.cashflow_npv(currentNodeIdx,:);
%                 
%                 callable.cashflow_e(currentNodeIdx) = nonCall.cashflow_e(currentNodeIdx);
%                 callable.cashflow_npv_e(currentNodeIdx) = nonCall.cashflow_npv_e(currentNodeIdx);
                  
                lastTimeIdx = currentTimeIdx;
                
                
                % flag for inclusive schedule
%                 simpleYN = false;
                simpleYN = 1;
                
                for i=periodCount:-1:1
                    % past schedule neglect
                    if DateDiff(valueDate,schedule.payDates(i)) >= 0, continue;end 
                    startTimeIdx = numMethodInfo.timeStepsInfo.startTimeIdx(i);
                    endTimeIdx = numMethodInfo.timeStepsInfo.endTimeIdx(i);

                    inclusiveYN = numMethodInfo.timeStepsInfo.inclusiveYN(i);
                     %% FromToInduction start

                    %modelsStates induct backward(lastTimeIdx to endTimeIdx)
                    while endTimeIdx < lastTimeIdx
                        lastTimeIdx = lastTimeIdx -1;
                        if lastTimeIdx == endTimeIdx break;end
                    end

                    currentTimeIdx = lastTimeIdx;
                    currentTime = totalTimeSteps(currentTimeIdx);
                    % now we are at endTimeIdx
                    % variables for coupon valuation initialized
                    ralower = security.lower;
                    raupper = security.upper;
                    tenorLong = security.tenorL;
                    tenorShort = security.tenorS;

                    cD = 0; % cumulated day count fraction
                    cP = 0; 
                    dI = 0; % dummy idx for loop
                    dcfC = 0;
                    dcfN = 0;
                    dD = 0;
                    idxL = zeros(1,pathSize);
                    idxS = zeros(1,pathSize);
                    spreadC = zeros(1,pathSize);
                    spreadN = zeros(1,pathSize);
                    probSpread = zeros(1,pathSize);
                    
                    numOfTimeSteps = numMethodInfo.timeStepsInfo.numOfTimeSteps(i);
                    
                    % if inclusive then we use simpleYN rangeProb
                    %if inclusive then we do not evaluate the last step but
                    % use fixing instead 
                    endTimeStep = 1;
                    if inclusiveYN
%                         simpleYN = true;
                        simpleYN = 1;
                        endTimeStep = 2;
                    end
                    
                    for j=numOfTimeSteps:-1:endTimeStep
                        % evaluate dealdescription
                        % last step is omitted
                        currentTime = totalTimeSteps(currentTimeIdx);
                        %dcfC = currentTime/365.0;
                        dcfC = currentTime;
                        
                        if j~= numOfTimeSteps
                            dD = (dcfN-dcfC);
                        end

                        toDate = AddDate(valueDate,currentTime,'day');
                        tpDate = schedule.payDates(i);
                        spreadC = model.CMSCMTSpreadDigitalRangeDate(valueDate,toDate,tpDate,tenorLong,tenorShort,ralower,raupper);
                        
                        %modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:numOfFactors*(currentTimeIdx-1)+2,:);
%                         idxL = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorLong,modelStateT);
%                         idxS = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorShort,modelStateT);
%                         spreadC = idxL - idxS;
%                         probSpread = model.SRPr(spreadC,spreadN,ralower,raupper,simpleYN,pathSize)*dD; 

                        if simpleYN
                            probSpread = spreadC*dD;
                        else
                            probSpread = spreadC*dD;
%                             probSpread = 0.5*(spreadC + spreadN)*dD;
                            
                        end

                        
                        cD = cD + dD;
                        cP = cP + probSpread;

                        dcfN = dcfC;
                        spreadN = spreadC;

                        %induct backward to the next time step
                        % one time backward induction
                        lastTimeIdx = currentTimeIdx;

                        if startTimeIdx < lastTimeIdx
                            lastTimeIdx = lastTimeIdx -1;
                            currentTimeIdx = lastTimeIdx;
                        end


                    end

                    %% FromToInduction end
                    %% AtDate evaluation
                    % Adding a new cashflow to the payoff

                    dcf = schedule.dayCountFraction(i);
                    
                    if inclusiveYN
                        % we add valuation date
                        cD = schedule.dayCount(i);
                        rangeDays =  security.rangeDays(i);
                        payoffStateA.cashflow = nominal * coupon * dcf * (rangeDays + cP)/cD;
                    else
                        payoffStateA.cashflow = nominal * coupon * dcf * cP/cD;
                    end
                    % eventDate(i) = startDate(i) therefore there is no induction in modelStates
                    % i.e. currentTimeIdx has not changed
                    % no need to update modelStateT
                    % modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:end,:);
                    % since cashflow's are added at the startDate(i) we need to
                    % multiply cashflow with the df(startDate(i), payDate(i))
                    payTime = DateDiff(schedule.payDates(i),valueDate);
                    
                    %event is at startDate so we need to discount the
                    %cashflow first from paydate to startdate
%                     df_sp = model.discountFactor(currentTime, payTime,numMatTime,modelStateT);

%                     for k=1:pathSize
%                         payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_sp(k);
%                     end
                    
                    % process payoff
                    
                    currentNodeIdx = i;
                    
                    % 1st nonCall column
                    nonCall.payoffStates.cashflow(currentNodeIdx,:) = payoffStateA.cashflow;
                    payoffStateA.cashflow_npv = payoffStateA.cashflow * model.DFRisky(payTime/365.0);
                    
%                     payoffStateA.cashflow_npv = model.discountPayoff(currentTime,numMatTime ...
%                     ,modelStateT,payoffStateA.cashflow);
                    nonCall.payoffStates.cashflow_npv(currentNodeIdx,:) = payoffStateA.cashflow_npv;
                    payoffStateA.df = model.DFRisky(payTime/365.0);
%                     payoffStateA.df = model.discountPayoff(currentTime,numMatTime ...
%                     ,modelStateT,1.0*ones(1,pathSize));
                    nonCall.payoffStates.df(currentNodeIdx,:) = payoffStateA.df;
                    
                    nonCall.payoff = nonCall.payoff + payoffStateA.cashflow_npv;
                    
                    nonCall.cashflow_e(currentNodeIdx) = mean(payoffStateA.cashflow);
                    nonCall.rangeProb_e(currentNodeIdx) = mean(nonCall.cashflow_e(currentNodeIdx)/(nominal * coupon * dcf));
                    nonCall.cashflow_npv_e(currentNodeIdx) = mean(payoffStateA.cashflow_npv);

                    nonCall.df_e(currentNodeIdx) = mean(payoffStateA.df);
                    nonCall.df_c(currentNodeIdx) = model.DFRisky(currentTime/365.0);
                    

                end

                nonCall.npv = mean(nonCall.payoff);
                out.pricerInfo.nonCall = nonCall;
                
%                 callable.npv = mean(callable.payoff);
%                 out.pricerInfo.callable = callable;
                out.daycountFraction = schedule.dayCountFraction;
                
        end
        

        
        
        
        
    end
    
end



In [ ]:
classdef DailyWeeklyRiskyPricer_CDRASwap < H_RootObject
    %Pricer(Security,Model,NumMethod)
    %Prcer is constructed from Security, Model, NumMethod 
    
    properties
        m_security
        m_model
        m_numMethod
        overHedgeYN
    end
    
    methods
        function pricer = DailyWeeklyRiskyPricer_CDRASwap(security,model,numMethod,overHedgeYN)
                pricer.m_security = security;
                pricer.m_model = model;
                pricer.m_numMethod = numMethod;
                pricer.overHedgeYN = overHedgeYN;
        end
        
        function out = computePriceMCCallableGeneric(pricer,valueDate)
                security = pricer.m_security;
                numMethod = pricer.m_numMethod;
                model = pricer.m_model;
                schedule = security.couponSchedule;
                callSchedule = security.callSchedule;
                
                floatingSchedule = security.floatingSchedule;
                floatingYN = security.floatingYN;
                floatingSpread = security.floatingSpread;
                floatingIndex = security.floatingIndex;
                
                % numMethodTimeSteps Generate
                mcOneTimeStep = numMethod.mcOneTimeStep;
                outInfo = numMethod.numMethodTimeStepsGenerateDW(valueDate,schedule,mcOneTimeStep);
            
                numMethodInfo.timeStepsInfo = outInfo;

                % mc random number init
                %pathSize = 2^16;
                pathSize = numMethod.pathSize;

                % generate random numbers and modelstates forward
                rng('default');
                rng(0);

                % LGM1F : one factor
                % LGM2F : one factor
                
                numOfFactors = model.numOfFactor;

                %Z= randn(numOfFactors*outInfo.totalTimeStepsSize,pathSize/2);
                Z= randn(pathSize/2,numOfFactors*outInfo.totalTimeStepsSize);
                Z=[Z;-Z];
                Z=Z';
                
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;

                % we check with the c code, so we use generated random numbers
%                 mc_random = importdata('mc_random.xlsx');
%                 mc_random = importdata('D:\mc_random_number_new_report.txt');
%                 Z= mc_random;
                
                numMethodInfo.pathSize = pathSize;
                numMethodInfo.numOfFactors = numOfFactors;
                numMethodInfo.totalTimeSteps = outInfo.totalTimeSteps;
                numMethodInfo.totalTimeStepsSize = outInfo.totalTimeStepsSize;
                numMethodInfo.Z = Z;



                %generate model states for the given timeSteps
                modelStates = numMethod.generateModelStatesNF(valueDate,schedule,numMethodInfo,model);

                % numeraire info
                numMatTime = numMethodInfo.totalTimeSteps(end);
                days = DateDiff(schedule.payDates(schedule.numOfPeriod),valueDate);
                numMatTime = max(numMatTime, days);
                dfNumMat = model.DFRisky(numMatTime/365.0);
                
                numeraire.numMatTime = numMatTime;
                numeraire.dfNumMat = dfNumMat;
                
                % induct backward and calculate cashflow
            
                periodCount = schedule.numOfPeriod;
                
                % pricer single info initialize
                
                nonCall = PricerInfo(periodCount +1, pathSize);
                callable = PricerInfo(periodCount +1, pathSize);
                dfCol = PricerInfo(periodCount +1, pathSize);
                
%                 dummy1 = PricerInfo(periodCount +1, pathSize);
%                 dummy2 = PricerInfo(periodCount +1, pathSize);
                
                
                % intermediate variables
                payoffStateA.cashflow = zeros(1, pathSize);
                payoffStateA.cashflow_npv = zeros(1, pathSize);
                
            
                % we calculate cashflow backward from maturityDate to valueDate
                % rangeAccrual coupons are added at the startDate of the Period
                % At maturity

                nominal = security.nominal;
                coupon = security.coupon;
                callYN = security.callYN;
                
                ralower = security.lower;
                raupper = security.upper;
                
                tenorCD = security.tenor;
                
                currentTime = numMethodInfo.totalTimeSteps(end);
                totalTimeStepsSize = numMethodInfo.totalTimeStepsSize;
                currentTimeIdx = totalTimeStepsSize;
                
                currentNodeIdx = periodCount + 1;
                
%                 modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1,:);
                modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:numOfFactors*(currentTimeIdx-1)+numOfFactors,:);
                
                % swap
                
%                 payoffStateA.cashflow = nominal*ones(1,pathSize);
                payoffStateA.cashflow = 0.0*nominal*ones(1,pathSize);
                
                payTime = DateDiff(schedule.payDates(schedule.numOfPeriod),valueDate);
                df_ep = model.discountFactorRisky(currentTime, payTime,numMatTime,modelStateT);
                for k=1:pathSize
                    payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                end
                
                %process payoff at maturity
                nonCall.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                
                callable.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                
                %df column
                payoffStateA.cashflow = ones(1,pathSize);
                for k=1:pathSize
                    payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                end
                
                dfCol.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                                 
%                 %dummy column
%                 payoffStateA.cashflow = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorLong,modelStateT);
%                 dummy1.ProcessPayoff(model,currentTime,numeraire, ...
%                                  modelStateT,payoffStateA.cashflow,currentNodeIdx);
% 
%                 payoffStateA.cashflow = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorShort,modelStateT);
%                 dummy2.ProcessPayoff(model,currentTime,numeraire, ...
%                                  modelStateT,payoffStateA.cashflow,currentNodeIdx);
                                 
                  

                lastTimeIdx = currentTimeIdx;
                totalTimeSteps = numMethodInfo.timeStepsInfo.totalTimeSteps;
                
                % flag for inclusive schedule
%                 simpleYN = false;
%                 simpleYN = true;
                if pricer.overHedgeYN == true 
                    simpleYN = 2;
                else
                    simpleYN = 1;
                end
                
                for i=periodCount:-1:1
                    % past schedule neglect
                    if DateDiff(valueDate,schedule.payDates(i)) >= 0, continue;end 
                    startTimeIdx = numMethodInfo.timeStepsInfo.startTimeIdx(i);
                    endTimeIdx = numMethodInfo.timeStepsInfo.endTimeIdx(i);

                    inclusiveYN = numMethodInfo.timeStepsInfo.inclusiveYN(i);
                 %% FromToInduction start

                    %modelsStates induct backward(lastTimeIdx to endTimeIdx)
                    while endTimeIdx < lastTimeIdx
                        lastTimeIdx = lastTimeIdx -1;
                        if lastTimeIdx == endTimeIdx break;end
                    end

                    currentTimeIdx = lastTimeIdx;
                    currentTime = totalTimeSteps(currentTimeIdx);
                    % now we are at endTimeIdx
                    % variables for coupon valuation initialized
                    

                    cD = 0; % cumulated day count fraction
                    cP = 0; 
                    dI = 0; % dummy idx for loop
                    dcfC = 0;
                    dcfN = 0;
                    dD = 0;
                    idxCD = zeros(1,pathSize);
                    
                    idxC = zeros(1,pathSize);
                    idxN = zeros(1,pathSize);
                
                    probSpread = zeros(1,pathSize);
                    
                    numOfTimeSteps = numMethodInfo.timeStepsInfo.numOfTimeSteps(i);
                    
                    % if inclusive then we use simpleYN rangeProb
                    %if inclusive then we do not evaluate the last step but
                    % use fixing instead 
                    endTimeStep = 1;
                    if inclusiveYN
%                         simpleYN = true;
                        simpleYN = 1;
                        endTimeStep = 2;
                        if pricer.overHedgeYN == true 
                            simpleYN = 2;
                        end
                    end
                    
                    for j=numOfTimeSteps:-1:endTimeStep
                        % evaluate dealdescription
                        % last step is omitted
                        currentTime = totalTimeSteps(currentTimeIdx);
                        %dcfC = currentTime/365.0;
                        dcfC = currentTime;
                        
                        if j~= numOfTimeSteps
                            dD = (dcfN-dcfC);
                        end

%                         modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1,:);
                        modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:numOfFactors*(currentTimeIdx-1)+numOfFactors,:);
                
                        idxCD = model.Libor_SimpleDate(valueDate,currentTime,currentTime,numMatTime, tenorCD,modelStateT);
                        idxC = idxCD;
                        probSpread = model.SRPr(idxC,idxN,ralower,raupper,simpleYN,pathSize)*dD;
                    
                        cD = cD + dD;
                        cP = cP + probSpread;

                        dcfN = dcfC;
                        
                        idxN = idxC;
                        
                        %induct backward to the next time step
                        % one time backward induction
                        lastTimeIdx = currentTimeIdx;

                        if startTimeIdx < lastTimeIdx
                            lastTimeIdx = lastTimeIdx -1;
                            currentTimeIdx = lastTimeIdx;
                        end


                    end

                    %% FromToInduction end
        % if the current period is inclusive, then the start date is not evaluated in the above for loop.
		% instead the fixing will be treated separately below.
		% however current_time_idx is updated at the end of the for loop.
		% so we need to update model_states accordingly.
		% if model_states were updated previously then the following additionaly step doesn't affect the result anyway.
                    currentTime = totalTimeSteps(currentTimeIdx);
%                     modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1,:);
                    modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:numOfFactors*(currentTimeIdx-1)+numOfFactors,:);
                
                    
                    %% AtDate evaluation
                    % Adding a new cashflow to the payoff

                    dcf = schedule.dayCountFraction(i);
                    
                    if inclusiveYN
                        % we add valuation date
                        cD = schedule.dayCount(i);
                        rangeDays =  security.rangeDays(i);
                        payoffStateA.cashflow = nominal * coupon * dcf * (rangeDays + cP)/cD;
                    else
                        payoffStateA.cashflow = nominal * coupon * dcf * cP/cD;
                    end
                    % eventDate(i) = startDate(i) therefore there is no induction in modelStates
                    % i.e. currentTimeIdx has not changed
                    % no need to update modelStateT
                    % modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:end,:);
                    % since cashflow's are added at the startDate(i) we need to
                    % multiply cashflow with the df(startDate(i), payDate(i))
                    payTime = DateDiff(schedule.payDates(i),valueDate);
                    
                    %event is at startDate so we need to discount the
                    %cashflow first from paydate to startdate
                    df_ep = model.discountFactorRisky(currentTime, payTime,numMatTime,modelStateT);

                    for k=1:pathSize
                        payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                    end
                    
                    % process payoff
                    
                    currentNodeIdx = i;
                    
                    nonCall.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                
                    callable.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                    
                    %df column
                    payoffStateA.cashflow = ones(1,pathSize);
                    for k=1:pathSize
                        payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                    end
                    
                    dfCol.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                                 
%                     %dummy column
%                     payoffStateA.cashflow = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorLong,modelStateT);
%                     dummy1.ProcessPayoff(model,currentTime,numeraire, ...
%                                      modelStateT,payoffStateA.cashflow,currentNodeIdx);
%                     
%                     payoffStateA.cashflow = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorShort,modelStateT);
%                     dummy2.ProcessPayoff(model,currentTime,numeraire, ...
%                                      modelStateT,payoffStateA.cashflow,currentNodeIdx);
                    
                    
                    % if exercise is possible
                    if i == 1
                        dummaaa = 1;
                    end
                    
                    % floating leg event happens at the start of the period
                    if floatingYN(i) 
                        dcfFloating = floatingSchedule.dayCountFraction(i);
                        %floatingIndex is not fixed yet
                        if DateDiff(valueDate,floatingSchedule.resetDates(i)) < 0
                            
                            if strcmp(floatingIndex,'Libor3M')
                                tenorFloatingCD = 0.25;
                                idxFloatingCD = model.Libor_SimpleDate(valueDate,currentTime,currentTime,numMatTime, tenorFloatingCD,modelStateT);
                                % -1 for floating pay 
                                payoffStateA.cashflow = -1.0 * nominal * (idxFloatingCD + floatingSpread) * dcfFloating;
                            end
                            
                        else %floatingIndex is fixed
                            if strcmp(floatingIndex,'Libor3M')
                                tenorFloatingCD = 0.25;
                                allFixings = security.allFixings;
                                USDLibor3MFixings = allFixings('USDLibor3M');
                                idxFloatingCD = USDLibor3MFixings(StrDate(floatingSchedule.resetDates(i),'str'));
%                                 idxFloatingCD = model.Libor_SimpleDate(valueDate,currentTime,currentTime,numMatTime, tenorFloatingCD,modelStateT);
                                % -1 for floating pay
                                for k=1:pathSize
                                    payoffStateA.cashflow(k) = -1.0 * nominal * (idxFloatingCD + floatingSpread) * dcfFloating;
                                end
                            end
                            
                        end
                        
                            payTime = DateDiff(floatingSchedule.payDates(i),valueDate);
                            %event is at startDate so we need to discount the
                            %cashflow first from paydate to startdate
                            df_ep = model.discountFactorRisky(currentTime, payTime,numMatTime,modelStateT);
                            
                            for k=1:pathSize
                                payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                            end

                            % process payoff

                            currentNodeIdx = i;

                            nonCall.ProcessAddPayoff(model,currentTime,numeraire, ...
                                             modelStateT,payoffStateA.cashflow,currentNodeIdx);

                            callable.ProcessAddPayoff(model,currentTime,numeraire, ...
                                             modelStateT,payoffStateA.cashflow,currentNodeIdx);
                            
                        
                    end
                    
                    
                    % call event happens  at the start of the period
                    % at call date, by default, call option is expired
                    
                    if callYN(i) && (DateDiff(valueDate,callSchedule.resetDates(i)) < 0)
                        % 1.undiscount callable.payoff
                        % 2.build the state variables matrix
                        % 3.regress 1's with the given state variables
                        % (model states 1,2)
                        % 4. exercise per each path in
                        % discount the new payoff and assign into callable
                        % column
                        
                        %1 undiscount callable.payoff and get the
                        %contiValue
                       
                        %exercise value
%                         exerciseValue = security.nominal*ones(1,pathSize);
                        
                        exerciseValue = security.nominal*security.callStrike*ones(1,pathSize);
                        
                        payTime = DateDiff(callSchedule.payDates(i),valueDate);
                        %event is at startDate so we need to discount the
                        %cashflow first from paydate to startdate
                        df_ep = model.discountFactorRisky(currentTime, payTime,numMatTime,modelStateT);
                        for k=1:pathSize
                            exerciseValue(k) = exerciseValue(k) * df_ep(k);
                        end
                        
                        contiValue = zeros(1,pathSize);
                        onePayoff = ones(1,pathSize);
                        onePayoffD = model.discountPayoffRisky(currentTime,numMatTime ...
                        ,modelStateT,onePayoff);
                        contiValue = callable.payoff./onePayoffD;
                        
                        %Nominal Regression test
                        contiValue = contiValue/10000;
                        
                        % build the state variabls matrix
                        dimensionLS = (numOfFactors + 1)*(numOfFactors + 2)/2;
                        
                        decisionVar = zeros(numOfFactors,size(modelStateT,2));
                        
                        for idxLS =1:numOfFactors
                            decisionVar(idxLS,:) = modelStateT(idxLS,:); 
                        end
%                         decisionVar1 = modelStateT(1,:);
%                         decisionVar2 = modelStateT(2,:);
                        % we use upto 2nd order polynomial as regression
                        % equatioin
                        
                        regMatrix = ones(pathSize,dimensionLS);
                        
                        for idx_r=1:pathSize
                            for idxLS=1:numOfFactors
                                regMatrix(idx_r,idxLS+1) = decisionVar(idxLS,idx_r);
                            end
                            
                            tempIdx = 1;
                            for idxLS1=1:numOfFactors
                                for idxLS2=idxLS1:numOfFactors
                                    regMatrix(idx_r,numOfFactors+1+tempIdx) = decisionVar(idxLS1,idx_r)*decisionVar(idxLS2,idx_r);
                                    tempIdx = tempIdx + 1;
                                end
                            end
                            
                        end
                        
                        % simple regression <- for path dependent option
                        % we should change it
                        % beta  : regression coefficients
                        beta =zeros(dimensionLS,1);
                        beta = regress(contiValue',regMatrix);
                        regContiValue = regMatrix*beta;
                       
                         %Nominal Regression test
                        regContiValue = regContiValue * 10000;
                        
                        exercisedPayoff = regContiValue;
                        % if regContiValue is bigger than exerciseValue
                        % then issue exercise call option
                        for idx_r=1:pathSize
                            if regContiValue(idx_r) >= exerciseValue(idx_r) 
                                exercisedPayoff(idx_r) = exerciseValue(idx_r);
                            end
                        end
                        
                        % we discount the new payoff and 
                        % assign it into the existing callable payoff
                        callable.payoff = model.discountPayoffRisky(currentTime,numMatTime ...
                        ,modelStateT,exercisedPayoff);
                        
                    end
                end

                nonCall.npv = mean(nonCall.payoff);
                out.nonCall = nonCall;
                callable.npv = mean(callable.payoff);
                out.callable = callable;
                out.dfCol = dfCol;
%                 out.dummy1 = dummy1;
%                 out.dummy2 = dummy2;
                out.daycountFraction = schedule.dayCountFraction;
                
        end
        
        function out = computePriceMCCallable(pricer,valueDate)
                security = pricer.m_security;
                numMethod = pricer.m_numMethod;
                model = pricer.m_model;
                schedule = security.couponSchedule;
                callSchedule = security.callSchedule;
                
                floatingSchedule = security.floatingSchedule;
                floatingYN = security.floatingYN;
                floatingSpread = security.floatingSpread;
                floatingIndex = security.floatingIndex;
                
                % numMethodTimeSteps Generate
                mcOneTimeStep = numMethod.mcOneTimeStep;
                outInfo = numMethod.numMethodTimeStepsGenerateDW(valueDate,schedule,mcOneTimeStep);
            
                numMethodInfo.timeStepsInfo = outInfo;

                % mc random number init
                %pathSize = 2^16;
                pathSize = numMethod.pathSize;

                % generate random numbers and modelstates forward
                rng('default');
                rng(0);

                % LGM1F : one factor 
                numOfFactors = 1;

                %Z= randn(numOfFactors*outInfo.totalTimeStepsSize,pathSize/2);
                Z= randn(pathSize/2,numOfFactors*outInfo.totalTimeStepsSize);
                Z=[Z;-Z];
                Z=Z';
                
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;

                % we check with the c code, so we use generated random numbers
%                 mc_random = importdata('mc_random.xlsx');
%                 mc_random = importdata('D:\mc_random_number_new_report.txt');
%                 Z= mc_random;
                
                numMethodInfo.pathSize = pathSize;
                numMethodInfo.numOfFactors = numOfFactors;
                numMethodInfo.totalTimeSteps = outInfo.totalTimeSteps;
                numMethodInfo.totalTimeStepsSize = outInfo.totalTimeStepsSize;
                numMethodInfo.Z = Z;



                %generate model states for the given timeSteps
                modelStates = numMethod.generateModelStates1(valueDate,schedule,numMethodInfo,model);

                % numeraire info
                numMatTime = numMethodInfo.totalTimeSteps(end);
                days = DateDiff(schedule.payDates(schedule.numOfPeriod),valueDate);
                numMatTime = max(numMatTime, days);
                dfNumMat = model.DFRisky(numMatTime/365.0);
                
                numeraire.numMatTime = numMatTime;
                numeraire.dfNumMat = dfNumMat;
                
                % induct backward and calculate cashflow
            
                periodCount = schedule.numOfPeriod;
                
                % pricer single info initialize
                
                nonCall = PricerInfo(periodCount +1, pathSize);
                callable = PricerInfo(periodCount +1, pathSize);
                dfCol = PricerInfo(periodCount +1, pathSize);
                
%                 dummy1 = PricerInfo(periodCount +1, pathSize);
%                 dummy2 = PricerInfo(periodCount +1, pathSize);
                
                
                % intermediate variables
                payoffStateA.cashflow = zeros(1, pathSize);
                payoffStateA.cashflow_npv = zeros(1, pathSize);
                
            
                % we calculate cashflow backward from maturityDate to valueDate
                % rangeAccrual coupons are added at the startDate of the Period
                % At maturity

                nominal = security.nominal;
                coupon = security.coupon;
                callYN = security.callYN;
                
                ralower = security.lower;
                raupper = security.upper;
                
                tenorCD = security.tenor;
                
                currentTime = numMethodInfo.totalTimeSteps(end);
                totalTimeStepsSize = numMethodInfo.totalTimeStepsSize;
                currentTimeIdx = totalTimeStepsSize;
                
                currentNodeIdx = periodCount + 1;
                
                modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1,:);
                
                 % swap
                 
%                 payoffStateA.cashflow = nominal*ones(1,pathSize);
                payoffStateA.cashflow = 0.0*nominal*ones(1,pathSize);
                
                payTime = DateDiff(schedule.payDates(schedule.numOfPeriod),valueDate);
                df_ep = model.discountFactorRisky(currentTime, payTime,numMatTime,modelStateT);
                for k=1:pathSize
                    payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                end
                
                %process payoff at maturity
                nonCall.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                
                callable.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                
                %df column
                payoffStateA.cashflow = ones(1,pathSize);
                for k=1:pathSize
                    payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                end
                
                dfCol.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                                 
%                 %dummy column
%                 payoffStateA.cashflow = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorLong,modelStateT);
%                 dummy1.ProcessPayoff(model,currentTime,numeraire, ...
%                                  modelStateT,payoffStateA.cashflow,currentNodeIdx);
% 
%                 payoffStateA.cashflow = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorShort,modelStateT);
%                 dummy2.ProcessPayoff(model,currentTime,numeraire, ...
%                                  modelStateT,payoffStateA.cashflow,currentNodeIdx);
                                 
                  

                lastTimeIdx = currentTimeIdx;
                totalTimeSteps = numMethodInfo.timeStepsInfo.totalTimeSteps;
                
                % flag for inclusive schedule
%                 simpleYN = false;
%                 simpleYN = true;
                if pricer.overHedgeYN == true 
                    simpleYN = 2;
                else
                    simpleYN = 1;
                end
                
                for i=periodCount:-1:1
                    % past schedule neglect
                    if DateDiff(valueDate,schedule.payDates(i)) >= 0, continue;end 
                    startTimeIdx = numMethodInfo.timeStepsInfo.startTimeIdx(i);
                    endTimeIdx = numMethodInfo.timeStepsInfo.endTimeIdx(i);

                    inclusiveYN = numMethodInfo.timeStepsInfo.inclusiveYN(i);
                 %% FromToInduction start

                    %modelsStates induct backward(lastTimeIdx to endTimeIdx)
                    while endTimeIdx < lastTimeIdx
                        lastTimeIdx = lastTimeIdx -1;
                        if lastTimeIdx == endTimeIdx break;end
                    end

                    currentTimeIdx = lastTimeIdx;
                    currentTime = totalTimeSteps(currentTimeIdx);
                    % now we are at endTimeIdx
                    % variables for coupon valuation initialized
                    

                    cD = 0; % cumulated day count fraction
                    cP = 0; 
                    dI = 0; % dummy idx for loop
                    dcfC = 0;
                    dcfN = 0;
                    dD = 0;
                    idxCD = zeros(1,pathSize);
                    
                    idxC = zeros(1,pathSize);
                    idxN = zeros(1,pathSize);
                
                    probSpread = zeros(1,pathSize);
                    
                    numOfTimeSteps = numMethodInfo.timeStepsInfo.numOfTimeSteps(i);
                    
                    % if inclusive then we use simpleYN rangeProb
                    %if inclusive then we do not evaluate the last step but
                    % use fixing instead 
                    endTimeStep = 1;
                    if inclusiveYN
%                         simpleYN = true;
                        simpleYN = 1;
                        endTimeStep = 2;
                        if pricer.overHedgeYN == true 
                            simpleYN = 2;
                        end
                    end
                    
                    for j=numOfTimeSteps:-1:endTimeStep
                        % evaluate dealdescription
                        % last step is omitted
                        currentTime = totalTimeSteps(currentTimeIdx);
                        %dcfC = currentTime/365.0;
                        dcfC = currentTime;
                        
                        if j~= numOfTimeSteps
                            dD = (dcfN-dcfC);
                        end

                        modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1,:);
                        
                        idxCD = model.Libor_SimpleDate(valueDate,currentTime,currentTime,numMatTime, tenorCD,modelStateT);
                        idxC = idxCD;
                        probSpread = model.SRPr(idxC,idxN,ralower,raupper,simpleYN,pathSize)*dD;
                    
                        cD = cD + dD;
                        cP = cP + probSpread;

                        dcfN = dcfC;
                        
                        idxN = idxC;
                        
                        %induct backward to the next time step
                        % one time backward induction
                        lastTimeIdx = currentTimeIdx;

                        if startTimeIdx < lastTimeIdx
                            lastTimeIdx = lastTimeIdx -1;
                            currentTimeIdx = lastTimeIdx;
                        end


                    end

                    %% FromToInduction end
        % if the current period is inclusive, then the start date is not evaluated in the above for loop.
		% instead the fixing will be treated separately below.
		% however current_time_idx is updated at the end of the for loop.
		% so we need to update model_states accordingly.
		% if model_states were updated previously then the following additionaly step doesn't affect the result anyway.
                    currentTime = totalTimeSteps(currentTimeIdx);
                    modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1,:);
                    
                    %% AtDate evaluation
                    % Adding a new cashflow to the payoff

                    dcf = schedule.dayCountFraction(i);
                    
                    if inclusiveYN
                        % we add valuation date
                        cD = schedule.dayCount(i);
                        rangeDays =  security.rangeDays(i);
                        payoffStateA.cashflow = nominal * coupon * dcf * (rangeDays + cP)/cD;
                    else
                        payoffStateA.cashflow = nominal * coupon * dcf * cP/cD;
                    end
                    % eventDate(i) = startDate(i) therefore there is no induction in modelStates
                    % i.e. currentTimeIdx has not changed
                    % no need to update modelStateT
                    % modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:end,:);
                    % since cashflow's are added at the startDate(i) we need to
                    % multiply cashflow with the df(startDate(i), payDate(i))
                    payTime = DateDiff(schedule.payDates(i),valueDate);
                    
                    %event is at startDate so we need to discount the
                    %cashflow first from paydate to startdate
                    df_ep = model.discountFactorRisky(currentTime, payTime,numMatTime,modelStateT);

                    for k=1:pathSize
                        payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                    end
                    
                    % process payoff
                    
                    currentNodeIdx = i;
                    
                    nonCall.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                
                    callable.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                    
                    %df column
                    payoffStateA.cashflow = ones(1,pathSize);
                    for k=1:pathSize
                        payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                    end
                    
                    dfCol.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                                 
%                     %dummy column
%                     payoffStateA.cashflow = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorLong,modelStateT);
%                     dummy1.ProcessPayoff(model,currentTime,numeraire, ...
%                                      modelStateT,payoffStateA.cashflow,currentNodeIdx);
%                     
%                     payoffStateA.cashflow = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorShort,modelStateT);
%                     dummy2.ProcessPayoff(model,currentTime,numeraire, ...
%                                      modelStateT,payoffStateA.cashflow,currentNodeIdx);
                    
                    
                    % if exercise is possible
                    if i == 1
                        dummaaa = 1;
                    end
                    
                    % floating leg event happens at the start of the period
                    if floatingYN(i) 
                        dcfFloating = floatingSchedule.dayCountFraction(i);
                        %floatingIndex is not fixed yet
                        if DateDiff(valueDate,floatingSchedule.resetDates(i)) < 0
                            
                            if strcmp(floatingIndex,'Libor3M')
                                tenorFloatingCD = 0.25;
                                idxFloatingCD = model.Libor_SimpleDate(valueDate,currentTime,currentTime,numMatTime, tenorFloatingCD,modelStateT);
                                % -1 for floating pay 
                                payoffStateA.cashflow = -1.0 * nominal * (idxFloatingCD + floatingSpread) * dcfFloating;
                            end
                            
                        else %floatingIndex is fixed
                            if strcmp(floatingIndex,'Libor3M')
                                tenorFloatingCD = 0.25;
                                allFixings = security.allFixings;
                                USDLibor3MFixings = allFixings('USDLibor3M');
                                idxFloatingCD = USDLibor3MFixings(StrDate(floatingSchedule.resetDates(i),'str'));
%                                 idxFloatingCD = model.Libor_SimpleDate(valueDate,currentTime,currentTime,numMatTime, tenorFloatingCD,modelStateT);
                                % -1 for floating pay
                                for k=1:pathSize
                                    payoffStateA.cashflow(k) = -1.0 * nominal * (idxFloatingCD + floatingSpread) * dcfFloating;
                                end
                            end
                            
                        end
                        
                            payTime = DateDiff(floatingSchedule.payDates(i),valueDate);
                            %event is at startDate so we need to discount the
                            %cashflow first from paydate to startdate
                            df_ep = model.discountFactorRisky(currentTime, payTime,numMatTime,modelStateT);
                            
                            for k=1:pathSize
                                payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                            end

                            % process payoff

                            currentNodeIdx = i;

                            nonCall.ProcessAddPayoff(model,currentTime,numeraire, ...
                                             modelStateT,payoffStateA.cashflow,currentNodeIdx);

                            callable.ProcessAddPayoff(model,currentTime,numeraire, ...
                                             modelStateT,payoffStateA.cashflow,currentNodeIdx);
                            
                        
                    end
                    
                    
                    
                    % call event happens  at the start of the period
                    % at call date, by default, call option is expired
                    
                    if callYN(i) && (DateDiff(valueDate,callSchedule.resetDates(i)) < 0)
                        % 1.undiscount callable.payoff
                        % 2.build the state variables matrix
                        % 3.regress 1's with the given state variables
                        % (model states 1,2)
                        % 4. exercise per each path in
                        % discount the new payoff and assign into callable
                        % column
                        
                        %1 undiscount callable.payoff and get the
                        %contiValue
                       
                        %exercise value
%                         exerciseValue = security.nominal*ones(1,pathSize);
                        
                        exerciseValue = security.nominal*security.callStrike*ones(1,pathSize);
                        
                        payTime = DateDiff(callSchedule.payDates(i),valueDate);
                        %event is at startDate so we need to discount the
                        %cashflow first from paydate to startdate
                        df_ep = model.discountFactorRisky(currentTime, payTime,numMatTime,modelStateT);
                        for k=1:pathSize
                            exerciseValue(k) = exerciseValue(k) * df_ep(k);
                        end
                        
                        contiValue = zeros(1,pathSize);
                        onePayoff = ones(1,pathSize);
                        onePayoffD = model.discountPayoffRisky(currentTime,numMatTime ...
                        ,modelStateT,onePayoff);
                        contiValue = callable.payoff./onePayoffD;
                        
                        %Nominal Regression test
                        contiValue = contiValue/10000;
                        
                        % build the state variabls matrix
                        decisionVar1 = modelStateT(1,:);
%                         decisionVar2 = modelStateT(2,:);
                        % we use upto 2nd order polynomial as regression
                        % equatioin
                        dimensionLS = 3;
                        regMatrix = ones(pathSize,dimensionLS);
                        
                        for idx_r=1:pathSize
                            regMatrix(idx_r,2) = decisionVar1(idx_r);
                            regMatrix(idx_r,3) = decisionVar1(idx_r)*decisionVar1(idx_r);
                        end
                        
                        % simple regression <- for path dependent option
                        % we should change it
                        % beta  : regression coefficients
                        beta =zeros(dimensionLS,1);
                        beta = regress(contiValue',regMatrix);
                        regContiValue = regMatrix*beta;
                       
                         %Nominal Regression test
                        regContiValue = regContiValue * 10000;
                        
                        exercisedPayoff = regContiValue;
                        % if regContiValue is bigger than exerciseValue
                        % then issue exercise call option
                        for idx_r=1:pathSize
                            if regContiValue(idx_r) >= exerciseValue(idx_r) 
                                exercisedPayoff(idx_r) = exerciseValue(idx_r);
                            end
                        end
                        
                        % we discount the new payoff and 
                        % assign it into the existing callable payoff
                        callable.payoff = model.discountPayoffRisky(currentTime,numMatTime ...
                        ,modelStateT,exercisedPayoff);
                        
                    end
                end

                nonCall.npv = mean(nonCall.payoff);
                out.nonCall = nonCall;
                callable.npv = mean(callable.payoff);
                out.callable = callable;
                out.dfCol = dfCol;
%                 out.dummy1 = dummy1;
%                 out.dummy2 = dummy2;
                out.daycountFraction = schedule.dayCountFraction;
                
        end
        
        function out = computePriceClosedSimple(pricer,valueDate)
                security = pricer.m_security;
                numMethod = pricer.m_numMethod;
                model = pricer.m_model;
                schedule = security.couponSchedule;
                
                %floating
                floatingSchedule = security.floatingSchedule;
                floatingYN = security.floatingYN;
                floatingSpread = security.floatingSpread;
                floatingIndex = security.floatingIndex;
                
                % numMethodTimeSteps Generate
                mcOneTimeStep = numMethod.mcOneTimeStep;
                outInfo = numMethod.numMethodTimeStepsGenerateDW(valueDate,schedule,mcOneTimeStep);
            
                numMethodInfo.timeStepsInfo = outInfo;

                % mc random number init
                %pathSize = 2^16;
                pathSize = numMethod.pathSize;
                
                pathSize = 1;
                
%                 % generate random numbers and modelstates forward
%                 rng('default');
%                 rng(0);
                numOfFactors = model.numOfFactor;
%                 numOfFactors = 1;
% 
%                 %Z= randn(numOfFactors*outInfo.totalTimeStepsSize,pathSize/2);
%                 Z= randn(pathSize/2,numOfFactors*outInfo.totalTimeStepsSize);
%                 Z=[Z;-Z];
%                 Z=Z';
%                 
%                 
%                 %TimeInversion
%                 sizeZ = size(Z,1);
%                 A = zeros(sizeZ,sizeZ);
%                 for i=1:sizeZ
%                     A(i,sizeZ + 1 -i) = 1;
%                 end
%                 Z = A*Z;
% 
%                 % we check with the c, so we use generated random numbers
% %                 mc_random = importdata('mc_random.xlsx');
% %                 mc_random = importdata('D:\mc_random_number_new_report.txt');
% %                 Z= mc_random;
%                 
                numMethodInfo.pathSize = pathSize;
                numMethodInfo.numOfFactors = numOfFactors;
                numMethodInfo.totalTimeSteps = outInfo.totalTimeSteps;
                numMethodInfo.totalTimeStepsSize = outInfo.totalTimeStepsSize;
%                 numMethodInfo.Z = Z;
% 
% 
% 
%                 %generate model states for the given timeSteps
%                 modelStates = numMethod.generateModelStates(valueDate,schedule,numMethodInfo,model);

                % numeraire info
                numMatTime = numMethodInfo.totalTimeSteps(end);
                dfNumMat = model.DF(numMatTime/365.0);
                
                % induct backward and calculate cashflow
            
                periodCount = schedule.numOfPeriod;
                
                % cashflow columns initialize
                % 1st nonCall
                nonCall.npv = 0;
                nonCall.cashflow_e = zeros(periodCount + 1,1);
                nonCall.cashflow_npv_e = zeros(periodCount + 1,1);
                nonCall.rangeProb_e = zeros(periodCount + 1,1);
                nonCall.df_e = zeros(periodCount + 1,1);
                nonCall.df_c = zeros(periodCount + 1,1);

                % debug information, cashflow & npv per path
                nonCall.payoff = zeros(1,pathSize);
                nonCall.payoffStates.cashflow = zeros(periodCount+1,pathSize);
                nonCall.payoffStates.cashflow_npv = zeros(periodCount+1,pathSize);
                nonCall.payoffStates.df = zeros(periodCount + 1,pathSize);
                
                % intermediate variables
                payoffStateA.cashflow = zeros(1, pathSize);
                payoffStateA.cashflow_npv = zeros(1, pathSize);
                payoffStateA.df = zeros(1,pathSize);
                
                % we calculate cashflow backward from maturityDate to valueDate
                % rangeAccrual coupons are added at the startDate of the Period
                % At maturity

                nominal = security.nominal;
                coupon = security.coupon;
                callYN = security.callYN;
                
                ralower = security.lower;
                raupper = security.upper;
                
                tenorCD = security.tenor;
                
                currentTime = numMethodInfo.totalTimeSteps(end);
                totalTimeSteps = numMethodInfo.timeStepsInfo.totalTimeSteps;
                totalTimeStepsSize = numMethodInfo.timeStepsInfo.totalTimeStepsSize;
                if totalTimeStepsSize == 0 || (totalTimeStepsSize == 1 && totalTimeSteps(1) == 0) 
                    out.nonCall = nonCall;
                    return;
                end
                
                currentTimeIdx = totalTimeStepsSize;
                
                currentNodeIdx = periodCount + 1;
                
                % swap
%                 payoffStateA.cashflow = nominal*ones(1,pathSize);
                payoffStateA.cashflow = 0.0*nominal*ones(1,pathSize);
                
                %process payoff at maturity
                nonCall.payoffStates.cashflow(currentNodeIdx,:) = payoffStateA.cashflow;
                
%                 payoffStateA.cashflow_npv = model.discountPayoff(currentTime,numMatTime ...
%                     ,modelStateT,payoffStateA.cashflow);
%                 
                payoffStateA.cashflow_npv = model.DFRisky(currentTime/365.0)*payoffStateA.cashflow;
                
                nonCall.payoffStates.cashflow_npv(currentNodeIdx,:) = payoffStateA.cashflow_npv ;
                
%                 payoffStateA.df = model.discountPayoff(currentTime,numMatTime ...
%                     ,modelStateT,1.0*ones(1,pathSize));
                payoffStateA.df = model.DFRisky(currentTime/365.0) *ones(1,pathSize);
                    

                nonCall.payoffStates.df(currentNodeIdx,:) = payoffStateA.df;
                nonCall.payoff = nonCall.payoff + payoffStateA.cashflow_npv;
                
                nonCall.cashflow_e(currentNodeIdx) = mean(payoffStateA.cashflow);
                nonCall.cashflow_npv_e(currentNodeIdx) = mean(payoffStateA.cashflow_npv);
                nonCall.df_e(currentNodeIdx) = mean(payoffStateA.df);
                nonCall.df_c(currentNodeIdx) = model.DFRisky(currentTime/365.0);
                
%                 % callable at maturity is the same as noncallable
%                 callable.payoffStates.cashflow(currentNodeIdx,:) = nonCall.payoffStates.cashflow(currentNodeIdx,:);
%                 callable.payoffStates.cashflow_npv(currentNodeIdx,:) = nonCall.payoffStates.cashflow_npv(currentNodeIdx,:);
%                 callable.payoff = callable.payoff + callable.payoffStates.cashflow_npv(currentNodeIdx,:);
%                 
%                 callable.cashflow_e(currentNodeIdx) = nonCall.cashflow_e(currentNodeIdx);
%                 callable.cashflow_npv_e(currentNodeIdx) = nonCall.cashflow_npv_e(currentNodeIdx);
                  
                lastTimeIdx = currentTimeIdx;
                
                
                % flag for inclusive schedule
%                 simpleYN = false;
                simpleYN = 1;
                
                for i=periodCount:-1:1
                    % past schedule neglect
                    if DateDiff(valueDate,schedule.payDates(i)) >= 0, continue;end 
                    startTimeIdx = numMethodInfo.timeStepsInfo.startTimeIdx(i);
                    endTimeIdx = numMethodInfo.timeStepsInfo.endTimeIdx(i);

                    inclusiveYN = numMethodInfo.timeStepsInfo.inclusiveYN(i);
                     %% FromToInduction start

                    %modelsStates induct backward(lastTimeIdx to endTimeIdx)
                    while endTimeIdx < lastTimeIdx
                        lastTimeIdx = lastTimeIdx -1;
                        if lastTimeIdx == endTimeIdx break;end
                    end

                    currentTimeIdx = lastTimeIdx;
                    currentTime = totalTimeSteps(currentTimeIdx);
                    % now we are at endTimeIdx
                    % variables for coupon valuation initialized
%                     ralower = security.lower;
%                     raupper = security.upper;
%                     tenorLong = security.tenorL;
%                     tenorShort = security.tenorS;

                    cD = 0; % cumulated day count fraction
                    cP = 0; 
                    dI = 0; % dummy idx for loop
                    dcfC = 0;
                    dcfN = 0;
                    dD = 0;
                    idxCD = zeros(1,pathSize);
                    
                    idxC = zeros(1,pathSize);
                    idxN = zeros(1,pathSize);
                    
                    probSpread = zeros(1,pathSize);
                    
                    numOfTimeSteps = numMethodInfo.timeStepsInfo.numOfTimeSteps(i);
                    
                    % if inclusive then we use simpleYN rangeProb
                    %if inclusive then we do not evaluate the last step but
                    % use fixing instead 
                    endTimeStep = 1;
                    if inclusiveYN
%                         simpleYN = true;
                        simpleYN = 1;
                        endTimeStep = 2;
                    end
                    
                    for j=numOfTimeSteps:-1:endTimeStep
                        % evaluate dealdescription
                        % last step is omitted
                        currentTime = totalTimeSteps(currentTimeIdx);
                        %dcfC = currentTime/365.0;
                        dcfC = currentTime;
                        
                        if j~= numOfTimeSteps
                            dD = (dcfN-dcfC);
                        end

                        toDate = AddDate(valueDate,currentTime,'day');
                        tpDate = schedule.payDates(i);
                        
                        
                        spreadC = model.CDDigitalRangeDate(valueDate,toDate,tpDate,tenorCD,0.25,ralower,raupper);
                        
%                         if numOfFactors == 2                                             
% %                             spreadC = model.CDCMSSpreadDigitalRangeDate(valueDate,toDate,tpDate,tenorCD,10,2,0.25,ralower,raupper,-10000,10000);
%                             spreadC = model.CDDigitalRangeDate(valueDate,toDate,tpDate,tenorCD,0.25,ralower,raupper);
%                         
%                         else
%                             spreadC = model.CDDigitalRangeDate(valueDate,toDate,tpDate,tenorCD,0.25,ralower,raupper);
%                         
%                         end
%                         %                         spreadC = model.CMSSpreadDigitalRangeDate(valueDate,toDate,tpDate,tenorLong,tenorShort,0.25,ralower,raupper);
                        
                        %modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:numOfFactors*(currentTimeIdx-1)+2,:);
%                         idxL = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorLong,modelStateT);
%                         idxS = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorShort,modelStateT);
%                         spreadC = idxL - idxS;
%                         probSpread = model.SRPr(spreadC,spreadN,ralower,raupper,simpleYN,pathSize)*dD; 

                        if simpleYN
                            probSpread = spreadC*dD;
                        else
                            probSpread = spreadC*dD;
%                             probSpread = 0.5*(spreadC + spreadN)*dD;
                            
                        end

                        
                        cD = cD + dD;
                        cP = cP + probSpread;

                        dcfN = dcfC;
                        spreadN = spreadC;

                        %induct backward to the next time step
                        % one time backward induction
                        lastTimeIdx = currentTimeIdx;

                        if startTimeIdx < lastTimeIdx
                            lastTimeIdx = lastTimeIdx -1;
                            currentTimeIdx = lastTimeIdx;
                        end


                    end

                    %% FromToInduction end
                    %% AtDate evaluation
                    % Adding a new cashflow to the payoff

                    dcf = schedule.dayCountFraction(i);
                    
                    if inclusiveYN
                        % we add valuation date
                        cD = schedule.dayCount(i);
                        rangeDays =  security.rangeDays(i);
                        payoffStateA.cashflow = nominal * coupon * dcf * (rangeDays + cP)/cD;
                    else
                        payoffStateA.cashflow = nominal * coupon * dcf * cP/cD;
                    end
                    % eventDate(i) = startDate(i) therefore there is no induction in modelStates
                    % i.e. currentTimeIdx has not changed
                    % no need to update modelStateT
                    % modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:end,:);
                    % since cashflow's are added at the startDate(i) we need to
                    % multiply cashflow with the df(startDate(i), payDate(i))
                    payTime = DateDiff(schedule.payDates(i),valueDate);
                    
                    %event is at startDate so we need to discount the
                    %cashflow first from paydate to startdate
%                     df_sp = model.discountFactor(currentTime, payTime,numMatTime,modelStateT);

%                     for k=1:pathSize
%                         payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_sp(k);
%                     end
                    
                    % process payoff
                    
                    currentNodeIdx = i;
                    
                    % 1st nonCall column
                    nonCall.payoffStates.cashflow(currentNodeIdx,:) = payoffStateA.cashflow;
                    payoffStateA.cashflow_npv = payoffStateA.cashflow * model.DFRisky(payTime/365.0);
                    
                    nonCall.payoffStates.cashflow_npv(currentNodeIdx,:) = payoffStateA.cashflow_npv;
                    payoffStateA.df = model.DFRisky(payTime/365.0);
                    nonCall.payoffStates.df(currentNodeIdx,:) = payoffStateA.df;
                    
                    nonCall.payoff = nonCall.payoff + payoffStateA.cashflow_npv;
                    
                    nonCall.cashflow_e(currentNodeIdx) = mean(payoffStateA.cashflow);
                    nonCall.rangeProb_e(currentNodeIdx) = mean(nonCall.cashflow_e(currentNodeIdx)/(nominal * coupon * dcf));
                    nonCall.cashflow_npv_e(currentNodeIdx) = mean(payoffStateA.cashflow_npv);

                    nonCall.df_e(currentNodeIdx) = mean(payoffStateA.df);
                    nonCall.df_c(currentNodeIdx) = model.DFRisky(currentTime/365.0);
                    
                    % if exercise is possible
                    if i == 1
                        dummaaa = 1;
                    end
                    
                    % floating leg event happens at the start of the period
                    if floatingYN(i) 
                        dcfFloating = floatingSchedule.dayCountFraction(i);
                        %floatingIndex is not fixed yet
                        if DateDiff(valueDate,floatingSchedule.resetDates(i)) < 0
                            
                            if strcmp(floatingIndex,'Libor3M')
                                tenorFloatingCD = 0.25;
%                                 idxFloatingCD = model.Libor_SimpleDate(valueDate,currentTime,currentTime,numMatTime, tenorFloatingCD,modelStateT);
                                idxFloatingCD = model.LiborDate(valueDate,currentTime,tenorFloatingCD);
                                % -1 for floating pay 
                                payoffStateA.cashflow = -1.0 * nominal * (idxFloatingCD + floatingSpread) * dcfFloating;
                            end
                            
                        else %floatingIndex is fixed
                            if strcmp(floatingIndex,'Libor3M')
                                tenorFloatingCD = 0.25;
                                allFixings = security.allFixings;
                                USDLibor3MFixings = allFixings('USDLibor3M');
                                idxFloatingCD = USDLibor3MFixings(StrDate(floatingSchedule.resetDates(i),'str'));
%                                 idxFloatingCD = model.Libor_SimpleDate(valueDate,currentTime,currentTime,numMatTime, tenorFloatingCD,modelStateT);
                                % -1 for floating pay
                                    payoffStateA.cashflow = -1.0 * nominal * (idxFloatingCD + floatingSpread) * dcfFloating;
                             end
                            
                        end
                        
                        payTime = DateDiff(floatingSchedule.payDates(i),valueDate);
                        %event is at startDate so we need to discount the
                        %cashflow first from paydate to startdate
                        nonCall.payoffStates.cashflow(currentNodeIdx,:) = nonCall.payoffStates.cashflow(currentNodeIdx,:) + payoffStateA.cashflow;
                        payoffStateA.cashflow_npv = payoffStateA.cashflow * model.DFRisky(payTime/365.0);

                        nonCall.payoffStates.cashflow_npv(currentNodeIdx,:) = nonCall.payoffStates.cashflow_npv(currentNodeIdx,:) + payoffStateA.cashflow_npv;
                        
                        nonCall.payoff = nonCall.payoff + payoffStateA.cashflow_npv;

                        nonCall.cashflow_e(currentNodeIdx) = nonCall.cashflow_e(currentNodeIdx) + mean(payoffStateA.cashflow);
                        nonCall.cashflow_npv_e(currentNodeIdx) = nonCall.cashflow_npv_e(currentNodeIdx) + mean(payoffStateA.cashflow_npv);
                            
                        
                    end
                    
                    % call event happens  at the start of the period
                    % at call date, by default, call option is expired
                    
%                     if callYN(i) && (DateDiff(valueDate,schedule.startDates(i)) < 0)
%                         % 1.undiscount callable.payoff
%                         % 2.build the state variables matrix
%                         % 3.regress 1's with the given state variables
%                         % (model states 1,2)
%                         % 4. exercise per each path in
%                         % discount the new payoff and assign into callable
%                         % column
%                         
%                         %1 undiscount callable.payoff and get the
%                         %contiValue
%                        
%                         %exercise value
%                         exerciseValue = security.nominal*ones(1,pathSize);
%                         contiValue = zeros(1,pathSize);
%                         onePayoff = ones(1,pathSize);
%                         onePayoffD = model.discountPayoff(currentTime,numMatTime ...
%                         ,modelStateT,onePayoff);
%                         contiValue = callable.payoff./onePayoffD;
%                         
%                         % build the state variabls matrix
%                         decisionVar1 = modelStateT(1,:);
%                         decisionVar2 = modelStateT(2,:);
%                         % we use upto 2nd order polynomial as regression
%                         % equatioin
%                         dimensionLS = 6;
%                         regMatrix = ones(pathSize,dimensionLS);
%                         
%                         for idx_r=1:pathSize
%                             regMatrix(idx_r,2) = decisionVar1(idx_r);
%                             regMatrix(idx_r,3) = decisionVar2(idx_r);
%                             regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r);
%                             regMatrix(idx_r,5) = decisionVar1(idx_r)*decisionVar2(idx_r);
%                             regMatrix(idx_r,6) = decisionVar2(idx_r)*decisionVar2(idx_r);
%                         end
%                         
%                         % simple regression <- for path dependent option
%                         % we should change it
%                         % beta  : regression coefficients
%                         beta =zeros(dimensionLS,1);
%                         beta = regress(contiValue',regMatrix);
%                         regContiValue = regMatrix*beta;
%                         
%                         exercisedPayoff = regContiValue;
%                         % if regContiValue is bigger than exerciseValue
%                         % then issue exercise call option
%                         for idx_r=1:pathSize
%                             if regContiValue(idx_r) >= exerciseValue(idx_r) 
%                                 exercisedPayoff(idx_r) = exerciseValue(idx_r);
%                             end
%                         end
%                         
%                         % we discount the new payoff and 
%                         % assign it into the existing callable payoff
%                         callable.payoff = model.discountPayoff(currentTime,numMatTime ...
%                         ,modelStateT,exercisedPayoff);
%                         
%                     end
                end

                nonCall.npv = mean(nonCall.payoff);
                out.pricerInfo.nonCall = nonCall;
                
%                 callable.npv = mean(callable.payoff);
%                 out.pricerInfo.callable = callable;
                out.daycountFraction = schedule.dayCountFraction;
                
        end
        
        
        
        
    end
    
end



In [ ]:
classdef DailyWeeklyRiskyPricer_CDRA2 < H_RootObject
    %Pricer(Security,Model,NumMethod)
    %Prcer is constructed from Security, Model, NumMethod 
    
    properties
        m_security
        m_model
        m_numMethod
        overHedgeYN
        
    end
    
    methods
        function pricer = DailyWeeklyRiskyPricer_CDRA2(security,model,numMethod,overHedgeYN)
                pricer.m_security = security;
                pricer.m_model = model;
                pricer.m_numMethod = numMethod;
                pricer.overHedgeYN = overHedgeYN;
                
        end
        
        function out = computePriceMCCallableGeneric(pricer,valueDate,pricerOption)
                security = pricer.m_security;
                numMethod = pricer.m_numMethod;
                model = pricer.m_model;
                schedule = security.couponSchedule;
                callSchedule = security.callSchedule;
                
                % numMethodTimeSteps Generate
                mcOneTimeStep = numMethod.mcOneTimeStep;
                outInfo = numMethod.numMethodTimeStepsGenerateDW(valueDate,schedule,mcOneTimeStep);
            
                numMethodInfo.timeStepsInfo = outInfo;

                % mc random number init
                %pathSize = 2^16;
                pathSize = numMethod.pathSize;

                % generate random numbers and modelstates forward
                rng('default');
                rng(0);

                % LGM1F : one factor
                % LGM2F : two factor
                
%                 numOfFactors = model.numOfFactor;
                numOfFactors = model.numOfFactors;

                %Z= randn(numOfFactors*outInfo.totalTimeStepsSize,pathSize/2);
                Z= randn(pathSize/2,numOfFactors*outInfo.totalTimeStepsSize);
                Z=[Z;-Z];
                Z=Z';
                
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;

                % we check with the c code, so we use generated random numbers
%                 mc_random = importdata('mc_random.xlsx');
%                 mc_random = importdata('D:\mc_random_number_new_report.txt');
%                 Z= mc_random;
                
                numMethodInfo.pathSize = pathSize;
                numMethodInfo.numOfFactors = numOfFactors;
                numMethodInfo.totalTimeSteps = outInfo.totalTimeSteps;
                numMethodInfo.totalTimeStepsSize = outInfo.totalTimeStepsSize;
                numMethodInfo.Z = Z;



                %numMethod generate model states for the given timeSteps
                modelStates = numMethod.generateModelStatesNF(valueDate,schedule,numMethodInfo,model);

                % numeraire info
                numMatTime = numMethodInfo.totalTimeSteps(end);
                days = DateDiff(schedule.payDates(schedule.numOfPeriod),valueDate);
                numMatTime = max(numMatTime, days);
                dfNumMat = model.DFRisky(numMatTime/365.0);
                
                numeraire.numMatTime = numMatTime;
                numeraire.dfNumMat = dfNumMat;
                
                % induct backward and calculate cashflow
            
                periodCount = schedule.numOfPeriod;
                
                % pricer single info initialize
                
                nonCall = PricerInfo(periodCount +1, pathSize);
                callable = PricerInfo(periodCount +1, pathSize);
                dfCol = PricerInfo(periodCount +1, pathSize);
                
                dummy1 = PricerInfo(periodCount +1, pathSize);
                dummy2 = PricerInfo(periodCount +1, pathSize);
                
                
                % intermediate variables
                payoffStateA.cashflow = zeros(1, pathSize);
                payoffStateA.cashflow_npv = zeros(1, pathSize);
                
            
                % we calculate cashflow backward from maturityDate to valueDate
                % rangeAccrual coupons are added at the startDate of the Period
                % At maturity

                nominal = security.nominal;
                coupon = security.coupon;
                callYN = security.callYN;
                
                ralower = security.lower;
                raupper = security.upper;
                
                ralowerS = security.lowerS;
                raupperS = security.upperS;
                
                tenorCD = security.tenor;
                
                tenorLong = security.tenorL;
                tenorShort = security.tenorS;
                
                currentTime = numMethodInfo.totalTimeSteps(end);
                totalTimeStepsSize = numMethodInfo.totalTimeStepsSize;
                currentTimeIdx = totalTimeStepsSize;
                
                currentNodeIdx = periodCount + 1;
                
                modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:numOfFactors*(currentTimeIdx-1)+numOfFactors,:);
                
                payoffStateA.cashflow = nominal*ones(1,pathSize);
                
                payTime = DateDiff(schedule.payDates(schedule.numOfPeriod),valueDate);
                df_ep = model.discountFactorRisky(currentTime, payTime,numMatTime,modelStateT);
                for k=1:pathSize
                    payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                end
                
                %process payoff at maturity
                nonCall.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                
                callable.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                
                %df column
                payoffStateA.cashflow = ones(1,pathSize);
                for k=1:pathSize
                    payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                end
                
                dfCol.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                                 
                %dummy column

                if strcmp(pricerOption,'PSA')
                    payoffStateA.cashflow = model.CMS_PSA_SimpleDate('KRW',valueDate,currentTime,numMatTime, tenorLong,modelStateT);
                    dummy1.ProcessPayoff(model,currentTime,numeraire, ...
                                 modelStateT,payoffStateA.cashflow,currentNodeIdx);
                    
                    payoffStateA.cashflow = model.CMS_PSA_SimpleDate('USD',valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                    dummy2.ProcessPayoff(model,currentTime,numeraire, ...
                                 modelStateT,payoffStateA.cashflow,currentNodeIdx);
                
                else
                    payoffStateA.cashflow = model.Libor_SimpleDate(valueDate,currentTime,currentTime,numMatTime, tenorLong,modelStateT); 
                    dummy1.ProcessPayoff(model,currentTime,numeraire, ...
                                 modelStateT,payoffStateA.cashflow,currentNodeIdx);
                             
%                     payoffStateA.cashflow = model.Libor_SimpleDate('USD',valueDate,currentTime,currentTime,numMatTime, tenorShort,modelStateT); 
%                     dummy2.ProcessPayoff(model,currentTime,numeraire, ...
%                                  modelStateT,payoffStateA.cashflow,currentNodeIdx);
%                     
                end
                

%                 payoffStateA.cashflow = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                
%                 payoffStateA.cashflow = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                
                
                                 
                  

                lastTimeIdx = currentTimeIdx;
                totalTimeSteps = numMethodInfo.timeStepsInfo.totalTimeSteps;
                
                % flag for inclusive schedule
%                 simpleYN = false;
%                 simpleYN = true;
                if pricer.overHedgeYN == true 
                    simpleYN = 2;
                else
                    simpleYN = 1;
                end
                
                for i=periodCount:-1:1
                    % past schedule neglect
                    if DateDiff(valueDate,schedule.payDates(i)) >= 0, continue;end 
                    startTimeIdx = numMethodInfo.timeStepsInfo.startTimeIdx(i);
                    endTimeIdx = numMethodInfo.timeStepsInfo.endTimeIdx(i);

                    inclusiveYN = numMethodInfo.timeStepsInfo.inclusiveYN(i);
                 %% FromToInduction start

                    %modelsStates induct backward(lastTimeIdx to endTimeIdx)
                    while endTimeIdx < lastTimeIdx
                        lastTimeIdx = lastTimeIdx -1;
                        if lastTimeIdx == endTimeIdx break;end
                    end

                    currentTimeIdx = lastTimeIdx;
                    currentTime = totalTimeSteps(currentTimeIdx);
                    % now we are at endTimeIdx
                    % variables for coupon valuation initialized
                    

                    cD = 0; % cumulated day count fraction
                    cP = 0; 
                    dI = 0; % dummy idx for loop
                    dcfC = 0;
                    dcfN = 0;
                    dD = 0;
                    idxCD = zeros(1,pathSize);
                    idxL = zeros(1,pathSize);
                    idxS = zeros(1,pathSize);
                    
                    idxC = zeros(1,pathSize);
                    idxN = zeros(1,pathSize);
                    
                    spreadC = zeros(1,pathSize);
                    spreadN = zeros(1,pathSize);
                    probSpread = zeros(1,pathSize);
                    
                    numOfTimeSteps = numMethodInfo.timeStepsInfo.numOfTimeSteps(i);
                    
                    % if inclusive then we use simpleYN rangeProb
                    %if inclusive then we do not evaluate the last step but
                    % use fixing instead 
                    endTimeStep = 1;
                    if inclusiveYN
%                         simpleYN = true;
                        simpleYN = 1;
                        endTimeStep = 2;
                        if pricer.overHedgeYN == true 
                            simpleYN = 2;
                        end
                    end
                    
                    for j=numOfTimeSteps:-1:endTimeStep
                        % evaluate dealdescription
                        % last step is omitted
                        currentTime = totalTimeSteps(currentTimeIdx);
                        %dcfC = currentTime/365.0;
                        dcfC = currentTime;
                        
                        if j~= numOfTimeSteps
                            dD = (dcfN-dcfC);
                        end

                        modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:numOfFactors*(currentTimeIdx-1)+numOfFactors,:);
                        
                        idxCD = model.Libor_SimpleDate(valueDate,currentTime,currentTime,numMatTime, tenorCD,modelStateT);
                        idxC = idxCD;
                        
%                         if strcmp(pricerOption,'PSA')
%                             idxL = model.CMS_PSA_SimpleDate('KRW',valueDate,currentTime,numMatTime, tenorLong,modelStateT);
%                             idxS = model.CMS_PSA_SimpleDate('USD',valueDate,currentTime,numMatTime, tenorShort,modelStateT);
%                         else
% %                             idxL = model.Libor_SimpleDate(valueDate,currentTime,currentTime,numMatTime, tenorLong,modelStateT);
% %                             idxS = model.Libor_SimpleDate(valueDate,currentTime,currentTime,numMatTime, tenorShort,modelStateT);
%                             
%                         end
                        spreadC = idxL - idxS;

%                         probSpread = model.DRPr(idxC,idxN,ralower,raupper,spreadC,spreadN,ralowerS,raupperS,simpleYN,pathSize)*dD;
                    
                        probSpread = model.SRPr(idxC,idxN,ralower,raupper,simpleYN,pathSize)*dD;
                        cD = cD + dD;
                        cP = cP + probSpread;

                        dcfN = dcfC;
                        
                        idxN = idxC;
                        spreadN = spreadC;

                        %induct backward to the next time step
                        % one time backward induction
                        lastTimeIdx = currentTimeIdx;

                        if startTimeIdx < lastTimeIdx
                            lastTimeIdx = lastTimeIdx -1;
                            currentTimeIdx = lastTimeIdx;
                        end


                    end

                    %% FromToInduction end
        % if the current period is inclusive, then the start date is not evaluated in the above for loop.
		% instead the fixing will be treated separately below.
		% however current_time_idx is updated at the end of the for loop.
		% so we need to update model_states accordingly.
		% if model_states were updated previously then the following additionaly step doesn't affect the result anyway.
                    currentTime = totalTimeSteps(currentTimeIdx);
                    modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:numOfFactors*(currentTimeIdx-1)+numOfFactors,:);
                    
                    %% AtDate evaluation
                    % Adding a new cashflow to the payoff

                    dcf = schedule.dayCountFraction(i);
                    
                    if inclusiveYN
                        % we add valuation date
                        cD = schedule.dayCount(i);
                        rangeDays =  security.rangeDays(i);
                        payoffStateA.cashflow = nominal * coupon * dcf * (rangeDays + cP)/cD;
                    else
                        payoffStateA.cashflow = nominal * coupon * dcf * cP/cD;
                    end
                    % eventDate(i) = startDate(i) therefore there is no induction in modelStates
                    % i.e. currentTimeIdx has not changed
                    % no need to update modelStateT
                    % modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:end,:);
                    % since cashflow's are added at the startDate(i) we need to
                    % multiply cashflow with the df(startDate(i), payDate(i))
                    payTime = DateDiff(schedule.payDates(i),valueDate);
                    
                    %event is at startDate so we need to discount the
                    %cashflow first from paydate to startdate
                    df_ep = model.discountFactorRisky(currentTime, payTime,numMatTime,modelStateT);

                    for k=1:pathSize
                        payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                    end
                    
                    % process payoff
                    
                    currentNodeIdx = i;
                    
                    nonCall.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                
                    callable.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                    
                    %df column
                    payoffStateA.cashflow = ones(1,pathSize);
                    for k=1:pathSize
                        payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                    end
                    
                    dfCol.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                                 
                    %dummy column
%                     if strcmp(pricerOption,'PSA')
%                         payoffStateA.cashflow = model.CMS_PSA_SimpleDate('KRW',valueDate,currentTime,numMatTime, tenorLong,modelStateT);
%                         dummy1.ProcessPayoff(model,currentTime,numeraire, ...
%                                          modelStateT,payoffStateA.cashflow,currentNodeIdx);
% 
% 
% 
%                         payoffStateA.cashflow = model.CMS_PSA_SimpleDate('USD',valueDate,currentTime,numMatTime, tenorShort,modelStateT);
%                         dummy2.ProcessPayoff(model,currentTime,numeraire, ...
%                                          modelStateT,payoffStateA.cashflow,currentNodeIdx);
%                     
%                     else
%                         payoffStateA.cashflow = model.Libor_SimpleDate('KRW',valueDate,currentTime,currentTime,numMatTime, tenorLong,modelStateT); 
%                         dummy1.ProcessPayoff(model,currentTime,numeraire, ...
%                                  modelStateT,payoffStateA.cashflow,currentNodeIdx);
%                         
%                         payoffStateA.cashflow = model.Libor_SimpleDate('USD',valueDate,currentTime,currentTime,numMatTime, tenorShort,modelStateT); 
%                         dummy2.ProcessPayoff(model,currentTime,numeraire, ...
%                                  modelStateT,payoffStateA.cashflow,currentNodeIdx);
%                         
%                     end

                                 
                    
                    
                    
                    % if exercise is possible
                    if i == 1
                        dummaaa = 1;
                    end
                    
                    % call event happens  at the start of the period
                    % at call date, by default, call option is expired
                    
                    if callYN(i) && (DateDiff(valueDate,callSchedule.resetDates(i)) < 0)
                        % 1.undiscount callable.payoff
                        % 2.build the state variables matrix
                        % 3.regress 1's with the given state variables
                        % (model states 1,2)
                        % 4. exercise per each path in
                        % discount the new payoff and assign into callable
                        % column
                        
                        %1 undiscount callable.payoff and get the
                        %contiValue
                       
                        %exercise value
                        exerciseValue = security.nominal*ones(1,pathSize);
                        payTime = DateDiff(callSchedule.payDates(i),valueDate);
                        %event is at startDate so we need to discount the
                        %cashflow first from paydate to startdate
                        df_ep = model.discountFactorRisky(currentTime, payTime,numMatTime,modelStateT);
                        for k=1:pathSize
                            exerciseValue(k) = exerciseValue(k) * df_ep(k);
                        end
                        
                        contiValue = zeros(1,pathSize);
                        onePayoff = ones(1,pathSize);
                        onePayoffD = model.discountPayoffRisky(currentTime,numMatTime ...
                        ,modelStateT,onePayoff);
                        contiValue = callable.payoff./onePayoffD;
                        
                        %Nominal Regression test
                        contiValue = contiValue/10000;
                        
                        % build the state variabls matrix
                        dimensionLS = (numOfFactors + 1)*(numOfFactors + 2)/2;
                        
                        decisionVar = zeros(numOfFactors,size(modelStateT,2));
                        
                        for idxLS =1:numOfFactors
                            decisionVar(idxLS,:) = modelStateT(idxLS,:); 
                        end
%                         decisionVar1 = modelStateT(1,:);
%                         decisionVar2 = modelStateT(2,:);
                        % we use upto 2nd order polynomial as regression
                        % equatioin
                        
                        regMatrix = ones(pathSize,dimensionLS);
                        
                        for idx_r=1:pathSize
                            for idxLS=1:numOfFactors
                                regMatrix(idx_r,idxLS+1) = decisionVar(idxLS,idx_r);
                            end
                            
                            tempIdx = 1;
                            for idxLS1=1:numOfFactors
                                for idxLS2=idxLS1:numOfFactors
                                    regMatrix(idx_r,numOfFactors+1+tempIdx) = decisionVar(idxLS1,idx_r)*decisionVar(idxLS2,idx_r);
                                    tempIdx = tempIdx + 1;
                                end
                            end
                            
                        end
                        
                        % simple regression <- for path dependent option
                        % we should change it
                        % beta  : regression coefficients
                        beta =zeros(dimensionLS,1);
                        beta = regress(contiValue',regMatrix);
                        regContiValue = regMatrix*beta;
                       
                         %Nominal Regression test
                        regContiValue = regContiValue * 10000;
                        
                        exercisedPayoff = regContiValue;
                        % if regContiValue is bigger than exerciseValue
                        % then issue exercise call option
                        for idx_r=1:pathSize
                            if regContiValue(idx_r) >= exerciseValue(idx_r) 
                                exercisedPayoff(idx_r) = exerciseValue(idx_r);
                            end
                        end
                        
                        % we discount the new payoff and 
                        % assign it into the existing callable payoff
                        callable.payoff = model.discountPayoffRisky(currentTime,numMatTime ...
                        ,modelStateT,exercisedPayoff);
                        
                    end
                end

                nonCall.npv = mean(nonCall.payoff);
                out.nonCall = nonCall;
                callable.npv = mean(callable.payoff);
                dfCol.npv = mean(dfCol.payoff);
                dummy1.npv = mean(dummy1.npv);
                dummy2.npv = mean(dummy2.npv);
                out.callable = callable;
                out.dfCol = dfCol;
                out.dummy1 = dummy1;
                out.dummy2 = dummy2;
                out.daycountFraction = schedule.dayCountFraction;
                
        end
        
        function out = computePriceMCCallableGenericReg2(pricer,valueDate,pricerOption)
                security = pricer.m_security;
                numMethod = pricer.m_numMethod;
                model = pricer.m_model;
                schedule = security.couponSchedule;
                callSchedule = security.callSchedule;
                
                % numMethodTimeSteps Generate
                mcOneTimeStep = numMethod.mcOneTimeStep;
                outInfo = numMethod.numMethodTimeStepsGenerateDW(valueDate,schedule,mcOneTimeStep);
            
                numMethodInfo.timeStepsInfo = outInfo;

                % mc random number init
                %pathSize = 2^16;
                pathSize = numMethod.pathSize;

                % generate random numbers and modelstates forward
                rng('default');
                rng(0);

                % LGM1F : one factor
                % LGM2F : two factor
                
                numOfFactors = model.numOfFactor;

                %Z= randn(numOfFactors*outInfo.totalTimeStepsSize,pathSize/2);
                Z= randn(pathSize/2,numOfFactors*outInfo.totalTimeStepsSize);
                Z=[Z;-Z];
                Z=Z';
                
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;

                % we check with the c code, so we use generated random numbers
%                 mc_random = importdata('mc_random.xlsx');
%                 mc_random = importdata('D:\mc_random_number_new_report.txt');
%                 Z= mc_random;
                
                numMethodInfo.pathSize = pathSize;
                numMethodInfo.numOfFactors = numOfFactors;
                numMethodInfo.totalTimeSteps = outInfo.totalTimeSteps;
                numMethodInfo.totalTimeStepsSize = outInfo.totalTimeStepsSize;
                numMethodInfo.Z = Z;



                %numMethod generate model states for the given timeSteps
                modelStates = numMethod.generateModelStatesNF(valueDate,schedule,numMethodInfo,model);

                % numeraire info
                numMatTime = numMethodInfo.totalTimeSteps(end);
                days = DateDiff(schedule.payDates(schedule.numOfPeriod),valueDate);
                numMatTime = max(numMatTime, days);
                dfNumMat = model.DFRisky(numMatTime/365.0);
                
                numeraire.numMatTime = numMatTime;
                numeraire.dfNumMat = dfNumMat;
                
                % induct backward and calculate cashflow
            
                periodCount = schedule.numOfPeriod;
                
                % pricer single info initialize
                
                nonCall = PricerInfo(periodCount +1, pathSize);
                callable = PricerInfo(periodCount +1, pathSize);
                dfCol = PricerInfo(periodCount +1, pathSize);
                
                dummy1 = PricerInfo(periodCount +1, pathSize);
                dummy2 = PricerInfo(periodCount +1, pathSize);
                
                
                % intermediate variables
                payoffStateA.cashflow = zeros(1, pathSize);
                payoffStateA.cashflow_npv = zeros(1, pathSize);
                
            
                % we calculate cashflow backward from maturityDate to valueDate
                % rangeAccrual coupons are added at the startDate of the Period
                % At maturity

                hedgeBuffer = security.hedgeBuffer;
                
                nominal = security.nominal;
                coupon = security.coupon;
                callYN = security.callYN;
                
                ralower = security.lower;
                raupper = security.upper;
                
                ralowerS = security.lowerS;
                raupperS = security.upperS;
                
                tenorCD = security.tenor;
                
                tenorLong = security.tenorL;
                tenorShort = security.tenorS;
                
                currentTime = numMethodInfo.totalTimeSteps(end);
                totalTimeStepsSize = numMethodInfo.totalTimeStepsSize;
                currentTimeIdx = totalTimeStepsSize;
                
                currentNodeIdx = periodCount + 1;
                
                modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:numOfFactors*(currentTimeIdx-1)+numOfFactors,:);
                
                payoffStateA.cashflow = nominal*ones(1,pathSize);
                
                payTime = DateDiff(schedule.payDates(schedule.numOfPeriod),valueDate);
                df_ep = model.discountFactorRisky(currentTime, payTime,numMatTime,modelStateT);
                for k=1:pathSize
                    payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                end
                
                %process payoff at maturity
                nonCall.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                
                callable.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                
                %df column
                payoffStateA.cashflow = ones(1,pathSize);
                for k=1:pathSize
                    payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                end
                
                dfCol.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                                 
                %dummy column

                if strcmp(pricerOption,'PSA')
                    payoffStateA.cashflow = model.CMS_PSA_SimpleDate('KRW',valueDate,currentTime,numMatTime, tenorLong,modelStateT);
                    dummy1.ProcessPayoff(model,currentTime,numeraire, ...
                                 modelStateT,payoffStateA.cashflow,currentNodeIdx);
                    
                    payoffStateA.cashflow = model.CMS_PSA_SimpleDate('USD',valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                    dummy2.ProcessPayoff(model,currentTime,numeraire, ...
                                 modelStateT,payoffStateA.cashflow,currentNodeIdx);
                
                else
                    payoffStateA.cashflow = model.Libor_SimpleDate('KRW',valueDate,currentTime,currentTime,numMatTime, tenorLong,modelStateT); 
                    dummy1.ProcessPayoff(model,currentTime,numeraire, ...
                                 modelStateT,payoffStateA.cashflow,currentNodeIdx);
                             
                    payoffStateA.cashflow = model.Libor_SimpleDate('USD',valueDate,currentTime,currentTime,numMatTime, tenorShort,modelStateT); 
                    dummy2.ProcessPayoff(model,currentTime,numeraire, ...
                                 modelStateT,payoffStateA.cashflow,currentNodeIdx);
                    
                end
                

%                 payoffStateA.cashflow = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                
%                 payoffStateA.cashflow = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                
                
                                 
                  

                lastTimeIdx = currentTimeIdx;
                totalTimeSteps = numMethodInfo.timeStepsInfo.totalTimeSteps;
                
                % flag for inclusive schedule
%                 simpleYN = false;
%                 simpleYN = true;
                if pricer.overHedgeYN == true 
                    simpleYN = 2;
                else
                    simpleYN = 1;
                end
                
                for i=periodCount:-1:1
                    % past schedule neglect
                    if DateDiff(valueDate,schedule.payDates(i)) >= 0, continue;end 
                    startTimeIdx = numMethodInfo.timeStepsInfo.startTimeIdx(i);
                    endTimeIdx = numMethodInfo.timeStepsInfo.endTimeIdx(i);

                    inclusiveYN = numMethodInfo.timeStepsInfo.inclusiveYN(i);
                 %% FromToInduction start

                    %modelsStates induct backward(lastTimeIdx to endTimeIdx)
                    while endTimeIdx < lastTimeIdx
                        lastTimeIdx = lastTimeIdx -1;
                        if lastTimeIdx == endTimeIdx break;end
                    end

                    currentTimeIdx = lastTimeIdx;
                    currentTime = totalTimeSteps(currentTimeIdx);
                    % now we are at endTimeIdx
                    % variables for coupon valuation initialized
                    

                    cD = 0; % cumulated day count fraction
                    cP = 0; 
                    dI = 0; % dummy idx for loop
                    dcfC = 0;
                    dcfN = 0;
                    dD = 0;
                    idxCD = zeros(1,pathSize);
                    idxL = zeros(1,pathSize);
                    idxS = zeros(1,pathSize);
                    
                    idxC = zeros(1,pathSize);
                    idxN = zeros(1,pathSize);
                    
                    spreadC = zeros(1,pathSize);
                    spreadN = zeros(1,pathSize);
                    probSpread = zeros(1,pathSize);
                    
                    numOfTimeSteps = numMethodInfo.timeStepsInfo.numOfTimeSteps(i);
                    
                    % if inclusive then we use simpleYN rangeProb
                    %if inclusive then we do not evaluate the last step but
                    % use fixing instead 
                    endTimeStep = 1;
                    if inclusiveYN
%                         simpleYN = true;
                        simpleYN = 1;
                        endTimeStep = 2;
                        if pricer.overHedgeYN == true 
                            simpleYN = 2;
                        end
                    end
                    
                    for j=numOfTimeSteps:-1:endTimeStep
                        % evaluate dealdescription
                        % last step is omitted
                        currentTime = totalTimeSteps(currentTimeIdx);
                        %dcfC = currentTime/365.0;
                        dcfC = currentTime;
                        
                        if j~= numOfTimeSteps
                            dD = (dcfN-dcfC);
                        end

                        modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:numOfFactors*(currentTimeIdx-1)+numOfFactors,:);
                        
                        idxCD = model.Libor_SimpleDate('KRW',valueDate,currentTime,currentTime,numMatTime, tenorCD,modelStateT);
                        idxC = idxCD;
                        
                        if strcmp(pricerOption,'PSA')
                            idxL = model.CMS_PSA_SimpleDate('KRW',valueDate,currentTime,numMatTime, tenorLong,modelStateT);
                            idxS = model.CMS_PSA_SimpleDate('USD',valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                        else
                            idxL = model.Libor_SimpleDate('KRW',valueDate,currentTime,currentTime,numMatTime, tenorLong,modelStateT);
                            idxS = model.Libor_SimpleDate('USD',valueDate,currentTime,currentTime,numMatTime, tenorShort,modelStateT);
                        end
                        spreadC = idxL - idxS;

                        probSpread = model.DRPr(idxC,idxN,ralower,raupper,spreadC,spreadN,ralowerS,raupperS,simpleYN,hedgeBuffer,pathSize)*dD;
                    
%                         probSpread = model.SRPr(spreadC,spreadN,ralower,raupper,simpleYN,pathSize)*dD;
                        cD = cD + dD;
                        cP = cP + probSpread;

                        dcfN = dcfC;
                        
                        idxN = idxC;
                        spreadN = spreadC;

                        %induct backward to the next time step
                        % one time backward induction
                        lastTimeIdx = currentTimeIdx;

                        if startTimeIdx < lastTimeIdx
                            lastTimeIdx = lastTimeIdx -1;
                            currentTimeIdx = lastTimeIdx;
                        end


                    end

                    %% FromToInduction end
        % if the current period is inclusive, then the start date is not evaluated in the above for loop.
		% instead the fixing will be treated separately below.
		% however current_time_idx is updated at the end of the for loop.
		% so we need to update model_states accordingly.
		% if model_states were updated previously then the following additionaly step doesn't affect the result anyway.
                    currentTime = totalTimeSteps(currentTimeIdx);
                    modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:numOfFactors*(currentTimeIdx-1)+numOfFactors,:);
                    
                    %% AtDate evaluation
                    % Adding a new cashflow to the payoff

                    dcf = schedule.dayCountFraction(i);
                    
                    if inclusiveYN
                        % we add valuation date
                        cD = schedule.dayCount(i);
                        rangeDays =  security.rangeDays(i);
                        payoffStateA.cashflow = nominal * coupon * dcf * (rangeDays + cP)/cD;
                    else
                        payoffStateA.cashflow = nominal * coupon * dcf * cP/cD;
                    end
                    % eventDate(i) = startDate(i) therefore there is no induction in modelStates
                    % i.e. currentTimeIdx has not changed
                    % no need to update modelStateT
                    % modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:end,:);
                    % since cashflow's are added at the startDate(i) we need to
                    % multiply cashflow with the df(startDate(i), payDate(i))
                    payTime = DateDiff(schedule.payDates(i),valueDate);
                    
                    %event is at startDate so we need to discount the
                    %cashflow first from paydate to startdate
                    df_ep = model.discountFactorRisky(currentTime, payTime,numMatTime,modelStateT);

                    for k=1:pathSize
                        payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                    end
                    
                    % process payoff
                    
                    currentNodeIdx = i;
                    
                    nonCall.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                
                    callable.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                    
                    %df column
                    payoffStateA.cashflow = ones(1,pathSize);
                    for k=1:pathSize
                        payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                    end
                    
                    dfCol.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                                 
                    %dummy column
                    if strcmp(pricerOption,'PSA')
                        payoffStateA.cashflow = model.CMS_PSA_SimpleDate('KRW',valueDate,currentTime,numMatTime, tenorLong,modelStateT);
                        dummy1.ProcessPayoff(model,currentTime,numeraire, ...
                                         modelStateT,payoffStateA.cashflow,currentNodeIdx);



                        payoffStateA.cashflow = model.CMS_PSA_SimpleDate('USD',valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                        dummy2.ProcessPayoff(model,currentTime,numeraire, ...
                                         modelStateT,payoffStateA.cashflow,currentNodeIdx);
                    
                    else
                        payoffStateA.cashflow = model.Libor_SimpleDate('KRW',valueDate,currentTime,currentTime,numMatTime, tenorLong,modelStateT); 
                        dummy1.ProcessPayoff(model,currentTime,numeraire, ...
                                 modelStateT,payoffStateA.cashflow,currentNodeIdx);
                        
                        payoffStateA.cashflow = model.Libor_SimpleDate('USD',valueDate,currentTime,currentTime,numMatTime, tenorShort,modelStateT); 
                        dummy2.ProcessPayoff(model,currentTime,numeraire, ...
                                 modelStateT,payoffStateA.cashflow,currentNodeIdx);
                        
                    end

                                 
                    
                    
                    
                    % if exercise is possible
                    if i == 1
                        dummaaa = 1;
                    end
                    
                    % call event happens  at the start of the period
                    % at call date, by default, call option is expired
                    
                    if callYN(i) && (DateDiff(valueDate,callSchedule.resetDates(i)) < 0)
                        % 1.undiscount callable.payoff
                        % 2.build the state variables matrix
                        % 3.regress 1's with the given state variables
                        % (model states 1,2)
                        % 4. exercise per each path in
                        % discount the new payoff and assign into callable
                        % column
                        
                        %1 undiscount callable.payoff and get the
                        %contiValue
                       
                        %exercise value
                        exerciseValue = security.nominal*ones(1,pathSize);
                        payTime = DateDiff(callSchedule.payDates(i),valueDate);
                        %event is at startDate so we need to discount the
                        %cashflow first from paydate to startdate
                        df_ep = model.discountFactorRisky(currentTime, payTime,numMatTime,modelStateT);
                        for k=1:pathSize
                            exerciseValue(k) = exerciseValue(k) * df_ep(k);
                        end
                        
                        contiValue = zeros(1,pathSize);
                        onePayoff = ones(1,pathSize);
                        onePayoffD = model.discountPayoffRisky(currentTime,numMatTime ...
                        ,modelStateT,onePayoff);
                        contiValue = callable.payoff./onePayoffD;
                        
                        %Nominal Regression test
                        contiValue = contiValue/10000;
                        
                        % build the state variabls matrix
                        %zeroth order
                        dimensionLS = 1;
                        
                        %1st order
                        for idxLS =1:numOfFactors
                            dimensionLS =  dimensionLS +1; 
                        end
                        
                        %2nd order
                        for idxLS1=1:numOfFactors
                            for idxLS2=idxLS1:numOfFactors
                                dimensionLS = dimensionLS + 1;
                            end
                        end
                        
                        %3rd order
                        for idxLS1=1:numOfFactors
                            for idxLS2=idxLS1:numOfFactors
                                for idxLS3=idxLS2:numOfFactors
                                    dimensionLS = dimensionLS + 1;
                                end
                            end
                        end
                        
%                         dimensionLS = (numOfFactors + 1)*(numOfFactors + 2)/2;
                        
                        decisionVar = zeros(numOfFactors,size(modelStateT,2));
                        
                        for idxLS =1:numOfFactors
                            decisionVar(idxLS,:) = modelStateT(idxLS,:); 
                        end
%                         decisionVar1 = modelStateT(1,:);
%                         decisionVar2 = modelStateT(2,:);
                        % we use upto 2nd order polynomial as regression
                        % equatioin
                        
                        regMatrix = ones(pathSize,dimensionLS);
                        
                        for idx_r=1:pathSize
                            for idxLS=1:numOfFactors
                                regMatrix(idx_r,idxLS+1) = decisionVar(idxLS,idx_r);
                            end
                            
                            tempIdx = 1;
                            for idxLS1=1:numOfFactors
                                for idxLS2=idxLS1:numOfFactors
                                    regMatrix(idx_r,numOfFactors+1+tempIdx) = decisionVar(idxLS1,idx_r)*decisionVar(idxLS2,idx_r);
                                    tempIdx = tempIdx + 1;
                                end
                            end
                            
                            tempIdx2 = 1;
                            for idxLS1=1:numOfFactors
                                for idxLS2=idxLS1:numOfFactors
                                    for idxLS3=idxLS2:numOfFactors
                                        regMatrix(idx_r,numOfFactors+1+tempIdx-1 + tempIdx2) = decisionVar(idxLS1,idx_r)*decisionVar(idxLS2,idx_r)*decisionVar(idxLS3,idx_r);
                                        tempIdx = tempIdx + 1;
                                    end
                                end
                            end
                            
                        end
                        
                        % simple regression <- for path dependent option
                        % we should change it
                        % beta  : regression coefficients
                        beta =zeros(dimensionLS,1);
                        beta = regress(contiValue',regMatrix);
                        regContiValue = regMatrix*beta;
                       
                         %Nominal Regression test
                        regContiValue = regContiValue * 10000;
                                                
%                         exercisedPayoff = regContiValue;
                        exercisedPayoff = contiValue * 10000;
                        % if regContiValue is bigger than exerciseValue
                        % then issue exercise call option
                        for idx_r=1:pathSize
                            if regContiValue(idx_r) >= exerciseValue(idx_r) 
                                exercisedPayoff(idx_r) = exerciseValue(idx_r);
                            end
                        end
                        
                        % we discount the new payoff and 
                        % assign it into the existing callable payoff
                        callable.payoff = model.discountPayoffRisky(currentTime,numMatTime ...
                        ,modelStateT,exercisedPayoff);
                        
                    end
                end

                nonCall.npv = mean(nonCall.payoff);
                out.nonCall = nonCall;
                callable.npv = mean(callable.payoff);
                dfCol.npv = mean(dfCol.payoff);
                dummy1.npv = mean(dummy1.npv);
                dummy2.npv = mean(dummy2.npv);
                out.callable = callable;
                out.dfCol = dfCol;
                out.dummy1 = dummy1;
                out.dummy2 = dummy2;
                out.daycountFraction = schedule.dayCountFraction;
                
        end
        
        
        function out = computePriceClosedSimple(pricer,valueDate)
                security = pricer.m_security;
                numMethod = pricer.m_numMethod;
                model = pricer.m_model;
                schedule = security.couponSchedule;
                
                % numMethodTimeSteps Generate
                mcOneTimeStep = numMethod.mcOneTimeStep;
                outInfo = numMethod.numMethodTimeStepsGenerateDW(valueDate,schedule,mcOneTimeStep);
            
                numMethodInfo.timeStepsInfo = outInfo;

                % mc random number init
                %pathSize = 2^16;
                pathSize = numMethod.pathSize;
                
                pathSize = 1;
                
%                 % generate random numbers and modelstates forward
%                 rng('default');
%                 rng(0);
% 
                numOfFactors = model.numOfFactor;
% 
%                 %Z= randn(numOfFactors*outInfo.totalTimeStepsSize,pathSize/2);
%                 Z= randn(pathSize/2,numOfFactors*outInfo.totalTimeStepsSize);
%                 Z=[Z;-Z];
%                 Z=Z';
%                 
%                 
%                 %TimeInversion
%                 sizeZ = size(Z,1);
%                 A = zeros(sizeZ,sizeZ);
%                 for i=1:sizeZ
%                     A(i,sizeZ + 1 -i) = 1;
%                 end
%                 Z = A*Z;
% 
%                 % we check with the c, so we use generated random numbers
% %                 mc_random = importdata('mc_random.xlsx');
% %                 mc_random = importdata('D:\mc_random_number_new_report.txt');
% %                 Z= mc_random;
%                 
                numMethodInfo.pathSize = pathSize;
                numMethodInfo.numOfFactors = numOfFactors;
                numMethodInfo.totalTimeSteps = outInfo.totalTimeSteps;
                numMethodInfo.totalTimeStepsSize = outInfo.totalTimeStepsSize;
%                 numMethodInfo.Z = Z;
% 
% 
% 
%                 %generate model states for the given timeSteps
%                 modelStates = numMethod.generateModelStates(valueDate,schedule,numMethodInfo,model);

                % numeraire info
                numMatTime = numMethodInfo.totalTimeSteps(end);
                dfNumMat = model.DFRisky(numMatTime/365.0);
                
                % induct backward and calculate cashflow
            
                periodCount = schedule.numOfPeriod;
                
                % cashflow columns initialize
                % 1st nonCall
                nonCall.npv = 0;
                nonCall.cashflow_e = zeros(periodCount + 1,1);
                nonCall.cashflow_npv_e = zeros(periodCount + 1,1);
                nonCall.rangeProb_e = zeros(periodCount + 1,1);
                nonCall.df_e = zeros(periodCount + 1,1);
                nonCall.df_c = zeros(periodCount + 1,1);
                
                nonCall.fwdCD = zeros(periodCount + 1,1);
                nonCall.fwdKTB = zeros(periodCount + 1,1);
                nonCall.fwdSpread = zeros(periodCount + 1,1);
                nonCall.fwdMu = zeros(periodCount + 1,1);
                nonCall.fwdSigma = zeros(periodCount + 1,1);

                % debug information, cashflow & npv per path
                nonCall.payoff = zeros(1,pathSize);
                nonCall.payoffStates.cashflow = zeros(periodCount+1,pathSize);
                nonCall.payoffStates.cashflow_npv = zeros(periodCount+1,pathSize);
                nonCall.payoffStates.df = zeros(periodCount + 1,pathSize);
                
                % intermediate variables
                payoffStateA.cashflow = zeros(1, pathSize);
                payoffStateA.cashflow_npv = zeros(1, pathSize);
                payoffStateA.df = zeros(1,pathSize);
                
                % we calculate cashflow backward from maturityDate to valueDate
                % rangeAccrual coupons are added at the startDate of the Period
                % At maturity

                nominal = security.nominal;
                coupon = security.coupon;
                callYN = security.callYN;
                
                currentTime = numMethodInfo.totalTimeSteps(end);
                totalTimeSteps = numMethodInfo.timeStepsInfo.totalTimeSteps;
                totalTimeStepsSize = numMethodInfo.timeStepsInfo.totalTimeStepsSize;
                if totalTimeStepsSize == 0 || (totalTimeStepsSize == 1 && totalTimeSteps(1) == 0) 
                    out.nonCall = nonCall;
                    return;
                end
                
                currentTimeIdx = totalTimeStepsSize;
                
                currentNodeIdx = periodCount + 1;
                
%                 modelStateT = modelStates(numOfFactors*(totalTimeStepsSize-1)+1:numOfFactors*(totalTimeStepsSize-1)+2,:);
                payoffStateA.cashflow = nominal*ones(1,pathSize);
                %process payoff at maturity
                nonCall.payoffStates.cashflow(currentNodeIdx,:) = payoffStateA.cashflow;
                
%                 payoffStateA.cashflow_npv = model.discountPayoff(currentTime,numMatTime ...
%                     ,modelStateT,payoffStateA.cashflow);
%                 
                payoffStateA.cashflow_npv = model.DFRisky(currentTime/365.0)*payoffStateA.cashflow;
                
                nonCall.payoffStates.cashflow_npv(currentNodeIdx,:) = payoffStateA.cashflow_npv ;
                
%                 payoffStateA.df = model.discountPayoff(currentTime,numMatTime ...
%                     ,modelStateT,1.0*ones(1,pathSize));
                payoffStateA.df = model.DFRisky(currentTime/365.0) *ones(1,pathSize);
                    

                nonCall.payoffStates.df(currentNodeIdx,:) = payoffStateA.df;
                nonCall.payoff = nonCall.payoff + payoffStateA.cashflow_npv;
                
                nonCall.cashflow_e(currentNodeIdx) = mean(payoffStateA.cashflow);
                nonCall.cashflow_npv_e(currentNodeIdx) = mean(payoffStateA.cashflow_npv);
                nonCall.df_e(currentNodeIdx) = mean(payoffStateA.df);
                nonCall.df_c(currentNodeIdx) = model.DFRisky(currentTime/365.0);
                
%                 % callable at maturity is the same as noncallable
%                 callable.payoffStates.cashflow(currentNodeIdx,:) = nonCall.payoffStates.cashflow(currentNodeIdx,:);
%                 callable.payoffStates.cashflow_npv(currentNodeIdx,:) = nonCall.payoffStates.cashflow_npv(currentNodeIdx,:);
%                 callable.payoff = callable.payoff + callable.payoffStates.cashflow_npv(currentNodeIdx,:);
%                 
%                 callable.cashflow_e(currentNodeIdx) = nonCall.cashflow_e(currentNodeIdx);
%                 callable.cashflow_npv_e(currentNodeIdx) = nonCall.cashflow_npv_e(currentNodeIdx);
                  
                lastTimeIdx = currentTimeIdx;
                
                
                % flag for inclusive schedule
%                 simpleYN = false;
                simpleYN = 1;
                
                for i=periodCount:-1:1
                    % past schedule neglect
                    if DateDiff(valueDate,schedule.payDates(i)) >= 0, continue;end 
                    startTimeIdx = numMethodInfo.timeStepsInfo.startTimeIdx(i);
                    endTimeIdx = numMethodInfo.timeStepsInfo.endTimeIdx(i);

                    inclusiveYN = numMethodInfo.timeStepsInfo.inclusiveYN(i);
                     %% FromToInduction start

                    %modelsStates induct backward(lastTimeIdx to endTimeIdx)
                    while endTimeIdx < lastTimeIdx
                        lastTimeIdx = lastTimeIdx -1;
                        if lastTimeIdx == endTimeIdx break;end
                    end

                    currentTimeIdx = lastTimeIdx;
                    currentTime = totalTimeSteps(currentTimeIdx);
                    % now we are at endTimeIdx
                    % variables for coupon valuation initialized
                    ralower = security.lowerS;
                    raupper = security.upperS;
                    tenorLong = security.tenorL;
                    tenorShort = security.tenorS;

                    cD = 0; % cumulated day count fraction
                    cP = 0; 
                    dI = 0; % dummy idx for loop
                    dcfC = 0;
                    dcfN = 0;
                    dD = 0;
                    idxL = zeros(1,pathSize);
                    idxS = zeros(1,pathSize);
                    spreadC = zeros(1,pathSize);
                    spreadN = zeros(1,pathSize);
                    probSpread = zeros(1,pathSize);
                    
                    numOfTimeSteps = numMethodInfo.timeStepsInfo.numOfTimeSteps(i);
                    
                    % if inclusive then we use simpleYN rangeProb
                    %if inclusive then we do not evaluate the last step but
                    % use fixing instead 
                    endTimeStep = 1;
                    if inclusiveYN
%                         simpleYN = true;
                        simpleYN = 1;
                        endTimeStep = 2;
                    end
                    
                    for j=numOfTimeSteps:-1:endTimeStep
                        % evaluate dealdescription
                        % last step is omitted
                        currentTime = totalTimeSteps(currentTimeIdx);
                        %dcfC = currentTime/365.0;
                        dcfC = currentTime;
                        
                        if j~= numOfTimeSteps
                            dD = (dcfN-dcfC);
                        end

                        toDate = AddDate(valueDate,currentTime,'day');
                        tpDate = schedule.payDates(i);
                        spreadCOut = model.CMSCMTSpreadDigitalRangeDate(valueDate,toDate,tpDate,tenorLong,tenorShort,ralower,raupper);
                        spreadC = spreadCOut.value;
                        if j== endTimeStep
                            fwdCD = spreadCOut.cms_fwd;
                            fwdKTB = spreadCOut.cmt_fwd;
                            fwdSpread = spreadCOut.fwdSpread;
                            fwdMu        = spreadCOut.mu;
                            fwdSigma     = spreadCOut.sigma;
                        end
                        %modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:numOfFactors*(currentTimeIdx-1)+2,:);
%                         idxL = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorLong,modelStateT);
%                         idxS = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorShort,modelStateT);
%                         spreadC = idxL - idxS;
%                         probSpread = model.SRPr(spreadC,spreadN,ralower,raupper,simpleYN,pathSize)*dD; 

                        if simpleYN
                            probSpread = spreadC*dD;
                        else
                            probSpread = spreadC*dD;
%                             probSpread = 0.5*(spreadC + spreadN)*dD;
                            
                        end

                        
                        cD = cD + dD;
                        cP = cP + probSpread;

                        dcfN = dcfC;
                        spreadN = spreadC;

                        %induct backward to the next time step
                        % one time backward induction
                        lastTimeIdx = currentTimeIdx;

                        if startTimeIdx < lastTimeIdx
                            lastTimeIdx = lastTimeIdx -1;
                            currentTimeIdx = lastTimeIdx;
                        end


                    end

                    %% FromToInduction end
                    %% AtDate evaluation
                    % Adding a new cashflow to the payoff

                    dcf = schedule.dayCountFraction(i);
                    
                    if inclusiveYN
                        % we add valuation date
                        cD = schedule.dayCount(i);
                        rangeDays =  security.rangeDays(i);
                        payoffStateA.cashflow = nominal * coupon * dcf * (rangeDays + cP)/cD;
                    else
                        payoffStateA.cashflow = nominal * coupon * dcf * cP/cD;
                    end
                    % eventDate(i) = startDate(i) therefore there is no induction in modelStates
                    % i.e. currentTimeIdx has not changed
                    % no need to update modelStateT
                    % modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:end,:);
                    % since cashflow's are added at the startDate(i) we need to
                    % multiply cashflow with the df(startDate(i), payDate(i))
                    payTime = DateDiff(schedule.payDates(i),valueDate);
                    
                    %event is at startDate so we need to discount the
                    %cashflow first from paydate to startdate
%                     df_sp = model.discountFactor(currentTime, payTime,numMatTime,modelStateT);

%                     for k=1:pathSize
%                         payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_sp(k);
%                     end
                    
                    % process payoff
                    
                    currentNodeIdx = i;
                    
                    % 1st nonCall column
                    nonCall.payoffStates.cashflow(currentNodeIdx,:) = payoffStateA.cashflow;
                    payoffStateA.cashflow_npv = payoffStateA.cashflow * model.DFRisky(payTime/365.0);
                    
%                     payoffStateA.cashflow_npv = model.discountPayoff(currentTime,numMatTime ...
%                     ,modelStateT,payoffStateA.cashflow);
                    nonCall.payoffStates.cashflow_npv(currentNodeIdx,:) = payoffStateA.cashflow_npv;
                    payoffStateA.df = model.DFRisky(payTime/365.0);
%                     payoffStateA.df = model.discountPayoff(currentTime,numMatTime ...
%                     ,modelStateT,1.0*ones(1,pathSize));
                    nonCall.payoffStates.df(currentNodeIdx,:) = payoffStateA.df;
                    
                    nonCall.payoff = nonCall.payoff + payoffStateA.cashflow_npv;
                    
                    nonCall.cashflow_e(currentNodeIdx) = mean(payoffStateA.cashflow);
                    nonCall.rangeProb_e(currentNodeIdx) = mean(nonCall.cashflow_e(currentNodeIdx)/(nominal * coupon * dcf));
                    nonCall.cashflow_npv_e(currentNodeIdx) = mean(payoffStateA.cashflow_npv);

                    nonCall.df_e(currentNodeIdx) = mean(payoffStateA.df);
                    nonCall.df_c(currentNodeIdx) = model.DFRisky(currentTime/365.0);
                    
                    nonCall.fwdCD(currentNodeIdx) = fwdCD;
                    nonCall.fwdKTB(currentNodeIdx) = fwdKTB;
                    
                    nonCall.fwdSpread(currentNodeIdx) = fwdSpread;
                    nonCall.fwdMu(currentNodeIdx) = fwdMu;
                    nonCall.fwdSigma(currentNodeIdx) = fwdSigma;

                end

                nonCall.npv = mean(nonCall.payoff);
                out.pricerInfo.nonCall = nonCall;
                
%                 callable.npv = mean(callable.payoff);
%                 out.pricerInfo.callable = callable;
                out.daycountFraction = schedule.dayCountFraction;
                
        end
        

        
        
        
        
    end
    
end



In [ ]:
classdef DailyWeeklyRiskyPricer_CDRA < H_RootObject
    %Pricer(Security,Model,NumMethod)
    %Prcer is constructed from Security, Model, NumMethod 
    
    properties
        m_security
        m_model
        m_numMethod
        overHedgeYN
    end
    
    methods
        function pricer = DailyWeeklyRiskyPricer_CDRA(security,model,numMethod,overHedgeYN)
                pricer.m_security = security;
                pricer.m_model = model;
                pricer.m_numMethod = numMethod;
                pricer.overHedgeYN = overHedgeYN;
        end
        
        function out = computePriceMCCallableGeneric(pricer,valueDate)
                security = pricer.m_security;
                numMethod = pricer.m_numMethod;
                model = pricer.m_model;
                schedule = security.couponSchedule;
                callSchedule = security.callSchedule;
                
                % numMethodTimeSteps Generate
                mcOneTimeStep = numMethod.mcOneTimeStep;
                outInfo = numMethod.numMethodTimeStepsGenerateDW(valueDate,schedule,mcOneTimeStep);
            
                numMethodInfo.timeStepsInfo = outInfo;

                % mc random number init
                %pathSize = 2^16;
                pathSize = numMethod.pathSize;

                % generate random numbers and modelstates forward
                rng('default');
                rng(0);

                % LGM1F : one factor 
                numOfFactors = 1;

                %Z= randn(numOfFactors*outInfo.totalTimeStepsSize,pathSize/2);
                Z= randn(pathSize/2,numOfFactors*outInfo.totalTimeStepsSize);
                Z=[Z;-Z];
                Z=Z';
                
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;

                % we check with the c code, so we use generated random numbers
%                 mc_random = importdata('mc_random.xlsx');
%                 mc_random = importdata('D:\mc_random_number_new_report.txt');
%                 Z= mc_random;
                
                numMethodInfo.pathSize = pathSize;
                numMethodInfo.numOfFactors = numOfFactors;
                numMethodInfo.totalTimeSteps = outInfo.totalTimeSteps;
                numMethodInfo.totalTimeStepsSize = outInfo.totalTimeStepsSize;
                numMethodInfo.Z = Z;



                %generate model states for the given timeSteps
                modelStates = numMethod.generateModelStates1(valueDate,schedule,numMethodInfo,model);

                % numeraire info
                numMatTime = numMethodInfo.totalTimeSteps(end);
                days = DateDiff(schedule.payDates(schedule.numOfPeriod),valueDate);
                numMatTime = max(numMatTime, days);
                dfNumMat = model.DFRisky(numMatTime/365.0);
                
                numeraire.numMatTime = numMatTime;
                numeraire.dfNumMat = dfNumMat;
                
                % induct backward and calculate cashflow
            
                periodCount = schedule.numOfPeriod;
                
                % pricer single info initialize
                
                nonCall = PricerInfo(periodCount +1, pathSize);
                callable = PricerInfo(periodCount +1, pathSize);
                dfCol = PricerInfo(periodCount +1, pathSize);
                
%                 dummy1 = PricerInfo(periodCount +1, pathSize);
%                 dummy2 = PricerInfo(periodCount +1, pathSize);
                
                
                % intermediate variables
                payoffStateA.cashflow = zeros(1, pathSize);
                payoffStateA.cashflow_npv = zeros(1, pathSize);
                
            
                % we calculate cashflow backward from maturityDate to valueDate
                % rangeAccrual coupons are added at the startDate of the Period
                % At maturity

                nominal = security.nominal;
                coupon = security.coupon;
                callYN = security.callYN;
                
                ralower = security.lower;
                raupper = security.upper;
                
                tenorCD = security.tenor;
                
                currentTime = numMethodInfo.totalTimeSteps(end);
                totalTimeStepsSize = numMethodInfo.totalTimeStepsSize;
                currentTimeIdx = totalTimeStepsSize;
                
                currentNodeIdx = periodCount + 1;
                
                modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1,:);
                payoffStateA.cashflow = nominal*ones(1,pathSize);
                
                payTime = DateDiff(schedule.payDates(schedule.numOfPeriod),valueDate);
                df_ep = model.discountFactorRisky(currentTime, payTime,numMatTime,modelStateT);
                for k=1:pathSize
                    payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                end
                
                %process payoff at maturity
                nonCall.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                
                callable.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                
                %df column
                payoffStateA.cashflow = ones(1,pathSize);
                for k=1:pathSize
                    payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                end
                
                dfCol.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                                 
%                 %dummy column
%                 payoffStateA.cashflow = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorLong,modelStateT);
%                 dummy1.ProcessPayoff(model,currentTime,numeraire, ...
%                                  modelStateT,payoffStateA.cashflow,currentNodeIdx);
% 
%                 payoffStateA.cashflow = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorShort,modelStateT);
%                 dummy2.ProcessPayoff(model,currentTime,numeraire, ...
%                                  modelStateT,payoffStateA.cashflow,currentNodeIdx);
                                 
                  

                lastTimeIdx = currentTimeIdx;
                totalTimeSteps = numMethodInfo.timeStepsInfo.totalTimeSteps;
                
                % flag for inclusive schedule
%                 simpleYN = false;
%                 simpleYN = true;
                if pricer.overHedgeYN == true 
                    simpleYN = 2;
                else
                    simpleYN = 1;
                end
                
                for i=periodCount:-1:1
                    % past schedule neglect
                    if DateDiff(valueDate,schedule.payDates(i)) >= 0, continue;end 
                    startTimeIdx = numMethodInfo.timeStepsInfo.startTimeIdx(i);
                    endTimeIdx = numMethodInfo.timeStepsInfo.endTimeIdx(i);

                    inclusiveYN = numMethodInfo.timeStepsInfo.inclusiveYN(i);
                 %% FromToInduction start

                    %modelsStates induct backward(lastTimeIdx to endTimeIdx)
                    while endTimeIdx < lastTimeIdx
                        lastTimeIdx = lastTimeIdx -1;
                        if lastTimeIdx == endTimeIdx break;end
                    end

                    currentTimeIdx = lastTimeIdx;
                    currentTime = totalTimeSteps(currentTimeIdx);
                    % now we are at endTimeIdx
                    % variables for coupon valuation initialized
                    

                    cD = 0; % cumulated day count fraction
                    cP = 0; 
                    dI = 0; % dummy idx for loop
                    dcfC = 0;
                    dcfN = 0;
                    dD = 0;
                    idxCD = zeros(1,pathSize);
                    
                    idxC = zeros(1,pathSize);
                    idxN = zeros(1,pathSize);
                
                    probSpread = zeros(1,pathSize);
                    
                    numOfTimeSteps = numMethodInfo.timeStepsInfo.numOfTimeSteps(i);
                    
                    % if inclusive then we use simpleYN rangeProb
                    %if inclusive then we do not evaluate the last step but
                    % use fixing instead 
                    endTimeStep = 1;
                    if inclusiveYN
%                         simpleYN = true;
                        simpleYN = 1;
                        endTimeStep = 2;
                        if pricer.overHedgeYN == true 
                            simpleYN = 2;
                        end
                    end
                    
                    for j=numOfTimeSteps:-1:endTimeStep
                        % evaluate dealdescription
                        % last step is omitted
                        currentTime = totalTimeSteps(currentTimeIdx);
                        %dcfC = currentTime/365.0;
                        dcfC = currentTime;
                        
                        if j~= numOfTimeSteps
                            dD = (dcfN-dcfC);
                        end

                        modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1,:);
                        
                        idxCD = model.Libor_SimpleDate(valueDate,currentTime,currentTime,numMatTime, tenorCD,modelStateT);
                        idxC = idxCD;
                        probSpread = model.SRPr(idxC,idxN,ralower,raupper,simpleYN,pathSize)*dD;
                    
                        cD = cD + dD;
                        cP = cP + probSpread;

                        dcfN = dcfC;
                        
                        idxN = idxC;
                        
                        %induct backward to the next time step
                        % one time backward induction
                        lastTimeIdx = currentTimeIdx;

                        if startTimeIdx < lastTimeIdx
                            lastTimeIdx = lastTimeIdx -1;
                            currentTimeIdx = lastTimeIdx;
                        end


                    end

                    %% FromToInduction end
        % if the current period is inclusive, then the start date is not evaluated in the above for loop.
		% instead the fixing will be treated separately below.
		% however current_time_idx is updated at the end of the for loop.
		% so we need to update model_states accordingly.
		% if model_states were updated previously then the following additionaly step doesn't affect the result anyway.
                    currentTime = totalTimeSteps(currentTimeIdx);
                    modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1,:);
                    
                    %% AtDate evaluation
                    % Adding a new cashflow to the payoff

                    dcf = schedule.dayCountFraction(i);
                    
                    if inclusiveYN
                        % we add valuation date
                        cD = schedule.dayCount(i);
                        rangeDays =  security.rangeDays(i);
                        payoffStateA.cashflow = nominal * coupon * dcf * (rangeDays(i) + cP)/cD;
                    else
                        payoffStateA.cashflow = nominal * coupon * dcf * cP/cD;
                    end
                    % eventDate(i) = startDate(i) therefore there is no induction in modelStates
                    % i.e. currentTimeIdx has not changed
                    % no need to update modelStateT
                    % modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:end,:);
                    % since cashflow's are added at the startDate(i) we need to
                    % multiply cashflow with the df(startDate(i), payDate(i))
                    payTime = DateDiff(schedule.payDates(i),valueDate);
                    
                    %event is at startDate so we need to discount the
                    %cashflow first from paydate to startdate
                    df_ep = model.discountFactorRisky(currentTime, payTime,numMatTime,modelStateT);

                    for k=1:pathSize
                        payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                    end
                    
                    % process payoff
                    
                    currentNodeIdx = i;
                    
                    nonCall.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                
                    callable.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                    
                    %df column
                    payoffStateA.cashflow = ones(1,pathSize);
                    for k=1:pathSize
                        payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                    end
                    
                    dfCol.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                                 
%                     %dummy column
%                     payoffStateA.cashflow = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorLong,modelStateT);
%                     dummy1.ProcessPayoff(model,currentTime,numeraire, ...
%                                      modelStateT,payoffStateA.cashflow,currentNodeIdx);
%                     
%                     payoffStateA.cashflow = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorShort,modelStateT);
%                     dummy2.ProcessPayoff(model,currentTime,numeraire, ...
%                                      modelStateT,payoffStateA.cashflow,currentNodeIdx);
                    
                    
                    % if exercise is possible
                    if i == 1
                        dummaaa = 1;
                    end
                    
                    % call event happens  at the start of the period
                    % at call date, by default, call option is expired
                    
                    if callYN(i) && (DateDiff(valueDate,callSchedule.resetDates(i)) < 0)
                        % 1.undiscount callable.payoff
                        % 2.build the state variables matrix
                        % 3.regress 1's with the given state variables
                        % (model states 1,2)
                        % 4. exercise per each path in
                        % discount the new payoff and assign into callable
                        % column
                        
                        %1 undiscount callable.payoff and get the
                        %contiValue
                       
                        %exercise value
                        exerciseValue = security.nominal*ones(1,pathSize);
                        payTime = DateDiff(callSchedule.payDates(i),valueDate);
                        %event is at startDate so we need to discount the
                        %cashflow first from paydate to startdate
                        df_ep = model.discountFactorRisky(currentTime, payTime,numMatTime,modelStateT);
                        for k=1:pathSize
                            exerciseValue(k) = exerciseValue(k) * df_ep(k);
                        end
                        
                        contiValue = zeros(1,pathSize);
                        onePayoff = ones(1,pathSize);
                        onePayoffD = model.discountPayoffRisky(currentTime,numMatTime ...
                        ,modelStateT,onePayoff);
                        contiValue = callable.payoff./onePayoffD;
                        
                        %Nominal Regression test
                        contiValue = contiValue/10000;
                        
                        % build the state variabls matrix
                        decisionVar1 = modelStateT(1,:);
%                         decisionVar2 = modelStateT(2,:);
                        % we use upto 2nd order polynomial as regression
                        % equatioin
                        dimensionLS = 3;
                        regMatrix = ones(pathSize,dimensionLS);
                        
                        for idx_r=1:pathSize
                            regMatrix(idx_r,2) = decisionVar1(idx_r);
                            regMatrix(idx_r,3) = decisionVar1(idx_r)*decisionVar1(idx_r);
                        end
                        
                        % simple regression <- for path dependent option
                        % we should change it
                        % beta  : regression coefficients
                        beta =zeros(dimensionLS,1);
                        beta = regress(contiValue',regMatrix);
                        regContiValue = regMatrix*beta;
                       
                         %Nominal Regression test
                        regContiValue = regContiValue * 10000;
                        
                        exercisedPayoff = regContiValue;
                        % if regContiValue is bigger than exerciseValue
                        % then issue exercise call option
                        for idx_r=1:pathSize
                            if regContiValue(idx_r) >= exerciseValue(idx_r) 
                                exercisedPayoff(idx_r) = exerciseValue(idx_r);
                            end
                        end
                        
                        % we discount the new payoff and 
                        % assign it into the existing callable payoff
                        callable.payoff = model.discountPayoffRisky(currentTime,numMatTime ...
                        ,modelStateT,exercisedPayoff);
                        
                    end
                end

                nonCall.npv = mean(nonCall.payoff);
                out.nonCall = nonCall;
                callable.npv = mean(callable.payoff);
                out.callable = callable;
                out.dfCol = dfCol;
%                 out.dummy1 = dummy1;
%                 out.dummy2 = dummy2;
                out.daycountFraction = schedule.dayCountFraction;
                
        end
        
        function out = computePriceMCCallable(pricer,valueDate)
                security = pricer.m_security;
                numMethod = pricer.m_numMethod;
                model = pricer.m_model;
                schedule = security.couponSchedule;
                callSchedule = security.callSchedule;
                
                % numMethodTimeSteps Generate
                mcOneTimeStep = numMethod.mcOneTimeStep;
                outInfo = numMethod.numMethodTimeStepsGenerateDW(valueDate,schedule,mcOneTimeStep);
            
                numMethodInfo.timeStepsInfo = outInfo;

                % mc random number init
                %pathSize = 2^16;
                pathSize = numMethod.pathSize;

                % generate random numbers and modelstates forward
                rng('default');
                rng(0);

                % LGM1F : one factor 
                numOfFactors = 1;

                %Z= randn(numOfFactors*outInfo.totalTimeStepsSize,pathSize/2);
                Z= randn(pathSize/2,numOfFactors*outInfo.totalTimeStepsSize);
                Z=[Z;-Z];
                Z=Z';
                
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;

                % we check with the c code, so we use generated random numbers
%                 mc_random = importdata('mc_random.xlsx');
%                 mc_random = importdata('D:\mc_random_number_new_report.txt');
%                 Z= mc_random;
                
                numMethodInfo.pathSize = pathSize;
                numMethodInfo.numOfFactors = numOfFactors;
                numMethodInfo.totalTimeSteps = outInfo.totalTimeSteps;
                numMethodInfo.totalTimeStepsSize = outInfo.totalTimeStepsSize;
                numMethodInfo.Z = Z;



                %generate model states for the given timeSteps
                modelStates = numMethod.generateModelStates1(valueDate,schedule,numMethodInfo,model);

                % numeraire info
                numMatTime = numMethodInfo.totalTimeSteps(end);
                days = DateDiff(schedule.payDates(schedule.numOfPeriod),valueDate);
                numMatTime = max(numMatTime, days);
                dfNumMat = model.DFRisky(numMatTime/365.0);
                
                numeraire.numMatTime = numMatTime;
                numeraire.dfNumMat = dfNumMat;
                
                % induct backward and calculate cashflow
            
                periodCount = schedule.numOfPeriod;
                
                % pricer single info initialize
                
                nonCall = PricerInfo(periodCount +1, pathSize);
                callable = PricerInfo(periodCount +1, pathSize);
                dfCol = PricerInfo(periodCount +1, pathSize);
                
%                 dummy1 = PricerInfo(periodCount +1, pathSize);
%                 dummy2 = PricerInfo(periodCount +1, pathSize);
                
                
                % intermediate variables
                payoffStateA.cashflow = zeros(1, pathSize);
                payoffStateA.cashflow_npv = zeros(1, pathSize);
                
            
                % we calculate cashflow backward from maturityDate to valueDate
                % rangeAccrual coupons are added at the startDate of the Period
                % At maturity

                nominal = security.nominal;
                coupon = security.coupon;
                callYN = security.callYN;
                
                ralower = security.lower;
                raupper = security.upper;
                
                tenorCD = security.tenor;
                
                currentTime = numMethodInfo.totalTimeSteps(end);
                totalTimeStepsSize = numMethodInfo.totalTimeStepsSize;
                currentTimeIdx = totalTimeStepsSize;
                
                currentNodeIdx = periodCount + 1;
                
                modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1,:);
                payoffStateA.cashflow = nominal*ones(1,pathSize);
                
                payTime = DateDiff(schedule.payDates(schedule.numOfPeriod),valueDate);
                df_ep = model.discountFactorRisky(currentTime, payTime,numMatTime,modelStateT);
                for k=1:pathSize
                    payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                end
                
                %process payoff at maturity
                nonCall.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                
                callable.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                
                %df column
                payoffStateA.cashflow = ones(1,pathSize);
                for k=1:pathSize
                    payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                end
                
                dfCol.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                                 
%                 %dummy column
%                 payoffStateA.cashflow = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorLong,modelStateT);
%                 dummy1.ProcessPayoff(model,currentTime,numeraire, ...
%                                  modelStateT,payoffStateA.cashflow,currentNodeIdx);
% 
%                 payoffStateA.cashflow = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorShort,modelStateT);
%                 dummy2.ProcessPayoff(model,currentTime,numeraire, ...
%                                  modelStateT,payoffStateA.cashflow,currentNodeIdx);
                                 
                  

                lastTimeIdx = currentTimeIdx;
                totalTimeSteps = numMethodInfo.timeStepsInfo.totalTimeSteps;
                
                % flag for inclusive schedule
%                 simpleYN = false;
%                 simpleYN = true;
                if pricer.overHedgeYN == true 
                    simpleYN = 2;
                else
                    simpleYN = 1;
                end
                
                for i=periodCount:-1:1
                    % past schedule neglect
                    if DateDiff(valueDate,schedule.payDates(i)) >= 0, continue;end 
                    startTimeIdx = numMethodInfo.timeStepsInfo.startTimeIdx(i);
                    endTimeIdx = numMethodInfo.timeStepsInfo.endTimeIdx(i);

                    inclusiveYN = numMethodInfo.timeStepsInfo.inclusiveYN(i);
                 %% FromToInduction start

                    %modelsStates induct backward(lastTimeIdx to endTimeIdx)
                    while endTimeIdx < lastTimeIdx
                        lastTimeIdx = lastTimeIdx -1;
                        if lastTimeIdx == endTimeIdx break;end
                    end

                    currentTimeIdx = lastTimeIdx;
                    currentTime = totalTimeSteps(currentTimeIdx);
                    % now we are at endTimeIdx
                    % variables for coupon valuation initialized
                    

                    cD = 0; % cumulated day count fraction
                    cP = 0; 
                    dI = 0; % dummy idx for loop
                    dcfC = 0;
                    dcfN = 0;
                    dD = 0;
                    idxCD = zeros(1,pathSize);
                    
                    idxC = zeros(1,pathSize);
                    idxN = zeros(1,pathSize);
                
                    probSpread = zeros(1,pathSize);
                    
                    numOfTimeSteps = numMethodInfo.timeStepsInfo.numOfTimeSteps(i);
                    
                    % if inclusive then we use simpleYN rangeProb
                    %if inclusive then we do not evaluate the last step but
                    % use fixing instead 
                    endTimeStep = 1;
                    if inclusiveYN
%                         simpleYN = true;
                        simpleYN = 1;
                        endTimeStep = 2;
                        if pricer.overHedgeYN == true 
                            simpleYN = 2;
                        end
                    end
                    
                    for j=numOfTimeSteps:-1:endTimeStep
                        % evaluate dealdescription
                        % last step is omitted
                        currentTime = totalTimeSteps(currentTimeIdx);
                        %dcfC = currentTime/365.0;
                        dcfC = currentTime;
                        
                        if j~= numOfTimeSteps
                            dD = (dcfN-dcfC);
                        end

                        modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1,:);
                        
                        idxCD = model.Libor_SimpleDate(valueDate,currentTime,currentTime,numMatTime, tenorCD,modelStateT);
                        idxC = idxCD;
                        probSpread = model.SRPr(idxC,idxN,ralower,raupper,simpleYN,pathSize)*dD;
                    
                        cD = cD + dD;
                        cP = cP + probSpread;

                        dcfN = dcfC;
                        
                        idxN = idxC;
                        
                        %induct backward to the next time step
                        % one time backward induction
                        lastTimeIdx = currentTimeIdx;

                        if startTimeIdx < lastTimeIdx
                            lastTimeIdx = lastTimeIdx -1;
                            currentTimeIdx = lastTimeIdx;
                        end


                    end

                    %% FromToInduction end
        % if the current period is inclusive, then the start date is not evaluated in the above for loop.
		% instead the fixing will be treated separately below.
		% however current_time_idx is updated at the end of the for loop.
		% so we need to update model_states accordingly.
		% if model_states were updated previously then the following additionaly step doesn't affect the result anyway.
                    currentTime = totalTimeSteps(currentTimeIdx);
                    modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1,:);
                    
                    %% AtDate evaluation
                    % Adding a new cashflow to the payoff

                    dcf = schedule.dayCountFraction(i);
                    
                    if inclusiveYN
                        % we add valuation date
                        cD = schedule.dayCount(i);
                        rangeDays =  security.rangeDays(i);
                        payoffStateA.cashflow = nominal * coupon * dcf * (rangeDays + cP)/cD;
                    else
                        payoffStateA.cashflow = nominal * coupon * dcf * cP/cD;
                    end
                    % eventDate(i) = startDate(i) therefore there is no induction in modelStates
                    % i.e. currentTimeIdx has not changed
                    % no need to update modelStateT
                    % modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:end,:);
                    % since cashflow's are added at the startDate(i) we need to
                    % multiply cashflow with the df(startDate(i), payDate(i))
                    payTime = DateDiff(schedule.payDates(i),valueDate);
                    
                    %event is at startDate so we need to discount the
                    %cashflow first from paydate to startdate
                    df_ep = model.discountFactorRisky(currentTime, payTime,numMatTime,modelStateT);

                    for k=1:pathSize
                        payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                    end
                    
                    % process payoff
                    
                    currentNodeIdx = i;
                    
                    nonCall.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                
                    callable.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                    
                    %df column
                    payoffStateA.cashflow = ones(1,pathSize);
                    for k=1:pathSize
                        payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                    end
                    
                    dfCol.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                                 
%                     %dummy column
%                     payoffStateA.cashflow = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorLong,modelStateT);
%                     dummy1.ProcessPayoff(model,currentTime,numeraire, ...
%                                      modelStateT,payoffStateA.cashflow,currentNodeIdx);
%                     
%                     payoffStateA.cashflow = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorShort,modelStateT);
%                     dummy2.ProcessPayoff(model,currentTime,numeraire, ...
%                                      modelStateT,payoffStateA.cashflow,currentNodeIdx);
                    
                    
                    % if exercise is possible
                    if i == 1
                        dummaaa = 1;
                    end
                    
                    % call event happens  at the start of the period
                    % at call date, by default, call option is expired
                    
                    if callYN(i) && (DateDiff(valueDate,callSchedule.resetDates(i)) < 0)
                        % 1.undiscount callable.payoff
                        % 2.build the state variables matrix
                        % 3.regress 1's with the given state variables
                        % (model states 1,2)
                        % 4. exercise per each path in
                        % discount the new payoff and assign into callable
                        % column
                        
                        %1 undiscount callable.payoff and get the
                        %contiValue
                       
                        %exercise value
%                         exerciseValue = security.nominal*ones(1,pathSize);
                        exerciseValue = security.nominal*security.callStrike*ones(1,pathSize);
                        
                        payTime = DateDiff(callSchedule.payDates(i),valueDate);
                        %event is at startDate so we need to discount the
                        %cashflow first from paydate to startdate
                        df_ep = model.discountFactorRisky(currentTime, payTime,numMatTime,modelStateT);
                        for k=1:pathSize
                            exerciseValue(k) = exerciseValue(k) * df_ep(k);
                        end
                        
                        contiValue = zeros(1,pathSize);
                        onePayoff = ones(1,pathSize);
                        onePayoffD = model.discountPayoffRisky(currentTime,numMatTime ...
                        ,modelStateT,onePayoff);
                        contiValue = callable.payoff./onePayoffD;
                        
                        %Nominal Regression test
                        contiValue = contiValue/10000;
                        
                        % build the state variabls matrix
                        decisionVar1 = modelStateT(1,:);
%                         decisionVar2 = modelStateT(2,:);
                        % we use upto 2nd order polynomial as regression
                        % equatioin
                        dimensionLS = 3;
                        regMatrix = ones(pathSize,dimensionLS);
                        
                        for idx_r=1:pathSize
                            regMatrix(idx_r,2) = decisionVar1(idx_r);
                            regMatrix(idx_r,3) = decisionVar1(idx_r)*decisionVar1(idx_r);
                        end
                        
                        % simple regression <- for path dependent option
                        % we should change it
                        % beta  : regression coefficients
                        beta =zeros(dimensionLS,1);
                        beta = regress(contiValue',regMatrix);
                        regContiValue = regMatrix*beta;
                       
                         %Nominal Regression test
                        regContiValue = regContiValue * 10000;
                        
                        exercisedPayoff = regContiValue;
                        % if regContiValue is bigger than exerciseValue
                        % then issue exercise call option
                        for idx_r=1:pathSize
                            if regContiValue(idx_r) >= exerciseValue(idx_r) 
                                exercisedPayoff(idx_r) = exerciseValue(idx_r);
                            end
                        end
                        
                        % we discount the new payoff and 
                        % assign it into the existing callable payoff
                        callable.payoff = model.discountPayoffRisky(currentTime,numMatTime ...
                        ,modelStateT,exercisedPayoff);
                        
                    end
                end

                nonCall.npv = mean(nonCall.payoff);
                out.nonCall = nonCall;
                callable.npv = mean(callable.payoff);
                out.callable = callable;
                out.dfCol = dfCol;
%                 out.dummy1 = dummy1;
%                 out.dummy2 = dummy2;
                out.daycountFraction = schedule.dayCountFraction;
                
        end
        
        function out = computePriceClosedSimple(pricer,valueDate)
                security = pricer.m_security;
                numMethod = pricer.m_numMethod;
                model = pricer.m_model;
                schedule = security.couponSchedule;
                
                % numMethodTimeSteps Generate
                mcOneTimeStep = numMethod.mcOneTimeStep;
                outInfo = numMethod.numMethodTimeStepsGenerateDW(valueDate,schedule,mcOneTimeStep);
            
                numMethodInfo.timeStepsInfo = outInfo;

                % mc random number init
                %pathSize = 2^16;
                pathSize = numMethod.pathSize;
                
                pathSize = 1;
                
%                 % generate random numbers and modelstates forward
%                 rng('default');
%                 rng(0);
% 
                numOfFactors = 1;
% 
%                 %Z= randn(numOfFactors*outInfo.totalTimeStepsSize,pathSize/2);
%                 Z= randn(pathSize/2,numOfFactors*outInfo.totalTimeStepsSize);
%                 Z=[Z;-Z];
%                 Z=Z';
%                 
%                 
%                 %TimeInversion
%                 sizeZ = size(Z,1);
%                 A = zeros(sizeZ,sizeZ);
%                 for i=1:sizeZ
%                     A(i,sizeZ + 1 -i) = 1;
%                 end
%                 Z = A*Z;
% 
%                 % we check with the c, so we use generated random numbers
% %                 mc_random = importdata('mc_random.xlsx');
% %                 mc_random = importdata('D:\mc_random_number_new_report.txt');
% %                 Z= mc_random;
%                 
                numMethodInfo.pathSize = pathSize;
                numMethodInfo.numOfFactors = numOfFactors;
                numMethodInfo.totalTimeSteps = outInfo.totalTimeSteps;
                numMethodInfo.totalTimeStepsSize = outInfo.totalTimeStepsSize;
%                 numMethodInfo.Z = Z;
% 
% 
% 
%                 %generate model states for the given timeSteps
%                 modelStates = numMethod.generateModelStates(valueDate,schedule,numMethodInfo,model);

                % numeraire info
                numMatTime = numMethodInfo.totalTimeSteps(end);
                dfNumMat = model.DF(numMatTime/365.0);
                
                % induct backward and calculate cashflow
            
                periodCount = schedule.numOfPeriod;
                
                % cashflow columns initialize
                % 1st nonCall
                nonCall.npv = 0;
                nonCall.cashflow_e = zeros(periodCount + 1,1);
                nonCall.cashflow_npv_e = zeros(periodCount + 1,1);
                nonCall.rangeProb_e = zeros(periodCount + 1,1);
                nonCall.df_e = zeros(periodCount + 1,1);
                nonCall.df_c = zeros(periodCount + 1,1);

                % debug information, cashflow & npv per path
                nonCall.payoff = zeros(1,pathSize);
                nonCall.payoffStates.cashflow = zeros(periodCount+1,pathSize);
                nonCall.payoffStates.cashflow_npv = zeros(periodCount+1,pathSize);
                nonCall.payoffStates.df = zeros(periodCount + 1,pathSize);
                
                % intermediate variables
                payoffStateA.cashflow = zeros(1, pathSize);
                payoffStateA.cashflow_npv = zeros(1, pathSize);
                payoffStateA.df = zeros(1,pathSize);
                
                % we calculate cashflow backward from maturityDate to valueDate
                % rangeAccrual coupons are added at the startDate of the Period
                % At maturity

                nominal = security.nominal;
                coupon = security.coupon;
                callYN = security.callYN;
                
                ralower = security.lower;
                raupper = security.upper;
                
                tenorCD = security.tenor;
                
                currentTime = numMethodInfo.totalTimeSteps(end);
                totalTimeSteps = numMethodInfo.timeStepsInfo.totalTimeSteps;
                totalTimeStepsSize = numMethodInfo.timeStepsInfo.totalTimeStepsSize;
                if totalTimeStepsSize == 0 || (totalTimeStepsSize == 1 && totalTimeSteps(1) == 0) 
                    out.nonCall = nonCall;
                    return;
                end
                
                currentTimeIdx = totalTimeStepsSize;
                
                currentNodeIdx = periodCount + 1;
                
%                 modelStateT = modelStates(numOfFactors*(totalTimeStepsSize-1)+1:numOfFactors*(totalTimeStepsSize-1)+2,:);
                payoffStateA.cashflow = nominal*ones(1,pathSize);
                %process payoff at maturity
                nonCall.payoffStates.cashflow(currentNodeIdx,:) = payoffStateA.cashflow;
                
%                 payoffStateA.cashflow_npv = model.discountPayoff(currentTime,numMatTime ...
%                     ,modelStateT,payoffStateA.cashflow);
%                 
                payoffStateA.cashflow_npv = model.DFRisky(currentTime/365.0)*payoffStateA.cashflow;
                
                nonCall.payoffStates.cashflow_npv(currentNodeIdx,:) = payoffStateA.cashflow_npv ;
                
%                 payoffStateA.df = model.discountPayoff(currentTime,numMatTime ...
%                     ,modelStateT,1.0*ones(1,pathSize));
                payoffStateA.df = model.DFRisky(currentTime/365.0) *ones(1,pathSize);
                    

                nonCall.payoffStates.df(currentNodeIdx,:) = payoffStateA.df;
                nonCall.payoff = nonCall.payoff + payoffStateA.cashflow_npv;
                
                nonCall.cashflow_e(currentNodeIdx) = mean(payoffStateA.cashflow);
                nonCall.cashflow_npv_e(currentNodeIdx) = mean(payoffStateA.cashflow_npv);
                nonCall.df_e(currentNodeIdx) = mean(payoffStateA.df);
                nonCall.df_c(currentNodeIdx) = model.DFRisky(currentTime/365.0);
                
%                 % callable at maturity is the same as noncallable
%                 callable.payoffStates.cashflow(currentNodeIdx,:) = nonCall.payoffStates.cashflow(currentNodeIdx,:);
%                 callable.payoffStates.cashflow_npv(currentNodeIdx,:) = nonCall.payoffStates.cashflow_npv(currentNodeIdx,:);
%                 callable.payoff = callable.payoff + callable.payoffStates.cashflow_npv(currentNodeIdx,:);
%                 
%                 callable.cashflow_e(currentNodeIdx) = nonCall.cashflow_e(currentNodeIdx);
%                 callable.cashflow_npv_e(currentNodeIdx) = nonCall.cashflow_npv_e(currentNodeIdx);
                  
                lastTimeIdx = currentTimeIdx;
                
                
                % flag for inclusive schedule
%                 simpleYN = false;
                simpleYN = 1;
                
                for i=periodCount:-1:1
                    % past schedule neglect
                    if DateDiff(valueDate,schedule.payDates(i)) >= 0, continue;end 
                    startTimeIdx = numMethodInfo.timeStepsInfo.startTimeIdx(i);
                    endTimeIdx = numMethodInfo.timeStepsInfo.endTimeIdx(i);

                    inclusiveYN = numMethodInfo.timeStepsInfo.inclusiveYN(i);
                     %% FromToInduction start

                    %modelsStates induct backward(lastTimeIdx to endTimeIdx)
                    while endTimeIdx < lastTimeIdx
                        lastTimeIdx = lastTimeIdx -1;
                        if lastTimeIdx == endTimeIdx break;end
                    end

                    currentTimeIdx = lastTimeIdx;
                    currentTime = totalTimeSteps(currentTimeIdx);
                    % now we are at endTimeIdx
                    % variables for coupon valuation initialized
%                     ralower = security.lower;
%                     raupper = security.upper;
%                     tenorLong = security.tenorL;
%                     tenorShort = security.tenorS;

                    cD = 0; % cumulated day count fraction
                    cP = 0; 
                    dI = 0; % dummy idx for loop
                    dcfC = 0;
                    dcfN = 0;
                    dD = 0;
                    idxCD = zeros(1,pathSize);
                    
                    idxC = zeros(1,pathSize);
                    idxN = zeros(1,pathSize);
                    
                    probSpread = zeros(1,pathSize);
                    
                    numOfTimeSteps = numMethodInfo.timeStepsInfo.numOfTimeSteps(i);
                    
                    % if inclusive then we use simpleYN rangeProb
                    %if inclusive then we do not evaluate the last step but
                    % use fixing instead 
                    endTimeStep = 1;
                    if inclusiveYN
%                         simpleYN = true;
                        simpleYN = 1;
                        endTimeStep = 2;
                    end
                    
                    for j=numOfTimeSteps:-1:endTimeStep
                        % evaluate dealdescription
                        % last step is omitted
                        currentTime = totalTimeSteps(currentTimeIdx);
                        %dcfC = currentTime/365.0;
                        dcfC = currentTime;
                        
                        if j~= numOfTimeSteps
                            dD = (dcfN-dcfC);
                        end

                        toDate = AddDate(valueDate,currentTime,'day');
                        tpDate = schedule.payDates(i);
                        
                                                                     
                        spreadC = model.CDDigitalRangeDate(valueDate,toDate,tpDate,tenorCD,0.25,ralower,raupper);
%                         spreadC = model.CMSSpreadDigitalRangeDate(valueDate,toDate,tpDate,tenorLong,tenorShort,0.25,ralower,raupper);
                        
                        %modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:numOfFactors*(currentTimeIdx-1)+2,:);
%                         idxL = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorLong,modelStateT);
%                         idxS = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorShort,modelStateT);
%                         spreadC = idxL - idxS;
%                         probSpread = model.SRPr(spreadC,spreadN,ralower,raupper,simpleYN,pathSize)*dD; 

                        if simpleYN
                            probSpread = spreadC*dD;
                        else
                            probSpread = spreadC*dD;
%                             probSpread = 0.5*(spreadC + spreadN)*dD;
                            
                        end

                        
                        cD = cD + dD;
                        cP = cP + probSpread;

                        dcfN = dcfC;
                        spreadN = spreadC;

                        %induct backward to the next time step
                        % one time backward induction
                        lastTimeIdx = currentTimeIdx;

                        if startTimeIdx < lastTimeIdx
                            lastTimeIdx = lastTimeIdx -1;
                            currentTimeIdx = lastTimeIdx;
                        end


                    end

                    %% FromToInduction end
                    %% AtDate evaluation
                    % Adding a new cashflow to the payoff

                    dcf = schedule.dayCountFraction(i);
                    
                    if inclusiveYN
                        % we add valuation date
                        cD = schedule.dayCount(i);
                        rangeDays =  security.rangeDays(i);
                        payoffStateA.cashflow = nominal * coupon * dcf * (rangeDays + cP)/cD;
                    else
                        payoffStateA.cashflow = nominal * coupon * dcf * cP/cD;
                    end
                    % eventDate(i) = startDate(i) therefore there is no induction in modelStates
                    % i.e. currentTimeIdx has not changed
                    % no need to update modelStateT
                    % modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:end,:);
                    % since cashflow's are added at the startDate(i) we need to
                    % multiply cashflow with the df(startDate(i), payDate(i))
                    payTime = DateDiff(schedule.payDates(i),valueDate);
                    
                    %event is at startDate so we need to discount the
                    %cashflow first from paydate to startdate
%                     df_sp = model.discountFactor(currentTime, payTime,numMatTime,modelStateT);

%                     for k=1:pathSize
%                         payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_sp(k);
%                     end
                    
                    % process payoff
                    
                    currentNodeIdx = i;
                    
                    % 1st nonCall column
                    nonCall.payoffStates.cashflow(currentNodeIdx,:) = payoffStateA.cashflow;
                    payoffStateA.cashflow_npv = payoffStateA.cashflow * model.DFRisky(payTime/365.0);
                    
%                     payoffStateA.cashflow_npv = model.discountPayoff(currentTime,numMatTime ...
%                     ,modelStateT,payoffStateA.cashflow);
                    nonCall.payoffStates.cashflow_npv(currentNodeIdx,:) = payoffStateA.cashflow_npv;
                    payoffStateA.df = model.DFRisky(payTime/365.0);
%                     payoffStateA.df = model.discountPayoff(currentTime,numMatTime ...
%                     ,modelStateT,1.0*ones(1,pathSize));
                    nonCall.payoffStates.df(currentNodeIdx,:) = payoffStateA.df;
                    
                    nonCall.payoff = nonCall.payoff + payoffStateA.cashflow_npv;
                    
                    nonCall.cashflow_e(currentNodeIdx) = mean(payoffStateA.cashflow);
                    nonCall.rangeProb_e(currentNodeIdx) = mean(nonCall.cashflow_e(currentNodeIdx)/(nominal * coupon * dcf));
                    nonCall.cashflow_npv_e(currentNodeIdx) = mean(payoffStateA.cashflow_npv);

                    nonCall.df_e(currentNodeIdx) = mean(payoffStateA.df);
                    nonCall.df_c(currentNodeIdx) = model.DFRisky(currentTime/365.0);
                    
%                     %2nd callable column
%                     callable.payoffStates.cashflow(currentNodeIdx,:) = nonCall.payoffStates.cashflow(currentNodeIdx,:);
%                     callable.payoffStates.cashflow_npv(currentNodeIdx,:) = nonCall.payoffStates.cashflow_npv(currentNodeIdx,:);
%                     callable.payoff = callable.payoff + callable.payoffStates.cashflow_npv(currentNodeIdx,:);
%                     
%                     callable.cashflow_e(currentNodeIdx) = nonCall.cashflow_e(currentNodeIdx);
%                     callable.cashflow_npv_e(currentNodeIdx) = nonCall.cashflow_npv_e(currentNodeIdx);

                    % if exercise is possible
                    if i == 1
                        dummaaa = 1;
                    end
                    
                    % call event happens  at the start of the period
                    % at call date, by default, call option is expired
                    
%                     if callYN(i) && (DateDiff(valueDate,schedule.startDates(i)) < 0)
%                         % 1.undiscount callable.payoff
%                         % 2.build the state variables matrix
%                         % 3.regress 1's with the given state variables
%                         % (model states 1,2)
%                         % 4. exercise per each path in
%                         % discount the new payoff and assign into callable
%                         % column
%                         
%                         %1 undiscount callable.payoff and get the
%                         %contiValue
%                        
%                         %exercise value
%                         exerciseValue = security.nominal*ones(1,pathSize);
%                         contiValue = zeros(1,pathSize);
%                         onePayoff = ones(1,pathSize);
%                         onePayoffD = model.discountPayoff(currentTime,numMatTime ...
%                         ,modelStateT,onePayoff);
%                         contiValue = callable.payoff./onePayoffD;
%                         
%                         % build the state variabls matrix
%                         decisionVar1 = modelStateT(1,:);
%                         decisionVar2 = modelStateT(2,:);
%                         % we use upto 2nd order polynomial as regression
%                         % equatioin
%                         dimensionLS = 6;
%                         regMatrix = ones(pathSize,dimensionLS);
%                         
%                         for idx_r=1:pathSize
%                             regMatrix(idx_r,2) = decisionVar1(idx_r);
%                             regMatrix(idx_r,3) = decisionVar2(idx_r);
%                             regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r);
%                             regMatrix(idx_r,5) = decisionVar1(idx_r)*decisionVar2(idx_r);
%                             regMatrix(idx_r,6) = decisionVar2(idx_r)*decisionVar2(idx_r);
%                         end
%                         
%                         % simple regression <- for path dependent option
%                         % we should change it
%                         % beta  : regression coefficients
%                         beta =zeros(dimensionLS,1);
%                         beta = regress(contiValue',regMatrix);
%                         regContiValue = regMatrix*beta;
%                         
%                         exercisedPayoff = regContiValue;
%                         % if regContiValue is bigger than exerciseValue
%                         % then issue exercise call option
%                         for idx_r=1:pathSize
%                             if regContiValue(idx_r) >= exerciseValue(idx_r) 
%                                 exercisedPayoff(idx_r) = exerciseValue(idx_r);
%                             end
%                         end
%                         
%                         % we discount the new payoff and 
%                         % assign it into the existing callable payoff
%                         callable.payoff = model.discountPayoff(currentTime,numMatTime ...
%                         ,modelStateT,exercisedPayoff);
%                         
%                     end
                end

                nonCall.npv = mean(nonCall.payoff);
                out.pricerInfo.nonCall = nonCall;
                
%                 callable.npv = mean(callable.payoff);
%                 out.pricerInfo.callable = callable;
                out.daycountFraction = schedule.dayCountFraction;
                
        end
        
        
        
        
    end
    
end



In [ ]:
classdef DailyWeeklyRiskyPricer_CDKTBSpreadRA < H_RootObject
    %Pricer(Security,Model,NumMethod)
    %Prcer is constructed from Security, Model, NumMethod 
    
    properties
        m_security
        m_model
        m_numMethod
        overHedgeYN
    end
    
    methods
        function pricer = DailyWeeklyRiskyPricer_CDKTBSpreadRA(security,model,numMethod,overHedgeYN)
                pricer.m_security = security;
                pricer.m_model = model;
                pricer.m_numMethod = numMethod;
                pricer.overHedgeYN = overHedgeYN;
        end
        
        function out = computePriceMCCallableGeneric(pricer,valueDate,pricerOption)
                security = pricer.m_security;
                numMethod = pricer.m_numMethod;
                model = pricer.m_model;
                schedule = security.couponSchedule;
                callSchedule = security.callSchedule;
                
                % numMethodTimeSteps Generate
                mcOneTimeStep = numMethod.mcOneTimeStep;
                outInfo = numMethod.numMethodTimeStepsGenerateDW(valueDate,schedule,mcOneTimeStep);
            
                numMethodInfo.timeStepsInfo = outInfo;

                % mc random number init
                %pathSize = 2^16;
                pathSize = numMethod.pathSize;

                % generate random numbers and modelstates forward
                rng('default');
                rng(0);

                % LGM1F : one factor
                % LGM2F : two factor
                
                numOfFactors = model.numOfFactor;

                %Z= randn(numOfFactors*outInfo.totalTimeStepsSize,pathSize/2);
                Z= randn(pathSize/2,numOfFactors*outInfo.totalTimeStepsSize);
                Z=[Z;-Z];
                Z=Z';
                
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;

                % we check with the c code, so we use generated random numbers
%                 mc_random = importdata('mc_random.xlsx');
%                 mc_random = importdata('D:\mc_random_number_new_report.txt');
%                 Z= mc_random;
                
                numMethodInfo.pathSize = pathSize;
                numMethodInfo.numOfFactors = numOfFactors;
                numMethodInfo.totalTimeSteps = outInfo.totalTimeSteps;
                numMethodInfo.totalTimeStepsSize = outInfo.totalTimeStepsSize;
                numMethodInfo.Z = Z;



                %numMethod generate model states for the given timeSteps
                modelStates = numMethod.generateModelStatesNF(valueDate,schedule,numMethodInfo,model);

                % numeraire info
                numMatTime = numMethodInfo.totalTimeSteps(end);
                days = DateDiff(schedule.payDates(schedule.numOfPeriod),valueDate);
                numMatTime = max(numMatTime, days);
                dfNumMat = model.DFRisky(numMatTime/365.0);
                
                numeraire.numMatTime = numMatTime;
                numeraire.dfNumMat = dfNumMat;
                
                % induct backward and calculate cashflow
            
                periodCount = schedule.numOfPeriod;
                
                % pricer single info initialize
                
                nonCall = PricerInfo(periodCount +1, pathSize);
                callable = PricerInfo(periodCount +1, pathSize);
                dfCol = PricerInfo(periodCount +1, pathSize);
                
                dummy1 = PricerInfo(periodCount +1, pathSize);
                dummy2 = PricerInfo(periodCount +1, pathSize);
                
                
                % intermediate variables
                payoffStateA.cashflow = zeros(1, pathSize);
                payoffStateA.cashflow_npv = zeros(1, pathSize);
                
            
                % we calculate cashflow backward from maturityDate to valueDate
                % rangeAccrual coupons are added at the startDate of the Period
                % At maturity

                nominal = security.nominal;
                coupon = security.coupon;
                callYN = security.callYN;
                
                ralower = security.lower;
                raupper = security.upper;
                tenorLong = security.tenorL;
                tenorShort = security.tenorS;
                
                currentTime = numMethodInfo.totalTimeSteps(end);
                totalTimeStepsSize = numMethodInfo.totalTimeStepsSize;
                currentTimeIdx = totalTimeStepsSize;
                
                currentNodeIdx = periodCount + 1;
                
                modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:numOfFactors*(currentTimeIdx-1)+numOfFactors,:);
                
                payoffStateA.cashflow = nominal*ones(1,pathSize);
                
                payTime = DateDiff(schedule.payDates(schedule.numOfPeriod),valueDate);
                df_ep = model.discountFactorRisky(currentTime, payTime,numMatTime,modelStateT);
                for k=1:pathSize
                    payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                end
                
                %process payoff at maturity
                nonCall.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                
                callable.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                
                %df column
                payoffStateA.cashflow = ones(1,pathSize);
                for k=1:pathSize
                    payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                end
                
                dfCol.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                                 
                %dummy column

                if strcmp(pricerOption,'PSA')
                    payoffStateA.cashflow = model.CMS_PSA_SimpleDate('KRW',valueDate,currentTime,numMatTime, tenorLong,modelStateT);
                    dummy1.ProcessPayoff(model,currentTime,numeraire, ...
                                 modelStateT,payoffStateA.cashflow,currentNodeIdx);
                    
                    payoffStateA.cashflow = model.CMS_PSA_SimpleDate('KTB',valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                    dummy2.ProcessPayoff(model,currentTime,numeraire, ...
                                 modelStateT,payoffStateA.cashflow,currentNodeIdx);
                
                else
                    payoffStateA.cashflow = model.Libor_SimpleDate('KRW',valueDate,currentTime,currentTime,numMatTime, tenorLong,modelStateT); 
                    dummy1.ProcessPayoff(model,currentTime,numeraire, ...
                                 modelStateT,payoffStateA.cashflow,currentNodeIdx);
                             
                    payoffStateA.cashflow = model.Libor_SimpleDate('KTB',valueDate,currentTime,currentTime,numMatTime, tenorShort,modelStateT); 
                    dummy2.ProcessPayoff(model,currentTime,numeraire, ...
                                 modelStateT,payoffStateA.cashflow,currentNodeIdx);
                    
                end
                

%                 payoffStateA.cashflow = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                
%                 payoffStateA.cashflow = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                
                
                                 
                  

                lastTimeIdx = currentTimeIdx;
                totalTimeSteps = numMethodInfo.timeStepsInfo.totalTimeSteps;
                
                % flag for inclusive schedule
%                 simpleYN = false;
%                 simpleYN = true;
                if pricer.overHedgeYN == true 
                    simpleYN = 2;
                else
                    simpleYN = 1;
                end
                
                for i=periodCount:-1:1
                    % past schedule neglect
                    if DateDiff(valueDate,schedule.payDates(i)) >= 0, continue;end 
                    startTimeIdx = numMethodInfo.timeStepsInfo.startTimeIdx(i);
                    endTimeIdx = numMethodInfo.timeStepsInfo.endTimeIdx(i);

                    inclusiveYN = numMethodInfo.timeStepsInfo.inclusiveYN(i);
                 %% FromToInduction start

                    %modelsStates induct backward(lastTimeIdx to endTimeIdx)
                    while endTimeIdx < lastTimeIdx
                        lastTimeIdx = lastTimeIdx -1;
                        if lastTimeIdx == endTimeIdx break;end
                    end

                    currentTimeIdx = lastTimeIdx;
                    currentTime = totalTimeSteps(currentTimeIdx);
                    % now we are at endTimeIdx
                    % variables for coupon valuation initialized
                    

                    cD = 0; % cumulated day count fraction
                    cP = 0; 
                    dI = 0; % dummy idx for loop
                    dcfC = 0;
                    dcfN = 0;
                    dD = 0;
                    idxL = zeros(1,pathSize);
                    idxS = zeros(1,pathSize);
                    spreadC = zeros(1,pathSize);
                    spreadN = zeros(1,pathSize);
                    probSpread = zeros(1,pathSize);
                    
                    numOfTimeSteps = numMethodInfo.timeStepsInfo.numOfTimeSteps(i);
                    
                    % if inclusive then we use simpleYN rangeProb
                    %if inclusive then we do not evaluate the last step but
                    % use fixing instead 
                    endTimeStep = 1;
                    if inclusiveYN
%                         simpleYN = true;
                        simpleYN = 1;
                        endTimeStep = 2;
                        if pricer.overHedgeYN == true 
                            simpleYN = 2;
                        end
                    end
                    
                    for j=numOfTimeSteps:-1:endTimeStep
                        % evaluate dealdescription
                        % last step is omitted
                        currentTime = totalTimeSteps(currentTimeIdx);
                        %dcfC = currentTime/365.0;
                        dcfC = currentTime;
                        
                        if j~= numOfTimeSteps
                            dD = (dcfN-dcfC);
                        end

                        modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:numOfFactors*(currentTimeIdx-1)+numOfFactors,:);
                        
                        if strcmp(pricerOption,'PSA')
                            idxL = model.CMS_PSA_SimpleDate('KRW',valueDate,currentTime,numMatTime, tenorLong,modelStateT);
                            idxS = model.CMS_PSA_SimpleDate('KTB',valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                        else
                            idxL = model.Libor_SimpleDate('KRW',valueDate,currentTime,currentTime,numMatTime, tenorLong,modelStateT);
                            idxS = model.Libor_SimpleDate('KTB',valueDate,currentTime,currentTime,numMatTime, tenorShort,modelStateT);
                        end
                        spreadC = idxL - idxS;

                        probSpread = model.SRPr(spreadC,spreadN,ralower,raupper,simpleYN,pathSize)*dD;
                        cD = cD + dD;
                        cP = cP + probSpread;

                        dcfN = dcfC;
                        spreadN = spreadC;

                        %induct backward to the next time step
                        % one time backward induction
                        lastTimeIdx = currentTimeIdx;

                        if startTimeIdx < lastTimeIdx
                            lastTimeIdx = lastTimeIdx -1;
                            currentTimeIdx = lastTimeIdx;
                        end


                    end

                    %% FromToInduction end
        % if the current period is inclusive, then the start date is not evaluated in the above for loop.
		% instead the fixing will be treated separately below.
		% however current_time_idx is updated at the end of the for loop.
		% so we need to update model_states accordingly.
		% if model_states were updated previously then the following additionaly step doesn't affect the result anyway.
                    currentTime = totalTimeSteps(currentTimeIdx);
                    modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:numOfFactors*(currentTimeIdx-1)+numOfFactors,:);
                    
                    %% AtDate evaluation
                    % Adding a new cashflow to the payoff

                    dcf = schedule.dayCountFraction(i);
                    
                    if inclusiveYN
                        % we add valuation date
                        cD = schedule.dayCount(i);
                        rangeDays =  security.rangeDays(i);
                        payoffStateA.cashflow = nominal * coupon * dcf * (rangeDays + cP)/cD;
                    else
                        payoffStateA.cashflow = nominal * coupon * dcf * cP/cD;
                    end
                    % eventDate(i) = startDate(i) therefore there is no induction in modelStates
                    % i.e. currentTimeIdx has not changed
                    % no need to update modelStateT
                    % modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:end,:);
                    % since cashflow's are added at the startDate(i) we need to
                    % multiply cashflow with the df(startDate(i), payDate(i))
                    payTime = DateDiff(schedule.payDates(i),valueDate);
                    
                    %event is at startDate so we need to discount the
                    %cashflow first from paydate to startdate
                    df_ep = model.discountFactorRisky(currentTime, payTime,numMatTime,modelStateT);

                    for k=1:pathSize
                        payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                    end
                    
                    % process payoff
                    
                    currentNodeIdx = i;
                    
                    nonCall.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                
                    callable.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                    
                    %df column
                    payoffStateA.cashflow = ones(1,pathSize);
                    for k=1:pathSize
                        payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                    end
                    
                    dfCol.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                                 
                    %dummy column
                    if strcmp(pricerOption,'PSA')
                        payoffStateA.cashflow = model.CMS_PSA_SimpleDate('KRW',valueDate,currentTime,numMatTime, tenorLong,modelStateT);
                        dummy1.ProcessPayoff(model,currentTime,numeraire, ...
                                         modelStateT,payoffStateA.cashflow,currentNodeIdx);



                        payoffStateA.cashflow = model.CMS_PSA_SimpleDate('KTB',valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                        dummy2.ProcessPayoff(model,currentTime,numeraire, ...
                                         modelStateT,payoffStateA.cashflow,currentNodeIdx);
                    
                    else
                        payoffStateA.cashflow = model.Libor_SimpleDate('KRW',valueDate,currentTime,currentTime,numMatTime, tenorLong,modelStateT); 
                        dummy1.ProcessPayoff(model,currentTime,numeraire, ...
                                 modelStateT,payoffStateA.cashflow,currentNodeIdx);
                        
                        payoffStateA.cashflow = model.Libor_SimpleDate('KTB',valueDate,currentTime,currentTime,numMatTime, tenorShort,modelStateT); 
                        dummy2.ProcessPayoff(model,currentTime,numeraire, ...
                                 modelStateT,payoffStateA.cashflow,currentNodeIdx);
                        
                    end

                                 
                    
                    
                    
                    % if exercise is possible
                    if i == 1
                        dummaaa = 1;
                    end
                    
                    % call event happens  at the start of the period
                    % at call date, by default, call option is expired
                    
                    if callYN(i) && (DateDiff(valueDate,callSchedule.resetDates(i)) < 0)
                        % 1.undiscount callable.payoff
                        % 2.build the state variables matrix
                        % 3.regress 1's with the given state variables
                        % (model states 1,2)
                        % 4. exercise per each path in
                        % discount the new payoff and assign into callable
                        % column
                        
                        %1 undiscount callable.payoff and get the
                        %contiValue
                       
                        %exercise value
                        exerciseValue = security.nominal*ones(1,pathSize);
                        payTime = DateDiff(callSchedule.payDates(i),valueDate);
                        %event is at startDate so we need to discount the
                        %cashflow first from paydate to startdate
                        df_ep = model.discountFactorRisky(currentTime, payTime,numMatTime,modelStateT);
                        for k=1:pathSize
                            exerciseValue(k) = exerciseValue(k) * df_ep(k);
                        end
                        
                        contiValue = zeros(1,pathSize);
                        onePayoff = ones(1,pathSize);
                        onePayoffD = model.discountPayoffRisky(currentTime,numMatTime ...
                        ,modelStateT,onePayoff);
                        contiValue = callable.payoff./onePayoffD;
                        
                        %Nominal Regression test
                        contiValue = contiValue/10000;
                        
                        % build the state variabls matrix
                        dimensionLS = (numOfFactors + 1)*(numOfFactors + 2)/2;
                        
                        decisionVar = zeros(numOfFactors,size(modelStateT,2));
                        
                        for idxLS =1:numOfFactors
                            decisionVar(idxLS,:) = modelStateT(idxLS,:); 
                        end
%                         decisionVar1 = modelStateT(1,:);
%                         decisionVar2 = modelStateT(2,:);
                        % we use upto 2nd order polynomial as regression
                        % equatioin
                        
                        regMatrix = ones(pathSize,dimensionLS);
                        
                        for idx_r=1:pathSize
                            for idxLS=1:numOfFactors
                                regMatrix(idx_r,idxLS+1) = decisionVar(idxLS,idx_r);
                            end
                            
                            tempIdx = 1;
                            for idxLS1=1:numOfFactors
                                for idxLS2=idxLS1:numOfFactors
                                    regMatrix(idx_r,numOfFactors+1+tempIdx) = decisionVar(idxLS1,idx_r)*decisionVar(idxLS2,idx_r);
                                    tempIdx = tempIdx + 1;
                                end
                            end
                            
                        end
                        
                        % simple regression <- for path dependent option
                        % we should change it
                        % beta  : regression coefficients
                        beta =zeros(dimensionLS,1);
                        beta = regress(contiValue',regMatrix);
                        regContiValue = regMatrix*beta;
                       
                         %Nominal Regression test
                        regContiValue = regContiValue * 10000;
                        
                        exercisedPayoff = regContiValue;
                        % if regContiValue is bigger than exerciseValue
                        % then issue exercise call option
                        for idx_r=1:pathSize
                            if regContiValue(idx_r) >= exerciseValue(idx_r) 
                                exercisedPayoff(idx_r) = exerciseValue(idx_r);
                            end
                        end
                        
                        % we discount the new payoff and 
                        % assign it into the existing callable payoff
                        callable.payoff = model.discountPayoffRisky(currentTime,numMatTime ...
                        ,modelStateT,exercisedPayoff);
                        
                    end
                end

                nonCall.npv = mean(nonCall.payoff);
                out.nonCall = nonCall;
                callable.npv = mean(callable.payoff);
                dfCol.npv = mean(dfCol.payoff);
                dummy1.npv = mean(dummy1.npv);
                dummy2.npv = mean(dummy2.npv);
                out.callable = callable;
                out.dfCol = dfCol;
                out.dummy1 = dummy1;
                out.dummy2 = dummy2;
                out.daycountFraction = schedule.dayCountFraction;
                
        end
        
        
        function out = computePriceClosedSimple(pricer,valueDate)
                security = pricer.m_security;
                numMethod = pricer.m_numMethod;
                model = pricer.m_model;
                schedule = security.couponSchedule;
                
                % numMethodTimeSteps Generate
                mcOneTimeStep = numMethod.mcOneTimeStep;
                outInfo = numMethod.numMethodTimeStepsGenerateDW(valueDate,schedule,mcOneTimeStep);
            
                numMethodInfo.timeStepsInfo = outInfo;

                % mc random number init
                %pathSize = 2^16;
                pathSize = numMethod.pathSize;
                
                pathSize = 1;
                
%                 % generate random numbers and modelstates forward
%                 rng('default');
%                 rng(0);
% 
                numOfFactors = model.numOfFactor;
% 
%                 %Z= randn(numOfFactors*outInfo.totalTimeStepsSize,pathSize/2);
%                 Z= randn(pathSize/2,numOfFactors*outInfo.totalTimeStepsSize);
%                 Z=[Z;-Z];
%                 Z=Z';
%                 
%                 
%                 %TimeInversion
%                 sizeZ = size(Z,1);
%                 A = zeros(sizeZ,sizeZ);
%                 for i=1:sizeZ
%                     A(i,sizeZ + 1 -i) = 1;
%                 end
%                 Z = A*Z;
% 
%                 % we check with the c, so we use generated random numbers
% %                 mc_random = importdata('mc_random.xlsx');
% %                 mc_random = importdata('D:\mc_random_number_new_report.txt');
% %                 Z= mc_random;
%                 
                numMethodInfo.pathSize = pathSize;
                numMethodInfo.numOfFactors = numOfFactors;
                numMethodInfo.totalTimeSteps = outInfo.totalTimeSteps;
                numMethodInfo.totalTimeStepsSize = outInfo.totalTimeStepsSize;
%                 numMethodInfo.Z = Z;
% 
% 
% 
%                 %generate model states for the given timeSteps
%                 modelStates = numMethod.generateModelStates(valueDate,schedule,numMethodInfo,model);

                % numeraire info
                numMatTime = numMethodInfo.totalTimeSteps(end);
                dfNumMat = model.DFRisky(numMatTime/365.0);
                
                % induct backward and calculate cashflow
            
                periodCount = schedule.numOfPeriod;
                
                % cashflow columns initialize
                % 1st nonCall
                nonCall.npv = 0;
                nonCall.cashflow_e = zeros(periodCount + 1,1);
                nonCall.cashflow_npv_e = zeros(periodCount + 1,1);
                nonCall.rangeProb_e = zeros(periodCount + 1,1);
                nonCall.df_e = zeros(periodCount + 1,1);
                nonCall.df_c = zeros(periodCount + 1,1);

                % debug information, cashflow & npv per path
                nonCall.payoff = zeros(1,pathSize);
                nonCall.payoffStates.cashflow = zeros(periodCount+1,pathSize);
                nonCall.payoffStates.cashflow_npv = zeros(periodCount+1,pathSize);
                nonCall.payoffStates.df = zeros(periodCount + 1,pathSize);
                
                % intermediate variables
                payoffStateA.cashflow = zeros(1, pathSize);
                payoffStateA.cashflow_npv = zeros(1, pathSize);
                payoffStateA.df = zeros(1,pathSize);
                
                % we calculate cashflow backward from maturityDate to valueDate
                % rangeAccrual coupons are added at the startDate of the Period
                % At maturity

                nominal = security.nominal;
                coupon = security.coupon;
                callYN = security.callYN;
                
                currentTime = numMethodInfo.totalTimeSteps(end);
                totalTimeSteps = numMethodInfo.timeStepsInfo.totalTimeSteps;
                totalTimeStepsSize = numMethodInfo.timeStepsInfo.totalTimeStepsSize;
                if totalTimeStepsSize == 0 || (totalTimeStepsSize == 1 && totalTimeSteps(1) == 0) 
                    out.nonCall = nonCall;
                    return;
                end
                
                currentTimeIdx = totalTimeStepsSize;
                
                currentNodeIdx = periodCount + 1;
                
%                 modelStateT = modelStates(numOfFactors*(totalTimeStepsSize-1)+1:numOfFactors*(totalTimeStepsSize-1)+2,:);
                payoffStateA.cashflow = nominal*ones(1,pathSize);
                %process payoff at maturity
                nonCall.payoffStates.cashflow(currentNodeIdx,:) = payoffStateA.cashflow;
                
%                 payoffStateA.cashflow_npv = model.discountPayoff(currentTime,numMatTime ...
%                     ,modelStateT,payoffStateA.cashflow);
%                 
                payoffStateA.cashflow_npv = model.DFRisky(currentTime/365.0)*payoffStateA.cashflow;
                
                nonCall.payoffStates.cashflow_npv(currentNodeIdx,:) = payoffStateA.cashflow_npv ;
                
%                 payoffStateA.df = model.discountPayoff(currentTime,numMatTime ...
%                     ,modelStateT,1.0*ones(1,pathSize));
                payoffStateA.df = model.DFRisky(currentTime/365.0) *ones(1,pathSize);
                    

                nonCall.payoffStates.df(currentNodeIdx,:) = payoffStateA.df;
                nonCall.payoff = nonCall.payoff + payoffStateA.cashflow_npv;
                
                nonCall.cashflow_e(currentNodeIdx) = mean(payoffStateA.cashflow);
                nonCall.cashflow_npv_e(currentNodeIdx) = mean(payoffStateA.cashflow_npv);
                nonCall.df_e(currentNodeIdx) = mean(payoffStateA.df);
                nonCall.df_c(currentNodeIdx) = model.DFRisky(currentTime/365.0);
                
%                 % callable at maturity is the same as noncallable
%                 callable.payoffStates.cashflow(currentNodeIdx,:) = nonCall.payoffStates.cashflow(currentNodeIdx,:);
%                 callable.payoffStates.cashflow_npv(currentNodeIdx,:) = nonCall.payoffStates.cashflow_npv(currentNodeIdx,:);
%                 callable.payoff = callable.payoff + callable.payoffStates.cashflow_npv(currentNodeIdx,:);
%                 
%                 callable.cashflow_e(currentNodeIdx) = nonCall.cashflow_e(currentNodeIdx);
%                 callable.cashflow_npv_e(currentNodeIdx) = nonCall.cashflow_npv_e(currentNodeIdx);
                  
                lastTimeIdx = currentTimeIdx;
                
                
                % flag for inclusive schedule
%                 simpleYN = false;
                simpleYN = 1;
                
                for i=periodCount:-1:1
                    % past schedule neglect
                    if DateDiff(valueDate,schedule.payDates(i)) >= 0, continue;end 
                    startTimeIdx = numMethodInfo.timeStepsInfo.startTimeIdx(i);
                    endTimeIdx = numMethodInfo.timeStepsInfo.endTimeIdx(i);

                    inclusiveYN = numMethodInfo.timeStepsInfo.inclusiveYN(i);
                     %% FromToInduction start

                    %modelsStates induct backward(lastTimeIdx to endTimeIdx)
                    while endTimeIdx < lastTimeIdx
                        lastTimeIdx = lastTimeIdx -1;
                        if lastTimeIdx == endTimeIdx break;end
                    end

                    currentTimeIdx = lastTimeIdx;
                    currentTime = totalTimeSteps(currentTimeIdx);
                    % now we are at endTimeIdx
                    % variables for coupon valuation initialized
                    ralower = security.lower;
                    raupper = security.upper;
                    tenorLong = security.tenorL;
                    tenorShort = security.tenorS;

                    cD = 0; % cumulated day count fraction
                    cP = 0; 
                    dI = 0; % dummy idx for loop
                    dcfC = 0;
                    dcfN = 0;
                    dD = 0;
                    idxL = zeros(1,pathSize);
                    idxS = zeros(1,pathSize);
                    spreadC = zeros(1,pathSize);
                    spreadN = zeros(1,pathSize);
                    probSpread = zeros(1,pathSize);
                    
                    numOfTimeSteps = numMethodInfo.timeStepsInfo.numOfTimeSteps(i);
                    
                    % if inclusive then we use simpleYN rangeProb
                    %if inclusive then we do not evaluate the last step but
                    % use fixing instead 
                    endTimeStep = 1;
                    if inclusiveYN
%                         simpleYN = true;
                        simpleYN = 1;
                        endTimeStep = 2;
                    end
                    
                    for j=numOfTimeSteps:-1:endTimeStep
                        % evaluate dealdescription
                        % last step is omitted
                        currentTime = totalTimeSteps(currentTimeIdx);
                        %dcfC = currentTime/365.0;
                        dcfC = currentTime;
                        
                        if j~= numOfTimeSteps
                            dD = (dcfN-dcfC);
                        end

                        toDate = AddDate(valueDate,currentTime,'day');
                        tpDate = schedule.payDates(i);
                        spreadC = model.CMSCMTSpreadDigitalRangeDate(valueDate,toDate,tpDate,tenorLong,tenorShort,ralower,raupper);
                        
                        %modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:numOfFactors*(currentTimeIdx-1)+2,:);
%                         idxL = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorLong,modelStateT);
%                         idxS = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorShort,modelStateT);
%                         spreadC = idxL - idxS;
%                         probSpread = model.SRPr(spreadC,spreadN,ralower,raupper,simpleYN,pathSize)*dD; 

                        if simpleYN
                            probSpread = spreadC*dD;
                        else
                            probSpread = spreadC*dD;
%                             probSpread = 0.5*(spreadC + spreadN)*dD;
                            
                        end

                        
                        cD = cD + dD;
                        cP = cP + probSpread;

                        dcfN = dcfC;
                        spreadN = spreadC;

                        %induct backward to the next time step
                        % one time backward induction
                        lastTimeIdx = currentTimeIdx;

                        if startTimeIdx < lastTimeIdx
                            lastTimeIdx = lastTimeIdx -1;
                            currentTimeIdx = lastTimeIdx;
                        end


                    end

                    %% FromToInduction end
                    %% AtDate evaluation
                    % Adding a new cashflow to the payoff

                    dcf = schedule.dayCountFraction(i);
                    
                    if inclusiveYN
                        % we add valuation date
                        cD = schedule.dayCount(i);
                        rangeDays =  security.rangeDays(i);
                        payoffStateA.cashflow = nominal * coupon * dcf * (rangeDays + cP)/cD;
                    else
                        payoffStateA.cashflow = nominal * coupon * dcf * cP/cD;
                    end
                    % eventDate(i) = startDate(i) therefore there is no induction in modelStates
                    % i.e. currentTimeIdx has not changed
                    % no need to update modelStateT
                    % modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:end,:);
                    % since cashflow's are added at the startDate(i) we need to
                    % multiply cashflow with the df(startDate(i), payDate(i))
                    payTime = DateDiff(schedule.payDates(i),valueDate);
                    
                    %event is at startDate so we need to discount the
                    %cashflow first from paydate to startdate
%                     df_sp = model.discountFactor(currentTime, payTime,numMatTime,modelStateT);

%                     for k=1:pathSize
%                         payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_sp(k);
%                     end
                    
                    % process payoff
                    
                    currentNodeIdx = i;
                    
                    % 1st nonCall column
                    nonCall.payoffStates.cashflow(currentNodeIdx,:) = payoffStateA.cashflow;
                    payoffStateA.cashflow_npv = payoffStateA.cashflow * model.DFRisky(payTime/365.0);
                    
%                     payoffStateA.cashflow_npv = model.discountPayoff(currentTime,numMatTime ...
%                     ,modelStateT,payoffStateA.cashflow);
                    nonCall.payoffStates.cashflow_npv(currentNodeIdx,:) = payoffStateA.cashflow_npv;
                    payoffStateA.df = model.DFRisky(payTime/365.0);
%                     payoffStateA.df = model.discountPayoff(currentTime,numMatTime ...
%                     ,modelStateT,1.0*ones(1,pathSize));
                    nonCall.payoffStates.df(currentNodeIdx,:) = payoffStateA.df;
                    
                    nonCall.payoff = nonCall.payoff + payoffStateA.cashflow_npv;
                    
                    nonCall.cashflow_e(currentNodeIdx) = mean(payoffStateA.cashflow);
                    nonCall.rangeProb_e(currentNodeIdx) = mean(nonCall.cashflow_e(currentNodeIdx)/(nominal * coupon * dcf));
                    nonCall.cashflow_npv_e(currentNodeIdx) = mean(payoffStateA.cashflow_npv);

                    nonCall.df_e(currentNodeIdx) = mean(payoffStateA.df);
                    nonCall.df_c(currentNodeIdx) = model.DFRisky(currentTime/365.0);
                    

                end

                nonCall.npv = mean(nonCall.payoff);
                out.pricerInfo.nonCall = nonCall;
                
%                 callable.npv = mean(callable.payoff);
%                 out.pricerInfo.callable = callable;
                out.daycountFraction = schedule.dayCountFraction;
                
        end
        

        
        
        
        
    end
    
end



In [ ]:
classdef DailyWeeklyRiskyPricer_CDKTBSpreadCMSSpreadDualRASimpleCall < H_RootObject
    %Pricer(Security,Model,NumMethod)
    %Prcer is constructed from Security, Model, NumMethod 
    
    properties
        m_security
        m_model
        m_numMethod
        overHedgeYN
    end
    
    methods
        function pricer = DailyWeeklyRiskyPricer_CDKTBSpreadCMSSpreadDualRASimpleCall(security,model,numMethod,overHedgeYN)
                pricer.m_security = security;
                pricer.m_model = model;
                pricer.m_numMethod = numMethod;
                pricer.overHedgeYN = overHedgeYN;
        end
        
        function out = computePriceMCCallableGeneric(pricer,valueDate,pricerOption)
                security = pricer.m_security;
                numMethod = pricer.m_numMethod;
                model = pricer.m_model;
                schedule = security.couponSchedule;
                callSchedule = security.callSchedule;
                
                % numMethodTimeSteps Generate
                mcOneTimeStep = numMethod.mcOneTimeStep;
%                 outInfo = numMethod.numMethodTimeStepsGenerateDW(valueDate,schedule,mcOneTimeStep);
                
                floatingSchedule = struct;
                outInfo = numMethod.numMethodTimeStepsGenerateDWSwapSimpleCall(valueDate,schedule,floatingSchedule,...
                                    callSchedule,mcOneTimeStep);
            
                numMethodInfo.timeStepsInfo = outInfo;

                % mc random number init
                %pathSize = 2^16;
                pathSize = numMethod.pathSize;

                % generate random numbers and modelstates forward
                rng('default');
                rng(0);

                % LGM2F : two factor
                % CK1F  : one factor
                
                numOfFactors = model.numOfFactors;

                %Z= randn(numOfFactors*outInfo.totalTimeStepsSize,pathSize/2);
%                 Z= randn(pathSize/2,numOfFactors*outInfo.totalTimeStepsSize);
                Z= randn(pathSize/2,numOfFactors*outInfo.modelTimeStepsSize);
                Z=[Z;-Z];
                Z=Z';
                
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;

                % we check with the c code, so we use generated random numbers
%                 mc_random = importdata('mc_random.xlsx');
%                 mc_random = importdata('D:\mc_random_number_new_report.txt');
%                 Z= mc_random;
                
                numMethodInfo.pathSize = pathSize;
                numMethodInfo.numOfFactors = numOfFactors;
                numMethodInfo.totalTimeSteps = outInfo.totalTimeSteps;
                numMethodInfo.totalTimeStepsSize = outInfo.totalTimeStepsSize;
                
                % for modelStateGeneration
                numMethodInfo.modelTimeSteps = outInfo.modelTimeSteps;
                numMethodInfo.modelTimeStepsSize = outInfo.modelTimeStepsSize;
                
                numMethodInfo.pastTimeStepsSize = outInfo.pastTimeStepsSize;
                
                pastTimeStepsSize = numMethodInfo.pastTimeStepsSize;
                
                numMethodInfo.Z = Z;



                %numMethod generate model states for the given timeSteps
%                 modelStates = numMethod.generateModelStatesNF(valueDate,schedule,numMethodInfo,model);
                 modelStates = numMethod.generateModelStatesNFNew(valueDate,schedule,numMethodInfo,model);
                % numeraire info
                numMatTime = numMethodInfo.totalTimeSteps(end);
                days = DateDiff(schedule.payDates(schedule.numOfPeriod),valueDate);
                numMatTime = max(numMatTime, days);
                dfNumMat = model.DFRisky(numMatTime/365.0);
                
                numeraire.numMatTime = numMatTime;
                numeraire.dfNumMat = dfNumMat;
                
                % induct backward and calculate cashflow
            
                periodCount = schedule.numOfPeriod;
                
                % pricer single info initialize
                
                nonCall = PricerInfo(periodCount +1, pathSize);
                callable = PricerInfo(periodCount +1, pathSize);
                dfCol = PricerInfo(periodCount +1, pathSize);
                
                dummy1 = PricerInfo(periodCount +1, pathSize);
                dummy2 = PricerInfo(periodCount +1, pathSize);
                
%                 % additional floating leg info
%                 if ~isempty(fieldnames(floatingSchedule))
%                     floatingLeg = PricerInfo(floatingSchedule.numOfPeriod, pathSize);
%                     % currentFloatingLegIdx for iteration in dealing with 
%                     % the floatingLeg cashflow
%                     currentFloatingLegIdx = floatingSchedule.numOfPeriod;
%                 end
                
                % intermediate variables
                payoffStateA.cashflow = zeros(1, pathSize);
                payoffStateA.cashflow_npv = zeros(1, pathSize);
                
            
                % we calculate cashflow backward from maturityDate to valueDate
                % rangeAccrual coupons are added at the startDate of the Period
                % At maturity

                nominal = security.nominal;
                coupon = security.coupon;
                callYN = security.callYN;
                
                tenorCMSL = security.tenorCMSL;
                tenorCMSS = security.tenorCMSS;
                
                ralowerCMSLS = security.lowerCMSLS;
                raupperCMSLS = security.upperCMSLS;
                
                tenorLong = security.tenorL;
                tenorShort = security.tenorS;
                
                ralowerS = security.lowerS;
                raupperS = security.upperS;
                
                currentTime = numMethodInfo.totalTimeSteps(end);
                modelTimeStepsSize = numMethodInfo.modelTimeStepsSize;
                currentTimeIdx = numMethodInfo.totalTimeStepsSize;
                
                currentNodeIdx = periodCount + 1;
                
                modelStateT = modelStates(numOfFactors*(max(currentTimeIdx-pastTimeStepsSize-1,0))+1:numOfFactors*(max(currentTimeIdx-pastTimeStepsSize-1,0))+numOfFactors,:);
%                 modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:numOfFactors*(currentTimeIdx-1)+numOfFactors,:);
                
                payTime = DateDiff(schedule.payDates(schedule.numOfPeriod),valueDate);
                
                %even though there is no cashflow at the last structured
                %enddate, we add dummy cashflow in line with the note
                if payTime > 0
                    payoffStateA.cashflow = nominal*ones(1,pathSize);

                    df_ep = model.discountFactorRisky(currentTime, payTime,numMatTime,modelStateT);
                    for k=1:pathSize
                        payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                    end

                    %process payoff at maturity
                    nonCall.ProcessPayoff(model,currentTime,numeraire, ...
                                         modelStateT,payoffStateA.cashflow,currentNodeIdx);

                    callable.ProcessPayoff(model,currentTime,numeraire, ...
                                         modelStateT,payoffStateA.cashflow,currentNodeIdx);

                    %df column
                    payoffStateA.cashflow = ones(1,pathSize);
                    for k=1:pathSize
                        payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                    end

                    dfCol.ProcessPayoff(model,currentTime,numeraire, ...
                                         modelStateT,payoffStateA.cashflow,currentNodeIdx);

                    %dummy column
                    foreignModelName = model.foreignModelName;

                    if strcmp(pricerOption,'PSA')
                        payoffStateA.cashflow = model.CMS_PSA_SimpleDate(model.refModelName,valueDate,currentTime,numMatTime, tenorLong,modelStateT);
                        dummy1.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);

                        payoffStateA.cashflow = model.CMS_PSA_SimpleDate(foreignModelName,valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                        dummy2.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);

                    else
                        payoffStateA.cashflow = model.CMS_SimpleDate(model.refModelName,valueDate,currentTime,numMatTime, tenorLong,modelStateT);
                        dummy1.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);

                        payoffStateA.cashflow = model.CMS_SimpleDate(foreignModelName,valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                        dummy2.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);

                    end
                
                end

                lastTimeIdx = currentTimeIdx;
                totalTimeSteps = numMethodInfo.timeStepsInfo.totalTimeSteps;
                
                % flag for inclusive schedule
%                 simpleYN = false;
%                 simpleYN = true;
                if pricer.overHedgeYN == true 
                    simpleYN = 2;
                else
                    simpleYN = 1;
                end
                
                for i=periodCount:-1:1
                    % we induct in the pastTimeSteps too
%                     if DateDiff(valueDate,schedule.payDates(i)) >= 0, continue;end 
                    
                    startTimeIdx = numMethodInfo.timeStepsInfo.startTimeIdx(i);
                    endTimeIdx = numMethodInfo.timeStepsInfo.endTimeIdx(i);

                    inclusiveYN = numMethodInfo.timeStepsInfo.inclusiveYN(i);
                 %% FromToInduction start

                    %modelsStates induct backward(lastTimeIdx to endTimeIdx)
                    while endTimeIdx < lastTimeIdx
                        lastTimeIdx = lastTimeIdx -1;
                        if lastTimeIdx == endTimeIdx break;end
                    end

                    currentTimeIdx = lastTimeIdx;
                    currentTime = totalTimeSteps(currentTimeIdx);
                    % now we are at endTimeIdx
                    % variables for coupon valuation initialized
                    

                    cD = 0; % cumulated day count fraction
                     %cP shoul be declared as an array
%                     cP = 0;
                    cP = zeros(1,pathSize);
                    dI = 0; % dummy idx for loop
                    dcfC = 0;
                    dcfN = 0;
                    dD = 0;
                    
%                     idxCMS = zeros(1,pathSize);
%                     idxC = zeros(1,pathSize);
%                     idxN = zeros(1,pathSize);
                    
                    idxCMSL = zeros(1,pathSize);
                    idxCMSS = zeros(1,pathSize);
                    
                    idxCMSSpreadC = zeros(1,pathSize);
                    idxCMSSpreadN = zeros(1,pathSize);
                    
                    
                    idxL = zeros(1,pathSize);
                    idxS = zeros(1,pathSize);
                    
                    spreadC = zeros(1,pathSize);
                    spreadN = zeros(1,pathSize);
                    probSpread = zeros(1,pathSize);
                    
                    numOfTimeSteps = numMethodInfo.timeStepsInfo.numOfTimeSteps(i);
                    
                    % if inclusive then we use simpleYN rangeProb
                    %if inclusive then we do not evaluate the last step but
                    % use fixing instead 
                    endTimeStep = 1;
                     % we let endTimeStep to be 1 and
                    % inside the for loop if currentTime ==0
                    % then we skip the range accrual check part
                    
                    if inclusiveYN
%                         simpleYN = true;
                        simpleYN = 1;
%                         endTimeStep = 2;
                        if pricer.overHedgeYN == true 
                            simpleYN = 2;
                        end
                    end
                    
                    for j=numOfTimeSteps:-1:endTimeStep
                        % we initialize payoffStateA as zeros to prevent
                        % reuse it;
                        payoffStateA.cashflow = zeros(1,pathSize);
                        % evaluate FromTo dealdescription
                        % last step is omitted
                        currentTime = totalTimeSteps(currentTimeIdx);
                        if currentTime > 0
                            dcfC = currentTime;

                            if j~= numOfTimeSteps
                                dD = (dcfN-dcfC);
                            end

%                             modelStateT = modelStates(max(currentTimeIdx-pastTimeStepsSize-1,0)+1:numOfFactors*(currentTimeIdx-1)+numOfFactors,:);
                            modelStateT = modelStates(numOfFactors*(max(currentTimeIdx-pastTimeStepsSize-1,0))+1:numOfFactors*(max(currentTimeIdx-pastTimeStepsSize-1,0))+numOfFactors,:);
                            if strcmp(pricerOption,'PSA')
                                idxCMSL = model.CMS_PSA_SimpleDate(model.refModelName,valueDate,currentTime,numMatTime, tenorCMSL,modelStateT);
                                idxCMSS = model.CMS_PSA_SimpleDate(model.refModelName,valueDate,currentTime,numMatTime, tenorCMSS,modelStateT);
                                
                                idxCMSSpreadC = idxCMSL - idxCMSS;
%                                 idxC = idxCMSL;

                                idxL = model.CMS_PSA_SimpleDate(model.refModelName,valueDate,currentTime,numMatTime, tenorLong,modelStateT);
                                idxS = model.CMS_PSA_SimpleDate(foreignModelName,valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                                spreadC = idxL - idxS;
                            else
                                idxCMS = model.CMS_SimpleDate(foreignModelName,valueDate,currentTime,numMatTime, tenorCMS,modelStateT);
                                idxC = idxCMS;

                                idxL = model.CMS_SimpleDate(model.refModelName,valueDate,currentTime,numMatTime, tenorLong,modelStateT);
                                idxS = model.CMS_SimpleDate(model.refModelName,valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                                spreadC = idxL - idxS;
                            end
    %                         probSpread = model.SRPr(spreadC,spreadN,ralower,raupper,simpleYN,pathSize)*dD;
                            probSpread = model.DRPr(idxCMSSpreadC,idxCMSSpreadN,ralowerCMSLS,raupperCMSLS,spreadC,spreadN,ralowerS,raupperS,simpleYN,pathSize)*dD;
                            cD = cD + dD;
                            cP = cP + probSpread;

                            dcfN = dcfC;

                            idxCMSSpreadN = idxCMSSpreadC;
%                             idxN = idxC;
                            
                            spreadN = spreadC;
                        else
                            dcfC = currentTime;

                            if j~= numOfTimeSteps
                                dD = (dcfN-dcfC);
                            end

                            cD = cD + dD;

                            dcfN = dcfC;
                            
                        end
                        
                        %floating event begins here
                        %floating event end here
                        
                        % callable event begin
                        idxCall = find(currentTime == outInfo.callEventTime);
                        
                        % we do not evaluate callable event at the start of
                        % coupon Schedule for avoding double counting
                        % and that make sense
                        if ~isempty(idxCall) && (j~= endTimeStep)
                            iterCall = 1;
                            foundIterCallYN= false;
                            for idx11 = length(callSchedule.resetDates):-1:1
                                % call event happens at call startDate !
                                % we change call event happens at call resetDate !
                                
                                if outInfo.callEventTime(idxCall) == DateDiff(callSchedule.resetDates(idx11),valueDate)
                                    iterCall = idx11;
                                    foundIterCallYN = true;
                                    break;
                                end
                            end
                            
                            if ~foundIterCallYN
                                error('callEventTime is not in the callSchedule startDates')
                            end
                            
                            % only those cashflow that paydate is after
                            % valueDate will be added
                            if DateDiff(callSchedule.payDates(iterCall),valueDate) > 0
                                % callEvent is not fixed
                                if DateDiff(valueDate,callSchedule.resetDates(iterCall)) < 0
                                    % 1.undiscount callable.payoff
                                    % 2.build the state variables matrix
                                    % 3.regress 1's with the given state variables
                                    % (model states 1,2)
                                    % 4. exercise per each path in
                                    % discount the new payoff and assign into callable
                                    % column

                                    %1 undiscount callable.payoff and get the
                                    %contiValue

                                    %exercise value Note
                                    exerciseValue = security.nominal*ones(1,pathSize) * 1.0;
                                    payTime = DateDiff(callSchedule.payDates(iterCall),valueDate);
                                    %event is at startDate so we need to discount the
                                    %cashflow first from paydate to startdate
                                    df_ep = model.discountFactorRisky(currentTime, payTime,numMatTime,modelStateT);
                                    for k=1:pathSize
                                        exerciseValue(k) = exerciseValue(k) * df_ep(k);
                                    end
                                    
                                    contiValue = zeros(1,pathSize);
                                    onePayoff = ones(1,pathSize);
                                    onePayoffD = model.discountPayoffRisky(currentTime,numMatTime ...
                                    ,modelStateT,onePayoff);
                                    contiValue = callable.payoff./onePayoffD;

                                    %Nominal Regression test
                                    contiValue = contiValue/10000;

                                    % build the state variabls matrix
                                    if numMethod.regressionOrder == 3
            %                             dimensionLS = (numOfFactors + 1)*(numOfFactors + 2)/2;

                                        decisionVar = zeros(numOfFactors,size(modelStateT,2));

                                        for idxLS =1:numOfFactors
                                            decisionVar(idxLS,:) = modelStateT(idxLS,:); 
                                        end

                                        % calculate number of dimension first
                                        % 0th order
                                        dimensionLS = 1;

                                        % 1st order
                                        for idxLS=1:numOfFactors
                                            dimensionLS = dimensionLS + 1;
                                        end

                                        %2nd order
                                        for idxLS1=1:numOfFactors
                                            for idxLS2=idxLS1:numOfFactors
                                                dimensionLS = dimensionLS + 1;
                                            end
                                        end

                                        %3rd order
                                        for idxLS1=1:numOfFactors
                                            for idxLS2=idxLS1:numOfFactors
                                                for idxLS3=idxLS2:numOfFactors
                                                    dimensionLS = dimensionLS + 1;
                                                end
                                            end
                                        end


                %                         decisionVar1 = modelStateT(1,:);
                %                         decisionVar2 = modelStateT(2,:);
                                        % we use upto 2nd order polynomial as regression
                                        % equatioin

                                        regMatrix = ones(pathSize,dimensionLS);

                                        for idx_r=1:pathSize
                                            for idxLS=1:numOfFactors
                                                regMatrix(idx_r,idxLS+1) = decisionVar(idxLS,idx_r);
                                            end

                                            tempIdx = 0;
                                            for idxLS1=1:numOfFactors
                                                for idxLS2=idxLS1:numOfFactors
                                                    tempIdx = tempIdx + 1;
                                                    regMatrix(idx_r,numOfFactors+1+tempIdx) = decisionVar(idxLS1,idx_r)*decisionVar(idxLS2,idx_r);
                                                end
                                            end

                                            tempIdx2 = 0;
                                            for idxLS1=1:numOfFactors
                                                for idxLS2=idxLS1:numOfFactors
                                                    for idxLS3=idxLS2:numOfFactors
                                                        tempIdx2 = tempIdx2 + 1;
                                                        regMatrix(idx_r,numOfFactors+1+tempIdx + tempIdx2) = decisionVar(idxLS1,idx_r)*decisionVar(idxLS2,idx_r)*decisionVar(idxLS3,idx_r);
                                                    end
                                                end
                                            end


                                        end

                                    else
                                        dimensionLS = (numOfFactors + 1)*(numOfFactors + 2)/2;

                                        decisionVar = zeros(numOfFactors,size(modelStateT,2));

                                        for idxLS =1:numOfFactors
                                            decisionVar(idxLS,:) = modelStateT(idxLS,:); 
                                        end
                %                         decisionVar1 = modelStateT(1,:);
                %                         decisionVar2 = modelStateT(2,:);
                                        % we use upto 2nd order polynomial as regression
                                        % equatioin

                                        regMatrix = ones(pathSize,dimensionLS);

                                        for idx_r=1:pathSize
                                            for idxLS=1:numOfFactors
                                                regMatrix(idx_r,idxLS+1) = decisionVar(idxLS,idx_r);
                                            end

                                            tempIdx = 1;
                                            for idxLS1=1:numOfFactors
                                                for idxLS2=idxLS1:numOfFactors
                                                    regMatrix(idx_r,numOfFactors+1+tempIdx) = decisionVar(idxLS1,idx_r)*decisionVar(idxLS2,idx_r);
                                                    tempIdx = tempIdx + 1;
                                                end
                                            end

                                        end
                                    end
                                    % simple regression <- for path dependent option
                                    % we should change it
                                    % beta  : regression coefficients
                                    beta =zeros(dimensionLS,1);
                                    beta = regress(contiValue',regMatrix);
                                    regContiValue = regMatrix*beta;

                                     %Nominal Regression test
                                    regContiValue = regContiValue * 10000;

                                    exercisedPayoff = regContiValue;
                                    % if regContiValue is bigger than exerciseValue
                                    % then issue exercise call option
                                    for idx_r=1:pathSize
                                        if regContiValue(idx_r) >= exerciseValue(idx_r) 
                                            exercisedPayoff(idx_r) = exerciseValue(idx_r);
                                        end
                                    end

                                    % we discount the new payoff and 
                                    % assign it into the existing callable payoff
                                    callable.payoff = model.discountPayoffRisky(currentTime,numMatTime ...
                                    ,modelStateT,exercisedPayoff);

%                                     contiValue = zeros(1,pathSize);
%                                     onePayoff = ones(1,pathSize);
%                                     onePayoffD = model.discountPayoffRisky(currentTime,numMatTime ...
%                                     ,modelStateT,onePayoff);
%                                     contiValue = callable.payoff./onePayoffD;
% 
%                                     %Nominal Regression test
%                                     contiValue = contiValue/10000;
% 
%                                     % build the state variabls matrix
%                                     decisionVar1 = modelStateT(1,:);
%                                     decisionVar2 = modelStateT(2,:);
%                                     % we use upto 2nd order polynomial as regression
%                                     % equatioin
%                                     dimensionLS = 6;
%                                     regMatrix = ones(pathSize,dimensionLS);
% 
%                                     for idx_r=1:pathSize
%                                         regMatrix(idx_r,2) = decisionVar1(idx_r);
%                                         regMatrix(idx_r,3) = decisionVar2(idx_r);
%                                         regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r);
%                                         regMatrix(idx_r,5) = decisionVar1(idx_r)*decisionVar2(idx_r);
%                                         regMatrix(idx_r,6) = decisionVar2(idx_r)*decisionVar2(idx_r);
%                                     end
% 
%                                     % simple regression <- for path dependent option
%                                     % we should change it
%                                     % beta  : regression coefficients
%                                     beta =zeros(dimensionLS,1);
%                                     beta = regress(contiValue',regMatrix);
%                                     regContiValue = regMatrix*beta;
% 
%                                      %Nominal Regression test
%                                     regContiValue = regContiValue * 10000;
% 
%                                     exercisedPayoff = regContiValue;
%                                     % if regContiValue is bigger than exerciseValue
%                                     % then issue exercise call option
%                                     for idx_r=1:pathSize
%                                         if regContiValue(idx_r) >= exerciseValue(idx_r) 
%                                             exercisedPayoff(idx_r) = exerciseValue(idx_r);
%                                         end
%                                     end
% 
%                                     % we discount the new payoff and 
%                                     % assign it into the existing callable payoff
%                                     callable.payoff = model.discountPayoffRisky(currentTime,numMatTime ...
%                                     ,modelStateT,exercisedPayoff);

                                end
                                
                                
                            end
                        end
                        
                        % callable event end
                        
                        %induct backward to the next time step
                        % one time backward induction
                        lastTimeIdx = currentTimeIdx;

                        if startTimeIdx < lastTimeIdx
                            lastTimeIdx = lastTimeIdx -1;
                            currentTimeIdx = lastTimeIdx;
                        end


                    end

                    %% FromToInduction end
        % if the current period is inclusive, then the start date is not evaluated in the above for loop.
		% instead the fixing will be treated separately below.
		% however current_time_idx is updated at the end of the for loop.
		% so we need to update model_states accordingly.
		% if model_states were updated previously then the following additionaly step doesn't affect the result anyway.
                    currentTime = totalTimeSteps(currentTimeIdx);
%                     modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:numOfFactors*(currentTimeIdx-1)+numOfFactors,:);
                    modelStateT = modelStates(numOfFactors*(max(currentTimeIdx-pastTimeStepsSize-1,0))+1:numOfFactors*(max(currentTimeIdx-pastTimeStepsSize-1,0))+numOfFactors,:);
                    
                    %% AtDate evaluation
                    % Adding a new cashflow to the payoff
                    if DateDiff(schedule.payDates(i),valueDate) > 0
                        dcf = schedule.dayCountFraction(i);

                        if inclusiveYN
                            % we add valuation date
                            cD = schedule.dayCount(i);
                            rangeDays =  security.rangeDays(i);
                            payoffStateA.cashflow = nominal * coupon * dcf * (rangeDays + cP)/cD;
                        else
                            payoffStateA.cashflow = nominal * coupon * dcf * cP/cD;
                        end
                        % eventDate(i) = startDate(i) therefore there is no induction in modelStates
                        % i.e. currentTimeIdx has not changed
                        % no need to update modelStateT
                        % modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:end,:);
                        % since cashflow's are added at the startDate(i) we need to
                        % multiply cashflow with the df(startDate(i), payDate(i))
                        payTime = DateDiff(schedule.payDates(i),valueDate);

                        %event is at startDate so we need to discount the
                        %cashflow first from paydate to startdate
                        df_ep = model.discountFactorRisky(currentTime, payTime,numMatTime,modelStateT);

                        for k=1:pathSize
                            payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                        end

                        % process payoff

                        currentNodeIdx = i;

                        nonCall.ProcessPayoff(model,currentTime,numeraire, ...
                                         modelStateT,payoffStateA.cashflow,currentNodeIdx);

                        callable.ProcessPayoff(model,currentTime,numeraire, ...
                                         modelStateT,payoffStateA.cashflow,currentNodeIdx);

                        %df column
                        payoffStateA.cashflow = ones(1,pathSize);
                        for k=1:pathSize
                            payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                        end

                        dfCol.ProcessPayoff(model,currentTime,numeraire, ...
                                         modelStateT,payoffStateA.cashflow,currentNodeIdx);

                        %dummy column
                        if strcmp(pricerOption,'PSA')
                            payoffStateA.cashflow = model.CMS_PSA_SimpleDate(model.refModelName,valueDate,currentTime,numMatTime, tenorLong,modelStateT);
                            dummy1.ProcessPayoff(model,currentTime,numeraire, ...
                                             modelStateT,payoffStateA.cashflow,currentNodeIdx);



                            payoffStateA.cashflow = model.CMS_PSA_SimpleDate(foreignModelName,valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                            dummy2.ProcessPayoff(model,currentTime,numeraire, ...
                                             modelStateT,payoffStateA.cashflow,currentNodeIdx);

                        else
                            payoffStateA.cashflow = model.CMS_SimpleDate(model.refModelName,valueDate,currentTime,numMatTime, tenorLong,modelStateT);
                            dummy1.ProcessPayoff(model,currentTime,numeraire, ...
                                             modelStateT,payoffStateA.cashflow,currentNodeIdx);



                            payoffStateA.cashflow = model.CMS_SimpleDate(foreignModelName,valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                            dummy2.ProcessPayoff(model,currentTime,numeraire, ...
                                             modelStateT,payoffStateA.cashflow,currentNodeIdx);

                        end

                    end           
                    
                    
                    
                    % if exercise is possible
                    if i == 1
                        dummaaa = 1;
                    end
                    
                    % call event happens  at the start of the period
                    % at call date, by default, call option is expired
                    
%                     if callYN(i) && (DateDiff(valueDate,callSchedule.resetDates(i)) < 0)
%                         % 1.undiscount callable.payoff
%                         % 2.build the state variables matrix
%                         % 3.regress 1's with the given state variables
%                         % (model states 1,2)
%                         % 4. exercise per each path in
%                         % discount the new payoff and assign into callable
%                         % column
%                         
%                         %1 undiscount callable.payoff and get the
%                         %contiValue
%                        
%                         %exercise value
%                         exerciseValue = security.nominal*ones(1,pathSize);
%                         payTime = DateDiff(callSchedule.payDates(i),valueDate);
%                         %event is at startDate so we need to discount the
%                         %cashflow first from paydate to startdate
%                         df_ep = model.discountFactorRisky(currentTime, payTime,numMatTime,modelStateT);
%                         for k=1:pathSize
%                             exerciseValue(k) = exerciseValue(k) * df_ep(k);
%                         end
%                         
%                         contiValue = zeros(1,pathSize);
%                         onePayoff = ones(1,pathSize);
%                         onePayoffD = model.discountPayoffRisky(currentTime,numMatTime ...
%                         ,modelStateT,onePayoff);
%                         contiValue = callable.payoff./onePayoffD;
%                         
%                         %Nominal Regression test
%                         contiValue = contiValue/10000;
%                         
%                         % build the state variabls matrix
%                         if numMethod.regressionOrder == 3
% %                             dimensionLS = (numOfFactors + 1)*(numOfFactors + 2)/2;
% 
%                             decisionVar = zeros(numOfFactors,size(modelStateT,2));
% 
%                             for idxLS =1:numOfFactors
%                                 decisionVar(idxLS,:) = modelStateT(idxLS,:); 
%                             end
%                             
%                             % calculate number of dimension first
%                             % 0th order
%                             dimensionLS = 1;
%                             
%                             % 1st order
%                             for idxLS=1:numOfFactors
%                                 dimensionLS = dimensionLS + 1;
%                             end
%                             
%                             %2nd order
%                             for idxLS1=1:numOfFactors
%                                 for idxLS2=idxLS1:numOfFactors
%                                     dimensionLS = dimensionLS + 1;
%                                 end
%                             end
%                             
%                             %3rd order
%                             for idxLS1=1:numOfFactors
%                                 for idxLS2=idxLS1:numOfFactors
%                                     for idxLS3=idxLS2:numOfFactors
%                                         dimensionLS = dimensionLS + 1;
%                                     end
%                                 end
%                             end
% 
%                             
%     %                         decisionVar1 = modelStateT(1,:);
%     %                         decisionVar2 = modelStateT(2,:);
%                             % we use upto 2nd order polynomial as regression
%                             % equatioin
% 
%                             regMatrix = ones(pathSize,dimensionLS);
% 
%                             for idx_r=1:pathSize
%                                 for idxLS=1:numOfFactors
%                                     regMatrix(idx_r,idxLS+1) = decisionVar(idxLS,idx_r);
%                                 end
% 
%                                 tempIdx = 0;
%                                 for idxLS1=1:numOfFactors
%                                     for idxLS2=idxLS1:numOfFactors
%                                         tempIdx = tempIdx + 1;
%                                         regMatrix(idx_r,numOfFactors+1+tempIdx) = decisionVar(idxLS1,idx_r)*decisionVar(idxLS2,idx_r);
%                                     end
%                                 end
%                                 
%                                 tempIdx2 = 0;
%                                 for idxLS1=1:numOfFactors
%                                     for idxLS2=idxLS1:numOfFactors
%                                         for idxLS3=idxLS2:numOfFactors
%                                             tempIdx2 = tempIdx2 + 1;
%                                             regMatrix(idx_r,numOfFactors+1+tempIdx + tempIdx2) = decisionVar(idxLS1,idx_r)*decisionVar(idxLS2,idx_r)*decisionVar(idxLS3,idx_r);
%                                         end
%                                     end
%                                 end
%                                 
% 
%                             end
%                         
%                         else
%                             dimensionLS = (numOfFactors + 1)*(numOfFactors + 2)/2;
% 
%                             decisionVar = zeros(numOfFactors,size(modelStateT,2));
% 
%                             for idxLS =1:numOfFactors
%                                 decisionVar(idxLS,:) = modelStateT(idxLS,:); 
%                             end
%     %                         decisionVar1 = modelStateT(1,:);
%     %                         decisionVar2 = modelStateT(2,:);
%                             % we use upto 2nd order polynomial as regression
%                             % equatioin
% 
%                             regMatrix = ones(pathSize,dimensionLS);
% 
%                             for idx_r=1:pathSize
%                                 for idxLS=1:numOfFactors
%                                     regMatrix(idx_r,idxLS+1) = decisionVar(idxLS,idx_r);
%                                 end
% 
%                                 tempIdx = 1;
%                                 for idxLS1=1:numOfFactors
%                                     for idxLS2=idxLS1:numOfFactors
%                                         regMatrix(idx_r,numOfFactors+1+tempIdx) = decisionVar(idxLS1,idx_r)*decisionVar(idxLS2,idx_r);
%                                         tempIdx = tempIdx + 1;
%                                     end
%                                 end
% 
%                             end
%                         end
%                         % simple regression <- for path dependent option
%                         % we should change it
%                         % beta  : regression coefficients
%                         beta =zeros(dimensionLS,1);
%                         beta = regress(contiValue',regMatrix);
%                         regContiValue = regMatrix*beta;
%                        
%                          %Nominal Regression test
%                         regContiValue = regContiValue * 10000;
%                         
%                         exercisedPayoff = regContiValue;
%                         % if regContiValue is bigger than exerciseValue
%                         % then issue exercise call option
%                         for idx_r=1:pathSize
%                             if regContiValue(idx_r) >= exerciseValue(idx_r) 
%                                 exercisedPayoff(idx_r) = exerciseValue(idx_r);
%                             end
%                         end
%                         
%                         % we discount the new payoff and 
%                         % assign it into the existing callable payoff
%                         callable.payoff = model.discountPayoffRisky(currentTime,numMatTime ...
%                         ,modelStateT,exercisedPayoff);
%                         
%                     end
                end

                nonCall.npv = mean(nonCall.payoff);
                out.nonCall = nonCall;
                callable.npv = mean(callable.payoff);
                dfCol.npv = mean(dfCol.payoff);
                dummy1.npv = mean(dummy1.npv);
                dummy2.npv = mean(dummy2.npv);
                out.callable = callable;
                out.dfCol = dfCol;
                out.dummy1 = dummy1;
                out.dummy2 = dummy2;
                out.daycountFraction = schedule.dayCountFraction;
                
        end
        
        
        function out = computePriceClosedSimple(pricer,valueDate)
                security = pricer.m_security;
                numMethod = pricer.m_numMethod;
                model = pricer.m_model;
                schedule = security.couponSchedule;
                
                % numMethodTimeSteps Generate
                mcOneTimeStep = numMethod.mcOneTimeStep;
                outInfo = numMethod.numMethodTimeStepsGenerateDW(valueDate,schedule,mcOneTimeStep);
            
                numMethodInfo.timeStepsInfo = outInfo;

                % mc random number init
                %pathSize = 2^16;
                pathSize = numMethod.pathSize;
                
                pathSize = 1;
                
%                 % generate random numbers and modelstates forward
%                 rng('default');
%                 rng(0);
% 
                numOfFactors = model.numOfFactor;
% 
%                 %Z= randn(numOfFactors*outInfo.totalTimeStepsSize,pathSize/2);
%                 Z= randn(pathSize/2,numOfFactors*outInfo.totalTimeStepsSize);
%                 Z=[Z;-Z];
%                 Z=Z';
%                 
%                 
%                 %TimeInversion
%                 sizeZ = size(Z,1);
%                 A = zeros(sizeZ,sizeZ);
%                 for i=1:sizeZ
%                     A(i,sizeZ + 1 -i) = 1;
%                 end
%                 Z = A*Z;
% 
%                 % we check with the c, so we use generated random numbers
% %                 mc_random = importdata('mc_random.xlsx');
% %                 mc_random = importdata('D:\mc_random_number_new_report.txt');
% %                 Z= mc_random;
%                 
                numMethodInfo.pathSize = pathSize;
                numMethodInfo.numOfFactors = numOfFactors;
                numMethodInfo.totalTimeSteps = outInfo.totalTimeSteps;
                numMethodInfo.totalTimeStepsSize = outInfo.totalTimeStepsSize;
%                 numMethodInfo.Z = Z;
% 
% 
% 
%                 %generate model states for the given timeSteps
%                 modelStates = numMethod.generateModelStates(valueDate,schedule,numMethodInfo,model);

                % numeraire info
                numMatTime = numMethodInfo.totalTimeSteps(end);
                dfNumMat = model.DFRisky(numMatTime/365.0);
                
                % induct backward and calculate cashflow
            
                periodCount = schedule.numOfPeriod;
                
                % cashflow columns initialize
                % 1st nonCall
                nonCall.npv = 0;
                nonCall.cashflow_e = zeros(periodCount + 1,1);
                nonCall.cashflow_npv_e = zeros(periodCount + 1,1);
                nonCall.rangeProb_e = zeros(periodCount + 1,1);
                nonCall.df_e = zeros(periodCount + 1,1);
                nonCall.df_c = zeros(periodCount + 1,1);

                % debug information, cashflow & npv per path
                nonCall.payoff = zeros(1,pathSize);
                nonCall.payoffStates.cashflow = zeros(periodCount+1,pathSize);
                nonCall.payoffStates.cashflow_npv = zeros(periodCount+1,pathSize);
                nonCall.payoffStates.df = zeros(periodCount + 1,pathSize);
                
                % intermediate variables
                payoffStateA.cashflow = zeros(1, pathSize);
                payoffStateA.cashflow_npv = zeros(1, pathSize);
                payoffStateA.df = zeros(1,pathSize);
                
                % we calculate cashflow backward from maturityDate to valueDate
                % rangeAccrual coupons are added at the startDate of the Period
                % At maturity

                nominal = security.nominal;
                coupon = security.coupon;
                callYN = security.callYN;
                
                tenorCMS = security.tenor;
                
                ralower = security.lower;
                raupper = security.upper;
                
                tenorLong = security.tenorL;
                tenorShort = security.tenorS;
                
                ralowerS = security.lowerS;
                raupperS = security.upperS;
                
                currentTime = numMethodInfo.totalTimeSteps(end);
                totalTimeSteps = numMethodInfo.timeStepsInfo.totalTimeSteps;
                totalTimeStepsSize = numMethodInfo.timeStepsInfo.totalTimeStepsSize;
                if totalTimeStepsSize == 0 || (totalTimeStepsSize == 1 && totalTimeSteps(1) == 0) 
                    out.nonCall = nonCall;
                    return;
                end
                
                currentTimeIdx = totalTimeStepsSize;
                
                currentNodeIdx = periodCount + 1;
                
%                 modelStateT = modelStates(numOfFactors*(totalTimeStepsSize-1)+1:numOfFactors*(totalTimeStepsSize-1)+2,:);
                payoffStateA.cashflow = nominal*ones(1,pathSize);
                %process payoff at maturity
                nonCall.payoffStates.cashflow(currentNodeIdx,:) = payoffStateA.cashflow;
                
%                 payoffStateA.cashflow_npv = model.discountPayoff(currentTime,numMatTime ...
%                     ,modelStateT,payoffStateA.cashflow);
%                 
                payoffStateA.cashflow_npv = model.DFRisky(currentTime/365.0)*payoffStateA.cashflow;
                
                nonCall.payoffStates.cashflow_npv(currentNodeIdx,:) = payoffStateA.cashflow_npv ;
                
%                 payoffStateA.df = model.discountPayoff(currentTime,numMatTime ...
%                     ,modelStateT,1.0*ones(1,pathSize));
                payoffStateA.df = model.DFRisky(currentTime/365.0) *ones(1,pathSize);
                    

                nonCall.payoffStates.df(currentNodeIdx,:) = payoffStateA.df;
                nonCall.payoff = nonCall.payoff + payoffStateA.cashflow_npv;
                
                nonCall.cashflow_e(currentNodeIdx) = mean(payoffStateA.cashflow);
                nonCall.cashflow_npv_e(currentNodeIdx) = mean(payoffStateA.cashflow_npv);
                nonCall.df_e(currentNodeIdx) = mean(payoffStateA.df);
                nonCall.df_c(currentNodeIdx) = model.DFRisky(currentTime/365.0);
                
%                 % callable at maturity is the same as noncallable
%                 callable.payoffStates.cashflow(currentNodeIdx,:) = nonCall.payoffStates.cashflow(currentNodeIdx,:);
%                 callable.payoffStates.cashflow_npv(currentNodeIdx,:) = nonCall.payoffStates.cashflow_npv(currentNodeIdx,:);
%                 callable.payoff = callable.payoff + callable.payoffStates.cashflow_npv(currentNodeIdx,:);
%                 
%                 callable.cashflow_e(currentNodeIdx) = nonCall.cashflow_e(currentNodeIdx);
%                 callable.cashflow_npv_e(currentNodeIdx) = nonCall.cashflow_npv_e(currentNodeIdx);
                  
                lastTimeIdx = currentTimeIdx;
                
                
                % flag for inclusive schedule
%                 simpleYN = false;
                simpleYN = 1;
                
                for i=periodCount:-1:1
                    % past schedule neglect
                    if DateDiff(valueDate,schedule.payDates(i)) >= 0, continue;end 
                    startTimeIdx = numMethodInfo.timeStepsInfo.startTimeIdx(i);
                    endTimeIdx = numMethodInfo.timeStepsInfo.endTimeIdx(i);

                    inclusiveYN = numMethodInfo.timeStepsInfo.inclusiveYN(i);
                     %% FromToInduction start

                    %modelsStates induct backward(lastTimeIdx to endTimeIdx)
                    while endTimeIdx < lastTimeIdx
                        lastTimeIdx = lastTimeIdx -1;
                        if lastTimeIdx == endTimeIdx break;end
                    end

                    currentTimeIdx = lastTimeIdx;
                    currentTime = totalTimeSteps(currentTimeIdx);
                    % now we are at endTimeIdx
                    % variables for coupon valuation initialized
%                     ralower = security.lower;
%                     raupper = security.upper;
%                     tenorLong = security.tenorL;
%                     tenorShort = security.tenorS;

                    cD = 0; % cumulated day count fraction
                    cP = 0; 
                    dI = 0; % dummy idx for loop
                    dcfC = 0;
                    dcfN = 0;
                    dD = 0;
                    idxCMS = zeros(1,pathSize);
                    
                    idxC = zeros(1,pathSize);
                    idxN = zeros(1,pathSize);
                    
                    idxL = zeros(1,pathSize);
                    idxS = zeros(1,pathSize);
                    spreadC = zeros(1,pathSize);
                    spreadN = zeros(1,pathSize);
                    probSpread = zeros(1,pathSize);
                    
                    numOfTimeSteps = numMethodInfo.timeStepsInfo.numOfTimeSteps(i);
                    
                    % if inclusive then we use simpleYN rangeProb
                    %if inclusive then we do not evaluate the last step but
                    % use fixing instead 
                    endTimeStep = 1;
                    if inclusiveYN
%                         simpleYN = true;
                        simpleYN = 1;
                        endTimeStep = 2;
                    end
                    
                    for j=numOfTimeSteps:-1:endTimeStep
                        % evaluate dealdescription
                        % last step is omitted
                        currentTime = totalTimeSteps(currentTimeIdx);
                        %dcfC = currentTime/365.0;
                        dcfC = currentTime;
                        
                        if j~= numOfTimeSteps
                            dD = (dcfN-dcfC);
                        end

                        toDate = AddDate(valueDate,currentTime,'day');
                        tpDate = schedule.payDates(i);
%                         spreadC = model.CMSCMTSpreadDigitalRangeDate(valueDate,toDate,tpDate,tenorLong,tenorShort,ralower,raupper);
                        spreadC = model.CMSCMSSpreadDigitalRangeDate(valueDate,toDate,tpDate,tenorCMS,tenorLong,tenorShort,0.25,ralower,raupper,ralowerS,raupperS);
                        
                        %modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:numOfFactors*(currentTimeIdx-1)+2,:);
%                         idxL = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorLong,modelStateT);
%                         idxS = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorShort,modelStateT);
%                         spreadC = idxL - idxS;
%                         probSpread = model.SRPr(spreadC,spreadN,ralower,raupper,simpleYN,pathSize)*dD; 

                        if simpleYN
                            probSpread = spreadC*dD;
                        else
                            probSpread = spreadC*dD;
%                             probSpread = 0.5*(spreadC + spreadN)*dD;
                            
                        end

                        
                        cD = cD + dD;
                        cP = cP + probSpread;

                        dcfN = dcfC;
                        spreadN = spreadC;

                        %induct backward to the next time step
                        % one time backward induction
                        lastTimeIdx = currentTimeIdx;

                        if startTimeIdx < lastTimeIdx
                            lastTimeIdx = lastTimeIdx -1;
                            currentTimeIdx = lastTimeIdx;
                        end


                    end

                    %% FromToInduction end
                    %% AtDate evaluation
                    % Adding a new cashflow to the payoff

                    dcf = schedule.dayCountFraction(i);
                    
                    if inclusiveYN
                        % we add valuation date
                        cD = schedule.dayCount(i);
                        rangeDays =  security.rangeDays(i);
                        payoffStateA.cashflow = nominal * coupon * dcf * (rangeDays + cP)/cD;
                    else
                        payoffStateA.cashflow = nominal * coupon * dcf * cP/cD;
                    end
                    % eventDate(i) = startDate(i) therefore there is no induction in modelStates
                    % i.e. currentTimeIdx has not changed
                    % no need to update modelStateT
                    % modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:end,:);
                    % since cashflow's are added at the startDate(i) we need to
                    % multiply cashflow with the df(startDate(i), payDate(i))
                    payTime = DateDiff(schedule.payDates(i),valueDate);
                    
                    %event is at startDate so we need to discount the
                    %cashflow first from paydate to startdate
%                     df_sp = model.discountFactor(currentTime, payTime,numMatTime,modelStateT);

%                     for k=1:pathSize
%                         payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_sp(k);
%                     end
                    
                    % process payoff
                    
                    currentNodeIdx = i;
                    
                    % 1st nonCall column
                    nonCall.payoffStates.cashflow(currentNodeIdx,:) = payoffStateA.cashflow;
                    payoffStateA.cashflow_npv = payoffStateA.cashflow * model.DFRisky(payTime/365.0);
                    
%                     payoffStateA.cashflow_npv = model.discountPayoff(currentTime,numMatTime ...
%                     ,modelStateT,payoffStateA.cashflow);
                    nonCall.payoffStates.cashflow_npv(currentNodeIdx,:) = payoffStateA.cashflow_npv;
                    payoffStateA.df = model.DFRisky(payTime/365.0);
%                     payoffStateA.df = model.discountPayoff(currentTime,numMatTime ...
%                     ,modelStateT,1.0*ones(1,pathSize));
                    nonCall.payoffStates.df(currentNodeIdx,:) = payoffStateA.df;
                    
                    nonCall.payoff = nonCall.payoff + payoffStateA.cashflow_npv;
                    
                    nonCall.cashflow_e(currentNodeIdx) = mean(payoffStateA.cashflow);
                    nonCall.rangeProb_e(currentNodeIdx) = mean(nonCall.cashflow_e(currentNodeIdx)/(nominal * coupon * dcf));
                    nonCall.cashflow_npv_e(currentNodeIdx) = mean(payoffStateA.cashflow_npv);

                    nonCall.df_e(currentNodeIdx) = mean(payoffStateA.df);
                    nonCall.df_c(currentNodeIdx) = model.DFRisky(currentTime/365.0);
                    

                end

                nonCall.npv = mean(nonCall.payoff);
                out.pricerInfo.nonCall = nonCall;
                
%                 callable.npv = mean(callable.payoff);
%                 out.pricerInfo.callable = callable;
                out.daycountFraction = schedule.dayCountFraction;
                
        end
        

        
        
        
        
    end
    
end



In [ ]:
classdef DailyWeeklyRiskyPricer_CDCMSSpreadRASwapSimple < H_RootObject
    %Pricer(Security,Model,NumMethod)
    %Prcer is constructed from Security, Model, NumMethod 
    
    properties
        m_security
        m_model
        m_numMethod
        overHedgeYN
    end
    
    methods
        function pricer = DailyWeeklyRiskyPricer_CDCMSSpreadRASwapSimple(security,model,numMethod,overHedgeYN)
                pricer.m_security = security;
                pricer.m_model = model;
                pricer.m_numMethod = numMethod;
                pricer.overHedgeYN = overHedgeYN;
        end
        
        function out = computePriceMCCallableGeneric(pricer,valueDate)
                security = pricer.m_security;
                numMethod = pricer.m_numMethod;
                model = pricer.m_model;
                schedule = security.couponSchedule;
                callSchedule = security.callSchedule;
                
                floatingSchedule = security.floatingSchedule;
                floatingSpread = security.floatingSpread;
                floatingIndex = security.floatingIndex;
                
                % numMethodTimeSteps Generate
                mcOneTimeStep = numMethod.mcOneTimeStep;
%                 outInfo = numMethod.numMethodTimeStepsGenerateDW(valueDate,...
%                                     schedule,mcOneTimeStep);
%                                 

                outInfo = numMethod.numMethodTimeStepsGenerateDWSwapSimple(valueDate,...
                                    schedule,floatingSchedule,mcOneTimeStep);
                
                numMethodInfo.timeStepsInfo = outInfo;

                % mc random number init
                %pathSize = 2^16;
                pathSize = numMethod.pathSize;

                % generate random numbers and modelstates forward
                rng('default');
                rng(0);

                numOfFactors = model.numOfFactor;

                %Z= randn(numOfFactors*outInfo.totalTimeStepsSize,pathSize/2);
%                 Z= randn(pathSize/2,numOfFactors*outInfo.totalTimeStepsSize);
                Z= randn(pathSize/2,numOfFactors*outInfo.modelTimeStepsSize);
                Z=[Z;-Z];
                Z=Z';
                
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;

                % we check with the c code, so we use generated random numbers
%                 mc_random = importdata('mc_random.xlsx');
%                 mc_random = importdata('D:\mc_random_number_new_report.txt');
%                 Z= mc_random;
                
                numMethodInfo.pathSize = pathSize;
                numMethodInfo.numOfFactors = numOfFactors;
                
                numMethodInfo.totalTimeSteps = outInfo.totalTimeSteps;
                numMethodInfo.totalTimeStepsSize = outInfo.totalTimeStepsSize;
                
                totalTimeSteps = numMethodInfo.totalTimeSteps;
                totalTimeStepsSize = numMethodInfo.totalTimeStepsSize;
                
                % for modelStateGeneration
                numMethodInfo.modelTimeSteps = outInfo.modelTimeSteps;
                numMethodInfo.modelTimeStepsSize = outInfo.modelTimeStepsSize;
                
                numMethodInfo.pastTimeStepsSize = outInfo.pastTimeStepsSize;
                
                pastTimeStepsSize = numMethodInfo.pastTimeStepsSize;
                
                numMethodInfo.Z = Z;

                

                %generate model states for the given timeSteps
                modelStates = numMethod.generateModelStatesNFNew(valueDate,schedule,numMethodInfo,model);

                % numeraire info
%                 numMatTime = numMethodInfo.totalTimeSteps(end - numMethodInfo.pastTimeStepsSize);
                numMatTime = numMethodInfo.totalTimeSteps(end);
                
                days = DateDiff(schedule.payDates(schedule.numOfPeriod),valueDate);
                numMatTime = max(numMatTime, days);
                dfNumMat = model.DFRisky(numMatTime/365.0);
                
                numeraire.numMatTime = numMatTime;
                numeraire.dfNumMat = dfNumMat;
                
                % induct backward and calculate cashflow
                periodCount = schedule.numOfPeriod;
                
                % pricer single info initialize
                
                nonCall = PricerInfo(periodCount +1, pathSize);
                callable = PricerInfo(periodCount +1, pathSize);
                dfCol = PricerInfo(periodCount +1, pathSize);
                
                dummy1 = PricerInfo(periodCount +1, pathSize);
                dummy2 = PricerInfo(periodCount +1, pathSize);
                
                % additional floating leg info
                floatingLeg = PricerInfo(floatingSchedule.numOfPeriod, pathSize);
                % currentFloatingLegIdx for iteration in dealing with 
                % the floatingLeg cashflow
                currentFloatingLegIdx = floatingSchedule.numOfPeriod;
                
                % intermediate variables
                payoffStateA.cashflow = zeros(1, pathSize);
                payoffStateA.cashflow_npv = zeros(1, pathSize);
                
            
                % we calculate cashflow backward from maturityDate to valueDate
                % rangeAccrual coupons are added at the startDate of the Period
                % At maturity

                nominal = security.nominal;
                coupon = security.coupon;
                callYN = security.callYN;
                
                ralower = security.lower;
                raupper = security.upper;
                tenorLong = security.tenorL;
                tenorShort = security.tenorS;
                
                ralowerCD = security.lowerCD;
                raupperCD = security.upperCD;
                tenorCD = security.tenorCD;
                
                currentTime = numMethodInfo.totalTimeSteps(end);
                modelTimeStepsSize = numMethodInfo.modelTimeStepsSize;
                currentTimeIdx = numMethodInfo.totalTimeStepsSize;
%                 currentTimeIdx = numMethodInfo.totalTimeStepsSize - numMethodInfo.pastTimeStepsSize;
                
                currentNodeIdx = periodCount + 1;
                
                modelStateT = modelStates(numOfFactors*(max(currentTimeIdx-pastTimeStepsSize-1,0))+1:numOfFactors*(max(currentTimeIdx-pastTimeStepsSize-1,0))+2,:);
                
                %swap no nominal payoff
                payTime = DateDiff(schedule.payDates(schedule.numOfPeriod),valueDate);
                
                %even though there is no cashflow at the last structured
                %enddate, we add dummy cashflow in line with the note
                
                if payTime > 0
                    payoffStateA.cashflow = nominal*ones(1,pathSize)*0.0;

                    df_ep = model.discountFactorRisky(currentTime, payTime,numMatTime,modelStateT);
                    for k=1:pathSize
                        payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                    end

                    %process payoff at maturity
                    nonCall.ProcessAddPayoff(model,currentTime,numeraire, ...
                                         modelStateT,payoffStateA.cashflow,currentNodeIdx);

                    callable.ProcessAddPayoff(model,currentTime,numeraire, ...
                                         modelStateT,payoffStateA.cashflow,currentNodeIdx);

                    %df column
                    payoffStateA.cashflow = ones(1,pathSize);
                    for k=1:pathSize
                        payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                    end

                    dfCol.ProcessAddPayoff(model,currentTime,numeraire, ...
                                         modelStateT,payoffStateA.cashflow,currentNodeIdx);

                    %dummy column
                    payoffStateA.cashflow = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorLong,modelStateT);
                    dummy1.ProcessAddPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);

                    payoffStateA.cashflow = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                    dummy2.ProcessAddPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);

                  
                end
                
                lastTimeIdx = currentTimeIdx;
                totalTimeSteps = numMethodInfo.timeStepsInfo.totalTimeSteps;
                
                % flag for inclusive schedule
%                 simpleYN = false;
%                 simpleYN = true;
                if pricer.overHedgeYN == true 
                    simpleYN = 2;
                else
                    simpleYN = 1;
                end
                
                for i=periodCount:-1:1
                    % past schedule neglect
                    % we induct in the pastTimeSteps too
                    
%                     if DateDiff(valueDate,schedule.payDates(i)) >= 0, continue;end 
                    
                    startTimeIdx = numMethodInfo.timeStepsInfo.startTimeIdx(i);
                    endTimeIdx = numMethodInfo.timeStepsInfo.endTimeIdx(i);

                    inclusiveYN = numMethodInfo.timeStepsInfo.inclusiveYN(i);
                 %% FromToInduction start

                    %modelsStates induct backward(lastTimeIdx to endTimeIdx)
                    while endTimeIdx < lastTimeIdx
                        lastTimeIdx = lastTimeIdx -1;
                        if lastTimeIdx == endTimeIdx break;end
                    end

                    currentTimeIdx = lastTimeIdx;
                    currentTime = totalTimeSteps(currentTimeIdx);
                    % now we are at endTimeIdx
                    % variables for coupon valuation initialized
                    

                    cD = 0; % cumulated day count fraction
                    %cP shoul be declared as an array
%                     cP = 0;
                    cP = zeros(1,pathSize);
                    dI = 0; % dummy idx for loop
                    dcfC = 0;
                    dcfN = 0;
                    dD = 0;
                    idxCD = zeros(1,pathSize);
                    
                    idxL = zeros(1,pathSize);
                    idxS = zeros(1,pathSize);
                    
                    idxC = zeros(1,pathSize);
                    idxN = zeros(1,pathSize);
                    
                    spreadC = zeros(1,pathSize);
                    spreadN = zeros(1,pathSize);
                    probSpread = zeros(1,pathSize);
                    
                    numOfTimeSteps = numMethodInfo.timeStepsInfo.numOfTimeSteps(i);
                    
                    % if inclusive then we use simpleYN rangeProb
                    %if inclusive then we do not evaluate the last step but
                    % use fixing instead 
                    endTimeStep = 1;
                    % we let endTimeStep to be 1 and
                    % inside the for loop if currentTime ==0
                    % then we skip the range accrual check part
                    
                    if inclusiveYN
%                         simpleYN = true;
                        simpleYN = 1;
%                         endTimeStep = 2;
                        if pricer.overHedgeYN == true 
                            simpleYN = 2;
                        end
                    end
                    
                    for j=numOfTimeSteps:-1:endTimeStep
                        % we initialize payoffStateA as zeros to prevent
                        % reuse it;
                        payoffStateA.cashflow = zeros(1,pathSize);
                        % evaluate FromTo dealdescription
                        % last step is omitted
                        currentTime = totalTimeSteps(currentTimeIdx);
                        if currentTime > 0
                            dcfC = currentTime;

                            if j~= numOfTimeSteps
                                dD = (dcfN-dcfC);
                            end

                            modelStateT = modelStates(numOfFactors*(max(currentTimeIdx-pastTimeStepsSize-1,0))+1:numOfFactors*(max(currentTimeIdx-pastTimeStepsSize-1,0))+2,:);
                            
                            idxCD = model.Libor_SimpleDate(valueDate,currentTime,currentTime,numMatTime, tenorCD,modelStateT);
                            idxC = idxCD;
                        
                            idxL = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorLong,modelStateT);
                            idxS = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                            spreadC = idxL - idxS;

%                             probSpread = model.SRPr(spreadC,spreadN,ralower,raupper,simpleYN,pathSize)*dD;
                            probSpread = model.DRPr(idxC,idxN,ralowerCD,raupperCD,spreadC,spreadN,ralower,raupper,simpleYN,pathSize)*dD;
                            cD = cD + dD;
                            cP = cP + probSpread;

                            dcfN = dcfC;
                            
                            idxN = idxC;
                            spreadN = spreadC;
                        else
                            % we updated cD only since fixing is treated in
                            % rangedays
                            % fix it later !
                            
                            dcfC = currentTime;

                            if j~= numOfTimeSteps
                                dD = (dcfN-dcfC);
                            end
                            cD = cD + dD;
                            
                            dcfN = dcfC;
                            aaa =1.0;
                        end 
                        % evaluate secodary dealDescription(floating)
                        % floatingLeg
                        idxFloat = find(currentTime == outInfo.floatingEventTime);
                        
                        if currentTime == 5297
                            aaa =1;
                        end
                        
                        % we do not evaluate floating leg at the end of
                        % coupon Schedule for avoding double counting
                        if ~isempty(idxFloat) && (j~= numOfTimeSteps)
                            iterFloat = 1;
                            foundIterFloatYN= false;
                            for idx11 = length(floatingSchedule.startDates):-1:1
                                if outInfo.floatingEventTime(idxFloat) == DateDiff(floatingSchedule.startDates(idx11),valueDate)
                                    iterFloat = idx11;
                                    foundIterFloatYN = true;
                                    break;
                                end
                            end
                            
                            if ~foundIterFloatYN
                                error('floatingEventTime is not in the floatingSchedule startDates')
                            end
                            
                            % only those cashflow that paydate is after
                            % valueDate will be added
                            if DateDiff(floatingSchedule.payDates(idx11),valueDate) > 0
                                % floatingIndex is not fixed    
                                if outInfo.floatingEventTime(idxFloat) > 0
                                    dcfFloating = floatingSchedule.dayCountFraction(iterFloat);
                                    if strcmp(floatingIndex,'Libor3M')
                                        tenorFloatingCD = 0.25;
                                        idxFloatingCD = model.Libor_SimpleDate(valueDate,currentTime,currentTime,numMatTime, tenorFloatingCD,modelStateT);
                                        % -1 for floating pay 
                                        payoffStateA.cashflow = -1.0 * nominal * (idxFloatingCD + floatingSpread) * dcfFloating;
                                    end

                                else
                                    % floatingIndex is fixed
                                    sizeIdxFloat = length(idxFloat);
                                    if strcmp(floatingIndex,'Libor3M')
                                        tenorFloatingCD = 0.25;
                                        allFixings = security.allFixings;
                                        KRwLibor3MFixings = allFixings('KRWLibor3M');

                                        dcfFloating = floatingSchedule.dayCountFraction(iterFloat);
                                        idxFloatingCD = KRwLibor3MFixings(StrDate(floatingSchedule.resetDates(iterFloat),'str'));
        %                                 idxFloatingCD = model.Libor_SimpleDate(valueDate,currentTime,currentTime,numMatTime, tenorFloatingCD,modelStateT);
                                        % -1 for floating pay
                                        for k=1:pathSize
                                            payoffStateA.cashflow(k) = -1.0 * nominal * (idxFloatingCD + floatingSpread) * dcfFloating;
                                        end
                                    end
                                end
                                
                                payTime = DateDiff(floatingSchedule.payDates(iterFloat),valueDate);
                                %event is at startDate so we need to discount the
                                %cashflow first from paydate to startdate
                                df_ep = model.discountFactorRisky(currentTime, payTime,numMatTime,modelStateT);

                                for k=1:pathSize
                                    payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                                end
                                
                                % process floatingLeg payoff

                                currentNodeIdx = i;

                                nonCall.ProcessAddPayoff(model,currentTime,numeraire, ...
                                                 modelStateT,payoffStateA.cashflow,currentNodeIdx);

                                callable.ProcessAddPayoff(model,currentTime,numeraire, ...
                                                 modelStateT,payoffStateA.cashflow,currentNodeIdx);

                                % process additional payoff
                                floatingLeg.ProcessAddPayoff(model,currentTime,numeraire, ...
                                                 modelStateT,payoffStateA.cashflow,iterFloat);
                                
                            end
                        end
                            

                            
                            
                        
                        %induct backward to the next time step
                        % one time backward induction
                        lastTimeIdx = currentTimeIdx;

                        if startTimeIdx < lastTimeIdx
                            lastTimeIdx = lastTimeIdx -1;
                            currentTimeIdx = lastTimeIdx;
                        end

                    end
                    
                    %% FromToInduction end
        % if the current period is inclusive, then the start date is not evaluated in the above for loop.
		% instead the fixing will be treated separately below.
		% however current_time_idx is updated at the end of the for loop.
		% so we need to update model_states accordingly.
		% if model_states were updated previously then the following additionaly step doesn't affect the result anyway.
                    currentTime = totalTimeSteps(currentTimeIdx);
                    
                    % if 
                    modelStateT = modelStates(numOfFactors*(max(currentTimeIdx-pastTimeStepsSize-1,0))+1:numOfFactors*(max(currentTimeIdx-pastTimeStepsSize-1,0))+2,:);
                    
                    %% AtDate evaluation
                    % Adding a new cashflow to the payoff
                    % for structured coupon payDate should be after
                    % valueDate
                    
                    if DateDiff(schedule.payDates(i),valueDate) > 0
                        dcf = schedule.dayCountFraction(i);

                        if inclusiveYN
                            % we add valuation date
                            cD = schedule.dayCount(i);
                            rangeDays =  security.rangeDays(i);
                            payoffStateA.cashflow = nominal * coupon * dcf * (rangeDays + cP)/cD;
                        else
                            payoffStateA.cashflow = nominal * coupon * dcf * cP/cD;
                        end
                        % eventDate(i) = startDate(i) therefore there is no induction in modelStates
                        % i.e. currentTimeIdx has not changed
                        % no need to update modelStateT
                        % modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:end,:);
                        % since cashflow's are added at the startDate(i) we need to
                        % multiply cashflow with the df(startDate(i), payDate(i))
                        payTime = DateDiff(schedule.payDates(i),valueDate);

                        %event is at startDate so we need to discount the
                        %cashflow first from paydate to startdate
                        df_ep = model.discountFactorRisky(currentTime, payTime,numMatTime,modelStateT);

                        for k=1:pathSize
                            payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                        end

                        % process payoff

                        currentNodeIdx = i;

                        nonCall.ProcessAddPayoff(model,currentTime,numeraire, ...
                                         modelStateT,payoffStateA.cashflow,currentNodeIdx);

                        callable.ProcessAddPayoff(model,currentTime,numeraire, ...
                                         modelStateT,payoffStateA.cashflow,currentNodeIdx);

                        %df column
                        payoffStateA.cashflow = ones(1,pathSize);
                        for k=1:pathSize
                            payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                        end

                        dfCol.ProcessAddPayoff(model,currentTime,numeraire, ...
                                         modelStateT,payoffStateA.cashflow,currentNodeIdx);

                        %dummy column
                        payoffStateA.cashflow = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorLong,modelStateT);
                        dummy1.ProcessAddPayoff(model,currentTime,numeraire, ...
                                         modelStateT,payoffStateA.cashflow,currentNodeIdx);

                        payoffStateA.cashflow = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                        dummy2.ProcessAddPayoff(model,currentTime,numeraire, ...
                                         modelStateT,payoffStateA.cashflow,currentNodeIdx);

                    end
                    % if exercise is possible
                    if i == 1
                        dummaaa = 1;
                    end
                    
                    % call event happens  at the start of the period
                    % at call date, by default, call option is expired
                    
                    if callYN(i) && (DateDiff(valueDate,callSchedule.resetDates(i)) < 0)
                        % 1.undiscount callable.payoff
                        % 2.build the state variables matrix
                        % 3.regress 1's with the given state variables
                        % (model states 1,2)
                        % 4. exercise per each path in
                        % discount the new payoff and assign into callable
                        % column
                        
                        %1 undiscount callable.payoff and get the
                        %contiValue
                       
                        %exercise value swap
                        exerciseValue = security.nominal*ones(1,pathSize) * 0.0;
                        payTime = DateDiff(callSchedule.payDates(i),valueDate);
                        %event is at startDate so we need to discount the
                        %cashflow first from paydate to startdate
                        df_ep = model.discountFactorRisky(currentTime, payTime,numMatTime,modelStateT);
                        for k=1:pathSize
                            exerciseValue(k) = exerciseValue(k) * df_ep(k);
                        end
                        
                        contiValue = zeros(1,pathSize);
                        onePayoff = ones(1,pathSize);
                        onePayoffD = model.discountPayoffRisky(currentTime,numMatTime ...
                        ,modelStateT,onePayoff);
                        contiValue = callable.payoff./onePayoffD;
                        
                        %Nominal Regression test
                        contiValue = contiValue/10000;
                        
                        % build the state variabls matrix
                        decisionVar1 = modelStateT(1,:);
                        decisionVar2 = modelStateT(2,:);
                        % we use upto 2nd order polynomial as regression
                        % equatioin
                        dimensionLS = 6;
                        regMatrix = ones(pathSize,dimensionLS);
                        
                        for idx_r=1:pathSize
                            regMatrix(idx_r,2) = decisionVar1(idx_r);
                            regMatrix(idx_r,3) = decisionVar2(idx_r);
                            regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r);
                            regMatrix(idx_r,5) = decisionVar1(idx_r)*decisionVar2(idx_r);
                            regMatrix(idx_r,6) = decisionVar2(idx_r)*decisionVar2(idx_r);
                        end
                        
                        % simple regression <- for path dependent option
                        % we should change it
                        % beta  : regression coefficients
                        beta =zeros(dimensionLS,1);
                        beta = regress(contiValue',regMatrix);
                        regContiValue = regMatrix*beta;
                       
                         %Nominal Regression test
                        regContiValue = regContiValue * 10000;
                        
%                         exercisedPayoff = regContiValue;
                        exercisedPayoff = contiValue * 10000;
                        
                        % if regContiValue is bigger than exerciseValue
                        % then issue exercise call option
                        for idx_r=1:pathSize
                            if regContiValue(idx_r) >= exerciseValue(idx_r) 
                                exercisedPayoff(idx_r) = exerciseValue(idx_r);
                            end
                        end
                        
                        % we discount the new payoff and 
                        % assign it into the existing callable payoff
                        callable.payoff = model.discountPayoffRisky(currentTime,numMatTime ...
                        ,modelStateT,exercisedPayoff);
                        
                    end
                end

                
                nonCall.npv = mean(nonCall.payoff);
                out.nonCall = nonCall;
                callable.npv = mean(callable.payoff);
                out.callable = callable;
                
                floatingLeg.npv = mean(floatingLeg.payoff);
                out.floatingLeg = floatingLeg;
                
                out.dfCol = dfCol;
                out.dummy1 = dummy1;
                out.dummy2 = dummy2;
                out.daycountFraction = schedule.dayCountFraction;
                
        end
        
        function out = computePriceClosedSimple(pricer,valueDate)
                security = pricer.m_security;
                numMethod = pricer.m_numMethod;
                model = pricer.m_model;
                schedule = security.couponSchedule;
                
                 %floating
                floatingSchedule = security.floatingSchedule;
                floatingSpread = security.floatingSpread;
                floatingIndex = security.floatingIndex;
                
                % numMethodTimeSteps Generate
                mcOneTimeStep = numMethod.mcOneTimeStep;
%                 outInfo = numMethod.numMethodTimeStepsGenerateDW(valueDate,schedule,mcOneTimeStep);
                
                outInfo = numMethod.numMethodTimeStepsGenerateDWSwapSimple(valueDate,...
                                    schedule,floatingSchedule,mcOneTimeStep);
                                
                numMethodInfo.timeStepsInfo = outInfo;

                % mc random number init
                %pathSize = 2^16;
                pathSize = numMethod.pathSize;
                
                pathSize = 1;
                
%                 % generate random numbers and modelstates forward
%                 rng('default');
%                 rng(0);
% 
                numOfFactors = 2;
% 
%                 %Z= randn(numOfFactors*outInfo.totalTimeStepsSize,pathSize/2);
%                 Z= randn(pathSize/2,numOfFactors*outInfo.totalTimeStepsSize);
%                 Z=[Z;-Z];
%                 Z=Z';
%                 
%                 
%                 %TimeInversion
%                 sizeZ = size(Z,1);
%                 A = zeros(sizeZ,sizeZ);
%                 for i=1:sizeZ
%                     A(i,sizeZ + 1 -i) = 1;
%                 end
%                 Z = A*Z;
% 
%                 % we check with the c, so we use generated random numbers
% %                 mc_random = importdata('mc_random.xlsx');
% %                 mc_random = importdata('D:\mc_random_number_new_report.txt');
% %                 Z= mc_random;
%                 
                numMethodInfo.pathSize = pathSize;
                numMethodInfo.numOfFactors = numOfFactors;
                
                numMethodInfo.totalTimeSteps = outInfo.totalTimeSteps;
                numMethodInfo.totalTimeStepsSize = outInfo.totalTimeStepsSize;
                
                totalTimeSteps = numMethodInfo.totalTimeSteps;
                totalTimeStepsSize = numMethodInfo.totalTimeStepsSize;
                
                numMethodInfo.modelTimeSteps = outInfo.modelTimeSteps;
                numMethodInfo.modelTimeStepsSize = outInfo.modelTimeStepsSize;
                
                numMethodInfo.pastTimeStepsSize = outInfo.pastTimeStepsSize;
                
                pastTimeStepsSize = numMethodInfo.pastTimeStepsSize;
%                 numMethodInfo.Z = Z;
% 
% 
% 
%                 %generate model states for the given timeSteps
%                 modelStates = numMethod.generateModelStates(valueDate,schedule,numMethodInfo,model);

                % numeraire info
                numMatTime = numMethodInfo.totalTimeSteps(end);
                dfNumMat = model.DF(numMatTime/365.0);
                
                % induct backward and calculate cashflow
            
                periodCount = schedule.numOfPeriod;
                fltPeriodCount = floatingSchedule.numOfPeriod;
                % cashflow columns initialize
                % 1st nonCall
                nonCall.npv = 0;
                nonCall.cashflow_e = zeros(periodCount + 1,1);
                nonCall.cashflow_npv_e = zeros(periodCount + 1,1);
                nonCall.rangeProb_e = zeros(periodCount + 1,1);
                nonCall.df_e = zeros(periodCount + 1,1);
                nonCall.df_c = zeros(periodCount + 1,1);

                % debug information, cashflow & npv per path
                nonCall.payoff = zeros(1,pathSize);
                nonCall.payoffStates.cashflow = zeros(periodCount+1,pathSize);
                nonCall.payoffStates.cashflow_npv = zeros(periodCount+1,pathSize);
                nonCall.payoffStates.df = zeros(periodCount + 1,pathSize);
                
                %floatingLeg
                floatingLeg.npv = 0;
                floatingLeg.cashflow_e = zeros(fltPeriodCount,1);
                floatingLeg.cashflow_npv_e = zeros(fltPeriodCount,1);
                floatingLeg.rangeProb_e = zeros(fltPeriodCount,1);
                floatingLeg.df_e = zeros(fltPeriodCount,1);
                floatingLeg.df_c = zeros(fltPeriodCount,1);

                floatingLeg.payoff = zeros(1,pathSize);
                floatingLeg.payoffStates.cashflow = zeros(fltPeriodCount,pathSize);
                floatingLeg.payoffStates.cashflow_npv = zeros(fltPeriodCount,pathSize);
                floatingLeg.payoffStates.df = zeros(fltPeriodCount,pathSize);
                
                % intermediate variables
                payoffStateA.cashflow = zeros(1, pathSize);
                payoffStateA.cashflow_npv = zeros(1, pathSize);
                payoffStateA.df = zeros(1,pathSize);
                
                % we calculate cashflow backward from maturityDate to valueDate
                % rangeAccrual coupons are added at the startDate of the Period
                % At maturity

                nominal = security.nominal;
                coupon = security.coupon;
                callYN = security.callYN;
                
                currentTime = numMethodInfo.totalTimeSteps(end);
                totalTimeSteps = numMethodInfo.timeStepsInfo.totalTimeSteps;
                totalTimeStepsSize = numMethodInfo.timeStepsInfo.totalTimeStepsSize;
                if totalTimeStepsSize == 0 || (totalTimeStepsSize == 1 && totalTimeSteps(1) == 0) 
                    out.nonCall = nonCall;
                    return;
                end
                
                currentTimeIdx = totalTimeStepsSize;
                
                currentNodeIdx = periodCount + 1;
                
%                 modelStateT = modelStates(numOfFactors*(totalTimeStepsSize-1)+1:numOfFactors*(totalTimeStepsSize-1)+2,:);
                payoffStateA.cashflow = nominal*ones(1,pathSize)*0.0;
                %process payoff at maturity
                nonCall.payoffStates.cashflow(currentNodeIdx,:) = payoffStateA.cashflow;
                
%                 payoffStateA.cashflow_npv = model.discountPayoff(currentTime,numMatTime ...
%                     ,modelStateT,payoffStateA.cashflow);
%                 
                payoffStateA.cashflow_npv = model.DFRisky(currentTime/365.0)*payoffStateA.cashflow;
                
                nonCall.payoffStates.cashflow_npv(currentNodeIdx,:) = payoffStateA.cashflow_npv ;
                
%                 payoffStateA.df = model.discountPayoff(currentTime,numMatTime ...
%                     ,modelStateT,1.0*ones(1,pathSize));
                payoffStateA.df = model.DFRisky(currentTime/365.0) *ones(1,pathSize);
                    

                nonCall.payoffStates.df(currentNodeIdx,:) = payoffStateA.df;
                nonCall.payoff = nonCall.payoff + payoffStateA.cashflow_npv;
                
                nonCall.cashflow_e(currentNodeIdx) = mean(payoffStateA.cashflow);
                nonCall.cashflow_npv_e(currentNodeIdx) = mean(payoffStateA.cashflow_npv);
                nonCall.df_e(currentNodeIdx) = mean(payoffStateA.df);
                nonCall.df_c(currentNodeIdx) = model.DFRisky(currentTime/365.0);
                
%                 % callable at maturity is the same as noncallable
%                 callable.payoffStates.cashflow(currentNodeIdx,:) = nonCall.payoffStates.cashflow(currentNodeIdx,:);
%                 callable.payoffStates.cashflow_npv(currentNodeIdx,:) = nonCall.payoffStates.cashflow_npv(currentNodeIdx,:);
%                 callable.payoff = callable.payoff + callable.payoffStates.cashflow_npv(currentNodeIdx,:);
%                 
%                 callable.cashflow_e(currentNodeIdx) = nonCall.cashflow_e(currentNodeIdx);
%                 callable.cashflow_npv_e(currentNodeIdx) = nonCall.cashflow_npv_e(currentNodeIdx);
                  
                lastTimeIdx = currentTimeIdx;
                
                
                % flag for inclusive schedule
%                 simpleYN = false;
                simpleYN = 1;
                
                for i=periodCount:-1:1
                    % past schedule neglect
%                     if DateDiff(valueDate,schedule.payDates(i)) >= 0, continue;end 
                    startTimeIdx = numMethodInfo.timeStepsInfo.startTimeIdx(i);
                    endTimeIdx = numMethodInfo.timeStepsInfo.endTimeIdx(i);

                    inclusiveYN = numMethodInfo.timeStepsInfo.inclusiveYN(i);
                     %% FromToInduction start

                    %modelsStates induct backward(lastTimeIdx to endTimeIdx)
                    while endTimeIdx < lastTimeIdx
                        lastTimeIdx = lastTimeIdx -1;
                        if lastTimeIdx == endTimeIdx break;end
                    end

                    currentTimeIdx = lastTimeIdx;
                    currentTime = totalTimeSteps(currentTimeIdx);
                    % now we are at endTimeIdx
                    % variables for coupon valuation initialized
                    ralower = security.lower;
                    raupper = security.upper;
                    tenorLong = security.tenorL;
                    tenorShort = security.tenorS;
                    
                    ralowerCD = security.lowerCD;
                    raupperCD = security.upperCD;
                    tenorCD = security.tenorCD;
                
                    cD = 0; % cumulated day count fraction
                    cP = zeros(1,pathSize);
                    dI = 0; % dummy idx for loop
                    dcfC = 0;
                    dcfN = 0;
                    dD = 0;
                    idxL = zeros(1,pathSize);
                    idxS = zeros(1,pathSize);
                    idxCD = zeros(1,pathSize);
                    
                    spreadC = zeros(1,pathSize);
                    spreadN = zeros(1,pathSize);
                    
                    idxC = zeros(1,pathSize);
                    idxN = zeros(1,pathSize);
                    
                    probSpread = zeros(1,pathSize);
                    
                    numOfTimeSteps = numMethodInfo.timeStepsInfo.numOfTimeSteps(i);
                    
                    % if inclusive then we use simpleYN rangeProb
                    %if inclusive then we do not evaluate the last step but
                    % use fixing instead 
                    endTimeStep = 1;
                    if inclusiveYN
%                         simpleYN = true;
                        simpleYN = 1;
%                         endTimeStep = 2;
                    end
                    
                    for j=numOfTimeSteps:-1:endTimeStep
                        payoffStateA.cashflow = zeros(1,pathSize);
                        % evaluate dealdescription
                        % last step is omitted
                        currentTime = totalTimeSteps(currentTimeIdx);
                        if currentTime > 0
                            %dcfC = currentTime/365.0;
                            dcfC = currentTime;

                            if j~= numOfTimeSteps
                                dD = (dcfN-dcfC);
                            end

                            toDate = AddDate(valueDate,currentTime,'day');
                            tpDate = schedule.payDates(i);
%                             spreadC = model.CMSSpreadDigitalRangeDate(valueDate,toDate,tpDate,tenorLong,tenorShort,0.25,ralower,raupper);
%                             spreadC = model.CMSSpreadDigitalRangeDate(valueDate,toDate,tpDate,tenorLong,tenorShort,0.25,ralower,raupper);
                            spreadC = model.CDCMSSpreadDigitalRangeDate(valueDate,toDate,tpDate,tenorCD,tenorLong,tenorShort,0.25,ralowerCD,raupperCD,ralower,raupper);
    
                            if simpleYN
                                probSpread = spreadC*dD;
                            else
                                probSpread = spreadC*dD;
    %                             probSpread = 0.5*(spreadC + spreadN)*dD;

                            end


                            cD = cD + dD;
                            cP = cP + probSpread;

                            dcfN = dcfC;
                            spreadN = spreadC;
                        else
                            % we updated cD only since fixing is treated in
                            % rangedays
                            % fix it later !
                            
                            dcfC = currentTime;

                            if j~= numOfTimeSteps
                                dD = (dcfN-dcfC);
                            end
                            cD = cD + dD;
                            
                            dcfN = dcfC;
                            aaa =1.0;
                        end
                        % evaluate secodary dealDescription(floating)
                        % floatingLeg
                        idxFloat = find(currentTime == outInfo.floatingEventTime);
                        
                        % we do not evaluate floating leg at the endof
                        % coupon Schedule for avoding double counting
                        if ~isempty(idxFloat) && (j~= numOfTimeSteps)
                            iterFloat = 1;
                            foundIterFloatYN= false;
                            for idx11 = length(floatingSchedule.startDates):-1:1
                                if outInfo.floatingEventTime(idxFloat) == DateDiff(floatingSchedule.startDates(idx11),valueDate)
                                    iterFloat = idx11;
                                    foundIterFloatYN = true;
                                    break;
                                end
                            end

                            if ~foundIterFloatYN
                                error('floatingEventTime is not in the floatingSchedule startDates')
                            end
                            
                            if DateDiff(floatingSchedule.payDates(idx11),valueDate) > 0
                                if outInfo.floatingEventTime(idxFloat) > 0

                                    dcfFloating = floatingSchedule.dayCountFraction(iterFloat);
                                    if strcmp(floatingIndex,'Libor3M')
                                        tenorFloatingCD = 0.25;
    %                                         idxFloatingCD = model.Libor_SimpleDate(valueDate,currentTime,currentTime,numMatTime, tenorFloatingCD,modelStateT);
                                        idxFloatingCD = model.LiborDate(valueDate,currentTime,tenorFloatingCD);
                                        % -1 for floating pay 
                                        payoffStateA.cashflow = -1.0 * nominal * (idxFloatingCD + floatingSpread) * dcfFloating;
                                    end

                                else
                                    % floatingIndex is fixed
                                    %if floatingCashflow 1 is fixed not paid
                                    %and also next floatingCashflow2 is fixed
                                    %then
                                    %idxFloat = find(currentTime == outInfo.floatingEventTime);
                                    % will give two answers
                                    % then we use sizeIdxFloat, length(idxfloat) to 
                                    % access the right floating Cashflow
                                    sizeIdxFloat = length(idxFloat);
                                    %
                                    if strcmp(floatingIndex,'Libor3M')
                                        tenorFloatingCD = 0.25;
                                        allFixings = security.allFixings;
                                        KRwLibor3MFixings = allFixings('KRWLibor3M');
        
                                        dcfFloating = floatingSchedule.dayCountFraction(iterFloat);
                                        idxFloatingCD = KRwLibor3MFixings(StrDate(floatingSchedule.resetDates(iterFloat),'str'));
        %                                 idxFloatingCD = model.Libor_SimpleDate(valueDate,currentTime,currentTime,numMatTime, tenorFloatingCD,modelStateT);
                                        % -1 for floating pay
                                        for k=1:pathSize
                                            payoffStateA.cashflow(k) = -1.0 * nominal * (idxFloatingCD + floatingSpread) * dcfFloating;
                                        end
                                    end
                                end
                                
                                payTime = DateDiff(floatingSchedule.payDates(iterFloat),valueDate);
                                %event is at startDate so we need to discount the
                                %cashflow first from paydate to startdate
                                %cashflow first from paydate to startdate
                                currentNodeIdx = i;
                                nonCall.payoffStates.cashflow(currentNodeIdx,:) = nonCall.payoffStates.cashflow(currentNodeIdx,:) + payoffStateA.cashflow;
                                payoffStateA.cashflow_npv = payoffStateA.cashflow * model.DFRisky(payTime/365.0);

                                nonCall.payoffStates.cashflow_npv(currentNodeIdx,:) = nonCall.payoffStates.cashflow_npv(currentNodeIdx,:) + payoffStateA.cashflow_npv;

                                nonCall.payoff = nonCall.payoff + payoffStateA.cashflow_npv;

                                nonCall.cashflow_e(currentNodeIdx) = nonCall.cashflow_e(currentNodeIdx) + mean(payoffStateA.cashflow);
                                nonCall.cashflow_npv_e(currentNodeIdx) = nonCall.cashflow_npv_e(currentNodeIdx) + mean(payoffStateA.cashflow_npv);

                                floatingLeg.payoffStates.cashflow(iterFloat,:) = floatingLeg.payoffStates.cashflow(iterFloat,:) + payoffStateA.cashflow;
                                floatingLeg.payoffStates.cashflow_npv(iterFloat,:) = floatingLeg.payoffStates.cashflow_npv(iterFloat,:) + payoffStateA.cashflow_npv;
                                floatingLeg.payoff = floatingLeg.payoff + payoffStateA.cashflow_npv;

                                floatingLeg.cashflow_e(iterFloat) = floatingLeg.cashflow_e(iterFloat) + mean(payoffStateA.cashflow);
                                floatingLeg.cashflow_npv_e(iterFloat) = floatingLeg.cashflow_npv_e(iterFloat) + mean(payoffStateA.cashflow_npv);

                            end
                        end
                               
                            
                        
                        %induct backward to the next time step
                        % one time backward induction
                        lastTimeIdx = currentTimeIdx;

                        if startTimeIdx < lastTimeIdx
                            lastTimeIdx = lastTimeIdx -1;
                            currentTimeIdx = lastTimeIdx;
                        end
                    end

                    
                    %% FromToInduction end
                    %% AtDate evaluation
                    % Adding a new cashflow to the payoff

                    dcf = schedule.dayCountFraction(i);
                    if DateDiff(schedule.payDates(i),valueDate) > 0
                        if inclusiveYN
                            % we add valuation date
                            cD = schedule.dayCount(i);
                            rangeDays =  security.rangeDays(i);
                            payoffStateA.cashflow = nominal * coupon * dcf * (rangeDays + cP)/cD;
                        else
                            payoffStateA.cashflow = nominal * coupon * dcf * cP/cD;
                        end
           
                        payTime = DateDiff(schedule.payDates(i),valueDate);

                        currentNodeIdx = i;

                        % 1st nonCall column
                        nonCall.payoffStates.cashflow(currentNodeIdx,:) = nonCall.payoffStates.cashflow(currentNodeIdx,:) + payoffStateA.cashflow;
                        payoffStateA.cashflow_npv = payoffStateA.cashflow * model.DFRisky(payTime/365.0);

    %                     payoffStateA.cashflow_npv = model.discountPayoff(currentTime,numMatTime ...
    %                     ,modelStateT,payoffStateA.cashflow);
                        nonCall.payoffStates.cashflow_npv(currentNodeIdx,:) = nonCall.payoffStates.cashflow_npv(currentNodeIdx,:) + payoffStateA.cashflow_npv;
                        payoffStateA.df = model.DFRisky(payTime/365.0);
    %                     payoffStateA.df = model.discountPayoff(currentTime,numMatTime ...
    %                     ,modelStateT,1.0*ones(1,pathSize));
                        nonCall.payoffStates.df(currentNodeIdx,:) = nonCall.payoffStates.df(currentNodeIdx,:) + payoffStateA.df;

                        nonCall.payoff = nonCall.payoff + payoffStateA.cashflow_npv;

                        nonCall.cashflow_e(currentNodeIdx) = nonCall.cashflow_e(currentNodeIdx) + mean(payoffStateA.cashflow);
                        nonCall.cashflow_npv_e(currentNodeIdx) = nonCall.cashflow_npv_e(currentNodeIdx) + mean(payoffStateA.cashflow_npv);

                        nonCall.df_e(currentNodeIdx) = nonCall.df_e(currentNodeIdx) + mean(payoffStateA.df);
                        nonCall.df_c(currentNodeIdx) = nonCall.df_c(currentNodeIdx) + model.DFRisky(currentTime/365.0);
 
                    end
                    % if exercise is possible
                    if i == 1
                        dummaaa = 1;
                    end
       
                end

                nonCall.npv = mean(nonCall.payoff);
                out.nonCall = nonCall;
                floatingLeg.npv = mean(floatingLeg.payoff);
                out.floatingLeg = floatingLeg;
%                 callable.npv = mean(callable.payoff);
%                 out.pricerInfo.callable = callable;
                out.daycountFraction = schedule.dayCountFraction;
                
        end
        
        
        
        
    end
    
end



In [ ]:
classdef DailyWeeklyRiskyPricer_CDCMSSpreadDualRA < H_RootObject
    %Pricer(Security,Model,NumMethod)
    %Prcer is constructed from Security, Model, NumMethod 
    
    properties
        m_security
        m_model
        m_numMethod
        overHedgeYN
    end
    
    methods
        function pricer = DailyWeeklyRiskyPricer_CDCMSSpreadDualRA(security,model,numMethod,overHedgeYN)
                pricer.m_security = security;
                pricer.m_model = model;
                pricer.m_numMethod = numMethod;
                pricer.overHedgeYN = overHedgeYN;
        end
        
        function out = computePriceMCCallable(pricer,valueDate)
                security = pricer.m_security;
                numMethod = pricer.m_numMethod;
                model = pricer.m_model;
                schedule = security.couponSchedule;
                callSchedule = security.callSchedule;
                
                % numMethodTimeSteps Generate
                mcOneTimeStep = numMethod.mcOneTimeStep;
                outInfo = numMethod.numMethodTimeStepsGenerateDW(valueDate,schedule,mcOneTimeStep);
            
                numMethodInfo.timeStepsInfo = outInfo;

                % mc random number init
                %pathSize = 2^16;
                pathSize = numMethod.pathSize;

                % generate random numbers and modelstates forward
                rng('default');
                rng(0);

                numOfFactors = 2;

                %Z= randn(numOfFactors*outInfo.totalTimeStepsSize,pathSize/2);
                Z= randn(pathSize/2,numOfFactors*outInfo.totalTimeStepsSize);
                Z=[Z;-Z];
                Z=Z';
                
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;

                % we check with the c code, so we use generated random numbers
%                 mc_random = importdata('mc_random.xlsx');
%                 mc_random = importdata('D:\mc_random_number_new_report.txt');
%                 Z= mc_random;
                
                numMethodInfo.pathSize = pathSize;
                numMethodInfo.numOfFactors = numOfFactors;
                numMethodInfo.totalTimeSteps = outInfo.totalTimeSteps;
                numMethodInfo.totalTimeStepsSize = outInfo.totalTimeStepsSize;
                numMethodInfo.Z = Z;



                %generate model states for the given timeSteps
                modelStates = numMethod.generateModelStates(valueDate,schedule,numMethodInfo,model);

                % numeraire info
                numMatTime = numMethodInfo.totalTimeSteps(end);
                days = DateDiff(schedule.payDates(schedule.numOfPeriod),valueDate);
                numMatTime = max(numMatTime, days);
                dfNumMat = model.DFRisky(numMatTime/365.0);
                
                numeraire.numMatTime = numMatTime;
                numeraire.dfNumMat = dfNumMat;
                
                % induct backward and calculate cashflow
            
                periodCount = schedule.numOfPeriod;
                
                % pricer single info initialize
                
                nonCall = PricerInfo(periodCount +1, pathSize);
                callable = PricerInfo(periodCount +1, pathSize);
                dfCol = PricerInfo(periodCount +1, pathSize);
                
                dummy1 = PricerInfo(periodCount +1, pathSize);
                dummy2 = PricerInfo(periodCount +1, pathSize);
                
                
                % intermediate variables
                payoffStateA.cashflow = zeros(1, pathSize);
                payoffStateA.cashflow_npv = zeros(1, pathSize);
                
            
                % we calculate cashflow backward from maturityDate to valueDate
                % rangeAccrual coupons are added at the startDate of the Period
                % At maturity

                nominal = security.nominal;
                coupon = security.coupon;
                callYN = security.callYN;
                
                ralower = security.lower;
                raupper = security.upper;
                
                ralowerS = security.lowerS;
                raupperS = security.upperS;
                
                tenorCD = security.tenor;
                
                tenorLong = security.tenorL;
                tenorShort = security.tenorS;
                
                currentTime = numMethodInfo.totalTimeSteps(end);
                totalTimeStepsSize = numMethodInfo.totalTimeStepsSize;
                currentTimeIdx = totalTimeStepsSize;
                
                currentNodeIdx = periodCount + 1;
                
                modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:numOfFactors*(currentTimeIdx-1)+2,:);
                payoffStateA.cashflow = nominal*ones(1,pathSize);
                
                payTime = DateDiff(schedule.payDates(schedule.numOfPeriod),valueDate);
                df_ep = model.discountFactorRisky(currentTime, payTime,numMatTime,modelStateT);
                for k=1:pathSize
                    payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                end
                
                %process payoff at maturity
                nonCall.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                
                callable.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                
                %df column
                payoffStateA.cashflow = ones(1,pathSize);
                for k=1:pathSize
                    payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                end
                
                dfCol.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                                 
                %dummy column
                payoffStateA.cashflow = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorLong,modelStateT);
                dummy1.ProcessPayoff(model,currentTime,numeraire, ...
                                 modelStateT,payoffStateA.cashflow,currentNodeIdx);

                payoffStateA.cashflow = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                dummy2.ProcessPayoff(model,currentTime,numeraire, ...
                                 modelStateT,payoffStateA.cashflow,currentNodeIdx);
                                 
                  

                lastTimeIdx = currentTimeIdx;
                totalTimeSteps = numMethodInfo.timeStepsInfo.totalTimeSteps;
                
                % flag for inclusive schedule
%                 simpleYN = false;
%                 simpleYN = true;
                if pricer.overHedgeYN == true 
                    simpleYN = 2;
                else
                    simpleYN = 1;
                end
                
                for i=periodCount:-1:1
                    % past schedule neglect
                    if DateDiff(valueDate,schedule.payDates(i)) >= 0, continue;end 
                    startTimeIdx = numMethodInfo.timeStepsInfo.startTimeIdx(i);
                    endTimeIdx = numMethodInfo.timeStepsInfo.endTimeIdx(i);

                    inclusiveYN = numMethodInfo.timeStepsInfo.inclusiveYN(i);
                 %% FromToInduction start

                    %modelsStates induct backward(lastTimeIdx to endTimeIdx)
                    while endTimeIdx < lastTimeIdx
                        lastTimeIdx = lastTimeIdx -1;
                        if lastTimeIdx == endTimeIdx break;end
                    end

                    currentTimeIdx = lastTimeIdx;
                    currentTime = totalTimeSteps(currentTimeIdx);
                    % now we are at endTimeIdx
                    % variables for coupon valuation initialized
                    

                    cD = 0; % cumulated day count fraction
                    cP = 0; 
                    dI = 0; % dummy idx for loop
                    dcfC = 0;
                    dcfN = 0;
                    dD = 0;
                    idxCD = zeros(1,pathSize);
                    
                    idxL = zeros(1,pathSize);
                    idxS = zeros(1,pathSize);
                    
                    idxC = zeros(1,pathSize);
                    idxN = zeros(1,pathSize);
                
                    spreadC = zeros(1,pathSize);
                    spreadN = zeros(1,pathSize);
                    probSpread = zeros(1,pathSize);
                    
                    numOfTimeSteps = numMethodInfo.timeStepsInfo.numOfTimeSteps(i);
                    
                    % if inclusive then we use simpleYN rangeProb
                    %if inclusive then we do not evaluate the last step but
                    % use fixing instead 
                    endTimeStep = 1;
                    if inclusiveYN
%                         simpleYN = true;
                        simpleYN = 1;
                        endTimeStep = 2;
                        if pricer.overHedgeYN == true 
                            simpleYN = 2;
                        end
                    end
                    
                    for j=numOfTimeSteps:-1:endTimeStep
                        % evaluate dealdescription
                        % last step is omitted
                        currentTime = totalTimeSteps(currentTimeIdx);
                        %dcfC = currentTime/365.0;
                        dcfC = currentTime;
                        
                        if j~= numOfTimeSteps
                            dD = (dcfN-dcfC);
                        end

                        modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:numOfFactors*(currentTimeIdx-1)+2,:);
                        
                        idxCD = model.Libor_SimpleDate(valueDate,currentTime,currentTime,numMatTime, tenorCD,modelStateT);
                        idxC = idxCD;
                    
                        idxL = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorLong,modelStateT);
                        idxS = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                        spreadC = idxL - idxS;

%                         probSpread = model.SRPr(spreadC,spreadN,ralower,raupper,simpleYN,pathSize)*dD;
                        
                        probSpread = model.DRPr(idxC,idxN,ralower,raupper,spreadC,spreadN,ralowerS,raupperS,simpleYN,pathSize)*dD;
                    
                        cD = cD + dD;
                        cP = cP + probSpread;

                        dcfN = dcfC;
                        
                        idxN = idxC;
                        spreadN = spreadC;

                        %induct backward to the next time step
                        % one time backward induction
                        lastTimeIdx = currentTimeIdx;

                        if startTimeIdx < lastTimeIdx
                            lastTimeIdx = lastTimeIdx -1;
                            currentTimeIdx = lastTimeIdx;
                        end


                    end

                    %% FromToInduction end
        % if the current period is inclusive, then the start date is not evaluated in the above for loop.
		% instead the fixing will be treated separately below.
		% however current_time_idx is updated at the end of the for loop.
		% so we need to update model_states accordingly.
		% if model_states were updated previously then the following additionaly step doesn't affect the result anyway.
                    currentTime = totalTimeSteps(currentTimeIdx);
                    modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:numOfFactors*(currentTimeIdx-1)+2,:);
                    
                    %% AtDate evaluation
                    % Adding a new cashflow to the payoff

                    dcf = schedule.dayCountFraction(i);
                    
                    if inclusiveYN
                        % we add valuation date
                        cD = schedule.dayCount(i);
                        rangeDays =  security.rangeDays(i);
                        payoffStateA.cashflow = nominal * coupon * dcf * (rangeDays + cP)/cD;
                    else
                        payoffStateA.cashflow = nominal * coupon * dcf * cP/cD;
                    end
                    % eventDate(i) = startDate(i) therefore there is no induction in modelStates
                    % i.e. currentTimeIdx has not changed
                    % no need to update modelStateT
                    % modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:end,:);
                    % since cashflow's are added at the startDate(i) we need to
                    % multiply cashflow with the df(startDate(i), payDate(i))
                    payTime = DateDiff(schedule.payDates(i),valueDate);
                    
                    %event is at startDate so we need to discount the
                    %cashflow first from paydate to startdate
                    df_ep = model.discountFactorRisky(currentTime, payTime,numMatTime,modelStateT);

                    for k=1:pathSize
                        payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                    end
                    
                    % process payoff
                    
                    currentNodeIdx = i;
                    
                    nonCall.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                
                    callable.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                    
                    %df column
                    payoffStateA.cashflow = ones(1,pathSize);
                    for k=1:pathSize
                        payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                    end
                    
                    dfCol.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                                 
                    %dummy column
                    payoffStateA.cashflow = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorLong,modelStateT);
                    dummy1.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                    
                    payoffStateA.cashflow = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                    dummy2.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                    
                    
                    % if exercise is possible
                    if i == 1
                        dummaaa = 1;
                    end
                    
                    % call event happens  at the start of the period
                    % at call date, by default, call option is expired
                    
                    if callYN(i) && (DateDiff(valueDate,callSchedule.resetDates(i)) < 0)
                        % 1.undiscount callable.payoff
                        % 2.build the state variables matrix
                        % 3.regress 1's with the given state variables
                        % (model states 1,2)
                        % 4. exercise per each path in
                        % discount the new payoff and assign into callable
                        % column
                        
                        %1 undiscount callable.payoff and get the
                        %contiValue
                       
                        %exercise value
                        exerciseValue = security.nominal*ones(1,pathSize);
                        payTime = DateDiff(callSchedule.payDates(i),valueDate);
                        %event is at startDate so we need to discount the
                        %cashflow first from paydate to startdate
                        df_ep = model.discountFactorRisky(currentTime, payTime,numMatTime,modelStateT);
                        for k=1:pathSize
                            exerciseValue(k) = exerciseValue(k) * df_ep(k);
                        end
                        
                        contiValue = zeros(1,pathSize);
                        onePayoff = ones(1,pathSize);
                        onePayoffD = model.discountPayoffRisky(currentTime,numMatTime ...
                        ,modelStateT,onePayoff);
                        contiValue = callable.payoff./onePayoffD;
                        
                        %Nominal Regression test
                        contiValue = contiValue/10000;
                        
                        % build the state variabls matrix
                        decisionVar1 = modelStateT(1,:);
                        decisionVar2 = modelStateT(2,:);
                        % we use upto 2nd order polynomial as regression
                        % equatioin
                        dimensionLS = 6;
                        regMatrix = ones(pathSize,dimensionLS);
                        
                        for idx_r=1:pathSize
                            regMatrix(idx_r,2) = decisionVar1(idx_r);
                            regMatrix(idx_r,3) = decisionVar2(idx_r);
                            regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r);
                            regMatrix(idx_r,5) = decisionVar1(idx_r)*decisionVar2(idx_r);
                            regMatrix(idx_r,6) = decisionVar2(idx_r)*decisionVar2(idx_r);
                        end
                        
                        % simple regression <- for path dependent option
                        % we should change it
                        % beta  : regression coefficients
                        beta =zeros(dimensionLS,1);
                        beta = regress(contiValue',regMatrix);
                        regContiValue = regMatrix*beta;
                       
                         %Nominal Regression test
                        regContiValue = regContiValue * 10000;
                        
                        exercisedPayoff = regContiValue;
                        % if regContiValue is bigger than exerciseValue
                        % then issue exercise call option
                        for idx_r=1:pathSize
                            if regContiValue(idx_r) >= exerciseValue(idx_r) 
                                exercisedPayoff(idx_r) = exerciseValue(idx_r);
                            end
                        end
                        
                        % we discount the new payoff and 
                        % assign it into the existing callable payoff
                        callable.payoff = model.discountPayoffRisky(currentTime,numMatTime ...
                        ,modelStateT,exercisedPayoff);
                        
                    end
                end

                nonCall.npv = mean(nonCall.payoff);
                out.nonCall = nonCall;
                callable.npv = mean(callable.payoff);
                out.callable = callable;
                out.dfCol = dfCol;
                out.dummy1 = dummy1;
                out.dummy2 = dummy2;
                out.daycountFraction = schedule.dayCountFraction;
                
        end
        
        function out = computePriceClosedSimple(pricer,valueDate)
                security = pricer.m_security;
                numMethod = pricer.m_numMethod;
                model = pricer.m_model;
                schedule = security.couponSchedule;
                
                % numMethodTimeSteps Generate
                mcOneTimeStep = numMethod.mcOneTimeStep;
                outInfo = numMethod.numMethodTimeStepsGenerateDW(valueDate,schedule,mcOneTimeStep);
            
                numMethodInfo.timeStepsInfo = outInfo;

                % mc random number init
                %pathSize = 2^16;
                pathSize = numMethod.pathSize;
                
                pathSize = 1;
                
%                 % generate random numbers and modelstates forward
%                 rng('default');
%                 rng(0);
% 
                numOfFactors = 2;
% 
%                 %Z= randn(numOfFactors*outInfo.totalTimeStepsSize,pathSize/2);
%                 Z= randn(pathSize/2,numOfFactors*outInfo.totalTimeStepsSize);
%                 Z=[Z;-Z];
%                 Z=Z';
%                 
%                 
%                 %TimeInversion
%                 sizeZ = size(Z,1);
%                 A = zeros(sizeZ,sizeZ);
%                 for i=1:sizeZ
%                     A(i,sizeZ + 1 -i) = 1;
%                 end
%                 Z = A*Z;
% 
%                 % we check with the c, so we use generated random numbers
% %                 mc_random = importdata('mc_random.xlsx');
% %                 mc_random = importdata('D:\mc_random_number_new_report.txt');
% %                 Z= mc_random;
%                 
                numMethodInfo.pathSize = pathSize;
                numMethodInfo.numOfFactors = numOfFactors;
                numMethodInfo.totalTimeSteps = outInfo.totalTimeSteps;
                numMethodInfo.totalTimeStepsSize = outInfo.totalTimeStepsSize;
%                 numMethodInfo.Z = Z;
% 
% 
% 
%                 %generate model states for the given timeSteps
%                 modelStates = numMethod.generateModelStates(valueDate,schedule,numMethodInfo,model);

                % numeraire info
                numMatTime = numMethodInfo.totalTimeSteps(end);
                dfNumMat = model.DF(numMatTime/365.0);
                
                % induct backward and calculate cashflow
            
                periodCount = schedule.numOfPeriod;
                
                % cashflow columns initialize
                % 1st nonCall
                nonCall.npv = 0;
                nonCall.cashflow_e = zeros(periodCount + 1,1);
                nonCall.cashflow_npv_e = zeros(periodCount + 1,1);
                nonCall.rangeProb_e = zeros(periodCount + 1,1);
                nonCall.df_e = zeros(periodCount + 1,1);
                nonCall.df_c = zeros(periodCount + 1,1);

                % debug information, cashflow & npv per path
                nonCall.payoff = zeros(1,pathSize);
                nonCall.payoffStates.cashflow = zeros(periodCount+1,pathSize);
                nonCall.payoffStates.cashflow_npv = zeros(periodCount+1,pathSize);
                nonCall.payoffStates.df = zeros(periodCount + 1,pathSize);
                
                % intermediate variables
                payoffStateA.cashflow = zeros(1, pathSize);
                payoffStateA.cashflow_npv = zeros(1, pathSize);
                payoffStateA.df = zeros(1,pathSize);
                
                % we calculate cashflow backward from maturityDate to valueDate
                % rangeAccrual coupons are added at the startDate of the Period
                % At maturity

                nominal = security.nominal;
                coupon = security.coupon;
                callYN = security.callYN;
                
                ralower = security.lower;
                raupper = security.upper;
                
                ralowerS = security.lowerS;
                raupperS = security.upperS;
                
                tenorCD = security.tenor;
                
                tenorLong = security.tenorL;
                tenorShort = security.tenorS;
                
                currentTime = numMethodInfo.totalTimeSteps(end);
                totalTimeSteps = numMethodInfo.timeStepsInfo.totalTimeSteps;
                totalTimeStepsSize = numMethodInfo.timeStepsInfo.totalTimeStepsSize;
                if totalTimeStepsSize == 0 || (totalTimeStepsSize == 1 && totalTimeSteps(1) == 0) 
                    out.nonCall = nonCall;
                    return;
                end
                
                currentTimeIdx = totalTimeStepsSize;
                
                currentNodeIdx = periodCount + 1;
                
%                 modelStateT = modelStates(numOfFactors*(totalTimeStepsSize-1)+1:numOfFactors*(totalTimeStepsSize-1)+2,:);
                payoffStateA.cashflow = nominal*ones(1,pathSize);
                %process payoff at maturity
                nonCall.payoffStates.cashflow(currentNodeIdx,:) = payoffStateA.cashflow;
                
%                 payoffStateA.cashflow_npv = model.discountPayoff(currentTime,numMatTime ...
%                     ,modelStateT,payoffStateA.cashflow);
%                 
                payoffStateA.cashflow_npv = model.DFRisky(currentTime/365.0)*payoffStateA.cashflow;
                
                nonCall.payoffStates.cashflow_npv(currentNodeIdx,:) = payoffStateA.cashflow_npv ;
                
%                 payoffStateA.df = model.discountPayoff(currentTime,numMatTime ...
%                     ,modelStateT,1.0*ones(1,pathSize));
                payoffStateA.df = model.DFRisky(currentTime/365.0) *ones(1,pathSize);
                    

                nonCall.payoffStates.df(currentNodeIdx,:) = payoffStateA.df;
                nonCall.payoff = nonCall.payoff + payoffStateA.cashflow_npv;
                
                nonCall.cashflow_e(currentNodeIdx) = mean(payoffStateA.cashflow);
                nonCall.cashflow_npv_e(currentNodeIdx) = mean(payoffStateA.cashflow_npv);
                nonCall.df_e(currentNodeIdx) = mean(payoffStateA.df);
                nonCall.df_c(currentNodeIdx) = model.DFRisky(currentTime/365.0);
                
%                 % callable at maturity is the same as noncallable
%                 callable.payoffStates.cashflow(currentNodeIdx,:) = nonCall.payoffStates.cashflow(currentNodeIdx,:);
%                 callable.payoffStates.cashflow_npv(currentNodeIdx,:) = nonCall.payoffStates.cashflow_npv(currentNodeIdx,:);
%                 callable.payoff = callable.payoff + callable.payoffStates.cashflow_npv(currentNodeIdx,:);
%                 
%                 callable.cashflow_e(currentNodeIdx) = nonCall.cashflow_e(currentNodeIdx);
%                 callable.cashflow_npv_e(currentNodeIdx) = nonCall.cashflow_npv_e(currentNodeIdx);
                  
                lastTimeIdx = currentTimeIdx;
                
                
                % flag for inclusive schedule
%                 simpleYN = false;
                simpleYN = 1;
                
                for i=periodCount:-1:1
                    % past schedule neglect
                    if DateDiff(valueDate,schedule.payDates(i)) >= 0, continue;end 
                    startTimeIdx = numMethodInfo.timeStepsInfo.startTimeIdx(i);
                    endTimeIdx = numMethodInfo.timeStepsInfo.endTimeIdx(i);

                    inclusiveYN = numMethodInfo.timeStepsInfo.inclusiveYN(i);
                     %% FromToInduction start

                    %modelsStates induct backward(lastTimeIdx to endTimeIdx)
                    while endTimeIdx < lastTimeIdx
                        lastTimeIdx = lastTimeIdx -1;
                        if lastTimeIdx == endTimeIdx break;end
                    end

                    currentTimeIdx = lastTimeIdx;
                    currentTime = totalTimeSteps(currentTimeIdx);
                    % now we are at endTimeIdx
                    % variables for coupon valuation initialized
%                     ralower = security.lower;
%                     raupper = security.upper;
%                     tenorLong = security.tenorL;
%                     tenorShort = security.tenorS;

                    cD = 0; % cumulated day count fraction
                    cP = 0; 
                    dI = 0; % dummy idx for loop
                    dcfC = 0;
                    dcfN = 0;
                    dD = 0;
                    idxCD = zeros(1,pathSize);
                    
                    idxL = zeros(1,pathSize);
                    idxS = zeros(1,pathSize);
                    
                    idxC = zeros(1,pathSize);
                    idxN = zeros(1,pathSize);
                    
                    spreadC = zeros(1,pathSize);
                    spreadN = zeros(1,pathSize);
                    probSpread = zeros(1,pathSize);
                    
                    numOfTimeSteps = numMethodInfo.timeStepsInfo.numOfTimeSteps(i);
                    
                    % if inclusive then we use simpleYN rangeProb
                    %if inclusive then we do not evaluate the last step but
                    % use fixing instead 
                    endTimeStep = 1;
                    if inclusiveYN
%                         simpleYN = true;
                        simpleYN = 1;
                        endTimeStep = 2;
                    end
                    
                    for j=numOfTimeSteps:-1:endTimeStep
                        % evaluate dealdescription
                        % last step is omitted
                        currentTime = totalTimeSteps(currentTimeIdx);
                        %dcfC = currentTime/365.0;
                        dcfC = currentTime;
                        
                        if j~= numOfTimeSteps
                            dD = (dcfN-dcfC);
                        end

                        toDate = AddDate(valueDate,currentTime,'day');
                        tpDate = schedule.payDates(i);
                        
                                                                     
                        spreadC = model.CDCMSSpreadDigitalRangeDate(valueDate,toDate,tpDate,tenorCD,tenorLong,tenorShort,0.25,ralower,raupper,ralowerS,raupperS);
%                         spreadC = model.CMSSpreadDigitalRangeDate(valueDate,toDate,tpDate,tenorLong,tenorShort,0.25,ralower,raupper);
                        
                        %modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:numOfFactors*(currentTimeIdx-1)+2,:);
%                         idxL = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorLong,modelStateT);
%                         idxS = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorShort,modelStateT);
%                         spreadC = idxL - idxS;
%                         probSpread = model.SRPr(spreadC,spreadN,ralower,raupper,simpleYN,pathSize)*dD; 

                        if simpleYN
                            probSpread = spreadC*dD;
                        else
                            probSpread = spreadC*dD;
%                             probSpread = 0.5*(spreadC + spreadN)*dD;
                            
                        end

                        
                        cD = cD + dD;
                        cP = cP + probSpread;

                        dcfN = dcfC;
                        spreadN = spreadC;

                        %induct backward to the next time step
                        % one time backward induction
                        lastTimeIdx = currentTimeIdx;

                        if startTimeIdx < lastTimeIdx
                            lastTimeIdx = lastTimeIdx -1;
                            currentTimeIdx = lastTimeIdx;
                        end


                    end

                    %% FromToInduction end
                    %% AtDate evaluation
                    % Adding a new cashflow to the payoff

                    dcf = schedule.dayCountFraction(i);
                    
                    if inclusiveYN
                        % we add valuation date
                        cD = schedule.dayCount(i);
                        rangeDays =  security.rangeDays(i);
                        payoffStateA.cashflow = nominal * coupon * dcf * (rangeDays + cP)/cD;
                    else
                        payoffStateA.cashflow = nominal * coupon * dcf * cP/cD;
                    end
                    % eventDate(i) = startDate(i) therefore there is no induction in modelStates
                    % i.e. currentTimeIdx has not changed
                    % no need to update modelStateT
                    % modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:end,:);
                    % since cashflow's are added at the startDate(i) we need to
                    % multiply cashflow with the df(startDate(i), payDate(i))
                    payTime = DateDiff(schedule.payDates(i),valueDate);
                    
                    %event is at startDate so we need to discount the
                    %cashflow first from paydate to startdate
%                     df_sp = model.discountFactor(currentTime, payTime,numMatTime,modelStateT);

%                     for k=1:pathSize
%                         payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_sp(k);
%                     end
                    
                    % process payoff
                    
                    currentNodeIdx = i;
                    
                    % 1st nonCall column
                    nonCall.payoffStates.cashflow(currentNodeIdx,:) = payoffStateA.cashflow;
                    payoffStateA.cashflow_npv = payoffStateA.cashflow * model.DFRisky(payTime/365.0);
                    
%                     payoffStateA.cashflow_npv = model.discountPayoff(currentTime,numMatTime ...
%                     ,modelStateT,payoffStateA.cashflow);
                    nonCall.payoffStates.cashflow_npv(currentNodeIdx,:) = payoffStateA.cashflow_npv;
                    payoffStateA.df = model.DFRisky(payTime/365.0);
%                     payoffStateA.df = model.discountPayoff(currentTime,numMatTime ...
%                     ,modelStateT,1.0*ones(1,pathSize));
                    nonCall.payoffStates.df(currentNodeIdx,:) = payoffStateA.df;
                    
                    nonCall.payoff = nonCall.payoff + payoffStateA.cashflow_npv;
                    
                    nonCall.cashflow_e(currentNodeIdx) = mean(payoffStateA.cashflow);
                    nonCall.rangeProb_e(currentNodeIdx) = mean(nonCall.cashflow_e(currentNodeIdx)/(nominal * coupon * dcf));
                    nonCall.cashflow_npv_e(currentNodeIdx) = mean(payoffStateA.cashflow_npv);

                    nonCall.df_e(currentNodeIdx) = mean(payoffStateA.df);
                    nonCall.df_c(currentNodeIdx) = model.DFRisky(currentTime/365.0);
                    
%                     %2nd callable column
%                     callable.payoffStates.cashflow(currentNodeIdx,:) = nonCall.payoffStates.cashflow(currentNodeIdx,:);
%                     callable.payoffStates.cashflow_npv(currentNodeIdx,:) = nonCall.payoffStates.cashflow_npv(currentNodeIdx,:);
%                     callable.payoff = callable.payoff + callable.payoffStates.cashflow_npv(currentNodeIdx,:);
%                     
%                     callable.cashflow_e(currentNodeIdx) = nonCall.cashflow_e(currentNodeIdx);
%                     callable.cashflow_npv_e(currentNodeIdx) = nonCall.cashflow_npv_e(currentNodeIdx);

                    % if exercise is possible
                    if i == 1
                        dummaaa = 1;
                    end
                    
                    % call event happens  at the start of the period
                    % at call date, by default, call option is expired
                    
%                     if callYN(i) && (DateDiff(valueDate,schedule.startDates(i)) < 0)
%                         % 1.undiscount callable.payoff
%                         % 2.build the state variables matrix
%                         % 3.regress 1's with the given state variables
%                         % (model states 1,2)
%                         % 4. exercise per each path in
%                         % discount the new payoff and assign into callable
%                         % column
%                         
%                         %1 undiscount callable.payoff and get the
%                         %contiValue
%                        
%                         %exercise value
%                         exerciseValue = security.nominal*ones(1,pathSize);
%                         contiValue = zeros(1,pathSize);
%                         onePayoff = ones(1,pathSize);
%                         onePayoffD = model.discountPayoff(currentTime,numMatTime ...
%                         ,modelStateT,onePayoff);
%                         contiValue = callable.payoff./onePayoffD;
%                         
%                         % build the state variabls matrix
%                         decisionVar1 = modelStateT(1,:);
%                         decisionVar2 = modelStateT(2,:);
%                         % we use upto 2nd order polynomial as regression
%                         % equatioin
%                         dimensionLS = 6;
%                         regMatrix = ones(pathSize,dimensionLS);
%                         
%                         for idx_r=1:pathSize
%                             regMatrix(idx_r,2) = decisionVar1(idx_r);
%                             regMatrix(idx_r,3) = decisionVar2(idx_r);
%                             regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r);
%                             regMatrix(idx_r,5) = decisionVar1(idx_r)*decisionVar2(idx_r);
%                             regMatrix(idx_r,6) = decisionVar2(idx_r)*decisionVar2(idx_r);
%                         end
%                         
%                         % simple regression <- for path dependent option
%                         % we should change it
%                         % beta  : regression coefficients
%                         beta =zeros(dimensionLS,1);
%                         beta = regress(contiValue',regMatrix);
%                         regContiValue = regMatrix*beta;
%                         
%                         exercisedPayoff = regContiValue;
%                         % if regContiValue is bigger than exerciseValue
%                         % then issue exercise call option
%                         for idx_r=1:pathSize
%                             if regContiValue(idx_r) >= exerciseValue(idx_r) 
%                                 exercisedPayoff(idx_r) = exerciseValue(idx_r);
%                             end
%                         end
%                         
%                         % we discount the new payoff and 
%                         % assign it into the existing callable payoff
%                         callable.payoff = model.discountPayoff(currentTime,numMatTime ...
%                         ,modelStateT,exercisedPayoff);
%                         
%                     end
                end

                nonCall.npv = mean(nonCall.payoff);
                out.pricerInfo.nonCall = nonCall;
                
%                 callable.npv = mean(callable.payoff);
%                 out.pricerInfo.callable = callable;
                out.daycountFraction = schedule.dayCountFraction;
                
        end
        
        
        
        
    end
    
end



In [ ]:
classdef DailyWeeklyRiskyPricer_CDAndKOSPI200RAHyb < H_RootObject
    %Pricer(Security,Model,NumMethod)
    %Prcer is constructed from Security, Model, NumMethod 
    
    properties
        m_security
        m_model
        m_numMethod
        overHedgeYN
        
    end
    
    methods
        function pricer = DailyWeeklyRiskyPricer_CDAndKOSPI200RAHyb(security,model,numMethod,overHedgeYN)
                pricer.m_security = security;
                pricer.m_model = model;
                pricer.m_numMethod = numMethod;
                pricer.overHedgeYN = overHedgeYN;
                
        end
        
        function out = computePriceMCCallableGeneric(pricer,valueDate,pricerOption)
                security = pricer.m_security;
                numMethod = pricer.m_numMethod;
                model = pricer.m_model;
                schedule = security.couponSchedule;
                callSchedule = security.callSchedule;
                
                % numMethodTimeSteps Generate
                mcOneTimeStep = numMethod.mcOneTimeStep;
                outInfo = numMethod.numMethodTimeStepsGenerateDW(valueDate,schedule,mcOneTimeStep);
            
                numMethodInfo.timeStepsInfo = outInfo;

                % mc random number init
                %pathSize = 2^16;
                pathSize = numMethod.pathSize;

                % generate random numbers and modelstates forward
                rng('default');
                rng(0);

                % LGM1F : one factor
                % LGM2F : two factor
                
                numOfFactors = model.numOfFactor;

                %Z= randn(numOfFactors*outInfo.totalTimeStepsSize,pathSize/2);
                Z= randn(pathSize/2,numOfFactors*outInfo.totalTimeStepsSize);
                Z=[Z;-Z];
                Z=Z';
                
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;

                % we check with the c code, so we use generated random numbers
%                 mc_random = importdata('mc_random.xlsx');
%                 mc_random = importdata('D:\mc_random_number_new_report.txt');
%                 Z= mc_random;
                
                numMethodInfo.pathSize = pathSize;
                numMethodInfo.numOfFactors = numOfFactors;
                numMethodInfo.totalTimeSteps = outInfo.totalTimeSteps;
                numMethodInfo.totalTimeStepsSize = outInfo.totalTimeStepsSize;
                numMethodInfo.Z = Z;



                %numMethod generate model states for the given timeSteps
                modelStates = numMethod.generateModelStatesNF(valueDate,schedule,numMethodInfo,model);

                % numeraire info
                numMatTime = numMethodInfo.totalTimeSteps(end);
                days = DateDiff(schedule.payDates(schedule.numOfPeriod),valueDate);
                numMatTime = max(numMatTime, days);
                dfNumMat = model.DFRisky(numMatTime/365.0);
                
                numeraire.numMatTime = numMatTime;
                numeraire.dfNumMat = dfNumMat;
                
                % induct backward and calculate cashflow
            
                periodCount = schedule.numOfPeriod;
                
                % pricer single info initialize
                
                nonCall = PricerInfo(periodCount +1, pathSize);
                callable = PricerInfo(periodCount +1, pathSize);
                dfCol = PricerInfo(periodCount +1, pathSize);
                
                dummy1 = PricerInfo(periodCount +1, pathSize);
                dummy2 = PricerInfo(periodCount +1, pathSize);
                
                
                % intermediate variables
                payoffStateA.cashflow = zeros(1, pathSize);
                payoffStateA.cashflow_npv = zeros(1, pathSize);
                
            
                % we calculate cashflow backward from maturityDate to valueDate
                % rangeAccrual coupons are added at the startDate of the Period
                % At maturity

                nominal = security.nominal;
                coupon = security.coupon;
                callYN = security.callYN;
                
                ralower = security.lower;
                raupper = security.upper;
                
                ralowerS = security.lowerS;
                raupperS = security.upperS;
                
                tenorCD = security.tenor;
                
                tenorLong = security.tenorL;
                tenorShort = security.tenorS;
                
                currentTime = numMethodInfo.totalTimeSteps(end);
                totalTimeStepsSize = numMethodInfo.totalTimeStepsSize;
                currentTimeIdx = totalTimeStepsSize;
                
                currentNodeIdx = periodCount + 1;
                
                modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:numOfFactors*(currentTimeIdx-1)+numOfFactors,:);
                
                payoffStateA.cashflow = nominal*ones(1,pathSize);
                
                payTime = DateDiff(schedule.payDates(schedule.numOfPeriod),valueDate);
                df_ep = model.discountFactorRisky(currentTime, payTime,numMatTime,modelStateT);
                for k=1:pathSize
                    payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                end
                
                %process payoff at maturity
                nonCall.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                
                callable.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                
                %df column
                payoffStateA.cashflow = ones(1,pathSize);
                for k=1:pathSize
                    payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                end
                
                dfCol.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                                 
                %dummy column

                if strcmp(pricerOption,'PSA')
                    payoffStateA.cashflow = model.CMS_PSA_SimpleDate('KRW',valueDate,currentTime,numMatTime, tenorLong,modelStateT);
                    dummy1.ProcessPayoff(model,currentTime,numeraire, ...
                                 modelStateT,payoffStateA.cashflow,currentNodeIdx);
                    
                    payoffStateA.cashflow = model.CMS_PSA_SimpleDate('USD',valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                    dummy2.ProcessPayoff(model,currentTime,numeraire, ...
                                 modelStateT,payoffStateA.cashflow,currentNodeIdx);
                
                else
                    payoffStateA.cashflow = model.Libor_SimpleDate('KRW',valueDate,currentTime,currentTime,numMatTime, tenorLong,modelStateT); 
                    dummy1.ProcessPayoff(model,currentTime,numeraire, ...
                                 modelStateT,payoffStateA.cashflow,currentNodeIdx);
                             
                    payoffStateA.cashflow = model.Libor_SimpleDate('USD',valueDate,currentTime,currentTime,numMatTime, tenorShort,modelStateT); 
                    dummy2.ProcessPayoff(model,currentTime,numeraire, ...
                                 modelStateT,payoffStateA.cashflow,currentNodeIdx);
                    
                end
                

%                 payoffStateA.cashflow = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                
%                 payoffStateA.cashflow = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                
                
                                 
                  

                lastTimeIdx = currentTimeIdx;
                totalTimeSteps = numMethodInfo.timeStepsInfo.totalTimeSteps;
                
                % flag for inclusive schedule
%                 simpleYN = false;
%                 simpleYN = true;
                if pricer.overHedgeYN == true 
                    simpleYN = 2;
                else
                    simpleYN = 1;
                end
                
                for i=periodCount:-1:1
                    % past schedule neglect
                    if DateDiff(valueDate,schedule.payDates(i)) >= 0, continue;end 
                    startTimeIdx = numMethodInfo.timeStepsInfo.startTimeIdx(i);
                    endTimeIdx = numMethodInfo.timeStepsInfo.endTimeIdx(i);

                    inclusiveYN = numMethodInfo.timeStepsInfo.inclusiveYN(i);
                 %% FromToInduction start

                    %modelsStates induct backward(lastTimeIdx to endTimeIdx)
                    while endTimeIdx < lastTimeIdx
                        lastTimeIdx = lastTimeIdx -1;
                        if lastTimeIdx == endTimeIdx break;end
                    end

                    currentTimeIdx = lastTimeIdx;
                    currentTime = totalTimeSteps(currentTimeIdx);
                    % now we are at endTimeIdx
                    % variables for coupon valuation initialized
                    

                    cD = 0; % cumulated day count fraction
                    cP = 0; 
                    dI = 0; % dummy idx for loop
                    dcfC = 0;
                    dcfN = 0;
                    dD = 0;
                    idxCD = zeros(1,pathSize);
                    idxL = zeros(1,pathSize);
                    idxS = zeros(1,pathSize);
                    
                    idxC = zeros(1,pathSize);
                    idxN = zeros(1,pathSize);
                    
                    spreadC = zeros(1,pathSize);
                    spreadN = zeros(1,pathSize);
                    probSpread = zeros(1,pathSize);
                    
                    numOfTimeSteps = numMethodInfo.timeStepsInfo.numOfTimeSteps(i);
                    
                    % if inclusive then we use simpleYN rangeProb
                    %if inclusive then we do not evaluate the last step but
                    % use fixing instead 
                    endTimeStep = 1;
                    if inclusiveYN
%                         simpleYN = true;
                        simpleYN = 1;
                        endTimeStep = 2;
                        if pricer.overHedgeYN == true 
                            simpleYN = 2;
                        end
                    end
                    
                    for j=numOfTimeSteps:-1:endTimeStep
                        % evaluate dealdescription
                        % last step is omitted
                        currentTime = totalTimeSteps(currentTimeIdx);
                        %dcfC = currentTime/365.0;
                        dcfC = currentTime;
                        
                        if j~= numOfTimeSteps
                            dD = (dcfN-dcfC);
                        end

                        modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:numOfFactors*(currentTimeIdx-1)+numOfFactors,:);
                        
                        idxCD = model.Libor_SimpleDate('KRW',valueDate,currentTime,currentTime,numMatTime, tenorCD,modelStateT);
                        idxC = idxCD;
                        
                        if strcmp(pricerOption,'PSA')
                            idxL = model.CMS_PSA_SimpleDate('KRW',valueDate,currentTime,numMatTime, tenorLong,modelStateT);
                            idxS = model.CMS_PSA_SimpleDate('USD',valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                        else
                            idxL = model.Libor_SimpleDate('KRW',valueDate,currentTime,currentTime,numMatTime, tenorLong,modelStateT);
                            idxS = model.Libor_SimpleDate('USD',valueDate,currentTime,currentTime,numMatTime, tenorShort,modelStateT);
                        end
                        spreadC = idxL - idxS;

                        probSpread = model.DRPr(idxC,idxN,ralower,raupper,spreadC,spreadN,ralowerS,raupperS,simpleYN,pathSize)*dD;
                    
%                         probSpread = model.SRPr(spreadC,spreadN,ralower,raupper,simpleYN,pathSize)*dD;
                        cD = cD + dD;
                        cP = cP + probSpread;

                        dcfN = dcfC;
                        
                        idxN = idxC;
                        spreadN = spreadC;

                        %induct backward to the next time step
                        % one time backward induction
                        lastTimeIdx = currentTimeIdx;

                        if startTimeIdx < lastTimeIdx
                            lastTimeIdx = lastTimeIdx -1;
                            currentTimeIdx = lastTimeIdx;
                        end


                    end

                    %% FromToInduction end
        % if the current period is inclusive, then the start date is not evaluated in the above for loop.
		% instead the fixing will be treated separately below.
		% however current_time_idx is updated at the end of the for loop.
		% so we need to update model_states accordingly.
		% if model_states were updated previously then the following additionaly step doesn't affect the result anyway.
                    currentTime = totalTimeSteps(currentTimeIdx);
                    modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:numOfFactors*(currentTimeIdx-1)+numOfFactors,:);
                    
                    %% AtDate evaluation
                    % Adding a new cashflow to the payoff

                    dcf = schedule.dayCountFraction(i);
                    
                    if inclusiveYN
                        % we add valuation date
                        cD = schedule.dayCount(i);
                        rangeDays =  security.rangeDays(i);
                        payoffStateA.cashflow = nominal * coupon * dcf * (rangeDays + cP)/cD;
                    else
                        payoffStateA.cashflow = nominal * coupon * dcf * cP/cD;
                    end
                    % eventDate(i) = startDate(i) therefore there is no induction in modelStates
                    % i.e. currentTimeIdx has not changed
                    % no need to update modelStateT
                    % modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:end,:);
                    % since cashflow's are added at the startDate(i) we need to
                    % multiply cashflow with the df(startDate(i), payDate(i))
                    payTime = DateDiff(schedule.payDates(i),valueDate);
                    
                    %event is at startDate so we need to discount the
                    %cashflow first from paydate to startdate
                    df_ep = model.discountFactorRisky(currentTime, payTime,numMatTime,modelStateT);

                    for k=1:pathSize
                        payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                    end
                    
                    % process payoff
                    
                    currentNodeIdx = i;
                    
                    nonCall.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                
                    callable.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                    
                    %df column
                    payoffStateA.cashflow = ones(1,pathSize);
                    for k=1:pathSize
                        payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                    end
                    
                    dfCol.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                                 
                    %dummy column
                    if strcmp(pricerOption,'PSA')
                        payoffStateA.cashflow = model.CMS_PSA_SimpleDate('KRW',valueDate,currentTime,numMatTime, tenorLong,modelStateT);
                        dummy1.ProcessPayoff(model,currentTime,numeraire, ...
                                         modelStateT,payoffStateA.cashflow,currentNodeIdx);



                        payoffStateA.cashflow = model.CMS_PSA_SimpleDate('USD',valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                        dummy2.ProcessPayoff(model,currentTime,numeraire, ...
                                         modelStateT,payoffStateA.cashflow,currentNodeIdx);
                    
                    else
                        payoffStateA.cashflow = model.Libor_SimpleDate('KRW',valueDate,currentTime,currentTime,numMatTime, tenorLong,modelStateT); 
                        dummy1.ProcessPayoff(model,currentTime,numeraire, ...
                                 modelStateT,payoffStateA.cashflow,currentNodeIdx);
                        
                        payoffStateA.cashflow = model.Libor_SimpleDate('USD',valueDate,currentTime,currentTime,numMatTime, tenorShort,modelStateT); 
                        dummy2.ProcessPayoff(model,currentTime,numeraire, ...
                                 modelStateT,payoffStateA.cashflow,currentNodeIdx);
                        
                    end

                                 
                    
                    
                    
                    % if exercise is possible
                    if i == 1
                        dummaaa = 1;
                    end
                    
                    % call event happens  at the start of the period
                    % at call date, by default, call option is expired
                    
                    if callYN(i) && (DateDiff(valueDate,callSchedule.resetDates(i)) < 0)
                        % 1.undiscount callable.payoff
                        % 2.build the state variables matrix
                        % 3.regress 1's with the given state variables
                        % (model states 1,2)
                        % 4. exercise per each path in
                        % discount the new payoff and assign into callable
                        % column
                        
                        %1 undiscount callable.payoff and get the
                        %contiValue
                       
                        %exercise value
                        exerciseValue = security.nominal*ones(1,pathSize);
                        payTime = DateDiff(callSchedule.payDates(i),valueDate);
                        %event is at startDate so we need to discount the
                        %cashflow first from paydate to startdate
                        df_ep = model.discountFactorRisky(currentTime, payTime,numMatTime,modelStateT);
                        for k=1:pathSize
                            exerciseValue(k) = exerciseValue(k) * df_ep(k);
                        end
                        
                        contiValue = zeros(1,pathSize);
                        onePayoff = ones(1,pathSize);
                        onePayoffD = model.discountPayoffRisky(currentTime,numMatTime ...
                        ,modelStateT,onePayoff);
                        contiValue = callable.payoff./onePayoffD;
                        
                        %Nominal Regression test
                        contiValue = contiValue/10000;
                        
                        % build the state variabls matrix
                        dimensionLS = (numOfFactors + 1)*(numOfFactors + 2)/2;
                        
                        decisionVar = zeros(numOfFactors,size(modelStateT,2));
                        
                        for idxLS =1:numOfFactors
                            decisionVar(idxLS,:) = modelStateT(idxLS,:); 
                        end
%                         decisionVar1 = modelStateT(1,:);
%                         decisionVar2 = modelStateT(2,:);
                        % we use upto 2nd order polynomial as regression
                        % equatioin
                        
                        regMatrix = ones(pathSize,dimensionLS);
                        
                        for idx_r=1:pathSize
                            for idxLS=1:numOfFactors
                                regMatrix(idx_r,idxLS+1) = decisionVar(idxLS,idx_r);
                            end
                            
                            tempIdx = 1;
                            for idxLS1=1:numOfFactors
                                for idxLS2=idxLS1:numOfFactors
                                    regMatrix(idx_r,numOfFactors+1+tempIdx) = decisionVar(idxLS1,idx_r)*decisionVar(idxLS2,idx_r);
                                    tempIdx = tempIdx + 1;
                                end
                            end
                            
                        end
                        
                        % simple regression <- for path dependent option
                        % we should change it
                        % beta  : regression coefficients
                        beta =zeros(dimensionLS,1);
                        beta = regress(contiValue',regMatrix);
                        regContiValue = regMatrix*beta;
                       
                         %Nominal Regression test
                        regContiValue = regContiValue * 10000;
                        
                        exercisedPayoff = regContiValue;
                        % if regContiValue is bigger than exerciseValue
                        % then issue exercise call option
                        for idx_r=1:pathSize
                            if regContiValue(idx_r) >= exerciseValue(idx_r) 
                                exercisedPayoff(idx_r) = exerciseValue(idx_r);
                            end
                        end
                        
                        % we discount the new payoff and 
                        % assign it into the existing callable payoff
                        callable.payoff = model.discountPayoffRisky(currentTime,numMatTime ...
                        ,modelStateT,exercisedPayoff);
                        
                    end
                end

                nonCall.npv = mean(nonCall.payoff);
                out.nonCall = nonCall;
                callable.npv = mean(callable.payoff);
                dfCol.npv = mean(dfCol.payoff);
                dummy1.npv = mean(dummy1.npv);
                dummy2.npv = mean(dummy2.npv);
                out.callable = callable;
                out.dfCol = dfCol;
                out.dummy1 = dummy1;
                out.dummy2 = dummy2;
                out.daycountFraction = schedule.dayCountFraction;
                
        end
        
        function out = computePriceMCCallableGenericReg2(pricer,valueDate,pricerOption)
                security = pricer.m_security;
                numMethod = pricer.m_numMethod;
                model = pricer.m_model;
                schedule = security.couponSchedule;
                callSchedule = security.callSchedule;
                
                % numMethodTimeSteps Generate
                mcOneTimeStep = numMethod.mcOneTimeStep;
                outInfo = numMethod.numMethodTimeStepsGenerateDW(valueDate,schedule,mcOneTimeStep);
            
                numMethodInfo.timeStepsInfo = outInfo;

                % mc random number init
                %pathSize = 2^16;
                pathSize = numMethod.pathSize;

                % generate random numbers and modelstates forward
                rng('default');
                rng(0);

                % LGM1F : one factor
                % LGM2F : two factor
                
                numOfFactors = model.numOfFactor;

                %Z= randn(numOfFactors*outInfo.totalTimeStepsSize,pathSize/2);
                Z= randn(pathSize/2,numOfFactors*outInfo.totalTimeStepsSize);
                Z=[Z;-Z];
                Z=Z';
                
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;

                % we check with the c code, so we use generated random numbers
%                 mc_random = importdata('mc_random.xlsx');
%                 mc_random = importdata('D:\mc_random_number_new_report.txt');
%                 Z= mc_random;
                
                numMethodInfo.pathSize = pathSize;
                numMethodInfo.numOfFactors = numOfFactors;
                numMethodInfo.totalTimeSteps = outInfo.totalTimeSteps;
                numMethodInfo.totalTimeStepsSize = outInfo.totalTimeStepsSize;
                numMethodInfo.Z = Z;


                % numeraire info
                numMatTime = numMethodInfo.totalTimeSteps(end);
                days = DateDiff(schedule.payDates(schedule.numOfPeriod),valueDate);
                numMatTime = max(numMatTime, days);
                dfNumMat = model.DFRisky(numMatTime/365.0);
                
                numeraire.numMatTime = numMatTime;
                numeraire.dfNumMat = dfNumMat;
                numMethodInfo.numMatTime = numMatTime; 
                %numMethod generate model states for the given timeSteps
%                 modelStates = numMethod.generateModelStatesNF(valueDate,schedule,numMethodInfo,model);
                modelStates = numMethod.generateModelStatesNFHyb(valueDate,schedule,numMethodInfo,model);

                
                
                % induct backward and calculate cashflow
            
                periodCount = schedule.numOfPeriod;
                
                % pricer single info initialize
                
                nonCall = PricerInfo(periodCount +1, pathSize);
                callable = PricerInfo(periodCount +1, pathSize);
                dfCol = PricerInfo(periodCount +1, pathSize);
                
                dummy1 = PricerInfo(periodCount +1, pathSize);
                dummy2 = PricerInfo(periodCount +1, pathSize);
                
                
                % intermediate variables
                payoffStateA.cashflow = zeros(1, pathSize);
                payoffStateA.cashflow_npv = zeros(1, pathSize);
                
                payoffStateB.cashflow = zeros(1, pathSize);
                payoffStateB.cashflow_npv = zeros(1, pathSize);
                
                % we calculate cashflow backward from maturityDate to valueDate
                % rangeAccrual coupons are added at the startDate of the Period
                % At maturity

                hedgeBuffer = security.hedgeBuffer;
                
                nominal = security.nominal;
                coupon = security.coupon;
                callYN = security.callYN;
                
                ralower = security.lower;
                raupper = security.upper;
                
                ralowerS = security.lowerS;
                raupperS = security.upperS;
                
                tenorCD = security.tenor;
                
                tenorLong = security.tenorL;
                tenorShort = security.tenorS;
                
                currentTime = numMethodInfo.totalTimeSteps(end);
                totalTimeStepsSize = numMethodInfo.totalTimeStepsSize;
                currentTimeIdx = totalTimeStepsSize;
                
                currentNodeIdx = periodCount + 1;
                
                modelStateT = modelStates((numOfFactors+1)*(currentTimeIdx-1)+1:(numOfFactors+1)*(currentTimeIdx-1)+(numOfFactors+1),:);
                
                payoffStateA.cashflow = nominal*ones(1,pathSize);
                
                payTime = DateDiff(schedule.payDates(schedule.numOfPeriod),valueDate);
                df_ep = model.discountFactorRisky(currentTime, payTime,numMatTime,modelStateT);
                for k=1:pathSize
                    payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                end
                
                %process payoff at maturity
                nonCall.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                
                callable.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                
                %df column
                payoffStateA.cashflow = ones(1,pathSize);
                for k=1:pathSize
                    payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                end
                
                dfCol.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                                 
                %dummy column

%                 if strcmp(pricerOption,'PSA')
%                     payoffStateA.cashflow = model.CMS_PSA_SimpleDate('KRW',valueDate,currentTime,numMatTime, tenorLong,modelStateT);
%                     dummy1.ProcessPayoff(model,currentTime,numeraire, ...
%                                  modelStateT,payoffStateA.cashflow,currentNodeIdx);
%                     
%                     payoffStateA.cashflow = model.CMS_PSA_SimpleDate('KRW',valueDate,currentTime,numMatTime, tenorShort,modelStateT);
%                     dummy2.ProcessPayoff(model,currentTime,numeraire, ...
%                                  modelStateT,payoffStateA.cashflow,currentNodeIdx);
%                 
%                 else
% %                     payoffStateA.cashflow = model.Libor_SimpleDate('KRW',valueDate,currentTime,currentTime,numMatTime, tenorLong,modelStateT); 
% %                     dummy1.ProcessPayoff(model,currentTime,numeraire, ...
% %                                  modelStateT,payoffStateA.cashflow,currentNodeIdx);
% %                              
%                     payoffStateB.cashflow = model.EQSpotMC('KOSPI200',valueDate,currentTime,numMatTime,modelStateT); 
%                     dummy2.ProcessPayoff(model,currentTime,numeraire, ...
%                                  modelStateT,payoffStateA.cashflow,currentNodeIdx);
%                     
%                 end
                

%                 payoffStateA.cashflow = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                
%                 payoffStateA.cashflow = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                
                
                                 
                  

                lastTimeIdx = currentTimeIdx;
                totalTimeSteps = numMethodInfo.timeStepsInfo.totalTimeSteps;
                
                % flag for inclusive schedule
%                 simpleYN = false;
%                 simpleYN = true;
                if pricer.overHedgeYN == true 
                    simpleYN = 2;
                else
                    simpleYN = 1;
                end
                
                for i=periodCount:-1:1
                    % past schedule neglect
                    if DateDiff(valueDate,schedule.payDates(i)) >= 0, continue;end 
                    startTimeIdx = numMethodInfo.timeStepsInfo.startTimeIdx(i);
                    endTimeIdx = numMethodInfo.timeStepsInfo.endTimeIdx(i);

                    inclusiveYN = numMethodInfo.timeStepsInfo.inclusiveYN(i);
                 %% FromToInduction start

                    %modelsStates induct backward(lastTimeIdx to endTimeIdx)
                    while endTimeIdx < lastTimeIdx
                        lastTimeIdx = lastTimeIdx -1;
                        if lastTimeIdx == endTimeIdx break;end
                    end

                    currentTimeIdx = lastTimeIdx;
                    currentTime = totalTimeSteps(currentTimeIdx);
                    % now we are at endTimeIdx
                    % variables for coupon valuation initialized
                    

                    cD = 0; % cumulated day count fraction
                    cP = 0; 
                    dI = 0; % dummy idx for loop
                    dcfC = 0;
                    dcfN = 0;
                    dD = 0;
                    idxCD = zeros(1,pathSize);
                    idxL = zeros(1,pathSize);
                    idxS = zeros(1,pathSize);
                    
                    idxC = zeros(1,pathSize);
                    idxN = zeros(1,pathSize);
                    
                    spreadC = zeros(1,pathSize);
                    spreadN = zeros(1,pathSize);
                    probSpread = zeros(1,pathSize);
                    
                    numOfTimeSteps = numMethodInfo.timeStepsInfo.numOfTimeSteps(i);
                    
                    % if inclusive then we use simpleYN rangeProb
                    %if inclusive then we do not evaluate the last step but
                    % use fixing instead 
                    endTimeStep = 1;
                    if inclusiveYN
%                         simpleYN = true;
                        simpleYN = 1;
                        endTimeStep = 2;
                        if pricer.overHedgeYN == true 
                            simpleYN = 2;
                        end
                    end
                    
                    for j=numOfTimeSteps:-1:endTimeStep
                        % evaluate dealdescription
                        % last step is omitted
                        currentTime = totalTimeSteps(currentTimeIdx);
                        %dcfC = currentTime/365.0;
                        dcfC = currentTime;
                        
                        if j~= numOfTimeSteps
                            dD = (dcfN-dcfC);
                        end

%                         modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:numOfFactors*(currentTimeIdx-1)+numOfFactors,:);
                        modelStateT = modelStates((numOfFactors+1)*(currentTimeIdx-1)+1:(numOfFactors+1)*(currentTimeIdx-1)+(numOfFactors+1),:);
                        
                        idxCD = model.Libor_SimpleDate('KRW',valueDate,currentTime,currentTime,numMatTime, tenorCD,modelStateT);
                        idxC = idxCD;
                        
                        if strcmp(pricerOption,'PSA')
                            idxL = model.CMS_PSA_SimpleDate('KRW',valueDate,currentTime,numMatTime, tenorLong,modelStateT);
                            idxS = model.CMS_PSA_SimpleDate('KRW',valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                        else
                            idxL = model.EQSpotMC('KOSPI200',valueDate,currentTime,numMatTime,modelStateT); 
%                             idxL = model.Libor_SimpleDate('KRW',valueDate,currentTime,currentTime,numMatTime, tenorLong,modelStateT);
%                             idxS = model.Libor_SimpleDate('USD',valueDate,currentTime,currentTime,numMatTime, tenorShort,modelStateT);
                        end
                        spreadC = idxL;
%                         spreadC = idxL - idxS;

                        probSpread = model.DRPr(idxC,idxN,ralower,raupper,spreadC,spreadN,80,120,simpleYN,hedgeBuffer,pathSize)*dD;
                    
%                         probSpread = model.SRPr(spreadC,spreadN,ralower,raupper,simpleYN,pathSize)*dD;
                        cD = cD + dD;
                        cP = cP + probSpread;

                        dcfN = dcfC;
                        
                        idxN = idxC;
                        spreadN = spreadC;

                        %induct backward to the next time step
                        % one time backward induction
                        lastTimeIdx = currentTimeIdx;

                        if startTimeIdx < lastTimeIdx
                            lastTimeIdx = lastTimeIdx -1;
                            currentTimeIdx = lastTimeIdx;
                        end


                    end

                    %% FromToInduction end
        % if the current period is inclusive, then the start date is not evaluated in the above for loop.
		% instead the fixing will be treated separately below.
		% however current_time_idx is updated at the end of the for loop.
		% so we need to update model_states accordingly.
		% if model_states were updated previously then the following additionaly step doesn't affect the result anyway.
                    currentTime = totalTimeSteps(currentTimeIdx);
%                     modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:numOfFactors*(currentTimeIdx-1)+numOfFactors,:);
                    modelStateT = modelStates((numOfFactors+1)*(currentTimeIdx-1)+1:(numOfFactors+1)*(currentTimeIdx-1)+(numOfFactors+1),:);
                    %% AtDate evaluation
                    % Adding a new cashflow to the payoff

                    dcf = schedule.dayCountFraction(i);
                    
                    if inclusiveYN
                        % we add valuation date
                        cD = schedule.dayCount(i);
                        rangeDays =  security.rangeDays(i);
                        payoffStateA.cashflow = nominal * coupon * dcf * (rangeDays + cP)/cD;
                    else
                        payoffStateA.cashflow = nominal * coupon * dcf * cP/cD;
                    end
                    % eventDate(i) = startDate(i) therefore there is no induction in modelStates
                    % i.e. currentTimeIdx has not changed
                    % no need to update modelStateT
                    % modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:end,:);
                    % since cashflow's are added at the startDate(i) we need to
                    % multiply cashflow with the df(startDate(i), payDate(i))
                    payTime = DateDiff(schedule.payDates(i),valueDate);
                    
                    %event is at startDate so we need to discount the
                    %cashflow first from paydate to startdate
                    df_ep = model.discountFactorRisky(currentTime, payTime,numMatTime,modelStateT);

                    for k=1:pathSize
                        payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                    end
                    
                    % process payoff
                    
                    currentNodeIdx = i;
                    
                    nonCall.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                
                    callable.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                    
                    %df column
                    payoffStateA.cashflow = ones(1,pathSize);
                    for k=1:pathSize
                        payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                    end
                    
                    dfCol.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                                 
                    %dummy column
                    if strcmp(pricerOption,'PSA')
                        payoffStateA.cashflow = model.CMS_PSA_SimpleDate('KRW',valueDate,currentTime,numMatTime, tenorLong,modelStateT);
                        dummy1.ProcessPayoff(model,currentTime,numeraire, ...
                                         modelStateT,payoffStateA.cashflow,currentNodeIdx);



                        payoffStateA.cashflow = model.CMS_PSA_SimpleDate('KRW',valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                        dummy2.ProcessPayoff(model,currentTime,numeraire, ...
                                         modelStateT,payoffStateA.cashflow,currentNodeIdx);
                    
                    else
%                         payoffStateA.cashflow = model.Libor_SimpleDate('KRW',valueDate,currentTime,currentTime,numMatTime, tenorLong,modelStateT); 
%                         dummy1.ProcessPayoff(model,currentTime,numeraire, ...
%                                  modelStateT,payoffStateA.cashflow,currentNodeIdx);
                        
%                         payoffStateA.cashflow = model.Libor_SimpleDate('USD',valueDate,currentTime,currentTime,numMatTime, tenorShort,modelStateT); 
%                         dummy2.ProcessPayoff(model,currentTime,numeraire, ...
%                                  modelStateT,payoffStateA.cashflow,currentNodeIdx);
                        
                    end

                                 
                    
                    
                    
                    % if exercise is possible
                    if i == 1
                        dummaaa = 1;
                    end
                    
                    % call event happens  at the start of the period
                    % at call date, by default, call option is expired
                    
                    if callYN(i) && (DateDiff(valueDate,callSchedule.resetDates(i)) < 0)
                        % 1.undiscount callable.payoff
                        % 2.build the state variables matrix
                        % 3.regress 1's with the given state variables
                        % (model states 1,2)
                        % 4. exercise per each path in
                        % discount the new payoff and assign into callable
                        % column
                        
                        %1 undiscount callable.payoff and get the
                        %contiValue
                       
                        %exercise value
                        exerciseValue = security.nominal*ones(1,pathSize);
                        payTime = DateDiff(callSchedule.payDates(i),valueDate);
                        %event is at startDate so we need to discount the
                        %cashflow first from paydate to startdate
                        df_ep = model.discountFactorRisky(currentTime, payTime,numMatTime,modelStateT);
                        for k=1:pathSize
                            exerciseValue(k) = exerciseValue(k) * df_ep(k);
                        end
                        
                        contiValue = zeros(1,pathSize);
                        onePayoff = ones(1,pathSize);
                        onePayoffD = model.discountPayoffRisky(currentTime,numMatTime ...
                        ,modelStateT,onePayoff);
                        contiValue = callable.payoff./onePayoffD;
                        
                        %Nominal Regression test
                        contiValue = contiValue/10000;
                        
                        % build the state variabls matrix
                        %zeroth order
                        dimensionLS = 1;
                        
                        %1st order
                        for idxLS =1:numOfFactors
                            dimensionLS =  dimensionLS +1; 
                        end
                        
                        %2nd order
                        for idxLS1=1:numOfFactors
                            for idxLS2=idxLS1:numOfFactors
                                dimensionLS = dimensionLS + 1;
                            end
                        end
                        
                        %3rd order
                        for idxLS1=1:numOfFactors
                            for idxLS2=idxLS1:numOfFactors
                                for idxLS3=idxLS2:numOfFactors
                                    dimensionLS = dimensionLS + 1;
                                end
                            end
                        end
                        
%                         dimensionLS = (numOfFactors + 1)*(numOfFactors + 2)/2;
                        
                        decisionVar = zeros(numOfFactors,size(modelStateT,2));
                        
                        for idxLS =1:numOfFactors
                            decisionVar(idxLS,:) = modelStateT(idxLS,:); 
                        end
%                         decisionVar1 = modelStateT(1,:);
%                         decisionVar2 = modelStateT(2,:);
                        % we use upto 2nd order polynomial as regression
                        % equatioin
                        
                        regMatrix = ones(pathSize,dimensionLS);
                        
                        for idx_r=1:pathSize
                            for idxLS=1:numOfFactors
                                regMatrix(idx_r,idxLS+1) = decisionVar(idxLS,idx_r);
                            end
                            
                            tempIdx = 1;
                            for idxLS1=1:numOfFactors
                                for idxLS2=idxLS1:numOfFactors
                                    regMatrix(idx_r,numOfFactors+1+tempIdx) = decisionVar(idxLS1,idx_r)*decisionVar(idxLS2,idx_r);
                                    tempIdx = tempIdx + 1;
                                end
                            end
                            
                            tempIdx2 = 1;
                            for idxLS1=1:numOfFactors
                                for idxLS2=idxLS1:numOfFactors
                                    for idxLS3=idxLS2:numOfFactors
                                        regMatrix(idx_r,numOfFactors+1+tempIdx-1 + tempIdx2) = decisionVar(idxLS1,idx_r)*decisionVar(idxLS2,idx_r)*decisionVar(idxLS3,idx_r);
                                        tempIdx = tempIdx + 1;
                                    end
                                end
                            end
                            
                        end
                        
                        % simple regression <- for path dependent option
                        % we should change it
                        % beta  : regression coefficients
                        beta =zeros(dimensionLS,1);
                        beta = regress(contiValue',regMatrix);
                        regContiValue = regMatrix*beta;
                       
                         %Nominal Regression test
                        regContiValue = regContiValue * 10000;
                                                
%                         exercisedPayoff = regContiValue;
                        exercisedPayoff = contiValue * 10000;
                        % if regContiValue is bigger than exerciseValue
                        % then issue exercise call option
                        for idx_r=1:pathSize
                            if regContiValue(idx_r) >= exerciseValue(idx_r) 
                                exercisedPayoff(idx_r) = exerciseValue(idx_r);
                            end
                        end
                        
                        % we discount the new payoff and 
                        % assign it into the existing callable payoff
                        callable.payoff = model.discountPayoffRisky(currentTime,numMatTime ...
                        ,modelStateT,exercisedPayoff);
                        
                    end
                end

                nonCall.npv = mean(nonCall.payoff);
                out.nonCall = nonCall;
                callable.npv = mean(callable.payoff);
                dfCol.npv = mean(dfCol.payoff);
%                 dummy1.npv = mean(dummy1.npv);
%                 dummy2.npv = mean(dummy2.npv);
                out.callable = callable;
                out.dfCol = dfCol;
%                 out.dummy1 = dummy1;
%                 out.dummy2 = dummy2;
                out.daycountFraction = schedule.dayCountFraction;
                
        end
        
        
        function out = computePriceClosedSimple(pricer,valueDate)
                security = pricer.m_security;
                numMethod = pricer.m_numMethod;
                model = pricer.m_model;
                schedule = security.couponSchedule;
                
                % numMethodTimeSteps Generate
                mcOneTimeStep = numMethod.mcOneTimeStep;
                outInfo = numMethod.numMethodTimeStepsGenerateDW(valueDate,schedule,mcOneTimeStep);
            
                numMethodInfo.timeStepsInfo = outInfo;

                % mc random number init
                %pathSize = 2^16;
                pathSize = numMethod.pathSize;
                
                pathSize = 1;
                
%                 % generate random numbers and modelstates forward
%                 rng('default');
%                 rng(0);
% 
                numOfFactors = model.numOfFactor;
% 
%                 %Z= randn(numOfFactors*outInfo.totalTimeStepsSize,pathSize/2);
%                 Z= randn(pathSize/2,numOfFactors*outInfo.totalTimeStepsSize);
%                 Z=[Z;-Z];
%                 Z=Z';
%                 
%                 
%                 %TimeInversion
%                 sizeZ = size(Z,1);
%                 A = zeros(sizeZ,sizeZ);
%                 for i=1:sizeZ
%                     A(i,sizeZ + 1 -i) = 1;
%                 end
%                 Z = A*Z;
% 
%                 % we check with the c, so we use generated random numbers
% %                 mc_random = importdata('mc_random.xlsx');
% %                 mc_random = importdata('D:\mc_random_number_new_report.txt');
% %                 Z= mc_random;
%                 
                numMethodInfo.pathSize = pathSize;
                numMethodInfo.numOfFactors = numOfFactors;
                numMethodInfo.totalTimeSteps = outInfo.totalTimeSteps;
                numMethodInfo.totalTimeStepsSize = outInfo.totalTimeStepsSize;
%                 numMethodInfo.Z = Z;
% 
% 
% 
%                 %generate model states for the given timeSteps
%                 modelStates = numMethod.generateModelStates(valueDate,schedule,numMethodInfo,model);

                % numeraire info
                numMatTime = numMethodInfo.totalTimeSteps(end);
                dfNumMat = model.DFRisky(numMatTime/365.0);
                
                % induct backward and calculate cashflow
            
                periodCount = schedule.numOfPeriod;
                
                % cashflow columns initialize
                % 1st nonCall
                nonCall.npv = 0;
                nonCall.cashflow_e = zeros(periodCount + 1,1);
                nonCall.cashflow_npv_e = zeros(periodCount + 1,1);
                nonCall.rangeProb_e = zeros(periodCount + 1,1);
                nonCall.df_e = zeros(periodCount + 1,1);
                nonCall.df_c = zeros(periodCount + 1,1);
                
                nonCall.fwdCD = zeros(periodCount + 1,1);
                nonCall.fwdKTB = zeros(periodCount + 1,1);
                nonCall.fwdSpread = zeros(periodCount + 1,1);
                nonCall.fwdMu = zeros(periodCount + 1,1);
                nonCall.fwdSigma = zeros(periodCount + 1,1);

                % debug information, cashflow & npv per path
                nonCall.payoff = zeros(1,pathSize);
                nonCall.payoffStates.cashflow = zeros(periodCount+1,pathSize);
                nonCall.payoffStates.cashflow_npv = zeros(periodCount+1,pathSize);
                nonCall.payoffStates.df = zeros(periodCount + 1,pathSize);
                
                % intermediate variables
                payoffStateA.cashflow = zeros(1, pathSize);
                payoffStateA.cashflow_npv = zeros(1, pathSize);
                payoffStateA.df = zeros(1,pathSize);
                
                % we calculate cashflow backward from maturityDate to valueDate
                % rangeAccrual coupons are added at the startDate of the Period
                % At maturity

                nominal = security.nominal;
                coupon = security.coupon;
                callYN = security.callYN;
                
                currentTime = numMethodInfo.totalTimeSteps(end);
                totalTimeSteps = numMethodInfo.timeStepsInfo.totalTimeSteps;
                totalTimeStepsSize = numMethodInfo.timeStepsInfo.totalTimeStepsSize;
                if totalTimeStepsSize == 0 || (totalTimeStepsSize == 1 && totalTimeSteps(1) == 0) 
                    out.nonCall = nonCall;
                    return;
                end
                
                currentTimeIdx = totalTimeStepsSize;
                
                currentNodeIdx = periodCount + 1;
                
%                 modelStateT = modelStates(numOfFactors*(totalTimeStepsSize-1)+1:numOfFactors*(totalTimeStepsSize-1)+2,:);
                payoffStateA.cashflow = nominal*ones(1,pathSize);
                %process payoff at maturity
                nonCall.payoffStates.cashflow(currentNodeIdx,:) = payoffStateA.cashflow;
                
%                 payoffStateA.cashflow_npv = model.discountPayoff(currentTime,numMatTime ...
%                     ,modelStateT,payoffStateA.cashflow);
%                 
                payoffStateA.cashflow_npv = model.DFRisky(currentTime/365.0)*payoffStateA.cashflow;
                
                nonCall.payoffStates.cashflow_npv(currentNodeIdx,:) = payoffStateA.cashflow_npv ;
                
%                 payoffStateA.df = model.discountPayoff(currentTime,numMatTime ...
%                     ,modelStateT,1.0*ones(1,pathSize));
                payoffStateA.df = model.DFRisky(currentTime/365.0) *ones(1,pathSize);
                    

                nonCall.payoffStates.df(currentNodeIdx,:) = payoffStateA.df;
                nonCall.payoff = nonCall.payoff + payoffStateA.cashflow_npv;
                
                nonCall.cashflow_e(currentNodeIdx) = mean(payoffStateA.cashflow);
                nonCall.cashflow_npv_e(currentNodeIdx) = mean(payoffStateA.cashflow_npv);
                nonCall.df_e(currentNodeIdx) = mean(payoffStateA.df);
                nonCall.df_c(currentNodeIdx) = model.DFRisky(currentTime/365.0);
                
%                 % callable at maturity is the same as noncallable
%                 callable.payoffStates.cashflow(currentNodeIdx,:) = nonCall.payoffStates.cashflow(currentNodeIdx,:);
%                 callable.payoffStates.cashflow_npv(currentNodeIdx,:) = nonCall.payoffStates.cashflow_npv(currentNodeIdx,:);
%                 callable.payoff = callable.payoff + callable.payoffStates.cashflow_npv(currentNodeIdx,:);
%                 
%                 callable.cashflow_e(currentNodeIdx) = nonCall.cashflow_e(currentNodeIdx);
%                 callable.cashflow_npv_e(currentNodeIdx) = nonCall.cashflow_npv_e(currentNodeIdx);
                  
                lastTimeIdx = currentTimeIdx;
                
                
                % flag for inclusive schedule
%                 simpleYN = false;
                simpleYN = 1;
                
                for i=periodCount:-1:1
                    % past schedule neglect
                    if DateDiff(valueDate,schedule.payDates(i)) >= 0, continue;end 
                    startTimeIdx = numMethodInfo.timeStepsInfo.startTimeIdx(i);
                    endTimeIdx = numMethodInfo.timeStepsInfo.endTimeIdx(i);

                    inclusiveYN = numMethodInfo.timeStepsInfo.inclusiveYN(i);
                     %% FromToInduction start

                    %modelsStates induct backward(lastTimeIdx to endTimeIdx)
                    while endTimeIdx < lastTimeIdx
                        lastTimeIdx = lastTimeIdx -1;
                        if lastTimeIdx == endTimeIdx break;end
                    end

                    currentTimeIdx = lastTimeIdx;
                    currentTime = totalTimeSteps(currentTimeIdx);
                    % now we are at endTimeIdx
                    % variables for coupon valuation initialized
                    ralower = security.lowerS;
                    raupper = security.upperS;
                    tenorLong = security.tenorL;
                    tenorShort = security.tenorS;

                    cD = 0; % cumulated day count fraction
                    cP = 0; 
                    dI = 0; % dummy idx for loop
                    dcfC = 0;
                    dcfN = 0;
                    dD = 0;
                    idxL = zeros(1,pathSize);
                    idxS = zeros(1,pathSize);
                    spreadC = zeros(1,pathSize);
                    spreadN = zeros(1,pathSize);
                    probSpread = zeros(1,pathSize);
                    
                    numOfTimeSteps = numMethodInfo.timeStepsInfo.numOfTimeSteps(i);
                    
                    % if inclusive then we use simpleYN rangeProb
                    %if inclusive then we do not evaluate the last step but
                    % use fixing instead 
                    endTimeStep = 1;
                    if inclusiveYN
%                         simpleYN = true;
                        simpleYN = 1;
                        endTimeStep = 2;
                    end
                    
                    for j=numOfTimeSteps:-1:endTimeStep
                        % evaluate dealdescription
                        % last step is omitted
                        currentTime = totalTimeSteps(currentTimeIdx);
                        %dcfC = currentTime/365.0;
                        dcfC = currentTime;
                        
                        if j~= numOfTimeSteps
                            dD = (dcfN-dcfC);
                        end

                        toDate = AddDate(valueDate,currentTime,'day');
                        tpDate = schedule.payDates(i);
                        spreadCOut = model.CMSCMTSpreadDigitalRangeDate(valueDate,toDate,tpDate,tenorLong,tenorShort,ralower,raupper);
                        spreadC = spreadCOut.value;
                        if j== endTimeStep
                            fwdCD = spreadCOut.cms_fwd;
                            fwdKTB = spreadCOut.cmt_fwd;
                            fwdSpread = spreadCOut.fwdSpread;
                            fwdMu        = spreadCOut.mu;
                            fwdSigma     = spreadCOut.sigma;
                        end
                        %modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:numOfFactors*(currentTimeIdx-1)+2,:);
%                         idxL = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorLong,modelStateT);
%                         idxS = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorShort,modelStateT);
%                         spreadC = idxL - idxS;
%                         probSpread = model.SRPr(spreadC,spreadN,ralower,raupper,simpleYN,pathSize)*dD; 

                        if simpleYN
                            probSpread = spreadC*dD;
                        else
                            probSpread = spreadC*dD;
%                             probSpread = 0.5*(spreadC + spreadN)*dD;
                            
                        end

                        
                        cD = cD + dD;
                        cP = cP + probSpread;

                        dcfN = dcfC;
                        spreadN = spreadC;

                        %induct backward to the next time step
                        % one time backward induction
                        lastTimeIdx = currentTimeIdx;

                        if startTimeIdx < lastTimeIdx
                            lastTimeIdx = lastTimeIdx -1;
                            currentTimeIdx = lastTimeIdx;
                        end


                    end

                    %% FromToInduction end
                    %% AtDate evaluation
                    % Adding a new cashflow to the payoff

                    dcf = schedule.dayCountFraction(i);
                    
                    if inclusiveYN
                        % we add valuation date
                        cD = schedule.dayCount(i);
                        rangeDays =  security.rangeDays(i);
                        payoffStateA.cashflow = nominal * coupon * dcf * (rangeDays + cP)/cD;
                    else
                        payoffStateA.cashflow = nominal * coupon * dcf * cP/cD;
                    end
                    % eventDate(i) = startDate(i) therefore there is no induction in modelStates
                    % i.e. currentTimeIdx has not changed
                    % no need to update modelStateT
                    % modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:end,:);
                    % since cashflow's are added at the startDate(i) we need to
                    % multiply cashflow with the df(startDate(i), payDate(i))
                    payTime = DateDiff(schedule.payDates(i),valueDate);
                    
                    %event is at startDate so we need to discount the
                    %cashflow first from paydate to startdate
%                     df_sp = model.discountFactor(currentTime, payTime,numMatTime,modelStateT);

%                     for k=1:pathSize
%                         payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_sp(k);
%                     end
                    
                    % process payoff
                    
                    currentNodeIdx = i;
                    
                    % 1st nonCall column
                    nonCall.payoffStates.cashflow(currentNodeIdx,:) = payoffStateA.cashflow;
                    payoffStateA.cashflow_npv = payoffStateA.cashflow * model.DFRisky(payTime/365.0);
                    
%                     payoffStateA.cashflow_npv = model.discountPayoff(currentTime,numMatTime ...
%                     ,modelStateT,payoffStateA.cashflow);
                    nonCall.payoffStates.cashflow_npv(currentNodeIdx,:) = payoffStateA.cashflow_npv;
                    payoffStateA.df = model.DFRisky(payTime/365.0);
%                     payoffStateA.df = model.discountPayoff(currentTime,numMatTime ...
%                     ,modelStateT,1.0*ones(1,pathSize));
                    nonCall.payoffStates.df(currentNodeIdx,:) = payoffStateA.df;
                    
                    nonCall.payoff = nonCall.payoff + payoffStateA.cashflow_npv;
                    
                    nonCall.cashflow_e(currentNodeIdx) = mean(payoffStateA.cashflow);
                    nonCall.rangeProb_e(currentNodeIdx) = mean(nonCall.cashflow_e(currentNodeIdx)/(nominal * coupon * dcf));
                    nonCall.cashflow_npv_e(currentNodeIdx) = mean(payoffStateA.cashflow_npv);

                    nonCall.df_e(currentNodeIdx) = mean(payoffStateA.df);
                    nonCall.df_c(currentNodeIdx) = model.DFRisky(currentTime/365.0);
                    
                    nonCall.fwdCD(currentNodeIdx) = fwdCD;
                    nonCall.fwdKTB(currentNodeIdx) = fwdKTB;
                    
                    nonCall.fwdSpread(currentNodeIdx) = fwdSpread;
                    nonCall.fwdMu(currentNodeIdx) = fwdMu;
                    nonCall.fwdSigma(currentNodeIdx) = fwdSigma;

                end

                nonCall.npv = mean(nonCall.payoff);
                out.pricerInfo.nonCall = nonCall;
                
%                 callable.npv = mean(callable.payoff);
%                 out.pricerInfo.callable = callable;
                out.daycountFraction = schedule.dayCountFraction;
                
        end
        

        
        
        
        
    end
    
end



In [ ]:
classdef DailyWeeklyRiskyPricer_CDAndKOSPI200RA < H_RootObject
    %Pricer(Security,Model,NumMethod)
    %Prcer is constructed from Security, Model, NumMethod 
    
    properties
        m_security
        m_model
        m_numMethod
        overHedgeYN
        
    end
    
    methods
        function pricer = DailyWeeklyRiskyPricer_CDAndKOSPI200RA(security,model,numMethod,overHedgeYN)
                pricer.m_security = security;
                pricer.m_model = model;
                pricer.m_numMethod = numMethod;
                pricer.overHedgeYN = overHedgeYN;
                
        end
        
        function out = computePriceMCCallableGeneric(pricer,valueDate,pricerOption)
                security = pricer.m_security;
                numMethod = pricer.m_numMethod;
                model = pricer.m_model;
                schedule = security.couponSchedule;
                callSchedule = security.callSchedule;
                
                % numMethodTimeSteps Generate
                mcOneTimeStep = numMethod.mcOneTimeStep;
                outInfo = numMethod.numMethodTimeStepsGenerateDW(valueDate,schedule,mcOneTimeStep);
            
                numMethodInfo.timeStepsInfo = outInfo;

                % mc random number init
                %pathSize = 2^16;
                pathSize = numMethod.pathSize;

                % generate random numbers and modelstates forward
                rng('default');
                rng(0);

                % LGM1F : one factor
                % LGM2F : two factor
                
                numOfFactors = model.numOfFactor;

                %Z= randn(numOfFactors*outInfo.totalTimeStepsSize,pathSize/2);
                Z= randn(pathSize/2,numOfFactors*outInfo.totalTimeStepsSize);
                Z=[Z;-Z];
                Z=Z';
                
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;

                % we check with the c code, so we use generated random numbers
%                 mc_random = importdata('mc_random.xlsx');
%                 mc_random = importdata('D:\mc_random_number_new_report.txt');
%                 Z= mc_random;
                
                numMethodInfo.pathSize = pathSize;
                numMethodInfo.numOfFactors = numOfFactors;
                numMethodInfo.totalTimeSteps = outInfo.totalTimeSteps;
                numMethodInfo.totalTimeStepsSize = outInfo.totalTimeStepsSize;
                numMethodInfo.Z = Z;



                %numMethod generate model states for the given timeSteps
                modelStates = numMethod.generateModelStatesNF(valueDate,schedule,numMethodInfo,model);

                % numeraire info
                numMatTime = numMethodInfo.totalTimeSteps(end);
                days = DateDiff(schedule.payDates(schedule.numOfPeriod),valueDate);
                numMatTime = max(numMatTime, days);
                dfNumMat = model.DFRisky(numMatTime/365.0);
                
                numeraire.numMatTime = numMatTime;
                numeraire.dfNumMat = dfNumMat;
                
                % induct backward and calculate cashflow
            
                periodCount = schedule.numOfPeriod;
                
                % pricer single info initialize
                
                nonCall = PricerInfo(periodCount +1, pathSize);
                callable = PricerInfo(periodCount +1, pathSize);
                dfCol = PricerInfo(periodCount +1, pathSize);
                
                dummy1 = PricerInfo(periodCount +1, pathSize);
                dummy2 = PricerInfo(periodCount +1, pathSize);
                
                
                % intermediate variables
                payoffStateA.cashflow = zeros(1, pathSize);
                payoffStateA.cashflow_npv = zeros(1, pathSize);
                
            
                % we calculate cashflow backward from maturityDate to valueDate
                % rangeAccrual coupons are added at the startDate of the Period
                % At maturity

                nominal = security.nominal;
                coupon = security.coupon;
                callYN = security.callYN;
                
                ralower = security.lower;
                raupper = security.upper;
                
                ralowerS = security.lowerS;
                raupperS = security.upperS;
                
                tenorCD = security.tenor;
                
                tenorLong = security.tenorL;
                tenorShort = security.tenorS;
                
                currentTime = numMethodInfo.totalTimeSteps(end);
                totalTimeStepsSize = numMethodInfo.totalTimeStepsSize;
                currentTimeIdx = totalTimeStepsSize;
                
                currentNodeIdx = periodCount + 1;
                
                modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:numOfFactors*(currentTimeIdx-1)+numOfFactors,:);
                
                payoffStateA.cashflow = nominal*ones(1,pathSize);
                
                payTime = DateDiff(schedule.payDates(schedule.numOfPeriod),valueDate);
                df_ep = model.discountFactorRisky(currentTime, payTime,numMatTime,modelStateT);
                for k=1:pathSize
                    payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                end
                
                %process payoff at maturity
                nonCall.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                
                callable.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                
                %df column
                payoffStateA.cashflow = ones(1,pathSize);
                for k=1:pathSize
                    payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                end
                
                dfCol.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                                 
                %dummy column

                if strcmp(pricerOption,'PSA')
                    payoffStateA.cashflow = model.CMS_PSA_SimpleDate('KRW',valueDate,currentTime,numMatTime, tenorLong,modelStateT);
                    dummy1.ProcessPayoff(model,currentTime,numeraire, ...
                                 modelStateT,payoffStateA.cashflow,currentNodeIdx);
                    
                    payoffStateA.cashflow = model.CMS_PSA_SimpleDate('USD',valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                    dummy2.ProcessPayoff(model,currentTime,numeraire, ...
                                 modelStateT,payoffStateA.cashflow,currentNodeIdx);
                
                else
                    payoffStateA.cashflow = model.Libor_SimpleDate('KRW',valueDate,currentTime,currentTime,numMatTime, tenorLong,modelStateT); 
                    dummy1.ProcessPayoff(model,currentTime,numeraire, ...
                                 modelStateT,payoffStateA.cashflow,currentNodeIdx);
                             
                    payoffStateA.cashflow = model.Libor_SimpleDate('USD',valueDate,currentTime,currentTime,numMatTime, tenorShort,modelStateT); 
                    dummy2.ProcessPayoff(model,currentTime,numeraire, ...
                                 modelStateT,payoffStateA.cashflow,currentNodeIdx);
                    
                end
                

%                 payoffStateA.cashflow = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                
%                 payoffStateA.cashflow = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                
                
                                 
                  

                lastTimeIdx = currentTimeIdx;
                totalTimeSteps = numMethodInfo.timeStepsInfo.totalTimeSteps;
                
                % flag for inclusive schedule
%                 simpleYN = false;
%                 simpleYN = true;
                if pricer.overHedgeYN == true 
                    simpleYN = 2;
                else
                    simpleYN = 1;
                end
                
                for i=periodCount:-1:1
                    % past schedule neglect
                    if DateDiff(valueDate,schedule.payDates(i)) >= 0, continue;end 
                    startTimeIdx = numMethodInfo.timeStepsInfo.startTimeIdx(i);
                    endTimeIdx = numMethodInfo.timeStepsInfo.endTimeIdx(i);

                    inclusiveYN = numMethodInfo.timeStepsInfo.inclusiveYN(i);
                 %% FromToInduction start

                    %modelsStates induct backward(lastTimeIdx to endTimeIdx)
                    while endTimeIdx < lastTimeIdx
                        lastTimeIdx = lastTimeIdx -1;
                        if lastTimeIdx == endTimeIdx break;end
                    end

                    currentTimeIdx = lastTimeIdx;
                    currentTime = totalTimeSteps(currentTimeIdx);
                    % now we are at endTimeIdx
                    % variables for coupon valuation initialized
                    

                    cD = 0; % cumulated day count fraction
                    cP = 0; 
                    dI = 0; % dummy idx for loop
                    dcfC = 0;
                    dcfN = 0;
                    dD = 0;
                    idxCD = zeros(1,pathSize);
                    idxL = zeros(1,pathSize);
                    idxS = zeros(1,pathSize);
                    
                    idxC = zeros(1,pathSize);
                    idxN = zeros(1,pathSize);
                    
                    spreadC = zeros(1,pathSize);
                    spreadN = zeros(1,pathSize);
                    probSpread = zeros(1,pathSize);
                    
                    numOfTimeSteps = numMethodInfo.timeStepsInfo.numOfTimeSteps(i);
                    
                    % if inclusive then we use simpleYN rangeProb
                    %if inclusive then we do not evaluate the last step but
                    % use fixing instead 
                    endTimeStep = 1;
                    if inclusiveYN
%                         simpleYN = true;
                        simpleYN = 1;
                        endTimeStep = 2;
                        if pricer.overHedgeYN == true 
                            simpleYN = 2;
                        end
                    end
                    
                    for j=numOfTimeSteps:-1:endTimeStep
                        % evaluate dealdescription
                        % last step is omitted
                        currentTime = totalTimeSteps(currentTimeIdx);
                        %dcfC = currentTime/365.0;
                        dcfC = currentTime;
                        
                        if j~= numOfTimeSteps
                            dD = (dcfN-dcfC);
                        end

                        modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:numOfFactors*(currentTimeIdx-1)+numOfFactors,:);
                        
                        idxCD = model.Libor_SimpleDate('KRW',valueDate,currentTime,currentTime,numMatTime, tenorCD,modelStateT);
                        idxC = idxCD;
                        
                        if strcmp(pricerOption,'PSA')
                            idxL = model.CMS_PSA_SimpleDate('KRW',valueDate,currentTime,numMatTime, tenorLong,modelStateT);
                            idxS = model.CMS_PSA_SimpleDate('USD',valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                        else
                            idxL = model.Libor_SimpleDate('KRW',valueDate,currentTime,currentTime,numMatTime, tenorLong,modelStateT);
                            idxS = model.Libor_SimpleDate('USD',valueDate,currentTime,currentTime,numMatTime, tenorShort,modelStateT);
                        end
                        spreadC = idxL - idxS;

                        probSpread = model.DRPr(idxC,idxN,ralower,raupper,spreadC,spreadN,ralowerS,raupperS,simpleYN,pathSize)*dD;
                    
%                         probSpread = model.SRPr(spreadC,spreadN,ralower,raupper,simpleYN,pathSize)*dD;
                        cD = cD + dD;
                        cP = cP + probSpread;

                        dcfN = dcfC;
                        
                        idxN = idxC;
                        spreadN = spreadC;

                        %induct backward to the next time step
                        % one time backward induction
                        lastTimeIdx = currentTimeIdx;

                        if startTimeIdx < lastTimeIdx
                            lastTimeIdx = lastTimeIdx -1;
                            currentTimeIdx = lastTimeIdx;
                        end


                    end

                    %% FromToInduction end
        % if the current period is inclusive, then the start date is not evaluated in the above for loop.
		% instead the fixing will be treated separately below.
		% however current_time_idx is updated at the end of the for loop.
		% so we need to update model_states accordingly.
		% if model_states were updated previously then the following additionaly step doesn't affect the result anyway.
                    currentTime = totalTimeSteps(currentTimeIdx);
                    modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:numOfFactors*(currentTimeIdx-1)+numOfFactors,:);
                    
                    %% AtDate evaluation
                    % Adding a new cashflow to the payoff

                    dcf = schedule.dayCountFraction(i);
                    
                    if inclusiveYN
                        % we add valuation date
                        cD = schedule.dayCount(i);
                        rangeDays =  security.rangeDays(i);
                        payoffStateA.cashflow = nominal * coupon * dcf * (rangeDays + cP)/cD;
                    else
                        payoffStateA.cashflow = nominal * coupon * dcf * cP/cD;
                    end
                    % eventDate(i) = startDate(i) therefore there is no induction in modelStates
                    % i.e. currentTimeIdx has not changed
                    % no need to update modelStateT
                    % modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:end,:);
                    % since cashflow's are added at the startDate(i) we need to
                    % multiply cashflow with the df(startDate(i), payDate(i))
                    payTime = DateDiff(schedule.payDates(i),valueDate);
                    
                    %event is at startDate so we need to discount the
                    %cashflow first from paydate to startdate
                    df_ep = model.discountFactorRisky(currentTime, payTime,numMatTime,modelStateT);

                    for k=1:pathSize
                        payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                    end
                    
                    % process payoff
                    
                    currentNodeIdx = i;
                    
                    nonCall.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                
                    callable.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                    
                    %df column
                    payoffStateA.cashflow = ones(1,pathSize);
                    for k=1:pathSize
                        payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                    end
                    
                    dfCol.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                                 
                    %dummy column
                    if strcmp(pricerOption,'PSA')
                        payoffStateA.cashflow = model.CMS_PSA_SimpleDate('KRW',valueDate,currentTime,numMatTime, tenorLong,modelStateT);
                        dummy1.ProcessPayoff(model,currentTime,numeraire, ...
                                         modelStateT,payoffStateA.cashflow,currentNodeIdx);



                        payoffStateA.cashflow = model.CMS_PSA_SimpleDate('USD',valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                        dummy2.ProcessPayoff(model,currentTime,numeraire, ...
                                         modelStateT,payoffStateA.cashflow,currentNodeIdx);
                    
                    else
                        payoffStateA.cashflow = model.Libor_SimpleDate('KRW',valueDate,currentTime,currentTime,numMatTime, tenorLong,modelStateT); 
                        dummy1.ProcessPayoff(model,currentTime,numeraire, ...
                                 modelStateT,payoffStateA.cashflow,currentNodeIdx);
                        
                        payoffStateA.cashflow = model.Libor_SimpleDate('USD',valueDate,currentTime,currentTime,numMatTime, tenorShort,modelStateT); 
                        dummy2.ProcessPayoff(model,currentTime,numeraire, ...
                                 modelStateT,payoffStateA.cashflow,currentNodeIdx);
                        
                    end

                                 
                    
                    
                    
                    % if exercise is possible
                    if i == 1
                        dummaaa = 1;
                    end
                    
                    % call event happens  at the start of the period
                    % at call date, by default, call option is expired
                    
                    if callYN(i) && (DateDiff(valueDate,callSchedule.resetDates(i)) < 0)
                        % 1.undiscount callable.payoff
                        % 2.build the state variables matrix
                        % 3.regress 1's with the given state variables
                        % (model states 1,2)
                        % 4. exercise per each path in
                        % discount the new payoff and assign into callable
                        % column
                        
                        %1 undiscount callable.payoff and get the
                        %contiValue
                       
                        %exercise value
                        exerciseValue = security.nominal*ones(1,pathSize);
                        payTime = DateDiff(callSchedule.payDates(i),valueDate);
                        %event is at startDate so we need to discount the
                        %cashflow first from paydate to startdate
                        df_ep = model.discountFactorRisky(currentTime, payTime,numMatTime,modelStateT);
                        for k=1:pathSize
                            exerciseValue(k) = exerciseValue(k) * df_ep(k);
                        end
                        
                        contiValue = zeros(1,pathSize);
                        onePayoff = ones(1,pathSize);
                        onePayoffD = model.discountPayoffRisky(currentTime,numMatTime ...
                        ,modelStateT,onePayoff);
                        contiValue = callable.payoff./onePayoffD;
                        
                        %Nominal Regression test
                        contiValue = contiValue/10000;
                        
                        % build the state variabls matrix
                        dimensionLS = (numOfFactors + 1)*(numOfFactors + 2)/2;
                        
                        decisionVar = zeros(numOfFactors,size(modelStateT,2));
                        
                        for idxLS =1:numOfFactors
                            decisionVar(idxLS,:) = modelStateT(idxLS,:); 
                        end
%                         decisionVar1 = modelStateT(1,:);
%                         decisionVar2 = modelStateT(2,:);
                        % we use upto 2nd order polynomial as regression
                        % equatioin
                        
                        regMatrix = ones(pathSize,dimensionLS);
                        
                        for idx_r=1:pathSize
                            for idxLS=1:numOfFactors
                                regMatrix(idx_r,idxLS+1) = decisionVar(idxLS,idx_r);
                            end
                            
                            tempIdx = 1;
                            for idxLS1=1:numOfFactors
                                for idxLS2=idxLS1:numOfFactors
                                    regMatrix(idx_r,numOfFactors+1+tempIdx) = decisionVar(idxLS1,idx_r)*decisionVar(idxLS2,idx_r);
                                    tempIdx = tempIdx + 1;
                                end
                            end
                            
                        end
                        
                        % simple regression <- for path dependent option
                        % we should change it
                        % beta  : regression coefficients
                        beta =zeros(dimensionLS,1);
                        beta = regress(contiValue',regMatrix);
                        regContiValue = regMatrix*beta;
                       
                         %Nominal Regression test
                        regContiValue = regContiValue * 10000;
                        
                        exercisedPayoff = regContiValue;
                        % if regContiValue is bigger than exerciseValue
                        % then issue exercise call option
                        for idx_r=1:pathSize
                            if regContiValue(idx_r) >= exerciseValue(idx_r) 
                                exercisedPayoff(idx_r) = exerciseValue(idx_r);
                            end
                        end
                        
                        % we discount the new payoff and 
                        % assign it into the existing callable payoff
                        callable.payoff = model.discountPayoffRisky(currentTime,numMatTime ...
                        ,modelStateT,exercisedPayoff);
                        
                    end
                end

                nonCall.npv = mean(nonCall.payoff);
                out.nonCall = nonCall;
                callable.npv = mean(callable.payoff);
                dfCol.npv = mean(dfCol.payoff);
                dummy1.npv = mean(dummy1.npv);
                dummy2.npv = mean(dummy2.npv);
                out.callable = callable;
                out.dfCol = dfCol;
                out.dummy1 = dummy1;
                out.dummy2 = dummy2;
                out.daycountFraction = schedule.dayCountFraction;
                
        end
        
        function out = computePriceMCCallableGenericReg2(pricer,valueDate,pricerOption)
                security = pricer.m_security;
                numMethod = pricer.m_numMethod;
                model = pricer.m_model;
                schedule = security.couponSchedule;
                callSchedule = security.callSchedule;
                
                % numMethodTimeSteps Generate
                mcOneTimeStep = numMethod.mcOneTimeStep;
                outInfo = numMethod.numMethodTimeStepsGenerateDW(valueDate,schedule,mcOneTimeStep);
            
                numMethodInfo.timeStepsInfo = outInfo;

                % mc random number init
                %pathSize = 2^16;
                pathSize = numMethod.pathSize;

                % generate random numbers and modelstates forward
                rng('default');
                rng(0);

                % LGM1F : one factor
                % LGM2F : two factor
                
                numOfFactors = model.numOfFactor;

                %Z= randn(numOfFactors*outInfo.totalTimeStepsSize,pathSize/2);
                Z= randn(pathSize/2,numOfFactors*outInfo.totalTimeStepsSize);
                Z=[Z;-Z];
                Z=Z';
                
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;

                % we check with the c code, so we use generated random numbers
%                 mc_random = importdata('mc_random.xlsx');
%                 mc_random = importdata('D:\mc_random_number_new_report.txt');
%                 Z= mc_random;
                
                numMethodInfo.pathSize = pathSize;
                numMethodInfo.numOfFactors = numOfFactors;
                numMethodInfo.totalTimeSteps = outInfo.totalTimeSteps;
                numMethodInfo.totalTimeStepsSize = outInfo.totalTimeStepsSize;
                numMethodInfo.Z = Z;


                % numeraire info
                numMatTime = numMethodInfo.totalTimeSteps(end);
                days = DateDiff(schedule.payDates(schedule.numOfPeriod),valueDate);
                numMatTime = max(numMatTime, days);
                dfNumMat = model.DFRisky(numMatTime/365.0);
                
                numeraire.numMatTime = numMatTime;
                numeraire.dfNumMat = dfNumMat;
                %numMethod generate model states for the given timeSteps
%                 modelStates = numMethod.generateModelStatesNF(valueDate,schedule,numMethodInfo,model);
                modelStates = numMethod.generateModelStatesNFHyb(valueDate,schedule,numMethodInfo,model);

                
                
                % induct backward and calculate cashflow
            
                periodCount = schedule.numOfPeriod;
                
                % pricer single info initialize
                
                nonCall = PricerInfo(periodCount +1, pathSize);
                callable = PricerInfo(periodCount +1, pathSize);
                dfCol = PricerInfo(periodCount +1, pathSize);
                
                dummy1 = PricerInfo(periodCount +1, pathSize);
                dummy2 = PricerInfo(periodCount +1, pathSize);
                
                
                % intermediate variables
                payoffStateA.cashflow = zeros(1, pathSize);
                payoffStateA.cashflow_npv = zeros(1, pathSize);
                
                payoffStateB.cashflow = zeros(1, pathSize);
                payoffStateB.cashflow_npv = zeros(1, pathSize);
                
                % we calculate cashflow backward from maturityDate to valueDate
                % rangeAccrual coupons are added at the startDate of the Period
                % At maturity

                hedgeBuffer = security.hedgeBuffer;
                
                nominal = security.nominal;
                coupon = security.coupon;
                callYN = security.callYN;
                
                ralower = security.lower;
                raupper = security.upper;
                
                ralowerS = security.lowerS;
                raupperS = security.upperS;
                
                tenorCD = security.tenor;
                
                tenorLong = security.tenorL;
                tenorShort = security.tenorS;
                
                currentTime = numMethodInfo.totalTimeSteps(end);
                totalTimeStepsSize = numMethodInfo.totalTimeStepsSize;
                currentTimeIdx = totalTimeStepsSize;
                
                currentNodeIdx = periodCount + 1;
                
                modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:numOfFactors*(currentTimeIdx-1)+numOfFactors,:);
                
                payoffStateA.cashflow = nominal*ones(1,pathSize);
                
                payTime = DateDiff(schedule.payDates(schedule.numOfPeriod),valueDate);
                df_ep = model.discountFactorRisky(currentTime, payTime,numMatTime,modelStateT);
                for k=1:pathSize
                    payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                end
                
                %process payoff at maturity
                nonCall.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                
                callable.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                
                %df column
                payoffStateA.cashflow = ones(1,pathSize);
                for k=1:pathSize
                    payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                end
                
                dfCol.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                                 
                %dummy column

                if strcmp(pricerOption,'PSA')
                    payoffStateA.cashflow = model.CMS_PSA_SimpleDate('KRW',valueDate,currentTime,numMatTime, tenorLong,modelStateT);
                    dummy1.ProcessPayoff(model,currentTime,numeraire, ...
                                 modelStateT,payoffStateA.cashflow,currentNodeIdx);
                    
                    payoffStateA.cashflow = model.CMS_PSA_SimpleDate('USD',valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                    dummy2.ProcessPayoff(model,currentTime,numeraire, ...
                                 modelStateT,payoffStateA.cashflow,currentNodeIdx);
                
                else
                    payoffStateA.cashflow = model.Libor_SimpleDate('KRW',valueDate,currentTime,currentTime,numMatTime, tenorLong,modelStateT); 
                    dummy1.ProcessPayoff(model,currentTime,numeraire, ...
                                 modelStateT,payoffStateA.cashflow,currentNodeIdx);
                             
                    payoffStateB.cashflow = model.EQSpotMC('KOSPI200',valueDate,currentTime,numMatTime,modelStateT); 
                    dummy2.ProcessPayoff(model,currentTime,numeraire, ...
                                 modelStateT,payoffStateA.cashflow,currentNodeIdx);
                    
                end
                

%                 payoffStateA.cashflow = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                
%                 payoffStateA.cashflow = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                
                
                                 
                  

                lastTimeIdx = currentTimeIdx;
                totalTimeSteps = numMethodInfo.timeStepsInfo.totalTimeSteps;
                
                % flag for inclusive schedule
%                 simpleYN = false;
%                 simpleYN = true;
                if pricer.overHedgeYN == true 
                    simpleYN = 2;
                else
                    simpleYN = 1;
                end
                
                for i=periodCount:-1:1
                    % past schedule neglect
                    if DateDiff(valueDate,schedule.payDates(i)) >= 0, continue;end 
                    startTimeIdx = numMethodInfo.timeStepsInfo.startTimeIdx(i);
                    endTimeIdx = numMethodInfo.timeStepsInfo.endTimeIdx(i);

                    inclusiveYN = numMethodInfo.timeStepsInfo.inclusiveYN(i);
                 %% FromToInduction start

                    %modelsStates induct backward(lastTimeIdx to endTimeIdx)
                    while endTimeIdx < lastTimeIdx
                        lastTimeIdx = lastTimeIdx -1;
                        if lastTimeIdx == endTimeIdx break;end
                    end

                    currentTimeIdx = lastTimeIdx;
                    currentTime = totalTimeSteps(currentTimeIdx);
                    % now we are at endTimeIdx
                    % variables for coupon valuation initialized
                    

                    cD = 0; % cumulated day count fraction
                    cP = 0; 
                    dI = 0; % dummy idx for loop
                    dcfC = 0;
                    dcfN = 0;
                    dD = 0;
                    idxCD = zeros(1,pathSize);
                    idxL = zeros(1,pathSize);
                    idxS = zeros(1,pathSize);
                    
                    idxC = zeros(1,pathSize);
                    idxN = zeros(1,pathSize);
                    
                    spreadC = zeros(1,pathSize);
                    spreadN = zeros(1,pathSize);
                    probSpread = zeros(1,pathSize);
                    
                    numOfTimeSteps = numMethodInfo.timeStepsInfo.numOfTimeSteps(i);
                    
                    % if inclusive then we use simpleYN rangeProb
                    %if inclusive then we do not evaluate the last step but
                    % use fixing instead 
                    endTimeStep = 1;
                    if inclusiveYN
%                         simpleYN = true;
                        simpleYN = 1;
                        endTimeStep = 2;
                        if pricer.overHedgeYN == true 
                            simpleYN = 2;
                        end
                    end
                    
                    for j=numOfTimeSteps:-1:endTimeStep
                        % evaluate dealdescription
                        % last step is omitted
                        currentTime = totalTimeSteps(currentTimeIdx);
                        %dcfC = currentTime/365.0;
                        dcfC = currentTime;
                        
                        if j~= numOfTimeSteps
                            dD = (dcfN-dcfC);
                        end

                        modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:numOfFactors*(currentTimeIdx-1)+numOfFactors,:);
                        
                        idxCD = model.Libor_SimpleDate('KRW',valueDate,currentTime,currentTime,numMatTime, tenorCD,modelStateT);
                        idxC = idxCD;
                        
                        if strcmp(pricerOption,'PSA')
                            idxL = model.CMS_PSA_SimpleDate('KRW',valueDate,currentTime,numMatTime, tenorLong,modelStateT);
                            idxS = model.CMS_PSA_SimpleDate('USD',valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                        else
                            idxL = model.EQSpotMC('KOSPI200',valueDate,currentTime,numMatTime,modelStateT); 
%                             idxL = model.Libor_SimpleDate('KRW',valueDate,currentTime,currentTime,numMatTime, tenorLong,modelStateT);
%                             idxS = model.Libor_SimpleDate('USD',valueDate,currentTime,currentTime,numMatTime, tenorShort,modelStateT);
                        end
                        spreadC = idxL;
%                         spreadC = idxL - idxS;

                        probSpread = model.DRPr(idxC,idxN,ralower,raupper,spreadC,spreadN,80,120,simpleYN,hedgeBuffer,pathSize)*dD;
                    
%                         probSpread = model.SRPr(spreadC,spreadN,ralower,raupper,simpleYN,pathSize)*dD;
                        cD = cD + dD;
                        cP = cP + probSpread;

                        dcfN = dcfC;
                        
                        idxN = idxC;
                        spreadN = spreadC;

                        %induct backward to the next time step
                        % one time backward induction
                        lastTimeIdx = currentTimeIdx;

                        if startTimeIdx < lastTimeIdx
                            lastTimeIdx = lastTimeIdx -1;
                            currentTimeIdx = lastTimeIdx;
                        end


                    end

                    %% FromToInduction end
        % if the current period is inclusive, then the start date is not evaluated in the above for loop.
		% instead the fixing will be treated separately below.
		% however current_time_idx is updated at the end of the for loop.
		% so we need to update model_states accordingly.
		% if model_states were updated previously then the following additionaly step doesn't affect the result anyway.
                    currentTime = totalTimeSteps(currentTimeIdx);
                    modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:numOfFactors*(currentTimeIdx-1)+numOfFactors,:);
                    
                    %% AtDate evaluation
                    % Adding a new cashflow to the payoff

                    dcf = schedule.dayCountFraction(i);
                    
                    if inclusiveYN
                        % we add valuation date
                        cD = schedule.dayCount(i);
                        rangeDays =  security.rangeDays(i);
                        payoffStateA.cashflow = nominal * coupon * dcf * (rangeDays + cP)/cD;
                    else
                        payoffStateA.cashflow = nominal * coupon * dcf * cP/cD;
                    end
                    % eventDate(i) = startDate(i) therefore there is no induction in modelStates
                    % i.e. currentTimeIdx has not changed
                    % no need to update modelStateT
                    % modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:end,:);
                    % since cashflow's are added at the startDate(i) we need to
                    % multiply cashflow with the df(startDate(i), payDate(i))
                    payTime = DateDiff(schedule.payDates(i),valueDate);
                    
                    %event is at startDate so we need to discount the
                    %cashflow first from paydate to startdate
                    df_ep = model.discountFactorRisky(currentTime, payTime,numMatTime,modelStateT);

                    for k=1:pathSize
                        payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                    end
                    
                    % process payoff
                    
                    currentNodeIdx = i;
                    
                    nonCall.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                
                    callable.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                    
                    %df column
                    payoffStateA.cashflow = ones(1,pathSize);
                    for k=1:pathSize
                        payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                    end
                    
                    dfCol.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                                 
                    %dummy column
                    if strcmp(pricerOption,'PSA')
                        payoffStateA.cashflow = model.CMS_PSA_SimpleDate('KRW',valueDate,currentTime,numMatTime, tenorLong,modelStateT);
                        dummy1.ProcessPayoff(model,currentTime,numeraire, ...
                                         modelStateT,payoffStateA.cashflow,currentNodeIdx);



                        payoffStateA.cashflow = model.CMS_PSA_SimpleDate('USD',valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                        dummy2.ProcessPayoff(model,currentTime,numeraire, ...
                                         modelStateT,payoffStateA.cashflow,currentNodeIdx);
                    
                    else
                        payoffStateA.cashflow = model.Libor_SimpleDate('KRW',valueDate,currentTime,currentTime,numMatTime, tenorLong,modelStateT); 
                        dummy1.ProcessPayoff(model,currentTime,numeraire, ...
                                 modelStateT,payoffStateA.cashflow,currentNodeIdx);
                        
%                         payoffStateA.cashflow = model.Libor_SimpleDate('USD',valueDate,currentTime,currentTime,numMatTime, tenorShort,modelStateT); 
%                         dummy2.ProcessPayoff(model,currentTime,numeraire, ...
%                                  modelStateT,payoffStateA.cashflow,currentNodeIdx);
                        
                    end

                                 
                    
                    
                    
                    % if exercise is possible
                    if i == 1
                        dummaaa = 1;
                    end
                    
                    % call event happens  at the start of the period
                    % at call date, by default, call option is expired
                    
                    if callYN(i) && (DateDiff(valueDate,callSchedule.resetDates(i)) < 0)
                        % 1.undiscount callable.payoff
                        % 2.build the state variables matrix
                        % 3.regress 1's with the given state variables
                        % (model states 1,2)
                        % 4. exercise per each path in
                        % discount the new payoff and assign into callable
                        % column
                        
                        %1 undiscount callable.payoff and get the
                        %contiValue
                       
                        %exercise value
                        exerciseValue = security.nominal*ones(1,pathSize);
                        payTime = DateDiff(callSchedule.payDates(i),valueDate);
                        %event is at startDate so we need to discount the
                        %cashflow first from paydate to startdate
                        df_ep = model.discountFactorRisky(currentTime, payTime,numMatTime,modelStateT);
                        for k=1:pathSize
                            exerciseValue(k) = exerciseValue(k) * df_ep(k);
                        end
                        
                        contiValue = zeros(1,pathSize);
                        onePayoff = ones(1,pathSize);
                        onePayoffD = model.discountPayoffRisky(currentTime,numMatTime ...
                        ,modelStateT,onePayoff);
                        contiValue = callable.payoff./onePayoffD;
                        
                        %Nominal Regression test
                        contiValue = contiValue/10000;
                        
                        % build the state variabls matrix
                        %zeroth order
                        dimensionLS = 1;
                        
                        %1st order
                        for idxLS =1:numOfFactors
                            dimensionLS =  dimensionLS +1; 
                        end
                        
                        %2nd order
                        for idxLS1=1:numOfFactors
                            for idxLS2=idxLS1:numOfFactors
                                dimensionLS = dimensionLS + 1;
                            end
                        end
                        
                        %3rd order
                        for idxLS1=1:numOfFactors
                            for idxLS2=idxLS1:numOfFactors
                                for idxLS3=idxLS2:numOfFactors
                                    dimensionLS = dimensionLS + 1;
                                end
                            end
                        end
                        
%                         dimensionLS = (numOfFactors + 1)*(numOfFactors + 2)/2;
                        
                        decisionVar = zeros(numOfFactors,size(modelStateT,2));
                        
                        for idxLS =1:numOfFactors
                            decisionVar(idxLS,:) = modelStateT(idxLS,:); 
                        end
%                         decisionVar1 = modelStateT(1,:);
%                         decisionVar2 = modelStateT(2,:);
                        % we use upto 2nd order polynomial as regression
                        % equatioin
                        
                        regMatrix = ones(pathSize,dimensionLS);
                        
                        for idx_r=1:pathSize
                            for idxLS=1:numOfFactors
                                regMatrix(idx_r,idxLS+1) = decisionVar(idxLS,idx_r);
                            end
                            
                            tempIdx = 1;
                            for idxLS1=1:numOfFactors
                                for idxLS2=idxLS1:numOfFactors
                                    regMatrix(idx_r,numOfFactors+1+tempIdx) = decisionVar(idxLS1,idx_r)*decisionVar(idxLS2,idx_r);
                                    tempIdx = tempIdx + 1;
                                end
                            end
                            
                            tempIdx2 = 1;
                            for idxLS1=1:numOfFactors
                                for idxLS2=idxLS1:numOfFactors
                                    for idxLS3=idxLS2:numOfFactors
                                        regMatrix(idx_r,numOfFactors+1+tempIdx-1 + tempIdx2) = decisionVar(idxLS1,idx_r)*decisionVar(idxLS2,idx_r)*decisionVar(idxLS3,idx_r);
                                        tempIdx = tempIdx + 1;
                                    end
                                end
                            end
                            
                        end
                        
                        % simple regression <- for path dependent option
                        % we should change it
                        % beta  : regression coefficients
                        beta =zeros(dimensionLS,1);
                        beta = regress(contiValue',regMatrix);
                        regContiValue = regMatrix*beta;
                       
                         %Nominal Regression test
                        regContiValue = regContiValue * 10000;
                                                
%                         exercisedPayoff = regContiValue;
                        exercisedPayoff = contiValue * 10000;
                        % if regContiValue is bigger than exerciseValue
                        % then issue exercise call option
                        for idx_r=1:pathSize
                            if regContiValue(idx_r) >= exerciseValue(idx_r) 
                                exercisedPayoff(idx_r) = exerciseValue(idx_r);
                            end
                        end
                        
                        % we discount the new payoff and 
                        % assign it into the existing callable payoff
                        callable.payoff = model.discountPayoffRisky(currentTime,numMatTime ...
                        ,modelStateT,exercisedPayoff);
                        
                    end
                end

                nonCall.npv = mean(nonCall.payoff);
                out.nonCall = nonCall;
                callable.npv = mean(callable.payoff);
                dfCol.npv = mean(dfCol.payoff);
                dummy1.npv = mean(dummy1.npv);
                dummy2.npv = mean(dummy2.npv);
                out.callable = callable;
                out.dfCol = dfCol;
                out.dummy1 = dummy1;
                out.dummy2 = dummy2;
                out.daycountFraction = schedule.dayCountFraction;
                
        end
        
        
        function out = computePriceClosedSimple(pricer,valueDate)
                security = pricer.m_security;
                numMethod = pricer.m_numMethod;
                model = pricer.m_model;
                schedule = security.couponSchedule;
                
                % numMethodTimeSteps Generate
                mcOneTimeStep = numMethod.mcOneTimeStep;
                outInfo = numMethod.numMethodTimeStepsGenerateDW(valueDate,schedule,mcOneTimeStep);
            
                numMethodInfo.timeStepsInfo = outInfo;

                % mc random number init
                %pathSize = 2^16;
                pathSize = numMethod.pathSize;
                
                pathSize = 1;
                
%                 % generate random numbers and modelstates forward
%                 rng('default');
%                 rng(0);
% 
                numOfFactors = model.numOfFactor;
% 
%                 %Z= randn(numOfFactors*outInfo.totalTimeStepsSize,pathSize/2);
%                 Z= randn(pathSize/2,numOfFactors*outInfo.totalTimeStepsSize);
%                 Z=[Z;-Z];
%                 Z=Z';
%                 
%                 
%                 %TimeInversion
%                 sizeZ = size(Z,1);
%                 A = zeros(sizeZ,sizeZ);
%                 for i=1:sizeZ
%                     A(i,sizeZ + 1 -i) = 1;
%                 end
%                 Z = A*Z;
% 
%                 % we check with the c, so we use generated random numbers
% %                 mc_random = importdata('mc_random.xlsx');
% %                 mc_random = importdata('D:\mc_random_number_new_report.txt');
% %                 Z= mc_random;
%                 
                numMethodInfo.pathSize = pathSize;
                numMethodInfo.numOfFactors = numOfFactors;
                numMethodInfo.totalTimeSteps = outInfo.totalTimeSteps;
                numMethodInfo.totalTimeStepsSize = outInfo.totalTimeStepsSize;
%                 numMethodInfo.Z = Z;
% 
% 
% 
%                 %generate model states for the given timeSteps
%                 modelStates = numMethod.generateModelStates(valueDate,schedule,numMethodInfo,model);

                % numeraire info
                numMatTime = numMethodInfo.totalTimeSteps(end);
                dfNumMat = model.DFRisky(numMatTime/365.0);
                
                % induct backward and calculate cashflow
            
                periodCount = schedule.numOfPeriod;
                
                % cashflow columns initialize
                % 1st nonCall
                nonCall.npv = 0;
                nonCall.cashflow_e = zeros(periodCount + 1,1);
                nonCall.cashflow_npv_e = zeros(periodCount + 1,1);
                nonCall.rangeProb_e = zeros(periodCount + 1,1);
                nonCall.df_e = zeros(periodCount + 1,1);
                nonCall.df_c = zeros(periodCount + 1,1);
                
                nonCall.fwdCD = zeros(periodCount + 1,1);
                nonCall.fwdKTB = zeros(periodCount + 1,1);
                nonCall.fwdSpread = zeros(periodCount + 1,1);
                nonCall.fwdMu = zeros(periodCount + 1,1);
                nonCall.fwdSigma = zeros(periodCount + 1,1);

                % debug information, cashflow & npv per path
                nonCall.payoff = zeros(1,pathSize);
                nonCall.payoffStates.cashflow = zeros(periodCount+1,pathSize);
                nonCall.payoffStates.cashflow_npv = zeros(periodCount+1,pathSize);
                nonCall.payoffStates.df = zeros(periodCount + 1,pathSize);
                
                % intermediate variables
                payoffStateA.cashflow = zeros(1, pathSize);
                payoffStateA.cashflow_npv = zeros(1, pathSize);
                payoffStateA.df = zeros(1,pathSize);
                
                % we calculate cashflow backward from maturityDate to valueDate
                % rangeAccrual coupons are added at the startDate of the Period
                % At maturity

                nominal = security.nominal;
                coupon = security.coupon;
                callYN = security.callYN;
                
                currentTime = numMethodInfo.totalTimeSteps(end);
                totalTimeSteps = numMethodInfo.timeStepsInfo.totalTimeSteps;
                totalTimeStepsSize = numMethodInfo.timeStepsInfo.totalTimeStepsSize;
                if totalTimeStepsSize == 0 || (totalTimeStepsSize == 1 && totalTimeSteps(1) == 0) 
                    out.nonCall = nonCall;
                    return;
                end
                
                currentTimeIdx = totalTimeStepsSize;
                
                currentNodeIdx = periodCount + 1;
                
%                 modelStateT = modelStates(numOfFactors*(totalTimeStepsSize-1)+1:numOfFactors*(totalTimeStepsSize-1)+2,:);
                payoffStateA.cashflow = nominal*ones(1,pathSize);
                %process payoff at maturity
                nonCall.payoffStates.cashflow(currentNodeIdx,:) = payoffStateA.cashflow;
                
%                 payoffStateA.cashflow_npv = model.discountPayoff(currentTime,numMatTime ...
%                     ,modelStateT,payoffStateA.cashflow);
%                 
                payoffStateA.cashflow_npv = model.DFRisky(currentTime/365.0)*payoffStateA.cashflow;
                
                nonCall.payoffStates.cashflow_npv(currentNodeIdx,:) = payoffStateA.cashflow_npv ;
                
%                 payoffStateA.df = model.discountPayoff(currentTime,numMatTime ...
%                     ,modelStateT,1.0*ones(1,pathSize));
                payoffStateA.df = model.DFRisky(currentTime/365.0) *ones(1,pathSize);
                    

                nonCall.payoffStates.df(currentNodeIdx,:) = payoffStateA.df;
                nonCall.payoff = nonCall.payoff + payoffStateA.cashflow_npv;
                
                nonCall.cashflow_e(currentNodeIdx) = mean(payoffStateA.cashflow);
                nonCall.cashflow_npv_e(currentNodeIdx) = mean(payoffStateA.cashflow_npv);
                nonCall.df_e(currentNodeIdx) = mean(payoffStateA.df);
                nonCall.df_c(currentNodeIdx) = model.DFRisky(currentTime/365.0);
                
%                 % callable at maturity is the same as noncallable
%                 callable.payoffStates.cashflow(currentNodeIdx,:) = nonCall.payoffStates.cashflow(currentNodeIdx,:);
%                 callable.payoffStates.cashflow_npv(currentNodeIdx,:) = nonCall.payoffStates.cashflow_npv(currentNodeIdx,:);
%                 callable.payoff = callable.payoff + callable.payoffStates.cashflow_npv(currentNodeIdx,:);
%                 
%                 callable.cashflow_e(currentNodeIdx) = nonCall.cashflow_e(currentNodeIdx);
%                 callable.cashflow_npv_e(currentNodeIdx) = nonCall.cashflow_npv_e(currentNodeIdx);
                  
                lastTimeIdx = currentTimeIdx;
                
                
                % flag for inclusive schedule
%                 simpleYN = false;
                simpleYN = 1;
                
                for i=periodCount:-1:1
                    % past schedule neglect
                    if DateDiff(valueDate,schedule.payDates(i)) >= 0, continue;end 
                    startTimeIdx = numMethodInfo.timeStepsInfo.startTimeIdx(i);
                    endTimeIdx = numMethodInfo.timeStepsInfo.endTimeIdx(i);

                    inclusiveYN = numMethodInfo.timeStepsInfo.inclusiveYN(i);
                     %% FromToInduction start

                    %modelsStates induct backward(lastTimeIdx to endTimeIdx)
                    while endTimeIdx < lastTimeIdx
                        lastTimeIdx = lastTimeIdx -1;
                        if lastTimeIdx == endTimeIdx break;end
                    end

                    currentTimeIdx = lastTimeIdx;
                    currentTime = totalTimeSteps(currentTimeIdx);
                    % now we are at endTimeIdx
                    % variables for coupon valuation initialized
                    ralower = security.lowerS;
                    raupper = security.upperS;
                    tenorLong = security.tenorL;
                    tenorShort = security.tenorS;

                    cD = 0; % cumulated day count fraction
                    cP = 0; 
                    dI = 0; % dummy idx for loop
                    dcfC = 0;
                    dcfN = 0;
                    dD = 0;
                    idxL = zeros(1,pathSize);
                    idxS = zeros(1,pathSize);
                    spreadC = zeros(1,pathSize);
                    spreadN = zeros(1,pathSize);
                    probSpread = zeros(1,pathSize);
                    
                    numOfTimeSteps = numMethodInfo.timeStepsInfo.numOfTimeSteps(i);
                    
                    % if inclusive then we use simpleYN rangeProb
                    %if inclusive then we do not evaluate the last step but
                    % use fixing instead 
                    endTimeStep = 1;
                    if inclusiveYN
%                         simpleYN = true;
                        simpleYN = 1;
                        endTimeStep = 2;
                    end
                    
                    for j=numOfTimeSteps:-1:endTimeStep
                        % evaluate dealdescription
                        % last step is omitted
                        currentTime = totalTimeSteps(currentTimeIdx);
                        %dcfC = currentTime/365.0;
                        dcfC = currentTime;
                        
                        if j~= numOfTimeSteps
                            dD = (dcfN-dcfC);
                        end

                        toDate = AddDate(valueDate,currentTime,'day');
                        tpDate = schedule.payDates(i);
                        spreadCOut = model.CMSCMTSpreadDigitalRangeDate(valueDate,toDate,tpDate,tenorLong,tenorShort,ralower,raupper);
                        spreadC = spreadCOut.value;
                        if j== endTimeStep
                            fwdCD = spreadCOut.cms_fwd;
                            fwdKTB = spreadCOut.cmt_fwd;
                            fwdSpread = spreadCOut.fwdSpread;
                            fwdMu        = spreadCOut.mu;
                            fwdSigma     = spreadCOut.sigma;
                        end
                        %modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:numOfFactors*(currentTimeIdx-1)+2,:);
%                         idxL = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorLong,modelStateT);
%                         idxS = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorShort,modelStateT);
%                         spreadC = idxL - idxS;
%                         probSpread = model.SRPr(spreadC,spreadN,ralower,raupper,simpleYN,pathSize)*dD; 

                        if simpleYN
                            probSpread = spreadC*dD;
                        else
                            probSpread = spreadC*dD;
%                             probSpread = 0.5*(spreadC + spreadN)*dD;
                            
                        end

                        
                        cD = cD + dD;
                        cP = cP + probSpread;

                        dcfN = dcfC;
                        spreadN = spreadC;

                        %induct backward to the next time step
                        % one time backward induction
                        lastTimeIdx = currentTimeIdx;

                        if startTimeIdx < lastTimeIdx
                            lastTimeIdx = lastTimeIdx -1;
                            currentTimeIdx = lastTimeIdx;
                        end


                    end

                    %% FromToInduction end
                    %% AtDate evaluation
                    % Adding a new cashflow to the payoff

                    dcf = schedule.dayCountFraction(i);
                    
                    if inclusiveYN
                        % we add valuation date
                        cD = schedule.dayCount(i);
                        rangeDays =  security.rangeDays(i);
                        payoffStateA.cashflow = nominal * coupon * dcf * (rangeDays + cP)/cD;
                    else
                        payoffStateA.cashflow = nominal * coupon * dcf * cP/cD;
                    end
                    % eventDate(i) = startDate(i) therefore there is no induction in modelStates
                    % i.e. currentTimeIdx has not changed
                    % no need to update modelStateT
                    % modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:end,:);
                    % since cashflow's are added at the startDate(i) we need to
                    % multiply cashflow with the df(startDate(i), payDate(i))
                    payTime = DateDiff(schedule.payDates(i),valueDate);
                    
                    %event is at startDate so we need to discount the
                    %cashflow first from paydate to startdate
%                     df_sp = model.discountFactor(currentTime, payTime,numMatTime,modelStateT);

%                     for k=1:pathSize
%                         payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_sp(k);
%                     end
                    
                    % process payoff
                    
                    currentNodeIdx = i;
                    
                    % 1st nonCall column
                    nonCall.payoffStates.cashflow(currentNodeIdx,:) = payoffStateA.cashflow;
                    payoffStateA.cashflow_npv = payoffStateA.cashflow * model.DFRisky(payTime/365.0);
                    
%                     payoffStateA.cashflow_npv = model.discountPayoff(currentTime,numMatTime ...
%                     ,modelStateT,payoffStateA.cashflow);
                    nonCall.payoffStates.cashflow_npv(currentNodeIdx,:) = payoffStateA.cashflow_npv;
                    payoffStateA.df = model.DFRisky(payTime/365.0);
%                     payoffStateA.df = model.discountPayoff(currentTime,numMatTime ...
%                     ,modelStateT,1.0*ones(1,pathSize));
                    nonCall.payoffStates.df(currentNodeIdx,:) = payoffStateA.df;
                    
                    nonCall.payoff = nonCall.payoff + payoffStateA.cashflow_npv;
                    
                    nonCall.cashflow_e(currentNodeIdx) = mean(payoffStateA.cashflow);
                    nonCall.rangeProb_e(currentNodeIdx) = mean(nonCall.cashflow_e(currentNodeIdx)/(nominal * coupon * dcf));
                    nonCall.cashflow_npv_e(currentNodeIdx) = mean(payoffStateA.cashflow_npv);

                    nonCall.df_e(currentNodeIdx) = mean(payoffStateA.df);
                    nonCall.df_c(currentNodeIdx) = model.DFRisky(currentTime/365.0);
                    
                    nonCall.fwdCD(currentNodeIdx) = fwdCD;
                    nonCall.fwdKTB(currentNodeIdx) = fwdKTB;
                    
                    nonCall.fwdSpread(currentNodeIdx) = fwdSpread;
                    nonCall.fwdMu(currentNodeIdx) = fwdMu;
                    nonCall.fwdSigma(currentNodeIdx) = fwdSigma;

                end

                nonCall.npv = mean(nonCall.payoff);
                out.pricerInfo.nonCall = nonCall;
                
%                 callable.npv = mean(callable.payoff);
%                 out.pricerInfo.callable = callable;
                out.daycountFraction = schedule.dayCountFraction;
                
        end
        

        
        
        
        
    end
    
end



In [ ]:
classdef DailyWeeklyRiskyPricer_CDAndCDKTBSpreadRA < H_RootObject
    %Pricer(Security,Model,NumMethod)
    %Prcer is constructed from Security, Model, NumMethod 
    
    properties
        m_security
        m_model
        m_numMethod
        overHedgeYN
        
    end
    
    methods
        function pricer = DailyWeeklyRiskyPricer_CDAndCDKTBSpreadRA(security,model,numMethod,overHedgeYN)
                pricer.m_security = security;
                pricer.m_model = model;
                pricer.m_numMethod = numMethod;
                pricer.overHedgeYN = overHedgeYN;
                
        end
        
        function out = computePriceMCCallableGeneric(pricer,valueDate,pricerOption)
                security = pricer.m_security;
                numMethod = pricer.m_numMethod;
                model = pricer.m_model;
                schedule = security.couponSchedule;
                callSchedule = security.callSchedule;
                
                % numMethodTimeSteps Generate
                mcOneTimeStep = numMethod.mcOneTimeStep;
                outInfo = numMethod.numMethodTimeStepsGenerateDW(valueDate,schedule,mcOneTimeStep);
            
                numMethodInfo.timeStepsInfo = outInfo;

                % mc random number init
                %pathSize = 2^16;
                pathSize = numMethod.pathSize;

                % generate random numbers and modelstates forward
                rng('default');
                rng(0);

                % LGM1F : one factor
                % LGM2F : two factor
                
                numOfFactors = model.numOfFactor;

                %Z= randn(numOfFactors*outInfo.totalTimeStepsSize,pathSize/2);
                Z= randn(pathSize/2,numOfFactors*outInfo.totalTimeStepsSize);
                Z=[Z;-Z];
                Z=Z';
                
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;

                % we check with the c code, so we use generated random numbers
%                 mc_random = importdata('mc_random.xlsx');
%                 mc_random = importdata('D:\mc_random_number_new_report.txt');
%                 Z= mc_random;
                
                numMethodInfo.pathSize = pathSize;
                numMethodInfo.numOfFactors = numOfFactors;
                numMethodInfo.totalTimeSteps = outInfo.totalTimeSteps;
                numMethodInfo.totalTimeStepsSize = outInfo.totalTimeStepsSize;
                numMethodInfo.Z = Z;



                %numMethod generate model states for the given timeSteps
                modelStates = numMethod.generateModelStatesNF(valueDate,schedule,numMethodInfo,model);

                % numeraire info
                numMatTime = numMethodInfo.totalTimeSteps(end);
                days = DateDiff(schedule.payDates(schedule.numOfPeriod),valueDate);
                numMatTime = max(numMatTime, days);
                dfNumMat = model.DFRisky(numMatTime/365.0);
                
                numeraire.numMatTime = numMatTime;
                numeraire.dfNumMat = dfNumMat;
                
                % induct backward and calculate cashflow
            
                periodCount = schedule.numOfPeriod;
                
                % pricer single info initialize
                
                nonCall = PricerInfo(periodCount +1, pathSize);
                callable = PricerInfo(periodCount +1, pathSize);
                dfCol = PricerInfo(periodCount +1, pathSize);
                
                dummy1 = PricerInfo(periodCount +1, pathSize);
                dummy2 = PricerInfo(periodCount +1, pathSize);
                
                
                % intermediate variables
                payoffStateA.cashflow = zeros(1, pathSize);
                payoffStateA.cashflow_npv = zeros(1, pathSize);
                
            
                % we calculate cashflow backward from maturityDate to valueDate
                % rangeAccrual coupons are added at the startDate of the Period
                % At maturity

                nominal = security.nominal;
                coupon = security.coupon;
                callYN = security.callYN;
                
                ralower = security.lower;
                raupper = security.upper;
                
                ralowerS = security.lowerS;
                raupperS = security.upperS;
                
                tenorCD = security.tenor;
                
                tenorLong = security.tenorL;
                tenorShort = security.tenorS;
                
                currentTime = numMethodInfo.totalTimeSteps(end);
                totalTimeStepsSize = numMethodInfo.totalTimeStepsSize;
                currentTimeIdx = totalTimeStepsSize;
                
                currentNodeIdx = periodCount + 1;
                
                modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:numOfFactors*(currentTimeIdx-1)+numOfFactors,:);
                
                payoffStateA.cashflow = nominal*ones(1,pathSize);
                
                payTime = DateDiff(schedule.payDates(schedule.numOfPeriod),valueDate);
                df_ep = model.discountFactorRisky(currentTime, payTime,numMatTime,modelStateT);
                for k=1:pathSize
                    payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                end
                
                %process payoff at maturity
                nonCall.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                
                callable.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                
                %df column
                payoffStateA.cashflow = ones(1,pathSize);
                for k=1:pathSize
                    payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                end
                
                dfCol.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                                 
                %dummy column

                if strcmp(pricerOption,'PSA')
                    payoffStateA.cashflow = model.CMS_PSA_SimpleDate('KRW',valueDate,currentTime,numMatTime, tenorLong,modelStateT);
                    dummy1.ProcessPayoff(model,currentTime,numeraire, ...
                                 modelStateT,payoffStateA.cashflow,currentNodeIdx);
                    
                    payoffStateA.cashflow = model.CMS_PSA_SimpleDate('USD',valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                    dummy2.ProcessPayoff(model,currentTime,numeraire, ...
                                 modelStateT,payoffStateA.cashflow,currentNodeIdx);
                
                else
                    payoffStateA.cashflow = model.Libor_SimpleDate('KRW',valueDate,currentTime,currentTime,numMatTime, tenorLong,modelStateT); 
                    dummy1.ProcessPayoff(model,currentTime,numeraire, ...
                                 modelStateT,payoffStateA.cashflow,currentNodeIdx);
                             
                    payoffStateA.cashflow = model.Libor_SimpleDate('USD',valueDate,currentTime,currentTime,numMatTime, tenorShort,modelStateT); 
                    dummy2.ProcessPayoff(model,currentTime,numeraire, ...
                                 modelStateT,payoffStateA.cashflow,currentNodeIdx);
                    
                end
                

%                 payoffStateA.cashflow = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                
%                 payoffStateA.cashflow = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                
                
                                 
                  

                lastTimeIdx = currentTimeIdx;
                totalTimeSteps = numMethodInfo.timeStepsInfo.totalTimeSteps;
                
                % flag for inclusive schedule
%                 simpleYN = false;
%                 simpleYN = true;
                if pricer.overHedgeYN == true 
                    simpleYN = 2;
                else
                    simpleYN = 1;
                end
                
                for i=periodCount:-1:1
                    % past schedule neglect
                    if DateDiff(valueDate,schedule.payDates(i)) >= 0, continue;end 
                    startTimeIdx = numMethodInfo.timeStepsInfo.startTimeIdx(i);
                    endTimeIdx = numMethodInfo.timeStepsInfo.endTimeIdx(i);

                    inclusiveYN = numMethodInfo.timeStepsInfo.inclusiveYN(i);
                 %% FromToInduction start

                    %modelsStates induct backward(lastTimeIdx to endTimeIdx)
                    while endTimeIdx < lastTimeIdx
                        lastTimeIdx = lastTimeIdx -1;
                        if lastTimeIdx == endTimeIdx break;end
                    end

                    currentTimeIdx = lastTimeIdx;
                    currentTime = totalTimeSteps(currentTimeIdx);
                    % now we are at endTimeIdx
                    % variables for coupon valuation initialized
                    

                    cD = 0; % cumulated day count fraction
                    cP = 0; 
                    dI = 0; % dummy idx for loop
                    dcfC = 0;
                    dcfN = 0;
                    dD = 0;
                    idxCD = zeros(1,pathSize);
                    idxL = zeros(1,pathSize);
                    idxS = zeros(1,pathSize);
                    
                    idxC = zeros(1,pathSize);
                    idxN = zeros(1,pathSize);
                    
                    spreadC = zeros(1,pathSize);
                    spreadN = zeros(1,pathSize);
                    probSpread = zeros(1,pathSize);
                    
                    numOfTimeSteps = numMethodInfo.timeStepsInfo.numOfTimeSteps(i);
                    
                    % if inclusive then we use simpleYN rangeProb
                    %if inclusive then we do not evaluate the last step but
                    % use fixing instead 
                    endTimeStep = 1;
                    if inclusiveYN
%                         simpleYN = true;
                        simpleYN = 1;
                        endTimeStep = 2;
                        if pricer.overHedgeYN == true 
                            simpleYN = 2;
                        end
                    end
                    
                    for j=numOfTimeSteps:-1:endTimeStep
                        % evaluate dealdescription
                        % last step is omitted
                        currentTime = totalTimeSteps(currentTimeIdx);
                        %dcfC = currentTime/365.0;
                        dcfC = currentTime;
                        
                        if j~= numOfTimeSteps
                            dD = (dcfN-dcfC);
                        end

                        modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:numOfFactors*(currentTimeIdx-1)+numOfFactors,:);
                        
                        idxCD = model.Libor_SimpleDate('KRW',valueDate,currentTime,currentTime,numMatTime, tenorCD,modelStateT);
                        idxC = idxCD;
                        
                        if strcmp(pricerOption,'PSA')
                            idxL = model.CMS_PSA_SimpleDate('KRW',valueDate,currentTime,numMatTime, tenorLong,modelStateT);
                            idxS = model.CMS_PSA_SimpleDate('USD',valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                        else
                            idxL = model.Libor_SimpleDate('KRW',valueDate,currentTime,currentTime,numMatTime, tenorLong,modelStateT);
                            idxS = model.Libor_SimpleDate('USD',valueDate,currentTime,currentTime,numMatTime, tenorShort,modelStateT);
                        end
                        spreadC = idxL - idxS;
%                         hedgeBuffer = 0.0;
                        hedgeBuffer = 0.0030;
                        
                        probSpread = model.DRPr(idxC,idxN,ralower,raupper,spreadC,spreadN,ralowerS,raupperS,simpleYN,hedgeBuffer,pathSize)*dD;
                    
%                         probSpread = model.SRPr(spreadC,spreadN,ralower,raupper,simpleYN,pathSize)*dD;
                        cD = cD + dD;
                        cP = cP + probSpread;

                        dcfN = dcfC;
                        
                        idxN = idxC;
                        spreadN = spreadC;

                        %induct backward to the next time step
                        % one time backward induction
                        lastTimeIdx = currentTimeIdx;

                        if startTimeIdx < lastTimeIdx
                            lastTimeIdx = lastTimeIdx -1;
                            currentTimeIdx = lastTimeIdx;
                        end


                    end

                    %% FromToInduction end
        % if the current period is inclusive, then the start date is not evaluated in the above for loop.
		% instead the fixing will be treated separately below.
		% however current_time_idx is updated at the end of the for loop.
		% so we need to update model_states accordingly.
		% if model_states were updated previously then the following additionaly step doesn't affect the result anyway.
                    currentTime = totalTimeSteps(currentTimeIdx);
                    modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:numOfFactors*(currentTimeIdx-1)+numOfFactors,:);
                    
                    %% AtDate evaluation
                    % Adding a new cashflow to the payoff

                    dcf = schedule.dayCountFraction(i);
                    
                    if inclusiveYN
                        % we add valuation date
                        cD = schedule.dayCount(i);
                        rangeDays =  security.rangeDays(i);
                        payoffStateA.cashflow = nominal * coupon * dcf * (rangeDays + cP)/cD;
                    else
                        payoffStateA.cashflow = nominal * coupon * dcf * cP/cD;
                    end
                    % eventDate(i) = startDate(i) therefore there is no induction in modelStates
                    % i.e. currentTimeIdx has not changed
                    % no need to update modelStateT
                    % modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:end,:);
                    % since cashflow's are added at the startDate(i) we need to
                    % multiply cashflow with the df(startDate(i), payDate(i))
                    payTime = DateDiff(schedule.payDates(i),valueDate);
                    
                    %event is at startDate so we need to discount the
                    %cashflow first from paydate to startdate
                    df_ep = model.discountFactorRisky(currentTime, payTime,numMatTime,modelStateT);

                    for k=1:pathSize
                        payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                    end
                    
                    % process payoff
                    
                    currentNodeIdx = i;
                    
                    nonCall.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                
                    callable.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                    
                    %df column
                    payoffStateA.cashflow = ones(1,pathSize);
                    for k=1:pathSize
                        payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                    end
                    
                    dfCol.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                                 
                    %dummy column
                    if strcmp(pricerOption,'PSA')
                        payoffStateA.cashflow = model.CMS_PSA_SimpleDate('KRW',valueDate,currentTime,numMatTime, tenorLong,modelStateT);
                        dummy1.ProcessPayoff(model,currentTime,numeraire, ...
                                         modelStateT,payoffStateA.cashflow,currentNodeIdx);



                        payoffStateA.cashflow = model.CMS_PSA_SimpleDate('USD',valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                        dummy2.ProcessPayoff(model,currentTime,numeraire, ...
                                         modelStateT,payoffStateA.cashflow,currentNodeIdx);
                    
                    else
                        payoffStateA.cashflow = model.Libor_SimpleDate('KRW',valueDate,currentTime,currentTime,numMatTime, tenorLong,modelStateT); 
                        dummy1.ProcessPayoff(model,currentTime,numeraire, ...
                                 modelStateT,payoffStateA.cashflow,currentNodeIdx);
                        
                        payoffStateA.cashflow = model.Libor_SimpleDate('USD',valueDate,currentTime,currentTime,numMatTime, tenorShort,modelStateT); 
                        dummy2.ProcessPayoff(model,currentTime,numeraire, ...
                                 modelStateT,payoffStateA.cashflow,currentNodeIdx);
                        
                    end

                                 
                    
                    
                    
                    % if exercise is possible
                    if i == 1
                        dummaaa = 1;
                    end
                    
                    % call event happens  at the start of the period
                    % at call date, by default, call option is expired
                    
                    if callYN(i) && (DateDiff(valueDate,callSchedule.resetDates(i)) < 0)
                        % 1.undiscount callable.payoff
                        % 2.build the state variables matrix
                        % 3.regress 1's with the given state variables
                        % (model states 1,2)
                        % 4. exercise per each path in
                        % discount the new payoff and assign into callable
                        % column
                        
                        %1 undiscount callable.payoff and get the
                        %contiValue
                       
                        %exercise value
                        exerciseValue = security.nominal*ones(1,pathSize);
                        payTime = DateDiff(callSchedule.payDates(i),valueDate);
                        %event is at startDate so we need to discount the
                        %cashflow first from paydate to startdate
                        df_ep = model.discountFactorRisky(currentTime, payTime,numMatTime,modelStateT);
                        for k=1:pathSize
                            exerciseValue(k) = exerciseValue(k) * df_ep(k);
                        end
                        
                        contiValue = zeros(1,pathSize);
                        onePayoff = ones(1,pathSize);
                        onePayoffD = model.discountPayoffRisky(currentTime,numMatTime ...
                        ,modelStateT,onePayoff);
                        contiValue = callable.payoff./onePayoffD;
                        
                        %Nominal Regression test
                        contiValue = contiValue/10000;
                        
                        % build the state variabls matrix
                        dimensionLS = (numOfFactors + 1)*(numOfFactors + 2)/2;
                        
                        decisionVar = zeros(numOfFactors,size(modelStateT,2));
                        
                        for idxLS =1:numOfFactors
                            decisionVar(idxLS,:) = modelStateT(idxLS,:); 
                        end
%                         decisionVar1 = modelStateT(1,:);
%                         decisionVar2 = modelStateT(2,:);
                        % we use upto 2nd order polynomial as regression
                        % equatioin
                        
                        regMatrix = ones(pathSize,dimensionLS);
                        
                        for idx_r=1:pathSize
                            for idxLS=1:numOfFactors
                                regMatrix(idx_r,idxLS+1) = decisionVar(idxLS,idx_r);
                            end
                            
                            tempIdx = 1;
                            for idxLS1=1:numOfFactors
                                for idxLS2=idxLS1:numOfFactors
                                    regMatrix(idx_r,numOfFactors+1+tempIdx) = decisionVar(idxLS1,idx_r)*decisionVar(idxLS2,idx_r);
                                    tempIdx = tempIdx + 1;
                                end
                            end
                            
                        end
                        
                        % simple regression <- for path dependent option
                        % we should change it
                        % beta  : regression coefficients
                        beta =zeros(dimensionLS,1);
                        beta = regress(contiValue',regMatrix);
                        regContiValue = regMatrix*beta;
                       
                         %Nominal Regression test
                        regContiValue = regContiValue * 10000;
                        
                        exercisedPayoff = regContiValue;
                        % if regContiValue is bigger than exerciseValue
                        % then issue exercise call option
                        for idx_r=1:pathSize
                            if regContiValue(idx_r) >= exerciseValue(idx_r) 
                                exercisedPayoff(idx_r) = exerciseValue(idx_r);
                            end
                        end
                        
                        % we discount the new payoff and 
                        % assign it into the existing callable payoff
                        callable.payoff = model.discountPayoffRisky(currentTime,numMatTime ...
                        ,modelStateT,exercisedPayoff);
                        
                    end
                end

                nonCall.npv = mean(nonCall.payoff);
                out.nonCall = nonCall;
                callable.npv = mean(callable.payoff);
                dfCol.npv = mean(dfCol.payoff);
                dummy1.npv = mean(dummy1.npv);
                dummy2.npv = mean(dummy2.npv);
                out.callable = callable;
                out.dfCol = dfCol;
                out.dummy1 = dummy1;
                out.dummy2 = dummy2;
                out.daycountFraction = schedule.dayCountFraction;
                
        end
        
        function out = computePriceMCCallableGenericReg2(pricer,valueDate,pricerOption)
                security = pricer.m_security;
                numMethod = pricer.m_numMethod;
                model = pricer.m_model;
                schedule = security.couponSchedule;
                callSchedule = security.callSchedule;
                
                % numMethodTimeSteps Generate
                mcOneTimeStep = numMethod.mcOneTimeStep;
                outInfo = numMethod.numMethodTimeStepsGenerateDW(valueDate,schedule,mcOneTimeStep);
            
                numMethodInfo.timeStepsInfo = outInfo;

                % mc random number init
                %pathSize = 2^16;
                pathSize = numMethod.pathSize;

                % generate random numbers and modelstates forward
                rng('default');
                rng(0);

                % LGM1F : one factor
                % LGM2F : two factor
                
                numOfFactors = model.numOfFactor;

                %Z= randn(numOfFactors*outInfo.totalTimeStepsSize,pathSize/2);
                Z= randn(pathSize/2,numOfFactors*outInfo.totalTimeStepsSize);
                Z=[Z;-Z];
                Z=Z';
                
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;

                % we check with the c code, so we use generated random numbers
%                 mc_random = importdata('mc_random.xlsx');
%                 mc_random = importdata('D:\mc_random_number_new_report.txt');
%                 Z= mc_random;
                
                numMethodInfo.pathSize = pathSize;
                numMethodInfo.numOfFactors = numOfFactors;
                numMethodInfo.totalTimeSteps = outInfo.totalTimeSteps;
                numMethodInfo.totalTimeStepsSize = outInfo.totalTimeStepsSize;
                numMethodInfo.Z = Z;



                %numMethod generate model states for the given timeSteps
                modelStates = numMethod.generateModelStatesNF(valueDate,schedule,numMethodInfo,model);

                % numeraire info
                numMatTime = numMethodInfo.totalTimeSteps(end);
                days = DateDiff(schedule.payDates(schedule.numOfPeriod),valueDate);
                numMatTime = max(numMatTime, days);
                dfNumMat = model.DFRisky(numMatTime/365.0);
                
                numeraire.numMatTime = numMatTime;
                numeraire.dfNumMat = dfNumMat;
                
                % induct backward and calculate cashflow
            
                periodCount = schedule.numOfPeriod;
                
                % pricer single info initialize
                
                nonCall = PricerInfo(periodCount +1, pathSize);
                callable = PricerInfo(periodCount +1, pathSize);
                dfCol = PricerInfo(periodCount +1, pathSize);
                
                dummy1 = PricerInfo(periodCount +1, pathSize);
                dummy2 = PricerInfo(periodCount +1, pathSize);
                
                
                % intermediate variables
                payoffStateA.cashflow = zeros(1, pathSize);
                payoffStateA.cashflow_npv = zeros(1, pathSize);
                
            
                % we calculate cashflow backward from maturityDate to valueDate
                % rangeAccrual coupons are added at the startDate of the Period
                % At maturity

                hedgeBuffer = security.hedgeBuffer;
                
                nominal = security.nominal;
                coupon = security.coupon;
                callYN = security.callYN;
                
                ralower = security.lower;
                raupper = security.upper;
                
                ralowerS = security.lowerS;
                raupperS = security.upperS;
                
                tenorCD = security.tenor;
                
                tenorLong = security.tenorL;
                tenorShort = security.tenorS;
                
                currentTime = numMethodInfo.totalTimeSteps(end);
                totalTimeStepsSize = numMethodInfo.totalTimeStepsSize;
                currentTimeIdx = totalTimeStepsSize;
                
                currentNodeIdx = periodCount + 1;
                
                modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:numOfFactors*(currentTimeIdx-1)+numOfFactors,:);
                
                payoffStateA.cashflow = nominal*ones(1,pathSize);
                
                payTime = DateDiff(schedule.payDates(schedule.numOfPeriod),valueDate);
                df_ep = model.discountFactorRisky(currentTime, payTime,numMatTime,modelStateT);
                for k=1:pathSize
                    payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                end
                
                %process payoff at maturity
                nonCall.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                
                callable.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                
                %df column
                payoffStateA.cashflow = ones(1,pathSize);
                for k=1:pathSize
                    payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                end
                
                dfCol.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                                 
                %dummy column

                if strcmp(pricerOption,'PSA')
                    payoffStateA.cashflow = model.CMS_PSA_SimpleDate('KRW',valueDate,currentTime,numMatTime, tenorLong,modelStateT);
                    dummy1.ProcessPayoff(model,currentTime,numeraire, ...
                                 modelStateT,payoffStateA.cashflow,currentNodeIdx);
                    
                    payoffStateA.cashflow = model.CMS_PSA_SimpleDate('USD',valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                    dummy2.ProcessPayoff(model,currentTime,numeraire, ...
                                 modelStateT,payoffStateA.cashflow,currentNodeIdx);
                
                else
                    payoffStateA.cashflow = model.Libor_SimpleDate('KRW',valueDate,currentTime,currentTime,numMatTime, tenorLong,modelStateT); 
                    dummy1.ProcessPayoff(model,currentTime,numeraire, ...
                                 modelStateT,payoffStateA.cashflow,currentNodeIdx);
                             
                    payoffStateA.cashflow = model.Libor_SimpleDate('USD',valueDate,currentTime,currentTime,numMatTime, tenorShort,modelStateT); 
                    dummy2.ProcessPayoff(model,currentTime,numeraire, ...
                                 modelStateT,payoffStateA.cashflow,currentNodeIdx);
                    
                end
                

%                 payoffStateA.cashflow = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                
%                 payoffStateA.cashflow = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                
                
                                 
                  

                lastTimeIdx = currentTimeIdx;
                totalTimeSteps = numMethodInfo.timeStepsInfo.totalTimeSteps;
                
                % flag for inclusive schedule
%                 simpleYN = false;
%                 simpleYN = true;
                if pricer.overHedgeYN == true 
                    simpleYN = 2;
                else
                    simpleYN = 1;
                end
                
                for i=periodCount:-1:1
                    % past schedule neglect
                    if DateDiff(valueDate,schedule.payDates(i)) >= 0, continue;end 
                    startTimeIdx = numMethodInfo.timeStepsInfo.startTimeIdx(i);
                    endTimeIdx = numMethodInfo.timeStepsInfo.endTimeIdx(i);

                    inclusiveYN = numMethodInfo.timeStepsInfo.inclusiveYN(i);
                 %% FromToInduction start

                    %modelsStates induct backward(lastTimeIdx to endTimeIdx)
                    while endTimeIdx < lastTimeIdx
                        lastTimeIdx = lastTimeIdx -1;
                        if lastTimeIdx == endTimeIdx break;end
                    end

                    currentTimeIdx = lastTimeIdx;
                    currentTime = totalTimeSteps(currentTimeIdx);
                    % now we are at endTimeIdx
                    % variables for coupon valuation initialized
                    

                    cD = 0; % cumulated day count fraction
                    cP = 0; 
                    dI = 0; % dummy idx for loop
                    dcfC = 0;
                    dcfN = 0;
                    dD = 0;
                    idxCD = zeros(1,pathSize);
                    idxL = zeros(1,pathSize);
                    idxS = zeros(1,pathSize);
                    
                    idxC = zeros(1,pathSize);
                    idxN = zeros(1,pathSize);
                    
                    spreadC = zeros(1,pathSize);
                    spreadN = zeros(1,pathSize);
                    probSpread = zeros(1,pathSize);
                    
                    numOfTimeSteps = numMethodInfo.timeStepsInfo.numOfTimeSteps(i);
                    
                    % if inclusive then we use simpleYN rangeProb
                    %if inclusive then we do not evaluate the last step but
                    % use fixing instead 
                    endTimeStep = 1;
                    if inclusiveYN
%                         simpleYN = true;
                        simpleYN = 1;
                        endTimeStep = 2;
                        if pricer.overHedgeYN == true 
                            simpleYN = 2;
                        end
                    end
                    
                    for j=numOfTimeSteps:-1:endTimeStep
                        % evaluate dealdescription
                        % last step is omitted
                        currentTime = totalTimeSteps(currentTimeIdx);
                        %dcfC = currentTime/365.0;
                        dcfC = currentTime;
                        
                        if j~= numOfTimeSteps
                            dD = (dcfN-dcfC);
                        end

                        modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:numOfFactors*(currentTimeIdx-1)+numOfFactors,:);
                        
                        idxCD = model.Libor_SimpleDate('KRW',valueDate,currentTime,currentTime,numMatTime, tenorCD,modelStateT);
                        idxC = idxCD;
                        
                        if strcmp(pricerOption,'PSA')
                            idxL = model.CMS_PSA_SimpleDate('KRW',valueDate,currentTime,numMatTime, tenorLong,modelStateT);
                            idxS = model.CMS_PSA_SimpleDate('USD',valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                        else
                            idxL = model.Libor_SimpleDate('KRW',valueDate,currentTime,currentTime,numMatTime, tenorLong,modelStateT);
                            idxS = model.Libor_SimpleDate('USD',valueDate,currentTime,currentTime,numMatTime, tenorShort,modelStateT);
                        end
                        spreadC = idxL - idxS;

                        probSpread = model.DRPr(idxC,idxN,ralower,raupper,spreadC,spreadN,ralowerS,raupperS,simpleYN,hedgeBuffer,pathSize)*dD;
                    
%                         probSpread = model.SRPr(spreadC,spreadN,ralower,raupper,simpleYN,pathSize)*dD;
                        cD = cD + dD;
                        cP = cP + probSpread;

                        dcfN = dcfC;
                        
                        idxN = idxC;
                        spreadN = spreadC;

                        %induct backward to the next time step
                        % one time backward induction
                        lastTimeIdx = currentTimeIdx;

                        if startTimeIdx < lastTimeIdx
                            lastTimeIdx = lastTimeIdx -1;
                            currentTimeIdx = lastTimeIdx;
                        end


                    end

                    %% FromToInduction end
        % if the current period is inclusive, then the start date is not evaluated in the above for loop.
		% instead the fixing will be treated separately below.
		% however current_time_idx is updated at the end of the for loop.
		% so we need to update model_states accordingly.
		% if model_states were updated previously then the following additionaly step doesn't affect the result anyway.
                    currentTime = totalTimeSteps(currentTimeIdx);
                    modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:numOfFactors*(currentTimeIdx-1)+numOfFactors,:);
                    
                    %% AtDate evaluation
                    % Adding a new cashflow to the payoff

                    dcf = schedule.dayCountFraction(i);
                    
                    if inclusiveYN
                        % we add valuation date
                        cD = schedule.dayCount(i);
                        rangeDays =  security.rangeDays(i);
                        payoffStateA.cashflow = nominal * coupon * dcf * (rangeDays + cP)/cD;
                    else
                        payoffStateA.cashflow = nominal * coupon * dcf * cP/cD;
                    end
                    % eventDate(i) = startDate(i) therefore there is no induction in modelStates
                    % i.e. currentTimeIdx has not changed
                    % no need to update modelStateT
                    % modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:end,:);
                    % since cashflow's are added at the startDate(i) we need to
                    % multiply cashflow with the df(startDate(i), payDate(i))
                    payTime = DateDiff(schedule.payDates(i),valueDate);
                    
                    %event is at startDate so we need to discount the
                    %cashflow first from paydate to startdate
                    df_ep = model.discountFactorRisky(currentTime, payTime,numMatTime,modelStateT);

                    for k=1:pathSize
                        payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                    end
                    
                    % process payoff
                    
                    currentNodeIdx = i;
                    
                    nonCall.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                
                    callable.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                    
                    %df column
                    payoffStateA.cashflow = ones(1,pathSize);
                    for k=1:pathSize
                        payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                    end
                    
                    dfCol.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                                 
                    %dummy column
                    if strcmp(pricerOption,'PSA')
                        payoffStateA.cashflow = model.CMS_PSA_SimpleDate('KRW',valueDate,currentTime,numMatTime, tenorLong,modelStateT);
                        dummy1.ProcessPayoff(model,currentTime,numeraire, ...
                                         modelStateT,payoffStateA.cashflow,currentNodeIdx);



                        payoffStateA.cashflow = model.CMS_PSA_SimpleDate('USD',valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                        dummy2.ProcessPayoff(model,currentTime,numeraire, ...
                                         modelStateT,payoffStateA.cashflow,currentNodeIdx);
                    
                    else
                        payoffStateA.cashflow = model.Libor_SimpleDate('KRW',valueDate,currentTime,currentTime,numMatTime, tenorLong,modelStateT); 
                        dummy1.ProcessPayoff(model,currentTime,numeraire, ...
                                 modelStateT,payoffStateA.cashflow,currentNodeIdx);
                        
                        payoffStateA.cashflow = model.Libor_SimpleDate('USD',valueDate,currentTime,currentTime,numMatTime, tenorShort,modelStateT); 
                        dummy2.ProcessPayoff(model,currentTime,numeraire, ...
                                 modelStateT,payoffStateA.cashflow,currentNodeIdx);
                        
                    end

                                 
                    
                    
                    
                    % if exercise is possible
                    if i == 1
                        dummaaa = 1;
                    end
                    
                    % call event happens  at the start of the period
                    % at call date, by default, call option is expired
                    
                    if callYN(i) && (DateDiff(valueDate,callSchedule.resetDates(i)) < 0)
                        % 1.undiscount callable.payoff
                        % 2.build the state variables matrix
                        % 3.regress 1's with the given state variables
                        % (model states 1,2)
                        % 4. exercise per each path in
                        % discount the new payoff and assign into callable
                        % column
                        
                        %1 undiscount callable.payoff and get the
                        %contiValue
                       
                        %exercise value
                        exerciseValue = security.nominal*ones(1,pathSize);
                        payTime = DateDiff(callSchedule.payDates(i),valueDate);
                        %event is at startDate so we need to discount the
                        %cashflow first from paydate to startdate
                        df_ep = model.discountFactorRisky(currentTime, payTime,numMatTime,modelStateT);
                        for k=1:pathSize
                            exerciseValue(k) = exerciseValue(k) * df_ep(k);
                        end
                        
                        contiValue = zeros(1,pathSize);
                        onePayoff = ones(1,pathSize);
                        onePayoffD = model.discountPayoffRisky(currentTime,numMatTime ...
                        ,modelStateT,onePayoff);
                        contiValue = callable.payoff./onePayoffD;
                        
                        %Nominal Regression test
                        contiValue = contiValue/10000;
                        
                        % build the state variabls matrix
                        %zeroth order
                        dimensionLS = 1;
                        
                        %1st order
                        for idxLS =1:numOfFactors
                            dimensionLS =  dimensionLS +1; 
                        end
                        
                        %2nd order
                        for idxLS1=1:numOfFactors
                            for idxLS2=idxLS1:numOfFactors
                                dimensionLS = dimensionLS + 1;
                            end
                        end
                        
                        %3rd order
                        for idxLS1=1:numOfFactors
                            for idxLS2=idxLS1:numOfFactors
                                for idxLS3=idxLS2:numOfFactors
                                    dimensionLS = dimensionLS + 1;
                                end
                            end
                        end
                        
%                         dimensionLS = (numOfFactors + 1)*(numOfFactors + 2)/2;
                        
                        decisionVar = zeros(numOfFactors,size(modelStateT,2));
                        
                        for idxLS =1:numOfFactors
                            decisionVar(idxLS,:) = modelStateT(idxLS,:); 
                        end
%                         decisionVar1 = modelStateT(1,:);
%                         decisionVar2 = modelStateT(2,:);
                        % we use upto 2nd order polynomial as regression
                        % equatioin
                        
                        regMatrix = ones(pathSize,dimensionLS);
                        
                        for idx_r=1:pathSize
                            for idxLS=1:numOfFactors
                                regMatrix(idx_r,idxLS+1) = decisionVar(idxLS,idx_r);
                            end
                            
                            tempIdx = 1;
                            for idxLS1=1:numOfFactors
                                for idxLS2=idxLS1:numOfFactors
                                    regMatrix(idx_r,numOfFactors+1+tempIdx) = decisionVar(idxLS1,idx_r)*decisionVar(idxLS2,idx_r);
                                    tempIdx = tempIdx + 1;
                                end
                            end
                            
                            tempIdx2 = 1;
                            for idxLS1=1:numOfFactors
                                for idxLS2=idxLS1:numOfFactors
                                    for idxLS3=idxLS2:numOfFactors
                                        regMatrix(idx_r,numOfFactors+1+tempIdx-1 + tempIdx2) = decisionVar(idxLS1,idx_r)*decisionVar(idxLS2,idx_r)*decisionVar(idxLS3,idx_r);
                                        tempIdx = tempIdx + 1;
                                    end
                                end
                            end
                            
                        end
                        
                        % simple regression <- for path dependent option
                        % we should change it
                        % beta  : regression coefficients
                        beta =zeros(dimensionLS,1);
                        beta = regress(contiValue',regMatrix);
                        regContiValue = regMatrix*beta;
                       
                         %Nominal Regression test
                        regContiValue = regContiValue * 10000;
                                                
%                         exercisedPayoff = regContiValue;
                        exercisedPayoff = contiValue * 10000;
                        % if regContiValue is bigger than exerciseValue
                        % then issue exercise call option
                        for idx_r=1:pathSize
                            if regContiValue(idx_r) >= exerciseValue(idx_r) 
                                exercisedPayoff(idx_r) = exerciseValue(idx_r);
                            end
                        end
                        
                        % we discount the new payoff and 
                        % assign it into the existing callable payoff
                        callable.payoff = model.discountPayoffRisky(currentTime,numMatTime ...
                        ,modelStateT,exercisedPayoff);
                        
                    end
                end

                nonCall.npv = mean(nonCall.payoff);
                out.nonCall = nonCall;
                callable.npv = mean(callable.payoff);
                dfCol.npv = mean(dfCol.payoff);
                dummy1.npv = mean(dummy1.npv);
                dummy2.npv = mean(dummy2.npv);
                out.callable = callable;
                out.dfCol = dfCol;
                out.dummy1 = dummy1;
                out.dummy2 = dummy2;
                out.daycountFraction = schedule.dayCountFraction;
                
        end
        
        
        function out = computePriceClosedSimple(pricer,valueDate)
                security = pricer.m_security;
                numMethod = pricer.m_numMethod;
                model = pricer.m_model;
                schedule = security.couponSchedule;
                
                % numMethodTimeSteps Generate
                mcOneTimeStep = numMethod.mcOneTimeStep;
                outInfo = numMethod.numMethodTimeStepsGenerateDW(valueDate,schedule,mcOneTimeStep);
            
                numMethodInfo.timeStepsInfo = outInfo;

                % mc random number init
                %pathSize = 2^16;
                pathSize = numMethod.pathSize;
                
                pathSize = 1;
                
%                 % generate random numbers and modelstates forward
%                 rng('default');
%                 rng(0);
% 
                numOfFactors = model.numOfFactor;
% 
%                 %Z= randn(numOfFactors*outInfo.totalTimeStepsSize,pathSize/2);
%                 Z= randn(pathSize/2,numOfFactors*outInfo.totalTimeStepsSize);
%                 Z=[Z;-Z];
%                 Z=Z';
%                 
%                 
%                 %TimeInversion
%                 sizeZ = size(Z,1);
%                 A = zeros(sizeZ,sizeZ);
%                 for i=1:sizeZ
%                     A(i,sizeZ + 1 -i) = 1;
%                 end
%                 Z = A*Z;
% 
%                 % we check with the c, so we use generated random numbers
% %                 mc_random = importdata('mc_random.xlsx');
% %                 mc_random = importdata('D:\mc_random_number_new_report.txt');
% %                 Z= mc_random;
%                 
                numMethodInfo.pathSize = pathSize;
                numMethodInfo.numOfFactors = numOfFactors;
                numMethodInfo.totalTimeSteps = outInfo.totalTimeSteps;
                numMethodInfo.totalTimeStepsSize = outInfo.totalTimeStepsSize;
%                 numMethodInfo.Z = Z;
% 
% 
% 
%                 %generate model states for the given timeSteps
%                 modelStates = numMethod.generateModelStates(valueDate,schedule,numMethodInfo,model);

                % numeraire info
                numMatTime = numMethodInfo.totalTimeSteps(end);
                dfNumMat = model.DFRisky(numMatTime/365.0);
                
                % induct backward and calculate cashflow
            
                periodCount = schedule.numOfPeriod;
                
                % cashflow columns initialize
                % 1st nonCall
                nonCall.npv = 0;
                nonCall.cashflow_e = zeros(periodCount + 1,1);
                nonCall.cashflow_npv_e = zeros(periodCount + 1,1);
                nonCall.rangeProb_e = zeros(periodCount + 1,1);
                nonCall.df_e = zeros(periodCount + 1,1);
                nonCall.df_c = zeros(periodCount + 1,1);
                
                nonCall.fwdCD = zeros(periodCount + 1,1);
                nonCall.fwdKTB = zeros(periodCount + 1,1);
                nonCall.fwdSpread = zeros(periodCount + 1,1);
                nonCall.fwdMu = zeros(periodCount + 1,1);
                nonCall.fwdSigma = zeros(periodCount + 1,1);

                % debug information, cashflow & npv per path
                nonCall.payoff = zeros(1,pathSize);
                nonCall.payoffStates.cashflow = zeros(periodCount+1,pathSize);
                nonCall.payoffStates.cashflow_npv = zeros(periodCount+1,pathSize);
                nonCall.payoffStates.df = zeros(periodCount + 1,pathSize);
                
                % intermediate variables
                payoffStateA.cashflow = zeros(1, pathSize);
                payoffStateA.cashflow_npv = zeros(1, pathSize);
                payoffStateA.df = zeros(1,pathSize);
                
                % we calculate cashflow backward from maturityDate to valueDate
                % rangeAccrual coupons are added at the startDate of the Period
                % At maturity

                nominal = security.nominal;
                coupon = security.coupon;
                callYN = security.callYN;
                
                currentTime = numMethodInfo.totalTimeSteps(end);
                totalTimeSteps = numMethodInfo.timeStepsInfo.totalTimeSteps;
                totalTimeStepsSize = numMethodInfo.timeStepsInfo.totalTimeStepsSize;
                if totalTimeStepsSize == 0 || (totalTimeStepsSize == 1 && totalTimeSteps(1) == 0) 
                    out.nonCall = nonCall;
                    return;
                end
                
                currentTimeIdx = totalTimeStepsSize;
                
                currentNodeIdx = periodCount + 1;
                
%                 modelStateT = modelStates(numOfFactors*(totalTimeStepsSize-1)+1:numOfFactors*(totalTimeStepsSize-1)+2,:);
                payoffStateA.cashflow = nominal*ones(1,pathSize);
                %process payoff at maturity
                nonCall.payoffStates.cashflow(currentNodeIdx,:) = payoffStateA.cashflow;
                
%                 payoffStateA.cashflow_npv = model.discountPayoff(currentTime,numMatTime ...
%                     ,modelStateT,payoffStateA.cashflow);
%                 
                payoffStateA.cashflow_npv = model.DFRisky(currentTime/365.0)*payoffStateA.cashflow;
                
                nonCall.payoffStates.cashflow_npv(currentNodeIdx,:) = payoffStateA.cashflow_npv ;
                
%                 payoffStateA.df = model.discountPayoff(currentTime,numMatTime ...
%                     ,modelStateT,1.0*ones(1,pathSize));
                payoffStateA.df = model.DFRisky(currentTime/365.0) *ones(1,pathSize);
                    

                nonCall.payoffStates.df(currentNodeIdx,:) = payoffStateA.df;
                nonCall.payoff = nonCall.payoff + payoffStateA.cashflow_npv;
                
                nonCall.cashflow_e(currentNodeIdx) = mean(payoffStateA.cashflow);
                nonCall.cashflow_npv_e(currentNodeIdx) = mean(payoffStateA.cashflow_npv);
                nonCall.df_e(currentNodeIdx) = mean(payoffStateA.df);
                nonCall.df_c(currentNodeIdx) = model.DFRisky(currentTime/365.0);
                
%                 % callable at maturity is the same as noncallable
%                 callable.payoffStates.cashflow(currentNodeIdx,:) = nonCall.payoffStates.cashflow(currentNodeIdx,:);
%                 callable.payoffStates.cashflow_npv(currentNodeIdx,:) = nonCall.payoffStates.cashflow_npv(currentNodeIdx,:);
%                 callable.payoff = callable.payoff + callable.payoffStates.cashflow_npv(currentNodeIdx,:);
%                 
%                 callable.cashflow_e(currentNodeIdx) = nonCall.cashflow_e(currentNodeIdx);
%                 callable.cashflow_npv_e(currentNodeIdx) = nonCall.cashflow_npv_e(currentNodeIdx);
                  
                lastTimeIdx = currentTimeIdx;
                
                
                % flag for inclusive schedule
%                 simpleYN = false;
                simpleYN = 1;
                
                for i=periodCount:-1:1
                    % past schedule neglect
                    if DateDiff(valueDate,schedule.payDates(i)) >= 0, continue;end 
                    startTimeIdx = numMethodInfo.timeStepsInfo.startTimeIdx(i);
                    endTimeIdx = numMethodInfo.timeStepsInfo.endTimeIdx(i);

                    inclusiveYN = numMethodInfo.timeStepsInfo.inclusiveYN(i);
                     %% FromToInduction start

                    %modelsStates induct backward(lastTimeIdx to endTimeIdx)
                    while endTimeIdx < lastTimeIdx
                        lastTimeIdx = lastTimeIdx -1;
                        if lastTimeIdx == endTimeIdx break;end
                    end

                    currentTimeIdx = lastTimeIdx;
                    currentTime = totalTimeSteps(currentTimeIdx);
                    % now we are at endTimeIdx
                    % variables for coupon valuation initialized
                    ralower = security.lowerS;
                    raupper = security.upperS;
                    tenorLong = security.tenorL;
                    tenorShort = security.tenorS;

                    cD = 0; % cumulated day count fraction
                    cP = 0; 
                    dI = 0; % dummy idx for loop
                    dcfC = 0;
                    dcfN = 0;
                    dD = 0;
                    idxL = zeros(1,pathSize);
                    idxS = zeros(1,pathSize);
                    spreadC = zeros(1,pathSize);
                    spreadN = zeros(1,pathSize);
                    probSpread = zeros(1,pathSize);
                    
                    numOfTimeSteps = numMethodInfo.timeStepsInfo.numOfTimeSteps(i);
                    
                    % if inclusive then we use simpleYN rangeProb
                    %if inclusive then we do not evaluate the last step but
                    % use fixing instead 
                    endTimeStep = 1;
                    if inclusiveYN
%                         simpleYN = true;
                        simpleYN = 1;
                        endTimeStep = 2;
                    end
                    
                    for j=numOfTimeSteps:-1:endTimeStep
                        % evaluate dealdescription
                        % last step is omitted
                        currentTime = totalTimeSteps(currentTimeIdx);
                        %dcfC = currentTime/365.0;
                        dcfC = currentTime;
                        
                        if j~= numOfTimeSteps
                            dD = (dcfN-dcfC);
                        end

                        toDate = AddDate(valueDate,currentTime,'day');
                        tpDate = schedule.payDates(i);
                        spreadCOut = model.CMSCMTSpreadDigitalRangeDate(valueDate,toDate,tpDate,tenorLong,tenorShort,ralower,raupper);
                        spreadC = spreadCOut.value;
                        if j== endTimeStep
                            fwdCD = spreadCOut.cms_fwd;
                            fwdKTB = spreadCOut.cmt_fwd;
                            fwdSpread = spreadCOut.fwdSpread;
                            fwdMu        = spreadCOut.mu;
                            fwdSigma     = spreadCOut.sigma;
                        end
                        %modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:numOfFactors*(currentTimeIdx-1)+2,:);
%                         idxL = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorLong,modelStateT);
%                         idxS = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorShort,modelStateT);
%                         spreadC = idxL - idxS;
%                         probSpread = model.SRPr(spreadC,spreadN,ralower,raupper,simpleYN,pathSize)*dD; 

                        if simpleYN
                            probSpread = spreadC*dD;
                        else
                            probSpread = spreadC*dD;
%                             probSpread = 0.5*(spreadC + spreadN)*dD;
                            
                        end

                        
                        cD = cD + dD;
                        cP = cP + probSpread;

                        dcfN = dcfC;
                        spreadN = spreadC;

                        %induct backward to the next time step
                        % one time backward induction
                        lastTimeIdx = currentTimeIdx;

                        if startTimeIdx < lastTimeIdx
                            lastTimeIdx = lastTimeIdx -1;
                            currentTimeIdx = lastTimeIdx;
                        end


                    end

                    %% FromToInduction end
                    %% AtDate evaluation
                    % Adding a new cashflow to the payoff

                    dcf = schedule.dayCountFraction(i);
                    
                    if inclusiveYN
                        % we add valuation date
                        cD = schedule.dayCount(i);
                        rangeDays =  security.rangeDays(i);
                        payoffStateA.cashflow = nominal * coupon * dcf * (rangeDays + cP)/cD;
                    else
                        payoffStateA.cashflow = nominal * coupon * dcf * cP/cD;
                    end
                    % eventDate(i) = startDate(i) therefore there is no induction in modelStates
                    % i.e. currentTimeIdx has not changed
                    % no need to update modelStateT
                    % modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:end,:);
                    % since cashflow's are added at the startDate(i) we need to
                    % multiply cashflow with the df(startDate(i), payDate(i))
                    payTime = DateDiff(schedule.payDates(i),valueDate);
                    
                    %event is at startDate so we need to discount the
                    %cashflow first from paydate to startdate
%                     df_sp = model.discountFactor(currentTime, payTime,numMatTime,modelStateT);

%                     for k=1:pathSize
%                         payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_sp(k);
%                     end
                    
                    % process payoff
                    
                    currentNodeIdx = i;
                    
                    % 1st nonCall column
                    nonCall.payoffStates.cashflow(currentNodeIdx,:) = payoffStateA.cashflow;
                    payoffStateA.cashflow_npv = payoffStateA.cashflow * model.DFRisky(payTime/365.0);
                    
%                     payoffStateA.cashflow_npv = model.discountPayoff(currentTime,numMatTime ...
%                     ,modelStateT,payoffStateA.cashflow);
                    nonCall.payoffStates.cashflow_npv(currentNodeIdx,:) = payoffStateA.cashflow_npv;
                    payoffStateA.df = model.DFRisky(payTime/365.0);
%                     payoffStateA.df = model.discountPayoff(currentTime,numMatTime ...
%                     ,modelStateT,1.0*ones(1,pathSize));
                    nonCall.payoffStates.df(currentNodeIdx,:) = payoffStateA.df;
                    
                    nonCall.payoff = nonCall.payoff + payoffStateA.cashflow_npv;
                    
                    nonCall.cashflow_e(currentNodeIdx) = mean(payoffStateA.cashflow);
                    nonCall.rangeProb_e(currentNodeIdx) = mean(nonCall.cashflow_e(currentNodeIdx)/(nominal * coupon * dcf));
                    nonCall.cashflow_npv_e(currentNodeIdx) = mean(payoffStateA.cashflow_npv);

                    nonCall.df_e(currentNodeIdx) = mean(payoffStateA.df);
                    nonCall.df_c(currentNodeIdx) = model.DFRisky(currentTime/365.0);
                    
                    nonCall.fwdCD(currentNodeIdx) = fwdCD;
                    nonCall.fwdKTB(currentNodeIdx) = fwdKTB;
                    
                    nonCall.fwdSpread(currentNodeIdx) = fwdSpread;
                    nonCall.fwdMu(currentNodeIdx) = fwdMu;
                    nonCall.fwdSigma(currentNodeIdx) = fwdSigma;

                end

                nonCall.npv = mean(nonCall.payoff);
                out.pricerInfo.nonCall = nonCall;
                
%                 callable.npv = mean(callable.payoff);
%                 out.pricerInfo.callable = callable;
                out.daycountFraction = schedule.dayCountFraction;
                
        end
        

        
        
        
        
    end
    
end



In [ ]:
classdef DailyWeeklyRiskyPricer < H_RootObject
    %Pricer(Security,Model,NumMethod)
    %Prcer is constructed from Security, Model, NumMethod 
    
    properties
        m_security
        m_model
        m_numMethod
        overHedgeYN
    end
    
    methods
        function pricer = DailyWeeklyRiskyPricer(security,model,numMethod,overHedgeYN)
                pricer.m_security = security;
                pricer.m_model = model;
                pricer.m_numMethod = numMethod;
                pricer.overHedgeYN = overHedgeYN;
        end
        
        function out = computePriceMCCallable(pricer,valueDate)
                security = pricer.m_security;
                numMethod = pricer.m_numMethod;
                model = pricer.m_model;
                schedule = security.couponSchedule;
                callSchedule = security.callSchedule;
                
                % numMethodTimeSteps Generate
                mcOneTimeStep = numMethod.mcOneTimeStep;
                outInfo = numMethod.numMethodTimeStepsGenerateDW(valueDate,schedule,mcOneTimeStep);
            
                numMethodInfo.timeStepsInfo = outInfo;

                % mc random number init
                %pathSize = 2^16;
                pathSize = numMethod.pathSize;

                % generate random numbers and modelstates forward
                rng('default');
                rng(0);

                numOfFactors = 2;

                %Z= randn(numOfFactors*outInfo.totalTimeStepsSize,pathSize/2);
                Z= randn(pathSize/2,numOfFactors*outInfo.totalTimeStepsSize);
                Z=[Z;-Z];
                Z=Z';
                
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;

                % we check with the c code, so we use generated random numbers
%                 mc_random = importdata('mc_random.xlsx');
%                 mc_random = importdata('D:\mc_random_number_new_report.txt');
%                 Z= mc_random;
                
                numMethodInfo.pathSize = pathSize;
                numMethodInfo.numOfFactors = numOfFactors;
                numMethodInfo.totalTimeSteps = outInfo.totalTimeSteps;
                numMethodInfo.totalTimeStepsSize = outInfo.totalTimeStepsSize;
                numMethodInfo.Z = Z;



                %generate model states for the given timeSteps
                modelStates = numMethod.generateModelStates(valueDate,schedule,numMethodInfo,model);

                % numeraire info
                numMatTime = numMethodInfo.totalTimeSteps(end);
                days = DateDiff(schedule.payDates(schedule.numOfPeriod),valueDate);
                numMatTime = max(numMatTime, days);
                dfNumMat = model.DFRisky(numMatTime/365.0);
                
                numeraire.numMatTime = numMatTime;
                numeraire.dfNumMat = dfNumMat;
                
                % induct backward and calculate cashflow
            
                periodCount = schedule.numOfPeriod;
                
                % pricer single info initialize
                
                nonCall = PricerInfo(periodCount +1, pathSize);
                callable = PricerInfo(periodCount +1, pathSize);
                dfCol = PricerInfo(periodCount +1, pathSize);
                
                dummy1 = PricerInfo(periodCount +1, pathSize);
                dummy2 = PricerInfo(periodCount +1, pathSize);
                
                
                % intermediate variables
                payoffStateA.cashflow = zeros(1, pathSize);
                payoffStateA.cashflow_npv = zeros(1, pathSize);
                
            
                % we calculate cashflow backward from maturityDate to valueDate
                % rangeAccrual coupons are added at the startDate of the Period
                % At maturity

                nominal = security.nominal;
                coupon = security.coupon;
                callYN = security.callYN;
                
                ralower = security.lower;
                raupper = security.upper;
                tenorLong = security.tenorL;
                tenorShort = security.tenorS;
                
                currentTime = numMethodInfo.totalTimeSteps(end);
                totalTimeStepsSize = numMethodInfo.totalTimeStepsSize;
                currentTimeIdx = totalTimeStepsSize;
                
                currentNodeIdx = periodCount + 1;
                
                modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:numOfFactors*(currentTimeIdx-1)+2,:);
                payoffStateA.cashflow = nominal*ones(1,pathSize);
                
                payTime = DateDiff(schedule.payDates(schedule.numOfPeriod),valueDate);
                df_ep = model.discountFactorRisky(currentTime, payTime,numMatTime,modelStateT);
                for k=1:pathSize
                    payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                end
                
                %process payoff at maturity
                nonCall.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                
                callable.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                
                %df column
                payoffStateA.cashflow = ones(1,pathSize);
                for k=1:pathSize
                    payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                end
                
                dfCol.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                                 
                %dummy column
                payoffStateA.cashflow = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorLong,modelStateT);
                dummy1.ProcessPayoff(model,currentTime,numeraire, ...
                                 modelStateT,payoffStateA.cashflow,currentNodeIdx);

                payoffStateA.cashflow = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                dummy2.ProcessPayoff(model,currentTime,numeraire, ...
                                 modelStateT,payoffStateA.cashflow,currentNodeIdx);
                                 
                  

                lastTimeIdx = currentTimeIdx;
                totalTimeSteps = numMethodInfo.timeStepsInfo.totalTimeSteps;
                
                % flag for inclusive schedule
%                 simpleYN = false;
%                 simpleYN = true;
                if pricer.overHedgeYN == true 
                    simpleYN = 2;
                else
                    simpleYN = 1;
                end
                
                for i=periodCount:-1:1
                    % past schedule neglect
                    if DateDiff(valueDate,schedule.payDates(i)) >= 0, continue;end 
                    startTimeIdx = numMethodInfo.timeStepsInfo.startTimeIdx(i);
                    endTimeIdx = numMethodInfo.timeStepsInfo.endTimeIdx(i);

                    inclusiveYN = numMethodInfo.timeStepsInfo.inclusiveYN(i);
                 %% FromToInduction start

                    %modelsStates induct backward(lastTimeIdx to endTimeIdx)
                    while endTimeIdx < lastTimeIdx
                        lastTimeIdx = lastTimeIdx -1;
                        if lastTimeIdx == endTimeIdx break;end
                    end

                    currentTimeIdx = lastTimeIdx;
                    currentTime = totalTimeSteps(currentTimeIdx);
                    % now we are at endTimeIdx
                    % variables for coupon valuation initialized
                    

                    cD = 0; % cumulated day count fraction
                    cP = 0; 
                    dI = 0; % dummy idx for loop
                    dcfC = 0;
                    dcfN = 0;
                    dD = 0;
                    idxL = zeros(1,pathSize);
                    idxS = zeros(1,pathSize);
                    spreadC = zeros(1,pathSize);
                    spreadN = zeros(1,pathSize);
                    probSpread = zeros(1,pathSize);
                    
                    numOfTimeSteps = numMethodInfo.timeStepsInfo.numOfTimeSteps(i);
                    
                    % if inclusive then we use simpleYN rangeProb
                    %if inclusive then we do not evaluate the last step but
                    % use fixing instead 
                    endTimeStep = 1;
                    if inclusiveYN
%                         simpleYN = true;
                        simpleYN = 1;
                        endTimeStep = 2;
                        if pricer.overHedgeYN == true 
                            simpleYN = 2;
                        end
                    end
                    
                    for j=numOfTimeSteps:-1:endTimeStep
                        % evaluate dealdescription
                        % last step is omitted
                        currentTime = totalTimeSteps(currentTimeIdx);
                        %dcfC = currentTime/365.0;
                        dcfC = currentTime;
                        
                        if j~= numOfTimeSteps
                            dD = (dcfN-dcfC);
                        end

                        modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:numOfFactors*(currentTimeIdx-1)+2,:);
                        idxL = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorLong,modelStateT);
                        idxS = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                        spreadC = idxL - idxS;

                        probSpread = model.SRPr(spreadC,spreadN,ralower,raupper,simpleYN,pathSize)*dD;
                        cD = cD + dD;
                        cP = cP + probSpread;

                        dcfN = dcfC;
                        spreadN = spreadC;

                        %induct backward to the next time step
                        % one time backward induction
                        lastTimeIdx = currentTimeIdx;

                        if startTimeIdx < lastTimeIdx
                            lastTimeIdx = lastTimeIdx -1;
                            currentTimeIdx = lastTimeIdx;
                        end


                    end

                    %% FromToInduction end
        % if the current period is inclusive, then the start date is not evaluated in the above for loop.
		% instead the fixing will be treated separately below.
		% however current_time_idx is updated at the end of the for loop.
		% so we need to update model_states accordingly.
		% if model_states were updated previously then the following additionaly step doesn't affect the result anyway.
                    currentTime = totalTimeSteps(currentTimeIdx);
                    modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:numOfFactors*(currentTimeIdx-1)+2,:);
                    
                    %% AtDate evaluation
                    % Adding a new cashflow to the payoff

                    dcf = schedule.dayCountFraction(i);
                    
                    if inclusiveYN
                        % we add valuation date
                        cD = schedule.dayCount(i);
                        rangeDays =  security.rangeDays(i);
                        payoffStateA.cashflow = nominal * coupon * dcf * (rangeDays + cP)/cD;
                    else
                        payoffStateA.cashflow = nominal * coupon * dcf * cP/cD;
                    end
                    % eventDate(i) = startDate(i) therefore there is no induction in modelStates
                    % i.e. currentTimeIdx has not changed
                    % no need to update modelStateT
                    % modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:end,:);
                    % since cashflow's are added at the startDate(i) we need to
                    % multiply cashflow with the df(startDate(i), payDate(i))
                    payTime = DateDiff(schedule.payDates(i),valueDate);
                    
                    %event is at startDate so we need to discount the
                    %cashflow first from paydate to startdate
                    df_ep = model.discountFactorRisky(currentTime, payTime,numMatTime,modelStateT);

                    for k=1:pathSize
                        payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                    end
                    
                    % process payoff
                    
                    currentNodeIdx = i;
                    
                    nonCall.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                
                    callable.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                    
                    %df column
                    payoffStateA.cashflow = ones(1,pathSize);
                    for k=1:pathSize
                        payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                    end
                    
                    dfCol.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                                 
                    %dummy column
                    payoffStateA.cashflow = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorLong,modelStateT);
                    dummy1.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                    
                    payoffStateA.cashflow = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                    dummy2.ProcessPayoff(model,currentTime,numeraire, ...
                                     modelStateT,payoffStateA.cashflow,currentNodeIdx);
                    
                    
                    % if exercise is possible
                    if i == 1
                        dummaaa = 1;
                    end
                    
                    % call event happens  at the start of the period
                    % at call date, by default, call option is expired
                    
                    if callYN(i) && (DateDiff(valueDate,callSchedule.resetDates(i)) < 0)
                        % 1.undiscount callable.payoff
                        % 2.build the state variables matrix
                        % 3.regress 1's with the given state variables
                        % (model states 1,2)
                        % 4. exercise per each path in
                        % discount the new payoff and assign into callable
                        % column
                        
                        %1 undiscount callable.payoff and get the
                        %contiValue
                       
                        %exercise value
                        exerciseValue = security.nominal*ones(1,pathSize);
                        payTime = DateDiff(callSchedule.payDates(i),valueDate);
                        %event is at startDate so we need to discount the
                        %cashflow first from paydate to startdate
                        df_ep = model.discountFactorRisky(currentTime, payTime,numMatTime,modelStateT);
                        for k=1:pathSize
                            exerciseValue(k) = exerciseValue(k) * df_ep(k);
                        end
                        
                        contiValue = zeros(1,pathSize);
                        onePayoff = ones(1,pathSize);
                        onePayoffD = model.discountPayoffRisky(currentTime,numMatTime ...
                        ,modelStateT,onePayoff);
                        contiValue = callable.payoff./onePayoffD;
                        
                        %Nominal Regression test
                        contiValue = contiValue/10000;
                        
                        % build the state variabls matrix
                        decisionVar1 = modelStateT(1,:);
                        decisionVar2 = modelStateT(2,:);
                        % we use upto 2nd order polynomial as regression
                        % equatioin
                        dimensionLS = 6;
                        regMatrix = ones(pathSize,dimensionLS);
                        
                        for idx_r=1:pathSize
                            regMatrix(idx_r,2) = decisionVar1(idx_r);
                            regMatrix(idx_r,3) = decisionVar2(idx_r);
                            regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r);
                            regMatrix(idx_r,5) = decisionVar1(idx_r)*decisionVar2(idx_r);
                            regMatrix(idx_r,6) = decisionVar2(idx_r)*decisionVar2(idx_r);
                        end
                        
                        % simple regression <- for path dependent option
                        % we should change it
                        % beta  : regression coefficients
                        beta =zeros(dimensionLS,1);
                        beta = regress(contiValue',regMatrix);
                        regContiValue = regMatrix*beta;
                       
                         %Nominal Regression test
                        regContiValue = regContiValue * 10000;
                        
                        exercisedPayoff = regContiValue;
                        % if regContiValue is bigger than exerciseValue
                        % then issue exercise call option
                        for idx_r=1:pathSize
                            if regContiValue(idx_r) >= exerciseValue(idx_r) 
                                exercisedPayoff(idx_r) = exerciseValue(idx_r);
                            end
                        end
                        
                        % we discount the new payoff and 
                        % assign it into the existing callable payoff
                        callable.payoff = model.discountPayoffRisky(currentTime,numMatTime ...
                        ,modelStateT,exercisedPayoff);
                        
                    end
                end

                nonCall.npv = mean(nonCall.payoff);
                out.nonCall = nonCall;
                callable.npv = mean(callable.payoff);
                out.callable = callable;
                out.dfCol = dfCol;
                out.dummy1 = dummy1;
                out.dummy2 = dummy2;
                out.daycountFraction = schedule.dayCountFraction;
                
        end
        
        function out = computePriceClosedTRZ(pricer,valueDate)
                security = pricer.m_security;
                numMethod = pricer.m_numMethod;
                model = pricer.m_model;
                schedule = security.couponSchedule;
                
                % numMethodTimeSteps Generate
                mcOneTimeStep = numMethod.mcOneTimeStep;
                outInfo = numMethod.numMethodTimeStepsGenerateDW(valueDate,schedule,mcOneTimeStep);
            
                numMethodInfo.timeStepsInfo = outInfo;

                % mc random number init
                %pathSize = 2^16;
                pathSize = numMethod.pathSize;
                
                pathSize = 1;
                
%                 % generate random numbers and modelstates forward
%                 rng('default');
%                 rng(0);
% 
                numOfFactors = 2;
% 
%                 %Z= randn(numOfFactors*outInfo.totalTimeStepsSize,pathSize/2);
%                 Z= randn(pathSize/2,numOfFactors*outInfo.totalTimeStepsSize);
%                 Z=[Z;-Z];
%                 Z=Z';
%                 
%                 
%                 %TimeInversion
%                 sizeZ = size(Z,1);
%                 A = zeros(sizeZ,sizeZ);
%                 for i=1:sizeZ
%                     A(i,sizeZ + 1 -i) = 1;
%                 end
%                 Z = A*Z;
% 
%                 % we check with the c, so we use generated random numbers
% %                 mc_random = importdata('mc_random.xlsx');
% %                 mc_random = importdata('D:\mc_random_number_new_report.txt');
% %                 Z= mc_random;
%                 
                numMethodInfo.pathSize = pathSize;
                numMethodInfo.numOfFactors = numOfFactors;
                numMethodInfo.totalTimeSteps = outInfo.totalTimeSteps;
                numMethodInfo.totalTimeStepsSize = outInfo.totalTimeStepsSize;
%                 numMethodInfo.Z = Z;
% 
% 
% 
%                 %generate model states for the given timeSteps
%                 modelStates = numMethod.generateModelStates(valueDate,schedule,numMethodInfo,model);

                % numeraire info
                numMatTime = numMethodInfo.totalTimeSteps(end);
                dfNumMat = model.DF(numMatTime/365.0);
                
                % induct backward and calculate cashflow
            
                periodCount = schedule.numOfPeriod;
                
                % cashflow columns initialize
                % 1st nonCall
                nonCall.npv = 0;
                nonCall.cashflow_e = zeros(periodCount + 1,1);
                nonCall.cashflow_npv_e = zeros(periodCount + 1,1);
                nonCall.rangeProb_e = zeros(periodCount + 1,1);
                nonCall.df_e = zeros(periodCount + 1,1);
                nonCall.df_c = zeros(periodCount + 1,1);

                % debug information, cashflow & npv per path
                nonCall.payoff = zeros(1,pathSize);
                nonCall.payoffStates.cashflow = zeros(periodCount+1,pathSize);
                nonCall.payoffStates.cashflow_npv = zeros(periodCount+1,pathSize);
                nonCall.payoffStates.df = zeros(periodCount + 1,pathSize);
                
                % intermediate variables
                payoffStateA.cashflow = zeros(1, pathSize);
                payoffStateA.cashflow_npv = zeros(1, pathSize);
                payoffStateA.df = zeros(1,pathSize);
                
                % we calculate cashflow backward from maturityDate to valueDate
                % rangeAccrual coupons are added at the startDate of the Period
                % At maturity

                nominal = security.nominal;
                coupon = security.coupon;
                callYN = security.callYN;
                
                currentTime = numMethodInfo.totalTimeSteps(end);
                totalTimeSteps = numMethodInfo.timeStepsInfo.totalTimeSteps;
                totalTimeStepsSize = numMethodInfo.timeStepsInfo.totalTimeStepsSize;
                if totalTimeStepsSize == 0 || (totalTimeStepsSize == 1 && totalTimeSteps(1) == 0) 
                    out.nonCall = nonCall;
                    return;
                end
                
                currentTimeIdx = totalTimeStepsSize;
                
                currentNodeIdx = periodCount + 1;
                
%                 modelStateT = modelStates(numOfFactors*(totalTimeStepsSize-1)+1:numOfFactors*(totalTimeStepsSize-1)+2,:);
                payoffStateA.cashflow = nominal*ones(1,pathSize);
                %process payoff at maturity
                nonCall.payoffStates.cashflow(currentNodeIdx,:) = payoffStateA.cashflow;
                
%                 payoffStateA.cashflow_npv = model.discountPayoff(currentTime,numMatTime ...
%                     ,modelStateT,payoffStateA.cashflow);
%                 
                payoffStateA.cashflow_npv = model.DFRisky(currentTime/365.0)*payoffStateA.cashflow;
                
                nonCall.payoffStates.cashflow_npv(currentNodeIdx,:) = payoffStateA.cashflow_npv ;
                
%                 payoffStateA.df = model.discountPayoff(currentTime,numMatTime ...
%                     ,modelStateT,1.0*ones(1,pathSize));
                payoffStateA.df = model.DFRisky(currentTime/365.0) *ones(1,pathSize);
                    

                nonCall.payoffStates.df(currentNodeIdx,:) = payoffStateA.df;
                nonCall.payoff = nonCall.payoff + payoffStateA.cashflow_npv;
                
                nonCall.cashflow_e(currentNodeIdx) = mean(payoffStateA.cashflow);
                nonCall.cashflow_npv_e(currentNodeIdx) = mean(payoffStateA.cashflow_npv);
                nonCall.df_e(currentNodeIdx) = mean(payoffStateA.df);
                nonCall.df_c(currentNodeIdx) = model.DFRisky(currentTime/365.0);
                
%                 % callable at maturity is the same as noncallable
%                 callable.payoffStates.cashflow(currentNodeIdx,:) = nonCall.payoffStates.cashflow(currentNodeIdx,:);
%                 callable.payoffStates.cashflow_npv(currentNodeIdx,:) = nonCall.payoffStates.cashflow_npv(currentNodeIdx,:);
%                 callable.payoff = callable.payoff + callable.payoffStates.cashflow_npv(currentNodeIdx,:);
%                 
%                 callable.cashflow_e(currentNodeIdx) = nonCall.cashflow_e(currentNodeIdx);
%                 callable.cashflow_npv_e(currentNodeIdx) = nonCall.cashflow_npv_e(currentNodeIdx);
                  
                lastTimeIdx = currentTimeIdx;
                
                
                % flag for inclusive schedule
%                 simpleYN = false;
                simpleYN = 1;
                
                for i=periodCount:-1:1
                    % past schedule neglect
                    if DateDiff(valueDate,schedule.payDates(i)) >= 0, continue;end 
                    startTimeIdx = numMethodInfo.timeStepsInfo.startTimeIdx(i);
                    endTimeIdx = numMethodInfo.timeStepsInfo.endTimeIdx(i);

                    inclusiveYN = numMethodInfo.timeStepsInfo.inclusiveYN(i);
                     %% FromToInduction start

                    %modelsStates induct backward(lastTimeIdx to endTimeIdx)
                    while endTimeIdx < lastTimeIdx
                        lastTimeIdx = lastTimeIdx -1;
                        if lastTimeIdx == endTimeIdx break;end
                    end

                    currentTimeIdx = lastTimeIdx;
                    currentTime = totalTimeSteps(currentTimeIdx);
                    % now we are at endTimeIdx
                    % variables for coupon valuation initialized
                    ralower = security.lower;
                    raupper = security.upper;
                    tenorLong = security.tenorL;
                    tenorShort = security.tenorS;

                    cD = 0; % cumulated day count fraction
                    cP = 0; 
                    dI = 0; % dummy idx for loop
                    dcfC = 0;
                    dcfN = 0;
                    dD = 0;
                    idxL = zeros(1,pathSize);
                    idxS = zeros(1,pathSize);
                    spreadC = zeros(1,pathSize);
                    spreadN = zeros(1,pathSize);
                    probSpread = zeros(1,pathSize);
                    
                    numOfTimeSteps = numMethodInfo.timeStepsInfo.numOfTimeSteps(i);
                    
                    % if inclusive then we use simpleYN rangeProb
                    %if inclusive then we do not evaluate the last step but
                    % use fixing instead 
                    endTimeStep = 1;
                    if inclusiveYN
%                         simpleYN = true;
                        simpleYN = 1;
                        endTimeStep = 2;
                    end
                    
                    for j=numOfTimeSteps:-1:endTimeStep
                        % evaluate dealdescription
                        % last step is omitted
                        currentTime = totalTimeSteps(currentTimeIdx);
                        %dcfC = currentTime/365.0;
                        dcfC = currentTime;
                        
                        if j~= numOfTimeSteps
                            dD = (dcfN-dcfC);
                        end

                        toDate = AddDate(valueDate,currentTime,'day');
                        tpDate = schedule.payDates(i);
%                         spreadC = model.CMSSpreadDigitalRangeDate(valueDate,toDate,tpDate,tenorLong,tenorShort,0.25,ralower,raupper);
                        if isfield(model,'tau')
                            spreadC = model.CMSSpreadDigitalRangeDate(valueDate,toDate,tpDate,tenorLong,tenorShort,model.tau,ralower,raupper);
                        else
                            spreadC = model.CMSSpreadDigitalRangeDate(valueDate,toDate,tpDate,tenorLong,tenorShort,0.25,ralower,raupper);
                        
                        end
                        %modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:numOfFactors*(currentTimeIdx-1)+2,:);
%                         idxL = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorLong,modelStateT);
%                         idxS = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorShort,modelStateT);
%                         spreadC = idxL - idxS;
%                         probSpread = model.SRPr(spreadC,spreadN,ralower,raupper,simpleYN,pathSize)*dD; 

                        if simpleYN
                            probSpread = spreadC*dD;
                        else
                            probSpread = spreadC*dD;
%                             probSpread = 0.5*(spreadC + spreadN)*dD;
                            
                        end

                        
                        cD = cD + dD;
                        cP = cP + probSpread;

                        dcfN = dcfC;
                        spreadN = spreadC;

                        %induct backward to the next time step
                        % one time backward induction
                        lastTimeIdx = currentTimeIdx;

                        if startTimeIdx < lastTimeIdx
                            lastTimeIdx = lastTimeIdx -1;
                            currentTimeIdx = lastTimeIdx;
                        end


                    end

                    %% FromToInduction end
                    %% AtDate evaluation
                    % Adding a new cashflow to the payoff

                    dcf = schedule.dayCountFraction(i);
                    
                    if inclusiveYN
                        % we add valuation date
                        cD = schedule.dayCount(i);
                        rangeDays =  security.rangeDays(i);
                        payoffStateA.cashflow = nominal * coupon * dcf * (rangeDays + cP)/cD;
                    else
                        payoffStateA.cashflow = nominal * coupon * dcf * cP/cD;
                    end
                    % eventDate(i) = startDate(i) therefore there is no induction in modelStates
                    % i.e. currentTimeIdx has not changed
                    % no need to update modelStateT
                    % modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:end,:);
                    % since cashflow's are added at the startDate(i) we need to
                    % multiply cashflow with the df(startDate(i), payDate(i))
                    payTime = DateDiff(schedule.payDates(i),valueDate);
                    
                    %event is at startDate so we need to discount the
                    %cashflow first from paydate to startdate
%                     df_sp = model.discountFactor(currentTime, payTime,numMatTime,modelStateT);

%                     for k=1:pathSize
%                         payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_sp(k);
%                     end
                    
                    % process payoff
                    
                    currentNodeIdx = i;
                    
                    % 1st nonCall column
                    nonCall.payoffStates.cashflow(currentNodeIdx,:) = payoffStateA.cashflow;
                    payoffStateA.cashflow_npv = payoffStateA.cashflow * model.DFRisky(payTime/365.0);
                    
%                     payoffStateA.cashflow_npv = model.discountPayoff(currentTime,numMatTime ...
%                     ,modelStateT,payoffStateA.cashflow);
                    nonCall.payoffStates.cashflow_npv(currentNodeIdx,:) = payoffStateA.cashflow_npv;
                    payoffStateA.df = model.DFRisky(payTime/365.0);
%                     payoffStateA.df = model.discountPayoff(currentTime,numMatTime ...
%                     ,modelStateT,1.0*ones(1,pathSize));
                    nonCall.payoffStates.df(currentNodeIdx,:) = payoffStateA.df;
                    
                    nonCall.payoff = nonCall.payoff + payoffStateA.cashflow_npv;
                    
                    nonCall.cashflow_e(currentNodeIdx) = mean(payoffStateA.cashflow);
                    nonCall.rangeProb_e(currentNodeIdx) = mean(nonCall.cashflow_e(currentNodeIdx)/(nominal * coupon * dcf));
                    nonCall.cashflow_npv_e(currentNodeIdx) = mean(payoffStateA.cashflow_npv);

                    nonCall.df_e(currentNodeIdx) = mean(payoffStateA.df);
                    nonCall.df_c(currentNodeIdx) = model.DFRisky(currentTime/365.0);
                    
%                     %2nd callable column
%                     callable.payoffStates.cashflow(currentNodeIdx,:) = nonCall.payoffStates.cashflow(currentNodeIdx,:);
%                     callable.payoffStates.cashflow_npv(currentNodeIdx,:) = nonCall.payoffStates.cashflow_npv(currentNodeIdx,:);
%                     callable.payoff = callable.payoff + callable.payoffStates.cashflow_npv(currentNodeIdx,:);
%                     
%                     callable.cashflow_e(currentNodeIdx) = nonCall.cashflow_e(currentNodeIdx);
%                     callable.cashflow_npv_e(currentNodeIdx) = nonCall.cashflow_npv_e(currentNodeIdx);

                    % if exercise is possible
                    if i == 1
                        dummaaa = 1;
                    end
                    
                    % call event happens  at the start of the period
                    % at call date, by default, call option is expired
                    
%                     if callYN(i) && (DateDiff(valueDate,schedule.startDates(i)) < 0)
%                         % 1.undiscount callable.payoff
%                         % 2.build the state variables matrix
%                         % 3.regress 1's with the given state variables
%                         % (model states 1,2)
%                         % 4. exercise per each path in
%                         % discount the new payoff and assign into callable
%                         % column
%                         
%                         %1 undiscount callable.payoff and get the
%                         %contiValue
%                        
%                         %exercise value
%                         exerciseValue = security.nominal*ones(1,pathSize);
%                         contiValue = zeros(1,pathSize);
%                         onePayoff = ones(1,pathSize);
%                         onePayoffD = model.discountPayoff(currentTime,numMatTime ...
%                         ,modelStateT,onePayoff);
%                         contiValue = callable.payoff./onePayoffD;
%                         
%                         % build the state variabls matrix
%                         decisionVar1 = modelStateT(1,:);
%                         decisionVar2 = modelStateT(2,:);
%                         % we use upto 2nd order polynomial as regression
%                         % equatioin
%                         dimensionLS = 6;
%                         regMatrix = ones(pathSize,dimensionLS);
%                         
%                         for idx_r=1:pathSize
%                             regMatrix(idx_r,2) = decisionVar1(idx_r);
%                             regMatrix(idx_r,3) = decisionVar2(idx_r);
%                             regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r);
%                             regMatrix(idx_r,5) = decisionVar1(idx_r)*decisionVar2(idx_r);
%                             regMatrix(idx_r,6) = decisionVar2(idx_r)*decisionVar2(idx_r);
%                         end
%                         
%                         % simple regression <- for path dependent option
%                         % we should change it
%                         % beta  : regression coefficients
%                         beta =zeros(dimensionLS,1);
%                         beta = regress(contiValue',regMatrix);
%                         regContiValue = regMatrix*beta;
%                         
%                         exercisedPayoff = regContiValue;
%                         % if regContiValue is bigger than exerciseValue
%                         % then issue exercise call option
%                         for idx_r=1:pathSize
%                             if regContiValue(idx_r) >= exerciseValue(idx_r) 
%                                 exercisedPayoff(idx_r) = exerciseValue(idx_r);
%                             end
%                         end
%                         
%                         % we discount the new payoff and 
%                         % assign it into the existing callable payoff
%                         callable.payoff = model.discountPayoff(currentTime,numMatTime ...
%                         ,modelStateT,exercisedPayoff);
%                         
%                     end
                end

                nonCall.npv = mean(nonCall.payoff);
                out.pricerInfo.nonCall = nonCall;
                
%                 callable.npv = mean(callable.payoff);
%                 out.pricerInfo.callable = callable;
                out.daycountFraction = schedule.dayCountFraction;
                
        end
        
        function out = computePriceClosedGaussianCopola(pricer,termCorr,valueDate)
                security = pricer.m_security;
                numMethod = pricer.m_numMethod;
                model = pricer.m_model;
                schedule = security.couponSchedule;
                
                % numMethodTimeSteps Generate
                mcOneTimeStep = numMethod.mcOneTimeStep;
                outInfo = numMethod.numMethodTimeStepsGenerateDW(valueDate,schedule,mcOneTimeStep);
            
                numMethodInfo.timeStepsInfo = outInfo;

                % mc random number init
                %pathSize = 2^16;
                pathSize = numMethod.pathSize;
                
                pathSize = 1;

                numOfFactors = 2;
                numMethodInfo.pathSize = pathSize;
                numMethodInfo.numOfFactors = numOfFactors;
                numMethodInfo.totalTimeSteps = outInfo.totalTimeSteps;
                numMethodInfo.totalTimeStepsSize = outInfo.totalTimeStepsSize;

                % numeraire info
                numMatTime = numMethodInfo.totalTimeSteps(end);
                dfNumMat = model.DF(numMatTime/365.0);
                
                % induct backward and calculate cashflow
            
                periodCount = schedule.numOfPeriod;
                
                % cashflow columns initialize
                % 1st nonCall
                nonCall.npv = 0;
                nonCall.cashflow_e = zeros(periodCount + 1,1);
                nonCall.cashflow_npv_e = zeros(periodCount + 1,1);
                nonCall.rangeProb_e = zeros(periodCount + 1,1);
                nonCall.df_e = zeros(periodCount + 1,1);
                nonCall.df_c = zeros(periodCount + 1,1);

                % debug information, cashflow & npv per path
                nonCall.payoff = zeros(1,pathSize);
                nonCall.payoffStates.cashflow = zeros(periodCount+1,pathSize);
                nonCall.payoffStates.cashflow_npv = zeros(periodCount+1,pathSize);
                nonCall.payoffStates.df = zeros(periodCount + 1,pathSize);
                
                % intermediate variables
                payoffStateA.cashflow = zeros(1, pathSize);
                payoffStateA.cashflow_npv = zeros(1, pathSize);
                payoffStateA.df = zeros(1,pathSize);
                
                % we calculate cashflow backward from maturityDate to valueDate
                % rangeAccrual coupons are added at the startDate of the Period
                % At maturity

                nominal = security.nominal;
                coupon = security.coupon;
                callYN = security.callYN;
                
                currentTime = numMethodInfo.totalTimeSteps(end);
                totalTimeSteps = numMethodInfo.timeStepsInfo.totalTimeSteps;
                totalTimeStepsSize = numMethodInfo.timeStepsInfo.totalTimeStepsSize;
                if totalTimeStepsSize == 0 || (totalTimeStepsSize == 1 && totalTimeSteps(1) == 0) 
                    out.nonCall = nonCall;
                    return;
                end
                
                currentTimeIdx = totalTimeStepsSize;
                
                currentNodeIdx = periodCount + 1;
                
%                 modelStateT = modelStates(numOfFactors*(totalTimeStepsSize-1)+1:numOfFactors*(totalTimeStepsSize-1)+2,:);
                payoffStateA.cashflow = nominal*ones(1,pathSize);
                %process payoff at maturity
                nonCall.payoffStates.cashflow(currentNodeIdx,:) = payoffStateA.cashflow;
                
%                 payoffStateA.cashflow_npv = model.discountPayoff(currentTime,numMatTime ...
%                     ,modelStateT,payoffStateA.cashflow);
%                 
                payoffStateA.cashflow_npv = model.DFRisky(currentTime/365.0)*payoffStateA.cashflow;
                
                nonCall.payoffStates.cashflow_npv(currentNodeIdx,:) = payoffStateA.cashflow_npv ;
                
%                 payoffStateA.df = model.discountPayoff(currentTime,numMatTime ...
%                     ,modelStateT,1.0*ones(1,pathSize));
                payoffStateA.df = model.DFRisky(currentTime/365.0) *ones(1,pathSize);
                    

                nonCall.payoffStates.df(currentNodeIdx,:) = payoffStateA.df;
                nonCall.payoff = nonCall.payoff + payoffStateA.cashflow_npv;
                
                nonCall.cashflow_e(currentNodeIdx) = mean(payoffStateA.cashflow);
                nonCall.cashflow_npv_e(currentNodeIdx) = mean(payoffStateA.cashflow_npv);
                nonCall.df_e(currentNodeIdx) = mean(payoffStateA.df);
                nonCall.df_c(currentNodeIdx) = model.DFRisky(currentTime/365.0);
                
                  
                lastTimeIdx = currentTimeIdx;
                
                
                % flag for inclusive schedule
%                 simpleYN = false;
                simpleYN = 1;
                
                for i=periodCount:-1:1
                    % past schedule neglect
                    if DateDiff(valueDate,schedule.payDates(i)) >= 0, continue;end 
                    startTimeIdx = numMethodInfo.timeStepsInfo.startTimeIdx(i);
                    endTimeIdx = numMethodInfo.timeStepsInfo.endTimeIdx(i);

                    inclusiveYN = numMethodInfo.timeStepsInfo.inclusiveYN(i);
                     %% FromToInduction start

                    %modelsStates induct backward(lastTimeIdx to endTimeIdx)
                    while endTimeIdx < lastTimeIdx
                        lastTimeIdx = lastTimeIdx -1;
                        if lastTimeIdx == endTimeIdx break;end
                    end

                    currentTimeIdx = lastTimeIdx;
                    currentTime = totalTimeSteps(currentTimeIdx);
                    % now we are at endTimeIdx
                    % variables for coupon valuation initialized
                    ralower = security.lower;
                    raupper = security.upper;
                    tenorLong = security.tenorL;
                    tenorShort = security.tenorS;

                    cD = 0; % cumulated day count fraction
                    cP = 0; 
                    dI = 0; % dummy idx for loop
                    dcfC = 0;
                    dcfN = 0;
                    dD = 0;
                    idxL = zeros(1,pathSize);
                    idxS = zeros(1,pathSize);
                    spreadC = zeros(1,pathSize);
                    spreadN = zeros(1,pathSize);
                    probSpread = zeros(1,pathSize);
                    
                    numOfTimeSteps = numMethodInfo.timeStepsInfo.numOfTimeSteps(i);
                    
                    % if inclusive then we use simpleYN rangeProb
                    %if inclusive then we do not evaluate the last step but
                    % use fixing instead 
                    endTimeStep = 1;
                    if inclusiveYN
%                         simpleYN = true;
                        simpleYN = 1;
                        endTimeStep = 2;
                    end
                    
                    for j=numOfTimeSteps:-1:endTimeStep
                        % evaluate dealdescription
                        % last step is omitted
                        currentTime = totalTimeSteps(currentTimeIdx);
                        %dcfC = currentTime/365.0;
                        dcfC = currentTime;
                        
                        if j~= numOfTimeSteps
                            dD = (dcfN-dcfC);
                        end

                        toDate = AddDate(valueDate,currentTime,'day');
                        tpDate = schedule.payDates(i);
                        if isfield(model,'tau')
%                             spreadC = model.CMSSpreadDigitalRangeDate(valueDate,toDate,tpDate,tenorLong,tenorShort,model.tau,ralower,raupper);
                            spreadC = model.CMSSpreadDigitalRangeGCDate(valueDate,toDate,tpDate,tenorLong,tenorShort,model.tau,ralower,raupper,termCorr);
                       
                        else
%                             spreadC = model.CMSSpreadDigitalRangeDate(valueDate,toDate,tpDate,tenorLong,tenorShort,0.25,ralower,raupper);
                            spreadC = model.CMSSpreadDigitalRangeGCDate(valueDate,toDate,tpDate,tenorLong,tenorShort,0.25,ralower,raupper,termCorr);
                        
                        end

                        if simpleYN
                            probSpread = spreadC*dD;
                        else
                            probSpread = spreadC*dD;
                            
                        end

                        
                        cD = cD + dD;
                        cP = cP + probSpread;

                        dcfN = dcfC;
                        spreadN = spreadC;

                        %induct backward to the next time step
                        % one time backward induction
                        lastTimeIdx = currentTimeIdx;

                        if startTimeIdx < lastTimeIdx
                            lastTimeIdx = lastTimeIdx -1;
                            currentTimeIdx = lastTimeIdx;
                        end


                    end

                    %% FromToInduction end
                    %% AtDate evaluation
                    % Adding a new cashflow to the payoff

                    dcf = schedule.dayCountFraction(i);
                    
                    if inclusiveYN
                        % we add valuation date
                        cD = schedule.dayCount(i);
                        rangeDays =  security.rangeDays(i);
                        payoffStateA.cashflow = nominal * coupon * dcf * (rangeDays + cP)/cD;
                    else
                        payoffStateA.cashflow = nominal * coupon * dcf * cP/cD;
                    end
                    % eventDate(i) = startDate(i) therefore there is no induction in modelStates
                    % i.e. currentTimeIdx has not changed
                    % no need to update modelStateT
                    % modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:end,:);
                    % since cashflow's are added at the startDate(i) we need to
                    % multiply cashflow with the df(startDate(i), payDate(i))
                    payTime = DateDiff(schedule.payDates(i),valueDate);
                    
                    %event is at startDate so we need to discount the
                    %cashflow first from paydate to startdate
%                     df_sp = model.discountFactor(currentTime, payTime,numMatTime,modelStateT);

%                     for k=1:pathSize
%                         payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_sp(k);
%                     end
                    
                    % process payoff
                    
                    currentNodeIdx = i;
                    
                    % 1st nonCall column
                    nonCall.payoffStates.cashflow(currentNodeIdx,:) = payoffStateA.cashflow;
                    payoffStateA.cashflow_npv = payoffStateA.cashflow * model.DFRisky(payTime/365.0);
                    
%                     payoffStateA.cashflow_npv = model.discountPayoff(currentTime,numMatTime ...
%                     ,modelStateT,payoffStateA.cashflow);
                    nonCall.payoffStates.cashflow_npv(currentNodeIdx,:) = payoffStateA.cashflow_npv;
                    payoffStateA.df = model.DFRisky(payTime/365.0);
%                     payoffStateA.df = model.discountPayoff(currentTime,numMatTime ...
%                     ,modelStateT,1.0*ones(1,pathSize));
                    nonCall.payoffStates.df(currentNodeIdx,:) = payoffStateA.df;
                    
                    nonCall.payoff = nonCall.payoff + payoffStateA.cashflow_npv;
                    
                    nonCall.cashflow_e(currentNodeIdx) = mean(payoffStateA.cashflow);
                    nonCall.rangeProb_e(currentNodeIdx) = mean(nonCall.cashflow_e(currentNodeIdx)/(nominal * coupon * dcf));
                    nonCall.cashflow_npv_e(currentNodeIdx) = mean(payoffStateA.cashflow_npv);

                    nonCall.df_e(currentNodeIdx) = mean(payoffStateA.df);
                    nonCall.df_c(currentNodeIdx) = model.DFRisky(currentTime/365.0);
                    
%                     %2nd callable column
%                     callable.payoffStates.cashflow(currentNodeIdx,:) = nonCall.payoffStates.cashflow(currentNodeIdx,:);
%                     callable.payoffStates.cashflow_npv(currentNodeIdx,:) = nonCall.payoffStates.cashflow_npv(currentNodeIdx,:);
%                     callable.payoff = callable.payoff + callable.payoffStates.cashflow_npv(currentNodeIdx,:);
%                     
%                     callable.cashflow_e(currentNodeIdx) = nonCall.cashflow_e(currentNodeIdx);
%                     callable.cashflow_npv_e(currentNodeIdx) = nonCall.cashflow_npv_e(currentNodeIdx);

                    % if exercise is possible
                    if i == 1
                        dummaaa = 1;
                    end
                    
                    % call event happens  at the start of the period
                    % at call date, by default, call option is expired
                    
%                     if callYN(i) && (DateDiff(valueDate,schedule.startDates(i)) < 0)
%                         % 1.undiscount callable.payoff
%                         % 2.build the state variables matrix
%                         % 3.regress 1's with the given state variables
%                         % (model states 1,2)
%                         % 4. exercise per each path in
%                         % discount the new payoff and assign into callable
%                         % column
%                         
%                         %1 undiscount callable.payoff and get the
%                         %contiValue
%                        
%                         %exercise value
%                         exerciseValue = security.nominal*ones(1,pathSize);
%                         contiValue = zeros(1,pathSize);
%                         onePayoff = ones(1,pathSize);
%                         onePayoffD = model.discountPayoff(currentTime,numMatTime ...
%                         ,modelStateT,onePayoff);
%                         contiValue = callable.payoff./onePayoffD;
%                         
%                         % build the state variabls matrix
%                         decisionVar1 = modelStateT(1,:);
%                         decisionVar2 = modelStateT(2,:);
%                         % we use upto 2nd order polynomial as regression
%                         % equatioin
%                         dimensionLS = 6;
%                         regMatrix = ones(pathSize,dimensionLS);
%                         
%                         for idx_r=1:pathSize
%                             regMatrix(idx_r,2) = decisionVar1(idx_r);
%                             regMatrix(idx_r,3) = decisionVar2(idx_r);
%                             regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r);
%                             regMatrix(idx_r,5) = decisionVar1(idx_r)*decisionVar2(idx_r);
%                             regMatrix(idx_r,6) = decisionVar2(idx_r)*decisionVar2(idx_r);
%                         end
%                         
%                         % simple regression <- for path dependent option
%                         % we should change it
%                         % beta  : regression coefficients
%                         beta =zeros(dimensionLS,1);
%                         beta = regress(contiValue',regMatrix);
%                         regContiValue = regMatrix*beta;
%                         
%                         exercisedPayoff = regContiValue;
%                         % if regContiValue is bigger than exerciseValue
%                         % then issue exercise call option
%                         for idx_r=1:pathSize
%                             if regContiValue(idx_r) >= exerciseValue(idx_r) 
%                                 exercisedPayoff(idx_r) = exerciseValue(idx_r);
%                             end
%                         end
%                         
%                         % we discount the new payoff and 
%                         % assign it into the existing callable payoff
%                         callable.payoff = model.discountPayoff(currentTime,numMatTime ...
%                         ,modelStateT,exercisedPayoff);
%                         
%                     end
                end

                nonCall.npv = mean(nonCall.payoff);
                out.pricerInfo.nonCall = nonCall;
                
%                 callable.npv = mean(callable.payoff);
%                 out.pricerInfo.callable = callable;
                out.daycountFraction = schedule.dayCountFraction;
                
        end
        
        
        
        
    end
    
end



In [ ]:
classdef DailyWeeklyPricer < H_RootObject
    %Pricer(Security,Model,NumMethod)
    %Prcer is constructed from Security, Model, NumMethod 
    
    properties
        m_security
        m_model
        m_numMethod
    end
    
    methods
        function pricer = DailyWeeklyPricer(security,model,numMethod)
                pricer.m_security = security;
                pricer.m_model = model;
                pricer.m_numMethod = numMethod;
        end
        
        function out = computePriceMCNonCall(pricer,valueDate)
                security = pricer.m_security;
                numMethod = pricer.m_numMethod;
                model = pricer.m_model;
                schedule = security.schedule;
                % numMethodTimeSteps Generate
                mcOneTimeStep = numMethod.mcOneTimeStep;
                outInfo = numMethod.numMethodTimeStepsGenerateDW(valueDate,schedule,mcOneTimeStep);
            
                numMethodInfo.timeStepsInfo = outInfo;

                % mc random number init
                %pathSize = 2^16;
                pathSize = numMethod.pathSize;

                % generate random numbers and modelstates forward
                rng('default');
                rng(0);

                numOfFactors = 2;

                %Z= randn(numOfFactors*outInfo.totalTimeStepsSize,pathSize/2);
                Z= randn(pathSize/2,numOfFactors*outInfo.totalTimeStepsSize);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;

                numMethodInfo.pathSize = pathSize;
                numMethodInfo.numOfFactors = numOfFactors;
                numMethodInfo.totalTimeSteps = outInfo.totalTimeSteps;
                numMethodInfo.totalTimeStepsSize = outInfo.totalTimeStepsSize;
                numMethodInfo.Z = Z;



                %generate model states for the given timeSteps
                modelStates = numMethod.generateModelStates(valueDate,schedule,numMethodInfo,model);

                % numeraire info
                numMatTime = numMethodInfo.totalTimeSteps(end);
                dfNumMat = model.DF(numMatTime/365.0);
                
                % induct backward and calculate cashflow
            
                periodCount = schedule.numOfPeriod;
                
                
                nonCall.npv = 0;
                nonCall.cashflow_e = zeros(periodCount + 1,1);
                nonCall.cashflow_npv_e = zeros(periodCount + 1,1);
                nonCall.rangeProb_e = zeros(periodCount + 1,1);
                nonCall.df_e = zeros(periodCount + 1,1);
                nonCall.df_c = zeros(periodCount + 1,1);

                % debug information, cashflow & npv per path
                nonCall.payoff = zeros(1,pathSize);
                nonCall.payoffStates.cashflow = zeros(periodCount+1,pathSize);
                nonCall.payoffStates.cashflow_npv = zeros(periodCount+1,pathSize);
                nonCall.payoffStates.df = zeros(periodCount + 1,pathSize);
                
                % intermediate variables
                payoffStateA.cashflow = zeros(1, pathSize);
                payoffStateA.cashflow_npv = zeros(1, pathSize);
                payoffStateA.df = zeros(1,pathSize);

                % we calculate cashflow backward from maturityDate to valueDate
                % rangeAccrual coupons are added at the startDate of the Period
                % At maturity

                nominal = security.nominal;
                coupon = security.coupon;

                currentTime = numMethodInfo.totalTimeSteps(end);
                totalTimeStepsSize = numMethodInfo.totalTimeStepsSize;
                currentTimeIdx = totalTimeStepsSize;
                
                currentNodeIdx = periodCount + 1;
                
                modelStateT = modelStates(numOfFactors*(totalTimeStepsSize-1)+1:numOfFactors*(totalTimeStepsSize-1)+2,:);
                payoffStateA.cashflow = nominal*ones(1,pathSize);
                %process payoff at maturity
                nonCall.payoffStates.cashflow(currentNodeIdx,:) = payoffStateA.cashflow;
                payoffStateA.cashflow_npv = model.discountPayoff(currentTime,numMatTime ...
                    ,modelStateT,payoffStateA.cashflow);
                nonCall.payoffStates.cashflow_npv(currentNodeIdx,:) = payoffStateA.cashflow_npv ;
                payoffStateA.df = model.discountPayoff(currentTime,numMatTime ...
                    ,modelStateT,1.0*ones(1,pathSize));
                nonCall.payoffStates.df(currentNodeIdx,:) = payoffStateA.df;
                nonCall.payoff = nonCall.payoff + payoffStateA.cashflow_npv;
                
                nonCall.cashflow_e(currentNodeIdx) = mean(payoffStateA.cashflow);
                nonCall.cashflow_npv_e(currentNodeIdx) = mean(payoffStateA.cashflow_npv);
                nonCall.df_e(currentNodeIdx) = mean(payoffStateA.df);
                nonCall.df_c(currentNodeIdx) = model.DF(currentTime/365.0);
    %             
    %             nonCall.cashflow_npv_df(end,:) =  nonCall.npv/nominal;
    %             nonCall.cashflow_npv_df_e(end) = mean(nonCall.cashflow_npv_df(end,:));

                lastTimeIdx = currentTimeIdx;
                totalTimeSteps = numMethodInfo.timeStepsInfo.totalTimeSteps;
                
                % flag for inclusive schedule
                inclusiveYN = false;
                simpleYN = false;
                
                for i=periodCount:-1:1
                    % past schedule neglect
                    if DateDiff(valueDate,schedule.payDates(i)) >= 0, continue;end 
                    startTimeIdx = numMethodInfo.timeStepsInfo.startTimeIdx(i);
                    endTimeIdx = numMethodInfo.timeStepsInfo.endTimeIdx(i);

                    if DateDiff(valueDate,schedule.startDates(i)) >= 0
                        inclusiveYN = true;
                    end
                  %% FromToInduction start

                    %modelsStates induct backward(lastTimeIdx to endTimeIdx)
                    while endTimeIdx < lastTimeIdx
                        lastTimeIdx = lastTimeIdx -1;
                        if lastTimeIdx == endTimeIdx break;end
                    end

                    currentTimeIdx = lastTimeIdx;
                    currentTime = totalTimeSteps(currentTimeIdx);
                    % now we are at endTimeIdx
                    % variables for coupon valuation initialized
                    ralower = security.lower;
                    raupper = security.upper;
                    tenorLong = security.tenorL;
                    tenorShort = security.tenorS;

                    cD = 0; % cumulated day count fraction
                    cP = 0; 
                    dI = 0; % dummy idx for loop
                    dcfC = 0;
                    dcfN = 0;
                    dD = 0;
                    idxL = zeros(1,pathSize);
                    idxS = zeros(1,pathSize);
                    spreadC = zeros(1,pathSize);
                    spreadN = zeros(1,pathSize);
                    probSpread = zeros(1,pathSize);
                    
%                     if inclusiveYN
%                         simpleYN = true;
%                     end
                     
                    % if inclusive then we use simpleYN rangeProb
                    
                    if inclusiveYN
                        simpleYN = inclusiveYN;
                    end
                    
                    numOfTimeSteps = numMethodInfo.timeStepsInfo.numOfTimeSteps(i);
                    
                    %if inclusive then we do not evaluate the last step but
                    % use fixing instead 
                    endTimeStep = 1;
                    if inclusiveYN
                        endTimeStep = 2;
                    end
                    
                    for j=numOfTimeSteps:-1:endTimeStep
                        % evaluate dealdescription
                        % last step is omitted
                        currentTime = totalTimeSteps(currentTimeIdx);
                        %dcfC = currentTime/365.0;
                        %we change dcf into days for comparative purpose
                        dcfC = currentTime;
                        if j~= numOfTimeSteps
                            dD = (dcfN-dcfC);
                        end

                        modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:numOfFactors*(currentTimeIdx-1)+2,:);
                        idxL = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorLong,modelStateT);
                        idxS = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                        spreadC = idxL - idxS;

                        probSpread = model.SRPr(spreadC,spreadN,ralower,raupper,simpleYN,pathSize)*dD;
                        cD = cD + dD;
                        cP = cP + probSpread;

                        dcfN = dcfC;
                        spreadN = spreadC;

                        %induct backward to the next time step
                        % one time backward induction
                        lastTimeIdx = currentTimeIdx;

                        if startTimeIdx < lastTimeIdx
                            lastTimeIdx = lastTimeIdx -1;
                            currentTimeIdx = lastTimeIdx;
                        end


                    end

                    %% FromToInduction end
                    %% AtDate operation
                    % Adding a new cashflow to the payoff

                    dcf = schedule.dayCountFraction(i);
                    
                    if inclusiveYN
                        % we add valuation date
                        cD = schedule.dayCountFraction(i);
                        rangeDays =  security.rangeDays(i);
                        payoffStateA.cashflow = nominal * coupon * dcf * (rangeDays + cP)/cD;
                    else
                        payoffStateA.cashflow = nominal * coupon * dcf * cP/cD;
                    end
                    % eventDate(i) = startDate(i) therefore there is no induction in modelStates
                    % i.e. currentTimeIdx has not changed
                    % no need to update modelStateT
                    % modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:end,:);
                    % since cashflow's are added at the startDate(i) we need to
                    % multiply cashflow with the df(startDate(i), payDate(i))
                    payTime = DateDiff(schedule.payDates(i),valueDate);
                    
                    %event is at startDate so we need to discount the
                    %cashflow first from paydate to startdate
                    df_sp = model.discountFactor(currentTime, payTime,numMatTime,modelStateT);

                    for k=1:pathSize
                        payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_sp(k);
                    end
                    
                    % process payoff
                    
                    currentNodeIdx = i;
                    nonCall.payoffStates.cashflow(currentNodeIdx,:) = payoffStateA.cashflow;
                    payoffStateA.cashflow_npv = model.discountPayoff(currentTime,numMatTime ...
                    ,modelStateT,payoffStateA.cashflow);
                    nonCall.payoffStates.cashflow_npv(currentNodeIdx,:) = payoffStateA.cashflow_npv;
                    payoffStateA.df = model.discountPayoff(currentTime,numMatTime ...
                    ,modelStateT,1.0*ones(1,pathSize));
                    nonCall.payoffStates.df(currentNodeIdx,:) = payoffStateA.df;
                    
                    nonCall.payoff = nonCall.payoff + payoffStateA.cashflow_npv;
                    
                    nonCall.cashflow_e(currentNodeIdx) = mean(payoffStateA.cashflow);
                    nonCall.rangeProb_e(currentNodeIdx) = mean(nonCall.cashflow_e(currentNodeIdx)/(nominal * coupon * dcf));
                    nonCall.cashflow_npv_e(currentNodeIdx) = mean(payoffStateA.cashflow_npv);

                    nonCall.df_e(currentNodeIdx) = mean(payoffStateA.df);
                    nonCall.df_c(currentNodeIdx) = model.DF(currentTime/365.0);

                end

                nonCall.npv = mean(nonCall.payoff);

                out.cashflow= nonCall.cashflow_e;
                out.cashflowNpv = nonCall.cashflow_npv_e;
                out.df_e = nonCall.df_e;
                out.df_c = nonCall.df_c;
                out.daycountFraction = schedule.dayCountFraction;
                out.rangeProb = nonCall.rangeProb_e;
                out.npv = nonCall.npv;
            
        end
        
        function out = computePriceMCCallable(pricer,valueDate)
                security = pricer.m_security;
                numMethod = pricer.m_numMethod;
                model = pricer.m_model;
                schedule = security.couponSchedule;
                callSchedule = security.callSchedule;
                
                % numMethodTimeSteps Generate
                mcOneTimeStep = numMethod.mcOneTimeStep;
                outInfo = numMethod.numMethodTimeStepsGenerateDW(valueDate,schedule,mcOneTimeStep);
            
                numMethodInfo.timeStepsInfo = outInfo;

                % mc random number init
                %pathSize = 2^16;
                pathSize = numMethod.pathSize;

                % generate random numbers and modelstates forward
                rng('default');
                rng(0);

                numOfFactors = 2;

                %Z= randn(numOfFactors*outInfo.totalTimeStepsSize,pathSize/2);
                Z= randn(pathSize/2,numOfFactors*outInfo.totalTimeStepsSize);
                Z=[Z;-Z];
                Z=Z';
                
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;

                % we check with the c code, so we use generated random numbers
%                 mc_random = importdata('mc_random.xlsx');
%                 mc_random = importdata('D:\mc_random_number_new_report.txt');
%                 Z= mc_random;
                
                numMethodInfo.pathSize = pathSize;
                numMethodInfo.numOfFactors = numOfFactors;
                numMethodInfo.totalTimeSteps = outInfo.totalTimeSteps;
                numMethodInfo.totalTimeStepsSize = outInfo.totalTimeStepsSize;
                numMethodInfo.Z = Z;



                %generate model states for the given timeSteps
                modelStates = numMethod.generateModelStates(valueDate,schedule,numMethodInfo,model);

                % numeraire info
                numMatTime = numMethodInfo.totalTimeSteps(end);
                days = DateDiff(schedule.payDates(schedule.numOfPeriod),valueDate);
                numMatTime = max(numMatTime, days);
                dfNumMat = model.DF(numMatTime/365.0);
                
                % induct backward and calculate cashflow
            
                periodCount = schedule.numOfPeriod;
                
                % pricer single info initialize
                % 1st nonCall
                nonCall.npv = 0;
                nonCall.cashflowSize = periodCount +1;
                nonCall.pathSize = pathSize;
                nonCall.cashflow_e = zeros(periodCount + 1,1);
                nonCall.cashflow_npv_e = zeros(periodCount + 1,1);
                nonCall.payoff = zeros(1,pathSize);
                nonCall.payoffStates.cashflow = zeros(periodCount+1,pathSize);
                nonCall.payoffStates.cashflow_npv = zeros(periodCount+1,pathSize);
                
                % 2nd callable
                callable.npv = 0;
                callable.cashflowSize = periodCount +1;
                callable.pathSize = pathSize;
                callable.cashflow_e = zeros(periodCount + 1,1);
                callable.cashflow_npv_e = zeros(periodCount + 1,1);
                callable.payoff = zeros(1,pathSize);
                callable.payoffStates.cashflow = zeros(periodCount+1,pathSize);
                callable.payoffStates.cashflow_npv = zeros(periodCount+1,pathSize);
                
                % 3rd dfCol
                dfCol.npv = 0;
                dfCol.cashflowSize = periodCount +1;
                dfCol.pathSize = pathSize;
                dfCol.cashflow_e = zeros(periodCount + 1,1);
                dfCol.cashflow_npv_e = zeros(periodCount + 1,1);
                dfCol.payoff = zeros(1,pathSize);
                dfCol.payoffStates.cashflow = zeros(periodCount+1,pathSize);
                dfCol.payoffStates.cashflow_npv = zeros(periodCount+1,pathSize);
                
                % intermediate variables
                payoffStateA.cashflow = zeros(1, pathSize);
                payoffStateA.cashflow_npv = zeros(1, pathSize);
                

                % we calculate cashflow backward from maturityDate to valueDate
                % rangeAccrual coupons are added at the startDate of the Period
                % At maturity

                nominal = security.nominal;
                coupon = security.coupon;
                callYN = security.callYN;
                
                currentTime = numMethodInfo.totalTimeSteps(end);
                totalTimeStepsSize = numMethodInfo.totalTimeStepsSize;
                currentTimeIdx = totalTimeStepsSize;
                
                currentNodeIdx = periodCount + 1;
                
                modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:numOfFactors*(currentTimeIdx-1)+2,:);
                payoffStateA.cashflow = nominal*ones(1,pathSize);
                
                payTime = DateDiff(schedule.payDates(schedule.numOfPeriod),valueDate);
                df_ep = model.discountFactor(currentTime, payTime,numMatTime,modelStateT);
                for k=1:pathSize
                    payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                end
                
                %process payoff at maturity
                nonCall.payoffStates.cashflow(currentNodeIdx,:) = payoffStateA.cashflow;
                payoffStateA.cashflow_npv = model.discountPayoff(currentTime,numMatTime ...
                    ,modelStateT,payoffStateA.cashflow);
                nonCall.payoffStates.cashflow_npv(currentNodeIdx,:) = payoffStateA.cashflow_npv ;
                nonCall.payoff = nonCall.payoff + payoffStateA.cashflow_npv;
                nonCall.cashflow_e(currentNodeIdx) = mean(payoffStateA.cashflow);
                nonCall.cashflow_npv_e(currentNodeIdx) = mean(payoffStateA.cashflow_npv);
                
                
                % callable at maturity is the same as noncallable
                callable.payoffStates.cashflow(currentNodeIdx,:) = nonCall.payoffStates.cashflow(currentNodeIdx,:);
                callable.payoffStates.cashflow_npv(currentNodeIdx,:) = nonCall.payoffStates.cashflow_npv(currentNodeIdx,:);
                callable.payoff = callable.payoff + callable.payoffStates.cashflow_npv(currentNodeIdx,:);
                callable.cashflow_e(currentNodeIdx) = nonCall.cashflow_e(currentNodeIdx);
                callable.cashflow_npv_e(currentNodeIdx) = nonCall.cashflow_npv_e(currentNodeIdx);
                
                %df column
                payoffStateA.cashflow = ones(1,pathSize);
                for k=1:pathSize
                    payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                end
                dfCol.payoffStates.cashflow(currentNodeIdx,:) = payoffStateA.cashflow;
                payoffStateA.cashflow_npv = model.discountPayoff(currentTime,numMatTime ...
                    ,modelStateT,payoffStateA.cashflow);
                dfCol.payoffStates.cashflow_npv(currentNodeIdx,:) = payoffStateA.cashflow_npv ;
                dfCol.payoff = dfCol.payoff + payoffStateA.cashflow_npv;
                dfCol.cashflow_e(currentNodeIdx) = mean(payoffStateA.cashflow);
                dfCol.cashflow_npv_e(currentNodeIdx) = mean(payoffStateA.cashflow_npv);
                  

                lastTimeIdx = currentTimeIdx;
                totalTimeSteps = numMethodInfo.timeStepsInfo.totalTimeSteps;
                
                % flag for inclusive schedule
                simpleYN = false;
                
                for i=periodCount:-1:1
                    % past schedule neglect
                    if DateDiff(valueDate,schedule.payDates(i)) >= 0, continue;end 
                    startTimeIdx = numMethodInfo.timeStepsInfo.startTimeIdx(i);
                    endTimeIdx = numMethodInfo.timeStepsInfo.endTimeIdx(i);

                    inclusiveYN = numMethodInfo.timeStepsInfo.inclusiveYN(i);
                 %% FromToInduction start

                    %modelsStates induct backward(lastTimeIdx to endTimeIdx)
                    while endTimeIdx < lastTimeIdx
                        lastTimeIdx = lastTimeIdx -1;
                        if lastTimeIdx == endTimeIdx break;end
                    end

                    currentTimeIdx = lastTimeIdx;
                    currentTime = totalTimeSteps(currentTimeIdx);
                    % now we are at endTimeIdx
                    % variables for coupon valuation initialized
                    ralower = security.lower;
                    raupper = security.upper;
                    tenorLong = security.tenorL;
                    tenorShort = security.tenorS;

                    cD = 0; % cumulated day count fraction
                    cP = 0; 
                    dI = 0; % dummy idx for loop
                    dcfC = 0;
                    dcfN = 0;
                    dD = 0;
                    idxL = zeros(1,pathSize);
                    idxS = zeros(1,pathSize);
                    spreadC = zeros(1,pathSize);
                    spreadN = zeros(1,pathSize);
                    probSpread = zeros(1,pathSize);
                    
                    numOfTimeSteps = numMethodInfo.timeStepsInfo.numOfTimeSteps(i);
                    
                    % if inclusive then we use simpleYN rangeProb
                    %if inclusive then we do not evaluate the last step but
                    % use fixing instead 
                    endTimeStep = 1;
                    if inclusiveYN
                        simpleYN = true;
                        endTimeStep = 2;
                    end
                    
                    for j=numOfTimeSteps:-1:endTimeStep
                        % evaluate dealdescription
                        % last step is omitted
                        currentTime = totalTimeSteps(currentTimeIdx);
                        %dcfC = currentTime/365.0;
                        dcfC = currentTime;
                        
                        if j~= numOfTimeSteps
                            dD = (dcfN-dcfC);
                        end

                        modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:numOfFactors*(currentTimeIdx-1)+2,:);
                        idxL = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorLong,modelStateT);
                        idxS = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorShort,modelStateT);
                        spreadC = idxL - idxS;

                        probSpread = model.SRPr(spreadC,spreadN,ralower,raupper,simpleYN,pathSize)*dD;
                        cD = cD + dD;
                        cP = cP + probSpread;

                        dcfN = dcfC;
                        spreadN = spreadC;

                        %induct backward to the next time step
                        % one time backward induction
                        lastTimeIdx = currentTimeIdx;

                        if startTimeIdx < lastTimeIdx
                            lastTimeIdx = lastTimeIdx -1;
                            currentTimeIdx = lastTimeIdx;
                        end


                    end

                    %% FromToInduction end
        % if the current period is inclusive, then the start date is not evaluated in the above for loop.
		% instead the fixing will be treated separately below.
		% however current_time_idx is updated at the end of the for loop.
		% so we need to update model_states accordingly.
		% if model_states were updated previously then the following additionaly step doesn't affect the result anyway.
                    currentTime = totalTimeSteps(currentTimeIdx);
                    modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:numOfFactors*(currentTimeIdx-1)+2,:);
                    
                    %% AtDate evaluation
                    % Adding a new cashflow to the payoff

                    dcf = schedule.dayCountFraction(i);
                    
                    if inclusiveYN
                        % we add valuation date
                        cD = schedule.dayCount(i);
                        rangeDays =  security.rangeDays(i);
                        payoffStateA.cashflow = nominal * coupon * dcf * (rangeDays(i) + cP)/cD;
                    else
                        payoffStateA.cashflow = nominal * coupon * dcf * cP/cD;
                    end
                    % eventDate(i) = startDate(i) therefore there is no induction in modelStates
                    % i.e. currentTimeIdx has not changed
                    % no need to update modelStateT
                    % modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:end,:);
                    % since cashflow's are added at the startDate(i) we need to
                    % multiply cashflow with the df(startDate(i), payDate(i))
                    payTime = DateDiff(schedule.payDates(i),valueDate);
                    
                    %event is at startDate so we need to discount the
                    %cashflow first from paydate to startdate
                    df_ep = model.discountFactor(currentTime, payTime,numMatTime,modelStateT);

                    for k=1:pathSize
                        payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                    end
                    
                    % process payoff
                    
                    currentNodeIdx = i;
                    
                    %nonCall
                    nonCall.payoffStates.cashflow(currentNodeIdx,:) = payoffStateA.cashflow;
                    payoffStateA.cashflow_npv = model.discountPayoff(currentTime,numMatTime ...
                        ,modelStateT,payoffStateA.cashflow);
                    nonCall.payoffStates.cashflow_npv(currentNodeIdx,:) = payoffStateA.cashflow_npv ;
                    nonCall.payoff = nonCall.payoff + payoffStateA.cashflow_npv;
                    nonCall.cashflow_e(currentNodeIdx) = mean(payoffStateA.cashflow);
                    nonCall.cashflow_npv_e(currentNodeIdx) = mean(payoffStateA.cashflow_npv);

                    % callable
                    callable.payoffStates.cashflow(currentNodeIdx,:) = nonCall.payoffStates.cashflow(currentNodeIdx,:);
                    callable.payoffStates.cashflow_npv(currentNodeIdx,:) = nonCall.payoffStates.cashflow_npv(currentNodeIdx,:);
                    callable.payoff = callable.payoff + callable.payoffStates.cashflow_npv(currentNodeIdx,:);
                    callable.cashflow_e(currentNodeIdx) = nonCall.cashflow_e(currentNodeIdx);
                    callable.cashflow_npv_e(currentNodeIdx) = nonCall.cashflow_npv_e(currentNodeIdx);

                    %df column
                    payoffStateA.cashflow = ones(1,pathSize);
                    for k=1:pathSize
                        payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_ep(k);
                    end
                    dfCol.payoffStates.cashflow(currentNodeIdx,:) = payoffStateA.cashflow;
                    payoffStateA.cashflow_npv = model.discountPayoff(currentTime,numMatTime ...
                        ,modelStateT,payoffStateA.cashflow);
                    dfCol.payoffStates.cashflow_npv(currentNodeIdx,:) = payoffStateA.cashflow_npv ;
                    dfCol.payoff = dfCol.payoff + payoffStateA.cashflow_npv;
                    dfCol.cashflow_e(currentNodeIdx) = mean(payoffStateA.cashflow);
                    dfCol.cashflow_npv_e(currentNodeIdx) = mean(payoffStateA.cashflow_npv);
                    
                    % if exercise is possible
                    if i == 1
                        dummaaa = 1;
                    end
                    
                    % call event happens  at the start of the period
                    % at call date, by default, call option is expired
                    
                    if callYN(i) && (DateDiff(valueDate,callSchedule.resetDates(i)) < 0)
                        % 1.undiscount callable.payoff
                        % 2.build the state variables matrix
                        % 3.regress 1's with the given state variables
                        % (model states 1,2)
                        % 4. exercise per each path in
                        % discount the new payoff and assign into callable
                        % column
                        
                        %1 undiscount callable.payoff and get the
                        %contiValue
                       
                        %exercise value
                        exerciseValue = security.nominal*ones(1,pathSize);
                        payTime = DateDiff(callSchedule.payDates(i),valueDate);
                        %event is at startDate so we need to discount the
                        %cashflow first from paydate to startdate
                        df_ep = model.discountFactor(currentTime, payTime,numMatTime,modelStateT);
                        for k=1:pathSize
                            exerciseValue(k) = exerciseValue(k) * df_ep(k);
                        end
                        
                        contiValue = zeros(1,pathSize);
                        onePayoff = ones(1,pathSize);
                        onePayoffD = model.discountPayoff(currentTime,numMatTime ...
                        ,modelStateT,onePayoff);
                        contiValue = callable.payoff./onePayoffD;
                        
                        % build the state variabls matrix
                        decisionVar1 = modelStateT(1,:);
                        decisionVar2 = modelStateT(2,:);
                        % we use upto 2nd order polynomial as regression
                        % equatioin
                        dimensionLS = 6;
                        regMatrix = ones(pathSize,dimensionLS);
                        
                        for idx_r=1:pathSize
                            regMatrix(idx_r,2) = decisionVar1(idx_r);
                            regMatrix(idx_r,3) = decisionVar2(idx_r);
                            regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r);
                            regMatrix(idx_r,5) = decisionVar1(idx_r)*decisionVar2(idx_r);
                            regMatrix(idx_r,6) = decisionVar2(idx_r)*decisionVar2(idx_r);
                        end
                        
                        % simple regression <- for path dependent option
                        % we should change it
                        % beta  : regression coefficients
                        beta =zeros(dimensionLS,1);
                        beta = regress(contiValue',regMatrix);
                        regContiValue = regMatrix*beta;
                        
                        exercisedPayoff = regContiValue;
                        % if regContiValue is bigger than exerciseValue
                        % then issue exercise call option
                        for idx_r=1:pathSize
                            if regContiValue(idx_r) >= exerciseValue(idx_r) 
                                exercisedPayoff(idx_r) = exerciseValue(idx_r);
                            end
                        end
                        
                        % we discount the new payoff and 
                        % assign it into the existing callable payoff
                        callable.payoff = model.discountPayoff(currentTime,numMatTime ...
                        ,modelStateT,exercisedPayoff);
                        
                    end
                end

                nonCall.npv = mean(nonCall.payoff);
                out.nonCall = nonCall;
                callable.npv = mean(callable.payoff);
                out.callable = callable;
                out.daycountFraction = schedule.dayCountFraction;
                
        end
        
        function out = computePriceClosedTRZ(pricer,valueDate)
                security = pricer.m_security;
                numMethod = pricer.m_numMethod;
                model = pricer.m_model;
                schedule = security.schedule;
                % numMethodTimeSteps Generate
                mcOneTimeStep = numMethod.mcOneTimeStep;
                outInfo = numMethod.numMethodTimeStepsGenerateDW(valueDate,schedule,mcOneTimeStep);
            
                numMethodInfo.timeStepsInfo = outInfo;

                % mc random number init
                %pathSize = 2^16;
                pathSize = numMethod.pathSize;
                
                pathSize = 1;
                
%                 % generate random numbers and modelstates forward
%                 rng('default');
%                 rng(0);
% 
                numOfFactors = 2;
% 
%                 %Z= randn(numOfFactors*outInfo.totalTimeStepsSize,pathSize/2);
%                 Z= randn(pathSize/2,numOfFactors*outInfo.totalTimeStepsSize);
%                 Z=[Z;-Z];
%                 Z=Z';
%                 
%                 
%                 %TimeInversion
%                 sizeZ = size(Z,1);
%                 A = zeros(sizeZ,sizeZ);
%                 for i=1:sizeZ
%                     A(i,sizeZ + 1 -i) = 1;
%                 end
%                 Z = A*Z;
% 
%                 % we check with the c, so we use generated random numbers
% %                 mc_random = importdata('mc_random.xlsx');
% %                 mc_random = importdata('D:\mc_random_number_new_report.txt');
% %                 Z= mc_random;
%                 
                numMethodInfo.pathSize = pathSize;
                numMethodInfo.numOfFactors = numOfFactors;
                numMethodInfo.totalTimeSteps = outInfo.totalTimeSteps;
                numMethodInfo.totalTimeStepsSize = outInfo.totalTimeStepsSize;
%                 numMethodInfo.Z = Z;
% 
% 
% 
%                 %generate model states for the given timeSteps
%                 modelStates = numMethod.generateModelStates(valueDate,schedule,numMethodInfo,model);

                % numeraire info
                numMatTime = numMethodInfo.totalTimeSteps(end);
                dfNumMat = model.DF(numMatTime/365.0);
                
                % induct backward and calculate cashflow
            
                periodCount = schedule.numOfPeriod;
                
                % cashflow columns initialize
                % 1st nonCall
                nonCall.npv = 0;
                nonCall.cashflow_e = zeros(periodCount + 1,1);
                nonCall.cashflow_npv_e = zeros(periodCount + 1,1);
                nonCall.rangeProb_e = zeros(periodCount + 1,1);
                nonCall.df_e = zeros(periodCount + 1,1);
                nonCall.df_c = zeros(periodCount + 1,1);

                % debug information, cashflow & npv per path
                nonCall.payoff = zeros(1,pathSize);
                nonCall.payoffStates.cashflow = zeros(periodCount+1,pathSize);
                nonCall.payoffStates.cashflow_npv = zeros(periodCount+1,pathSize);
                nonCall.payoffStates.df = zeros(periodCount + 1,pathSize);
                
                % intermediate variables
                payoffStateA.cashflow = zeros(1, pathSize);
                payoffStateA.cashflow_npv = zeros(1, pathSize);
                payoffStateA.df = zeros(1,pathSize);
                
                % 2nd callable
                callable.npv = 0;
                callable.cashflow_e = zeros(periodCount + 1,1);
                callable.cashflow_npv_e = zeros(periodCount + 1,1);
                callable.rangeProb_e = zeros(periodCount + 1,1);
                callable.df_e = zeros(periodCount + 1,1);
                callable.df_c = zeros(periodCount + 1,1);

                % debug information, cashflow & npv per path
                callable.payoff = zeros(1,pathSize);
                callable.payoffStates.cashflow = zeros(periodCount+1,pathSize);
                callable.payoffStates.cashflow_npv = zeros(periodCount+1,pathSize);
                callable.payoffStates.df = zeros(periodCount + 1,pathSize);
                

                % we calculate cashflow backward from maturityDate to valueDate
                % rangeAccrual coupons are added at the startDate of the Period
                % At maturity

                nominal = security.nominal;
                coupon = security.coupon;
                callYN = security.callYN;
                
                currentTime = numMethodInfo.totalTimeSteps(end);
                totalTimeSteps = numMethodInfo.timeStepsInfo.totalTimeSteps;
                totalTimeStepsSize = numMethodInfo.timeStepsInfo.totalTimeStepsSize;
                if totalTimeStepsSize == 0 || (totalTimeStepsSize == 1 && totalTimeSteps(1) == 0) 
                    out.nonCall = nonCall;
                    out.callable = callable;
                    out.dfCol = dfCol;
                    return;
                end
                
                currentTimeIdx = totalTimeStepsSize;
                
                currentNodeIdx = periodCount + 1;
                
%                 modelStateT = modelStates(numOfFactors*(totalTimeStepsSize-1)+1:numOfFactors*(totalTimeStepsSize-1)+2,:);
                payoffStateA.cashflow = nominal*ones(1,pathSize);
                %process payoff at maturity
                nonCall.payoffStates.cashflow(currentNodeIdx,:) = payoffStateA.cashflow;
                
%                 payoffStateA.cashflow_npv = model.discountPayoff(currentTime,numMatTime ...
%                     ,modelStateT,payoffStateA.cashflow);
%                 
                payoffStateA.cashflow_npv = model.DF(currentTime/365.0)*payoffStateA.cashflow;
                
                nonCall.payoffStates.cashflow_npv(currentNodeIdx,:) = payoffStateA.cashflow_npv ;
                
%                 payoffStateA.df = model.discountPayoff(currentTime,numMatTime ...
%                     ,modelStateT,1.0*ones(1,pathSize));
                payoffStateA.df = model.DF(currentTime/365.0) *ones(1,pathSize);
                    

                nonCall.payoffStates.df(currentNodeIdx,:) = payoffStateA.df;
                nonCall.payoff = nonCall.payoff + payoffStateA.cashflow_npv;
                
                nonCall.cashflow_e(currentNodeIdx) = mean(payoffStateA.cashflow);
                nonCall.cashflow_npv_e(currentNodeIdx) = mean(payoffStateA.cashflow_npv);
                nonCall.df_e(currentNodeIdx) = mean(payoffStateA.df);
                nonCall.df_c(currentNodeIdx) = model.DF(currentTime/365.0);
                
                % callable at maturity is the same as noncallable
                callable.payoffStates.cashflow(currentNodeIdx,:) = nonCall.payoffStates.cashflow(currentNodeIdx,:);
                callable.payoffStates.cashflow_npv(currentNodeIdx,:) = nonCall.payoffStates.cashflow_npv(currentNodeIdx,:);
                callable.payoff = callable.payoff + callable.payoffStates.cashflow_npv(currentNodeIdx,:);
                
                callable.cashflow_e(currentNodeIdx) = nonCall.cashflow_e(currentNodeIdx);
                callable.cashflow_npv_e(currentNodeIdx) = nonCall.cashflow_npv_e(currentNodeIdx);
                  
                lastTimeIdx = currentTimeIdx;
                
                
                % flag for inclusive schedule
                simpleYN = false;
                
                for i=periodCount:-1:1
                    % past schedule neglect
                    if DateDiff(valueDate,schedule.payDates(i)) >= 0, continue;end 
                    startTimeIdx = numMethodInfo.timeStepsInfo.startTimeIdx(i);
                    endTimeIdx = numMethodInfo.timeStepsInfo.endTimeIdx(i);

                    inclusiveYN = numMethodInfo.timeStepsInfo.inclusiveYN(i);
                     %% FromToInduction start

                    %modelsStates induct backward(lastTimeIdx to endTimeIdx)
                    while endTimeIdx < lastTimeIdx
                        lastTimeIdx = lastTimeIdx -1;
                        if lastTimeIdx == endTimeIdx break;end
                    end

                    currentTimeIdx = lastTimeIdx;
                    currentTime = totalTimeSteps(currentTimeIdx);
                    % now we are at endTimeIdx
                    % variables for coupon valuation initialized
                    ralower = security.lower;
                    raupper = security.upper;
                    tenorLong = security.tenorL;
                    tenorShort = security.tenorS;

                    cD = 0; % cumulated day count fraction
                    cP = 0; 
                    dI = 0; % dummy idx for loop
                    dcfC = 0;
                    dcfN = 0;
                    dD = 0;
                    idxL = zeros(1,pathSize);
                    idxS = zeros(1,pathSize);
                    spreadC = zeros(1,pathSize);
                    spreadN = zeros(1,pathSize);
                    probSpread = zeros(1,pathSize);
                    
                    numOfTimeSteps = numMethodInfo.timeStepsInfo.numOfTimeSteps(i);
                    
                    % if inclusive then we use simpleYN rangeProb
                    %if inclusive then we do not evaluate the last step but
                    % use fixing instead 
                    endTimeStep = 1;
                    if inclusiveYN
                        simpleYN = true;
                        endTimeStep = 2;
                    end
                    
                    for j=numOfTimeSteps:-1:endTimeStep
                        % evaluate dealdescription
                        % last step is omitted
                        currentTime = totalTimeSteps(currentTimeIdx);
                        %dcfC = currentTime/365.0;
                        dcfC = currentTime;
                        
                        if j~= numOfTimeSteps
                            dD = (dcfN-dcfC);
                        end

                        toDate = AddDate(valueDate,currentTime,'day');
                        tpDate = schedule.payDates(i);
                        spreadC = model.CMSSpreadDigitalRangeDate(valueDate,toDate,tpDate,tenorLong,tenorShort,0.25,ralower,raupper);
                        
                        %modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:numOfFactors*(currentTimeIdx-1)+2,:);
%                         idxL = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorLong,modelStateT);
%                         idxS = model.CMS_PSA_SimpleDate(valueDate,currentTime,numMatTime, tenorShort,modelStateT);
%                         spreadC = idxL - idxS;
%                         probSpread = model.SRPr(spreadC,spreadN,ralower,raupper,simpleYN,pathSize)*dD; 

                        if simpleYN
                            probSpread = spreadC*dD;
                        else
                            probSpread = 0.5*(spreadC + spreadN)*dD;
                        end

                        
                        cD = cD + dD;
                        cP = cP + probSpread;

                        dcfN = dcfC;
                        spreadN = spreadC;

                        %induct backward to the next time step
                        % one time backward induction
                        lastTimeIdx = currentTimeIdx;

                        if startTimeIdx < lastTimeIdx
                            lastTimeIdx = lastTimeIdx -1;
                            currentTimeIdx = lastTimeIdx;
                        end


                    end

                    %% FromToInduction end
                    %% AtDate evaluation
                    % Adding a new cashflow to the payoff

                    dcf = schedule.dayCountFraction(i);
                    
                    if inclusiveYN
                        % we add valuation date
                        cD = schedule.dayCount(i);
                        rangeDays =  security.rangeDays(i);
                        payoffStateA.cashflow = nominal * coupon * dcf * (rangeDays(i) + cP)/cD;
                    else
                        payoffStateA.cashflow = nominal * coupon * dcf * cP/cD;
                    end
                    % eventDate(i) = startDate(i) therefore there is no induction in modelStates
                    % i.e. currentTimeIdx has not changed
                    % no need to update modelStateT
                    % modelStateT = modelStates(numOfFactors*(currentTimeIdx-1)+1:end,:);
                    % since cashflow's are added at the startDate(i) we need to
                    % multiply cashflow with the df(startDate(i), payDate(i))
                    payTime = DateDiff(schedule.payDates(i),valueDate);
                    
                    %event is at startDate so we need to discount the
                    %cashflow first from paydate to startdate
%                     df_sp = model.discountFactor(currentTime, payTime,numMatTime,modelStateT);

%                     for k=1:pathSize
%                         payoffStateA.cashflow(k) = payoffStateA.cashflow(k) * df_sp(k);
%                     end
                    
                    % process payoff
                    
                    currentNodeIdx = i;
                    
                    % 1st nonCall column
                    nonCall.payoffStates.cashflow(currentNodeIdx,:) = payoffStateA.cashflow;
                    payoffStateA.cashflow_npv = payoffStateA.cashflow * model.DF(payTime/365.0);
                    
%                     payoffStateA.cashflow_npv = model.discountPayoff(currentTime,numMatTime ...
%                     ,modelStateT,payoffStateA.cashflow);
                    nonCall.payoffStates.cashflow_npv(currentNodeIdx,:) = payoffStateA.cashflow_npv;
                    payoffStateA.df = model.DF(payTime/365.0);
%                     payoffStateA.df = model.discountPayoff(currentTime,numMatTime ...
%                     ,modelStateT,1.0*ones(1,pathSize));
                    nonCall.payoffStates.df(currentNodeIdx,:) = payoffStateA.df;
                    
                    nonCall.payoff = nonCall.payoff + payoffStateA.cashflow_npv;
                    
                    nonCall.cashflow_e(currentNodeIdx) = mean(payoffStateA.cashflow);
                    nonCall.rangeProb_e(currentNodeIdx) = mean(nonCall.cashflow_e(currentNodeIdx)/(nominal * coupon * dcf));
                    nonCall.cashflow_npv_e(currentNodeIdx) = mean(payoffStateA.cashflow_npv);

                    nonCall.df_e(currentNodeIdx) = mean(payoffStateA.df);
                    nonCall.df_c(currentNodeIdx) = model.DF(currentTime/365.0);
                    
                    %2nd callable column
                    callable.payoffStates.cashflow(currentNodeIdx,:) = nonCall.payoffStates.cashflow(currentNodeIdx,:);
                    callable.payoffStates.cashflow_npv(currentNodeIdx,:) = nonCall.payoffStates.cashflow_npv(currentNodeIdx,:);
                    callable.payoff = callable.payoff + callable.payoffStates.cashflow_npv(currentNodeIdx,:);
                    
                    callable.cashflow_e(currentNodeIdx) = nonCall.cashflow_e(currentNodeIdx);
                    callable.cashflow_npv_e(currentNodeIdx) = nonCall.cashflow_npv_e(currentNodeIdx);

                    % if exercise is possible
                    if i == 1
                        dummaaa = 1;
                    end
                    
                    % call event happens  at the start of the period
                    % at call date, by default, call option is expired
                    
%                     if callYN(i) && (DateDiff(valueDate,schedule.startDates(i)) < 0)
%                         % 1.undiscount callable.payoff
%                         % 2.build the state variables matrix
%                         % 3.regress 1's with the given state variables
%                         % (model states 1,2)
%                         % 4. exercise per each path in
%                         % discount the new payoff and assign into callable
%                         % column
%                         
%                         %1 undiscount callable.payoff and get the
%                         %contiValue
%                        
%                         %exercise value
%                         exerciseValue = security.nominal*ones(1,pathSize);
%                         contiValue = zeros(1,pathSize);
%                         onePayoff = ones(1,pathSize);
%                         onePayoffD = model.discountPayoff(currentTime,numMatTime ...
%                         ,modelStateT,onePayoff);
%                         contiValue = callable.payoff./onePayoffD;
%                         
%                         % build the state variabls matrix
%                         decisionVar1 = modelStateT(1,:);
%                         decisionVar2 = modelStateT(2,:);
%                         % we use upto 2nd order polynomial as regression
%                         % equatioin
%                         dimensionLS = 6;
%                         regMatrix = ones(pathSize,dimensionLS);
%                         
%                         for idx_r=1:pathSize
%                             regMatrix(idx_r,2) = decisionVar1(idx_r);
%                             regMatrix(idx_r,3) = decisionVar2(idx_r);
%                             regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r);
%                             regMatrix(idx_r,5) = decisionVar1(idx_r)*decisionVar2(idx_r);
%                             regMatrix(idx_r,6) = decisionVar2(idx_r)*decisionVar2(idx_r);
%                         end
%                         
%                         % simple regression <- for path dependent option
%                         % we should change it
%                         % beta  : regression coefficients
%                         beta =zeros(dimensionLS,1);
%                         beta = regress(contiValue',regMatrix);
%                         regContiValue = regMatrix*beta;
%                         
%                         exercisedPayoff = regContiValue;
%                         % if regContiValue is bigger than exerciseValue
%                         % then issue exercise call option
%                         for idx_r=1:pathSize
%                             if regContiValue(idx_r) >= exerciseValue(idx_r) 
%                                 exercisedPayoff(idx_r) = exerciseValue(idx_r);
%                             end
%                         end
%                         
%                         % we discount the new payoff and 
%                         % assign it into the existing callable payoff
%                         callable.payoff = model.discountPayoff(currentTime,numMatTime ...
%                         ,modelStateT,exercisedPayoff);
%                         
%                     end
                end

                nonCall.npv = mean(nonCall.payoff);
                out.pricerInfo.nonCall = nonCall;
                
                callable.npv = mean(callable.payoff);
                out.pricerInfo.callable = callable;
                out.daycountFraction = schedule.dayCountFraction;
                
        end
        
        
        
    end
    
end



In [ ]:
import pandas as pd
import requests
from lxml import html
from tqdm import tqdm

In [ ]:
# 삼성전자
sample_code = '005930'

In [ ]:
# parsing URL
# 우리 컴퓨터 -> [접속] -> 에프앤가이드(Data Source) -> [크롤링/웹 스크래핑] -> 우리 컴퓨터
# client -> request -> [Server] -> response -> client

SNAP_URL = 'https://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?pGB=1&gicode=A{}&cID=&MenuYn=Y&ReportGB=&NewMenuID=101&stkGb=701'
RATIO_URL = 'https://comp.fnguide.com/SVO2/ASP/SVD_FinanceRatio.asp?pGB=1&gicode=A{}&cID=&MenuYn=Y&ReportGB=&NewMenuID=104&stkGb=701'


In [ ]:
# object -> 데이터 덩어리(추상화)

snap_url = SNAP_URL.format(sample_code)
snap_content = requests.get(snap_url).content # 문자열 binary
snap_tree = html.fromstring(snap_content) # 객체(object)
per = snap_tree.xpath('//*[@id="corp_group2"]/dl[1]/dd')[0].text
per = float(per)

In [ ]:
per

8.15

In [ ]:
ratio_url = RATIO_URL.format(sample_code)
ratio_content = requests.get(ratio_url).content
ratio_tree = html.fromstring(ratio_content)
debt_ratio = ratio_tree.xpath('//*[@id="p_grid1_3"]/td[5]')[0].text
debt_ratio = float(debt_ratio)

In [ ]:
# stockMkt => KOSPI
# kosdaqMkt => KOSDAQ
# konexMkt => KONEX

In [ ]:
def get_stock_list(market):
    market_code = ''
    if market == 'kospi':
        market_code = 'stockMkt'
    elif market == 'kosdaq':
        market_code = 'kosdaqMkt'
    elif market == 'konex':
        market_code = 'konexMkt'
    kind_url = 'https://kind.krx.co.kr/corpgeneral/corpList.do?method=download&pageIndex=1&currentPageSize=3000&comAbbrv=&beginIndex=&orderMode=3&orderStat=D&isurCd=&repIsuSrtCd=&searchCodeType=&marketType={}&searchType=13&industry=&fiscalYearEnd=all&comAbbrvTmp=&location=all'.format(market_code)                                                                 

    return pd.read_html(kind_url, converters={'종목코드':lambda x: str(x)})[0]


In [ ]:
def converter(x):
    return str(x)

# 람다 함수 == 무명(anonymous) 함수 == 일회용 함수
lambda x: str(x)

<function __main__.<lambda>(x)>

In [ ]:
kospi_df = get_stock_list('kospi')
print(kospi_df.shape)

(829, 9)


In [ ]:
kosdaq_df = get_stock_list('kosdaq')
print(kosdaq_df.shape)

(1632, 9)


In [ ]:
# merge -> SQL Join
# append | kospi_df.append([kosdaq_df])
# concatenate(합치다)
stock_list_df = pd.concat([kospi_df, kosdaq_df] )

In [ ]:
print(stock_list_df.shape)

(2461, 9)


In [ ]:
# stock_list_df['종목코드'].dropna()
stock_list_df = stock_list_df[stock_list_df['종목코드'].notnull()]

In [ ]:
stock_list_df = stock_list_df[~stock_list_df['회사명'].str.contains('스팩|리츠')]
print(stock_list_df.shape)

(2364, 9)


In [ ]:
# list comprehension
stock_list_df.index = [x for x in range(len(stock_list_df))]

In [ ]:
stock_list_df.to_csv('kospi_kosdaq_stock_list.csv', encoding='utf-8', index=True)

In [ ]:
code_list = stock_list_df['종목코드']
code_list

0       100090
1       453340
2       452260
3       450140
4       377740
         ...  
2359    013030
2360    019550
2361    019570
2362    019590
2363    006920
Name: 종목코드, Length: 2364, dtype: object

In [ ]:
sample_df = pd.DataFrame(
    {'005930':['삼성전자', 1, 2], '035720':['카카오', 1, 2], '015720':['카카오', 1, 2], '025720':['카카오', 1, 2]}
).transpose()



In [ ]:
sample_df.columns = ['name', 'PER', 'Debt_ratio']

In [ ]:
sample_df

,name,PER,Debt_ratio
005930,삼성전자,1,2
035720,카카오,1,2
015720,카카오,1,2
025720,카카오,1,2


In [ ]:
def FinanceInfoCrawler(li, df):
    result_dict = {}
    error_codes = []
    
    for code in tqdm(li):
        try:
            # Parsing URL setting
            SNAP_URL = 'https://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?pGB=1&gicode=A{}&cID=&MenuYn=Y&ReportGB=&NewMenuID=101&stkGb=701'
            RATIO_URL = 'https://comp.fnguide.com/SVO2/ASP/SVD_FinanceRatio.asp?pGB=1&gicode=A{}&cID=&MenuYn=Y&ReportGB=&NewMenuID=104&stkGb=701'

            # company name
            company_name = df[df['종목코드'] == code]['회사명'].values[0]

            # Get PER
            snap_url = SNAP_URL.format(code)
            snap_content = requests.get(snap_url).content
            snap_tree = html.fromstring(snap_content)
            per = snap_tree.xpath('//*[@id="corp_group2"]/dl[1]/dd')[0].text
            per = float(per)

            # Get Debt ratio
            ratio_url = RATIO_URL.format(code)
            ratio_content = requests.get(ratio_url).content
            ratio_tree = html.fromstring(ratio_content)
            debt_ratio = ratio_tree.xpath('//*[@id="p_grid1_3"]/td[5]')[0].text
            debt_ratio = float(debt_ratio)
            
            result_dict[code] = [company_name, per, debt_ratio]
            
        except (TypeError, IndexError, AttributeError, ValueError):
            pass
#             print(code)
            error_codes.append(code)
    
    # convert dict to DataFrame
    result_df = pd.DataFrame(result_dict)
    
    # transpose DataFrame
    result_df = result_df.transpose()
    
    # Setting column names
    result_df.columns = ['Name', 'PER', 'Debt_ratio']
    
    return result_df, error_codes

In [ ]:
crawling_result_df = FinanceInfoCrawler(code_list[:50],stock_list_df)

100%|██████████| 50/50 [00:46<00:00,  1.07it/s]


In [ ]:
print(crawling_result_df[0].shape)
crawling_result_df[0].head()

(39, 3)


,Name,PER,Debt_ratio
100090,SK오션플랜트,38.11,132.0
377740,바이오노트,1.89,9.8
446070,유니드비티플러스,104.18,11.2
108320,LX세미콘,7.5,35.7
126720,수산인더스트리,6.3,24.7


In [ ]:
# original data
# crawling_result_df

# copy data
copy_df = crawling_result_df[0].copy()

In [ ]:
# PER 10 이하
# 부채비율 50 이하
# (상위) 20개 종목

final_result_df = copy_df[
    (copy_df['PER'] <= 10)&(copy_df['Debt_ratio'] <= 50)&(copy_df['PER'] > 0)
].sort_values(
    by='PER', ascending=True
).iloc[:20]


In [ ]:
import datetime

# 시간까지 포함한 날짜
now = datetime.datetime.now()

final_result_df.to_csv('LowPER_LowDR_{}.csv'.format(now.strftime('%Y%m%d')))

In [ ]:
pd.read_csv('LowPER_LowDR_{}.csv'.format(now.strftime('%Y%m%d')))

,Unnamed: 0,Name,PER,Debt_ratio
0,377740,바이오노트,1.89,9.8
1,137310,에스디바이오센서,2.39,10.7
2,383800,LX홀딩스,3.90,1.5
3,353200,대덕전자,6.16,39.4
4,363280,티와이홀딩스,6.17,47.6
5,126720,수산인더스트리,6.30,24.7
6,108320,LX세미콘,7.50,35.7
